# Memory Analysis Lab Deeper Dive (Iveep)
Description: This notebook documents the analysis of a suspicious disk image (iveep.7z) through various forensic techniques, including memory dump analysis, process analysis, binary examination, and network traffic analysis. The goal is to identify Indicators of Compromise (IOCs) and suspicious activities that may indicate malicious behavior.

We will begin with a quick recap on the various indicators identified from the previous commands from the `iveep_lab file` in the `Process-Memory-Analysis` repository



#### Process Analysis 

From the previous notes, the commands below were used to analyze the various processes on the iveep.mem file: 

`vol -f iveep.mem windows.pslist`
`vol -f iveep.mem windows.pstree`
`vol -f iveep.mem windows.psscan`
`vol -f iveep.mem windows.psxview`
`vol -f iveep.mem windows.netscan`
`vol -f iveep.mem windows.cmdline`



**Analysis**: The process `PSEXESVC.exe` was launched from `C:\Windows\PSEXESVC.exe`, a suspicious location for a Windows service. It then spawned `excel-update.exe`, which is also located in `C:\Windows\excel-update.exe`, another unusual directory for program executables when it should run from a program file directory. These processes were executed around 17:00 UTC, with the command `excel-update.exe` launching `cmd.exe` and the command prompt launching `powershell.exe` shortly after.


- The chain of execution (`PSEXESVC.exe` → `excel-update.exe` → `cmd.exe` → `powershell.exe`) indicates possible attacker behavior, including PowerShell script execution and command-line access.
- Remote access via `ScreenConnect` was also detected, indicating potential remote exploitation or C2 activity.

**Evidence**:
```
|PID|---|PPID|--|Imagefilename|-|Offset(v)|-----------------------------------------|CreateTime|
1296	1000	PSEXESVC.exe	0x840e99474080	6	-	0	True	2025-03-17 17:01:17.000000 UTC	N/A	\Device\HarddiskVolume3\Windows\PSEXESVC.exe	C:\Windows\PSEXESVC.exe	C:\Windows\PSEXESVC.exe
4672    11296	excel-update.e	0x840e9e6f3080	2	-	0	True	2025-03-17 17:01:18.000000 UTC	N/A	\Device\HarddiskVolume3\Windows\excel-update.exe	"excel-update.exe" 	C:\Windows\excel-update.exe
13752	4672	cmd.exe	0x840e978ed080	1	-	0	True	2025-03-17 17:04:00.000000 UTC	N/A	\Device\HarddiskVolume3\Windows\SysWOW64\cmd.exe	C:\Windows\system32\cmd.exe	C:\Windows\SysWOW64\cmd.exe
13156	13752	powershell.exe	0x840e9ee430c0	10	-	0	True	2025-03-17 17:19:23.000000 UTC	N/A	\Device\HarddiskVolume3\Windows\SysWOW64\WindowsPowerShell\v1.0\powershell.exe	powershell	C:\Windows\SysWOW64\WindowsPowerShell\v1.0\powershell.exe
```
                                                                                                                                                                                                                                                                                                                                                                                          
- **Timestamps**: All processes occurred within a short timeframe (17:00–18:00 UTC).


### Network Analysis
The `windows.netscan` command helps to determine IP addresses or ports associated with the screenconnect suspicion identified to be a remote desktop connection for further analysis


`vol -f iveep.mem windows.netscan`

```
Volatility 3 Framework 2.26.2                                                                                                                                                                                       Progress:  100.00               PDB scanning finished                                                                                                                                                               Offset  Proto   LocalAddr       LocalPort       ForeignAddr     ForeignPort     State   PID     Owner   Created                                                                                                                                                                                                                                                                                                                         0x840e9028e470  TCPv4   0.0.0.0 49668   0.0.0.0 0       LISTENING       2904    spoolsv.exe     2025-03-12 16:40:03.000000 UTC                                                                                      0x840e9028e890  TCPv4   0.0.0.0 445     0.0.0.0 0       LISTENING       4       System  2025-03-12 16:40:05.000000 UTC                                                                                              0x840e9028e890  TCPv6   ::      445     ::      0       LISTENING       4       System  2025-03-12 16:40:05.000000 UTC                                                                                              0x840e90aa1b40  TCPv4   10.20.20.23     59848   20.10.127.192   443     ESTABLISHED     10224   MsSense.exe     2025-03-17 17:26:17.000000 UTC                                                                      0x840e90e64010  TCPv4   10.20.20.23     50167   23.194.127.11   443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:49.000000 UTC                                                                      0x840e90ed0500  UDPv4   10.20.20.23     138     *       0               4       System  2025-03-12 16:39:44.000000 UTC                                                                                              0x840e90ed0820  UDPv4   10.20.20.23     137     *       0               4       System  2025-03-12 16:39:44.000000 UTC                                                                                              0x840e911dd470  TCPv4   10.20.20.23     139     0.0.0.0 0       LISTENING       4       System  2025-03-12 16:39:44.000000 UTC                                                                                      0x840e911dd730  TCPv4   0.0.0.0 135     0.0.0.0 0       LISTENING       1076    svchost.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e911de650  TCPv4   0.0.0.0 49667   0.0.0.0 0       LISTENING       1580    svchost.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e91b4d230  TCPv6   ::1     42050   ::      0       LISTENING       5176    Microsoft.Shar  2025-03-17 16:51:22.000000 UTC                                                                                      0x840e91b4da70  TCPv4   0.0.0.0 5040    0.0.0.0 0       LISTENING       1480    svchost.exe     2025-03-12 16:41:36.000000 UTC                                                                                      0x840e91f22050  UDPv4   0.0.0.0 5353    *       0               1336    msedge.exe      2025-03-17 16:51:15.000000 UTC                                                                                              0x840e91f43690  UDPv4   0.0.0.0 55871   *       0               15568   SenseNdr.exe    2025-03-17 16:51:22.000000 UTC                                                                                              0x840e91f43cd0  UDPv4   0.0.0.0 61832   *       0               15568   SenseNdr.exe    2025-03-17 16:51:24.000000 UTC                                                                                              0x840e92041050  TCPv4   0.0.0.0 49667   0.0.0.0 0       LISTENING       1580    svchost.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e92041050  TCPv6   ::      49667   ::      0       LISTENING       1580    svchost.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e920411b0  TCPv4   0.0.0.0 49664   0.0.0.0 0       LISTENING       548     lsass.exe       2025-03-12 16:39:53.000000 UTC                                                                                      0x840e920411b0  TCPv6   ::      49664   ::      0       LISTENING       548     lsass.exe       2025-03-12 16:39:53.000000 UTC                                                                                      0x840e92041470  TCPv4   0.0.0.0 135     0.0.0.0 0       LISTENING       1076    svchost.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e92041470  TCPv6   ::      135     ::      0       LISTENING       1076    svchost.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e92041730  TCPv4   0.0.0.0 49665   0.0.0.0 0       LISTENING       880     wininit.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e92041730  TCPv6   ::      49665   ::      0       LISTENING       880     wininit.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e920419f0  TCPv4   0.0.0.0 49664   0.0.0.0 0       LISTENING       548     lsass.exe       2025-03-12 16:39:53.000000 UTC                                                                                      0x840e92041b50  TCPv4   0.0.0.0 49665   0.0.0.0 0       LISTENING       880     wininit.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e92041cb0  TCPv4   0.0.0.0 49666   0.0.0.0 0       LISTENING       1524    svchost.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e92041cb0  TCPv6   ::      49666   ::      0       LISTENING       1524    svchost.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e92041e10  TCPv4   0.0.0.0 49668   0.0.0.0 0       LISTENING       2904    spoolsv.exe     2025-03-12 16:40:03.000000 UTC                                                                                      0x840e92041e10  TCPv6   ::      49668   ::      0       LISTENING       2904    spoolsv.exe     2025-03-12 16:40:03.000000 UTC                                                                                      0x840e92042bd0  TCPv4   0.0.0.0 5357    0.0.0.0 0       LISTENING       4       System  2025-03-12 16:40:03.000000 UTC                                                                                              0x840e92042bd0  TCPv6   ::      5357    ::      0       LISTENING       4       System  2025-03-12 16:40:03.000000 UTC                                                                                              0x840e92042e90  TCPv4   0.0.0.0 49666   0.0.0.0 0       LISTENING       1524    svchost.exe     2025-03-12 16:39:53.000000 UTC                                                                                      0x840e920f75e0  UDPv4   127.0.0.1       58652   *       0               15568   SenseNdr.exe    2025-03-17 16:51:24.000000 UTC                                                                                      0x840e920f7a90  UDPv4   0.0.0.0 58653   *       0               15568   SenseNdr.exe    2025-03-17 16:51:24.000000 UTC                                                                                              0x840e920f7c20  UDPv4   0.0.0.0 0       *       0               12440   svchost.exe     2025-03-13 14:54:32.000000 UTC                                                                                              0x840e920f7c20  UDPv6   ::      0       *       0               12440   svchost.exe     2025-03-13 14:54:32.000000 UTC                                                                                              0x840e920f8580  UDPv4   0.0.0.0 56440   *       0               15568   SenseNdr.exe    2025-03-17 16:51:25.000000 UTC                                                                                              0x840e922f3520  UDPv4   0.0.0.0 500     *       0               12440   svchost.exe     2025-03-13 14:54:32.000000 UTC                                                                                              0x840e922f3840  UDPv4   0.0.0.0 4500    *       0               12440   svchost.exe     2025-03-13 14:54:32.000000 UTC                                                                                              0x840e922f39d0  UDPv4   0.0.0.0 500     *       0               12440   svchost.exe     2025-03-13 14:54:32.000000 UTC                                                                                              0x840e922f39d0  UDPv6   ::      500     *       0               12440   svchost.exe     2025-03-13 14:54:32.000000 UTC                                                                                              0x840e922f3cf0  UDPv4   0.0.0.0 4500    *       0               12440   svchost.exe     2025-03-13 14:54:32.000000 UTC                                                                                              0x840e922f3cf0  UDPv6   ::      4500    *       0               12440   svchost.exe     2025-03-13 14:54:32.000000 UTC                                                                                              0x840e922f4330  UDPv4   0.0.0.0 123     *       0               9444    svchost.exe     2025-03-17 16:50:14.000000 UTC                                                                                              0x840e922f4330  UDPv6   ::      123     *       0               9444    svchost.exe     2025-03-17 16:50:14.000000 UTC                                                                                              0x840e922f5140  UDPv4   0.0.0.0 0       *       0               12440   svchost.exe     2025-03-13 14:54:32.000000 UTC                                                                                              0x840e922f55f0  UDPv4   0.0.0.0 123     *       0               9444    svchost.exe     2025-03-17 16:50:14.000000 UTC                                                                                              0x840e922f6270  UDPv4   127.0.0.1       58654   *       0               15568   SenseNdr.exe    2025-03-17 16:51:24.000000 UTC                                                                                      0x840e922f6720  UDPv4   0.0.0.0 58655   *       0               15568   SenseNdr.exe    2025-03-17 16:51:24.000000 UTC                                                                                              0x840e922f7210  UDPv4   127.0.0.1       58656   *       0               15568   SenseNdr.exe    2025-03-17 16:51:24.000000 UTC                                                                                      0x840e922f76c0  UDPv4   0.0.0.0 58657   *       0               15568   SenseNdr.exe    2025-03-17 16:51:24.000000 UTC                                                                                              0x840e922f8660  UDPv4   127.0.0.1       55870   *       0               15568   SenseNdr.exe    2025-03-17 16:51:22.000000 UTC                                                                                      0x840e9231fc30  UDPv4   0.0.0.0 5353    *       0               1336    msedge.exe      2025-03-17 16:51:15.000000 UTC                                                                                              0x840e9231fc30  UDPv6   ::      5353    *       0               1336    msedge.exe      2025-03-17 16:51:15.000000 UTC                                                                                              0x840e92322ca0  UDPv4   0.0.0.0 59539   *       0               15568   SenseNdr.exe    2025-03-17 16:51:21.000000 UTC                                                                                              0x840e92323150  UDPv4   127.0.0.1       61831   *       0               15568   SenseNdr.exe    2025-03-17 16:51:24.000000 UTC                                                                                      0x840e92323600  UDPv4   127.0.0.1       56439   *       0               15568   SenseNdr.exe    2025-03-17 16:51:25.000000 UTC                                                                                      0x840e923757e0  UDPv4   0.0.0.0 55149   *       0               10704   fontdrvhost.ex  2025-03-12 19:16:18.000000 UTC                                                                                              0x840e923757e0  UDPv6   ::      55149   *       0               10704   fontdrvhost.ex  2025-03-12 19:16:18.000000 UTC                                                                                              0x840e92388070  UDPv4   0.0.0.0 0       *       0               9444    svchost.exe     2025-03-13 21:55:51.000000 UTC                                                                                              0x840e92388200  UDPv4   127.0.0.1       63193   *       0               15568   SenseNdr.exe    2025-03-17 16:51:17.000000 UTC                                                                                      0x840e92389010  UDPv4   0.0.0.0 0       *       0               9444    svchost.exe     2025-03-13 21:55:51.000000 UTC                                                                                              0x840e92389010  UDPv6   ::      0       *       0               9444    svchost.exe     2025-03-13 21:55:51.000000 UTC                                                                                              0x840e9238aaa0  UDPv4   127.0.0.1       56437   *       0               15568   SenseNdr.exe    2025-03-17 16:51:25.000000 UTC                                                                                      0x840e9238b0e0  UDPv4   0.0.0.0 56438   *       0               15568   SenseNdr.exe    2025-03-17 16:51:25.000000 UTC                                                                                              0x840e9238b720  UDPv4   0.0.0.0 63194   *       0               15568   SenseNdr.exe    2025-03-17 16:51:17.000000 UTC                                                                                              0x840e9238ba40  UDPv4   0.0.0.0 54621   *       0               12476   msedge.exe      2025-03-17 16:51:25.000000 UTC                                                                                              0x840e9238bbd0  UDPv4   127.0.0.1       61596   *       0               15568   SenseNdr.exe    2025-03-17 16:51:30.000000 UTC                                                                                      0x840e9238c080  UDPv4   0.0.0.0 61597   *       0               15568   SenseNdr.exe    2025-03-17 16:51:30.000000 UTC                                                                                              0x840e9238d4d0  UDPv4   127.0.0.1       63533   *       0               15568   SenseNdr.exe    2025-03-17 16:51:21.000000 UTC                                                                                      0x840e9238edd0  UDPv4   0.0.0.0 63324   *       0               2376    svchost.exe     2025-03-17 16:51:40.000000 UTC                                                                                              0x840e9238edd0  UDPv6   ::      63324   *       0               2376    svchost.exe     2025-03-17 16:51:40.000000 UTC                                                                                              0x840e923d74e0  TCPv4   10.20.20.23     50173   23.45.84.152    443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:49.000000 UTC                                                                      0x840e92c62840  UDPv4   0.0.0.0 61481   *       0               -       -       2025-03-12 22:12:18.000000 UTC                                                                                                      0x840e92c65720  UDPv4   0.0.0.0 50013   *       0               -       -       2025-03-12 22:12:20.000000 UTC                                                                                                      0x840e92c674d0  UDPv4   0.0.0.0 62737   *       0               -       -       2025-03-12 22:12:20.000000 UTC                                                                                                      0x840e92c68470  UDPv4   0.0.0.0 63320   *       0               -       -       2025-03-12 22:12:17.000000 UTC                                                                                                      0x840e92c69280  UDPv4   0.0.0.0 63147   *       0               -       -       2025-03-12 22:12:17.000000 UTC                                                                                                      0x840e96647de0  UDPv4   0.0.0.0 0       *       0               2168    svchost.exe     2025-03-12 18:54:57.000000 UTC                                                                                              0x840e96647de0  UDPv6   ::      0       *       0               2168    svchost.exe     2025-03-12 18:54:57.000000 UTC                                                                                              0x840e9684aad0  UDPv4   0.0.0.0 0       *       0               2376    svchost.exe     2025-03-12 16:39:54.000000 UTC                                                                                              0x840e9684aad0  UDPv6   ::      0       *       0               2376    svchost.exe     2025-03-12 16:39:54.000000 UTC                                                                                              0x840e96941500  TCPv4   10.20.20.23     50153   23.211.124.202  443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:46.000000 UTC                                                                      0x840e96a21e40  UDPv6   ::1     1900    *       0               3172    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e96af1960  UDPv4   0.0.0.0 5355    *       0               2376    svchost.exe     2025-03-12 16:40:03.000000 UTC                                                                                              0x840e96af1960  UDPv6   ::      5355    *       0               2376    svchost.exe     2025-03-12 16:40:03.000000 UTC                                                                                              0x840e96af1af0  UDPv4   0.0.0.0 5353    *       0               2376    svchost.exe     2025-03-12 16:40:03.000000 UTC                                                                                              0x840e96af2450  UDPv4   0.0.0.0 5355    *       0               2376    svchost.exe     2025-03-12 16:40:03.000000 UTC                                                                                              0x840e96af30d0  UDPv4   0.0.0.0 5353    *       0               2376    svchost.exe     2025-03-12 16:40:03.000000 UTC                                                                                              0x840e96af30d0  UDPv6   ::      5353    *       0               2376    svchost.exe     2025-03-12 16:40:03.000000 UTC                                                                                              0x840e96afd4e0  UDPv6   fe80::ed2b:e322:d23f:739c       1900    *       0               3172    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                      0x840e96cb2a00  UDPv4   0.0.0.0 0       *       0               4800    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e96cb5750  UDPv4   0.0.0.0 0       *       0               4800    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e96cb5750  UDPv6   ::      0       *       0               4800    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e96cb6880  UDPv4   0.0.0.0 61627   *       0               2220    svchost.exe     2025-03-12 16:40:03.000000 UTC                                                                                              0x840e96cb7820  UDPv4   0.0.0.0 61628   *       0               2220    svchost.exe     2025-03-12 16:40:03.000000 UTC                                                                                              0x840e96cb7820  UDPv6   ::      61628   *       0               2220    svchost.exe     2025-03-12 16:40:03.000000 UTC                                                                                              0x840e96cc5470  UDPv4   10.20.20.23     52210   *       0               3172    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                      0x840e96cc5dd0  UDPv4   127.0.0.1       52211   *       0               3172    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                      0x840e96cc7b80  UDPv6   fe80::ed2b:e322:d23f:739c       52208   *       0               3172    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                      0x840e96cc81c0  UDPv6   ::1     52209   *       0               3172    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e96f744a0  UDPv4   127.0.0.1       61629   *       0               3376    svchost.exe     2025-03-12 16:40:04.000000 UTC                                                                                      0x840e96f93260  UDPv4   10.20.20.23     1900    *       0               3172    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                      0x840e96f99980  UDPv4   0.0.0.0 3702    *       0               2220    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e96f99980  UDPv6   ::      3702    *       0               2220    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e96f9ede0  UDPv4   127.0.0.1       1900    *       0               3172    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                      0x840e96fa03c0  UDPv4   0.0.0.0 3702    *       0               2220    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e96fa03c0  UDPv6   ::      3702    *       0               2220    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e96fa1810  UDPv4   0.0.0.0 3702    *       0               2220    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e96fa2940  UDPv4   0.0.0.0 3702    *       0               2220    svchost.exe     2025-03-12 16:40:06.000000 UTC                                                                                              0x840e97054750  TCPv4   10.20.20.23     50154   23.211.124.202  443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:46.000000 UTC                                                                      0x840e9708b270  TCPv4   10.20.20.23     50152   23.211.124.202  443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:46.000000 UTC                                                                      0x840e970eba20  TCPv4   10.20.20.23     50172   23.194.127.11   443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:49.000000 UTC                                                                      0x840e97447260  TCPv4   10.20.20.23     50164   23.194.127.43   443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:48.000000 UTC                                                                      0x840e9750e5e0  TCPv4   0.0.0.0 47001   0.0.0.0 0       LISTENING       4       System  2025-03-12 16:42:19.000000 UTC                                                                                              0x840e9750e5e0  TCPv6   ::      47001   ::      0       LISTENING       4       System  2025-03-12 16:42:19.000000 UTC                                                                                              0x840e9750e740  TCPv4   0.0.0.0 49677   0.0.0.0 0       LISTENING       1000    services.exe    2025-03-12 16:40:09.000000 UTC                                                                                      0x840e9750ea00  TCPv4   0.0.0.0 49677   0.0.0.0 0       LISTENING       1000    services.exe    2025-03-12 16:40:09.000000 UTC                                                                                      0x840e9750ea00  TCPv6   ::      49677   ::      0       LISTENING       1000    services.exe    2025-03-12 16:40:09.000000 UTC                                                                                      0x840e9750f3a0  TCPv4   0.0.0.0 5985    0.0.0.0 0       LISTENING       4       System  2025-03-12 16:42:19.000000 UTC                                                                                              0x840e9750f3a0  TCPv6   ::      5985    ::      0       LISTENING       4       System  2025-03-12 16:42:19.000000 UTC                                                                                              0x840e9750f500  TCPv4   0.0.0.0 5986    0.0.0.0 0       LISTENING       4       System  2025-03-12 16:42:19.000000 UTC                                                                                              0x840e9750f500  TCPv6   ::      5986    ::      0       LISTENING       4       System  2025-03-12 16:42:19.000000 UTC                                                                                              0x840e9750fbe0  TCPv4   0.0.0.0 58093   0.0.0.0 0       LISTENING       4800    svchost.exe     2025-03-13 14:54:33.000000 UTC                                                                                      0x840e9750fbe0  TCPv6   ::      58093   ::      0       LISTENING       4800    svchost.exe     2025-03-13 14:54:33.000000 UTC                                                                                      0x840e9750fea0  TCPv4   0.0.0.0 7680    0.0.0.0 0       LISTENING       2800    svchost.exe     2025-03-12 22:01:25.000000 UTC                                                                                      0x840e9750fea0  TCPv6   ::      7680    ::      0       LISTENING       2800    svchost.exe     2025-03-12 22:01:25.000000 UTC                                                                                      0x840e97510420  TCPv4   0.0.0.0 58093   0.0.0.0 0       LISTENING       4800    svchost.exe     2025-03-13 14:54:33.000000 UTC                                                                                      0x840e975d3760  UDPv4   0.0.0.0 5050    *       0               1480    svchost.exe     2025-03-12 16:41:36.000000 UTC                                                                                              0x840e97632010  TCPv4   10.20.20.23     59865   35.86.58.69     80      CLOSED  14536   spoolsv.exe     2025-03-17 17:26:37.000000 UTC                                                                              0x840e97643010  TCPv4   10.20.20.23     50165   23.194.127.43   443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:48.000000 UTC                                                                      0x840e9793ebc0  UDPv4   0.0.0.0 51100   *       0               3752    dasHost.exe     2025-03-12 16:40:36.000000 UTC                                                                                              0x840e9793fcf0  UDPv4   0.0.0.0 3702    *       0               3752    dasHost.exe     2025-03-12 16:40:36.000000 UTC                                                                                              0x840e9793fcf0  UDPv6   ::      3702    *       0               3752    dasHost.exe     2025-03-12 16:40:36.000000 UTC                                                                                              0x840e97940010  UDPv4   0.0.0.0 51101   *       0               3752    dasHost.exe     2025-03-12 16:40:36.000000 UTC                                                                                              0x840e97940010  UDPv6   ::      51101   *       0               3752    dasHost.exe     2025-03-12 16:40:36.000000 UTC                                                                                              0x840e97a87cd0  UDPv4   0.0.0.0 5353    *       0               10732   csrss.exe       2025-03-12 19:14:03.000000 UTC                                                                                              0x840e97a88180  UDPv4   0.0.0.0 5353    *       0               10732   csrss.exe       2025-03-12 19:14:03.000000 UTC                                                                                              0x840e97a88180  UDPv6   ::      5353    *       0               10732   csrss.exe       2025-03-12 19:14:03.000000 UTC                                                                                              0x840e97a8f070  UDPv4   0.0.0.0 64992   *       0               10704   fontdrvhost.ex  2025-03-12 19:15:56.000000 UTC                                                                                              0x840e97a904c0  UDPv4   0.0.0.0 54576   *       0               10704   fontdrvhost.ex  2025-03-12 19:15:56.000000 UTC                                                                                              0x840e97b2cdf0  UDPv4   0.0.0.0 60635   *       0               10704   fontdrvhost.ex  2025-03-12 19:15:19.000000 UTC                                                                                              0x840e97b2cdf0  UDPv6   ::      60635   *       0               10704   fontdrvhost.ex  2025-03-12 19:15:19.000000 UTC                                                                                              0x840e97b2fb40  UDPv4   0.0.0.0 64155   *       0               10704   fontdrvhost.ex  2025-03-12 19:14:15.000000 UTC                                                                                              0x840e97b320c0  UDPv4   0.0.0.0 55736   *       0               10704   fontdrvhost.ex  2025-03-12 19:14:15.000000 UTC                                                                                              0x840e97b355e0  UDPv4   0.0.0.0 61071   *       0               10704   fontdrvhost.ex  2025-03-12 19:14:15.000000 UTC                                                                                              0x840e97b36710  UDPv4   0.0.0.0 53180   *       0               10704   fontdrvhost.ex  2025-03-12 19:14:15.000000 UTC                                                                                              0x840e97b3c340  UDPv4   0.0.0.0 0       *       0               3308    svchost.exe     2025-03-12 18:23:25.000000 UTC                                                                                              0x840e97b3c340  UDPv6   ::      0       *       0               3308    svchost.exe     2025-03-12 18:23:25.000000 UTC                                                                                              0x840e97c40b00  UDPv4   0.0.0.0 3702    *       0               3752    dasHost.exe     2025-03-12 16:40:36.000000 UTC                                                                                              0x840e97c40e20  UDPv4   0.0.0.0 3702    *       0               3752    dasHost.exe     2025-03-12 16:40:36.000000 UTC                                                                                              0x840e97c415f0  UDPv4   0.0.0.0 3702    *       0               3752    dasHost.exe     2025-03-12 16:40:36.000000 UTC                                                                                              0x840e97c415f0  UDPv6   ::      3702    *       0               3752    dasHost.exe     2025-03-12 16:40:36.000000 UTC                                                                                              0x840e9804bb60  TCPv4   10.20.20.23     59428   23.209.84.48    443     CLOSE_WAIT      9092    SearchApp.exe   2025-03-17 17:15:00.000000 UTC                                                                      0x840e982f34b0  TCPv4   10.20.20.23     59231   35.86.58.69     4443    ESTABLISHED     4672    excel-update.e  2025-03-17 17:01:19.000000 UTC                                                                      0x840e98564050  TCPv4   10.20.20.23     50159   23.217.9.16     443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:48.000000 UTC                                                                      0x840e98aa9a60  TCPv4   10.20.20.23     50151   23.211.124.202  443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:46.000000 UTC                                                                      0x840e98ab3010  TCPv4   10.20.20.23     59590   20.42.144.52    443     ESTABLISHED     12476   msedge.exe      2025-03-17 17:19:15.000000 UTC                                                                      0x840e98b22010  TCPv4   10.20.20.23     445     10.20.20.22     50695   ESTABLISHED     4       System  2025-03-17 17:01:18.000000 UTC                                                                              0x840e98c11010  TCPv4   10.20.20.23     59434   20.125.62.241   443     CLOSED  9092    SearchApp.exe   2025-03-17 17:15:06.000000 UTC                                                                              0x840e98da4a20  TCPv4   10.20.20.23     50171   23.194.127.11   443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:49.000000 UTC                                                                      0x840e98fb7a80  TCPv4   10.20.20.23     59426   23.194.127.18   443     CLOSE_WAIT      9092    SearchApp.exe   2025-03-17 17:15:00.000000 UTC                                                                      0x840e98fc5a20  TCPv4   10.20.20.23     50162   23.194.127.43   443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:48.000000 UTC                                                                      0x840e99010520  TCPv4   10.20.20.23     59262   139.178.70.144  443     ESTABLISHED     6988    ScreenConnect.  2025-03-17 17:05:19.000000 UTC                                                                      0x840e990bda70  TCPv4   10.20.20.23     50149   150.171.28.12   443     CLOSED  4236    SearchApp.exe   2025-03-12 16:51:45.000000 UTC                                                                              0x840e990cd560  TCPv4   10.20.20.23     58310   20.59.87.225    443     ESTABLISHED     3560    svchost.exe     2025-03-13 21:54:57.000000 UTC                                                                      0x840e993d0010  TCPv4   10.20.20.23     50155   23.211.124.202  443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:46.000000 UTC                                                                      0x840e9952fb20  TCPv4   10.20.20.23     50150   204.79.197.203  443     CLOSED  4236    SearchApp.exe   2025-03-12 16:51:45.000000 UTC                                                                              0x840e99563320  TCPv4   10.20.20.23     59297   13.107.213.254  443     CLOSE_WAIT      9288    SearchApp.exe   2025-03-17 17:07:21.000000 UTC                                                                      0x840e9957d270  TCPv4   10.20.20.23     58252   54.67.68.88     443     ESTABLISHED     4       System  2025-03-13 15:03:17.000000 UTC                                                                              0x840e995d9560  TCPv4   10.20.20.23     50163   23.194.127.43   443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:48.000000 UTC                                                                      0x840e99698a20  TCPv4   10.20.20.23     50169   23.194.127.11   443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:49.000000 UTC                                                                      0x840e9969d930  TCPv4   10.20.20.23     59843   13.107.9.158    443     ESTABLISHED     9288    SearchApp.exe   2025-03-17 17:26:12.000000 UTC                                                                      0x840e9972db00  TCPv4   10.20.20.23     59861   13.107.21.239   443     ESTABLISHED     9016    msedge.exe      2025-03-17 17:26:33.000000 UTC                                                                      0x840e9972fb50  TCPv4   10.20.20.23     59431   23.45.84.152    443     CLOSE_WAIT      9092    SearchApp.exe   2025-03-17 17:15:01.000000 UTC                                                                      0x840e99743a20  TCPv4   10.20.20.23     50170   23.194.127.11   443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:49.000000 UTC                                                                      0x840e997c8010  TCPv4   10.20.20.23     59361   20.150.78.196   443     CLOSED  9092    SearchApp.exe   2025-03-17 17:12:52.000000 UTC                                                                              0x840e99827270  TCPv4   10.20.20.23     50161   23.217.9.16     443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:48.000000 UTC                                                                      0x840e99c10b60  TCPv6   fe80::ed2b:e322:d23f:739c       445     fe80::c05a:589f:16e6:1690       50692   ESTABLISHED     4       System  2025-03-17 17:01:17.000000 UTC                                              0x840e99c11a20  TCPv4   10.20.20.23     50168   23.194.127.11   443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:49.000000 UTC                                                                      0x840e99cb76e0  TCPv4   10.20.20.23     59879   35.86.58.69     80      CLOSED  14536   spoolsv.exe     2025-03-17 17:27:04.000000 UTC                                                                              0x840e99d24050  TCPv4   10.20.20.23     50160   23.217.9.16     443     CLOSE_WAIT      4236    SearchApp.exe   2025-03-12 16:51:48.000000 UTC                                                                      0x840e99d64b60  TCPv4   10.20.20.23     59867   35.86.58.69     80      CLOSED  14536   spoolsv.exe     2025-03-17 17:26:39.000000 UTC                                                                              0x840e99d66520  TCPv4   10.20.20.23     59329   35.86.58.69     4444    CLOSED  14536   spoolsv.exe     2025-03-17 17:12:10.000000 UTC                                                                              0x840e9ad23170  UDPv4   127.0.0.1       60255   *       0               15568   SenseNdr.exe    2025-03-17 17:01:18.000000 UTC                                                                                      0x840e9ad23300  UDPv4   127.0.0.1       60253   *       0               15568   SenseNdr.exe    2025-03-17 17:01:18.000000 UTC                                                                                      0x840e9ad23490  UDPv4   0.0.0.0 60258   *       0               15568   SenseNdr.exe    2025-03-17 17:01:18.000000 UTC                                                                                              0x840e9ad237b0  UDPv4   0.0.0.0 0       *       0               15172   powershell.exe  2025-03-17 17:08:10.000000 UTC                                                                                              0x840e9ad24110  UDPv4   0.0.0.0 0       *       0               15172   powershell.exe  2025-03-17 17:08:10.000000 UTC                                                                                              0x840e9ad24110  UDPv6   ::      0       *       0               15172   powershell.exe  2025-03-17 17:08:10.000000 UTC                                                                                              0x840e9ad242a0  UDPv4   127.0.0.1       60245   *       0               15568   SenseNdr.exe    2025-03-17 17:01:17.000000 UTC                                                                                      0x840e9ad24430  UDPv4   0.0.0.0 60256   *       0               15568   SenseNdr.exe    2025-03-17 17:01:18.000000 UTC                                                                                              0x840e9ad245c0  UDPv4   127.0.0.1       60257   *       0               15568   SenseNdr.exe    2025-03-17 17:01:18.000000 UTC                                                                                      0x840e9ad248e0  UDPv4   0.0.0.0 60250   *       0               15568   SenseNdr.exe    2025-03-17 17:01:17.000000 UTC                                                                                              0x840e9ad24d90  UDPv4   0.0.0.0 0       *       0               15172   powershell.exe  2025-03-17 17:08:10.000000 UTC                                                                                              0x840e9ad25560  UDPv4   0.0.0.0 60248   *       0               15568   SenseNdr.exe    2025-03-17 17:01:17.000000 UTC                                                                                              0x840e9ad256f0  UDPv4   127.0.0.1       60251   *       0               15568   SenseNdr.exe    2025-03-17 17:01:17.000000 UTC                                                                                      0x840e9ad25880  UDPv4   0.0.0.0 60252   *       0               15568   SenseNdr.exe    2025-03-17 17:01:17.000000 UTC                                                                                              0x840e9ad25a10  UDPv4   127.0.0.1       60247   *       0               15568   SenseNdr.exe    2025-03-17 17:01:17.000000 UTC                                                                                      0x840e9ad25ba0  UDPv4   0.0.0.0 60254   *       0               15568   SenseNdr.exe    2025-03-17 17:01:18.000000 UTC                                                                                              0x840e9ad26050  UDPv4   0.0.0.0 60246   *       0               15568   SenseNdr.exe    2025-03-17 17:01:17.000000 UTC                                                                                              0x840e9ad261e0  UDPv4   127.0.0.1       60249   *       0               15568   SenseNdr.exe    2025-03-17 17:01:17.000000 UTC                                                                                      0x840e9ad26cd0  UDPv4   0.0.0.0 0       *       0               12008   svchost.exe     2025-03-17 17:26:47.000000 UTC                                                                                              0x840e9ad26cd0  UDPv6   ::      0       *       0               12008   svchost.exe     2025-03-17 17:26:47.000000 UTC                                                                                              0x840e9ad27630  UDPv4   0.0.0.0 0       *       0               10352   powershell.exe  2025-03-17 17:26:44.000000 UTC                                                                                              0x840e9ad28120  UDPv4   0.0.0.0 0       *       0               15172   powershell.exe  2025-03-17 17:08:10.000000 UTC                                                                                              0x840e9ad28120  UDPv6   ::      0       *       0               15172   powershell.exe  2025-03-17 17:08:10.000000 UTC                                                                                              0x840e9ad288f0  UDPv4   0.0.0.0 0       *       0               12008   svchost.exe     2025-03-17 17:26:47.000000 UTC                                                                                              0x840e9af23490  UDPv4   127.0.0.1       56625   *       0               -       -       2025-03-12 17:25:48.000000 UTC                                                                                              0x840e9af23620  UDPv4   127.0.0.1       53542   *       0               -       -       2025-03-12 17:25:58.000000 UTC                                                                                              0x840e9af24110  UDPv4   0.0.0.0 56626   *       0               -       -       2025-03-12 17:25:48.000000 UTC                                                                                                      0x840e9af242a0  UDPv4   127.0.0.1       57961   *       0               -       -       2025-03-12 17:26:58.000000 UTC                                                                                              0x840e9af24430  UDPv4   127.0.0.1       61432   *       0               -       -       2025-03-12 17:26:08.000000 UTC                                                                                              0x840e9af248e0  UDPv4   127.0.0.1       57959   *       0               -       -       2025-03-12 17:26:58.000000 UTC                                                                                              0x840e9af24d90  UDPv4   0.0.0.0 50614   *       0               -       -       2025-03-12 17:25:52.000000 UTC                                                                                                      0x840e9af25240  UDPv4   127.0.0.1       50613   *       0               -       -       2025-03-12 17:25:52.000000 UTC                                                                                              0x840e9af256f0  UDPv4   0.0.0.0 61433   *       0               -       -       2025-03-12 17:26:08.000000 UTC                                                                                                      0x840e9af25ba0  UDPv4   0.0.0.0 53543   *       0               -       -       2025-03-12 17:25:58.000000 UTC                                                                                                      0x840e9af26050  UDPv4   0.0.0.0 57960   *       0               -       -       2025-03-12 17:26:58.000000 UTC                                                                                                      0x840e9af269b0  UDPv4   127.0.0.1       57965   *       0               -       -       2025-03-12 17:26:58.000000 UTC                                                                                              0x840e9af26b40  UDPv4   127.0.0.1       57963   *       0               -       -       2025-03-12 17:26:58.000000 UTC                                                                                              0x840e9af26cd0  UDPv4   0.0.0.0 57964   *       0               -       -       2025-03-12 17:26:58.000000 UTC                                                                                                      0x840e9af26e60  UDPv4   0.0.0.0 55312   *       0               10704   fontdrvhost.ex  2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af274a0  UDPv4   0.0.0.0 57966   *       0               -       -       2025-03-12 17:26:58.000000 UTC                                                                                                      0x840e9af27ae0  UDPv4   0.0.0.0 57997   *       0               10704   fontdrvhost.ex  2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af27c70  UDPv4   0.0.0.0 50577   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af27c70  UDPv6   ::      50577   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af27e00  UDPv4   0.0.0.0 55070   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af27e00  UDPv6   ::      55070   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af28120  UDPv4   0.0.0.0 59377   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af28120  UDPv6   ::      59377   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af282b0  UDPv4   0.0.0.0 58511   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af282b0  UDPv6   ::      58511   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af28440  UDPv4   0.0.0.0 63189   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af28440  UDPv6   ::      63189   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af285d0  UDPv4   0.0.0.0 57962   *       0               -       -       2025-03-12 17:26:58.000000 UTC                                                                                                      0x840e9af28a80  UDPv4   0.0.0.0 61153   *       0               10704   fontdrvhost.ex  2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af28da0  UDPv4   0.0.0.0 52553   *       0               10704   fontdrvhost.ex  2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af290c0  UDPv4   0.0.0.0 54713   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af290c0  UDPv6   ::      54713   *       0               2376    svchost.exe     2025-03-12 19:15:06.000000 UTC                                                                                              0x840e9af29890  UDPv4   0.0.0.0 63239   *       0               -       -       2025-03-12 17:30:30.000000 UTC                                                                                                      0x840e9af2a1f0  UDPv4   127.0.0.1       64029   *       0               -       -       2025-03-12 17:27:40.000000 UTC                                                                                              0x840e9af2a510  UDPv4   0.0.0.0 53115   *       0               -       -       2025-03-12 17:30:30.000000 UTC                                                                                                      0x840e9af2a6a0  UDPv4   0.0.0.0 64030   *       0               -       -       2025-03-12 17:27:40.000000 UTC                                                                                                      0x840e9af2ace0  UDPv4   0.0.0.0 64700   *       0               -       -       2025-03-12 17:27:51.000000 UTC                                                                                                      0x840e9af2b4b0  UDPv4   0.0.0.0 49304   *       0               2376    svchost.exe     2025-03-12 17:27:39.000000 UTC                                                                                              0x840e9af2b4b0  UDPv6   ::      49304   *       0               2376    svchost.exe     2025-03-12 17:27:39.000000 UTC                                                                                              0x840e9af2b640  UDPv4   0.0.0.0 61414   *       0               -       -       2025-03-12 17:27:40.000000 UTC                                                                                                      0x840e9af2b7d0  UDPv4   0.0.0.0 0       *       0               3308    svchost.exe     2025-03-12 17:30:25.000000 UTC                                                                                              0x840e9af2b7d0  UDPv6   ::      0       *       0               3308    svchost.exe     2025-03-12 17:30:25.000000 UTC                                                                                              0x840e9af2baf0  UDPv4   0.0.0.0 0       *       0               3308    svchost.exe     2025-03-12 17:30:25.000000 UTC                                                                                              0x840e9af2baf0  UDPv6   ::      0       *       0               3308    svchost.exe     2025-03-12 17:30:25.000000 UTC                                                                                              0x840e9af2c5e0  UDPv4   0.0.0.0 53991   *       0               -       -       2025-03-12 17:30:30.000000 UTC                                                                                                      0x840e9af2c5e0  UDPv6   ::      53991   *       0               -       -       2025-03-12 17:30:30.000000 UTC                                                                                                      0x840e9b199a70  TCPv4   10.20.20.23     59883   35.86.58.69     80      CLOSED  14536   spoolsv.exe     2025-03-17 17:27:06.000000 UTC                                                                              0x840e9b2dd010  TCPv4   10.20.20.23     59882   35.86.58.69     80      CLOSED  14536   spoolsv.exe     2025-03-17 17:27:06.000000 UTC                                                                              0x840e9bdeca20  TCPv4   10.20.20.23     59884   35.86.58.69     80      ESTABLISHED     14536   spoolsv.exe     2025-03-17 17:27:09.000000 UTC                                                                      0x840e9c0d75a0  TCPv4   10.20.20.23     59433   204.79.197.203  443     CLOSED  9092    SearchApp.exe   2025-03-17 17:15:06.000000 UTC                                                                              0x840e9ce6c880  TCPv4   10.20.20.23     59881   35.86.58.69     80      CLOSED  14536   spoolsv.exe     2025-03-17 17:27:06.000000 UTC                                                                              0x840e9d7df270  TCPv4   10.20.20.23     59430   20.189.173.28   443     CLOSED  9092    SearchApp.exe   2025-03-17 17:15:01.000000 UTC                                                                              0x840e9ddd75a0  TCPv4   10.20.20.23     59427   23.209.84.48    443     CLOSE_WAIT      9092    SearchApp.exe   2025-03-17 17:15:00.000000 UTC                                                                      0x840e9ea7da20  TCPv4   10.20.20.23     59875   35.86.58.69     80      CLOSED  14536   spoolsv.exe     2025-03-17 17:26:49.000000 UTC                                                                              0x840e9eb9a6e0  TCPv4   10.20.20.23     59880   10.30.0.107     9997    ESTABLISHED     13620   splunkd.exe     2025-03-17 17:27:06.000000 UTC                                                                      0x840e9f319010  TCPv4   10.20.20.23     59429   23.209.84.48    443     CLOSE_WAIT      9092    SearchApp.exe   2025-03-17 17:15:00.000000 UTC                                                                      
```

**notes**: The analysis of network connections reveals two suspicious activities. The first connection, established by the `excel-update.exe` process (PID 4672), communicates with the IP address `35.86.58.69` on port `4443`, a non-standard HTTPS port, suggesting potential C2 communication. The second connection, initiated by the `ScreenConnect` process (PID 6988), connects to `139.178.70.144` on port `433`, a standard HTTPS port typically used for remote access tools. Both connections are marked as `ESTABLISHED`, indicating active communication between the compromised machine `(10.20.20.23)` and the remote attacker’s servers. These connections point to potential malicious activity, such as malware exfiltration and persistent remote access.

### Deeper Memory Process Analysis
Now that we have identified established remote connections with possible C2 activity, we will examine handles, privileges, dlllist to further analyze processes to generate additional indicators of compromise.


##### Handles
Handles in Windows are references to system resources used by processes, and analyzing them helps identify what objects (files, registry keys, processes) a program is interacting with, including potential security risks like unauthorized access or malicious activity.

`vol -f iveep.mem windows.handles`

```
Volatility 3 Framework 2.26.2

PID     Process Offset  HandleValue     Type    GrantedAccess   Name

4       System  0x840e90261040  0x4     Process 0x1fffff        System Pid 4
4       System  0x840e902d1140  0x8     Thread  0x1fffff        Tid 28 Pid 4
4       System  0xb707fbd9db40  0xc     Key     0x10    MACHINE\SYSTEM\CONTROLSET001\CONTROL\NETWORKPROVIDER\PROVIDERORDER
4       System  0x840e9028b320  0x10    Mutant  0x1f0001        BcdSyncMutant
4       System  0xb707fb855a40  0x14    Directory       0xf000f GLOBAL??
4       System  0xb707fb8993c0  0x18    Directory       0xf000f -
4       System  0x840e902ac960  0x1c    Partition       0x1f0003        MemoryPartition0
4       System  0xb707fb899680  0x20    Directory       0xf000f KernelObjects    
4      System  0x840e96fde040  0x1548  Thread  0x1fffff        Tid 3996 Pid 4
4       System  0xb708080187d0  0x154c  SymbolicLink    0xf0001 SWD#MSRRAS#MS_NDISWANIPV6#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0x840e96f6d2c0  0x1550  Process 0x102a  svchost.exe Pid 3880
4       System  0x840e96f1d300  0x1554  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e96f6d2c0  0x1558  Process 0x1fffff        svchost.exe Pid 3880
4       System  0x840e9ca64940  0x155c  IoCompletion    0x1f0003        -
4       System  0x840e96f6d2c0  0x1560  Process 0x102a  svchost.exe Pid 3880
4       System  0x840e96de8260  0x1564  Event   0x1f0003        -
4       System  0x840e96ed1340  0x1568  IoCompletion    0x1f0003        -
4       System  0x840e96807380  0x156c  WaitCompletionPacket    0x1     -
4       System  0x840e96de86e0  0x1570  Event   0x1f0003        -
4       System  0x840e96ed1080  0x1574  IoCompletion    0x1f0003        -
4       System  0x840e9680b210  0x1578  WaitCompletionPacket    0x1     -
4       System  0x840e96ed3c40  0x157c  IoCompletion    0x1f0003        -
4       System  0x840e91bc2320  0x1580  WaitCompletionPacket    0x1     -
4       System  0x840e96fdc040  0x1584  Thread  0x1fffff        Tid 3988 Pid 4
4       System  0x840e970b8ac0  0x1588  IoCompletion    0x1f0003        -
4       System  0x840e96fdb040  0x158c  Thread  0x1fffff        Tid 3984 Pid 4
4       System  0x840e96fdf040  0x1590  Thread  0x1fffff        Tid 4000 Pid 4
4       System  0x840e96fe0040  0x1594  Thread  0x1fffff        Tid 4004 Pid 4
4       System  0x840e96fe1040  0x1598  Thread  0x1fffff        Tid 4008 Pid 4
4       System  0x840e96fe2040  0x159c  Thread  0x1fffff        Tid 4012 Pid 4
4       System  0x840e96fe3040  0x15a0  Thread  0x1fffff        Tid 4016 Pid 4
4       System  0x840e96fe4040  0x15a4  Thread  0x1fffff        Tid 4020 Pid 4
4       System  0x840e96fe5040  0x15a8  Thread  0x1fffff        Tid 4024 Pid 4
4       System  0x840e96fe6040  0x15ac  Thread  0x1fffff        Tid 4028 Pid 4
4       System  0x840eab2f08e0  0x15b0  Event   0x1f0003        -
4       System  0x840e9709ba00  0x15b4  Timer   0x2     -
4       System  0xb708069c9580  0x15b8  SymbolicLink    0xf0001 SWD#MSRRAS#MS_AGILEVPNMINIPORT#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0xb708079c1900  0x15bc  SymbolicLink    0xf0001 SWD#MSRRAS#MS_PPPOEMINIPORT#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0xb708079c1d80  0x15c0  SymbolicLink    0xf0001 SWD#MSRRAS#MS_L2TPMINIPORT#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0x840e971062e0  0x15c4  Event   0x1f0003        -
4       System  0x840e970bd600  0x15c8  IoCompletion    0x1f0003        -
4       System  0xb70808018500  0x15cc  SymbolicLink    0xf0001 SWD#MSRRAS#MS_NDISWANIPV6#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0xb708079c23b0  0x15d0  SymbolicLink    0xf0001 SWD#MSRRAS#MS_PPTPMINIPORT#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0xb708079c1630  0x15d4  SymbolicLink    0xf0001 SWD#MSRRAS#MS_PPPOEMINIPORT#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0xb708079c28c0  0x15d8  SymbolicLink    0xf0001 SWD#MSRRAS#MS_PPTPMINIPORT#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0x840e9715d080  0x15dc  Process 0x102a  Sysmon64.exe Pid 3696
4       System  0x840e97159300  0x15e0  Process 0x1fffff        CSFalconServic Pid 3648
4       System  0x840e92def080  0x15e4  Process 0x102a  svchost.exe Pid 1292
4       System  0xb708079c5b60  0x15e8  SymbolicLink    0xf0001 SWD#MSRRAS#MS_NDISWANBH#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0xb708079c6bb0  0x15ec  SymbolicLink    0xf0001 SWD#MSRRAS#MS_NDISWANBH#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0x840e97117de0  0x15f0  Event   0x1f0003        -
4       System  0x840e970f3080  0x15f8  Process 0x102a  svchost.exe Pid 3172
4       System  0x840ea3cf5060  0x15fc  Event   0x1f0003        -
4       System  0x840e98ec91c0  0x1600  Process 0x102a  RuntimeBroker. Pid 10968
4       System  0x840e99f5b500  0x1604  IoCompletion    0x1f0003        -
4       System  0x840e97135830  0x1608  WaitCompletionPacket    0x1     -
4       System  0x840e96e0d080  0x160c  Process 0x102a  svchost.exe Pid 3844
4       System  0x840e96dc4080  0x1610  Process 0x102a  svchost.exe Pid 3352
4       System  0x840e971154e0  0x1614  Event   0x1f0003        -
4       System  0x840e971697d0  0x1618  File    0x12019f        \Device\NamedPipe\
4       System  0x840e92ee5450  0x161c  WaitCompletionPacket    0x1     -
4       System  0x840e92def080  0x1624  Process 0x102a  svchost.exe Pid 1292
4       System  0x840e970f3080  0x1628  Process 0x102a  svchost.exe Pid 3172
4       System  0x840e970f3080  0x162c  Process 0x1fffff        svchost.exe Pid 3172
4       System  0x840e97159300  0x1630  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e9676e080  0x1634  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e91ba00c0  0x1638  Process 0x102a  svchost.exe Pid 7220
4       System  0x840e9715d080  0x163c  Process 0x1fffff        Sysmon64.exe Pid 3696
4       System  0x840e97358b80  0x1640  IoCompletion    0x1f0003        -
4       System  0x840e9b9850f0  0x1644  WaitCompletionPacket    0x1     -
4       System  0x840e984fccb0  0x1648  Timer   0x2     -
4       System  0x840e9a14e270  0x164c  File    0x3     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-5-21-3294271487-4292050549-1661813568-1001\SystemAppData\Helium\Cache\b4d622c333fcfdd7_COM15.dat.LOG2
4       System  0x840e9715d080  0x1650  Process 0x102a  Sysmon64.exe Pid 3696
4       System  0x840e970f3080  0x1654  Process 0x102a  svchost.exe Pid 3172
4       System  0x840e97442080  0x1658  Process 0x102a  CSFalconContai Pid 6624
4       System  0x840e97159300  0x165c  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e970c3380  0x1660  IoCompletion    0x1f0003        -
4       System  0x840e9713a7d0  0x1664  WaitCompletionPacket    0x1     -
4       System  0x840e97159300  0x1668  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e971c9ce0  0x166c  ALPC Port       0x1f0001        -
4       System  0x840e96e44300  0x1670  Process 0x102a  MsMpEng.exe Pid 3540
4       System  0x840e97159300  0x1674  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e971b2a80  0x1678  ALPC Port       0x1f0001        -
4       System  0x840e9710b960  0x167c  Event   0x1f0003        -
4       System  0x840e971faa80  0x1680  ALPC Port       0x1f0001        -
4       System  0x840e96e44300  0x1684  Process 0x102a  MsMpEng.exe Pid 3540
4       System  0x840e99bcb080  0x1688  Process 0x102a  explorer.exe Pid 9852
4       System  0x840e9ac2c560  0x168c  WaitCompletionPacket    0x1     -
4       System  0x840e97159300  0x1690  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e9c709f60  0x1694  Event   0x1f0003        -
4       System  0x840e970ce600  0x1698  IoCompletion    0x1f0003        -
4       System  0x840e971125e0  0x169c  Event   0x1f0003        -
4       System  0x840e9818fe40  0x16a4  WaitCompletionPacket    0x1     -
4       System  0x840e99f51c00  0x16a8  IoCompletion    0x1f0003        -
4       System  0x840e9f3a3c00  0x16ac  IoCompletion    0x1f0003        -
4       System  0x840e972491c0  0x16b0  Process 0x1fffff        svchost.exe Pid 4352
4       System  0x840e96e44300  0x16b4  Process 0x102a  MsMpEng.exe Pid 3540
4       System  0x840e96da72c0  0x16b8  Process 0x102a  svchost.exe Pid 3324
4       System  0x840e973d81c0  0x16bc  Process 0x102a  svchost.exe Pid 4800
4       System  0x840e96621060  0x16c0  FilterCommunicationPort 0x1f0001        -
4       System  0x840e97c4fa60  0x18ac  Event   0x1f0003        -
4       System  0x840e97108660  0x18b0  Event   0x1f0003        -
4       System  0x840e97b5c750  0x18b4  File    0x12019f        \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e91bcc670  0x18b8  WaitCompletionPacket    0x1     -
4       System  0x840e96f1d300  0x18bc  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e975ad080  0x18c0  Process 0x102a  CSFalconContai Pid 1192
4       System  0x840e96f1d300  0x18c8  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e96f1d300  0x18d0  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e99f508c0  0x18d4  IoCompletion    0x1f0003        -
4       System  0x840e9ac23730  0x18d8  WaitCompletionPacket    0x1     -
4       System  0x840e9ac34c40  0x18dc  WaitCompletionPacket    0x1     -
4       System  0x840e974b4080  0x18e0  Process 0x1fffff        CSFalconContai Pid 324
4       System  0x840e9979e6e0  0x18e4  Event   0x1f0003        -
4       System  0x840e98b374a0  0x18e8  WaitCompletionPacket    0x1     -
4       System  0x840e980e3c80  0x18ec  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTcs-etw-stp.etl
4       System  0x840e99c6e080  0x18f0  Process 0x1fffff        svchost.exe Pid 4636
4       System  0x840e98dbf2c0  0x18f8  Process 0x1fffff        svchost.exe Pid 8688
4       System  0x840e999e30e0  0x18fc  Event   0x1f0003        -
4       System  0x840e990373c0  0x1900  IoCompletion    0x1f0003        -
4       System  0x840e99bcb080  0x1908  Process 0x1fffff        explorer.exe Pid 9852
4       System  0x840e97d5cb00  0x190c  IoCompletion    0x1f0003        -
4       System  0x840e9b993950  0x1914  WaitCompletionPacket    0x1     -
4       System  0x840e9b280910  0x1920  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.ShellExperienceHost_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e96c5a080  0x1924  Process 0x102a  svchost.exe Pid 7108
4       System  0x840e97677940  0x1928  IoCompletion    0x1f0003        -
4       System  0x840e9c903fe0  0x192c  Event   0x1f0003        -
4       System  0xb70804826040  0x1930  Token   0xe     -
4       System  0x840e97655740  0x1934  IoCompletion    0x1f0003        -
4       System  0x840e98c1c660  0x1938  Event   0x1f0003        -
4       System  0xb7081e55c8a0  0x193c  Directory       0xf     RPC Control
4       System  0x840e91c2a080  0x1940  Process 0x102a  svchost.exe Pid 5340
4       System  0x840e96d9f080  0x1948  Process 0x102a  svchost.exe Pid 3308
4       System  0x840e90b0d080  0x194c  Process 0x0     svchost.exe Pid 9452
4       System  0x840e96f1d300  0x1950  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e968087d0  0x1954  WaitCompletionPacket    0x1     -
4       System  0x840e90ad3080  0x1958  Process 0x102a  svchost.exe Pid 7152
4       System  0x840e98e742c0  0x195c  Process 0x102a  svchost.exe Pid 8292
4       System  0x840e975c90a0  0x1960  File    0x120089        \Device\HarddiskVolume3\Windows\CSC
4       System  0x840e990382c0  0x1964  IoCompletion    0x1f0003        -
4       System  0xb708106b2c00  0x1968  Directory       0xf000f 00000000-0258d92d
4       System  0x840e9ca96a60  0x196c  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-536077884-713174666-1066051701-3219990555-339840825-1966734348-1611281757
4       System  0x840e96f1d300  0x1970  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e98a3f580  0x1974  IoCompletion    0x1f0003        -
4       System  0x840e9c8be7f0  0x1978  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee29-852e-11ef-ba7c-bc2411845bc2}.TxR.2.regtrans-ms
4       System  0x840e992afc40  0x197c  IoCompletion    0x1f0003        -
4       System  0xb70812e2d060  0x1980  Token   0xf01ff -
4       System  0x840e99ac7f40  0x1984  IoCompletion    0x1f0003        -
4       System  0x840e998450c0  0x1988  Process 0x1fffff        SearchApp.exe Pid 4236
4       System  0x840e91e7c5c0  0x198c  ALPC Port       0x1f0001        -
4       System  0x840e97c4ed60  0x1990  Event   0x1f0003        -
4       System  0xb708109e1c40  0x1994  SymbolicLink    0xf0001 SWD#PRINTENUM#{5921CECD-FF14-4BE5-8254-F33210B5C52D}#{0ecef634-6ef0-472a-8085-5ad023ecbccd}
4       System  0x840e9dacf2c0  0x1998  IoCompletion    0x1f0003        -
4       System  0x840e99a92080  0x199c  Process 0x1fffff        SecurityHealth Pid 12036
4       System  0x840e9bdf4340  0x19a0  Process 0x102a  ScreenConnect. Pid 14756
4       System  0x840e9229bc60  0x19a4  Event   0x1f0003        -
4       System  0x840e990a2230  0x19a8  WaitCompletionPacket    0x1     -
4       System  0x840e97180380  0x19ac  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTSYSMON TRACE.etl
4       System  0x840e9daeba80  0x19b0  IoCompletion    0x1f0003        -
4       System  0x840e92edd180  0x19b4  WaitCompletionPacket    0x1     -
4       System  0x840e98d06080  0x19b8  IoCompletion    0x1f0003        -
4       System  0x840e99096dd0  0x19bc  WaitCompletionPacket    0x1     -
4       System  0x840e9715d080  0x19c0  Process 0x1028  Sysmon64.exe Pid 3696
4       System  0x840e97182770  0x19c4  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTSysmonDnsEtwSession.etl
4       System  0x840e974b7080  0x19c8  Process 0x1fffff        CSFalconContai Pid 5980
4       System  0x840e99094d50  0x19cc  WaitCompletionPacket    0x1     -
4       System  0x840e9daeca40  0x19d0  IoCompletion    0x1f0003        -
4       System  0x840e993526e0  0x19d4  Event   0x1f0003        -
4       System  0x840e975ad080  0x19dc  Process 0x1fffff        CSFalconContai Pid 1192
4       System  0x840e974b7080  0x19e8  Process 0x102a  CSFalconContai Pid 5980
4       System  0x840e9715d080  0x19ec  Process 0x1028  Sysmon64.exe Pid 3696
4       System  0x840e99f79900  0x19f0  IoCompletion    0x1f0003        -
4       System  0x840e96c5a080  0x19f4  Process 0x102a  svchost.exe Pid 7108
4       System  0x840e98a5a1f0  0x19f8  IoCompletionReserve     0xf0003 -
4       System  0x840ea0ec9360  0x19fc  Event   0x1f0003        -
4       System  0x840e9817fa40  0x1a00  WaitCompletionPacket    0x1     -
4       System  0x840e9715d080  0x1a04  Process 0x1fffff        Sysmon64.exe Pid 3696
4       System  0x840e990ad690  0x1a08  WaitCompletionPacket    0x1     -
4       System  0x840e9a351920  0x1a0c  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\windows.immersivecontrolpanel_10.0.2.1000_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat
4       System  0x840e9715d080  0x1a10  Process 0x102a  Sysmon64.exe Pid 3696
4       System  0x840e9715d080  0x1a14  Process 0x102a  Sysmon64.exe Pid 3696
4       System  0x840e96bf6b60  0x1a18  Event   0x1f0003        -
4       System  0xb707fbcb4190  0x1a1c  SymbolicLink    0xf0001 Global
4       System  0x840e9b9b2dd0  0x1a20  WaitCompletionPacket    0x1     -
4       System  0x840e9ac0f230  0x1a24  WaitCompletionPacket    0x1     -
4       System  0x840e9a6280c0  0x1a28  IoCompletion    0x1f0003        -
4       System  0x840e9ee430c0  0x1a2c  Process 0x102a  powershell.exe Pid 13156
4       System  0x840ea47c0990  0x1a30  WaitCompletionPacket    0x1     -
4       System  0x840e97f80080  0x1a38  Process 0x102a  AggregatorHost Pid 5804
4       System  0x840e99bcb080  0x1a3c  Process 0x102a  explorer.exe Pid 9852
4       System  0x840e97d74b00  0x1a40  IoCompletion    0x1f0003        -
4       System  0xb708095b5d80  0x1a44  Directory       0xf000f S-1-15-2-233136611-892874861-1880231551-514580003-522390519-4292070654-795176919
4       System  0x840e999e5b60  0x1a48  Event   0x1f0003        -
4       System  0x840e97121f60  0x1a4c  Event   0x1f0003        -
4       System  0x840e97c36760  0x1a50  File    0x3     \Device\HarddiskVolume3\Windows\appcompat\Programs\Amcache.hve.LOG2
4       System  0x840e97949390  0x1a54  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e980e50a0  0x1a58  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTcs-etw.etl
4       System  0x840e9970dde0  0x1a6c  Event   0x1f0003        -
4       System  0x840e98fe0be0  0x1a70  Event   0x1f0003        -
4       System  0x840e9971d660  0x1a78  Event   0x1f0003        -
4       System  0x840e983e8380  0x1a7c  TmRm    0x1f007f        -
4       System  0x840e90bb8a60  0x1a80  Event   0x1f0003        -
4       System  0x840e992a7e40  0x1a84  IoCompletion    0x1f0003        -
4       System  0x840e97ab3ad0  0x1a8c  File    0x12019f        \Device\HarddiskVolume3\Users\admin\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000002.regtrans-ms
4       System  0x840e9b372080  0x1a90  Process 0x102a  taskhostw.exe Pid 10476
4       System  0x840e9ca3ecc0  0x1a94  IoCompletion    0x1f0003        -
4       System  0x840e9a95c8e0  0x1a98  Event   0x1f0003        -
4       System  0x840e9c250080  0x1a9c  Process 0x102a  SenseIR.exe Pid 7728
4       System  0x840e96d56340  0x1aa0  Process 0x0     SenseTVM.exe Pid 1552
4       System  0x840e96d09280  0x1aa4  IoCompletion    0x1f0003        -
4       System  0xb70810d63be0  0x1aa8  Key     0xf003f MACHINE\SYSTEM\DRIVERDATABASE\DRIVERPACKAGES
4       System  0x840e91d8d7e0  0x1ab0  Event   0x1f0003        -
4       System  0x840e9ac2e850  0x1ab4  WaitCompletionPacket    0x1     -
4       System  0x840e997210e0  0x1ab8  Event   0x1f0003        -
4       System  0x840e90cf27e0  0x1abc  Event   0x1f0003        -
4       System  0x840e90bb4b60  0x1ac0  Event   0x1f0003        -
4       System  0x840e97949500  0x1ac4  File    0x12019f        \Device\Tcp
4       System  0x840ea0ed43e0  0x1ac8  Event   0x1f0003        -
4       System  0x840e9794b8f0  0x1acc  File    0x12019f        \Device\Tcp
4       System  0x840e98ff32c0  0x1ad0  Process 0x102a  SecurityHealth Pid 8808
4       System  0x840e97460c60  0x1ad4  Event   0x1f0003        -
4       System  0x840e99f501c0  0x1ad8  IoCompletion    0x1f0003        -
4       System  0x840e974b7080  0x1adc  Process 0x102a  CSFalconContai Pid 5980
4       System  0x840e9ac374e0  0x1ae0  WaitCompletionPacket    0x1     -
4       System  0x840e978de2f0  0x1ae4  WaitCompletionPacket    0x1     -
4       System  0x840e97949670  0x1ae8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9716a7a0  0x1aec  File    0x180   \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e97add1c0  0x1af0  Process 0x102a  svchost.exe Pid 6020
4       System  0x840e979497e0  0x1af4  File    0x12019f        \Device\Tcp
4       System  0x840e9794b610  0x1af8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794b330  0x1afc  File    0x12019f        \Device\Tcp
4       System  0x840e97949950  0x1b00  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e97949ac0  0x1b04  File    0x12019f        \Device\Tcp
4       System  0x840e9794b780  0x1b08  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e97949c30  0x1b0c  File    0x12019f        \Device\Tcp
4       System  0x840e9794ba60  0x1b10  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794a640  0x1b14  File    0x12019f        \Device\Tcp
4       System  0x840e9794a920  0x1b18  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794a7b0  0x1b1c  File    0x12019f        \Device\Tcp
4       System  0x840e97949da0  0x1b20  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794a080  0x1b24  File    0x12019f        \Device\Tcp
4       System  0x840e97948dd0  0x1b28  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794ac00  0x1b2c  File    0x12019f        \Device\Tcp
4       System  0x840e9794b1c0  0x1b30  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794a4d0  0x1b34  File    0x12019f        \Device\Tcp
4       System  0x840e9794a1f0  0x1b38  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794aa90  0x1b3c  File    0x12019f        \Device\Tcp
4       System  0x840e9794a360  0x1b40  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794b4a0  0x1b44  File    0x12019f        \Device\Tcp
4       System  0x840e9794ad70  0x1b48  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794aee0  0x1b4c  File    0x12019f        \Device\Tcp
4       System  0x840e979490b0  0x1b50  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794c190  0x1b54  File    0x12019f        \Device\Tcp
4       System  0x840e9794ca30  0x1b58  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794c300  0x1b5c  File    0x12019f        \Device\Tcp
4       System  0x840e9794c5e0  0x1b60  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794cd10  0x1b64  File    0x12019f        \Device\Tcp
4       System  0x840e9794e580  0x1b68  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794e6f0  0x1b6c  File    0x12019f        \Device\Tcp
4       System  0x840e9794c470  0x1b70  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794cba0  0x1b74  File    0x12019f        \Device\Tcp
4       System  0x840e9794e410  0x1b78  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794e860  0x1b7c  File    0x12019f        \Device\Tcp
4       System  0x840e9794c750  0x1b80  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794c8c0  0x1b84  File    0x12019f        \Device\Tcp
4       System  0x840e9794ce80  0x1b88  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794d160  0x1b8c  File    0x12019f        \Device\Tcp
4       System  0x840e9794bbd0  0x1b90  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794beb0  0x1b94  File    0x12019f        \Device\Tcp
4       System  0x840e9794d720  0x1b98  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794d2d0  0x1b9c  File    0x12019f        \Device\Tcp
4       System  0x840e9794e130  0x1ba0  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794d890  0x1ba4  File    0x12019f        \Device\Tcp
4       System  0x840e9794d440  0x1ba8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794bd40  0x1bac  File    0x12019f        \Device\Tcp
4       System  0x840e9794d5b0  0x1bb0  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794db70  0x1bb4  File    0x12019f        \Device\Tcp
4       System  0x840e9794da00  0x1bb8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794de50  0x1bbc  File    0x12019f        \Device\Tcp
4       System  0x840e9794dce0  0x1bc0  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e979510a0  0x1bc4  File    0x12019f        \Device\Tcp
4       System  0x840e9794e2a0  0x1bc8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e97951210  0x1bcc  File    0x12019f        \Device\Tcp
4       System  0x840e9794ee20  0x1bd0  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e979500d0  0x1bd4  File    0x12019f        \Device\Tcp
4       System  0x840e9794ecb0  0x1bd8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e97951380  0x1bdc  File    0x12019f        \Device\Tcp
4       System  0x840e97950800  0x1be0  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e974b7080  0x1be4  Process 0x102a  CSFalconContai Pid 5980
4       System  0x840e9229ade0  0x1be8  Event   0x1f0003        -
4       System  0x840e97c640c0  0x1bec  Process 0x102a  powershell.exe Pid 10352
4       System  0x840e992bb340  0x1bf0  IoCompletion    0x1f0003        -
4       System  0x840e9737a3c0  0x1bf4  IoCompletion    0x1f0003        -
4       System  0x840e970c5e80  0x1bf8  IoCompletion    0x1f0003        -
4       System  0x840e97add1c0  0x1bfc  Process 0x102a  svchost.exe Pid 6020
4       System  0x840e97aefbd0  0x1c04  TmTm    0xf003f -
4       System  0x840e9c9332e0  0x1c08  Event   0x1f0003        -
4       System  0x840e9daca140  0x1c0c  IoCompletion    0x1f0003        -
4       System  0x840e96c23280  0x1c10  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e99ab9840  0x1c14  IoCompletion    0x1f0003        -
4       System  0x840e981ae900  0x1c18  WaitCompletionPacket    0x1     -
4       System  0x840e99840080  0x1c1c  Process 0x0     dllhost.exe Pid 2492
4       System  0x840e9c4747c0  0x1c20  IoCompletion    0x1f0003        -
4       System  0x840e98c136e0  0x1c24  Event   0x1f0003        -
4       System  0x840e9ba33a80  0x1c28  WaitCompletionPacket    0x1     -
4       System  0x840e98a1d7c0  0x1c2c  IoCompletion    0x1f0003        -
4       System  0xb7082045c5d0  0x1c30  Section 0x17    -
4       System  0x840e96c23280  0x1c34  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e9a34d140  0x1c38  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\windows.immersivecontrolpanel_10.0.2.1000_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG1
4       System  0x840e97c34820  0x1c3c  File    0x3     \Device\HarddiskVolume3\Windows\appcompat\Programs\Amcache.hve.LOG1
4       System  0xb70814b6d330  0x1c40  Section 0x5     -
4       System  0x840e9b998270  0x1c44  WaitCompletionPacket    0x1     -
4       System  0x840e99ac0a00  0x1c48  IoCompletion    0x1f0003        -
4       System  0x840ea534e160  0x1c50  Event   0x1f0003        -
4       System  0xb708044ae060  0x1c54  Token   0xf01ff -
4       System  0x840e97dc72c0  0x1c58  Process 0x102a  SearchIndexer. Pid 1852
4       System  0x840e99aab4c0  0x1c5c  IoCompletion    0x1f0003        -
4       System  0x840ea47c48f0  0x1c60  WaitCompletionPacket    0x1     -
4       System  0x840e97dc72c0  0x1c64  Process 0x1fffff        SearchIndexer. Pid 1852
4       System  0x840e97f80080  0x1c68  Process 0x1fffff        AggregatorHost Pid 5804
4       System  0x840e98c2d9e0  0x1c6c  Event   0x1f0003        -
4       System  0x840e97159300  0x1c74  Process 0x1028  CSFalconServic Pid 3648
4       System  0x840e99e240c0  0x1c78  Process 0x1fffff        ScreenConnect. Pid 6988
4       System  0x840e9cea20e0  0x1c7c  Event   0x1f0003        -
4       System  0x840e993240c0  0x1c80  Process 0x102a  sihost.exe Pid 10384
4       System  0x840ea7061080  0x1c84  Process 0x1fffff        svchost.exe Pid 16024
4       System  0x840e974b4080  0x1c8c  Process 0x102a  CSFalconContai Pid 324
4       System  0x840e92f32940  0x1c90  IoCompletion    0x1f0003        -
4       System  0x840e99e240c0  0x1c94  Process 0x102a  ScreenConnect. Pid 6988
4       System  0x840e97f7e080  0x1c98  Process 0x102a  svchost.exe Pid 5772
4       System  0x840e97ab5d30  0x1c9c  File    0x12019f        \Device\HarddiskVolume3\Users\admin\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM.blf
4       System  0x840e97c4a1e0  0x1ca0  Event   0x1f0003        -
4       System  0xb70805b847b0  0x1ca4  Token   0xe     -
4       System  0x840e9230b100  0x1ca8  Process 0x0     services.exe Pid 1000
4       System  0xb70808820040  0x1cac  Token   0xe     -
4       System  0x840e97d49b40  0x1cb0  IoCompletion    0x1f0003        -
4       System  0x840ea5a198e0  0x1cb4  Event   0x1f0003        -
4       System  0x840e98c271e0  0x1cb8  Event   0x1f0003        -
4       System  0x840e97c52a60  0x1cbc  Event   0x1f0003        -
4       System  0x840e97d4a400  0x1cc8  IoCompletion    0x1f0003        -
4       System  0x840e98bdf080  0x1ccc  Process 0x40    SystemSettings Pid 10020
4       System  0xb708053902d0  0x1cd0  Token   0xe     -
4       System  0x840e978e4470  0x1cd4  WaitCompletionPacket    0x1     -
4       System  0x840e990b2490  0x1cd8  WaitCompletionPacket    0x1     -
4       System  0x840e999e58e0  0x1cdc  Event   0x1f0003        -
4       System  0x840e92cc1090  0x1ce0  IoCompletionReserve     0xf0003 -
4       System  0x840e9760aca0  0x1ce4  ALPC Port       0x1f0001        -
4       System  0x840e96da72c0  0x1ce8  Process 0x102a  svchost.exe Pid 3324
4       System  0x840e980d08e0  0x1cec  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTDiagtrack-Listener.etl
4       System  0x840e91ee7080  0x1cf0  Process 0x102a  Sysmon64.exe Pid 1540
4       System  0x840e9c89a060  0x1cf4  Event   0x1f0003        -
4       System  0x840e97159300  0x1cf8  Process 0x1028  CSFalconServic Pid 3648
4       System  0x840e990ab7b0  0x1cfc  WaitCompletionPacket    0x1     -
4       System  0x840e91d9ace0  0x1d00  Event   0x1f0003        -
4       System  0x840e97fafb60  0x1d04  Event   0x1f0003        -
4       System  0x840e97f7e080  0x1d08  Process 0x102a  svchost.exe Pid 5772
4       System  0x840e9c5aef60  0x1d0c  Event   0x1f0003        -
4       System  0x840e97c2f230  0x1d10  File    0x20003 \Device\HarddiskVolume3\Windows\appcompat\Programs\Amcache.hve
4       System  0x840e9783b080  0x1d14  Process 0x102a  CSFalconContai Pid 6604
4       System  0x840e99bdd080  0x1d18  Process 0x40    SearchApp.exe Pid 9288
4       System  0x840e9907dd40  0x1d1c  WaitCompletionPacket    0x1     -
4       System  0x840e97d55b00  0x1d20  IoCompletion    0x1f0003        -
4       System  0x840e9b7bb080  0x1d24  Process 0x0     StartMenuExper Pid 8
4       System  0xb7080a2764b0  0x1d28  Section 0x17    -
4       System  0x840e9680cdb0  0x1d2c  WaitCompletionPacket    0x1     -
4       System  0x840e974644e0  0x1d30  Event   0x1f0003        -
4       System  0x840e97f7e080  0x1d34  Process 0x1fffff        svchost.exe Pid 5772
4       System  0x840e992b9200  0x1d38  IoCompletion    0x1f0003        -
4       System  0xb70816eba3a0  0x1d3c  Directory       0xf     S-1-15-2-2246530975-808720366-1776470054-230329187-4153223113-3550430174-4193313734
4       System  0x840e978e7600  0x1d40  WaitCompletionPacket    0x1     -
4       System  0x840e9170e740  0x1d44  WaitCompletionPacket    0x1     -
4       System  0x840e97811060  0x1d48  Event   0x1f0003        -
4       System  0x840e97d4c1c0  0x1d4c  IoCompletion    0x1f0003        -
4       System  0x840e975ad080  0x1d50  Process 0x102a  CSFalconContai Pid 1192
4       System  0x840e97349860  0x1d54  FilterCommunicationPort 0x1f0001        -
4       System  0xb7080874e870  0x1d58  Section 0x17    -
4       System  0x840e96f1d300  0x1d5c  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e999dd0e0  0x1d60  Event   0x1f0003        -
4       System  0x840e974b4080  0x1d64  Process 0x102a  CSFalconContai Pid 324
4       System  0x840e99fd2960  0x1d70  Event   0x1f0003        -
4       System  0x840e96f1d300  0x1d74  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840ea08f3080  0x1d7c  Process 0x102a  svchost.exe Pid 12008
4       System  0x840e98d0f980  0x1d80  IoCompletion    0x1f0003        -
4       System  0x840e97348a60  0x1d88  FilterCommunicationPort 0x1f0001        -
4       System  0x840e98c27d60  0x1d8c  Event   0x1f0003        -
4       System  0x840e97ab5ec0  0x1d90  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\admin\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0x840e97159300  0x1d94  Process 0x1028  CSFalconServic Pid 3648
4       System  0x840e91f131c0  0x1d98  Process 0x0     Sysmon64.exe Pid 1244
4       System  0x840e98c48080  0x1d9c  Process 0x102a  ApplicationFra Pid 4836
4       System  0x840e96f1d300  0x1da0  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e96f1d300  0x1da4  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e97136a10  0x1dac  WaitCompletionPacket    0x1     -
4       System  0x840e97ab3300  0x1db0  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\admin\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0x840e9b98f2a0  0x1db4  WaitCompletionPacket    0x1     -
4       System  0x840e97e0c8a0  0x1db8  TmRm    0x1f007f        -
4       System  0x840e974b2080  0x1dc0  Process 0x102a  UserOOBEBroker Pid 9620
4       System  0x840e9817d8f0  0x1dc4  WaitCompletionPacket    0x1     -
4       System  0x840e995680c0  0x1dc8  Process 0x102a  SgrmBroker.exe Pid 9948
4       System  0x840e978d4550  0x1dcc  WaitCompletionPacket    0x1     -
4       System  0x840e9c906860  0x1dd0  Event   0x1f0003        -
4       System  0x840e99f7bdc0  0x1dd4  IoCompletion    0x1f0003        -
4       System  0x840e97ab3c60  0x1dd8  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat
4       System  0x840e9b9867b0  0x1ddc  WaitCompletionPacket    0x1     -
4       System  0x840e98eb3080  0x1de0  Process 0x102a  conhost.exe Pid 10944
4       System  0x840e9819ef90  0x1de4  WaitCompletionPacket    0x1     -
4       System  0xb70808820040  0x1de8  Token   0xe     -
4       System  0xb70808820040  0x1dec  Token   0xe     -
4       System  0x840e9680d770  0x1df0  WaitCompletionPacket    0x1     -
4       System  0x840e97442080  0x1df4  Process 0x102a  CSFalconContai Pid 6624
4       System  0x840e91cd5cb0  0x1df8  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTcs-etw-sm.etl
4       System  0x840e97d5c780  0x1dfc  IoCompletion    0x1f0003        -
4       System  0x840e96808630  0x1e00  WaitCompletionPacket    0x1     -
4       System  0x840e90bb1ae0  0x1e04  Event   0x1f0003        -
4       System  0x840e97ab16e0  0x1e08  File    0x3     \Device\HarddiskVolume3\Users\admin\ntuser.dat.LOG1
4       System  0x840e981b37d0  0x1e0c  WaitCompletionPacket    0x1     -
4       System  0x840e96d4d350  0x1e10  Timer   0x1f0003        -
4       System  0x840e978e02a0  0x1e14  WaitCompletionPacket    0x1     -
4       System  0x840e96e0f080  0x1e18  Process 0x1fffff        sihost.exe Pid 6908
4       System  0x840e9971f860  0x1e1c  Event   0x1f0003        -
4       System  0x840e96e0f080  0x1e20  Process 0x102a  sihost.exe Pid 6908
4       System  0x840e97988ce0  0x1e24  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\MicrosoftWindows.Client.CBS_1000.19058.1000.0_x64__cw5n1h2txyewy\ActivationStore.dat
4       System  0xb70817928270  0x1e2c  Section 0x5     -
4       System  0x840e97ab5560  0x1e30  File    0x12019f        \Device\HarddiskVolume3\Users\admin\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000001.regtrans-ms
4       System  0x840e97648800  0x1e34  IoCompletion    0x1f0003        -
4       System  0x840e9a637b00  0x1e38  IoCompletion    0x1f0003        -
4       System  0x840e97ab0a60  0x1e3c  File    0x20003 \Device\HarddiskVolume3\Users\admin\NTUSER.DAT
4       System  0xb7080ec3ef60  0x1e40  Key     0xf003f USER\S-1-5-21-3294271487-4292050549-1661813568-1001\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\WINLOGON
4       System  0x840e97815760  0x1e44  Event   0x1f0003        -
4       System  0x840e9c9b2560  0x1e48  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e999f1760  0x1e50  Event   0x1f0003        -
4       System  0x840e91d8e6e0  0x1e54  Event   0x1f0003        -
4       System  0x840e9711e060  0x1e5c  Event   0x1f0003        -
4       System  0x840e9c70f6e0  0x1e60  Event   0x1f0003        -
4       System  0x840e97ab3620  0x1e64  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat.LOG1
4       System  0x840e9764de80  0x1e68  IoCompletion    0x1f0003        -
4       System  0x840e98d8a060  0x1e6c  TmTm    0xf003f -
4       System  0x840e9c236080  0x1e70  Process 0x102a  dllhost.exe Pid 5164
4       System  0x840e97fc5160  0x1e74  Event   0x1f0003        -
4       System  0x840e96c5a080  0x1e78  Process 0x102a  svchost.exe Pid 7108
4       System  0x840e97ab48e0  0x1e7c  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat.LOG2
4       System  0x840e98fe0ce0  0x1e84  Event   0x1f0003        -
4       System  0x840e98aef080  0x1e88  Process 0x1fffff        RuntimeBroker. Pid 7684
4       System  0x840e90ad3080  0x1e8c  Process 0x1fffff        svchost.exe Pid 7152
4       System  0x840e99bcb080  0x1e90  Process 0x0     explorer.exe Pid 9852
4       System  0x840e99016980  0x1e94  IoCompletion    0x1f0003        -
4       System  0x840e97468560  0x1e98  Event   0x1f0003        -
4       System  0x840e98b2cee0  0x1e9c  WaitCompletionPacket    0x1     -
4       System  0xb708087665b0  0x1ea0  Section 0x17    -
4       System  0x840e96771c40  0x1ea4  TmRm    0x1f007f        -
4       System  0x840e97ab6690  0x1ea8  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat{54a96616-79ef-11ef-ba74-bc2411d898d1}.TM
4       System  0x840e97ab6500  0x1eac  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat{54a96616-79ef-11ef-ba74-bc2411d898d1}.TM
4       System  0x840e97af0c60  0x1eb0  TmRm    0x1f007f        -
4       System  0x840e96f29060  0x1eb4  TmTm    0xf003f -

4       System  0x840e999e3c60  0x2164  Event   0x1f0003        -
4       System  0x840e90bc8660  0x2168  Event   0x1f0003        -
4       System  0xb708109e1610  0x216c  SymbolicLink    0xf0001 SWD#PRINTENUM#{A763D421-EDFD-4DAB-8863-40F4DE9FF327}#{0ecef634-6ef0-472a-8085-5ad023ecbccd}
4       System  0x840e90e10410  0x2170  Session 0x0     Session2
4       System  0x840e9745de60  0x2174  Event   0x1f0003        -
4       System  0x840ea3ce7560  0x2178  Event   0x1f0003        -
4       System  0x840e975ca9a0  0x217c  File    0x12019f        \Device\HarddiskVolume3\Windows\CSC\v2.0.6\pq
4       System  0x840e98fe2e60  0x2184  Event   0x1f0003        -
4       System  0x840e98e969e0  0x2188  Event   0x1f0003        -
4       System  0x840e9907bbf0  0x218c  WaitCompletionPacket    0x1     -
4       System  0x840e9908fce0  0x2190  WaitCompletionPacket    0x1     -
4       System  0x840e9ac0f4a0  0x2194  WaitCompletionPacket    0x1     -
4       System  0x840e98fd3d60  0x2198  Event   0x1f0003        -
4       System  0x840e9a651eb0  0x21a0  IoCompletionReserve     0xf0003 -
4       System  0x840e9b9c1630  0x21a8  WaitCompletionPacket    0x1     -
4       System  0x840e96c5a080  0x21ac  Process 0x102a  svchost.exe Pid 7108
4       System  0xb7081bb5b400  0x21b0  Directory       0xf     RPC Control
4       System  0x840e91da7e60  0x21b4  Event   0x1f0003        -
4       System  0xb7081bb5c8a0  0x21bc  Directory       0xf     S-1-15-2-515815643-2845804217-1874292103-218650560-777617685-4287762684-137415000
4       System  0x840e9c714860  0x21c0  Event   0x1f0003        -
4       System  0x840e90ad3080  0x21c4  Process 0x102a  svchost.exe Pid 7152
4       System  0x840e975bb5e0  0x21c8  File    0x100081        \Device\HarddiskVolume3炮☺
4       System  0xb7080a29c390  0x21cc  Section 0x17    -
4       System  0x840e9904fe80  0x21d0  IoCompletion    0x1f0003        -
4       System  0x840e9ca5aa00  0x21d4  IoCompletion    0x1f0003        -
4       System  0x840e9cb526c0  0x21d8  IoCompletion    0x1f0003        -
4       System  0x840e992b5f40  0x21dc  IoCompletion    0x1f0003        -
4       System  0xb70816ebdbe0  0x21e0  Directory       0xf     RPC Control
4       System  0x840e99c157e0  0x21e4  Event   0x1f0003        -
4       System  0x840e9dae0f00  0x21e8  IoCompletion    0x1f0003        -
4       System  0x840e975cacc0  0x21ec  File    0x120089        \Device\HarddiskVolume3\Windows\CSC\v2.0.6
4       System  0x840e975cae50  0x21f0  File    0x12019f        \Device\HarddiskVolume3\Windows\CSC\v2.0.6\temp
4       System  0x840e97e11080  0x21f4  Process 0x102a  TextInputHost. Pid 1256
4       System  0x840e9819bb90  0x21f8  WaitCompletionPacket    0x1     -
4       System  0x840e92eea590  0x21fc  WaitCompletionPacket    0x1     -
4       System  0x840e98d15200  0x2200  IoCompletion    0x1f0003        -
4       System  0x840e97374e00  0x2204  IoCompletion    0x1f0003        -
4       System  0xb708109e1e50  0x2208  SymbolicLink    0xf0001 SWD#PRINTENUM#{AEBD5C3D-F2AF-4E79-8041-0CF8E0AC0AE3}#{0ecef634-6ef0-472a-8085-5ad023ecbccd}
4       System  0x840e98d33240  0x220c  IoCompletion    0x1f0003        -
4       System  0x840e9903c540  0x2210  IoCompletion    0x1f0003        -
4       System  0xb708109e1b90  0x2214  SymbolicLink    0xf0001 SWD#PRINTENUM#{BC835CDC-D582-4230-83CB-5335521321E0}#{0ecef634-6ef0-472a-8085-5ad023ecbccd}
4       System  0x840e98d40980  0x2218  IoCompletion    0x1f0003        -
4       System  0xb708109e04e0  0x221c  SymbolicLink    0xf0001 SWD#PRINTENUM#{CDBBC391-ED2A-41F9-BC46-93FED66AB8AE}#{0ecef634-6ef0-472a-8085-5ad023ecbccd}
4       System  0x840e979bdd50  0x2228  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\MicrosoftWindows.Client.CBS_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e995cd0c0  0x2230  Process 0x102a  svchost.exe Pid 6356
4       System  0x840e9818a820  0x2234  WaitCompletionPacket    0x1     -
4       System  0x840e9a14be80  0x2238  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\UserClasses.dat
4       System  0x840e98c2c660  0x2240  Event   0x1f0003        -
4       System  0x840e98d14400  0x2244  IoCompletion    0x1f0003        -
4       System  0x840e98b38ea0  0x2248  WaitCompletionPacket    0x1     -
4       System  0x840e98d0d080  0x224c  IoCompletion    0x1f0003        -
4       System  0x840e97146ee0  0x2250  WaitCompletionPacket    0x1     -
4       System  0x840e995680c0  0x2254  Process 0x102a  SgrmBroker.exe Pid 9948
4       System  0x840e99f64100  0x2258  IoCompletion    0x1f0003        -
4       System  0x840e97e11080  0x225c  Process 0x1fffff        TextInputHost. Pid 1256
4       System  0x840e98e8ea60  0x2260  Event   0x1f0003        -
4       System  0x840e98a81670  0x2264  IoCompletionReserve     0xf0003 -
4       System  0x840e98d3adc0  0x2268  IoCompletion    0x1f0003        -
4       System  0x840e9969e080  0x2270  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e99fd8ae0  0x2274  Event   0x1f0003        -
4       System  0x840e9d9b1080  0x2278  Process 0x102a  msedge.exe Pid 6408
4       System  0x840e98b37230  0x227c  WaitCompletionPacket    0x1     -
4       System  0x840e968c00c0  0x2288  Process 0x0     svchost.exe Pid 2496
4       System  0x840e97140880  0x228c  WaitCompletionPacket    0x1     -
4       System  0x840e98db8080  0x2290  Process 0x102a  MsSense.exe Pid 10224
4       System  0x840e9e6f3080  0x2294  Process 0x102a  excel-update.e Pid 4672
4       System  0x840e9b8fdd90  0x2298  Timer   0x1f0003        -
4       System  0x840e98e9a560  0x22a4  Event   0x1f0003        -
4       System  0x840e98e89e60  0x22a8  Event   0x1f0003        -
4       System  0x840e98075620  0x22ac  CoreMessaging   0xf0000 -
4       System  0x840e98b388f0  0x22b0  WaitCompletionPacket    0x1     -
4       System  0x840e9ac0a910  0x22b4  WaitCompletionPacket    0x1     -
4       System  0x840e99651080  0x22b8  Process 0x1fffff        conhost.exe Pid 13176
4       System  0x840e9cf2c940  0x22bc  IoCompletion    0x1f0003        -
4       System  0x840e98a23ac0  0x22c0  IoCompletion    0x1f0003        -
4       System  0x840e98181370  0x22c4  WaitCompletionPacket    0x1     -
4       System  0x840e9b99e660  0x22c8  WaitCompletionPacket    0x1     -
4       System  0x840e9b99b9b0  0x22cc  WaitCompletionPacket    0x1     -
4       System  0x840e99084200  0x22d0  WaitCompletionPacket    0x1     -
4       System  0x840e98b8e340  0x22d4  Process 0x102a  RuntimeBroker. Pid 7612
4       System  0x840e9979ca60  0x22d8  Event   0x1f0003        -
4       System  0x840e90ee9dc0  0x22e4  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.blf
4       System  0x840e9f40c340  0x22e8  Process 0x102a  svchost.exe Pid 2800
4       System  0x840e97f80080  0x22ec  Process 0x0     AggregatorHost Pid 5804
4       System  0x840e98b26950  0x22f0  WaitCompletionPacket    0x1     -
4       System  0x840e9b203300  0x22f8  Process 0x102a  KeePass.exe Pid 11628
4       System  0x840e90bb7260  0x22fc  Event   0x1f0003        -
4       System  0x840e992c0d40  0x2300  IoCompletion    0x1f0003        -
4       System  0x840e981b3700  0x2304  WaitCompletionPacket    0x1     -
4       System  0x840e9969e080  0x2308  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e98fb61c0  0x230c  Process 0x102a  dllhost.exe Pid 8472
4       System  0x840e97e11080  0x2310  Process 0x102a  TextInputHost. Pid 1256
4       System  0x840e90bb8460  0x2314  Event   0x1f0003        -
4       System  0x840e992c0740  0x2318  IoCompletion    0x1f0003        -
4       System  0x840e981b4670  0x231c  WaitCompletionPacket    0x1     -
4       System  0x840e98d0f540  0x2328  IoCompletion    0x1f0003        -
4       System  0x840e98c29060  0x232c  Event   0x1f0003        -
4       System  0x840e9ec9c9a0  0x2334  File    0x3     \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\AppData\Local\Microsoft\Windows\DeliveryOptimization\State\dosvcState.dat.LOG1
4       System  0x840e99ab5780  0x2338  IoCompletion    0x1f0003        -
4       System  0x840e98c450c0  0x2340  Process 0x102a  dllhost.exe Pid 7800
4       System  0x840e9a628a40  0x2344  IoCompletion    0x1f0003        -
4       System  0x840e98c18de0  0x2348  Event   0x1f0003        -
4       System  0x840e92cfa140  0x2350  Process 0x0     dwm.exe Pid 1204
4       System  0x840e97159300  0x2354  Process 0x0     CSFalconServic Pid 3648
4       System  0x840e98c289e0  0x2358  Event   0x1f0003        -
4       System  0x840e98d16440  0x235c  IoCompletion    0x1f0003        -
4       System  0x840e9979b0e0  0x2360  Event   0x1f0003        -
4       System  0x840e98b385b0  0x2364  WaitCompletionPacket    0x1     -
4       System  0x840e9676e080  0x2368  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e99387080  0x2370  Process 0x0     StartMenuExper Pid 7408
4       System  0x840e98b45dd0  0x2374  WaitCompletionPacket    0x1     -
4       System  0x840e98d11bc0  0x2378  IoCompletion    0x1f0003        -
4       System  0x840e994d1080  0x237c  Process 0x102a  LogonUI.exe Pid 10680
4       System  0x840e9b9b2680  0x2380  WaitCompletionPacket    0x1     -
4       System  0x840e98e742c0  0x2384  Process 0x0     svchost.exe Pid 8292
4       System  0x840e92fc6080  0x2388  Process 0x0     svchost.exe Pid 1964
4       System  0xb7081304b060  0x238c  Token   0xf01ff -
4       System  0x840e993439e0  0x2390  Event   0x1f0003        -
4       System  0x840e9951c1c0  0x239c  Process 0x0     msedge.exe Pid 8732
4       System  0x840e98e8f0e0  0x23a0  Event   0x1f0003        -
4       System  0x840e970c9900  0x23a4  IoCompletion    0x1f0003        -
4       System  0x840e9d69a690  0x23a8  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\User.dat.LOG2
4       System  0x840e9daf05c0  0x23ac  IoCompletion    0x1f0003        -
4       System  0x840e995ea080  0x23b0  Process 0x102a  OneDrive.exe Pid 11592
4       System  0x840e90de9080  0x23b8  Process 0x1fffff        FileCoAuth.exe Pid 8592
4       System  0x840e98bba080  0x23c0  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e98e742c0  0x23c4  Process 0x102a  svchost.exe Pid 8292
4       System  0xb7081319be60  0x23c8  Directory       0xf     S-1-15-2-3251537155-1984446955-2931258699-841473695-1938553385-924012159-129201922
4       System  0x840e96907690  0x23cc  IoCompletionReserve     0xf0003 -
4       System  0x840e98e96660  0x23d0  Event   0x1f0003        -
4       System  0x840e98177df0  0x23d4  WaitCompletionPacket    0x1     -
4       System  0x840e981a5930  0x23d8  WaitCompletionPacket    0x1     -
4       System  0x840e92c9a340  0x23dc  Process 0x0     svchost.exe Pid 1076
4       System  0x840e995cd0c0  0x23e0  Process 0x0     svchost.exe Pid 6356
4       System  0x840e99adc680  0x23e4  IoCompletion    0x1f0003        -
4       System  0x840e9769f080  0x23e8  Process 0x102a  winlogon.exe Pid 2804
4       System  0x840e990a0420  0x23ec  WaitCompletionPacket    0x1     -
4       System  0x840e97442080  0x23f0  Process 0x0     CSFalconContai Pid 6624
4       System  0x840e992b4b00  0x23f4  IoCompletion    0x1f0003        -
4       System  0x840e9ac3f880  0x23f8  WaitCompletionPacket    0x1     -
4       System  0x840e978c0300  0x23fc  Process 0x40    explorer.exe Pid 5328
4       System  0x840e90c19340  0x2404  Process 0x1fffff        svchost.exe Pid 3604
4       System  0x840e98b15080  0x2408  Process 0x1fffff        fontdrvhost.ex Pid 10704
4       System  0x840e98bba080  0x240c  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e9ac13740  0x2410  WaitCompletionPacket    0x1     -
4       System  0x840e96e552c0  0x2414  Process 0x102a  svchost.exe Pid 3560
4       System  0x840e9ac2a4e0  0x2418  WaitCompletionPacket    0x1     -
4       System  0x840e9a94f4e0  0x241c  Event   0x1f0003        -
4       System  0x840ea0843780  0x2420  IoCompletion    0x1f0003        -
4       System  0x840e9933be60  0x2424  Event   0x1f0003        -
4       System  0x840e98b57210  0x2428  WaitCompletionPacket    0x1     -
4       System  0x840e9968d080  0x242c  Process 0x1fffff        splunkd.exe Pid 13620
4       System  0x840e9832a1c0  0x2430  Process 0x0     msedge.exe Pid 3832
4       System  0x840e996e91c0  0x2434  Process 0x0     msedge.exe Pid 9016
4       System  0x840e98d3a240  0x2438  IoCompletion    0x1f0003        -
4       System  0x840e98d37b00  0x243c  IoCompletion    0x1f0003        -
4       System  0x840e9901d440  0x2440  IoCompletion    0x1f0003        -
4       System  0x840e99338560  0x2444  Event   0x1f0003        -
4       System  0x840e9958a0c0  0x2448  Process 0x0     msedge.exe Pid 9128
4       System  0x840e98b50a10  0x2450  WaitCompletionPacket    0x1     -
4       System  0x840e99f76100  0x2454  IoCompletion    0x1f0003        -
4       System  0x840e9dad8400  0x245c  IoCompletion    0x1f0003        -
4       System  0x840e9969f080  0x2460  Process 0x102a  Microsoft.Shar Pid 5176
4       System  0x840e99f53480  0x2464  IoCompletion    0x1f0003        -
4       System  0x840e92cfa140  0x2468  Process 0x40    dwm.exe Pid 1204
4       System  0x840e9908dc60  0x246c  WaitCompletionPacket    0x1     -
4       System  0x840e9daba640  0x2470  IoCompletion    0x1f0003        -
4       System  0x840e9cf39a80  0x2474  IoCompletion    0x1f0003        -
4       System  0x840e97d58500  0x2478  IoCompletion    0x1f0003        -
4       System  0x840e9ba13c40  0x247c  WaitCompletionPacket    0x1     -
4       System  0xb708156d7990  0x2480  Token   0xf01ff -
4       System  0x840e98d19d40  0x2484  IoCompletion    0x1f0003        -
4       System  0x840e98d18f40  0x2488  IoCompletion    0x1f0003        -
4       System  0x840e98c13a60  0x248c  Event   0x1f0003        -
4       System  0x840e9a945de0  0x2490  Event   0x1f0003        -
4       System  0x840e98a27e00  0x2494  IoCompletion    0x1f0003        -
4       System  0x840e90b0d080  0x2498  Process 0x102a  svchost.exe Pid 9452
4       System  0x840e9ba11610  0x249c  WaitCompletionPacket    0x1     -
4       System  0x840e98c290e0  0x24a0  Event   0x1f0003        -
4       System  0x840e9909b7c0  0x24a4  WaitCompletionPacket    0x1     -
4       System  0x840e98b50390  0x24a8  WaitCompletionPacket    0x1     -
4       System  0x840e990ad0e0  0x24ac  WaitCompletionPacket    0x1     -
4       System  0x840e91ee7080  0x24b0  Process 0x102a  Sysmon64.exe Pid 1540
4       System  0x840e966735a0  0x24b4  WaitCompletionPacket    0x1     -
4       System  0x840e9e3d0760  0x24b8  Event   0x1f0003        -
4       System  0x840e9b9b9910  0x24bc  WaitCompletionPacket    0x1     -
4       System  0x840e978cd940  0x24c0  WaitCompletionPacket    0x1     -
4       System  0x840e9c47d3c0  0x24c4  IoCompletion    0x1f0003        -
4       System  0x840e998251c0  0x24cc  Process 0x0     msedge.exe Pid 9728
4       System  0xb70820e89060  0x24d0  Token   0xf01ff -
4       System  0x840e978d8650  0x24d8  WaitCompletionPacket    0x1     -
4       System  0x840e967e4460  0x24dc  Event   0x1f0003        -
4       System  0x840e9c252080  0x24e0  Process 0x102a  RuntimeBroker. Pid 10648
4       System  0x840e92cfa140  0x24e4  Process 0x102a  dwm.exe Pid 1204
4       System  0x840e9b99b810  0x24e8  WaitCompletionPacket    0x1     -
4       System  0x840e990aac50  0x24ec  WaitCompletionPacket    0x1     -
4       System  0x840e91bc7b00  0x24f0  FilterConnectionPort    0x1f0001        MicrosoftSecFilterControlPort
4       System  0x840e98fe5160  0x24f4  Event   0x1f0003        -
4       System  0x840e970ca9c0  0x24f8  IoCompletion    0x1f0003        -
4       System  0x840e9c89f460  0x24fc  Event   0x1f0003        -
4       System  0x840e923f41c0  0x2500  Process 0x0     fontdrvhost.ex Pid 800
4       System  0x840e90e4d080  0x2504  Thread  0x1fffff        Tid 6536 Pid 4
4       System  0x840e978e50a0  0x2508  WaitCompletionPacket    0x1     -
4       System  0x840e98e86960  0x250c  Event   0x1f0003        -
4       System  0x840e995680c0  0x2510  Process 0x0     SgrmBroker.exe Pid 9948
4       System  0x840e990374c0  0x2514  IoCompletion    0x1f0003        -
4       System  0x840e9e5a11c0  0x2518  Process 0x0     uhssvc.exe Pid 11740
4       System  0x840e91ba00c0  0x251c  Process 0x0     svchost.exe Pid 7220
4       System  0x840ea534e7e0  0x2520  Event   0x1f0003        -
4       System  0x840e96d4d860  0x2524  Timer   0x1f0003        -
4       System  0x840e9dabd900  0x2528  IoCompletion    0x1f0003        -
4       System  0x840e98077e10  0x252c  CoreMessaging   0xf0000 -
4       System  0x840e98c450c0  0x2530  Process 0x102a  dllhost.exe Pid 7800
4       System  0x840e90b0d080  0x2534  Process 0x102a  svchost.exe Pid 9452
4       System  0x840e92291140  0x2538  Process 0x0     csrss.exe Pid 888
4       System  0x840e9c846060  0x253c  Event   0x1f0003        -
4       System  0x840e970c78c0  0x2540  IoCompletion    0x1f0003        -
4       System  0x840e9b99fd20  0x2544  WaitCompletionPacket    0x1     -
4       System  0x840ea9ed2560  0x2548  Event   0x1f0003        -
4       System  0x840e9969e080  0x254c  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e9969e080  0x2550  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e9909b070  0x2554  WaitCompletionPacket    0x1     -
4       System  0x840e9ac234c0  0x2558  WaitCompletionPacket    0x1     -
4       System  0x840e9b7bb080  0x255c  Process 0x102a  StartMenuExper Pid 8
4       System  0x840e9b9b3110  0x2564  WaitCompletionPacket    0x1     -
4       System  0x840e90e10410  0x2568  Session 0x0     Session2
4       System  0x840e96672be0  0x256c  WaitCompletionPacket    0x1     -
4       System  0x840e9c8babf0  0x2570  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.1.regtrans-ms
4       System  0x840e997052e0  0x2574  Event   0x1f0003        -
4       System  0x840e9c9053e0  0x2578  Event   0x1f0003        -
4       System  0x840e9ac34de0  0x257c  WaitCompletionPacket    0x1     -
4       System  0x840e9952c080  0x2580  Process 0x40    csrss.exe Pid 10732
4       System  0x840e98bdf080  0x2584  Process 0x1fffff        SystemSettings Pid 10020
4       System  0x840e98e84860  0x2588  Event   0x1f0003        -
4       System  0x840ea0882780  0x258c  IoCompletion    0x1f0003        -
4       System  0x840e9c252080  0x2590  Process 0x102a  RuntimeBroker. Pid 10648
4       System  0x840e98bdf080  0x2594  Process 0x40    SystemSettings Pid 10020
4       System  0x840e98bba080  0x2598  Process 0x40    SearchApp.exe Pid 7932
4       System  0x840e9c895de0  0x259c  Event   0x1f0003        -
4       System  0x840e98c29b60  0x25a0  Event   0x1f0003        -
4       System  0x840e9909fe70  0x25a4  WaitCompletionPacket    0x1     -
4       System  0x840e92f27080  0x25a8  Process 0x0     svchost.exe Pid 1808
4       System  0x840e993384e0  0x25ac  Event   0x1f0003        -
4       System  0x840e98d113c0  0x25b0  IoCompletion    0x1f0003        -
4       System  0x840e98b44a50  0x25b4  WaitCompletionPacket    0x1     -
4       System  0x840e993fb680  0x25b8  TmRm    0x1f007f        -
4       System  0x840e98185130  0x25bc  WaitCompletionPacket    0x1     -
4       System  0x840e9952c080  0x25c4  Process 0x0     csrss.exe Pid 10732
4       System  0x840e98ec91c0  0x25c8  Process 0x0     RuntimeBroker. Pid 10968
4       System  0x840e9dacb2c0  0x25cc  IoCompletion    0x1f0003        -
4       System  0x840e92ee0990  0x25d4  WaitCompletionPacket    0x1     -
4       System  0x840e92297960  0x25d8  Event   0x1f0003        -
4       System  0x840e9832a1c0  0x25dc  Process 0x102a  msedge.exe Pid 3832
4       System  0x840e90bcff60  0x25e0  Event   0x1f0003        -
4       System  0x840e97145a90  0x25e4  WaitCompletionPacket    0x1     -
4       System  0x840e98d21180  0x25e8  IoCompletion    0x1f0003        -
4       System  0x840e9f3b7d00  0x25ec  IoCompletion    0x1f0003        -
4       System  0x840e9c9adc40  0x25f0  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Photos_2019.19071.12548.0_x64__8wekyb3d8bbwe\ActivationStore.dat.LOG1
4       System  0x840e99ab9140  0x25f8  IoCompletion    0x1f0003        -
4       System  0x840e98a82270  0x25fc  IoCompletionReserve     0xf0003 -
4       System  0x840e98a7ec70  0x2600  IoCompletionReserve     0xf0003 -
4       System  0x840e97e8c080  0x2604  Process 0x102a  svchost.exe Pid 3864
4       System  0xb7081a5c6180  0x2608  SymbolicLink    0xf0001 Session
4       System  0x840e990adaa0  0x260c  WaitCompletionPacket    0x1     -
4       System  0x840e9818d190  0x2610  WaitCompletionPacket    0x1     -
4       System  0x840e9904cc80  0x2614  IoCompletion    0x1f0003        -
4       System  0x840e96f5a080  0x2618  Process 0x1fffff        SenseNdr.exe Pid 15568
4       System  0x840e9907c680  0x261c  WaitCompletionPacket    0x1     -
4       System  0x840e98db8080  0x2620  Process 0x102a  MsSense.exe Pid 10224
4       System  0x840e99ae2b00  0x2624  IoCompletion    0x1f0003        -
4       System  0x840e9b807a30  0x2628  IoCompletionReserve     0xf0003 -
4       System  0x840e9e0b3040  0x262c  Thread  0x1fffff        Tid 6676 Pid 4
4       System  0x840e98bcace0  0x2630  ALPC Port       0x1f0001        -
4       System  0x840e978c0300  0x2634  Process 0x40    explorer.exe Pid 5328
4       System  0x840e98db4080  0x2638  Process 0x0     msedge.exe Pid 9888
4       System  0x840e98f07080  0x263c  Process 0x102a  msedge.exe Pid 9172
4       System  0x840e98b3b5a0  0x2640  WaitCompletionPacket    0x1     -
4       System  0x840e98fe2c60  0x2644  Event   0x1f0003        -
4       System  0x840e90c63df0  0x2648  WaitCompletionPacket    0x1     -
4       System  0x840e98d201c0  0x264c  IoCompletion    0x1f0003        -
4       System  0x840e98c2fc60  0x2650  Event   0x1f0003        -
4       System  0x840e9230b100  0x265c  Process 0x0     services.exe Pid 1000
4       System  0x840e990246c0  0x2660  IoCompletion    0x1f0003        -
4       System  0x840e98e861e0  0x2664  Event   0x1f0003        -
4       System  0x840e997450c0  0x2668  Process 0x102a  svchost.exe Pid 3052
4       System  0x840e92c4fde0  0x2670  WaitCompletionPacket    0x1     -
4       System  0x840e9ea14080  0x2674  Process 0x102a  ScreenConnect. Pid 1772
4       System  0xb70805fad060  0x2678  Token   0xf01ff -
4       System  0x840e91ba00c0  0x267c  Process 0x102a  svchost.exe Pid 7220
4       System  0x840e9c8968e0  0x2680  Event   0x1f0003        -
4       System  0x840e9b160b10  0x2684  File    0x100001        \Device\0000002e
4       System  0x840e98b56e00  0x2688  WaitCompletionPacket    0x1     -
4       System  0x840e90bce7e0  0x268c  Event   0x1f0003        -
4       System  0x840e9ac0e2c0  0x2690  WaitCompletionPacket    0x1     -
4       System  0x840e98c322e0  0x2694  Event   0x1f0003        -
4       System  0x840e9715d080  0x2698  Process 0x0     Sysmon64.exe Pid 3696
4       System  0x840e9f7a8140  0x269c  IoCompletion    0x1f0003        -
4       System  0x840e9229cee0  0x26a0  Event   0x1f0003        -
4       System  0x840e96dd3f60  0x26a4  Event   0x1f0003        -
4       System  0x840e9c8962e0  0x26a8  Event   0x1f0003        -
4       System  0x840e9769f080  0x26ac  Process 0x1fffff        winlogon.exe Pid 2804
4       System  0x840e92cab2c0  0x26b0  Process 0x0     svchost.exe Pid 1124
4       System  0x840e981ae760  0x26b4  WaitCompletionPacket    0x1     -
4       System  0x840e98fd0160  0x26b8  Event   0x1f0003        -
4       System  0x840e98d3d440  0x26bc  IoCompletion    0x1f0003        -
4       System  0x840e92eaa2c0  0x26c4  Process 0x0     svchost.exe Pid 1604
4       System  0x840e9856b2c0  0x26d0  Process 0x0     svchost.exe Pid 7780
4       System  0x840e9682f080  0x26d4  Process 0x40    OneDrive.exe Pid 3320
4       System  0x840e981a5380  0x26d8  WaitCompletionPacket    0x1     -
4       System  0x840e98f07080  0x26dc  Process 0x1fffff        msedge.exe Pid 9172
4       System  0x840e976752c0  0x26e4  IoCompletion    0x1f0003        -
4       System  0x840e971396c0  0x26f0  WaitCompletionPacket    0x1     -
4       System  0x840e9a606d80  0x26f4  IoCompletion    0x1f0003        -
4       System  0x840e90e10410  0x26f8  Session 0x0     Session2
4       System  0x840e98bba080  0x26fc  Process 0x40    SearchApp.exe Pid 7932
4       System  0x840e98e832e0  0x2700  Event   0x1f0003        -
4       System  0xb70834703950  0x2704  Token   0xf01ff -
4       System  0x840e98d3c480  0x2708  IoCompletion    0x1f0003        -
4       System  0x840e98bba080  0x270c  Process 0x0     SearchApp.exe Pid 7932
4       System  0x840e9979eae0  0x2710  Event   0x1f0003        -
4       System  0x840e98b2c6c0  0x2714  WaitCompletionPacket    0x1     -
4       System  0x840e9c8977e0  0x2718  Event   0x1f0003        -
4       System  0x840e99092da0  0x271c  WaitCompletionPacket    0x1     -
4       System  0x840e98e8aae0  0x2720  Event   0x1f0003        -
4       System  0x840e993240c0  0x2724  Process 0x102a  sihost.exe Pid 10384
4       System  0x840e9979cd60  0x2728  Event   0x1f0003        -
4       System  0x840e98d391c0  0x272c  IoCompletion    0x1f0003        -
4       System  0x840e9c52f4e0  0x2730  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0x840ea47d2d40  0x2734  WaitCompletionPacket    0x1     -
4       System  0x840e92f04340  0x2738  Process 0x0     svchost.exe Pid 1652
4       System  0x840e9c47dd40  0x273c  IoCompletion    0x1f0003        -
4       System  0x840e9c84dbe0  0x2740  Event   0x1f0003        -
4       System  0x840e981aac10  0x274c  WaitCompletionPacket    0x1     -
4       System  0x840e9c527ca0  0x2750  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000002.regtrans-ms
4       System  0x840e92e539e0  0x2754  Event   0x100000        -
4       System  0x840e99abf880  0x2758  IoCompletion    0x1f0003        -
4       System  0x840e92eac340  0x275c  Process 0x0     svchost.exe Pid 1612
4       System  0x840e9b985500  0x2760  WaitCompletionPacket    0x1     -
4       System  0x840e978c0300  0x2768  Process 0x102a  explorer.exe Pid 5328
4       System  0x840e96e552c0  0x2770  Process 0x102a  svchost.exe Pid 3560
4       System  0x840e978c0300  0x2774  Process 0x102a  explorer.exe Pid 5328
4       System  0x840e9c92b5e0  0x2778  Event   0x1f0003        -
4       System  0x840e98d19e00  0x277c  IoCompletion    0x1f0003        -
4       System  0x840e90bb8c60  0x2780  Event   0x1f0003        -
4       System  0x840e98c2eb60  0x2784  Event   0x1f0003        -
4       System  0x840e984072b0  0x2788  IoCompletionReserve     0xf0003 -
4       System  0x840e98b601e0  0x2790  WaitCompletionPacket    0x1     -
4       System  0x840e9b7bb080  0x2794  Process 0x1fffff        StartMenuExper Pid 8
4       System  0x840e98b447e0  0x2798  WaitCompletionPacket    0x1     -
4       System  0x840e9956e7d0  0x279c  TmRm    0x1f007f        -
4       System  0x840e978dfc20  0x27a0  WaitCompletionPacket    0x1     -
4       System  0x840e982f2ce0  0x27a4  ALPC Port       0x1f0001        -
4       System  0x840e90c19340  0x27a8  Process 0x102a  svchost.exe Pid 3604
4       System  0x840e90c19340  0x27ac  Process 0x102a  svchost.exe Pid 3604
4       System  0x840e96c61080  0x27b0  Process 0x0     svchost.exe Pid 2216
4       System  0xb70812ba6430  0x27b4  Section 0x17    -
4       System  0x840e96c582c0  0x27b8  Process 0x0     svchost.exe Pid 3028
4       System  0x840e96e552c0  0x27bc  Process 0x102a  svchost.exe Pid 3560
4       System  0x840e998251c0  0x27c0  Process 0x102a  msedge.exe Pid 9728
4       System  0x840e992ccec0  0x27c4  IoCompletion    0x1f0003        -
4       System  0x840e90bc18e0  0x27c8  Event   0x1f0003        -
4       System  0x840e90de2080  0x27cc  Process 0x1fffff        svchost.exe Pid 1480
4       System  0x840e92f23080  0x27d4  Process 0x0     svchost.exe Pid 1776
4       System  0x840e98d0dbc0  0x27d8  IoCompletion    0x1f0003        -
4       System  0x840e9a325a30  0x27dc  IoCompletionReserve     0xf0003 -
4       System  0x840e90bc3560  0x27e0  Event   0x1f0003        -
4       System  0x840e992d0cc0  0x27e4  IoCompletion    0x1f0003        -
4       System  0x840e978d2190  0x27e8  WaitCompletionPacket    0x1     -
4       System  0x840e978ce230  0x27ec  WaitCompletionPacket    0x1     -
4       System  0x840e9933bd60  0x27f0  Event   0x1f0003        -
4       System  0x840e90b0d080  0x27f4  Process 0x102a  svchost.exe Pid 9452
4       System  0x840e9ea14080  0x27f8  Process 0x102a  ScreenConnect. Pid 1772
4       System  0xb7081a5c91a0  0x2804  SymbolicLink    0xf0001 Global
4       System  0x840e978e9270  0x2808  WaitCompletionPacket    0x1     -
4       System  0x840e99f4f740  0x280c  IoCompletion    0x1f0003        -
4       System  0x840e9798adb0  0x2810  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e98bba080  0x2814  Process 0x40    SearchApp.exe Pid 7932
4       System  0x840e98bba080  0x281c  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e99083360  0x2820  WaitCompletionPacket    0x1     -
4       System  0xb7081792d450  0x2824  Section 0x17    -
4       System  0x840e96f6d2c0  0x2828  Process 0x0     svchost.exe Pid 3880
4       System  0x840e992e47c0  0x282c  IoCompletion    0x1f0003        -
4       System  0x840e981b5370  0x2830  WaitCompletionPacket    0x1     -
4       System  0xb7081120ac90  0x2834  SymbolicLink    0xf0001 Local
4       System  0x840ea3ce2f60  0x2838  Event   0x1f0003        -
4       System  0x840e981800c0  0x283c  WaitCompletionPacket    0x1     -
4       System  0x840e90bcb960  0x2840  Event   0x1f0003        -
4       System  0x840e98bdf080  0x2844  Process 0x102a  SystemSettings Pid 10020
4       System  0x840e9681d9d0  0x2848  WaitCompletionPacket    0x1     -
4       System  0xb70821b0e790  0x284c  Section 0x17    -
4       System  0x840e9b9baa20  0x2850  WaitCompletionPacket    0x1     -
4       System  0x840e9c896f60  0x2854  Event   0x1f0003        -
4       System  0x840e990817c0  0x2858  WaitCompletionPacket    0x1     -
4       System  0x840e98182f10  0x285c  WaitCompletionPacket    0x1     -
4       System  0x840e9b9b18b0  0x2860  WaitCompletionPacket    0x1     -
4       System  0x840e978de150  0x2864  WaitCompletionPacket    0x1     -
4       System  0x840e99f66dc0  0x2868  IoCompletion    0x1f0003        -
4       System  0x840e978e43a0  0x286c  WaitCompletionPacket    0x1     -
4       System  0x840e98db4080  0x2870  Process 0x102a  msedge.exe Pid 9888
4       System  0x840e98e93fe0  0x2874  Event   0x1f0003        -
4       System  0x840e992ddc40  0x2878  IoCompletion    0x1f0003        -
4       System  0x840e97ad9cb0  0x287c  ALPC Port       0x1f0001        -
4       System  0x840e90de2080  0x2880  Process 0x102a  svchost.exe Pid 1480
4       System  0x840e99ab85c0  0x2884  IoCompletion    0x1f0003        -
4       System  0x840e996e91c0  0x2888  Process 0x102a  msedge.exe Pid 9016
4       System  0x840e98d0c500  0x288c  IoCompletion    0x1f0003        -
4       System  0x840e98df3080  0x2890  Process 0x0     ShellExperienc Pid 704
4       System  0x840e9b9b7140  0x2894  WaitCompletionPacket    0x1     -
4       System  0xb7081129cb80  0x2898  Directory       0xf     S-1-15-2-515815643-2845804217-1874292103-218650560-777617685-4287762684-137415000
4       System  0x840e9ba0c8e0  0x289c  WaitCompletionPacket    0x1     -
4       System  0x840ea17e9080  0x28a0  Process 0x1fffff        svchost.exe Pid 10392
4       System  0x840e90bc6560  0x28a4  Event   0x1f0003        -
4       System  0x840e992d3c00  0x28a8  IoCompletion    0x1f0003        -
4       System  0x840e97146a00  0x28ac  WaitCompletionPacket    0x1     -
4       System  0x840e9a14f210  0x28b0  File    0x3     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-5-21-3294271487-4292050549-1661813568-1001\SystemAppData\Helium\Cache\b4d622c333fcfdd7_COM15.dat.LOG1
4       System  0x840e98a2ff80  0x28b4  IoCompletion    0x1f0003        -
4       System  0x840e997abee0  0x28b8  Event   0x1f0003        -
4       System  0xb7081129c5e0  0x28bc  Directory       0xf     RPC Control
4       System  0x840e978e3430  0x28c0  WaitCompletionPacket    0x1     -
4       System  0x840e9666fa50  0x28c4  WaitCompletionPacket    0x1     -
4       System  0x840e97654500  0x28c8  IoCompletion    0x1f0003        -
4       System  0x840e90bc9860  0x28cc  Event   0x1f0003        -
4       System  0x840e9c554940  0x28d0  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\UserClasses.dat.LOG2
4       System  0x840e978c0300  0x28d4  Process 0x40    explorer.exe Pid 5328
4       System  0x840e996cf080  0x28d8  Process 0x1fffff        svchost.exe Pid 9320
4       System  0x840e9676e080  0x28dc  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e98582040  0x28e0  Thread  0x1fffff        Tid 15852 Pid 4
4       System  0x840e998251c0  0x28e4  Process 0x102a  msedge.exe Pid 9728
4       System  0x840e98a02b00  0x28e8  IoCompletion    0x1f0003        -
4       System  0x840e992dc580  0x28ec  IoCompletion    0x1f0003        -
4       System  0x840e9933bf60  0x28f0  Event   0x1f0003        -
4       System  0x840e9ba24380  0x28f4  WaitCompletionPacket    0x1     -
4       System  0x840ea70570c0  0x28f8  Process 0x102a  msedge.exe Pid 1336
4       System  0x840e99f67f40  0x28fc  IoCompletion    0x1f0003        -
4       System  0x840e98e92160  0x2900  Event   0x1f0003        -
4       System  0x840e97989e10  0x2904  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\MicrosoftWindows.Client.CBS_1000.19058.1000.0_x64__cw5n1h2txyewy\ActivationStore.dat.LOG2
4       System  0x840e90de2080  0x2908  Process 0x102a  svchost.exe Pid 1480
4       System  0x840e978c0300  0x290c  Process 0x102a  explorer.exe Pid 5328
4       System  0x840e976577c0  0x2910  IoCompletion    0x1f0003        -
4       System  0x840e98b332d0  0x2918  WaitCompletionPacket    0x1     -
4       System  0x840e98a02a40  0x291c  IoCompletion    0x1f0003        -
4       System  0x840e9c862260  0x2920  Event   0x1f0003        -
4       System  0x840e92f202c0  0x2924  Process 0x0     svchost.exe Pid 1764
4       System  0x840e9a14b520  0x2928  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.ShellExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat
4       System  0x840e992cd400  0x292c  IoCompletion    0x1f0003        -
4       System  0x840e9ac0eae0  0x2930  WaitCompletionPacket    0x1     -
4       System  0x840e98df3080  0x2934  Process 0x102a  ShellExperienc Pid 704
4       System  0x840e90bc2b60  0x2938  Event   0x1f0003        -
4       System  0x840e9c9ac3e0  0x293c  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Photos_2019.19071.12548.0_x64__8wekyb3d8bbwe\ActivationStore.dat
4       System  0xb707fe074060  0x2940  Token   0xf01ff -
4       System  0x840e90bc6260  0x2948  Event   0x1f0003        -
4       System  0x840e992d6440  0x294c  IoCompletion    0x1f0003        -
4       System  0x840e978d05f0  0x2950  WaitCompletionPacket    0x1     -
4       System  0x840e98d3f200  0x2954  IoCompletion    0x1f0003        -
4       System  0x840e90de2080  0x295c  Process 0x102a  svchost.exe Pid 1480
4       System  0x840e97c49ae0  0x2960  Event   0x1f0003        -
4       System  0x840e97fc1560  0x2964  Event   0x1f0003        -
4       System  0x840e99038d40  0x2968  IoCompletion    0x1f0003        -
4       System  0xb7081a5a9240  0x296c  Directory       0xf     S-1-15-2-155514346-2573954481-755741238-1654018636-1233331829-3075935687-2861478708
4       System  0x840e99336c60  0x2970  Event   0x1f0003        -
4       System  0x840e90de2080  0x2974  Process 0x102a  svchost.exe Pid 1480
4       System  0x840e98d0cf80  0x2978  IoCompletion    0x1f0003        -
4       System  0x840e9a147830  0x297c  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\User.dat
4       System  0x840e9979af60  0x2980  Event   0x1f0003        -
4       System  0x840e9bcd1bb0  0x2984  IoCompletionReserve     0xf0003 -
4       System  0x840e9cf2c6c0  0x2988  IoCompletion    0x1f0003        -
4       System  0x840e99fd7760  0x298c  Event   0x1f0003        -
4       System  0x840e9ac3d800  0x2994  WaitCompletionPacket    0x1     -
4       System  0x840e98a1dd00  0x2998  IoCompletion    0x1f0003        -
4       System  0x840e9969e080  0x299c  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e97bab850  0x29a0  File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-515815643-2845804217-1874292103-218650560-777617685-4287762684-137415000
4       System  0x840e98fb61c0  0x29a4  Process 0x1fffff        dllhost.exe Pid 8472
4       System  0x840e9979a1e0  0x29a8  Event   0x1f0003        -
4       System  0x840e9ac17c50  0x29ac  WaitCompletionPacket    0x1     -
4       System  0x840e99387080  0x29b0  Process 0x1fffff        StartMenuExper Pid 7408
4       System  0x840e9a625240  0x29b4  IoCompletion    0x1f0003        -
4       System  0x840e98a10780  0x29bc  IoCompletion    0x1f0003        -
4       System  0x840e98a0f980  0x29c0  IoCompletion    0x1f0003        -
4       System  0x840e981a7b50  0x29c4  WaitCompletionPacket    0x1     -
4       System  0x840e98f07080  0x29cc  Process 0x102a  msedge.exe Pid 9172
4       System  0xb7081a5c51b0  0x29d0  SymbolicLink    0xf0001 Global
4       System  0x840e97465360  0x29d4  Event   0x1f0003        -
4       System  0x840e999e81e0  0x29d8  Event   0x1f0003        -
4       System  0x840e9781b160  0x29e0  Event   0x1f0003        -
4       System  0x840e9cb56a00  0x29e4  IoCompletion    0x1f0003        -
4       System  0x840e9908a110  0x29e8  WaitCompletionPacket    0x1     -
4       System  0x840e98c18460  0x29ec  Event   0x1f0003        -
4       System  0x840e9927e8f0  0x29f0  IoCompletionReserve     0xf0003 -
4       System  0x840e99387080  0x29f4  Process 0x102a  StartMenuExper Pid 7408
4       System  0x840e99ab8780  0x29f8  IoCompletion    0x1f0003        -
4       System  0x840e9e902340  0x29fc  Process 0x102a  msedge.exe Pid 5808
4       System  0x840e9c937260  0x2a00  Event   0x1f0003        -
4       System  0x840e9798a450  0x2a04  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Search_1.14.15.19041_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat
4       System  0x840e91bcf3f0  0x2a08  WaitCompletionPacket    0x1     -
4       System  0x840e97381300  0x2a0c  IoCompletion    0x1f0003        -
4       System  0x840e99f51b00  0x2a10  IoCompletion    0x1f0003        -
4       System  0x840e91dde840  0x2a14  Timer   0x2     -
4       System  0x840e96d4b340  0x2a18  Timer   0x1f0003        -
4       System  0x840e97d73980  0x2a1c  IoCompletion    0x1f0003        -
4       System  0x840e99aae780  0x2a20  IoCompletion    0x1f0003        -
4       System  0x840e92c52dd0  0x2a24  WaitCompletionPacket    0x1     -
4       System  0x840e99036940  0x2a28  IoCompletion    0x1f0003        -
4       System  0x840e985d8080  0x2a2c  Process 0x40    ShellExperienc Pid 7588
4       System  0x840e976a2300  0x2a30  Process 0x0     unsecapp.exe Pid 5220
4       System  0x840e91ee7080  0x2a34  Process 0x0     Sysmon64.exe Pid 1540
4       System  0xb708106add40  0x2a38  Directory       0xf000f 00000000-0258f3ec
4       System  0x840e99fd6a60  0x2a3c  Event   0x1f0003        -
4       System  0x840e981af2c0  0x2a40  WaitCompletionPacket    0x1     -
4       System  0x840e99a89080  0x2a48  Process 0x102a  svchost.exe Pid 11756
4       System  0x840e98f07080  0x2a4c  Process 0x102a  msedge.exe Pid 9172
4       System  0x840e91d8a360  0x2a50  Event   0x1f0003        -
4       System  0x840e978c0300  0x2a54  Process 0x40    explorer.exe Pid 5328
4       System  0x840e98bd6240  0x2a58  Process 0x1fffff        RuntimeBroker. Pid 8120
4       System  0x840e97a764a0  0x2a5c  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.StartMenuExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat
4       System  0x840e90de9080  0x2a60  Process 0x0     FileCoAuth.exe Pid 8592
4       System  0x840e9903f1c0  0x2a6c  IoCompletion    0x1f0003        -
4       System  0x840e9c716260  0x2a74  Event   0x1f0003        -
4       System  0x840e99fcd9e0  0x2a78  Event   0x1f0003        -
4       System  0xb70811698bc0  0x2a7c  Directory       0xf     RPC Control
4       System  0x840e98b26a20  0x2a80  WaitCompletionPacket    0x1     -
4       System  0x840e9681d4f0  0x2a84  WaitCompletionPacket    0x1     -
4       System  0x840e9c71c3e0  0x2a88  Event   0x1f0003        -
4       System  0x840e923de2c0  0x2a8c  Process 0x0     svchost.exe Pid 756
4       System  0x840e97a77da0  0x2a90  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.StartMenuExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG1
4       System  0x840e98191360  0x2a94  WaitCompletionPacket    0x1     -
4       System  0x840e98c1ade0  0x2a98  Event   0x1f0003        -
4       System  0x840e9c545a60  0x2a9c  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-95739096-486727260-2033287795-3853587803-1685597119-444378811-2746676523
4       System  0x840e9798d970  0x2aa0  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e96f7e920  0x2aa4  File    0x12008b        \Device\HarddiskVolume3\ProgramData\Microsoft\Windows Security Health\Logs\SHS-03122025-104154-7-7f-19041.1.amd64fre.vb_release.191206-1406.etl
4       System  0x840e98b3f500  0x2aac  WaitCompletionPacket    0x1     -
4       System  0x840e97e9daa0  0x2ab0  CoreMessaging   0xf0000 -
4       System  0x840e9934bce0  0x2ab4  Event   0x1f0003        -
4       System  0x840e9a60e200  0x2ab8  IoCompletion    0x1f0003        -
4       System  0x840e9783b080  0x2ac0  Process 0x0     CSFalconContai Pid 6604
4       System  0x840e980783b0  0x2ac4  CoreMessaging   0xf0000 -
4       System  0x840e967def60  0x2ac8  Event   0x1f0003        -
4       System  0x840e96904810  0x2acc  IoCompletionReserve     0xf0003 -
4       System  0x840e9676b300  0x2ad0  Process 0x0     svchost.exe Pid 2180
4       System  0x840e9904eb40  0x2ad4  IoCompletion    0x1f0003        -
4       System  0x840e9904cbc0  0x2ad8  IoCompletion    0x1f0003        -
4       System  0x840e99da3080  0x2adc  Process 0x102a  RuntimeBroker. Pid 4464
4       System  0x840e9037c040  0x2ae0  Process 0x0     Registry Pid 108
4       System  0x840e97146380  0x2ae4  WaitCompletionPacket    0x1     -
4       System  0x840e9969e080  0x2ae8  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e98a16a80  0x2aec  IoCompletion    0x1f0003        -
4       System  0x840e98b28f80  0x2af4  WaitCompletionPacket    0x1     -
4       System  0x840e9ec9fa60  0x2af8  File    0x3     \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\AppData\Local\Microsoft\Windows\DeliveryOptimization\State\dosvcState.dat.LOG2
4       System  0x840e98bba080  0x2afc  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e9a62b600  0x2b00  IoCompletion    0x1f0003        -
4       System  0x840e99fdb0e0  0x2b04  Event   0x1f0003        -
4       System  0x840e9c36f8e0  0x2b08  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e99083840  0x2b0c  WaitCompletionPacket    0x1     -
4       System  0x840e96c5e1c0  0x2b10  Process 0x0     svchost.exe Pid 2220
4       System  0x840e98b8e340  0x2b14  Process 0x0     RuntimeBroker. Pid 7612
4       System  0x840e9934d4e0  0x2b18  Event   0x1f0003        -
4       System  0x840e9b6cc4b0  0x2b28  IoCompletionReserve     0xf0003 -
4       System  0x840e9c5a7ee0  0x2b2c  Event   0x1f0003        -
4       System  0x840e9dabaf00  0x2b34  IoCompletion    0x1f0003        -
4       System  0x840e97a75820  0x2b38  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.StartMenuExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG2
4       System  0x840e98ff32c0  0x2b44  Process 0x102a  SecurityHealth Pid 8808
4       System  0x840e99b263e0  0x2b48  File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-155514346-2573954481-755741238-1654018636-1233331829-3075935687-2861478708
4       System  0x840e9c8b69f0  0x2b4c  File    0x12008b        \Device\HarddiskVolume3\Windows\Logs\WindowsUpdate\WindowsUpdate.20250313.155518.080.4.etl
4       System  0x840e9b739080  0x2b50  Process 0x0     Microsoft.Phot Pid 11148
4       System  0x840e999ef260  0x2b58  Event   0x1f0003        -
4       System  0x840e99043540  0x2b5c  IoCompletion    0x1f0003        -
4       System  0x840e98e742c0  0x2b64  Process 0x1fffff        svchost.exe Pid 8292
4       System  0x840e98c13de0  0x2b68  Event   0x1f0003        -
4       System  0x840e9798bd50  0x2b6c  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e998450c0  0x2b70  Process 0x0     SearchApp.exe Pid 4236
4       System  0x840e98bba080  0x2b74  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e9ba08300  0x2b78  WaitCompletionPacket    0x1     -
4       System  0x840e990ab880  0x2b7c  WaitCompletionPacket    0x1     -
4       System  0x840e90b0d080  0x2b80  Process 0x102a  svchost.exe Pid 9452
4       System  0x840e974b2080  0x2b84  Process 0x0     UserOOBEBroker Pid 9620
4       System  0x840e99387080  0x2b88  Process 0x102a  StartMenuExper Pid 7408
4       System  0x840e98ea0360  0x2b8c  Event   0x1f0003        -
4       System  0x840e98b37710  0x2b90  WaitCompletionPacket    0x1     -
4       System  0x840e97136530  0x2b94  WaitCompletionPacket    0x1     -
4       System  0x840e98b291f0  0x2b98  WaitCompletionPacket    0x1     -
4       System  0x840e9952c080  0x2ba4  Process 0x1fffff        csrss.exe Pid 10732
4       System  0x840e98b56ed0  0x2ba8  WaitCompletionPacket    0x1     -
4       System  0x840e9b992360  0x2bac  WaitCompletionPacket    0x1     -
4       System  0xb708087bb5b0  0x2bb4  Section 0x5     -
4       System  0x840e99037300  0x2bb8  IoCompletion    0x1f0003        -
4       System  0xb70812d09fd0  0x2bbc  SymbolicLink    0xf0001 Session
4       System  0x840e99fd97e0  0x2bc0  Event   0x1f0003        -
4       System  0x840e9817ed40  0x2bc4  WaitCompletionPacket    0x1     -
4       System  0x840e9b372080  0x2bc8  Process 0x1fffff        taskhostw.exe Pid 10476
4       System  0x840e9909f650  0x2bcc  WaitCompletionPacket    0x1     -
4       System  0x840e9c5554a0  0x2bd0  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-2246530975-808720366-1776470054-230329187-4153223113-3550430174-4193313734
4       System  0x840e97b09080  0x2bd4  Process 0x102a  powershell.exe Pid 15172
4       System  0x840e92e30080  0x2bd8  Process 0x0     svchost.exe Pid 1392
4       System  0x840e97387f80  0x2bdc  IoCompletion    0x1f0003        -
4       System  0xb7081121aaf0  0x2be0  SymbolicLink    0xf0001 Global
4       System  0x840e97e9e0d0  0x2be4  CoreMessaging   0xf0000 -
4       System  0x840e97f7e080  0x2be8  Process 0x0     svchost.exe Pid 5772
4       System  0x840e98c48080  0x2bec  Process 0x1fffff        ApplicationFra Pid 4836
4       System  0x840e990a8bd0  0x2bf0  WaitCompletionPacket    0x1     -
4       System  0x840e997b7de0  0x2bf4  Event   0x1f0003        -
4       System  0x840e97e9e9d0  0x2bf8  CoreMessaging   0xf0000 -
4       System  0x840e9909d1c0  0x2bfc  WaitCompletionPacket    0x1     -
4       System  0x840e996e91c0  0x2c08  Process 0x1fffff        msedge.exe Pid 9016
4       System  0xb70816ebda00  0x2c0c  Directory       0xf     S-1-15-2-155514346-2573954481-755741238-1654018636-1233331829-3075935687-2861478708
4       System  0x840e923de2c0  0x2c10  Process 0x102a  svchost.exe Pid 756
4       System  0x840e98bd6240  0x2c14  Process 0x102a  RuntimeBroker. Pid 8120
4       System  0x840e921650c0  0x2c1c  Process 0x0     csrss.exe Pid 804
4       System  0x840e9b99fdf0  0x2c20  WaitCompletionPacket    0x1     -
4       System  0xb708106afd20  0x2c24  Directory       0xf     S-1-15-2-95739096-486727260-2033287795-3853587803-1685597119-444378811-2746676523
4       System  0x840e9c8632e0  0x2c2c  Event   0x1f0003        -
4       System  0x840e9909e130  0x2c34  WaitCompletionPacket    0x1     -
4       System  0x840e99014680  0x2c38  IoCompletion    0x1f0003        -
4       System  0x840e9a148640  0x2c3c  File    0x1     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-5-21-3294271487-4292050549-1661813568-1001\SystemAppData\Helium\Cache\b4d622c333fcfdd7.dat
4       System  0x840e98189160  0x2c40  WaitCompletionPacket    0x1     -
4       System  0x840e9a652870  0x2c44  IoCompletionReserve     0xf0003 -
4       System  0x840e9c896f60  0x2c48  Event   0x1f0003        -
4       System  0x840e9908b630  0x2c4c  WaitCompletionPacket    0x1     -
4       System  0x840e98aef080  0x2c50  Process 0x102a  RuntimeBroker. Pid 7684
4       System  0x840e9ac0ea10  0x2c54  WaitCompletionPacket    0x1     -
4       System  0x840e9969e080  0x2c58  Process 0x102a  svchost.exe Pid 4156
4       System  0xb70811872430  0x2c5c  SymbolicLink    0xf0001 Global
4       System  0x840e99fcc960  0x2c60  Event   0x1f0003        -
4       System  0x840e990eddc0  0x2c64  ALPC Port       0x1f0001        -
4       System  0x840e9a014b70  0x2c68  IoCompletionReserve     0xf0003 -
4       System  0x840e98d2d580  0x2c6c  IoCompletion    0x1f0003        -
4       System  0x840e97989960  0x2c70  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Search_1.14.15.19041_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG1
4       System  0x840e97139e10  0x2c78  WaitCompletionPacket    0x1     -
4       System  0xb7080879c7f0  0x2c7c  Section 0x5     -
4       System  0x840e97d55e80  0x2c80  IoCompletion    0x1f0003        -
4       System  0x840e9ccec080  0x2c84  Process 0x102a  svchost.exe Pid 11776
4       System  0x840e9713fb80  0x2c88  WaitCompletionPacket    0x1     -
4       System  0x840e9bbc9120  0x2c8c  File    0x1     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-12-1-419028606-1323772725-945580434-2117921032\SystemAppData\Helium\Cache\b4d622c333fcfdd7.dat
4       System  0x840e99387080  0x2c90  Process 0x102a  StartMenuExper Pid 7408
4       System  0x840e99adfa00  0x2c94  IoCompletion    0x1f0003        -
4       System  0x840e99bcb080  0x2c98  Process 0x102a  explorer.exe Pid 9852
4       System  0x840e99474080  0x2c9c  Process 0x102a  PSEXESVC.exe Pid 11296
4       System  0x840e99f696c0  0x2ca0  IoCompletion    0x1f0003        -
4       System  0x840e99783440  0x2ca4  Thread  0x0     Tid 6556 Pid 4
4       System  0x840e9a9574e0  0x2ca8  Event   0x1f0003        -
4       System  0x840e98b38340  0x2cac  WaitCompletionPacket    0x1     -
4       System  0x840e9a624540  0x2cb0  IoCompletion    0x1f0003        -
4       System  0xb70811218ec0  0x2cb4  SymbolicLink    0xf0001 Session
4       System  0x840e968c22c0  0x2cb8  Process 0x0     svchost.exe Pid 2504
4       System  0x840e9ba0cb50  0x2cc4  WaitCompletionPacket    0x1     -
4       System  0x840e990835d0  0x2cc8  WaitCompletionPacket    0x1     -
4       System  0x840e9958a0c0  0x2ccc  Process 0x102a  msedge.exe Pid 9128
4       System  0x840e92349080  0x2cd0  Process 0x102a  lsass.exe Pid 548
4       System  0x840e98c17c60  0x2cd4  Event   0x1f0003        -
4       System  0x840e98dbf2c0  0x2cd8  Process 0x102a  svchost.exe Pid 8688
4       System  0x840e966d9300  0x2cdc  Process 0x1fffff        svchost.exe Pid 8912
4       System  0x840e997b44e0  0x2ce0  Event   0x1f0003        -
4       System  0x840e9798a2c0  0x2ce4  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Search_1.14.15.19041_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG2
4       System  0x840e9689a080  0x2ce8  Process 0x0     svchost.exe Pid 2472
4       System  0x840e910b4040  0x2cec  Process 0x0     smss.exe Pid 536
4       System  0x840e9c9be1e0  0x2cf0  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-515815643-2845804217-1874292103-218650560-777617685-4287762684-137415000
4       System  0x840e9a14aa30  0x2cf4  File    0x20003 \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-5-21-3294271487-4292050549-1661813568-1001\SystemAppData\Helium\Cache\b4d622c333fcfdd7_COM15.dat
4       System  0x840e98c48080  0x2cf8  Process 0x0     ApplicationFra Pid 4836
4       System  0x840e9682f080  0x2cfc  Process 0x0     OneDrive.exe Pid 3320
4       System  0x840e9682f080  0x2d00  Process 0x1fffff        OneDrive.exe Pid 3320
4       System  0x840e9cf352c0  0x2d04  IoCompletion    0x1f0003        -
4       System  0x840e98bba080  0x2d08  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e998450c0  0x2d0c  Process 0x40    SearchApp.exe Pid 4236
4       System  0x840e971385b0  0x2d14  WaitCompletionPacket    0x1     -
4       System  0x840e9cb38260  0x2d18  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\MicrosoftWindows.Client.CBS_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e98b297a0  0x2d1c  WaitCompletionPacket    0x1     -
4       System  0x840e98b3a220  0x2d20  WaitCompletionPacket    0x1     -
4       System  0x840e997a7ae0  0x2d24  Event   0x1f0003        -
4       System  0x840e98b15080  0x2d28  Process 0x102a  fontdrvhost.ex Pid 10704
4       System  0x840e98df3080  0x2d2c  Process 0x102a  ShellExperienc Pid 704
4       System  0x840e9ac24ec0  0x2d30  WaitCompletionPacket    0x1     -
4       System  0x840e9ac422c0  0x2d34  WaitCompletionPacket    0x1     -
4       System  0x840e99084540  0x2d38  WaitCompletionPacket    0x1     -
4       System  0x840e9ac2fbd0  0x2d40  WaitCompletionPacket    0x1     -
4       System  0x840e99ab78c0  0x2d44  IoCompletion    0x1f0003        -
4       System  0x840e999edee0  0x2d48  Event   0x1f0003        -
4       System  0x840e99c83080  0x2d4c  Process 0x102a  dwm.exe Pid 10628
4       System  0x840e979be390  0x2d54  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\MicrosoftWindows.Client.CBS_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e98b51a50  0x2d5c  WaitCompletionPacket    0x1     -
4       System  0x840e97391dc0  0x2d60  IoCompletion    0x1f0003        -
4       System  0x840e979911b0  0x2d64  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\MicrosoftWindows.Client.CBS_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e9937dd20  0x2d68  ALPC Port       0x1f0001        -
4       System  0x840e98192060  0x2d6c  WaitCompletionPacket    0x1     -
4       System  0x840e9b98a8b0  0x2d70  WaitCompletionPacket    0x1     -
4       System  0x840e98b5a950  0x2d74  WaitCompletionPacket    0x1     -
4       System  0x840ea534dae0  0x2d78  Event   0x1f0003        -
4       System  0x840e97582080  0x2d7c  Process 0x102a  msedge.exe Pid 16344
4       System  0x840e990a0b70  0x2d80  WaitCompletionPacket    0x1     -
4       System  0x840e923de2c0  0x2d84  Process 0x102a  svchost.exe Pid 756
4       System  0x840e9bee60c0  0x2d8c  Process 0x102a  svchost.exe Pid 12440
4       System  0x840e99093280  0x2d90  WaitCompletionPacket    0x1     -
4       System  0x840e996cf080  0x2d94  Process 0x102a  svchost.exe Pid 9320
4       System  0x840e978e12e0  0x2d98  WaitCompletionPacket    0x1     -
4       System  0x840e98fb61c0  0x2d9c  Process 0x102a  dllhost.exe Pid 8472
4       System  0x840e996cf080  0x2da0  Process 0x102a  svchost.exe Pid 9320
4       System  0x840e91d8ed60  0x2da4  Event   0x1f0003        -
4       System  0x840e90c13ac0  0x2da8  ALPC Port       0x1f0001        -
4       System  0x840e98d0c0c0  0x2dac  IoCompletion    0x1f0003        -
4       System  0x840e9a149db0  0x2db0  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\UserClasses.dat.LOG1
4       System  0x840e98db4080  0x2db4  Process 0x102a  msedge.exe Pid 9888
4       System  0x840e96d0c700  0x2db8  IoCompletion    0x1f0003        -
4       System  0x840e998450c0  0x2dbc  Process 0x102a  SearchApp.exe Pid 4236
4       System  0x840e9a14c4c0  0x2dc0  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.ShellExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG1
4       System  0x840e9c8961e0  0x2dc8  Event   0x1f0003        -
4       System  0x840e9856b2c0  0x2dcc  Process 0x102a  svchost.exe Pid 7780
4       System  0x840e98d0b540  0x2dd0  IoCompletion    0x1f0003        -
4       System  0x840e9a325d30  0x2dd4  IoCompletionReserve     0xf0003 -
4       System  0x840ea3ce2be0  0x2dd8  Event   0x1f0003        -
4       System  0x840e99ac8b80  0x2ddc  IoCompletion    0x1f0003        -
4       System  0x840e9951c1c0  0x2de0  Process 0x102a  msedge.exe Pid 8732
4       System  0x840e99075b40  0x2de4  WaitCompletionPacket    0x1     -
4       System  0x840e990ad4f0  0x2de8  WaitCompletionPacket    0x1     -
4       System  0x840e9908bf20  0x2dec  WaitCompletionPacket    0x1     -
4       System  0x840e99041b00  0x2df0  IoCompletion    0x1f0003        -
4       System  0x840e9b7bb080  0x2df4  Process 0x102a  StartMenuExper Pid 8
4       System  0x840e98bd6240  0x2df8  Process 0x102a  RuntimeBroker. Pid 8120
4       System  0x840e990aea10  0x2dfc  WaitCompletionPacket    0x1     -
4       System  0x840e96f38680  0x2e04  CoreMessaging   0xf0000 -
4       System  0x840e92fc5300  0x2e08  Process 0x0     svchost.exe Pid 1976
4       System  0x840e98b36a10  0x2e0c  WaitCompletionPacket    0x1     -
4       System  0x840e98fd2f60  0x2e10  Event   0x1f0003        -
4       System  0xb70811219de0  0x2e14  SymbolicLink    0xf0001 Local
4       System  0x840e97152ea0  0x2e1c  WaitCompletionPacket    0x1     -
4       System  0x840e96d4a920  0x2e20  Timer   0x1f0003        -
4       System  0x840e99f82940  0x2e24  IoCompletion    0x1f0003        -
4       System  0x840e9e463cc0  0x2e28  TmEn    0xf001f -
4       System  0x840e98e88960  0x2e2c  Event   0x1f0003        -
4       System  0x840e9ac2ed30  0x2e34  WaitCompletionPacket    0x1     -
4       System  0x840e98b50c80  0x2e38  WaitCompletionPacket    0x1     -
4       System  0x840e99f6b3c0  0x2e3c  IoCompletion    0x1f0003        -
4       System  0x840ea8d6a080  0x2e44  Process 0x1fffff        OpenHandleColl Pid 11408
4       System  0x840e99051a80  0x2e48  IoCompletion    0x1f0003        -
4       System  0xb70811697cc0  0x2e4c  Directory       0xf     S-1-15-2-536077884-713174666-1066051701-3219990555-339840825-1966734348-1611281757
4       System  0x840e90e10410  0x2e50  Session 0x0     Session2
4       System  0x840e98a30080  0x2e54  IoCompletion    0x1f0003        -
4       System  0x840e90e10410  0x2e58  Session 0x0     Session2
4       System  0x840e98c1fc60  0x2e5c  Event   0x1f0003        -
4       System  0x840e9969e080  0x2e60  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e9ba0dac0  0x2e64  WaitCompletionPacket    0x1     -
4       System  0x840e992c4d40  0x2e6c  IoCompletion    0x1f0003        -
4       System  0xb7080a748a30  0x2e70  Section 0x17    -
4       System  0x840e97d586c0  0x2e74  IoCompletion    0x1f0003        -
4       System  0x840e90bb8160  0x2e78  Event   0x1f0003        -
4       System  0x840e9713a490  0x2e7c  WaitCompletionPacket    0x1     -
4       System  0x840e90bcb760  0x2e80  Event   0x1f0003        -
4       System  0x840e99f69340  0x2e84  IoCompletion    0x1f0003        -
4       System  0x840e99b30b10  0x2e88  File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-3251537155-1984446955-2931258699-841473695-1938553385-924012159-129201922
4       System  0x840e9e3cbae0  0x2e8c  Event   0x1f0003        -
4       System  0x840e9856b2c0  0x2e90  Process 0x102a  svchost.exe Pid 7780
4       System  0x840e99089d00  0x2e98  WaitCompletionPacket    0x1     -
4       System  0x840e91da7560  0x2e9c  Event   0x1f0003        -
4       System  0x840e992e1880  0x2ea0  IoCompletion    0x1f0003        -
4       System  0x840e97e11080  0x2ea4  Process 0x40    TextInputHost. Pid 1256
4       System  0x840e99026800  0x2ea8  IoCompletion    0x1f0003        -
4       System  0x840e92f36a00  0x2eac  IoCompletion    0x1f0003        -
4       System  0x840e98e949e0  0x2eb0  Event   0x1f0003        -
4       System  0x840e9a60f440  0x2eb8  IoCompletion    0x1f0003        -
4       System  0x840e98b319a0  0x2ebc  WaitCompletionPacket    0x1     -
4       System  0x840e9c895c60  0x2ec4  Event   0x1f0003        -
4       System  0x840e98bba080  0x2ecc  Process 0x1fffff        SearchApp.exe Pid 7932
4       System  0x840e975e71e0  0x2ed0  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e91bc24c0  0x2ed4  WaitCompletionPacket    0x1     -
4       System  0x840e9dad3c40  0x2ed8  IoCompletion    0x1f0003        -
4       System  0x840e99036780  0x2edc  IoCompletion    0x1f0003        -
4       System  0xb70813b9c0c0  0x2ee0  Directory       0xf     S-1-15-2-2226957697-3030467180-2301525-4248967783-2024719031-2325529081-2915787518
4       System  0x840e98a128c0  0x2ee4  IoCompletion    0x1f0003        -
4       System  0x840e9856b2c0  0x2ee8  Process 0x102a  svchost.exe Pid 7780
4       System  0x840e96958900  0x2eec  IoCompletion    0x1f0003        -
4       System  0x840e98b8e340  0x2ef0  Process 0x102a  RuntimeBroker. Pid 7612
4       System  0x840e973d81c0  0x2ef8  Process 0x0     svchost.exe Pid 4800
4       System  0x840e92fc7080  0x2efc  Process 0x0     svchost.exe Pid 1988
4       System  0x840e9856b2c0  0x2f00  Process 0x102a  svchost.exe Pid 7780
4       System  0x840e978e99c0  0x2f04  WaitCompletionPacket    0x1     -
4       System  0x840e99971f30  0x2f08  IoCompletionReserve     0xf0003 -
4       System  0x840e97e59080  0x2f0c  Process 0x102a  conhost.exe Pid 12392
4       System  0x840e98e87a60  0x2f14  Event   0x1f0003        -
4       System  0x840e99f80c80  0x2f20  IoCompletion    0x1f0003        -
4       System  0x840e9c523ee0  0x2f24  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat
4       System  0x840e997a44e0  0x2f28  Event   0x1f0003        -
4       System  0x840e918e90c0  0x2f2c  Process 0x0     svchost.exe Pid 14128
4       System  0x840e9ac3db40  0x2f30  WaitCompletionPacket    0x1     -
4       System  0x840e9db77080  0x2f34  Process 0x102a  taskhostw.exe Pid 15784
4       System  0x840e92e04300  0x2f38  Process 0x0     svchost.exe Pid 1408
4       System  0x840e99aaf740  0x2f3c  IoCompletion    0x1f0003        -
4       System  0x840e9856b2c0  0x2f40  Process 0x1fffff        svchost.exe Pid 7780
4       System  0x840e970c8880  0x2f44  IoCompletion    0x1f0003        -
4       System  0x840e998450c0  0x2f48  Process 0x102a  SearchApp.exe Pid 4236
4       System  0x840e9856b2c0  0x2f4c  Process 0x102a  svchost.exe Pid 7780
4       System  0xb708087979d0  0x2f54  Section 0x17    -
4       System  0x840e98aef080  0x2f58  Process 0x102a  RuntimeBroker. Pid 7684
4       System  0x840e9856b2c0  0x2f5c  Process 0x102a  svchost.exe Pid 7780
4       System  0x840e97baf090  0x2f60  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e98b30ca0  0x2f68  WaitCompletionPacket    0x1     -
4       System  0x840e993240c0  0x2f6c  Process 0x0     sihost.exe Pid 10384
4       System  0x840e9c856160  0x2f74  Event   0x1f0003        -
4       System  0x840e98b8e340  0x2f78  Process 0x1fffff        RuntimeBroker. Pid 7612
4       System  0x840e90de9080  0x2f7c  Process 0x102a  FileCoAuth.exe Pid 8592
4       System  0x840e99474080  0x2f80  Process 0x1fffff        PSEXESVC.exe Pid 11296
4       System  0xb7081a5c65a0  0x2f84  SymbolicLink    0xf0001 Local
4       System  0x840e966b1080  0x2f88  Process 0x0     MemCompression Pid 1924
4       System  0x840e9ceb5160  0x2f8c  Event   0x1f0003        -
4       System  0xb70822a63a60  0x2f90  Token   0xe     -
4       System  0x840e98a2a740  0x2f94  IoCompletion    0x1f0003        -
4       System  0x840e98fe5d60  0x2f98  Event   0x1f0003        -
4       System  0x840ea0ed2360  0x2f9c  Event   0x1f0003        -
4       System  0x840e9934dc60  0x2fa0  Event   0x1f0003        -
4       System  0x840e9769f080  0x2fa4  Process 0x0     winlogon.exe Pid 2804
4       System  0x840e9bdf4340  0x2fa8  Process 0x102a  ScreenConnect. Pid 14756
4       System  0x840e96a0a7b0  0x2fac  File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-536077884-713174666-1066051701-3219990555-339840825-1966734348-1611281757
4       System  0x840e91d977e0  0x2fb0  Event   0x1f0003        -
4       System  0x840e98dbf2c0  0x2fb4  Process 0x102a  svchost.exe Pid 8688
4       System  0x840e97a65b00  0x2fc0  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e999ec460  0x2fc4  Event   0x1f0003        -
4       System  0x840e9f40c340  0x2fc8  Process 0x1fffff        svchost.exe Pid 2800
4       System  0x840e974b4080  0x2fcc  Process 0x0     CSFalconContai Pid 324
4       System  0x840e98c24a60  0x2fd0  Event   0x1f0003        -
4       System  0x840e99078990  0x2fd4  WaitCompletionPacket    0x1     -
4       System  0x840e99f697c0  0x2fd8  IoCompletion    0x1f0003        -
4       System  0x840e9745b760  0x2fdc  Event   0x1f0003        -
4       System  0x840e98e8e060  0x2fe0  Event   0x1f0003        -
4       System  0x840e9c71db60  0x2fe8  Event   0x1f0003        -
4       System  0x840e97656100  0x2fec  IoCompletion    0x1f0003        -
4       System  0x840e96da9300  0x2ff4  Process 0x0     svchost.exe Pid 3332
4       System  0x840e990b0680  0x2ff8  WaitCompletionPacket    0x1     -
4       System  0x840e990b0820  0x2ffc  WaitCompletionPacket    0x1     -
4       System  0x840e9f02adf0  0x3004  WaitCompletionPacket    0x1     -
4       System  0x840e997b1860  0x300c  Event   0x1f0003        -
4       System  0x840e99aad180  0x3010  IoCompletion    0x1f0003        -
4       System  0x840e99f82780  0x3014  IoCompletion    0x1f0003        -
4       System  0x840e98a11680  0x3018  IoCompletion    0x1f0003        -
4       System  0x840e98dc11c0  0x301c  Process 0x102a  SecurityHealth Pid 8736
4       System  0x840e9979f2e0  0x3020  Event   0x1f0003        -
4       System  0x840e90e10410  0x3024  Session 0x0     Session2
4       System  0xb7081319baa0  0x3028  Directory       0xf     RPC Control
4       System  0x840e9a94a460  0x302c  Event   0x1f0003        -
4       System  0x840e997b1c60  0x3030  Event   0x1f0003        -
4       System  0x840e997176e0  0x3034  Event   0x1f0003        -
4       System  0x840e9958a0c0  0x3038  Process 0x1fffff        msedge.exe Pid 9128
4       System  0x840e9979f560  0x303c  Event   0x1f0003        -
4       System  0x840e97142760  0x3040  WaitCompletionPacket    0x1     -
4       System  0x840e9666ee20  0x3044  WaitCompletionPacket    0x1     -
4       System  0x840e98d0a840  0x3048  IoCompletion    0x1f0003        -
4       System  0x840e98e92260  0x304c  Event   0x1f0003        -
4       System  0x840e98e85d60  0x3050  Event   0x1f0003        -
4       System  0x840e90bc5560  0x3054  Event   0x1f0003        -
4       System  0x840e9ac28ae0  0x3058  WaitCompletionPacket    0x1     -
4       System  0x840e99aa7140  0x305c  IoCompletion    0x1f0003        -
4       System  0x840e99bcb080  0x3060  Process 0x102a  explorer.exe Pid 9852
4       System  0x840e9958a0c0  0x3064  Process 0x102a  msedge.exe Pid 9128
4       System  0x840e99976eb0  0x3068  IoCompletionReserve     0xf0003 -
4       System  0x840e9a95cbe0  0x3070  Event   0x1f0003        -
4       System  0x840e9e99e080  0x3074  Process 0x102a  dllhost.exe Pid 7884
4       System  0x840e98f8b570  0x3078  IoCompletionReserve     0xf0003 -
4       System  0x840e999f60e0  0x307c  Event   0x1f0003        -
4       System  0x840e990799d0  0x3080  WaitCompletionPacket    0x1     -
4       System  0x840e997a2660  0x3084  Event   0x1f0003        -
4       System  0x840e985cee20  0x3088  FilterCommunicationPort 0x1f0001        -
4       System  0x840e98c2cce0  0x308c  Event   0x1f0003        -
4       System  0x840e98d0de80  0x3090  IoCompletion    0x1f0003        -
4       System  0x840e9907d2b0  0x3094  WaitCompletionPacket    0x1     -
4       System  0x840e990760f0  0x3098  WaitCompletionPacket    0x1     -
4       System  0x840e98d0c600  0x309c  IoCompletion    0x1f0003        -
4       System  0x840e971473c0  0x30a0  WaitCompletionPacket    0x1     -
4       System  0x840e98d28340  0x30a4  IoCompletion    0x1f0003        -
4       System  0x840e978e3500  0x30a8  WaitCompletionPacket    0x1     -
4       System  0x840e98d255c0  0x30ac  IoCompletion    0x1f0003        -
4       System  0x840e9951c1c0  0x30b0  Process 0x1fffff        msedge.exe Pid 8732
4       System  0x840e96794080  0x30b4  Process 0x0     svchost.exe Pid 2276
4       System  0x840e98ff32c0  0x30b8  Process 0x1fffff        SecurityHealth Pid 8808
4       System  0x840e98184430  0x30bc  WaitCompletionPacket    0x1     -
4       System  0x840e99f76700  0x30c0  IoCompletion    0x1f0003        -
4       System  0x840e990502c0  0x30c4  IoCompletion    0x1f0003        -
4       System  0x840e9c858d60  0x30c8  Event   0x1f0003        -
4       System  0x840e9904d1c0  0x30cc  IoCompletion    0x1f0003        -
4       System  0x840e9907be60  0x30d0  WaitCompletionPacket    0x1     -
4       System  0x840e98fd93e0  0x30d4  Event   0x1f0003        -
4       System  0x840e99f61d40  0x30d8  IoCompletion    0x1f0003        -
4       System  0x840e98fdd0e0  0x30dc  Event   0x1f0003        -
4       System  0x840e99aa4a00  0x30e4  IoCompletion    0x1f0003        -
4       System  0x840e99030a00  0x30e8  IoCompletion    0x1f0003        -
4       System  0x840e98dc11c0  0x30ec  Process 0x102a  SecurityHealth Pid 8736
4       System  0x840e92fbb0c0  0x30f0  Process 0x0     svchost.exe Pid 604
4       System  0x840e997aa260  0x30f4  Event   0x1f0003        -
4       System  0x840e99fd09e0  0x30f8  Event   0x1f0003        -
4       System  0x840e981809b0  0x30fc  WaitCompletionPacket    0x1     -
4       System  0x840e9979cde0  0x3100  Event   0x1f0003        -
4       System  0x840e9b73a080  0x3104  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e98a30a00  0x3108  IoCompletion    0x1f0003        -
4       System  0x840e98fd5060  0x310c  Event   0x1f0003        -
4       System  0x840e99095980  0x3110  WaitCompletionPacket    0x1     -
4       System  0x840e9979cae0  0x3114  Event   0x1f0003        -
4       System  0x840e99d66a20  0x3118  ALPC Port       0x1f0001        -
4       System  0x840e98dc11c0  0x311c  Process 0x1fffff        SecurityHealth Pid 8736
4       System  0x840e9f0024c0  0x3120  WaitCompletionPacket    0x1     -
4       System  0x840e9ba0f180  0x3128  WaitCompletionPacket    0x1     -
4       System  0x840e9902c680  0x312c  IoCompletion    0x1f0003        -
4       System  0x840e97add1c0  0x3130  Process 0x102a  svchost.exe Pid 6020
4       System  0x840e96e3d080  0x3134  Process 0x0     svchost.exe Pid 3528
4       System  0x840e98180c20  0x3138  WaitCompletionPacket    0x1     -
4       System  0x840ea46df1e0  0x313c  Event   0x1f0003        -
4       System  0x840e98a0e3c0  0x3140  IoCompletion    0x1f0003        -
4       System  0x840e97b71740  0x3144  IoCompletionReserve     0xf0003 -
4       System  0x840e9682f080  0x3148  Process 0x102a  OneDrive.exe Pid 3320
4       System  0x840e98fe9ce0  0x314c  Event   0x1f0003        -
4       System  0x840e9a603bc0  0x3150  IoCompletion    0x1f0003        -
4       System  0x840e96864380  0x3154  Process 0x0     svchost.exe Pid 2384
4       System  0x840e9f7a7e80  0x3158  IoCompletion    0x1f0003        -
4       System  0x840e9817e6c0  0x315c  WaitCompletionPacket    0x1     -
4       System  0x840e9a61e6c0  0x3160  IoCompletion    0x1f0003        -
4       System  0x840e9832a1c0  0x3164  Process 0x1fffff        msedge.exe Pid 3832
4       System  0x840e976562c0  0x3168  IoCompletion    0x1f0003        -
4       System  0x840e9951c1c0  0x316c  Process 0x102a  msedge.exe Pid 8732
4       System  0x840e99c2bae0  0x3170  Event   0x1f0003        -
4       System  0x840e98b63100  0x3174  WaitCompletionPacket    0x1     -
4       System  0x840e996e91c0  0x3178  Process 0x102a  msedge.exe Pid 9016
4       System  0x840e997b0660  0x317c  Event   0x1f0003        -
4       System  0x840e978e47b0  0x3180  WaitCompletionPacket    0x1     -
4       System  0x840e9bdf4340  0x3184  Process 0x0     ScreenConnect. Pid 14756
4       System  0x840e96e552c0  0x3188  Process 0x0     svchost.exe Pid 3560
4       System  0x840e96d9f080  0x318c  Process 0x0     svchost.exe Pid 3308
4       System  0x840e997ae1e0  0x3190  Event   0x1f0003        -
4       System  0x840e98ff32c0  0x3194  Process 0x102a  SecurityHealth Pid 8808
4       System  0x840e9ba113a0  0x3198  WaitCompletionPacket    0x1     -
4       System  0x840e99031480  0x319c  IoCompletion    0x1f0003        -
4       System  0x840e9c85fa60  0x31a0  Event   0x1f0003        -
4       System  0x840e98fd45e0  0x31a4  Event   0x1f0003        -
4       System  0x840e98dbf2c0  0x31a8  Process 0x102a  svchost.exe Pid 8688
4       System  0xb70818ebd310  0x31ac  Token   0xf01ff -
4       System  0x840e99077e30  0x31b4  WaitCompletionPacket    0x1     -
4       System  0x840e99ad29c0  0x31b8  IoCompletion    0x1f0003        -
4       System  0x840e9b9b96a0  0x31bc  WaitCompletionPacket    0x1     -
4       System  0x840e99092e70  0x31c0  WaitCompletionPacket    0x1     -
4       System  0x840e990422c0  0x31c4  IoCompletion    0x1f0003        -
4       System  0x840e998251c0  0x31c8  Process 0x102a  msedge.exe Pid 9728
4       System  0x840e9ceb47e0  0x31cc  Event   0x1f0003        -
4       System  0x840e99702d60  0x31d0  Event   0x1f0003        -
4       System  0x840e98fd03e0  0x31d4  Event   0x1f0003        -
4       System  0x840e98f07080  0x31d8  Process 0x102a  msedge.exe Pid 9172
4       System  0x840e98e8fb60  0x31dc  Event   0x1f0003        -
4       System  0x840ea87f9f40  0x31e0  IoCompletion    0x1f0003        -
4       System  0x840e990afb20  0x31e4  WaitCompletionPacket    0x1     -
4       System  0x840e91ba00c0  0x31e8  Process 0x102a  svchost.exe Pid 7220
4       System  0x840e96bf8660  0x31ec  Event   0x1f0003        -
4       System  0x840e9904df00  0x31f0  IoCompletion    0x1f0003        -
4       System  0x840e96d1a8c0  0x31f4  IoCompletion    0x1f0003        -
4       System  0x840e990af570  0x31f8  WaitCompletionPacket    0x1     -
4       System  0x840e9979a3e0  0x31fc  Event   0x1f0003        -
4       System  0x840e99aae5c0  0x3200  IoCompletion    0x1f0003        -
4       System  0x840e99f578c0  0x3204  IoCompletion    0x1f0003        -
4       System  0x840e9979aa60  0x3208  Event   0x1f0003        -
4       System  0x840e9dae9e80  0x320c  IoCompletion    0x1f0003        -
4       System  0x840e9713b9b0  0x3210  WaitCompletionPacket    0x1     -
4       System  0x840e9979a560  0x3214  Event   0x1f0003        -
4       System  0xb70812d09e70  0x3218  SymbolicLink    0xf0001 Local
4       System  0x840e996e91c0  0x321c  Process 0x102a  msedge.exe Pid 9016
4       System  0x840e9951c1c0  0x3220  Process 0x102a  msedge.exe Pid 8732
4       System  0xb70812d09840  0x3224  SymbolicLink    0xf0001 Global
4       System  0x840e97148260  0x3228  WaitCompletionPacket    0x1     -
4       System  0x840e92349080  0x322c  Process 0x0     lsass.exe Pid 548
4       System  0x840e9c45e540  0x3234  IoCompletion    0x1f0003        -
4       System  0x840e999dca60  0x3238  Event   0x1f0003        -
4       System  0x840e994d1080  0x323c  Process 0x102a  LogonUI.exe Pid 10680
4       System  0x840e98db8080  0x3240  Process 0x1fffff        MsSense.exe Pid 10224
4       System  0x840e9979c2e0  0x3244  Event   0x1f0003        -
4       System  0x840e92ee4c30  0x3248  WaitCompletionPacket    0x1     -
4       System  0x840e997a0560  0x324c  Event   0x1f0003        -
4       System  0x840e9676e080  0x3250  Process 0x0     svchost.exe Pid 2168
4       System  0x840e9909ad30  0x3254  WaitCompletionPacket    0x1     -
4       System  0x840e997a6260  0x3258  Event   0x1f0003        -
4       System  0x840e96999270  0x325c  WaitCompletionPacket    0x1     -
4       System  0x840e96da72c0  0x3260  Process 0x0     svchost.exe Pid 3324
4       System  0x840e9745f7e0  0x3264  Event   0x1f0003        -
4       System  0x840e99aaf200  0x3268  IoCompletion    0x1f0003        -
4       System  0x840e9817f700  0x326c  WaitCompletionPacket    0x1     -
4       System  0x840e98b5e090  0x3270  WaitCompletionPacket    0x1     -
4       System  0x840e98a1edc0  0x3274  IoCompletion    0x1f0003        -
4       System  0x840e96956c00  0x3278  IoCompletion    0x1f0003        -
4       System  0x840e96e042c0  0x327c  Process 0x0     svchost.exe Pid 3376
4       System  0x840e98db4080  0x3280  Process 0x1fffff        msedge.exe Pid 9888
4       System  0xb707fbcbaac0  0x3284  SymbolicLink    0xf0001 Session
4       System  0x840e99aa6440  0x3288  IoCompletion    0x1f0003        -
4       System  0x840e9bdf4340  0x328c  Process 0x1fffff        ScreenConnect. Pid 14756
4       System  0x840e99aa4c80  0x3290  IoCompletion    0x1f0003        -
4       System  0x840e91bc2a70  0x3294  WaitCompletionPacket    0x1     -
4       System  0x840e923f61c0  0x329c  Process 0x0     fontdrvhost.ex Pid 892
4       System  0x840e9c70bb60  0x32a0  Event   0x1f0003        -
4       System  0x840e980787a0  0x32a4  CoreMessaging   0xf0000 -
4       System  0x840e9b993c90  0x32a8  WaitCompletionPacket    0x1     -
4       System  0x840e96806a90  0x32ac  WaitCompletionPacket    0x1     -
4       System  0x840e997a7c60  0x32b0  Event   0x1f0003        -
4       System  0x840e92d4f440  0x32b8  IoCompletion    0x1f0003        -
4       System  0x840e99ad3e00  0x32bc  IoCompletion    0x1f0003        -
4       System  0x840e9a9443e0  0x32c0  Event   0x1f0003        -
4       System  0x840e92d48080  0x32c4  Process 0x0     svchost.exe Pid 1364
4       System  0x840e98d17f80  0x32c8  IoCompletion    0x1f0003        -
4       System  0x840e92c4fd10  0x32cc  WaitCompletionPacket    0x1     -
4       System  0x840e99f4ba00  0x32d0  IoCompletion    0x1f0003        -
4       System  0x840e9ac0a0f0  0x32d4  WaitCompletionPacket    0x1     -
4       System  0x840e98b48260  0x32d8  WaitCompletionPacket    0x1     -
4       System  0x840e91ba00c0  0x32dc  Process 0x1fffff        svchost.exe Pid 7220
4       System  0x840e98bdf080  0x32e0  Process 0x102a  SystemSettings Pid 10020
4       System  0x840e996cf080  0x32e4  Process 0x102a  svchost.exe Pid 9320
4       System  0x840e91f131c0  0x32e8  Process 0x102a  Sysmon64.exe Pid 1244
4       System  0x840e998251c0  0x32f0  Process 0x1fffff        msedge.exe Pid 9728
4       System  0x840e99aac640  0x32f4  IoCompletion    0x1f0003        -
4       System  0x840e9933d360  0x32f8  Event   0x1f0003        -
4       System  0x840e9f38b840  0x3300  IoCompletion    0x1f0003        -
4       System  0x840e999e2de0  0x3304  Event   0x1f0003        -
4       System  0x840e98bdf080  0x3308  Process 0x102a  SystemSettings Pid 10020
4       System  0x840e9ac13330  0x330c  WaitCompletionPacket    0x1     -
4       System  0x840e96dc4080  0x3310  Process 0x0     svchost.exe Pid 3352
4       System  0x840e997aab60  0x3314  Event   0x1f0003        -
4       System  0x840e92e48080  0x331c  Process 0x0     svchost.exe Pid 1524
4       System  0x840e97468b60  0x3320  Event   0x1f0003        -
4       System  0x840e999de660  0x3324  Event   0x1f0003        -
4       System  0x840e9a94e9e0  0x3328  Event   0x1f0003        -
4       System  0x840e9c121080  0x332c  Process 0x102a  RuntimeBroker. Pid 10764
4       System  0x840e9c8bb670  0x3330  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.2.regtrans-ms
4       System  0x840e9764e840  0x3334  IoCompletion    0x1f0003        -
4       System  0x840e990a1d50  0x3338  WaitCompletionPacket    0x1     -
4       System  0x840e98bdf080  0x333c  Process 0x102a  SystemSettings Pid 10020
4       System  0x840e98eb3080  0x3340  Process 0x0     conhost.exe Pid 10944
4       System  0x840e96860080  0x3344  Process 0x0     svchost.exe Pid 2396
4       System  0x840e97d58e80  0x3348  IoCompletion    0x1f0003        -
4       System  0x840e98db8080  0x334c  Process 0x102a  MsSense.exe Pid 10224
4       System  0x840e96769240  0x3350  Process 0x0     svchost.exe Pid 2160
4       System  0x840e96cdd1c0  0x3358  Process 0x0     svchost.exe Pid 740
4       System  0x840e99c29ce0  0x335c  Event   0x1f0003        -
4       System  0x840e98db8080  0x3364  Process 0x0     MsSense.exe Pid 10224
4       System  0x840e96bc00c0  0x3368  Process 0x0     svchost.exe Pid 2848
4       System  0x840e99aaa940  0x336c  IoCompletion    0x1f0003        -
4       System  0x840e990999b0  0x3370  WaitCompletionPacket    0x1     -
4       System  0x840e99aa3dc0  0x3374  IoCompletion    0x1f0003        -
4       System  0x840e90de2080  0x3378  Process 0x0     svchost.exe Pid 1480
4       System  0x840e90c19340  0x337c  Process 0x0     svchost.exe Pid 3604
4       System  0x840e97add1c0  0x3380  Process 0x0     svchost.exe Pid 6020
4       System  0x840e9b996d50  0x3384  WaitCompletionPacket    0x1     -
4       System  0x840e974b7080  0x3388  Process 0x0     CSFalconContai Pid 5980
4       System  0x840e999e0a60  0x338c  Event   0x1f0003        -
4       System  0x840e99720ce0  0x3390  Event   0x1f0003        -
4       System  0x840e99c83080  0x3394  Process 0x102a  dwm.exe Pid 10628
4       System  0x840e97e8c080  0x3398  Process 0x102a  svchost.exe Pid 3864
4       System  0x840e99fd62e0  0x33a0  Event   0x1f0003        -
4       System  0x840e98079880  0x33a4  CoreMessaging   0xf0000 -
4       System  0x840e96df8c70  0x33a8  Desktop 0xf0003 Disconnect
4       System  0x840e9682f080  0x33b0  Process 0x102a  OneDrive.exe Pid 3320
4       System  0x840ea08f3080  0x33b4  Process 0x102a  svchost.exe Pid 12008
4       System  0x840e99513a60  0x33b8  ALPC Port       0x1f0001        -
4       System  0x840e9933eb60  0x33bc  Event   0x1f0003        -
4       System  0x840e9ba21460  0x33c0  WaitCompletionPacket    0x1     -
4       System  0x840e997196e0  0x33c4  Event   0x1f0003        -
4       System  0x840e9676e080  0x33c8  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e990dc5c0  0x33cc  TmRm    0x1f007f        -
4       System  0x840e98db8080  0x33d0  Process 0x102a  MsSense.exe Pid 10224
4       System  0x840e98a26e40  0x33d4  IoCompletion    0x1f0003        -
4       System  0x840e97dc72c0  0x33d8  Process 0x0     SearchIndexer. Pid 1852
4       System  0x840e96862080  0x33dc  Process 0x0     svchost.exe Pid 2376
4       System  0x840e98b29e20  0x33e0  WaitCompletionPacket    0x1     -
4       System  0x840e98c450c0  0x33e4  Process 0x0     dllhost.exe Pid 7800
4       System  0x840e9676e080  0x33e8  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9682f080  0x33ec  Process 0x102a  OneDrive.exe Pid 3320
4       System  0x840e994d1080  0x33f0  Process 0x0     LogonUI.exe Pid 10680
4       System  0x840e92d2e240  0x33f4  Process 0x0     svchost.exe Pid 1320
4       System  0x840e98187b70  0x33f8  WaitCompletionPacket    0x1     -
4       System  0x840e98bce2b0  0x33fc  TmEn    0xf001f -
4       System  0x840e98dc11c0  0x3404  Process 0x0     SecurityHealth Pid 8736
4       System  0x840e92e732c0  0x3408  Process 0x0     svchost.exe Pid 1580
4       System  0xb7081a44b5e0  0x340c  Key     0xf003f USER\S-1-12-1-419028606-1323772725-945580434-2117921032\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\WINLOGON
4       System  0x840e96e350c0  0x3410  Process 0x0     svchost.exe Pid 3480
4       System  0x840e96e20080  0x3414  Process 0x0     svchost.exe Pid 3488
4       System  0x840e966d9300  0x3418  Process 0x0     svchost.exe Pid 8912
4       System  0x840e9ca6b240  0x341c  IoCompletion    0x1f0003        -
4       System  0x840e92305080  0x3420  Process 0x0     winlogon.exe Pid 980
4       System  0x840e995cd0c0  0x3424  Process 0x1fffff        svchost.exe Pid 6356
4       System  0x840e978e24c0  0x3428  WaitCompletionPacket    0x1     -
4       System  0x840e98bd6240  0x342c  Process 0x0     RuntimeBroker. Pid 8120
4       System  0x840e97e11080  0x3430  Process 0x0     TextInputHost. Pid 1256
4       System  0x840e96e44300  0x3434  Process 0x0     MsMpEng.exe Pid 3540
4       System  0xb7080878d570  0x3438  Section 0x17    -
4       System  0x840e970ce700  0x3440  IoCompletion    0x1f0003        -
4       System  0xb707fbcba960  0x3444  SymbolicLink    0xf0001 Local
4       System  0x840e96e77080  0x3448  Process 0x0     svchost.exe Pid 3636
4       System  0x840e972491c0  0x344c  Process 0x0     svchost.exe Pid 4352
4       System  0x840e98f07080  0x3450  Process 0x0     msedge.exe Pid 9172
4       System  0x840e9969e080  0x3454  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e98dbf2c0  0x3458  Process 0x0     svchost.exe Pid 8688
4       System  0x840e9e99e080  0x345c  Process 0x1fffff        dllhost.exe Pid 7884
4       System  0x840e996cf080  0x3460  Process 0x0     svchost.exe Pid 9320
4       System  0x840e9b9afde0  0x3468  WaitCompletionPacket    0x1     -
4       System  0x840ea3cf4760  0x3470  Event   0x1f0003        -
4       System  0x840e995680c0  0x3474  Process 0x1fffff        SgrmBroker.exe Pid 9948
4       System  0x840e970f3080  0x3478  Process 0x0     svchost.exe Pid 3172
4       System  0x840e99840080  0x347c  Process 0x1fffff        dllhost.exe Pid 2492
4       System  0x840e9f2d8ae0  0x3480  Event   0x1f0003        -
4       System  0x840e9933af60  0x3484  Event   0x1f0003        -
4       System  0x840e98db8080  0x3488  Process 0x1028  MsSense.exe Pid 10224
4       System  0x840e98b3cd30  0x348c  WaitCompletionPacket    0x1     -
4       System  0x840e9f016ea0  0x3490  WaitCompletionPacket    0x1     -
4       System  0x840e92ee45b0  0x3494  WaitCompletionPacket    0x1     -
4       System  0x840e981a6700  0x34a0  WaitCompletionPacket    0x1     -
4       System  0x840e99448990  0x34a4  ALPC Port       0x1f0001        -
4       System  0x840e98b3c5e0  0x34ac  WaitCompletionPacket    0x1     -
4       System  0x840e97681600  0x34b0  IoCompletion    0x1f0003        -
4       System  0x840e9f39a800  0x34b4  IoCompletion    0x1f0003        -
4       System  0x840e995cd0c0  0x34b8  Process 0x102a  svchost.exe Pid 6356
4       System  0x840e966d9300  0x34bc  Process 0x102a  svchost.exe Pid 8912
4       System  0x840e9b1a1240  0x34c0  Process 0x1fffff        RuntimeBroker. Pid 12140
4       System  0x840e9ca424c0  0x34c4  IoCompletion    0x1f0003        -
4       System  0x840e995680c0  0x34c8  Process 0x0     SgrmBroker.exe Pid 9948
4       System  0x840e98e94460  0x34cc  Event   0x1f0003        -
4       System  0x840ea70570c0  0x34d0  Process 0x0     msedge.exe Pid 1336
4       System  0x840e995680c0  0x34d4  Process 0x102a  SgrmBroker.exe Pid 9948
4       System  0x840e995680c0  0x34d8  Process 0x102a  SgrmBroker.exe Pid 9948
4       System  0x840e91c3b300  0x34e0  Process 0x102a  svchost.exe Pid 8312
4       System  0x840e99dbfb30  0x34e4  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTSgrmEtwSession.etl
4       System  0x840e9969e080  0x34e8  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e995ea080  0x34f0  Process 0x0     OneDrive.exe Pid 11592
4       System  0x840e99c201e0  0x34f4  Event   0x1f0003        -
4       System  0x840e9b9b25b0  0x34f8  WaitCompletionPacket    0x1     -
4       System  0xb70823a39930  0x3500  SymbolicLink    0xf0001 Global
4       System  0x840e9bce9070  0x3504  RawInputManager 0x3     -
4       System  0x840e966d9300  0x3508  Process 0x102a  svchost.exe Pid 8912
4       System  0x840e9971fce0  0x3510  Event   0x1f0003        -
4       System  0x840e990a65a0  0x3514  WaitCompletionPacket    0x1     -
4       System  0x840e9d699990  0x3518  File    0x1     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-12-1-419028606-1323772725-945580434-2117921032\SystemAppData\Helium\Cache\b4d622c333fcfdd7.dat
4       System  0xb70812b05280  0x351c  Directory       0xf     S-1-15-2-1910091885-1573563583-1104941280-2418270861-3411158377-2822700936-2990310272
4       System  0x840e99581080  0x3520  Process 0x102a  svchost.exe Pid 15508
4       System  0x840e9b98acc0  0x3524  WaitCompletionPacket    0x1     -
4       System  0x840e9c8647e0  0x352c  Event   0x1f0003        -
4       System  0x840e9a61d440  0x3530  IoCompletion    0x1f0003        -
4       System  0x840e997450c0  0x3534  Process 0x1fffff        svchost.exe Pid 3052
4       System  0x840e9969e080  0x3538  Process 0x1fffff        svchost.exe Pid 4156
4       System  0x840e997450c0  0x353c  Process 0x0     svchost.exe Pid 3052
4       System  0x840e99f57980  0x3540  IoCompletion    0x1f0003        -
4       System  0x840e9969e080  0x3544  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e98184290  0x3548  WaitCompletionPacket    0x1     -
4       System  0x840e997450c0  0x354c  Process 0x1fffff        svchost.exe Pid 3052
4       System  0x840e97ea9080  0x3550  Process 0x1fffff        RuntimeBroker. Pid 1832
4       System  0x840e966d9300  0x3554  Process 0x0     svchost.exe Pid 8912
4       System  0x840e9a625400  0x3558  IoCompletion    0x1f0003        -
4       System  0x840e99086350  0x355c  WaitCompletionPacket    0x1     -
4       System  0x840e922da080  0x3564  Thread  0x1fffff        Tid 3748 Pid 4
4       System  0x840e9979cf60  0x356c  Event   0x1f0003        -
4       System  0x840e9a61e880  0x3570  IoCompletion    0x1f0003        -
4       System  0x840e997450c0  0x3574  Process 0x102a  svchost.exe Pid 3052
4       System  0x840e98b28eb0  0x3578  WaitCompletionPacket    0x1     -
4       System  0x840e9a625880  0x357c  IoCompletion    0x1f0003        -
4       System  0x840e99707160  0x3580  Event   0x1f0003        -
4       System  0x840e9969e080  0x3584  Process 0x0     svchost.exe Pid 4156
4       System  0xb70829130960  0x3588  Token   0xf01ff -
4       System  0x840e96f5a080  0x358c  Process 0x0     SenseNdr.exe Pid 15568
4       System  0x840e9830b530  0x3590  ALPC Port       0x1f0001        -
4       System  0x840e91d8afe0  0x3594  Event   0x1f0003        -
4       System  0x840e9ac0e940  0x3598  WaitCompletionPacket    0x1     -
4       System  0x840e98183320  0x359c  WaitCompletionPacket    0x1     -
4       System  0x840e99712d60  0x35a4  Event   0x1f0003        -
4       System  0x840e96a5d5d0  0x35a8  IoCompletionReserve     0xf0003 -
4       System  0x840e9970ee60  0x35ac  Event   0x1f0003        -
4       System  0x840e99f49440  0x35b0  IoCompletion    0x1f0003        -
4       System  0x840e90c049f0  0x35b4  TmRm    0x1f007f        -
4       System  0x840e966d9300  0x35b8  Process 0x102a  svchost.exe Pid 8912
4       System  0x840e9e3df5e0  0x35bc  Event   0x1f0003        -
4       System  0x840ea9ecef60  0x35c0  Event   0x1f0003        -
4       System  0x840e9f3d1040  0x35c4  Thread  0x1fffff        Tid 5156 Pid 4
4       System  0x840e96bfbae0  0x35cc  Event   0x1f0003        -
4       System  0x840e92ee3640  0x35d4  WaitCompletionPacket    0x1     -
4       System  0x840e90e10410  0x35d8  Session 0x0     Session2
4       System  0x840e990a3d00  0x35dc  WaitCompletionPacket    0x1     -
4       System  0x840e9907f810  0x35e0  WaitCompletionPacket    0x1     -
4       System  0x840e91c3b300  0x35e4  Process 0x102a  svchost.exe Pid 8312
4       System  0x840e970d1480  0x35ec  IoCompletion    0x1f0003        -
4       System  0x840e99a92080  0x35f0  Process 0x0     SecurityHealth Pid 12036
4       System  0x840ea08f3080  0x35f4  Process 0x102a  svchost.exe Pid 12008
4       System  0x840ea08f3080  0x35f8  Process 0x1fffff        svchost.exe Pid 12008
4       System  0x840ea5362d60  0x35fc  Event   0x1f0003        -
4       System  0x840e9e3d0ee0  0x3604  Event   0x1f0003        -
4       System  0x840e9807ab10  0x3608  CoreMessaging   0xf0000 -
4       System  0x840e99fd75e0  0x360c  Event   0x1f0003        -
4       System  0x840e979ea310  0x3610  ALPC Port       0x1f0001        -
4       System  0x840e9ac0f3d0  0x3614  WaitCompletionPacket    0x1     -
4       System  0x840e9907e8a0  0x361c  WaitCompletionPacket    0x1     -
4       System  0x840e99fc7de0  0x3620  Event   0x1f0003        -
4       System  0x840e96818d70  0x3624  WaitCompletionPacket    0x1     -
4       System  0x840e90e10410  0x3628  Session 0x0     Session2
4       System  0x840e99847070  0x362c  RawInputManager 0x3     -
4       System  0x840e9f388f40  0x3630  IoCompletion    0x1f0003        -
4       System  0x840e981886d0  0x3638  WaitCompletionPacket    0x1     -
4       System  0x840e9909f0a0  0x363c  WaitCompletionPacket    0x1     -
4       System  0x840e99fd6960  0x3640  Event   0x1f0003        -
4       System  0x840e97ea9080  0x3644  Process 0x0     RuntimeBroker. Pid 1832
4       System  0x840e9c852460  0x3648  Event   0x1f0003        -
4       System  0x840e9ac30a70  0x364c  WaitCompletionPacket    0x1     -
4       System  0x840e98a0fb40  0x3650  IoCompletion    0x1f0003        -
4       System  0x840e98a27700  0x3654  IoCompletion    0x1f0003        -
4       System  0x840e970d0c80  0x3658  IoCompletion    0x1f0003        -
4       System  0xb70823891230  0x365c  Section 0x5     -
4       System  0x840e97c46c60  0x3660  Event   0x1f0003        -
4       System  0xb707fbcbacd0  0x3664  SymbolicLink    0xf0001 Global
4       System  0x840e9c9a9800  0x3668  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.UI.Xaml.2.0_2.1810.18004.0_x64__8wekyb3d8bbwe\ActivationStore.dat.LOG2
4       System  0x840e91da7360  0x366c  Event   0x1f0003        -
4       System  0x840e97211cc0  0x3670  IoCompletionReserve     0xf0003 -
4       System  0x840e91b3c0e0  0x3674  Timer   0x1f0003        -
4       System  0x840e98a0a300  0x3678  IoCompletion    0x1f0003        -
4       System  0x840e9a1476a0  0x367c  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\User.dat.LOG2
4       System  0x840e9a625dc0  0x3680  IoCompletion    0x1f0003        -
4       System  0x840e990a11f0  0x3684  WaitCompletionPacket    0x1     -
4       System  0xb707fbcb43a0  0x3688  SymbolicLink    0xf0001 Local
4       System  0x840e99c2c5e0  0x368c  Event   0x1f0003        -
4       System  0x840e97810a60  0x3694  Event   0x1f0003        -
4       System  0x840e96d56340  0x3698  Process 0x1fffff        SenseTVM.exe Pid 1552
4       System  0x840e9c9a87c0  0x369c  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.UI.Xaml.2.0_2.1810.18004.0_x64__8wekyb3d8bbwe\ActivationStore.dat.LOG1
4       System  0x840e99f6a940  0x36ac  IoCompletion    0x1f0003        -
4       System  0x840e9902a380  0x36b4  IoCompletion    0x1f0003        -
4       System  0x840e9933ea60  0x36b8  Event   0x1f0003        -
4       System  0x840e96956280  0x36bc  IoCompletion    0x1f0003        -
4       System  0x840e99f6a4c0  0x36c0  IoCompletion    0x1f0003        -
4       System  0x840e967df060  0x36c8  Event   0x1f0003        -
4       System  0x840e98192470  0x36cc  WaitCompletionPacket    0x1     -
4       System  0x840e9ac109c0  0x36d0  WaitCompletionPacket    0x1     -
4       System  0x840ea3cf7260  0x36d4  Event   0x1f0003        -
4       System  0x840e9db77080  0x36d8  Process 0x102a  taskhostw.exe Pid 15784
4       System  0x840e9a944de0  0x36e0  Event   0x1f0003        -
4       System  0x840e995ea080  0x36e4  Process 0x1fffff        OneDrive.exe Pid 11592
4       System  0x840e99fd2760  0x36e8  Event   0x1f0003        -
4       System  0x840e990a52f0  0x36f0  WaitCompletionPacket    0x1     -
4       System  0x840e9a615f00  0x36f4  IoCompletion    0x1f0003        -
4       System  0x840e99658510  0x36f8  ALPC Port       0x1f0001        -
4       System  0x840e9ba042d0  0x36fc  WaitCompletionPacket    0x1     -
4       System  0x840e98d887c0  0x3700  ALPC Port       0x1f0001        -
4       System  0x840e9dad0980  0x3704  IoCompletion    0x1f0003        -
4       System  0x840ea534d560  0x3708  Event   0x1f0003        -
4       System  0x840ea085e440  0x370c  IoCompletion    0x1f0003        -
4       System  0xb7081cbe29c0  0x3710  SymbolicLink    0xf0001 Global
4       System  0x840e985d8080  0x3714  Process 0x1fffff        ShellExperienc Pid 7588
4       System  0x840e99f6e680  0x3718  IoCompletion    0x1f0003        -
4       System  0x840e9ac0f980  0x371c  WaitCompletionPacket    0x1     -
4       System  0x840e973867c0  0x3724  IoCompletion    0x1f0003        -
4       System  0x840e99335b60  0x3728  Event   0x1f0003        -
4       System  0x840e98d04b80  0x372c  IoCompletion    0x1f0003        -
4       System  0x840e99093350  0x373c  WaitCompletionPacket    0x1     -
4       System  0x840e9c5a8d60  0x3740  Event   0x1f0003        -
4       System  0x840e98b50ef0  0x3744  WaitCompletionPacket    0x1     -
4       System  0x840e9b9ba200  0x3748  WaitCompletionPacket    0x1     -
4       System  0x840e97d72900  0x374c  IoCompletion    0x1f0003        -
4       System  0x840e9a629080  0x3750  IoCompletion    0x1f0003        -
4       System  0x840e9c52f820  0x3754  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0x840e97d72800  0x375c  IoCompletion    0x1f0003        -
4       System  0x840e99a92080  0x3760  Process 0x102a  SecurityHealth Pid 12036
4       System  0xb7081a5c8d80  0x3768  SymbolicLink    0xf0001 Session
4       System  0x840e90e10410  0x3770  Session 0x0     Session2
4       System  0x840e9cf27700  0x3774  IoCompletion    0x1f0003        -
4       System  0x840ea5357060  0x377c  Event   0x1f0003        -
4       System  0x840e9cf159c0  0x3784  IoCompletion    0x1f0003        -
4       System  0x840e98e90660  0x3788  Event   0x1f0003        -
4       System  0x840e98304a80  0x378c  ALPC Port       0x1f0001        -
4       System  0x840e9842baf0  0x3790  IoCompletionReserve     0xf0003 -
4       System  0x840e99c83080  0x3798  Process 0x1fffff        dwm.exe Pid 10628
4       System  0x840e974b2080  0x379c  Process 0x102a  UserOOBEBroker Pid 9620
4       System  0x840e99716f60  0x37a0  Event   0x1f0003        -
4       System  0x840e980788c0  0x37a4  CoreMessaging   0xf0000 -
4       System  0x840e9f28b080  0x37a8  Process 0x1fffff        WmiPrvSE.exe Pid 14028
4       System  0x840e99f67740  0x37ac  IoCompletion    0x1f0003        -
4       System  0x840e99c83080  0x37b0  Process 0x102a  dwm.exe Pid 10628
4       System  0x840e999d9560  0x37b4  Event   0x1f0003        -
4       System  0x840e98c48080  0x37b8  Process 0x102a  ApplicationFra Pid 4836
4       System  0x840e9a628c00  0x37bc  IoCompletion    0x1f0003        -
4       System  0x840e98bdf080  0x37c0  Process 0x0     SystemSettings Pid 10020
4       System  0x840e9c0cb080  0x37c4  Process 0x0     splunk-winevtl Pid 11256
4       System  0x840e999f5f60  0x37c8  Event   0x1f0003        -
4       System  0x840e98b51980  0x37d0  WaitCompletionPacket    0x1     -
4       System  0x840e9682f080  0x37d4  Process 0x102a  OneDrive.exe Pid 3320
4       System  0x840ea9ed28e0  0x37d8  Event   0x1f0003        -
4       System  0xb70809062060  0x37dc  Token   0xf01ff -
4       System  0x840e99f76480  0x37e0  IoCompletion    0x1f0003        -
4       System  0x840e9ba171e0  0x37e4  WaitCompletionPacket    0x1     -
4       System  0x840e990b28a0  0x37e8  WaitCompletionPacket    0x1     -
4       System  0x840e99f707c0  0x37ec  IoCompletion    0x1f0003        -
4       System  0x840e997162e0  0x37f0  Event   0x1f0003        -
4       System  0x840e997ade60  0x37f4  Event   0x1f0003        -
4       System  0x840e985cb7e0  0x37f8  FilterCommunicationPort 0x1f0001        -
4       System  0x840e9952c080  0x37fc  Process 0x102a  csrss.exe Pid 10732
4       System  0x840e990a5970  0x3804  WaitCompletionPacket    0x1     -
4       System  0x840e9c8964e0  0x3808  Event   0x1f0003        -
4       System  0x840e9ba2f160  0x3814  WaitCompletionPacket    0x1     -
4       System  0x840e90ee7240  0x3818  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.0.regtrans-ms
4       System  0x840e99fc6560  0x3820  Event   0x1f0003        -
4       System  0x840e9a94c160  0x3828  Event   0x1f0003        -
4       System  0x840ea3cdf7e0  0x382c  Event   0x1f0003        -
4       System  0x840e96956440  0x3830  IoCompletion    0x1f0003        -
4       System  0x840e96d56340  0x3834  Process 0x102a  SenseTVM.exe Pid 1552
4       System  0x840e98a2f280  0x3838  IoCompletion    0x1f0003        -
4       System  0x840e9c441080  0x3844  Process 0x102a  SearchApp.exe Pid 9092
4       System  0x840e99e240c0  0x3848  Process 0x102a  ScreenConnect. Pid 6988
4       System  0x840e97e8c080  0x384c  Process 0x0     svchost.exe Pid 3864
4       System  0x840e9ba24520  0x3850  WaitCompletionPacket    0x1     -
4       System  0x840e99c83080  0x3858  Process 0x40    dwm.exe Pid 10628
4       System  0x840e9c8bd2f0  0x3860  File    0x13019f        \Device\clfs\Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee29-852e-11ef-ba7c-bc2411845bc2}.TxR
4       System  0x840e990ab470  0x3864  WaitCompletionPacket    0x1     -
4       System  0x840e90b0d080  0x3868  Process 0x102a  svchost.exe Pid 9452
4       System  0x840e9a943460  0x386c  Event   0x1f0003        -
4       System  0x840e9908c330  0x3870  WaitCompletionPacket    0x1     -
4       System  0x840e9b5df1b0  0x3874  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat.LOG1
4       System  0x840e9ac1af80  0x3878  WaitCompletionPacket    0x1     -
4       System  0x840e9b9afb70  0x387c  WaitCompletionPacket    0x1     -
4       System  0x840e9c8a4060  0x3880  Event   0x1f0003        -
4       System  0x840e97e9e940  0x3888  CoreMessaging   0xf0000 -
4       System  0x840e9b2df080  0x388c  Process 0x102a  MoUsoCoreWorke Pid 10144
4       System  0x840e99f61640  0x3890  IoCompletion    0x1f0003        -
4       System  0x840e97e9dfb0  0x3898  CoreMessaging   0xf0000 -
4       System  0x840e990a6cf0  0x38a0  WaitCompletionPacket    0x1     -
4       System  0x840e98a06bc0  0x38a4  IoCompletion    0x1f0003        -
4       System  0x840e9ac098d0  0x38a8  WaitCompletionPacket    0x1     -
4       System  0x840ea0869180  0x38ac  IoCompletion    0x1f0003        -
4       System  0x840e99f776c0  0x38b0  IoCompletion    0x1f0003        -
4       System  0x840ea534e4e0  0x38b4  Event   0x1f0003        -
4       System  0x840e98df3080  0x38b8  Process 0x102a  ShellExperienc Pid 704
4       System  0xb70801532630  0x38bc  Token   0xf01ff -
4       System  0x840e91c3b300  0x38c0  Process 0x0     svchost.exe Pid 8312
4       System  0x840e99f799c0  0x38c4  IoCompletion    0x1f0003        -
4       System  0x840e9b372080  0x38cc  Process 0x0     taskhostw.exe Pid 10476
4       System  0x840e9cf2fdc0  0x38d0  IoCompletion    0x1f0003        -
4       System  0x840e994d1080  0x38d4  Process 0x102a  LogonUI.exe Pid 10680
4       System  0x840e9ea14080  0x38d8  Process 0x1fffff        ScreenConnect. Pid 1772
4       System  0x840e9b739080  0x38dc  Process 0x102a  Microsoft.Phot Pid 11148
4       System  0x840e9c713560  0x38e0  Event   0x1f0003        -
4       System  0x840e9c7120e0  0x38e4  Event   0x1f0003        -
4       System  0xb70829d62a70  0x38e8  Token   0xf01ff -
4       System  0x840e9c5277c0  0x38ec  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000001.regtrans-ms
4       System  0x840e9e99e080  0x38f0  Process 0x102a  dllhost.exe Pid 7884
4       System  0x840e9a6294c0  0x38f4  IoCompletion    0x1f0003        -
4       System  0x840e9a617a40  0x38f8  IoCompletion    0x1f0003        -
4       System  0x840e9a61c800  0x38fc  IoCompletion    0x1f0003        -
4       System  0x840e9f3460c0  0x3900  Process 0x0     svchost.exe Pid 11596
4       System  0x840e98df3080  0x3904  Process 0x40    ShellExperienc Pid 704
4       System  0x840e9daca840  0x390c  IoCompletion    0x1f0003        -
4       System  0x840e976767c0  0x3910  IoCompletion    0x1f0003        -
4       System  0x840e9c47c940  0x3914  IoCompletion    0x1f0003        -
4       System  0x840e91c3b300  0x3918  Process 0x1fffff        svchost.exe Pid 8312
4       System  0x840e9ac23590  0x391c  WaitCompletionPacket    0x1     -
4       System  0x840e98e83ce0  0x3920  Event   0x1f0003        -
4       System  0xb7081792d770  0x3924  Section 0x5     -
4       System  0x840e99fd7d60  0x3928  Event   0x1f0003        -
4       System  0x840ea08666c0  0x392c  IoCompletion    0x1f0003        -
4       System  0x840e9f327080  0x3930  Process 0x0     ApplicationFra Pid 7492
4       System  0x840e97e9e8b0  0x3934  CoreMessaging   0xf0000 -
4       System  0x840e98079520  0x393c  CoreMessaging   0xf0000 -
4       System  0x840e9c707b60  0x3940  Event   0x1f0003        -
4       System  0x840e9856b2c0  0x3944  Process 0x102a  svchost.exe Pid 7780
4       System  0x840ea085eec0  0x3948  IoCompletion    0x1f0003        -
4       System  0x840e99bcb080  0x394c  Process 0x40    explorer.exe Pid 9852
4       System  0x840e99e240c0  0x3950  Process 0x102a  ScreenConnect. Pid 6988
4       System  0x840e99ac04c0  0x3954  IoCompletion    0x1f0003        -
4       System  0x840e9a94db60  0x3958  Event   0x1f0003        -
4       System  0x840e98b24800  0x3960  WaitCompletionPacket    0x1     -
4       System  0x840e9708cdf0  0x3964  ALPC Port       0x1f0001        -
4       System  0x840e99ae0900  0x3968  IoCompletion    0x1f0003        -
4       System  0x840e9ac2e2a0  0x396c  WaitCompletionPacket    0x1     -
4       System  0x840e9c456140  0x3970  IoCompletion    0x1f0003        -
4       System  0x840e98a21180  0x3974  IoCompletion    0x1f0003        -
4       System  0x840e98b3b4d0  0x3978  WaitCompletionPacket    0x1     -
4       System  0x840e91bc3770  0x3980  WaitCompletionPacket    0x1     -
4       System  0x840e999ede60  0x3988  Event   0x1f0003        -
4       System  0x840e98f962b0  0x398c  IoCompletionReserve     0xf0003 -
4       System  0x840e99fd3fe0  0x3990  Event   0x1f0003        -
4       System  0xb7081a787190  0x3994  Section 0x5     -
4       System  0x840e9b73a080  0x3998  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e99840080  0x39a0  Process 0x102a  dllhost.exe Pid 2492
4       System  0x840e998450c0  0x39a4  Process 0x40    SearchApp.exe Pid 4236
4       System  0x840e9b6cd9b0  0x39a8  IoCompletionReserve     0xf0003 -
4       System  0x840e97360f80  0x39ac  IoCompletion    0x1f0003        -
4       System  0x840e99fd9560  0x39b0  Event   0x1f0003        -
4       System  0x840e970d2c00  0x39b4  IoCompletion    0x1f0003        -
4       System  0x840e99e51e70  0x39bc  IoCompletionReserve     0xf0003 -
4       System  0x840e9b805270  0x39c8  IoCompletionReserve     0xf0003 -
4       System  0x840e99738af0  0x39cc  ALPC Port       0x1f0001        -
4       System  0x840e9dabc940  0x39d0  IoCompletion    0x1f0003        -
4       System  0x840e98b3bcf0  0x39d4  WaitCompletionPacket    0x1     -
4       System  0xb7081792db30  0x39e0  Section 0x17    -
4       System  0x840e99fca660  0x39e4  Event   0x1f0003        -
4       System  0x840e9ba16410  0x39e8  WaitCompletionPacket    0x1     -
4       System  0x840e98b15080  0x39ec  Process 0x0     fontdrvhost.ex Pid 10704
4       System  0x840e993449e0  0x39f0  Event   0x1f0003        -
4       System  0x840e9907b7e0  0x39f4  WaitCompletionPacket    0x1     -
4       System  0x840e9a623900  0x39f8  IoCompletion    0x1f0003        -
4       System  0x840e998450c0  0x39fc  Process 0x102a  SearchApp.exe Pid 4236
4       System  0x840e9b98cba0  0x3a08  WaitCompletionPacket    0x1     -
4       System  0x840e97d56900  0x3a0c  IoCompletion    0x1f0003        -
4       System  0x840e9c5b50e0  0x3a14  Event   0x1f0003        -
4       System  0x840e9711ee60  0x3a18  Event   0x1f0003        -
4       System  0x840e9ba12310  0x3a1c  WaitCompletionPacket    0x1     -
4       System  0x840e9ba0fc10  0x3a24  WaitCompletionPacket    0x1     -
4       System  0x840e99acef40  0x3a28  IoCompletion    0x1f0003        -
4       System  0x840e9c35d460  0x3a30  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee2a-852e-11ef-ba7c-bc2411845bc2}.TMContainer00000000000000000001.regtrans-ms
4       System  0x840e9ac2bba0  0x3a34  WaitCompletionPacket    0x1     -
4       System  0x840e9ac0f8b0  0x3a38  WaitCompletionPacket    0x1     -
4       System  0x840e99fd75e0  0x3a3c  Event   0x1f0003        -
4       System  0x840e9a20b650  0x3a40  File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-2246530975-808720366-1776470054-230329187-4153223113-3550430174-4193313734
4       System  0xb7081c69be60  0x3a44  Directory       0xf     S-1-15-2-157618707-224758843-3162413466-2249835351-834486866-1672254014-2610752905
4       System  0x840e9c70b760  0x3a48  Event   0x1f0003        -
4       System  0xb7081b046a00  0x3a50  Directory       0xf     RPC Control
4       System  0x840e99840080  0x3a54  Process 0x102a  dllhost.exe Pid 2492
4       System  0x840e99c83080  0x3a58  Process 0x40    dwm.exe Pid 10628
4       System  0x840e9968d080  0x3a5c  Process 0x102a  splunkd.exe Pid 13620
4       System  0xb7081a5c8cd0  0x3a64  SymbolicLink    0xf0001 Local
4       System  0x840e9cf0c280  0x3a68  IoCompletion    0x1f0003        -
4       System  0x840e99037c80  0x3a6c  IoCompletion    0x1f0003        -
4       System  0x840e98c160e0  0x3a70  Event   0x1f0003        -
4       System  0x840e9676e080  0x3a74  Process 0x102a  svchost.exe Pid 2168
4       System  0xb7082389a650  0x3a78  Section 0x17    -
4       System  0x840e9cf2e1c0  0x3a7c  IoCompletion    0x1f0003        -
4       System  0x840e9c895e60  0x3a80  Event   0x1f0003        -
4       System  0x840e99c83080  0x3a88  Process 0x102a  dwm.exe Pid 10628
4       System  0x840e91a11080  0x3a8c  Process 0x102a  spoolsv.exe Pid 14536
4       System  0x840e9c458340  0x3a94  IoCompletion    0x1f0003        -
4       System  0x840e99da3080  0x3a98  Process 0x0     RuntimeBroker. Pid 4464
4       System  0x840e9a14a710  0x3aa4  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.ShellExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG2
4       System  0x840e9b99a7d0  0x3aa8  WaitCompletionPacket    0x1     -
4       System  0x840ea3cf83e0  0x3aac  Event   0x1f0003        -
4       System  0x840e9c896160  0x3ab0  Event   0x1f0003        -
4       System  0x840e96696790  0x3abc  IoCompletionReserve     0xf0003 -
4       System  0x840e9c528ce0  0x3ac0  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM.blf
4       System  0x840e91e919b0  0x3acc  IoCompletionReserve     0xf0003 -
4       System  0x840e99fd5560  0x3ad0  Event   0x1f0003        -
4       System  0x840e9b9a3bb0  0x3ad4  WaitCompletionPacket    0x1     -
4       System  0xb7082389ab50  0x3ad8  Section 0x17    -
4       System  0x840e9a60cdc0  0x3adc  IoCompletion    0x1f0003        -
4       System  0x840e98fe4b60  0x3ae0  Event   0x1f0003        -
4       System  0x840e92e48080  0x3af0  Process 0x1028  svchost.exe Pid 1524
4       System  0xb708109dc7b0  0x3af8  SymbolicLink    0xf0001 Session
4       System  0x840e98b57a30  0x3afc  WaitCompletionPacket    0x1     -
4       System  0x840e97d59900  0x3b00  IoCompletion    0x1f0003        -
4       System  0x840e9934a860  0x3b08  Event   0x1f0003        -
4       System  0x840e90b0d080  0x3b0c  Process 0x1fffff        svchost.exe Pid 9452
4       System  0x840e9a630e80  0x3b1c  IoCompletion    0x1f0003        -
4       System  0x840e9b9a2c40  0x3b24  WaitCompletionPacket    0x1     -
4       System  0x840e99451a80  0x3b2c  ALPC Port       0x1f0001        -
4       System  0x840e9f7bf880  0x3b30  IoCompletion    0x1f0003        -
4       System  0x840e97e8c080  0x3b34  Process 0x102a  svchost.exe Pid 3864
4       System  0x840e91ee7080  0x3b38  Process 0x1fffff        Sysmon64.exe Pid 1540
4       System  0x840e9a948e60  0x3b3c  Event   0x1f0003        -
4       System  0x840e96817850  0x3b40  WaitCompletionPacket    0x1     -
4       System  0x840e9f327080  0x3b44  Process 0x102a  ApplicationFra Pid 7492
4       System  0xb7081f175120  0x3b48  Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\PEERDIST\SERVICE
4       System  0x840e9f2d7c60  0x3b4c  Event   0x1f0003        -
4       System  0x840e9c856a60  0x3b50  Event   0x1f0003        -
4       System  0x840e9f7ac200  0x3b54  IoCompletion    0x1f0003        -
4       System  0x840e92f5a760  0x3b58  Event   0x1f0003        -
4       System  0x840e9daf5100  0x3b5c  IoCompletion    0x1f0003        -
4       System  0x840e98078710  0x3b60  CoreMessaging   0xf0000 -
4       System  0x840e99fdb6e0  0x3b64  Event   0x1f0003        -
4       System  0x840e99840080  0x3b68  Process 0x102a  dllhost.exe Pid 2492
4       System  0x840ea9ecf0e0  0x3b6c  Event   0x1f0003        -
4       System  0x840e9709ba00  0x3b70  Timer   0x1f0003        -
4       System  0x840e9d8809e0  0x3b74  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.ShellExperienceHost_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e9cb821c0  0x3b7c  IoCompletion    0x1f0003        -
4       System  0x840e97fc6ee0  0x3b80  Event   0x1f0003        -
4       System  0x840e999e53e0  0x3b84  Event   0x1f0003        -
4       System  0xb70805be7300  0x3b88  Token   0xf01ff -
4       System  0x840e9f791a80  0x3b90  IoCompletion    0x1f0003        -
4       System  0x840e9affc6d0  0x3b98  File    0x1f0006        \Device\NamedPipe\Sessions\0\AppContainerNamedObjects\S-1-15-2-157618707-224758843-3162413466-2249835351-834486866-1672254014-2610752905
4       System  0x840e9ca34ac0  0x3b9c  IoCompletion    0x1f0003        -
4       System  0x840e9ca782e0  0x3bac  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-155514346-2573954481-755741238-1654018636-1233331829-3075935687-2861478708
4       System  0x840e9ac36a50  0x3bb4  WaitCompletionPacket    0x1     -
4       System  0x840e9817f150  0x3bb8  WaitCompletionPacket    0x1     -
4       System  0x840e96d4c420  0x3bbc  Timer   0x1f0003        -
4       System  0x840e985d8080  0x3bc0  Process 0x102a  ShellExperienc Pid 7588
4       System  0x840e9cb70f00  0x3bc4  IoCompletion    0x1f0003        -
4       System  0x840e96d56340  0x3bc8  Process 0x102a  SenseTVM.exe Pid 1552
4       System  0x840ea5355fe0  0x3bcc  Event   0x1f0003        -
4       System  0x840e99c191e0  0x3bd0  Event   0x1f0003        -
4       System  0x840e99fd7a60  0x3bd4  Event   0x1f0003        -
4       System  0x840e9ea14080  0x3bd8  Process 0x40    ScreenConnect. Pid 1772
4       System  0x840e9901fc80  0x3bdc  IoCompletion    0x1f0003        -
4       System  0x840e98fd0460  0x3be0  Event   0x1f0003        -
4       System  0x840ea5d2f080  0x3be8  Process 0x1fffff        svchost.exe Pid 9444
4       System  0x840e98b3a700  0x3bec  WaitCompletionPacket    0x1     -
4       System  0x840e9b283d10  0x3bf0  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.ShellExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840ea53557e0  0x3bf4  Event   0x1f0003        -
4       System  0x840e98eb3080  0x3c04  Process 0x1fffff        conhost.exe Pid 10944
4       System  0x840e9c360d40  0x3c0c  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee2a-852e-11ef-ba7c-bc2411845bc2}.TMContainer00000000000000000002.regtrans-ms
4       System  0x840e9ba1d430  0x3c10  WaitCompletionPacket    0x1     -
4       System  0x840e90eb2d30  0x3c14  Event   0x1f0003        EventShutDownCSRSS
4       System  0x840e99fd83e0  0x3c18  Event   0x1f0003        -
4       System  0x840e97e59080  0x3c20  Process 0x0     conhost.exe Pid 12392
4       System  0x840e98c26de0  0x3c24  Event   0x1f0003        -
4       System  0x840e996cf080  0x3c2c  Process 0x102a  svchost.exe Pid 9320
4       System  0x840e9c441080  0x3c30  Process 0x102a  SearchApp.exe Pid 9092
4       System  0x840e98b501f0  0x3c38  WaitCompletionPacket    0x1     -
4       System  0x840e9dae2880  0x3c3c  IoCompletion    0x1f0003        -
4       System  0x840e97ea9080  0x3c40  Process 0x102a  RuntimeBroker. Pid 1832
4       System  0x840e994d1080  0x3c44  Process 0x40    LogonUI.exe Pid 10680
4       System  0x840e9920f230  0x3c50  IoCompletionReserve     0xf0003 -
4       System  0xb70804de1890  0x3c64  Token   0xf01ff -
4       System  0x840e98b38270  0x3c6c  WaitCompletionPacket    0x1     -
4       System  0x840e9a620100  0x3c70  IoCompletion    0x1f0003        -
4       System  0x840e98c450c0  0x3c74  Process 0x1fffff        dllhost.exe Pid 7800
4       System  0x840e9b9951b0  0x3c78  WaitCompletionPacket    0x1     -
4       System  0x840e99c2d960  0x3c7c  Event   0x1f0003        -
4       System  0x840e9c00de60  0x3c80  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.Photos_8wekyb3d8bbwe\Settings\settings.dat
4       System  0x840e9df4b8e0  0x3c84  Event   0x1f0003        -
4       System  0x840e9a630a00  0x3c90  IoCompletion    0x1f0003        -
4       System  0x840e9986c790  0x3c94  ALPC Port       0x1f0001        -
4       System  0x840e99581080  0x3c98  Process 0x102a  svchost.exe Pid 15508
4       System  0x840e98b3e320  0x3c9c  WaitCompletionPacket    0x1     -
4       System  0x840e9ba14bb0  0x3ca0  WaitCompletionPacket    0x1     -
4       System  0x840e9ac0f570  0x3ca4  WaitCompletionPacket    0x1     -
4       System  0x840e9dac0740  0x3ca8  IoCompletion    0x1f0003        -
4       System  0x840e98ea1360  0x3cb0  Event   0x1f0003        -
4       System  0x840e990aadf0  0x3cb4  WaitCompletionPacket    0x1     -
4       System  0x840e9f2d8be0  0x3cb8  Event   0x1f0003        -
4       System  0x840ea70570c0  0x3cbc  Process 0x1fffff        msedge.exe Pid 1336
4       System  0x840e9c895860  0x3cc0  Event   0x1f0003        -
4       System  0x840e90e10410  0x3cc4  Session 0x0     Session2
4       System  0xb70821bab350  0x3cc8  SymbolicLink    0xf0001 Session
4       System  0x840e992c7f00  0x3ccc  IoCompletion    0x1f0003        -
4       System  0x840e97ea9080  0x3cd4  Process 0x102a  RuntimeBroker. Pid 1832
4       System  0x840e9c236080  0x3cd8  Process 0x102a  dllhost.exe Pid 5164
4       System  0xb70814e70a50  0x3cdc  Token   0xf01ff -
4       System  0x840e978ed080  0x3ce0  Process 0x0     cmd.exe Pid 13752
4       System  0x840e9c5b9d60  0x3ce4  Event   0x1f0003        -
4       System  0x840e993240c0  0x3ce8  Process 0x1fffff        sihost.exe Pid 10384
4       System  0x840e9b281950  0x3cec  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.ShellExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e9bb45f70  0x3cf0  IoCompletionReserve     0xf0003 -
4       System  0x840e9cb8d0c0  0x3cf4  Process 0x0     msedge.exe Pid 5148
4       System  0x840e9ac0ebb0  0x3cf8  WaitCompletionPacket    0x1     -
4       System  0x840e92f35f80  0x3cfc  IoCompletion    0x1f0003        -
4       System  0x840e99aa66c0  0x3d00  IoCompletion    0x1f0003        -
4       System  0x840e98fd59e0  0x3d04  Event   0x1f0003        -
4       System  0x840e9c52a3a0  0x3d08  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\ntuser.dat.LOG2
4       System  0x840e91e8f670  0x3d0c  IoCompletionReserve     0xf0003 -
4       System  0x840e9a955c60  0x3d10  Event   0x1f0003        -
4       System  0x840e99bcb080  0x3d14  Process 0x102a  explorer.exe Pid 9852
4       System  0x840e99fd7a60  0x3d18  Event   0x1f0003        -
4       System  0xb70804f53060  0x3d1c  Token   0xf01ff -
4       System  0x840e9c121080  0x3d20  Process 0x0     RuntimeBroker. Pid 10764
4       System  0x840e9dadd540  0x3d24  IoCompletion    0x1f0003        -
4       System  0x840e9e902340  0x3d28  Process 0x0     msedge.exe Pid 5808
4       System  0x840eaa187a60  0x3d30  WaitCompletionPacket    0x1     -
4       System  0x840e9f78c740  0x3d34  IoCompletion    0x1f0003        -
4       System  0x840e90e10410  0x3d38  Session 0x0     Session2
4       System  0x840e9968d080  0x3d3c  Process 0x102a  splunkd.exe Pid 13620
4       System  0x840e97d6f640  0x3d40  IoCompletion    0x1f0003        -
4       System  0x840e9c52c420  0x3d44  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\ntuser.dat.LOG1
4       System  0x840e99fd7ce0  0x3d48  Event   0x1f0003        -
4       System  0x840e99adedc0  0x3d4c  IoCompletion    0x1f0003        -
4       System  0x840e9c441080  0x3d50  Process 0x0     SearchApp.exe Pid 9092
4       System  0x840e99acf1c0  0x3d54  IoCompletion    0x1f0003        -
4       System  0x840e99c203e0  0x3d58  Event   0x1f0003        -
4       System  0x840e99ad61c0  0x3d5c  IoCompletion    0x1f0003        -
4       System  0x840e97363600  0x3d68  IoCompletion    0x1f0003        -
4       System  0x840e994f6d20  0x3d6c  ALPC Port       0x1f0001        -
4       System  0x840e9f3a4c80  0x3d70  IoCompletion    0x1f0003        -
4       System  0x840e9b8fdd90  0x3d78  Timer   0x2     -
4       System  0x840e98a22bc0  0x3d7c  IoCompletion    0x1f0003        -
4       System  0xb708106aa8c0  0x3d80  Directory       0x4     WindowStations
4       System  0x840e91f7dd50  0x3d84  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTSenseNdrEtw.etl
4       System  0x840e9dad6700  0x3d8c  IoCompletion    0x1f0003        -
4       System  0x840e9ac20260  0x3d94  WaitCompletionPacket    0x1     -
4       System  0x840e9817f220  0x3d98  WaitCompletionPacket    0x1     -
4       System  0x840e9b73a080  0x3d9c  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e9b8f8780  0x3da0  Timer   0x2     -
4       System  0x840e9ac43090  0x3da4  WaitCompletionPacket    0x1     -
4       System  0xb70811872640  0x3dac  SymbolicLink    0xf0001 Local
4       System  0x840e98a37bc0  0x3db0  IoCompletion    0x1f0003        -
4       System  0x840e92e34d80  0x3db4  IoCompletion    0x1f0003        -
4       System  0xb708106ad5c0  0x3dbc  Directory       0xf     RPC Control
4       System  0x840e97e9e820  0x3dc4  CoreMessaging   0xf0000 -
4       System  0x840e9a637240  0x3dc8  IoCompletion    0x1f0003        -
4       System  0x840e9736d800  0x3dd0  IoCompletion    0x1f0003        -
4       System  0x840e99c19160  0x3dd4  Event   0x1f0003        -
4       System  0x840e9ac0deb0  0x3dd8  WaitCompletionPacket    0x1     -
4       System  0x840e99049c80  0x3de0  IoCompletion    0x1f0003        -
4       System  0x840e9da14340  0x3de4  Process 0x1fffff        TextInputHost. Pid 12740
4       System  0x840e9b995d10  0x3dec  WaitCompletionPacket    0x1     -
4       System  0x840e997b5ce0  0x3df0  Event   0x1f0003        -
4       System  0x840e9e920080  0x3df4  Process 0x0     FileCoAuth.exe Pid 7516
4       System  0x840e9ca5c8c0  0x3df8  IoCompletion    0x1f0003        -
4       System  0x840e99ad3d00  0x3e00  IoCompletion    0x1f0003        -
4       System  0x840ea5d2f080  0x3e04  Process 0x0     svchost.exe Pid 9444
4       System  0x840e91d94ee0  0x3e08  Event   0x1f0003        -
4       System  0xb707fbcb4f50  0x3e0c  SymbolicLink    0xf0001 Session
4       System  0x840e9a794e30  0x3e10  IoCompletionReserve     0xf0003 -
4       System  0x840e9ac3e840  0x3e18  WaitCompletionPacket    0x1     -
4       System  0x840e96d4c420  0x3e1c  Timer   0x2     -
4       System  0x840e99c2cbe0  0x3e20  Event   0x1f0003        -
4       System  0x840e97884d60  0x3e24  ALPC Port       0x1f0001        [CoreMsgK]-{873fd6d2-ff75-11ef-ba82-bc2411703d8b}
4       System  0x840e9ac0ec80  0x3e28  WaitCompletionPacket    0x1     -
4       System  0x840e99c6e080  0x3e2c  Process 0x102a  svchost.exe Pid 4636
4       System  0x840e9769f080  0x3e30  Process 0x102a  winlogon.exe Pid 2804
4       System  0x840e9ac2cd80  0x3e34  WaitCompletionPacket    0x1     -
4       System  0x840e976537c0  0x3e38  IoCompletion    0x1f0003        -
4       System  0x840e9ac32540  0x3e3c  WaitCompletionPacket    0x1     -
4       System  0x840e9c5b81e0  0x3e44  Event   0x1f0003        -
4       System  0x840e9f40c340  0x3e48  Process 0x0     svchost.exe Pid 2800
4       System  0x840e96808d80  0x3e4c  WaitCompletionPacket    0x1     -
4       System  0x840e92fc5300  0x3e50  Process 0x102a  svchost.exe Pid 1976
4       System  0x840e993240c0  0x3e54  Process 0x102a  sihost.exe Pid 10384
4       System  0x840e99c83080  0x3e58  Process 0x40    dwm.exe Pid 10628
4       System  0x840e994d1080  0x3e5c  Process 0x1fffff        LogonUI.exe Pid 10680
4       System  0x840e9c8bc6f0  0x3e60  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee29-852e-11ef-ba7c-bc2411845bc2}.TxR.0.regtrans-ms
4       System  0x840e9ac29bf0  0x3e64  WaitCompletionPacket    0x1     -
4       System  0x840e9ac2b930  0x3e68  WaitCompletionPacket    0x1     -
4       System  0x840e9ba0e210  0x3e6c  WaitCompletionPacket    0x1     -
4       System  0x840e99fd7ae0  0x3e70  Event   0x1f0003        -
4       System  0x840e9903a940  0x3e74  IoCompletion    0x1f0003        -
4       System  0x840e99c14660  0x3e78  Event   0x1f0003        -
4       System  0x840e99fdfee0  0x3e7c  Event   0x1f0003        -
4       System  0x840e9c8bd8f0  0x3e84  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee29-852e-11ef-ba7c-bc2411845bc2}.TxR.1.regtrans-ms
4       System  0x840e99c83080  0x3e88  Process 0x40    dwm.exe Pid 10628
4       System  0x840e9ac10d00  0x3e8c  WaitCompletionPacket    0x1     -
4       System  0x840e9cb83940  0x3e90  IoCompletion    0x1f0003        -
4       System  0x840e92db94e0  0x3e94  Event   0x1f0003        -
4       System  0x840e98bba080  0x3e98  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e9c9a4ba0  0x3ea0  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.UI.Xaml.2.0_2.1810.18004.0_x64__8wekyb3d8bbwe\ActivationStore.dat
4       System  0x840e9a947e60  0x3ea4  Event   0x1f0003        -
4       System  0x840e9ac3d660  0x3ea8  WaitCompletionPacket    0x1     -
4       System  0x840e9ac2c220  0x3eac  WaitCompletionPacket    0x1     -
4       System  0x840e99bcb080  0x3eb0  Process 0x40    explorer.exe Pid 9852
4       System  0x840e90ee88c0  0x3eb8  File    0x13019f        \Device\clfs\Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR
4       System  0x840e9ca37f40  0x3ebc  IoCompletion    0x1f0003        -
4       System  0x840e966bc6c0  0x3ec4  ALPC Port       0x1f0001        -
4       System  0x840e97e8c080  0x3ec8  Process 0x1fffff        svchost.exe Pid 3864
4       System  0x840e9c457540  0x3ecc  IoCompletion    0x1f0003        -
4       System  0x840e98d0fe00  0x3ed0  IoCompletion    0x1f0003        -
4       System  0x840e9c8966e0  0x3edc  Event   0x1f0003        -
4       System  0x840e9cea11e0  0x3ee0  Event   0x1f0003        -
4       System  0x840e9764ebc0  0x3ee4  IoCompletion    0x1f0003        -
4       System  0x840e978d3ac0  0x3ee8  WaitCompletionPacket    0x1     -
4       System  0x840e992c56c0  0x3eec  IoCompletion    0x1f0003        -
4       System  0x840e9f3460c0  0x3ef0  Process 0x102a  svchost.exe Pid 11596
4       System  0x840e9764b100  0x3ef4  IoCompletion    0x1f0003        -
4       System  0x840e9ba0ff50  0x3ef8  WaitCompletionPacket    0x1     -
4       System  0x840ea7061080  0x3efc  Process 0x102a  svchost.exe Pid 16024
4       System  0x840e9e5a11c0  0x3f04  Process 0x102a  uhssvc.exe Pid 11740
4       System  0x840e978f0420  0x3f08  TmEn    0xf001f -
4       System  0x840e98db8080  0x3f0c  Process 0x102a  MsSense.exe Pid 10224
4       System  0x840e9c35caa0  0x3f10  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e99c6e080  0x3f18  Process 0x0     svchost.exe Pid 4636
4       System  0x840e9ba02db0  0x3f1c  WaitCompletionPacket    0x1     -
4       System  0x840e99fda860  0x3f20  Event   0x1f0003        -
4       System  0x840e9c252080  0x3f24  Process 0x0     RuntimeBroker. Pid 10648
4       System  0xb708057063d0  0x3f28  Token   0xe     -
4       System  0x840e99c1ee60  0x3f2c  Event   0x1f0003        -
4       System  0xb7081c69c400  0x3f30  Directory       0xf     RPC Control
4       System  0x840e9c89ffe0  0x3f34  Event   0x1f0003        -
4       System  0x840e9c441080  0x3f38  Process 0x102a  SearchApp.exe Pid 9092
4       System  0x840e9c712b60  0x3f40  Event   0x1f0003        -
4       System  0x840e9ea14080  0x3f44  Process 0x102a  ScreenConnect. Pid 1772
4       System  0x840e985c5960  0x3f48  FilterCommunicationPort 0x1f0001        -
4       System  0x840e99fc6460  0x3f4c  Event   0x1f0003        -
4       System  0x840e97e9e310  0x3f50  CoreMessaging   0xf0000 -
4       System  0x840e9a94c2e0  0x3f54  Event   0x1f0003        -
4       System  0x840e9cea43e0  0x3f58  Event   0x1f0003        -
4       System  0x840e9a94cde0  0x3f60  Event   0x1f0003        -
4       System  0x840e99c2c7e0  0x3f64  Event   0x1f0003        -
4       System  0x840e997b16e0  0x3f68  Event   0x1f0003        -
4       System  0x840e9ca34200  0x3f6c  IoCompletion    0x1f0003        -
4       System  0x840e99347560  0x3f70  Event   0x1f0003        -
4       System  0x840e99ad4080  0x3f74  IoCompletion    0x1f0003        -
4       System  0x840e9ac0ce70  0x3f78  WaitCompletionPacket    0x1     -
4       System  0x840e99fe3ae0  0x3f80  Event   0x1f0003        -
4       System  0x840e98aded80  0x3f8c  TmRm    0x1f007f        -
4       System  0x840e98fd69e0  0x3f90  Event   0x1f0003        -
4       System  0x840e99aa7940  0x3f94  IoCompletion    0x1f0003        -
4       System  0x840e92e36e00  0x3f9c  IoCompletion    0x1f0003        -
4       System  0x840e98d11740  0x3fa0  IoCompletion    0x1f0003        -
4       System  0xb708129e0060  0x3fa8  Token   0xf01ff -
4       System  0x840e9cb8d0c0  0x3fac  Process 0x102a  msedge.exe Pid 5148
4       System  0x840e97f80080  0x3fb0  Process 0x102a  AggregatorHost Pid 5804
4       System  0x840e984fc5f0  0x3fb4  Timer   0x2     -
4       System  0x840e9969f080  0x3fb8  Process 0x0     Microsoft.Shar Pid 5176
4       System  0x840e96d0c600  0x3fbc  IoCompletion    0x1f0003        -
4       System  0x840e9c5299e0  0x3fc0  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT
4       System  0x840e97582080  0x3fc4  Process 0x0     msedge.exe Pid 16344
4       System  0x840e9d69bef0  0x3fc8  File    0x3     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-12-1-419028606-1323772725-945580434-2117921032\SystemAppData\Helium\Cache\b4d622c333fcfdd7_COM15.dat.LOG2
4       System  0x840e9ff21880  0x3fd0  IoCompletion    0x1f0003        -
4       System  0x840e9b739080  0x3fd4  Process 0x102a  Microsoft.Phot Pid 11148
4       System  0x840e9ac2d740  0x3fd8  WaitCompletionPacket    0x1     -
4       System  0x840e9c8963e0  0x3fdc  Event   0x1f0003        -
4       System  0x840e9a943d60  0x3fe0  Event   0x1f0003        -
4       System  0x840e91a11080  0x3fe4  Process 0x102a  spoolsv.exe Pid 14536
4       System  0x840e9c8975e0  0x3fe8  Event   0x1f0003        -
4       System  0x840e99fd8ae0  0x3fec  Event   0x1f0003        -
4       System  0x840e99c83080  0x3ff0  Process 0x102a  dwm.exe Pid 10628
4       System  0x840e90e10410  0x3ff4  Session 0x0     Session2
4       System  0x840e9b277d50  0x3ff8  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat.LOG2
4       System  0x840e98eb3080  0x3ffc  Process 0x102a  conhost.exe Pid 10944
4       System  0x840e9b739080  0x4008  Process 0x40    Microsoft.Phot Pid 11148
4       System  0x840ea53539e0  0x400c  Event   0x1f0003        -
4       System  0x840e9b985ec0  0x4014  WaitCompletionPacket    0x1     -
4       System  0x840e9cb90540  0x4018  TmEn    0xf001f -
4       System  0x840e9f38a440  0x401c  IoCompletion    0x1f0003        -
4       System  0x840e9eb950c0  0x402c  Process 0x1fffff        CloudNotificat Pid 14972
4       System  0x840ea9ed1c60  0x4030  Event   0x1f0003        -
4       System  0x840ea9ec9560  0x4038  Event   0x1f0003        -
4       System  0xb7081cbe3200  0x4044  SymbolicLink    0xf0001 Local
4       System  0x840e9cb8d0c0  0x404c  Process 0x102a  msedge.exe Pid 5148
4       System  0x840e97dc9aa0  0x4050  TmEn    0xf001f -
4       System  0x840e9c5b57e0  0x4058  Event   0x1f0003        -
4       System  0x840e9dde0a70  0x4060  TmEn    0xf001f -
4       System  0x840e9da4d040  0x4064  Thread  0x1fffff        Tid 11260 Pid 4
4       System  0x840e98b2a090  0x406c  WaitCompletionPacket    0x1     -
4       System  0xb708234e4060  0x4074  Token   0xf01ff -
4       System  0x840e985c8360  0x4084  FilterCommunicationPort 0x1f0001        -
4       System  0x840e9c85ca60  0x4088  Event   0x1f0003        -
4       System  0x840e9ac2b2b0  0x4090  WaitCompletionPacket    0x1     -
4       System  0x840e9b6f6080  0x4094  Process 0x102a  msedge.exe Pid 13488
4       System  0x840e91a11080  0x409c  Process 0x0     spoolsv.exe Pid 14536
4       System  0x840e9dace4c0  0x40a0  IoCompletion    0x1f0003        -
4       System  0x840e9ba04540  0x40ac  WaitCompletionPacket    0x1     -
4       System  0x840e9e6f3080  0x40b0  Process 0x1fffff        excel-update.e Pid 4672
4       System  0x840e9ee430c0  0x40b4  Process 0x1fffff        powershell.exe Pid 13156
4       System  0x840e9d9b1080  0x40bc  Process 0x102a  msedge.exe Pid 6408
4       System  0x840e9c709860  0x40c0  Event   0x1f0003        -
4       System  0x840e9f027920  0x40c4  WaitCompletionPacket    0x1     -
4       System  0x840e9b993470  0x40c8  WaitCompletionPacket    0x1     -
4       System  0x840e98c1afe0  0x40d0  Event   0x1f0003        -
4       System  0x840e9a641c00  0x40d4  IoCompletion    0x1f0003        -
4       System  0x840e99081550  0x40d8  WaitCompletionPacket    0x1     -
4       System  0x840e97dfcbc0  0x40e0  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTAdmin_PS_Provider.etl
4       System  0x840e9b6f6080  0x40ec  Process 0x1fffff        msedge.exe Pid 13488
4       System  0x840e9c47d480  0x40f4  IoCompletion    0x1f0003        -
4       System  0x840e9f3ace00  0x40f8  IoCompletion    0x1f0003        -
4       System  0x840e91a11080  0x40fc  Process 0x102a  spoolsv.exe Pid 14536
4       System  0x840e9e5bca70  0x4104  TmEn    0xf001f -
4       System  0x840e9934b3e0  0x4110  Event   0x1f0003        -
4       System  0x840e97c4bde0  0x4114  Event   0x1f0003        -
4       System  0x840e992b1780  0x4118  IoCompletion    0x1f0003        -
4       System  0x840e9909ce80  0x411c  WaitCompletionPacket    0x1     -
4       System  0x840e9f7a7340  0x4124  IoCompletion    0x1f0003        -
4       System  0x840e9b032b70  0x412c  TmEn    0xf001f -
4       System  0x840e9b991e80  0x4130  WaitCompletionPacket    0x1     -
4       System  0x840e9cf2b540  0x4134  IoCompletion    0x1f0003        -
4       System  0x840e999ee660  0x4138  Event   0x1f0003        -
4       System  0x840e9ba14c80  0x4144  WaitCompletionPacket    0x1     -
4       System  0x840e99a92080  0x4148  Process 0x102a  SecurityHealth Pid 12036
4       System  0x840e91da3b60  0x414c  Event   0x1f0003        -
4       System  0x840e9cb6cac0  0x4150  IoCompletion    0x1f0003        -
4       System  0xb7081b046460  0x4154  Directory       0xf     S-1-15-2-536077884-713174666-1066051701-3219990555-339840825-1966734348-1611281757
4       System  0x840e9cb4db80  0x415c  IoCompletion    0x1f0003        -
4       System  0x840e99bf3080  0x4160  Process 0x1fffff        msedge.exe Pid 12476
4       System  0x840e9cf08440  0x4164  IoCompletion    0x1f0003        -
4       System  0x840e9e99e080  0x416c  Process 0x0     dllhost.exe Pid 7884
4       System  0x840e9bee60c0  0x4170  Process 0x1fffff        svchost.exe Pid 12440
4       System  0x840ea085fa40  0x4174  IoCompletion    0x1f0003        -
4       System  0x840e98b635e0  0x417c  WaitCompletionPacket    0x1     -
4       System  0x840ea47c8ed0  0x4184  WaitCompletionPacket    0x1     -
4       System  0x840e9ce53300  0x4188  Process 0x102a  ctfmon.exe Pid 12188
4       System  0x840e9ba057f0  0x4190  WaitCompletionPacket    0x1     -
4       System  0x840e9c70d0e0  0x4194  Event   0x1f0003        -
4       System  0x840e985d8080  0x419c  Process 0x102a  ShellExperienc Pid 7588
4       System  0x840e9dabe700  0x41a8  IoCompletion    0x1f0003        -
4       System  0x840e9ba0bf20  0x41ac  WaitCompletionPacket    0x1     -
4       System  0x840e9dad4b00  0x41b0  IoCompletion    0x1f0003        -
4       System  0x840e9ac421f0  0x41b4  WaitCompletionPacket    0x1     -
4       System  0x840e9e6f3080  0x41b8  Process 0x0     excel-update.e Pid 4672
4       System  0x840e9a629e80  0x41c0  IoCompletion    0x1f0003        -
4       System  0xb7081a5a9420  0x41c4  Directory       0xf     RPC Control
4       System  0x840e9f327080  0x41c8  Process 0x1fffff        ApplicationFra Pid 7492
4       System  0x840e9e99e080  0x41cc  Process 0x102a  dllhost.exe Pid 7884
4       System  0x840e99bdd080  0x41d0  Process 0x1fffff        SearchApp.exe Pid 9288
4       System  0xb70823a3b350  0x41d4  SymbolicLink    0xf0001 Local
4       System  0x840e92e37c00  0x41d8  IoCompletion    0x1f0003        -
4       System  0x840e97383a40  0x41dc  IoCompletion    0x1f0003        -
4       System  0x840e9f28b080  0x41e0  Process 0x102a  WmiPrvSE.exe Pid 14028
4       System  0x840e9c852e60  0x41e4  Event   0x1f0003        -
4       System  0x840e9dac5380  0x41e8  IoCompletion    0x1f0003        -
4       System  0x840e9710da60  0x41ec  Event   0x1f0003        -
4       System  0x840e9f2d2b60  0x41f4  Event   0x1f0003        -
4       System  0x840e9ba169c0  0x41f8  WaitCompletionPacket    0x1     -
4       System  0x840e9ac2f6f0  0x41fc  WaitCompletionPacket    0x1     -
4       System  0x840e99bdd080  0x4204  Process 0x0     SearchApp.exe Pid 9288
4       System  0x840e9b7bb080  0x4208  Process 0x102a  StartMenuExper Pid 8
4       System  0x840e9cac9ee0  0x4210  Event   0x1f0003        -
4       System  0xb7082344e980  0x4220  Token   0xf01ff -
4       System  0x840e985d8080  0x4224  Process 0x102a  ShellExperienc Pid 7588
4       System  0x840e99089b60  0x4228  WaitCompletionPacket    0x1     -
4       System  0x840e9cb4fe80  0x4234  IoCompletion    0x1f0003        -
4       System  0x840e9b1a1240  0x423c  Process 0x0     RuntimeBroker. Pid 12140
4       System  0x840e9b73a080  0x4240  Process 0x102a  svchost.exe Pid 5116
4       System  0x840ea3ce29e0  0x4250  Event   0x1f0003        -
4       System  0x840e9cf35540  0x4254  IoCompletion    0x1f0003        -
4       System  0x840e99bdd080  0x426c  Process 0x40    SearchApp.exe Pid 9288
4       System  0x840e9f021390  0x4274  WaitCompletionPacket    0x1     -
4       System  0x840e9dacc7c0  0x4278  IoCompletion    0x1f0003        -
4       System  0x840e9b73a080  0x427c  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e9b2a4d30  0x4284  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-2226957697-3030467180-2301525-4248967783-2024719031-2325529081-2915787518
4       System  0x840e9b1a1240  0x4288  Process 0x102a  RuntimeBroker. Pid 12140
4       System  0x840e98b2b4e0  0x428c  WaitCompletionPacket    0x1     -
4       System  0x840e98ec91c0  0x4290  Process 0x1fffff        RuntimeBroker. Pid 10968
4       System  0x840e9cb45e40  0x4294  IoCompletion    0x1f0003        -
4       System  0x840e9c491840  0x4298  IoCompletion    0x1f0003        -
4       System  0x840e9a94a3e0  0x429c  Event   0x1f0003        -
4       System  0x840e9ba24ad0  0x42a0  WaitCompletionPacket    0x1     -
4       System  0x840e9c00c120  0x42a4  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.Photos_8wekyb3d8bbwe\Settings\settings.dat.LOG1
4       System  0x840e9cf2c080  0x42a8  IoCompletion    0x1f0003        -
4       System  0x840ea5a17060  0x42b4  Event   0x1f0003        -
4       System  0xb70823896230  0x42c0  Section 0x5     -
4       System  0x840e99bdd080  0x42c4  Process 0x102a  SearchApp.exe Pid 9288
4       System  0x840e9ac33a60  0x42d0  WaitCompletionPacket    0x1     -
4       System  0x840e9c48ae80  0x42d4  IoCompletion    0x1f0003        -
4       System  0x840e9cacf4e0  0x42d8  Event   0x1f0003        -
4       System  0x840e9dae96c0  0x42e0  IoCompletion    0x1f0003        -
4       System  0x840e9c8496e0  0x42e4  Event   0x1f0003        -
4       System  0x840e9c491780  0x42e8  IoCompletion    0x1f0003        -
4       System  0x840e99c27c60  0x42f4  Event   0x1f0003        -
4       System  0x840e9c474340  0x42f8  IoCompletion    0x1f0003        -
4       System  0xb7080815e950  0x4300  Token   0xf01ff -
4       System  0x840e9666cb30  0x4314  WaitCompletionPacket    0x1     -
4       System  0x840e9cb8d0c0  0x4320  Process 0x102a  msedge.exe Pid 5148
4       System  0x840e98ec91c0  0x4324  Process 0x102a  RuntimeBroker. Pid 10968
4       System  0x840e9cacade0  0x4328  Event   0x1f0003        -
4       System  0x840e9ca48b00  0x432c  IoCompletion    0x1f0003        -
4       System  0x840e9cba3080  0x4330  Process 0x0     WmiPrvSE.exe Pid 7964
4       System  0x840e9ba23820  0x4334  WaitCompletionPacket    0x1     -
4       System  0x840e9da14340  0x4338  Process 0x0     TextInputHost. Pid 12740
4       System  0x840e9cabffe0  0x433c  Event   0x1f0003        -
4       System  0x840e9ce53300  0x4340  Process 0x102a  ctfmon.exe Pid 12188
4       System  0x840e9b9a8740  0x4344  WaitCompletionPacket    0x1     -
4       System  0x840e9c47c140  0x4354  IoCompletion    0x1f0003        -
4       System  0x840e9cb7a3c0  0x436c  IoCompletion    0x1f0003        -
4       System  0x840e9c0cb080  0x437c  Process 0x1fffff        splunk-winevtl Pid 11256
4       System  0x840e99bdd080  0x4380  Process 0x102a  SearchApp.exe Pid 9288
4       System  0x840e9b98b270  0x4384  WaitCompletionPacket    0x1     -
4       System  0x840ea4a640c0  0x438c  Process 0x102a  msedge.exe Pid 13964
4       System  0x840e995ea080  0x4390  Process 0x102a  OneDrive.exe Pid 11592
4       System  0x840e9dabfdc0  0x4394  IoCompletion    0x1f0003        -
4       System  0x840e978e28d0  0x4398  WaitCompletionPacket    0x1     -
4       System  0x840e9dad7c00  0x439c  IoCompletion    0x1f0003        -
4       System  0x840e9dad38c0  0x43a0  IoCompletion    0x1f0003        -
4       System  0xb70813225850  0x43a8  Section 0x17    -
4       System  0x840e99702ae0  0x43b0  Event   0x1f0003        -
4       System  0xb70820c9f120  0x43b4  SymbolicLink    0xf0001 Global
4       System  0x840e98c1d3e0  0x43c0  Event   0x1f0003        -
4       System  0x840e9ba40190  0x43c4  WaitCompletionPacket    0x1     -
4       System  0x840e9676e080  0x43c8  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9c486780  0x43d4  IoCompletion    0x1f0003        -
4       System  0x840e98180810  0x43d8  WaitCompletionPacket    0x1     -
4       System  0x840e9dae64c0  0x43dc  IoCompletion    0x1f0003        -
4       System  0x840e997b2ce0  0x43e0  Event   0x1f0003        -
4       System  0x840eaa193950  0x43e8  WaitCompletionPacket    0x1     -
4       System  0x840e9f3460c0  0x43ec  Process 0x102a  svchost.exe Pid 11596
4       System  0x840e9e3d99e0  0x43f4  Event   0x1f0003        -
4       System  0x840e9bee60c0  0x43f8  Process 0x102a  svchost.exe Pid 12440
4       System  0x840e9ac3bac0  0x43fc  WaitCompletionPacket    0x1     -
4       System  0xb708318f16b0  0x4404  Token   0xf01ff -
4       System  0x840e92c51f30  0x4410  WaitCompletionPacket    0x1     -
4       System  0x840e9d69c710  0x4418  File    0x20003 \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-12-1-419028606-1323772725-945580434-2117921032\SystemAppData\Helium\Cache\b4d622c333fcfdd7_COM15.dat
4       System  0x840e9ca69800  0x441c  IoCompletion    0x1f0003        -
4       System  0x840e9c5ac5e0  0x4424  Event   0x1f0003        -
4       System  0x840e9f3df080  0x4428  Process 0x102a  conhost.exe Pid 7256
4       System  0x840e9cacd460  0x4430  Event   0x1f0003        -
4       System  0x840e990a6f60  0x443c  WaitCompletionPacket    0x1     -
4       System  0x840e99bdd080  0x4440  Process 0x102a  SearchApp.exe Pid 9288
4       System  0x840e9df4be60  0x4448  Event   0x1f0003        -
4       System  0x840ea9ecb0e0  0x444c  Event   0x1f0003        -
4       System  0x840ea08f3080  0x4454  Process 0x102a  svchost.exe Pid 12008
4       System  0x840e9f40c340  0x445c  Process 0x102a  svchost.exe Pid 2800
4       System  0x840e99651080  0x4460  Process 0x0     conhost.exe Pid 13176
4       System  0x840e9cb4df00  0x4464  IoCompletion    0x1f0003        -
4       System  0x840ea0860a00  0x4468  IoCompletion    0x1f0003        -
4       System  0x840e9b203300  0x4470  Process 0x102a  KeePass.exe Pid 11628
4       System  0x840e9c441080  0x447c  Process 0x40    SearchApp.exe Pid 9092
4       System  0x840e9c45dc80  0x4484  IoCompletion    0x1f0003        -
4       System  0x840e9ca6f680  0x4490  IoCompletion    0x1f0003        -
4       System  0x840e9b739080  0x449c  Process 0x102a  Microsoft.Phot Pid 11148
4       System  0x840e9daee480  0x44a4  IoCompletion    0x1f0003        -
4       System  0x840e990a0c40  0x44a8  WaitCompletionPacket    0x1     -
4       System  0x840e9dab8880  0x44b0  IoCompletion    0x1f0003        -
4       System  0x840e9c713ae0  0x44b8  Event   0x1f0003        -
4       System  0x840e99bdd080  0x44bc  Process 0x102a  SearchApp.exe Pid 9288
4       System  0x840e99bdd080  0x44c4  Process 0x40    SearchApp.exe Pid 9288
4       System  0xb70821ba5cb0  0x44c8  SymbolicLink    0xf0001 Global
4       System  0x840e9caccbe0  0x44cc  Event   0x1f0003        -
4       System  0x840e9b0d92b0  0x44d4  TmEn    0xf001f -
4       System  0x840e91e98070  0x44d8  IoCompletionReserve     0xf0003 -
4       System  0x840e9ac2f210  0x44e0  WaitCompletionPacket    0x1     -
4       System  0x840e9ce9b8e0  0x44e4  Event   0x1f0003        -
4       System  0x840e9b9a1990  0x44ec  WaitCompletionPacket    0x1     -
4       System  0x840e9c48ed80  0x44f0  IoCompletion    0x1f0003        -
4       System  0x840e9b203300  0x44f4  Process 0x102a  KeePass.exe Pid 11628
4       System  0x840e96c582c0  0x44f8  Process 0x1028  svchost.exe Pid 3028
4       System  0x840e99c143e0  0x4500  Event   0x1f0003        -
4       System  0x840e98bae930  0x4504  ALPC Port       0x1f0001        -
4       System  0x840e9caba960  0x450c  Event   0x1f0003        -
4       System  0x840e97714bf0  0x4510  TmEn    0xf001f -
4       System  0x840e9b98d220  0x4514  WaitCompletionPacket    0x1     -
4       System  0x840e9c477e00  0x451c  IoCompletion    0x1f0003        -
4       System  0x840e9ac31c50  0x4524  WaitCompletionPacket    0x1     -
4       System  0x840e9cb45f40  0x4528  IoCompletion    0x1f0003        -
4       System  0x840e9b99ef50  0x4530  WaitCompletionPacket    0x1     -
4       System  0x840e9d8669e0  0x4538  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.Photos_8wekyb3d8bbwe\Settings\settings.dat.LOG2
4       System  0x840e98fcb6e0  0x453c  Event   0x1f0003        -
4       System  0xb7082305a140  0x4544  SymbolicLink    0xf0001 Local
4       System  0x840e9d880ec0  0x4548  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.ShellExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e9c46b900  0x454c  IoCompletion    0x1f0003        -
4       System  0x840e9cab8f60  0x4550  Event   0x1f0003        -
4       System  0x840e9cb243e0  0x4554  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e9cb236e0  0x4564  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e9714bce0  0x4570  WaitCompletionPacket    0x1     -
4       System  0x840ea084acc0  0x4578  IoCompletion    0x1f0003        -
4       System  0x840e9dae5340  0x457c  IoCompletion    0x1f0003        -
4       System  0x840e9ac282c0  0x4580  WaitCompletionPacket    0x1     -
4       System  0x840e99fd37e0  0x4588  Event   0x1f0003        -
4       System  0x840e9ac31360  0x458c  WaitCompletionPacket    0x1     -
4       System  0x840e9c465640  0x4590  IoCompletion    0x1f0003        -
4       System  0x840e99bdd080  0x4594  Process 0x40    SearchApp.exe Pid 9288
4       System  0x840e9cad0160  0x4598  Event   0x1f0003        -
4       System  0x840e9dabdac0  0x459c  IoCompletion    0x1f0003        -
4       System  0x840e9ac351f0  0x45a0  WaitCompletionPacket    0x1     -
4       System  0x840e9ba12650  0x45a4  WaitCompletionPacket    0x1     -
4       System  0x840e9909b480  0x45ac  WaitCompletionPacket    0x1     -
4       System  0x840e9ba1bca0  0x45b4  WaitCompletionPacket    0x1     -
4       System  0x840e9cb23540  0x45b8  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e99bdd080  0x45bc  Process 0x102a  SearchApp.exe Pid 9288
4       System  0x840e9f38d7c0  0x45c0  IoCompletion    0x1f0003        -
4       System  0x840e9f2d0560  0x45cc  Event   0x1f0003        -
4       System  0x840e99bdd080  0x45d0  Process 0x102a  SearchApp.exe Pid 9288
4       System  0xb70823899f70  0x45d8  Section 0x17    -
4       System  0x840e9c494c00  0x45dc  IoCompletion    0x1f0003        -
4       System  0x840e9eaee0c0  0x45e0  Process 0x102a  dllhost.exe Pid 10124
4       System  0x840e91ea72b0  0x45e4  IoCompletionReserve     0xf0003 -
4       System  0x840e9b203300  0x45ec  Process 0x102a  KeePass.exe Pid 11628
4       System  0x840e9f327080  0x45f4  Process 0x102a  ApplicationFra Pid 7492
4       System  0x840e9ac33990  0x45fc  WaitCompletionPacket    0x1     -
4       System  0x840ea47c3980  0x4600  WaitCompletionPacket    0x1     -
4       System  0x840e9de2bee0  0x460c  Event   0x1f0003        -
4       System  0x840e9a9540e0  0x4610  Event   0x1f0003        -
4       System  0xb70820ca56e0  0x461c  SymbolicLink    0xf0001 Local
4       System  0x840e9dad9100  0x4620  IoCompletion    0x1f0003        -
4       System  0x840e9c860ae0  0x4624  Event   0x1f0003        -
4       System  0x840e9ec9c4c0  0x4638  File    0x20003 \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\AppData\Local\Microsoft\Windows\DeliveryOptimization\State\dosvcState.dat
4       System  0x840e9c85b860  0x463c  Event   0x1f0003        -
4       System  0x840e9f2d9be0  0x4640  Event   0x1f0003        -
4       System  0x840e9817ac40  0x4644  WaitCompletionPacket    0x1     -
4       System  0x840ea0605780  0x4648  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-1910091885-1573563583-1104941280-2418270861-3411158377-2822700936-2990310272
4       System  0x840e9ba2aec0  0x4650  WaitCompletionPacket    0x1     -
4       System  0x840e971413e0  0x4668  WaitCompletionPacket    0x1     -
4       System  0xb7081793cdb0  0x466c  Section 0x5     -
4       System  0x840e9ec48b40  0x4670  File    0x120089        \Device\NamedPipe\PSEXESVC-RKIPPER-3244-stdout
4       System  0x840e9ca60ec0  0x4674  IoCompletion    0x1f0003        -
4       System  0x840e9cba3080  0x4678  Process 0x1fffff        WmiPrvSE.exe Pid 7964
4       System  0x840e96d0b900  0x467c  IoCompletion    0x1f0003        -
4       System  0x840e9c5a8860  0x4680  Event   0x1f0003        -
4       System  0x840e9c236080  0x4688  Process 0x0     dllhost.exe Pid 5164
4       System  0x840e9bdf4340  0x4694  Process 0x102a  ScreenConnect. Pid 14756
4       System  0x840e98b599e0  0x46a4  WaitCompletionPacket    0x1     -
4       System  0x840e9ba195a0  0x46a8  WaitCompletionPacket    0x1     -
4       System  0x840e9c479140  0x46ac  IoCompletion    0x1f0003        -
4       System  0x840e9c236080  0x46b0  Process 0x1fffff        dllhost.exe Pid 5164
4       System  0x840eaa1873e0  0x46b8  WaitCompletionPacket    0x1     -
4       System  0x840e9668db50  0x46bc  IoCompletionReserve     0xf0003 -
4       System  0x840e9c441080  0x46c0  Process 0x40    SearchApp.exe Pid 9092
4       System  0x840e91bc84a0  0x46c4  WaitCompletionPacket    0x1     -
4       System  0x840e97385480  0x46cc  IoCompletion    0x1f0003        -
4       System  0x840e98d37e80  0x46d8  IoCompletion    0x1f0003        -
4       System  0x840e9c5b1260  0x46dc  Event   0x1f0003        -
4       System  0x840e9b9acf90  0x46e0  WaitCompletionPacket    0x1     -
4       System  0x840e9bdf4340  0x46ec  Process 0x102a  ScreenConnect. Pid 14756
4       System  0x840e9c215080  0x46f0  Process 0x102a  conhost.exe Pid 1600
4       System  0x840ea53483e0  0x46fc  Event   0x1f0003        -
4       System  0x840e974f7340  0x4704  Process 0x102a  svchost.exe Pid 15932
4       System  0x840e9dad14c0  0x470c  IoCompletion    0x1f0003        -
4       System  0x840e9b98b1a0  0x4710  WaitCompletionPacket    0x1     -
4       System  0x840e9d69c8b0  0x4718  File    0x3     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-12-1-419028606-1323772725-945580434-2117921032\SystemAppData\Helium\Cache\b4d622c333fcfdd7_COM15.dat.LOG1
4       System  0x840e9b73a080  0x4720  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e9ac145e0  0x4728  WaitCompletionPacket    0x1     -
4       System  0x840ea9ec9de0  0x4730  Event   0x1f0003        -
4       System  0x840e9c250080  0x4734  Process 0x102a  SenseIR.exe Pid 7728
4       System  0x840ea47d10d0  0x4738  WaitCompletionPacket    0x1     -
4       System  0xb70823a3b980  0x473c  SymbolicLink    0xf0001 Session
4       System  0xb7081ae8d930  0x4740  Token   0xf01ff -
4       System  0x840e9b812530  0x474c  IoCompletionReserve     0xf0003 -
4       System  0x840e9b984730  0x4750  WaitCompletionPacket    0x1     -
4       System  0x840e9de22660  0x4754  Event   0x1f0003        -
4       System  0x840e9f2cb2e0  0x4758  Event   0x1f0003        -
4       System  0x840e9f2d9060  0x475c  Event   0x1f0003        -
4       System  0x840e9676e080  0x4760  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9b203300  0x4764  Process 0x1fffff        KeePass.exe Pid 11628
4       System  0x840e9971b360  0x4774  Event   0x1f0003        -
4       System  0x840e9cea7260  0x4780  Event   0x1f0003        -
4       System  0x840e9c250080  0x4784  Process 0x102a  SenseIR.exe Pid 7728
4       System  0x840e9c8610e0  0x4794  Event   0x1f0003        -
4       System  0x840e9cf1dec0  0x4798  IoCompletion    0x1f0003        -
4       System  0x840e978ed080  0x479c  Process 0x1fffff        cmd.exe Pid 13752
4       System  0x840e9aa14930  0x47a0  IoCompletionReserve     0xf0003 -
4       System  0x840e9680f240  0x47a4  WaitCompletionPacket    0x1     -
4       System  0x840e9dabd9c0  0x47ac  IoCompletion    0x1f0003        -
4       System  0x840e9c5b5260  0x47b0  Event   0x1f0003        -
4       System  0x840e9ac39560  0x47bc  WaitCompletionPacket    0x1     -
4       System  0x840e9c5b3a60  0x47cc  Event   0x1f0003        -
4       System  0x840e9ba07600  0x47d0  WaitCompletionPacket    0x1     -
4       System  0x840e990143c0  0x47d8  IoCompletion    0x1f0003        -
4       System  0xb70814b5a730  0x47dc  Section 0x17    -
4       System  0x840ea5a11de0  0x47e0  Event   0x1f0003        -
4       System  0x840e97c640c0  0x47e4  Process 0x102a  powershell.exe Pid 10352
4       System  0x840e9c5acde0  0x47f4  Event   0x1f0003        -
4       System  0xb70813b9c660  0x47f8  Directory       0xf     RPC Control
4       System  0x840eab2f0260  0x4808  Event   0x1f0003        -
4       System  0x840e9c92fde0  0x4810  Event   0x1f0003        -
4       System  0x840e9e920080  0x4818  Process 0x102a  FileCoAuth.exe Pid 7516
4       System  0x840ea08f1080  0x481c  Process 0x102a  svchost.exe Pid 3444
4       System  0xb70808571830  0x4820  Token   0xf01ff -
4       System  0x840e9f40c340  0x4824  Process 0x102a  svchost.exe Pid 2800
4       System  0x840ea08516c0  0x4830  IoCompletion    0x1f0003        -
4       System  0x840e97455160  0x4834  Event   0x1f0003        -
4       System  0x840ea47c2870  0x483c  WaitCompletionPacket    0x1     -
4       System  0x840e99c1f360  0x4848  Event   0x1f0003        -
4       System  0xb70821baaa60  0x484c  SymbolicLink    0xf0001 Local
4       System  0x840e9c7069e0  0x4850  Event   0x1f0003        -
4       System  0x840e99a89080  0x4854  Process 0x102a  svchost.exe Pid 11756
4       System  0x840e99adf400  0x4858  IoCompletion    0x1f0003        -
4       System  0xb7082389c1d0  0x4860  Section 0x5     -
4       System  0x840e9ac3a400  0x4864  WaitCompletionPacket    0x1     -
4       System  0x840e9ac40240  0x486c  WaitCompletionPacket    0x1     -
4       System  0x840e9dac0200  0x4874  IoCompletion    0x1f0003        -
4       System  0x840e978d83e0  0x4880  WaitCompletionPacket    0x1     -
4       System  0x840e9c70f260  0x4888  Event   0x1f0003        -
4       System  0x840e9f38a6c0  0x488c  IoCompletion    0x1f0003        -
4       System  0x840e99bcb080  0x4890  Process 0x40    explorer.exe Pid 9852
4       System  0x840e96669db0  0x4894  WaitCompletionPacket    0x1     -
4       System  0x840e97380880  0x4898  IoCompletion    0x1f0003        -
4       System  0x840e9cf11680  0x48a0  IoCompletion    0x1f0003        -
4       System  0x840ea5364de0  0x48ac  Event   0x1f0003        -
4       System  0x840e99651080  0x48b0  Process 0x102a  conhost.exe Pid 13176
4       System  0x840e9e3d05e0  0x48b8  Event   0x1f0003        -
4       System  0x840e9b739080  0x48bc  Process 0x1fffff        Microsoft.Phot Pid 11148
4       System  0x840e98f832f0  0x48c0  IoCompletionReserve     0xf0003 -
4       System  0x840e98b60930  0x48c4  WaitCompletionPacket    0x1     -
4       System  0x840e9b011ce0  0x48c8  ALPC Port       0x1f0001        -
4       System  0x840e98b617d0  0x48d0  WaitCompletionPacket    0x1     -
4       System  0x840e99a89080  0x48d4  Process 0x1fffff        svchost.exe Pid 11756
4       System  0xb70814304540  0x48d8  Token   0xf01ff -
4       System  0x840e99a89080  0x48dc  Process 0x0     svchost.exe Pid 11756
4       System  0xb70816f07060  0x48e0  Token   0xf01ff -
4       System  0x840e9c5b52e0  0x48e4  Event   0x1f0003        -
4       System  0x840e9ea14080  0x48ec  Process 0x102a  ScreenConnect. Pid 1772
4       System  0x840e9ba25a40  0x48f0  WaitCompletionPacket    0x1     -
4       System  0x840e9c7027e0  0x48f8  Event   0x1f0003        -
4       System  0x840e9cf21500  0x4900  IoCompletion    0x1f0003        -
4       System  0xb7082305a8d0  0x4904  SymbolicLink    0xf0001 Session
4       System  0x840e9ce53300  0x4908  Process 0x0     ctfmon.exe Pid 12188
4       System  0x840e9ca6f780  0x4914  IoCompletion    0x1f0003        -
4       System  0x840e9ba1e7b0  0x4918  WaitCompletionPacket    0x1     -
4       System  0x840e99da3080  0x491c  Process 0x1fffff        RuntimeBroker. Pid 4464
4       System  0x840e9cb62480  0x4924  IoCompletion    0x1f0003        -
4       System  0x840e9cb61e40  0x4928  IoCompletion    0x1f0003        -
4       System  0x840e9c556d00  0x492c  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\User.dat
4       System  0x840e9e6f3080  0x4930  Process 0x102a  excel-update.e Pid 4672
4       System  0x840e9ba1e3a0  0x4934  WaitCompletionPacket    0x1     -
4       System  0x840e9ac18bc0  0x493c  WaitCompletionPacket    0x1     -
4       System  0x840e98183250  0x4944  WaitCompletionPacket    0x1     -
4       System  0x840e978d0520  0x4948  WaitCompletionPacket    0x1     -
4       System  0x840ea08593c0  0x494c  IoCompletion    0x1f0003        -
4       System  0x840e9c46ddc0  0x4950  IoCompletion    0x1f0003        -
4       System  0x840e9c721860  0x4958  Event   0x1f0003        -
4       System  0x840ea47c9b00  0x4960  WaitCompletionPacket    0x1     -
4       System  0x840ea9ecc8e0  0x496c  Event   0x1f0003        -
4       System  0x840e9b203300  0x4970  Process 0x0     KeePass.exe Pid 11628
4       System  0x840e9ba2a6a0  0x4974  WaitCompletionPacket    0x1     -
4       System  0x840e9969f080  0x4978  Process 0x102a  Microsoft.Shar Pid 5176
4       System  0x840e9ac356d0  0x497c  WaitCompletionPacket    0x1     -
4       System  0x840ea3ce5d60  0x4984  Event   0x1f0003        -
4       System  0x840ea084af80  0x4988  IoCompletion    0x1f0003        -
4       System  0x840e970cf080  0x498c  IoCompletion    0x1f0003        -
4       System  0x840e9eb950c0  0x4990  Process 0x40    CloudNotificat Pid 14972
4       System  0xb708042d4060  0x4998  Token   0xf01ff -
4       System  0x840e97f17040  0x499c  Thread  0x1fffff        Tid 10612 Pid 4
4       System  0xb7080892c060  0x49a0  Token   0xf01ff -
4       System  0x840e9b9bc690  0x49a4  WaitCompletionPacket    0x1     -
4       System  0x840e97c640c0  0x49a8  Process 0x0     powershell.exe Pid 10352
4       System  0x840e99bdd080  0x49ac  Process 0x102a  SearchApp.exe Pid 9288
4       System  0x840e993382e0  0x49b4  Event   0x1f0003        -
4       System  0x840e985d8080  0x49b8  Process 0x0     ShellExperienc Pid 7588
4       System  0x840e9c554120  0x49c0  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\User.dat.LOG1
4       System  0x840e9c711e60  0x49c8  Event   0x1f0003        -
4       System  0xb70820ca4660  0x49cc  SymbolicLink    0xf0001 Session
4       System  0x840e9ac2de90  0x49d4  WaitCompletionPacket    0x1     -
4       System  0x840e9c45c940  0x49d8  IoCompletion    0x1f0003        -
4       System  0x840e9ac2be10  0x49dc  WaitCompletionPacket    0x1     -
4       System  0x840e9cba3080  0x49e8  Process 0x102a  WmiPrvSE.exe Pid 7964
4       System  0x840ea3cee7e0  0x49f0  Event   0x1f0003        -
4       System  0x840e9ce53300  0x49f4  Process 0x1fffff        ctfmon.exe Pid 12188
4       System  0x840e9ccec080  0x49f8  Process 0x102a  svchost.exe Pid 11776
4       System  0x840e9b1a1240  0x4a00  Process 0x102a  RuntimeBroker. Pid 12140
4       System  0x840e99ad1080  0x4a08  IoCompletion    0x1f0003        -
4       System  0x840e9ac2f070  0x4a10  WaitCompletionPacket    0x1     -
4       System  0x840e9b2df080  0x4a14  Process 0x102a  MoUsoCoreWorke Pid 10144
4       System  0x840e9ee430c0  0x4a18  Process 0x0     powershell.exe Pid 13156
4       System  0x840e9817e1e0  0x4a24  WaitCompletionPacket    0x1     -
4       System  0x840e9ee430c0  0x4a28  Process 0x102a  powershell.exe Pid 13156
4       System  0x840e92f50460  0x4a34  Event   0x1f0003        -
4       System  0x840e9f7c0300  0x4a38  IoCompletion    0x1f0003        -
4       System  0x840e99703260  0x4a40  Event   0x1f0003        -
4       System  0x840ea08436c0  0x4a44  IoCompletion    0x1f0003        -
4       System  0x840e99474080  0x4a48  Process 0x102a  PSEXESVC.exe Pid 11296
4       System  0x840e9ba27440  0x4a4c  WaitCompletionPacket    0x1     -
4       System  0x840e9f398b40  0x4a50  IoCompletion    0x1f0003        -
4       System  0x840e9dad7b40  0x4a5c  IoCompletion    0x1f0003        -
4       System  0x840e9daee100  0x4a68  IoCompletion    0x1f0003        -
4       System  0x840e97c5e4e0  0x4a74  Event   0x1f0003        -
4       System  0x840e98b3fc50  0x4a78  WaitCompletionPacket    0x1     -
4       System  0x840e9e5a11c0  0x4a7c  Process 0x102a  uhssvc.exe Pid 11740
4       System  0x840e9ac19cd0  0x4a80  WaitCompletionPacket    0x1     -
4       System  0x840e9c5a8c60  0x4a84  Event   0x1f0003        -
4       System  0x840e9cf05600  0x4a88  IoCompletion    0x1f0003        -
4       System  0x840e9cac8ae0  0x4a8c  Event   0x1f0003        -
4       System  0x840e9ac24430  0x4a90  WaitCompletionPacket    0x1     -
4       System  0x840e9e5a11c0  0x4a94  Process 0x102a  uhssvc.exe Pid 11740
4       System  0x840e9969f080  0x4a9c  Process 0x1fffff        Microsoft.Shar Pid 5176
4       System  0x840e9ca5d6c0  0x4aa4  IoCompletion    0x1f0003        -
4       System  0x840e9b841ab0  0x4aa8  IoCompletionReserve     0xf0003 -
4       System  0x840ea5be0c00  0x4aac  IoCompletion    0x1f0003        -
4       System  0x840ea7061080  0x4ab0  Process 0x102a  svchost.exe Pid 16024
4       System  0x840e9b73a080  0x4ab8  Process 0x0     svchost.exe Pid 5116
4       System  0x840e9ce17d90  0x4abc  ALPC Port       0x1f0001        -
4       System  0x840ea70570c0  0x4ac4  Process 0x102a  msedge.exe Pid 1336
4       System  0x840e9cf18740  0x4ad4  IoCompletion    0x1f0003        -
4       System  0x840e9cba3080  0x4ad8  Process 0x102a  WmiPrvSE.exe Pid 7964
4       System  0x840e9d69d0d0  0x4ae4  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\UserClasses.dat.LOG1
4       System  0x840e9ceadc60  0x4aec  Event   0x1f0003        -
4       System  0x840e9f3a6500  0x4af0  IoCompletion    0x1f0003        -
4       System  0x840e9c84ed60  0x4af4  Event   0x1f0003        -
4       System  0x840e9ff0c980  0x4af8  IoCompletion    0x1f0003        -
4       System  0x840e995ea080  0x4b00  Process 0x102a  OneDrive.exe Pid 11592
4       System  0x840ea534b660  0x4b08  Event   0x1f0003        -
4       System  0x840e9f2cbf60  0x4b1c  Event   0x1f0003        -
4       System  0x840e9ba200e0  0x4b24  WaitCompletionPacket    0x1     -
4       System  0x840e909840b0  0x4b28  WaitCompletionPacket    0x1     -
4       System  0x840e9a9614e0  0x4b2c  Event   0x1f0003        -
4       System  0x840ea08700c0  0x4b30  IoCompletion    0x1f0003        -
4       System  0x840e9f0227e0  0x4b3c  WaitCompletionPacket    0x1     -
4       System  0x840e981915d0  0x4b40  WaitCompletionPacket    0x1     -
4       System  0x840e99335360  0x4b44  Event   0x1f0003        -
4       System  0x840e9ff09780  0x4b4c  IoCompletion    0x1f0003        -
4       System  0x840e9ba3a350  0x4b50  WaitCompletionPacket    0x1     -
4       System  0x840ea08f3080  0x4b54  Process 0x0     svchost.exe Pid 12008
4       System  0x840e9dad1400  0x4b6c  IoCompletion    0x1f0003        -
4       System  0x840e9935d510  0x4b80  ALPC Port       0x1f0001        -
4       System  0x840ea0ed7a60  0x4b94  Event   0x1f0003        -
4       System  0x840e9f3df080  0x4b98  Process 0x102a  conhost.exe Pid 7256
4       System  0x840ea3ce5ae0  0x4ba0  Event   0x1f0003        -
4       System  0x840e9e50c180  0x4ba8  Thread  0x1fffff        Tid 5336 Pid 4
4       System  0x840e9ff1ac00  0x4bb8  IoCompletion    0x1f0003        -
4       System  0x840e9ceb6960  0x4bbc  Event   0x1f0003        -
4       System  0x840e9b9c13c0  0x4bc0  WaitCompletionPacket    0x1     -
4       System  0x840e9e920080  0x4bc4  Process 0x102a  FileCoAuth.exe Pid 7516
4       System  0x840e99c83080  0x4bc8  Process 0x102a  dwm.exe Pid 10628
4       System  0x840ea0ed4060  0x4bcc  Event   0x1f0003        -
4       System  0x840e9dabbc00  0x4bd4  IoCompletion    0x1f0003        -
4       System  0x840ea53552e0  0x4bd8  Event   0x1f0003        -
4       System  0x840e9f791700  0x4be4  IoCompletion    0x1f0003        -
4       System  0x840e9b9978b0  0x4bf0  WaitCompletionPacket    0x1     -
4       System  0xb708123af960  0x4bf4  Token   0xf01ff -
4       System  0x840e99c83080  0x4bfc  Process 0x40    dwm.exe Pid 10628
4       System  0x840e91a11080  0x4c08  Process 0x1fffff        spoolsv.exe Pid 14536
4       System  0x840e9f794e00  0x4c0c  IoCompletion    0x1f0003        -
4       System  0x840e9ceb5760  0x4c10  Event   0x1f0003        -
4       System  0x840e9ba0a930  0x4c14  WaitCompletionPacket    0x1     -
4       System  0x840e9c84b2e0  0x4c18  Event   0x1f0003        -
4       System  0x840e9dacaf40  0x4c20  IoCompletion    0x1f0003        -
4       System  0x840ea5348fe0  0x4c2c  Event   0x1f0003        -
4       System  0x840e9e253340  0x4c30  Process 0x102a  conhost.exe Pid 13532
4       System  0x840ea9ec98e0  0x4c34  Event   0x1f0003        -
4       System  0x840e9f38f140  0x4c38  IoCompletion    0x1f0003        -
4       System  0x840e990a75e0  0x4c3c  WaitCompletionPacket    0x1     -
4       System  0x840ea5350960  0x4c40  Event   0x1f0003        -
4       System  0x840e9c9c60a0  0x4c48  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\MicrosoftWindows.Client.CBS_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e9f37f600  0x4c50  IoCompletion    0x1f0003        -
4       System  0x840e9c9ddce0  0x4c58  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\MicrosoftWindows.Client.CBS_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e9ba04f00  0x4c60  WaitCompletionPacket    0x1     -
4       System  0x840e9e7020c0  0x4c64  Process 0x0     taskhostw.exe Pid 6380
4       System  0x840e9ba411d0  0x4c6c  WaitCompletionPacket    0x1     -
4       System  0x840e9daca300  0x4c70  IoCompletion    0x1f0003        -
4       System  0x840e9cf06300  0x4c7c  IoCompletion    0x1f0003        -
4       System  0x840e9f3a3e80  0x4c80  IoCompletion    0x1f0003        -
4       System  0x840e9f2dc0e0  0x4c84  Event   0x1f0003        -
4       System  0x840e99bcb080  0x4c88  Process 0x40    explorer.exe Pid 9852
4       System  0x840e9ff105c0  0x4c90  IoCompletion    0x1f0003        -
4       System  0x840e9f3a1580  0x4c94  IoCompletion    0x1f0003        -
4       System  0x840e9c84bd60  0x4c9c  Event   0x1f0003        -
4       System  0xb708236e9a70  0x4ca0  Section 0x17    -
4       System  0x840e9f37e200  0x4ca8  IoCompletion    0x1f0003        -
4       System  0x840e9cab7260  0x4cb0  Event   0x1f0003        -
4       System  0x840e9ac44b60  0x4cb4  WaitCompletionPacket    0x1     -
4       System  0x840e9cf40440  0x4cbc  IoCompletion    0x1f0003        -
4       System  0x840e9cf26740  0x4cc0  IoCompletion    0x1f0003        -
4       System  0x840e9cf41080  0x4ccc  IoCompletion    0x1f0003        -
4       System  0x840e9ac3af60  0x4cd0  WaitCompletionPacket    0x1     -
4       System  0xb708217e2060  0x4cd8  Token   0xf01ff -
4       System  0x840e9cf3c800  0x4ce0  IoCompletion    0x1f0003        -
4       System  0x840e98b40880  0x4ce4  WaitCompletionPacket    0x1     -
4       System  0x840e9ba3cbf0  0x4ce8  WaitCompletionPacket    0x1     -
4       System  0x840e9ceb40e0  0x4cec  Event   0x1f0003        -
4       System  0x840e9c252080  0x4cf0  Process 0x1fffff        RuntimeBroker. Pid 10648
4       System  0x840e9da14340  0x4cf8  Process 0x40    TextInputHost. Pid 12740
4       System  0x840e9d69ab70  0x4cfc  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\UserClasses.dat
4       System  0x840e9cf36b40  0x4d04  IoCompletion    0x1f0003        -
4       System  0x840ea5355ee0  0x4d08  Event   0x1f0003        -
4       System  0x840e9a9538e0  0x4d14  Event   0x1f0003        -
4       System  0x840e9ba02c10  0x4d1c  WaitCompletionPacket    0x1     -
4       System  0x840e9daf4080  0x4d24  IoCompletion    0x1f0003        -
4       System  0x840e9b6ca8f0  0x4d28  IoCompletionReserve     0xf0003 -
4       System  0x840e9f792340  0x4d30  IoCompletion    0x1f0003        -
4       System  0x840e99080b90  0x4d34  WaitCompletionPacket    0x1     -
4       System  0x840e9d8816e0  0x4d38  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.ShellExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e9f3a4740  0x4d40  IoCompletion    0x1f0003        -
4       System  0x840e9dae2940  0x4d44  IoCompletion    0x1f0003        -
4       System  0x840e9f028a30  0x4d4c  WaitCompletionPacket    0x1     -
4       System  0x840ea535a7e0  0x4d50  Event   0x1f0003        -
4       System  0x840e9f005240  0x4d5c  WaitCompletionPacket    0x1     -
4       System  0x840e9da14340  0x4d64  Process 0x102a  TextInputHost. Pid 12740
4       System  0x840e92ee0da0  0x4d68  WaitCompletionPacket    0x1     -
4       System  0x840e9da14340  0x4d6c  Process 0x102a  TextInputHost. Pid 12740
4       System  0x840e9ba14530  0x4d70  WaitCompletionPacket    0x1     -
4       System  0x840e9c441080  0x4d74  Process 0x1fffff        SearchApp.exe Pid 9092
4       System  0x840e9a629a00  0x4d78  IoCompletion    0x1f0003        -
4       System  0x840e990b0d00  0x4d7c  WaitCompletionPacket    0x1     -
4       System  0x840e9df4a260  0x4d80  Event   0x1f0003        -
4       System  0x840e9cab60e0  0x4d88  Event   0x1f0003        -
4       System  0x840e9c92d9e0  0x4d8c  Event   0x1f0003        -
4       System  0x840e9f380b00  0x4d94  IoCompletion    0x1f0003        -
4       System  0x840ea0bf4040  0x4d98  Thread  0x1fffff        Tid 13132 Pid 4
4       System  0x840e98a445f0  0x4da4  IoCompletionReserve     0xf0003 -
4       System  0x840e9b73a080  0x4dac  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e9c5b9060  0x4db0  Event   0x1f0003        -
4       System  0x840e9eaee0c0  0x4db4  Process 0x1fffff        dllhost.exe Pid 10124
4       System  0x840e9daf4140  0x4dbc  IoCompletion    0x1f0003        -
4       System  0x840ea47cd650  0x4dc8  WaitCompletionPacket    0x1     -
4       System  0x840ea5353960  0x4dd8  Event   0x1f0003        -
4       System  0x840e9f2dbee0  0x4df0  Event   0x1f0003        -
4       System  0x840e9ba13350  0x4df4  WaitCompletionPacket    0x1     -
4       System  0x840e974f7340  0x4df8  Process 0x1fffff        svchost.exe Pid 15932
4       System  0x840ea534b3e0  0x4e08  Event   0x1f0003        -
4       System  0x840ea46eb3e0  0x4e0c  Event   0x1f0003        -
4       System  0x840e990934f0  0x4e10  WaitCompletionPacket    0x1     -
4       System  0x840e97c640c0  0x4e1c  Process 0x1fffff        powershell.exe Pid 10352
4       System  0xb70807f96060  0x4e24  Token   0xf01ff -
4       System  0x840ea9ecd360  0x4e28  Event   0x1f0003        -
4       System  0x840e9c30ba80  0x4e40  ALPC Port       0x1f0001        -
4       System  0x840e99bf3080  0x4e44  Process 0x0     msedge.exe Pid 12476
4       System  0x840e99e240c0  0x4e48  Process 0x102a  ScreenConnect. Pid 6988
4       System  0x840e978d4140  0x4e54  WaitCompletionPacket    0x1     -
4       System  0x840e9ee430c0  0x4e5c  Process 0x102a  powershell.exe Pid 13156
4       System  0x840e9dad1f40  0x4e60  IoCompletion    0x1f0003        -
4       System  0xb70835507240  0x4e68  Key     0x20019 MACHINE\SOFTWARE\POLICIES\MICROSOFT
4       System  0x840e9ceaabe0  0x4e6c  Event   0x1f0003        -
4       System  0x840e9681a360  0x4e70  WaitCompletionPacket    0x1     -
4       System  0x840e995ea080  0x4e78  Process 0x40    OneDrive.exe Pid 11592
4       System  0x840e9eb950c0  0x4e7c  Process 0x102a  CloudNotificat Pid 14972
4       System  0x840e9ca33880  0x4e80  IoCompletion    0x1f0003        -
4       System  0x840e9f785d80  0x4e8c  IoCompletion    0x1f0003        -
4       System  0x840ea70570c0  0x4e90  Process 0x102a  msedge.exe Pid 1336
4       System  0x840e9c484d40  0x4e98  IoCompletion    0x1f0003        -
4       System  0x840ea08f3080  0x4e9c  Process 0x102a  svchost.exe Pid 12008
4       System  0x840e995ea080  0x4ea0  Process 0x102a  OneDrive.exe Pid 11592
4       System  0x840ea5359ce0  0x4ea4  Event   0x1f0003        -
4       System  0x840ea087cc80  0x4eb8  IoCompletion    0x1f0003        -
4       System  0x840e9ac3c480  0x4ec0  WaitCompletionPacket    0x1     -
4       System  0x840e9d7e5cf0  0x4ec4  ALPC Port       0x1f0001        -
4       System  0x840e9969f080  0x4ed0  Process 0x102a  Microsoft.Shar Pid 5176
4       System  0x840ea08f1080  0x4ed4  Process 0x0     svchost.exe Pid 3444
4       System  0x840e9c9af300  0x4ee0  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Photos_2019.19071.12548.0_x64__8wekyb3d8bbwe\ActivationStore.dat.LOG2
4       System  0x840e9b73a080  0x4ee4  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e9ac20a80  0x4ee8  WaitCompletionPacket    0x1     -
4       System  0x840ea084dc00  0x4ef0  IoCompletion    0x1f0003        -
4       System  0x840e92e48080  0x4efc  Process 0x1028  svchost.exe Pid 1524
4       System  0x840ea47c2e20  0x4f08  WaitCompletionPacket    0x1     -
4       System  0x840e9f7a1680  0x4f0c  IoCompletion    0x1f0003        -
4       System  0x840e9f0141f0  0x4f20  WaitCompletionPacket    0x1     -
4       System  0x840e9e5a11c0  0x4f24  Process 0x1fffff        uhssvc.exe Pid 11740
4       System  0x840e9ba09f70  0x4f28  WaitCompletionPacket    0x1     -
4       System  0x840ea9ed1860  0x4f38  Event   0x1f0003        -
4       System  0x840e9aa1dc30  0x4f44  IoCompletionReserve     0xf0003 -
4       System  0x840e9b6f6080  0x4f54  Process 0x0     msedge.exe Pid 13488
4       System  0x840e9ce9a8e0  0x4f58  Event   0x1f0003        -
4       System  0x840e9e7020c0  0x4f64  Process 0x102a  taskhostw.exe Pid 6380
4       System  0x840e9ba2af90  0x4f68  WaitCompletionPacket    0x1     -
4       System  0x840e9e902340  0x4f6c  Process 0x102a  msedge.exe Pid 5808
4       System  0x840ea5349be0  0x4f70  Event   0x1f0003        -
4       System  0x840e918e90c0  0x4f80  Process 0x102a  svchost.exe Pid 14128
4       System  0x840e9817cb20  0x4f84  WaitCompletionPacket    0x1     -
4       System  0x840e9e3cd4e0  0x4f8c  Event   0x1f0003        -
4       System  0x840e9f2d0de0  0x4f9c  Event   0x1f0003        -
4       System  0x840e99090500  0x4fa0  WaitCompletionPacket    0x1     -
4       System  0x840e9b987990  0x4fa4  WaitCompletionPacket    0x1     -
4       System  0x840e9b9bfb60  0x4fa8  WaitCompletionPacket    0x1     -
4       System  0x840e9e3cfb60  0x4fb4  Event   0x1f0003        -
4       System  0x840e9bdf4340  0x4fb8  Process 0x102a  ScreenConnect. Pid 14756
4       System  0x840e9e3cc660  0x4fbc  Event   0x1f0003        -
4       System  0x840e9cf23800  0x4fc4  IoCompletion    0x1f0003        -
4       System  0x840e99081f10  0x4fc8  WaitCompletionPacket    0x1     -
4       System  0x840e9c441080  0x4fcc  Process 0x102a  SearchApp.exe Pid 9092
4       System  0x840e9c47aa80  0x4fd0  IoCompletion    0x1f0003        -
4       System  0x840ea47cc200  0x4fd4  WaitCompletionPacket    0x1     -
4       System  0x840e9f3460c0  0x4fe0  Process 0x1fffff        svchost.exe Pid 11596
4       System  0x840ea47cd8c0  0x4fe4  WaitCompletionPacket    0x1     -
4       System  0x840e9dacbac0  0x4fec  IoCompletion    0x1f0003        -
4       System  0x840e9803e2e0  0x4ff0  ALPC Port       0x1f0001        -
4       System  0x840e9f2d09e0  0x4ff4  Event   0x1f0003        -
4       System  0x840e9f7b16c0  0x5004  IoCompletion    0x1f0003        -
4       System  0x840e9c4636c0  0x5014  IoCompletion    0x1f0003        -
4       System  0x840ea0ed9a60  0x501c  Event   0x1f0003        -
4       System  0x840ea9ecf1e0  0x5020  Event   0x1f0003        -
4       System  0x840ea47c3640  0x5024  WaitCompletionPacket    0x1     -
4       System  0xb70828699060  0x5034  Token   0xf01ff -
4       System  0x840e9ec483c0  0x504c  File    0x12019f        \Device\NamedPipe\PSEXESVC
4       System  0x840e9ba1ac60  0x5050  WaitCompletionPacket    0x1     -
4       System  0x840e990b2080  0x5064  WaitCompletionPacket    0x1     -
4       System  0x840e9e253340  0x5080  Process 0x1fffff        conhost.exe Pid 13532
4       System  0x840eaa187990  0x5088  WaitCompletionPacket    0x1     -
4       System  0x840e97fb12e0  0x508c  Event   0x1f0003        -
4       System  0x840e99e240c0  0x50a0  Process 0x102a  ScreenConnect. Pid 6988
4       System  0x840e98190180  0x50a8  WaitCompletionPacket    0x1     -
4       System  0xb70828304f20  0x50b8  Key     0x10    MACHINE\SOFTWARE\POLICIES\MICROSOFT\WINDOWS
4       System  0x840eab2e57e0  0x50bc  Event   0x1f0003        -
4       System  0x840e9eb950c0  0x50c0  Process 0x0     CloudNotificat Pid 14972
4       System  0x840e9f7c0b00  0x50cc  IoCompletion    0x1f0003        -
4       System  0x840e9cb8d0c0  0x50dc  Process 0x1fffff        msedge.exe Pid 5148
4       System  0x840ea0ecb860  0x50e0  Event   0x1f0003        -
4       System  0x840eaa193bc0  0x50e4  WaitCompletionPacket    0x1     -
4       System  0x840e99bf3080  0x50e8  Process 0x102a  msedge.exe Pid 12476
4       System  0x840e9c250080  0x50ec  Process 0x0     SenseIR.exe Pid 7728
4       System  0x840e9ba0c740  0x50f8  WaitCompletionPacket    0x1     -
4       System  0x840e98fe9960  0x5100  Event   0x1f0003        -
4       System  0xb70823058300  0x5104  SymbolicLink    0xf0001 Global
4       System  0x840e9f7a6b40  0x5110  IoCompletion    0x1f0003        -
4       System  0x840e9969f080  0x5114  Process 0x102a  Microsoft.Shar Pid 5176
4       System  0x840e99f62600  0x5118  IoCompletion    0x1f0003        -
4       System  0x840e9c441080  0x5124  Process 0x102a  SearchApp.exe Pid 9092
4       System  0x840e9f7ab780  0x5130  IoCompletion    0x1f0003        -
4       System  0x840e9ba11470  0x513c  WaitCompletionPacket    0x1     -
4       System  0x840e99ad4180  0x5140  IoCompletion    0x1f0003        -
4       System  0x840e9676e080  0x5154  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9817f7d0  0x5160  WaitCompletionPacket    0x1     -
4       System  0x840e96f5a080  0x5170  Process 0x102a  SenseNdr.exe Pid 15568
4       System  0x840e9e7020c0  0x517c  Process 0x102a  taskhostw.exe Pid 6380
4       System  0x840ea4a640c0  0x5188  Process 0x102a  msedge.exe Pid 13964
4       System  0x840e9f78b6c0  0x5190  IoCompletion    0x1f0003        -
4       System  0x840e9c8a7a60  0x5198  Event   0x1f0003        -
4       System  0x840e97c640c0  0x51a0  Process 0x102a  powershell.exe Pid 10352
4       System  0x840ea3cdfa60  0x51b0  Event   0x1f0003        -
4       System  0x840e9e7020c0  0x51c4  Process 0x102a  taskhostw.exe Pid 6380
4       System  0x840e92e48080  0x51d0  Process 0x1028  svchost.exe Pid 1524
4       System  0x840e97fc0160  0x51e0  Event   0x1f0003        -
4       System  0x840e9ee430c0  0x51e4  Process 0x102a  powershell.exe Pid 13156
4       System  0x840ea5bea6c0  0x51ec  IoCompletion    0x1f0003        -
4       System  0x840e98b3ce00  0x51f0  WaitCompletionPacket    0x1     -
4       System  0x840e97fc1060  0x520c  Event   0x1f0003        -
4       System  0x840ea30987e0  0x5210  Event   0x1f0003        -
4       System  0x840e9970eb60  0x5214  Event   0x1f0003        -
4       System  0x840e9f3df080  0x5220  Process 0x1fffff        conhost.exe Pid 7256
4       System  0x840e9b988830  0x5230  WaitCompletionPacket    0x1     -
4       System  0x840eaa187c00  0x5234  WaitCompletionPacket    0x1     -
4       System  0x840e99091e30  0x5238  WaitCompletionPacket    0x1     -
4       System  0x840e9ff0df40  0x523c  IoCompletion    0x1f0003        -
4       System  0x840e9e7020c0  0x5244  Process 0x1fffff        taskhostw.exe Pid 6380
4       System  0x840ea7061080  0x5248  Process 0x0     svchost.exe Pid 16024
4       System  0x840e9daf6ec0  0x525c  IoCompletion    0x1f0003        -
4       System  0x840e9c848660  0x5260  Event   0x1f0003        -
4       System  0x840e9cf0c6c0  0x5268  IoCompletion    0x1f0003        -
4       System  0x840e9dabff80  0x5270  IoCompletion    0x1f0003        -
4       System  0x840e9ba02b40  0x527c  WaitCompletionPacket    0x1     -
4       System  0x840e99fd3360  0x5280  Event   0x100000        -
4       System  0x840e9818ddc0  0x5294  WaitCompletionPacket    0x1     -
4       System  0x840ea084a780  0x52a0  IoCompletion    0x1f0003        -
4       System  0x840eab2e3ce0  0x52c4  Event   0x1f0003        -
4       System  0x840e97b09080  0x52c8  Process 0x102a  powershell.exe Pid 15172
4       System  0x840e9f3df080  0x52dc  Process 0x0     conhost.exe Pid 7256
4       System  0x840e9680c3f0  0x52e8  WaitCompletionPacket    0x1     -
4       System  0x840e9f7b8500  0x5300  IoCompletion    0x1f0003        -
4       System  0x840e9ba238f0  0x530c  WaitCompletionPacket    0x1     -
4       System  0x840e9ba2ccd0  0x5324  WaitCompletionPacket    0x1     -
4       System  0x840e974f7340  0x5330  Process 0x102a  svchost.exe Pid 15932
4       System  0xb70812b03840  0x5338  Directory       0xf     RPC Control
4       System  0x840e99474080  0x533c  Process 0x0     PSEXESVC.exe Pid 11296
4       System  0x840e9ba024c0  0x5348  WaitCompletionPacket    0x1     -
4       System  0x840e9f39a900  0x534c  IoCompletion    0x1f0003        -
4       System  0x840ea47c15c0  0x5350  WaitCompletionPacket    0x1     -
4       System  0x840e9c845ce0  0x535c  Event   0x1f0003        -
4       System  0x840e98187830  0x5368  WaitCompletionPacket    0x1     -
4       System  0x840ea3cf4860  0x536c  Event   0x1f0003        -
4       System  0x840ea8d6a080  0x5370  Process 0x102a  OpenHandleColl Pid 11408
4       System  0x840e99fd8d60  0x5374  Event   0x1f0003        -
4       System  0xb70807783060  0x5378  Token   0xf01ff -
4       System  0x840e99651080  0x5388  Process 0x102a  conhost.exe Pid 13176
4       System  0x840e9dae2a40  0x5394  IoCompletion    0x1f0003        -
4       System  0x840e9f7b3900  0x539c  IoCompletion    0x1f0003        -
4       System  0x840e97714180  0x53a0  Thread  0x1fffff        Tid 14484 Pid 4
4       System  0x840e9ba0ca80  0x53a4  WaitCompletionPacket    0x1     -
4       System  0x840e9ff161c0  0x53b4  IoCompletion    0x1f0003        -
4       System  0x840ea086fd40  0x53b8  IoCompletion    0x1f0003        -
4       System  0x840ea17e9080  0x53c4  Process 0x102a  svchost.exe Pid 10392
4       System  0x840ea309fa60  0x53d0  Event   0x1f0003        -
4       System  0x840e9c926160  0x53d8  Event   0x1f0003        -
4       System  0x840e9e253340  0x53e4  Process 0x0     conhost.exe Pid 13532
4       System  0x840e96f5a080  0x53e8  Process 0x1028  SenseNdr.exe Pid 15568
4       System  0x840e971140e0  0x53f0  Event   0x1f0003        -
4       System  0x840e9b6f6080  0x540c  Process 0x102a  msedge.exe Pid 13488
4       System  0x840e99581080  0x541c  Process 0x0     svchost.exe Pid 15508
4       System  0x840e9dac19c0  0x5428  IoCompletion    0x1f0003        -
4       System  0x840ea4a640c0  0x542c  Process 0x102a  msedge.exe Pid 13964
4       System  0xb7082b761a70  0x5438  Token   0xf01ff -
4       System  0x840e9c250080  0x543c  Process 0x1fffff        SenseIR.exe Pid 7728
4       System  0x840e98fcd0e0  0x5440  Event   0x1f0003        -
4       System  0x840e9ba0a380  0x544c  WaitCompletionPacket    0x1     -
4       System  0x840e9b9b0a10  0x5454  WaitCompletionPacket    0x1     -
4       System  0x840e9b73a080  0x545c  Process 0x102a  svchost.exe Pid 5116
4       System  0x840ea47d5370  0x5460  WaitCompletionPacket    0x1     -
4       System  0x840e97d5ba40  0x5474  IoCompletion    0x1f0003        -
4       System  0x840e9ccec080  0x547c  Process 0x0     svchost.exe Pid 11776
4       System  0x840e9b996390  0x5488  WaitCompletionPacket    0x1     -
4       System  0x840ea8d6a080  0x5494  Process 0x102a  OpenHandleColl Pid 11408
4       System  0x840e971027e0  0x549c  Event   0x1f0003        -
4       System  0x840e9f39b540  0x54b0  IoCompletion    0x1f0003        -
4       System  0x840e96f5a080  0x54b4  Process 0x102a  SenseNdr.exe Pid 15568
4       System  0x840ea53515e0  0x54b8  Event   0x1f0003        -
4       System  0x840e9c84e3e0  0x54c0  Event   0x1f0003        -
4       System  0x840e9dad9ac0  0x54d8  IoCompletion    0x1f0003        -
4       System  0x840e9eb950c0  0x54e4  Process 0x102a  CloudNotificat Pid 14972
4       System  0x840e9d9b1080  0x54e8  Process 0x1fffff        msedge.exe Pid 6408
4       System  0x840e97b09080  0x54f8  Process 0x0     powershell.exe Pid 15172
4       System  0x840e9b73a080  0x5500  Process 0x102a  svchost.exe Pid 5116
4       System  0x840ea47c89f0  0x550c  WaitCompletionPacket    0x1     -
4       System  0x840e9ba13f80  0x5510  WaitCompletionPacket    0x1     -
4       System  0x840e9bee60c0  0x551c  Process 0x0     svchost.exe Pid 12440
4       System  0x840e918e90c0  0x5534  Process 0x1fffff        svchost.exe Pid 14128
4       System  0x840e9ba139d0  0x5540  WaitCompletionPacket    0x1     -
4       System  0x840e99bf3080  0x5544  Process 0x102a  msedge.exe Pid 12476
4       System  0x840e9c846160  0x5564  Event   0x1f0003        -
4       System  0x840e99bf3080  0x5570  Process 0x102a  msedge.exe Pid 12476
4       System  0x840e9dae3900  0x5578  IoCompletion    0x1f0003        -
4       System  0x840e9ea14080  0x5580  Process 0x0     ScreenConnect. Pid 1772
4       System  0x840e978ed080  0x559c  Process 0x102a  cmd.exe Pid 13752
4       System  0x840e9e6f3080  0x55a0  Process 0x102a  excel-update.e Pid 4672
4       System  0xb708193cf7e0  0x55a8  Directory       0xf     S-1-15-2-3251537155-1984446955-2931258699-841473695-1938553385-924012159-129201922
4       System  0xb7081d4d8710  0x55ac  SymbolicLink    0xf0001 Local
4       System  0xb708193cdbc0  0x55b0  Directory       0xf     RPC Control
4       System  0xb7081d4d9840  0x55b4  SymbolicLink    0xf0001 Global
4       System  0xb7081d4d8870  0x55b8  SymbolicLink    0xf0001 Session
4       System  0x840e9e8a2e40  0x55bc  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-3251537155-1984446955-2931258699-841473695-1938553385-924012159-129201922
4       System  0x840e9c469380  0x55c8  IoCompletion    0x1f0003        -
4       System  0x840e9819c7c0  0x55cc  WaitCompletionPacket    0x1     -
4       System  0xb70808571830  0x55d8  Token   0xf01ff -
4       System  0x840e9818cf20  0x55dc  WaitCompletionPacket    0x1     -
4       System  0x840e9f791600  0x55e4  IoCompletion    0x1f0003        -
4       System  0x840e9daf1200  0x55fc  IoCompletion    0x1f0003        -
4       System  0x840e99474080  0x5600  Process 0x102a  PSEXESVC.exe Pid 11296
4       System  0x840e9f7af900  0x560c  IoCompletion    0x1f0003        -
4       System  0x840e9ba08980  0x561c  WaitCompletionPacket    0x1     -
4       System  0x840ea0879d40  0x562c  IoCompletion    0x1f0003        -
4       System  0x840ea0850380  0x5630  IoCompletion    0x1f0003        -
4       System  0x840e9eb950c0  0x5638  Process 0x102a  CloudNotificat Pid 14972
4       System  0x840ea46eb860  0x564c  Event   0x1f0003        -
4       System  0x840e9ba11e30  0x5654  WaitCompletionPacket    0x1     -
4       System  0x840ea9ecff60  0x565c  Event   0x1f0003        -
4       System  0xb7081ca87d30  0x5690  Key     0x2001f MACHINE\SYSTEM\CONTROLSET001\CONTROL\PRODUCTOPTIONS
4       System  0x840e9ee430c0  0x5698  Process 0x102a  powershell.exe Pid 13156
4       System  0x840ea47c4c30  0x569c  WaitCompletionPacket    0x1     -
4       System  0x840e97e59080  0x56ac  Process 0x1fffff        conhost.exe Pid 12392
4       System  0x840e9b2df080  0x56b0  Process 0x0     MoUsoCoreWorke Pid 10144
4       System  0x840e9c5c54e0  0x56b4  Event   0x1f0003        -
4       System  0x840e98178540  0x56c4  WaitCompletionPacket    0x1     -
4       System  0x840ea0866cc0  0x56c8  IoCompletion    0x1f0003        -
4       System  0x840e9ba1fb30  0x56dc  WaitCompletionPacket    0x1     -
4       System  0x840ea08482c0  0x56e0  IoCompletion    0x1f0003        -
4       System  0x840ea47c9e40  0x56ec  WaitCompletionPacket    0x1     -
4       System  0x840e99581080  0x5728  Process 0x1fffff        svchost.exe Pid 15508
4       System  0x840e9b2df080  0x573c  Process 0x102a  MoUsoCoreWorke Pid 10144
4       System  0x840e981922d0  0x5748  WaitCompletionPacket    0x1     -
4       System  0x840ea08532c0  0x5778  IoCompletion    0x1f0003        -
4       System  0x840e918e90c0  0x5780  Process 0x102a  svchost.exe Pid 14128
4       System  0x840e9db77080  0x5790  Process 0x1fffff        taskhostw.exe Pid 15784
4       System  0x840e9713fdf0  0x5794  WaitCompletionPacket    0x1     -
4       System  0x840e9c485280  0x5798  IoCompletion    0x1f0003        -
4       System  0x840e99e240c0  0x57b0  Process 0x102a  ScreenConnect. Pid 6988
4       System  0x840e98a37a00  0x57c8  IoCompletion    0x1f0003        -
4       System  0x840e9b73a080  0x57d8  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e9f28b080  0x57dc  Process 0x102a  WmiPrvSE.exe Pid 14028
4       System  0x840e9e253340  0x57f4  Process 0x102a  conhost.exe Pid 13532
4       System  0x840e99078e70  0x57f8  WaitCompletionPacket    0x1     -
4       System  0x840e9f792cc0  0x57fc  IoCompletion    0x1f0003        -
4       System  0x840e9c250080  0x5808  Process 0x102a  SenseIR.exe Pid 7728
4       System  0x840e9e510540  0x5810  Thread  0x1fffff        Tid 15156 Pid 4
4       System  0x840ea5360a60  0x5814  Event   0x1f0003        -
4       System  0x840ea0872200  0x581c  IoCompletion    0x1f0003        -
4       System  0x840ea0ec7860  0x5820  Event   0x1f0003        -
4       System  0x840ea3ce1660  0x5834  Event   0x1f0003        -
4       System  0x840e9c7046e0  0x5838  Event   0x1f0003        -
4       System  0x840e9c848360  0x5844  Event   0x1f0003        -
4       System  0x840ea0856c80  0x5850  IoCompletion    0x1f0003        -
4       System  0x840e9c8a20e0  0x5858  Event   0x1f0003        -
4       System  0x840e9ccec080  0x5868  Process 0x1fffff        svchost.exe Pid 11776
4       System  0x840e9ac1d340  0x586c  WaitCompletionPacket    0x1     -
4       System  0x840e99ad7b40  0x5870  IoCompletion    0x1f0003        -
4       System  0x840e98b3e250  0x588c  WaitCompletionPacket    0x1     -
4       System  0x840e98207040  0x5890  Thread  0x1fffff        Tid 4952 Pid 4
4       System  0x840e97582080  0x5894  Process 0x1fffff        msedge.exe Pid 16344
4       System  0x840e9daca4c0  0x589c  IoCompletion    0x1f0003        -
4       System  0x840ea47c0310  0x58ec  WaitCompletionPacket    0x1     -
4       System  0x840e99086420  0x58f0  WaitCompletionPacket    0x1     -
4       System  0x840e9e6f3080  0x58fc  Process 0x102a  excel-update.e Pid 4672
4       System  0x840e9c215080  0x5908  Process 0x0     conhost.exe Pid 1600
4       System  0x840e9ba11540  0x5914  WaitCompletionPacket    0x1     -
4       System  0x840e9f3aae80  0x592c  IoCompletion    0x1f0003        -
4       System  0x840e9c0cb080  0x5930  Process 0x102a  splunk-winevtl Pid 11256
4       System  0x840eaa190480  0x594c  WaitCompletionPacket    0x1     -
4       System  0x840eaa189c80  0x5954  WaitCompletionPacket    0x1     -
4       System  0x840ea086b100  0x5964  IoCompletion    0x1f0003        -
4       System  0x840e9968d080  0x596c  Process 0x0     splunkd.exe Pid 13620
4       System  0x840ea5d2f080  0x5970  Process 0x102a  svchost.exe Pid 9444
4       System  0x840e9ba0a790  0x5978  WaitCompletionPacket    0x1     -
4       System  0x840e9780ee60  0x5988  Event   0x1f0003        -
4       System  0x840e9dae34c0  0x5994  IoCompletion    0x1f0003        -
4       System  0x840ea9ee71e0  0x5998  Event   0x1f0003        -
4       System  0x840ea5d2f080  0x599c  Process 0x102a  svchost.exe Pid 9444
4       System  0x840e97b09080  0x59ac  Process 0x102a  powershell.exe Pid 15172
4       System  0x840e9c47d100  0x59b4  IoCompletion    0x1f0003        -
4       System  0xb70807a028b0  0x59bc  Token   0xf01ff -
4       System  0x840e9f7950c0  0x59d0  IoCompletion    0x1f0003        -
4       System  0x840e99c25660  0x59d8  Event   0x1f0003        -
4       System  0x840e9817f8a0  0x59dc  WaitCompletionPacket    0x1     -
4       System  0x840e97fb59e0  0x59e0  Event   0x1f0003        -
4       System  0x840e99474080  0x59e8  Process 0x102a  PSEXESVC.exe Pid 11296
4       System  0x840e97b09080  0x59ec  Process 0x102a  powershell.exe Pid 15172
4       System  0x840e9c215080  0x59f4  Process 0x102a  conhost.exe Pid 1600
4       System  0x840e96f5a080  0x5a00  Process 0x102a  SenseNdr.exe Pid 15568
4       System  0x840ea9ed5ae0  0x5a14  Event   0x1f0003        -
4       System  0x840e9f289040  0x5a18  Thread  0x1fffff        Tid 12248 Pid 4
4       System  0x840e97fb2460  0x5a20  Event   0x1f0003        -
4       System  0x840ea0855b00  0x5a24  IoCompletion    0x1f0003        -
4       System  0x840e9bee60c0  0x5a34  Process 0x102a  svchost.exe Pid 12440
4       System  0x840e9c215080  0x5a40  Process 0x1fffff        conhost.exe Pid 1600
4       System  0x840e981867f0  0x5a44  WaitCompletionPacket    0x1     -
4       System  0x840e9bee60c0  0x5a48  Process 0x102a  svchost.exe Pid 12440
4       System  0x840ea9ec9460  0x5a50  Event   0x1f0003        -
4       System  0x840e9e1d6040  0x5a54  Thread  0x1fffff        Tid 14132 Pid 4
4       System  0x840e9676e080  0x5a5c  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9b73a080  0x5a70  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e9c5b6960  0x5a78  Event   0x1f0003        -
4       System  0x840e9bee60c0  0x5a88  Process 0x102a  svchost.exe Pid 12440
4       System  0x840e99e240c0  0x5a98  Process 0x0     ScreenConnect. Pid 6988
4       System  0x840ea3098160  0x5a9c  Event   0x1f0003        -
4       System  0x840e9c8b1460  0x5aa0  Event   0x1f0003        -
4       System  0x840e9818c8a0  0x5aa4  WaitCompletionPacket    0x1     -
4       System  0xb70817dac0a0  0x5aa8  Token   0xf01ff -
4       System  0x840ea08682c0  0x5ab0  IoCompletion    0x1f0003        -
4       System  0x840e91a11080  0x5ab8  Process 0x102a  spoolsv.exe Pid 14536
4       System  0x840e981870e0  0x5ad0  WaitCompletionPacket    0x1     -
4       System  0x840e97b09080  0x5ad8  Process 0x1fffff        powershell.exe Pid 15172
4       System  0x840e9bee60c0  0x5ae8  Process 0x102a  svchost.exe Pid 12440
4       System  0x840e9ca54100  0x5aec  IoCompletion    0x1f0003        -
4       System  0x840e9bee60c0  0x5af0  Process 0x102a  svchost.exe Pid 12440
4       System  0x840e98187350  0x5b04  WaitCompletionPacket    0x1     -
4       System  0x840e9ff35380  0x5b08  IoCompletion    0x1f0003        -
4       System  0x840e98180330  0x5b10  WaitCompletionPacket    0x1     -
4       System  0x840e96805980  0x5b14  WaitCompletionPacket    0x1     -
4       System  0x840e9f391dc0  0x5b1c  IoCompletion    0x1f0003        -
4       System  0x840ea0edc760  0x5b20  Event   0x1f0003        -
4       System  0x840e974f7340  0x5b24  Process 0x0     svchost.exe Pid 15932
4       System  0x840e9817ec70  0x5b28  WaitCompletionPacket    0x1     -
4       System  0x840e978ed080  0x5b30  Process 0x102a  cmd.exe Pid 13752
4       System  0x840e9676e080  0x5b34  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9c0cb080  0x5b38  Process 0x102a  splunk-winevtl Pid 11256
4       System  0x840e9b2df080  0x5b3c  Process 0x1fffff        MoUsoCoreWorke Pid 10144
4       System  0x840e9e3e4460  0x5b40  Event   0x1f0003        -
4       System  0x840e9b99ad80  0x5b48  WaitCompletionPacket    0x1     -
4       System  0x840ea4a640c0  0x5b58  Process 0x1fffff        msedge.exe Pid 13964
4       System  0x840e97e59080  0x5b64  Process 0x102a  conhost.exe Pid 12392
4       System  0x840e990824c0  0x5b68  WaitCompletionPacket    0x1     -
4       System  0x840e9b6f6080  0x5b6c  Process 0x102a  msedge.exe Pid 13488
4       System  0x840ea9eccae0  0x5b74  Event   0x1f0003        -
4       System  0x840e9979d560  0x5b78  Event   0x1f0003        -
4       System  0x840ea4a640c0  0x5b80  Process 0x0     msedge.exe Pid 13964
4       System  0x840ea0860200  0x5b84  IoCompletion    0x1f0003        -
4       System  0x840e98fcb2e0  0x5b88  Event   0x1f0003        -
4       System  0x840e9c45f7c0  0x5b94  IoCompletion    0x1f0003        -
4       System  0x840e9b1a2a80  0x5b9c  ALPC Port       0x1f0001        -
4       System  0x840e9d9b1080  0x5ba0  Process 0x0     msedge.exe Pid 6408
4       System  0x840ea17e9080  0x5ba8  Process 0x0     svchost.exe Pid 10392
4       System  0x840ea0ed45e0  0x5bb4  Event   0x1f0003        -
4       System  0x840e9f28b080  0x5bb8  Process 0x0     WmiPrvSE.exe Pid 14028
4       System  0x840ea087a540  0x5bbc  IoCompletion    0x1f0003        -
4       System  0x840e9ba18080  0x5bc4  WaitCompletionPacket    0x1     -
4       System  0x840e99720960  0x5bc8  Event   0x1f0003        -
4       System  0x840ea5a14f60  0x5bcc  Event   0x1f0003        -
4       System  0x840e9dabb0c0  0x5be4  IoCompletion    0x1f0003        -
4       System  0x840e9ec48840  0x5be8  File    0x120089        \Device\NamedPipe\PSEXESVC-RKIPPER-3244-stderr
4       System  0x840ea8d6a080  0x5bec  Process 0x0     OpenHandleColl Pid 11408
4       System  0x840e9f787f80  0x5bf4  IoCompletion    0x1f0003        -
4       System  0x840e98b3b190  0x5bf8  WaitCompletionPacket    0x1     -
4       System  0x840ea9ed7ae0  0x5bfc  Event   0x1f0003        -
536     smss.exe        0x840e91047a60  0x4     Event   0x1f0003        -
536     smss.exe        0x840e91047ee0  0x8     Event   0x1f0003        -
536     smss.exe        0x840e90987da0  0xc     WaitCompletionPacket    0x1     -
536     smss.exe        0x840e90b9a540  0x10    IoCompletion    0x1f0003        -
536     smss.exe        0x840e910bbcb0  0x14    TpWorkerFactory 0xf00ff -
536     smss.exe        0x840e90bf6810  0x18    IRTimer 0x100002        -
536     smss.exe        0x840e90987580  0x1c    WaitCompletionPacket    0x1     -
536     smss.exe        0x840e90bf65f0  0x20    IRTimer 0x100002        -
536     smss.exe        0x840e90987c00  0x24    WaitCompletionPacket    0x1     -
536     smss.exe        0x840e90b9a8a0  0x28    EtwRegistration 0x804   -
536     smss.exe        0x840e90b9b860  0x2c    EtwRegistration 0x804   -
536     smss.exe        0x840e90b9a980  0x30    EtwRegistration 0x804   -
536     smss.exe        0x840e91047de0  0x34    Event   0x1f0003        -
536     smss.exe        0x840e91047260  0x38    Event   0x1f0003        -
536     smss.exe        0x840e90ed1b50  0x3c    File    0x100020        \Device\HarddiskVolume3\Windows
536     smss.exe        0x840e90b9ac40  0x40    IoCompletion    0x1f0003        -
536     smss.exe        0x840e910bc060  0x44    TpWorkerFactory 0xf00ff -
536     smss.exe        0x840e90bf6700  0x48    IRTimer 0x100002        -
536     smss.exe        0x840e90987e70  0x4c    WaitCompletionPacket    0x1     -
536     smss.exe        0x840e90bf6920  0x50    IRTimer 0x100002        -
536     smss.exe        0x840e90987990  0x54    WaitCompletionPacket    0x1     -
536     smss.exe        0x840e902ac430  0x58    EtwRegistration 0x804   -
536     smss.exe        0x840e90b9b6a0  0x5c    EtwRegistration 0x804   -
536     smss.exe        0x840e90b9b160  0x60    EtwRegistration 0x804   -
536     smss.exe        0x840e91197ec0  0x64    IoCompletion    0x1f0003        -
536     smss.exe        0x840e9184eca0  0x68    TpWorkerFactory 0xf00ff -
536     smss.exe        0x840e919d40a0  0x6c    IRTimer 0x100002        -
536     smss.exe        0x840e91bc1ca0  0x70    WaitCompletionPacket    0x1     -
536     smss.exe        0x840e919d54d0  0x74    IRTimer 0x100002        -
536     smss.exe        0x840e91bc2db0  0x78    WaitCompletionPacket    0x1     -
536     smss.exe        0x840e917df090  0x7c    ALPC Port       0x1f0001        SmApiPort
536     smss.exe        0x840e921650c0  0x80    Process 0x101441        csrss.exe Pid 804
536     smss.exe        0xb7080174f1a0  0x84    Section 0x4     -
536     smss.exe        0x840e91f522e0  0x88    Event   0x100002        -
536     smss.exe        0xb707fb855a40  0x8c    Directory       0xf000f GLOBAL??
536     smss.exe        0x840e91f9a4c0  0x90    File    0x1f01ff        \Device\NamedPipe\ProtectedPrefix
536     smss.exe        0xb708015fe840  0x94    Directory       0xf000f Sessions
536     smss.exe        0x840e92291140  0x98    Process 0x101441        csrss.exe Pid 888
536     smss.exe        0x840e9952c080  0x9c    Process 0x101441        csrss.exe Pid 10732
536     smss.exe        0x840e92165dc0  0xa4    ALPC Port       0x1f0001        -
536     smss.exe        0x840e91f64db0  0xa8    ALPC Port       0x1f0001        -
536     smss.exe        0x840e91f9b2d0  0xac    File    0x1f01ff        \Device\NamedPipe\Sessions
536     smss.exe        0x840e9107a2d0  0xb0    ALPC Port       0x1f0001        -
536     smss.exe        0x840e920152d0  0xb4    ALPC Port       0x1f0001        -
536     smss.exe        0x840e91f9b460  0xc0    File    0x1f01ff        \Device\Mailslot\ProtectedPrefix\NetWorkService
536     smss.exe        0xb70801666b40  0xc4    Key     0x2001f MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER\MEMORY MANAGEMENT
536     smss.exe        0x840e91f9a650  0xc8    File    0x1f01ff        \Device\NamedPipe\ProtectedPrefix\LocalService
536     smss.exe        0x840e91f9bdc0  0xcc    File    0x1f01ff        \Device\NamedPipe\ProtectedPrefix\NetWorkService
536     smss.exe        0x840e91f9a7e0  0xd0    File    0x1f01ff        \Device\Mailslot\ProtectedPrefix\LocalService
536     smss.exe        0x840e91f98d50  0xd8    File    0x1f01ff        \Device\Mailslot\ProtectedPrefix\Administrators
536     smss.exe        0x840e91f983f0  0xdc    File    0x1f01ff        \Device\Mailslot\ProtectedPrefix
536     smss.exe        0x840e91f9ab00  0xe0    File    0x1f01ff        \Device\NamedPipe\ProtectedPrefix\Administrators
536     smss.exe        0x840e97837740  0xf8    ALPC Port       0x1f0001        -
536     smss.exe        0x840e92caac50  0x11c   ALPC Port       0x1f0001        -
536     smss.exe        0x840e97584070  0x128   ALPC Port       0x1f0001        -
536     smss.exe        0x840e92cab2c0  0x12c   Process 0x101441        svchost.exe Pid 1124
804     csrss.exe       0x840e91f56960  0x4     Event   0x1f0003        -
804     csrss.exe       0x840e91f56d60  0x8     Event   0x1f0003        -
804     csrss.exe       0x840e909844c0  0xc     WaitCompletionPacket    0x1     -
804     csrss.exe       0x840e91197700  0x10    IoCompletion    0x1f0003        -
804     csrss.exe       0x840e90cada80  0x14    TpWorkerFactory 0xf00ff -
804     csrss.exe       0x840e919d4d60  0x18    IRTimer 0x100002        -
804     csrss.exe       0x840e90984a70  0x1c    WaitCompletionPacket    0x1     -
804     csrss.exe       0x840e919d5b30  0x20    IRTimer 0x100002        -
804     csrss.exe       0x840e90984730  0x24    WaitCompletionPacket    0x1     -
804     csrss.exe       0x840e911986a0  0x28    EtwRegistration 0x804   -
804     csrss.exe       0x840e90b9be80  0x2c    EtwRegistration 0x804   -
804     csrss.exe       0x840e91198080  0x30    EtwRegistration 0x804   -
804     csrss.exe       0x840e91f56460  0x34    Event   0x1f0003        -
804     csrss.exe       0x840e91f564e0  0x38    Event   0x1f0003        -
804     csrss.exe       0x840e920f6e80  0x3c    File    0x100020        \Device\HarddiskVolume3\Windows\System32
804     csrss.exe       0x840e91197380  0x40    IoCompletion    0x1f0003        -
804     csrss.exe       0x840e91ff6120  0x44    TpWorkerFactory 0xf00ff -
804     csrss.exe       0x840e919d5800  0x48    IRTimer 0x100002        -
804     csrss.exe       0x840e90984e80  0x4c    WaitCompletionPacket    0x1     -
804     csrss.exe       0x840e919d5c40  0x50    IRTimer 0x100002        -
804     csrss.exe       0x840e909866e0  0x54    WaitCompletionPacket    0x1     -
804     csrss.exe       0x840e91197600  0x58    EtwRegistration 0x804   -
804     csrss.exe       0xb7080435e740  0x5c    Directory       0xf000f BNOLINKS
804     csrss.exe       0xb70804352410  0x60    SymbolicLink    0xf0001 0
804     csrss.exe       0xb7080435eb00  0x64    Directory       0xf000f 0
804     csrss.exe       0xb7080435d0c0  0x68    Directory       0xf000f DosDevices
804     csrss.exe       0xb708012fe840  0x6c    Directory       0xf000f Windows
804     csrss.exe       0x840e91f6d2d0  0x70    ALPC Port       0x1f0001        -
804     csrss.exe       0x840e90a5a920  0x74    ALPC Port       0x1f0001        ApiPort
804     csrss.exe       0x840e92031240  0x78    Thread  0x1fffff        Tid 820 Pid 804
804     csrss.exe       0xb70804351c80  0x7c    SymbolicLink    0xf0001 Windows
804     csrss.exe       0xb7080435ece0  0x80    Directory       0xf000f Restricted
804     csrss.exe       0xb7080425a8f0  0x84    Section 0xf001f SharedSection
804     csrss.exe       0xb7080435d660  0x88    Directory       0xf000f BaseNamedObjects
804     csrss.exe       0xb708042b6ea0  0x8c    Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER
804     csrss.exe       0xb7080435e920  0x90    Directory       0xf000f AppContainerNamedObjects
804     csrss.exe       0x840e920f58a0  0x94    File    0x1f01ff        \Device\NamedPipe\Sessions\0\AppContainerNamedObjects
804     csrss.exe       0x840e91197de0  0x98    EtwRegistration 0x804   -
804     csrss.exe       0x840e91198be0  0x9c    EtwRegistration 0x804   -
804     csrss.exe       0x840e91fb1f70  0xa0    Semaphore       0x1f0003        SM0:804:304:WilStaging_02_p0h
804     csrss.exe       0x840e911978a0  0xa4    EtwRegistration 0x804   -
804     csrss.exe       0xb7080435d660  0xa8    Directory       0xf     BaseNamedObjects
804     csrss.exe       0x840e9228d080  0xac    Thread  0x1fffff        Tid 884 Pid 880
804     csrss.exe       0x840e91fb1ed0  0xb0    Semaphore       0x1f0003        SM0:804:304:WilStaging_02_p0
804     csrss.exe       0x840e91f79e50  0xb4    Mutant  0x1f0001        SM0:804:304:WilStaging_02
804     csrss.exe       0x840e91197980  0xb8    EtwRegistration 0x804   -
804     csrss.exe       0x840e91197c20  0xbc    EtwRegistration 0x804   -
804     csrss.exe       0xb708042b9c50  0xc0    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NLS\SORTING\VERSIONS
804     csrss.exe       0x840e91198f60  0xc4    EtwRegistration 0x804   -
804     csrss.exe       0x840e91198e80  0xc8    EtwRegistration 0x804   -
804     csrss.exe       0x840e920f6520  0xcc    File    0x100001        \Device\HarddiskVolume3\Windows\System32\en-US\csrss.exe.mui
804     csrss.exe       0xb708016674d0  0xd0    Key     0xf003f MACHINE
804     csrss.exe       0xb708042b8a40  0xd4    Key     0x9     MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\IMAGE FILE EXECUTION OPTIONS
804     csrss.exe       0x840e9228c080  0xd8    Process 0x1fffff        wininit.exe Pid 880
804     csrss.exe       0x840e91197d00  0xdc    EtwRegistration 0x804   -
804     csrss.exe       0x840e91f56de0  0xe0    Event   0x1f0003        -
804     csrss.exe       0x840e90c616a0  0xe4    WaitCompletionPacket    0x1     -
804     csrss.exe       0xb708042b7d80  0xe8    Key     0x20019 MACHINE
804     csrss.exe       0xb708042b6950  0xec    Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\OLE
804     csrss.exe       0x840e91f56e60  0xf0    Event   0x1f0003        -
804     csrss.exe       0xb708042b61e0  0xf4    Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS\SOFTWARE\MICROSOFT
804     csrss.exe       0xb708042b9b40  0xf8    Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS
804     csrss.exe       0x840e91f56ee0  0xfc    Event   0x1f0003        -
804     csrss.exe       0x840e91198cc0  0x100   EtwRegistration 0x804   -
804     csrss.exe       0x840e91198da0  0x104   EtwRegistration 0x804   -
804     csrss.exe       0x840e920fb7c0  0x108   EtwRegistration 0x804   -
804     csrss.exe       0x840e91f56f60  0x10c   Event   0x1f0003        -
804     csrss.exe       0x840e91f56fe0  0x110   Event   0x1f0003        -
804     csrss.exe       0x840e91f57c60  0x114   Event   0x1f0003        -
804     csrss.exe       0x840e91f577e0  0x118   Event   0x1f0003        -
804     csrss.exe       0x840e91f573e0  0x11c   Event   0x1f0003        -
804     csrss.exe       0x840e91f57ce0  0x120   Event   0x1f0003        -
804     csrss.exe       0x840e920fc940  0x124   EtwRegistration 0x804   -
804     csrss.exe       0x840e920fb1a0  0x128   EtwRegistration 0x804   -
804     csrss.exe       0x840e920fbb40  0x12c   EtwRegistration 0x804   -
804     csrss.exe       0x840e920fbc20  0x130   EtwRegistration 0x804   -
804     csrss.exe       0x840e920fc240  0x134   EtwRegistration 0x804   -
804     csrss.exe       0x840e920fcbe0  0x138   EtwRegistration 0x804   -
804     csrss.exe       0x840e920fbec0  0x13c   EtwRegistration 0x804   -
804     csrss.exe       0x840e920f5580  0x140   File    0x120089        \Device\DeviceApi\CMApi
804     csrss.exe       0x840e920fca20  0x144   EtwRegistration 0x804   -
804     csrss.exe       0x840e920fc6a0  0x148   EtwRegistration 0x804   -
804     csrss.exe       0x840e91f57260  0x14c   Event   0x1f0003        -
804     csrss.exe       0x840e91f570e0  0x150   Event   0x1f0003        -
804     csrss.exe       0x840e918ac430  0x154   Event   0x1f0003        EventRitExited
804     csrss.exe       0x840e91f57960  0x158   Event   0x1f0003        -
804     csrss.exe       0x840e91f57360  0x15c   Event   0x1f0003        -
804     csrss.exe       0x840e91f58060  0x160   Event   0x1f0003        -
804     csrss.exe       0x840e91f572e0  0x164   Event   0x1f0003        -
804     csrss.exe       0xb708042b8e80  0x168   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\FONTS
804     csrss.exe       0x840e920f6b60  0x16c   File    0x100001        \Device\HarddiskVolume3\Windows\System32\en-US\winsrv.dll.mui
804     csrss.exe       0x840e918ab710  0x170   Event   0x1f0003        TermSrvReadyEvent
804     csrss.exe       0x840e9223b500  0x174   Thread  0x1fffff        Tid 852 Pid 804
804     csrss.exe       0x840e9225d080  0x178   Thread  0x1fffff        Tid 856 Pid 804
804     csrss.exe       0x840e9224c080  0x17c   Thread  0x1fffff        Tid 860 Pid 804
804     csrss.exe       0x840e9226d080  0x180   Thread  0x1fffff        Tid 864 Pid 804
804     csrss.exe       0x840e92cab2c0  0x184   Process 0x1fffff        svchost.exe Pid 1124
804     csrss.exe       0x840e92c9b080  0x188   Thread  0x1fffff        Tid 1128 Pid 1124
804     csrss.exe       0xb708042b95f0  0x18c   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\WINDOWS\CURRENTVERSION\SIDEBYSIDE
804     csrss.exe       0x840e92174e10  0x190   ALPC Port       0x1f0001        SbApiPort
804     csrss.exe       0x840e9226e080  0x194   Thread  0x1fffff        Tid 868 Pid 804
804     csrss.exe       0xb708042b93d0  0x198   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\PRIORITYCONTROL
804     csrss.exe       0x840e9107a070  0x19c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92015070  0x1a0   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e91f58360  0x1a4   Event   0x1f0003        -
804     csrss.exe       0x840e920fc500  0x1a8   IoCompletion    0x1f0003        -
804     csrss.exe       0x840e922c5080  0x1ac   Thread  0x1fffff        Tid 924 Pid 804
804     csrss.exe       0x840e922de080  0x1b0   Thread  0x1fffff        Tid 948 Pid 804
804     csrss.exe       0x840e92298960  0x1b4   Event   0x1f0003        -
804     csrss.exe       0x840e922d7dc0  0x1b8   IoCompletion    0x1f0003        -
804     csrss.exe       0x840e922df080  0x1bc   Thread  0x1fffff        Tid 952 Pid 804
804     csrss.exe       0x840e92298560  0x1c0   Event   0x1f0003        -
804     csrss.exe       0x840e922ede00  0x1c4   IoCompletion    0x1f0003        -
804     csrss.exe       0x840e92298460  0x1c8   Event   0x1f0003        -
804     csrss.exe       0x840e922981e0  0x1cc   Event   0x1f0003        -
804     csrss.exe       0x840e911dc180  0x1d0   Timer   0x1f0003        -
804     csrss.exe       0x840e911dc330  0x1d4   Timer   0x1f0003        -
804     csrss.exe       0x840e922982e0  0x1d8   Event   0x1f0003        -
804     csrss.exe       0x840e922985e0  0x1dc   Event   0x1f0003        -
804     csrss.exe       0x840e922983e0  0x1e0   Event   0x1f0003        -
804     csrss.exe       0x840e92298760  0x1e4   Event   0x1f0003        -
804     csrss.exe       0x840e922dea80  0x1e8   RawInputManager 0x3     -
804     csrss.exe       0x840e922eaee0  0x1ec   Event   0x1f0003        WinSta0_DesktopSwitch
804     csrss.exe       0x840e92299ee0  0x1f0   Event   0x1f0003        -
804     csrss.exe       0x840e922999e0  0x1f4   Event   0x1f0003        -
804     csrss.exe       0x840e902995b0  0x1f8   Timer   0x1f0003        -
804     csrss.exe       0x840e9029a9f0  0x1fc   Timer   0x1f0003        -
804     csrss.exe       0x840e9029aba0  0x200   Timer   0x1f0003        -
804     csrss.exe       0x840e911dc4e0  0x204   Timer   0x1f0003        -
804     csrss.exe       0x840e922fb250  0x208   Timer   0x1f0003        -
804     csrss.exe       0x840e922fb400  0x20c   Timer   0x1f0003        -
804     csrss.exe       0x840e9229a0e0  0x210   Event   0x1f0003        -
804     csrss.exe       0x840e92299e60  0x214   Event   0x1f0003        -
804     csrss.exe       0x840e92299ae0  0x218   Event   0x1f0003        -
804     csrss.exe       0x840e92299460  0x21c   Event   0x1f0003        -
804     csrss.exe       0x840e92299de0  0x220   Event   0x1f0003        -
804     csrss.exe       0x840e92299660  0x224   Event   0x1f0003        -
804     csrss.exe       0x840e91f6cc00  0x228   RawInputManager 0x3     -
804     csrss.exe       0x840e92299160  0x22c   Event   0x1f0003        -
804     csrss.exe       0x840e922991e0  0x230   Event   0x1f0003        -
804     csrss.exe       0x840e922fc840  0x234   Timer   0x1f0003        -
804     csrss.exe       0x840e922998e0  0x238   Event   0x1f0003        -
804     csrss.exe       0x840e92299360  0x23c   Event   0x1f0003        -
804     csrss.exe       0x840e922994e0  0x240   Event   0x1f0003        -
804     csrss.exe       0x840e9228b900  0x244   RawInputManager 0x3     -
804     csrss.exe       0x840e920d4920  0x248   WindowStation   0xf0003 WinSta0
804     csrss.exe       0x840e9230b100  0x24c   Process 0x1fffff        services.exe Pid 1000
804     csrss.exe       0x840e992bb340  0x250   IoCompletion    0x1f0003        -
804     csrss.exe       0x840e922708a0  0x254   EtwRegistration 0x804   -
804     csrss.exe       0x840e9229da60  0x258   Event   0x1f0003        -
804     csrss.exe       0x840e922eb140  0x25c   Mutant  0x1f0001        -
804     csrss.exe       0x840e922714e0  0x260   EtwRegistration 0x804   -
804     csrss.exe       0xb708044574a0  0x264   Section 0xf0007 -
804     csrss.exe       0x840e92270440  0x268   EtwRegistration 0x804   -
804     csrss.exe       0x840e92271080  0x26c   IoCompletion    0x1f0003        -
804     csrss.exe       0x840e922f8090  0x270   File    0x100001        \Device\CNG
804     csrss.exe       0x840e92349080  0x274   Process 0x1fffff        lsass.exe Pid 548
804     csrss.exe       0xb7082454c810  0x278   Section 0xf0007 -
804     csrss.exe       0x840e922bba80  0x27c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92c9cd00  0x280   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e923de2c0  0x284   Process 0x1fffff        svchost.exe Pid 756
804     csrss.exe       0x840e923ca080  0x288   Thread  0x1fffff        Tid 752 Pid 756
804     csrss.exe       0xb708292355d0  0x290   Section 0xf0007 -
804     csrss.exe       0x840e9226ea80  0x294   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9cf72080  0x298   Thread  0x1fffff        Tid 4924 Pid 15932
804     csrss.exe       0x840e92cdf070  0x29c   ALPC Port       0x1f0001        -
804     csrss.exe       0xb70811b50370  0x2a0   Section 0xf0007 -
804     csrss.exe       0x840e923caa80  0x2a4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e91812cb0  0x2a8   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e923f61c0  0x2ac   Process 0x1fffff        fontdrvhost.ex Pid 892
804     csrss.exe       0x840e923f0080  0x2b0   Thread  0x1fffff        Tid 876 Pid 892
804     csrss.exe       0x840e923edce0  0x2b4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92c9a340  0x2b8   Process 0x1fffff        svchost.exe Pid 1076
804     csrss.exe       0x840e92313080  0x2bc   Thread  0x1fffff        Tid 956 Pid 804
804     csrss.exe       0x840e923ab960  0x2c0   Event   0x1f0003        -
804     csrss.exe       0x840e923c1f00  0x2c4   IoCompletion    0x1f0003        -
804     csrss.exe       0x840e923f2080  0x2c8   Thread  0x1fffff        Tid 1080 Pid 1076
804     csrss.exe       0x840e98dd4080  0x2d0   Thread  0x1fffff        Tid 8692 Pid 8688
804     csrss.exe       0x840e9ea1e080  0x2d4   Thread  0x1fffff        Tid 13372 Pid 756
804     csrss.exe       0x840e92d48080  0x2d8   Process 0x1fffff        svchost.exe Pid 1364
804     csrss.exe       0x840e97e8c080  0x2dc   Process 0x1fffff        svchost.exe Pid 3864
804     csrss.exe       0x840e92def080  0x2e0   Process 0x1fffff        svchost.exe Pid 1292
804     csrss.exe       0x840e92dcb080  0x2e4   Thread  0x1fffff        Tid 1296 Pid 1292
804     csrss.exe       0xb70824243490  0x2e8   Section 0xf0007 -
804     csrss.exe       0x840e92d26080  0x2ec   Thread  0x1fffff        Tid 1324 Pid 1320
804     csrss.exe       0x840e92d2e240  0x2f0   Process 0x1fffff        svchost.exe Pid 1320
804     csrss.exe       0x840e92d29de0  0x2f4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92ded080  0x2f8   Thread  0x1fffff        Tid 1368 Pid 1364
804     csrss.exe       0x840e92e2b080  0x2fc   Thread  0x1fffff        Tid 1396 Pid 1392
804     csrss.exe       0x840e975a5080  0x300   Thread  0x1fffff        Tid 4516 Pid 324
804     csrss.exe       0x840e9968d080  0x304   Process 0x1fffff        splunkd.exe Pid 13620
804     csrss.exe       0x840e903a2d40  0x308   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92e30080  0x30c   Process 0x1fffff        svchost.exe Pid 1392
804     csrss.exe       0x840e92d25a80  0x310   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e98be3d90  0x314   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92e28080  0x318   Thread  0x1fffff        Tid 1412 Pid 1408
804     csrss.exe       0x840e92e04300  0x31c   Process 0x1fffff        svchost.exe Pid 1408
804     csrss.exe       0x840e92325de0  0x320   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92ef7ce0  0x324   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9694bcd0  0x328   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92e2bce0  0x32c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92e27a80  0x330   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92e48080  0x334   Process 0x1fffff        svchost.exe Pid 1524
804     csrss.exe       0x840e92e2f080  0x338   Thread  0x1fffff        Tid 1528 Pid 1524
804     csrss.exe       0x840e92e68a80  0x33c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92e732c0  0x340   Process 0x1fffff        svchost.exe Pid 1580
804     csrss.exe       0x840e99d641c0  0x344   Thread  0x1fffff        Tid 10408 Pid 13156
804     csrss.exe       0x840e92e67080  0x348   Thread  0x1fffff        Tid 1584 Pid 1580
804     csrss.exe       0x840e92309a80  0x34c   ALPC Port       0x1f0001        -
804     csrss.exe       0xb7080a764af0  0x350   Section 0xf0007 -
804     csrss.exe       0x840e92eaa2c0  0x358   Process 0x1fffff        svchost.exe Pid 1604
804     csrss.exe       0x840e92e47380  0x35c   Thread  0x1fffff        Tid 1608 Pid 1604
804     csrss.exe       0x840e92eac340  0x360   Process 0x1fffff        svchost.exe Pid 1612
804     csrss.exe       0x840e92ebc080  0x364   Thread  0x1fffff        Tid 1616 Pid 1612
804     csrss.exe       0x840e92eb1a80  0x368   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92eb2080  0x36c   Thread  0x1fffff        Tid 1656 Pid 1652
804     csrss.exe       0x840e92f04340  0x370   Process 0x1fffff        svchost.exe Pid 1652
804     csrss.exe       0x840e996ea080  0x374   Thread  0x1fffff        Tid 4184 Pid 8312
804     csrss.exe       0x840e92f202c0  0x378   Process 0x1fffff        svchost.exe Pid 1764
804     csrss.exe       0x840e92eb9d20  0x37c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92ef9080  0x380   Thread  0x1fffff        Tid 1768 Pid 1764
804     csrss.exe       0x840e92f23080  0x384   Process 0x1fffff        svchost.exe Pid 1776
804     csrss.exe       0x840e92ef7080  0x388   Thread  0x1fffff        Tid 1780 Pid 1776
804     csrss.exe       0x840e9c13a080  0x38c   Thread  0x1fffff        Tid 7508 Pid 13156
804     csrss.exe       0x840e98c92080  0x390   Thread  0x1fffff        Tid 6636 Pid 15568
804     csrss.exe       0x840e974f7340  0x394   Process 0x1fffff        svchost.exe Pid 15932
804     csrss.exe       0x840e92ef8ce0  0x398   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92f27080  0x39c   Process 0x1fffff        svchost.exe Pid 1808
804     csrss.exe       0x840e92ef1080  0x3a0   Thread  0x1fffff        Tid 1812 Pid 1808
804     csrss.exe       0x840e96f5a080  0x3a4   Process 0x1fffff        SenseNdr.exe Pid 15568
804     csrss.exe       0x840e99587080  0x3a8   Thread  0x1fffff        Tid 2236 Pid 10224
804     csrss.exe       0x840e96e411c0  0x3ac   Thread  0x1fffff        Tid 5780 Pid 5804
804     csrss.exe       0x840e92f7adc0  0x3b4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92fc6080  0x3b8   Process 0x1fffff        svchost.exe Pid 1964
804     csrss.exe       0x840e98ceb080  0x3bc   Thread  0x1fffff        Tid 12388 Pid 756
804     csrss.exe       0x840e92fed080  0x3c0   Thread  0x1fffff        Tid 1968 Pid 1964
804     csrss.exe       0xb7083023f570  0x3c4   Section 0xf0007 -
804     csrss.exe       0x840e92feb080  0x3c8   Thread  0x1fffff        Tid 1980 Pid 1976
804     csrss.exe       0x840e92fc5300  0x3cc   Process 0x1fffff        svchost.exe Pid 1976
804     csrss.exe       0x840e96631a80  0x3d0   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9db76300  0x3d4   Thread  0x1fffff        Tid 12868 Pid 10144
804     csrss.exe       0xb70824543350  0x3d8   Section 0xf0007 -
804     csrss.exe       0x840e92fc7080  0x3dc   Process 0x1fffff        svchost.exe Pid 1988
804     csrss.exe       0x840e96630080  0x3e0   Thread  0x1fffff        Tid 1992 Pid 1988
804     csrss.exe       0x840e92fabd90  0x3e4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92fa7d90  0x3e8   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96630a80  0x3ec   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92fbb0c0  0x3f0   Process 0x1fffff        svchost.exe Pid 604
804     csrss.exe       0x840e92fc8080  0x3f4   Thread  0x1fffff        Tid 696 Pid 604
804     csrss.exe       0x840e91ed1080  0x3f8   Thread  0x1fffff        Tid 12860 Pid 13752
804     csrss.exe       0x840e9c250080  0x3fc   Process 0x1fffff        SenseIR.exe Pid 7728
804     csrss.exe       0x840e966d62c0  0x404   Process 0x1fffff        svchost.exe Pid 2092
804     csrss.exe       0x840e966b8080  0x408   Thread  0x1fffff        Tid 2096 Pid 2092
804     csrss.exe       0x840e966db080  0x40c   Thread  0x1fffff        Tid 2188 Pid 804
804     csrss.exe       0x840e966df080  0x410   Thread  0x1fffff        Tid 2164 Pid 2160
804     csrss.exe       0x840e96769240  0x414   Process 0x1fffff        svchost.exe Pid 2160
804     csrss.exe       0x840e966c2d20  0x418   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9676e080  0x41c   Process 0x1fffff        svchost.exe Pid 2168
804     csrss.exe       0x840e966de080  0x420   Thread  0x1fffff        Tid 2172 Pid 2168
804     csrss.exe       0x840e96625a80  0x424   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96bc00c0  0x428   Process 0x1fffff        svchost.exe Pid 2848
804     csrss.exe       0x840e9976f310  0x42c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9676b300  0x430   Process 0x1fffff        svchost.exe Pid 2180
804     csrss.exe       0x840e966dc080  0x434   Thread  0x1fffff        Tid 2184 Pid 2180
804     csrss.exe       0x840e92f6bce0  0x438   Event   0x1f0003        -
804     csrss.exe       0x840e96794080  0x43c   Process 0x1fffff        svchost.exe Pid 2276
804     csrss.exe       0x840e966eb8c0  0x440   IoCompletion    0x1f0003        -
804     csrss.exe       0x840e966bba80  0x444   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e966e1a80  0x448   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9677e080  0x44c   Thread  0x1fffff        Tid 2280 Pid 2276
804     csrss.exe       0x840e96bc5ce0  0x450   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e967ec080  0x454   Thread  0x1fffff        Tid 2380 Pid 2376
804     csrss.exe       0x840e96862080  0x458   Process 0x1fffff        svchost.exe Pid 2376
804     csrss.exe       0x840e96790070  0x45c   ALPC Port       0x1f0001        -
804     csrss.exe       0xb7082c5474d0  0x460   Section 0xf0007 -
804     csrss.exe       0xb70824246690  0x464   Section 0xf0007 -
804     csrss.exe       0x840ea0de1080  0x468   Thread  0x1fffff        Tid 10072 Pid 13620
804     csrss.exe       0x840e96860080  0x46c   Process 0x1fffff        svchost.exe Pid 2396
804     csrss.exe       0x840e96864380  0x470   Process 0x1fffff        svchost.exe Pid 2384
804     csrss.exe       0x840e9689a080  0x474   Process 0x1fffff        svchost.exe Pid 2472
804     csrss.exe       0x840e9685c080  0x478   Thread  0x1fffff        Tid 2400 Pid 2396
804     csrss.exe       0x840e9685e080  0x47c   Thread  0x1fffff        Tid 2388 Pid 2384
804     csrss.exe       0xb708303023f0  0x480   Section 0xf0007 -
804     csrss.exe       0x840e96779900  0x484   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9688b080  0x488   Thread  0x1fffff        Tid 2476 Pid 2472
804     csrss.exe       0x840e9687d7d0  0x48c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9687d570  0x490   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96886080  0x494   Thread  0x1fffff        Tid 2500 Pid 2496
804     csrss.exe       0x840e968c00c0  0x498   Process 0x1fffff        svchost.exe Pid 2496
804     csrss.exe       0x840e96bafdf0  0x49c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96bab0c0  0x4a0   Thread  0x1fffff        Tid 2900 Pid 2504
804     csrss.exe       0x840e96c959a0  0x4a4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e968c22c0  0x4a8   Process 0x1fffff        svchost.exe Pid 2504
804     csrss.exe       0x840e9685f080  0x4ac   Thread  0x1fffff        Tid 2508 Pid 2504
804     csrss.exe       0x840e96930ce0  0x4b0   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e968982d0  0x4b4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96896ce0  0x4b8   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e969b3080  0x4c0   Thread  0x1fffff        Tid 2852 Pid 2848
804     csrss.exe       0x840e96d3b080  0x4c4   Thread  0x1fffff        Tid 5712 Pid 5772
804     csrss.exe       0x840e96d9f080  0x4cc   Process 0x1fffff        svchost.exe Pid 3308
804     csrss.exe       0xb7083023e350  0x4d0   Section 0xf0007 -
804     csrss.exe       0x840e97442080  0x4d4   Process 0x1fffff        CSFalconContai Pid 6624
804     csrss.exe       0xb708241c9170  0x4d8   Section 0xf0007 -
804     csrss.exe       0x840e96c23280  0x4dc   Process 0x1fffff        spoolsv.exe Pid 2904
804     csrss.exe       0x840e96c211c0  0x4e0   Thread  0x1fffff        Tid 2908 Pid 2904
804     csrss.exe       0x840e99474080  0x4e4   Process 0x1fffff        PSEXESVC.exe Pid 11296
804     csrss.exe       0xb708211066d0  0x4e8   Section 0xf0007 -
804     csrss.exe       0x840e9b1a5640  0x4ec   Thread  0x1fffff        Tid 11292 Pid 16024
804     csrss.exe       0x840e96c5e1c0  0x4f0   Process 0x1fffff        svchost.exe Pid 2220
804     csrss.exe       0x840e96bba070  0x4f8   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9944e080  0x4fc   Thread  0x1fffff        Tid 16220 Pid 6988
804     csrss.exe       0x840e96c26080  0x500   Thread  0x1fffff        Tid 2232 Pid 2220
804     csrss.exe       0x840e96c582c0  0x504   Process 0x1fffff        svchost.exe Pid 3028
804     csrss.exe       0x840e96c561c0  0x508   Thread  0x1fffff        Tid 3032 Pid 3028
804     csrss.exe       0x840e96c4bcd0  0x510   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96c91300  0x514   Thread  0x1fffff        Tid 2372 Pid 3028
804     csrss.exe       0x840e96bc1d80  0x518   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96c4dce0  0x51c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96c61080  0x520   Process 0x1fffff        svchost.exe Pid 2216
804     csrss.exe       0x840e96c60380  0x524   Thread  0x1fffff        Tid 2348 Pid 2216
804     csrss.exe       0x840e96cdd1c0  0x528   Process 0x1fffff        svchost.exe Pid 740
804     csrss.exe       0x840e976baac0  0x52c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96cdf380  0x530   Thread  0x1fffff        Tid 2468 Pid 740
804     csrss.exe       0x840e96c28a80  0x534   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96da72c0  0x538   Process 0x1fffff        svchost.exe Pid 3324
804     csrss.exe       0x840e96da0080  0x53c   Thread  0x1fffff        Tid 3312 Pid 3308
804     csrss.exe       0x840e96da51c0  0x548   Thread  0x1fffff        Tid 3328 Pid 3324
804     csrss.exe       0x840e96da9300  0x54c   Process 0x1fffff        svchost.exe Pid 3332
804     csrss.exe       0x840e96d5d080  0x550   Thread  0x1fffff        Tid 3336 Pid 3332
804     csrss.exe       0x840e96cf7070  0x554   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96e350c0  0x558   Process 0x1fffff        svchost.exe Pid 3480
804     csrss.exe       0x840e9078d080  0x55c   Thread  0x1fffff        Tid 3524 Pid 1852
804     csrss.exe       0x840e96dc4080  0x560   Process 0x1fffff        svchost.exe Pid 3352
804     csrss.exe       0x840e96d5b9f0  0x564   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96dc3380  0x568   Thread  0x1fffff        Tid 3356 Pid 3352
804     csrss.exe       0x840e99335360  0x56c   Event   0x1f0003        -
804     csrss.exe       0x840e96dcc1c0  0x570   Thread  0x1fffff        Tid 3380 Pid 3376
804     csrss.exe       0x840e96e042c0  0x574   Process 0x1fffff        svchost.exe Pid 3376
804     csrss.exe       0x840e96db8a80  0x578   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96bb8a80  0x57c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e97dc72c0  0x580   Process 0x1fffff        SearchIndexer. Pid 1852
804     csrss.exe       0x840e96e39340  0x584   Thread  0x1fffff        Tid 3484 Pid 3480
804     csrss.exe       0x840e96e50080  0x588   Thread  0x1fffff        Tid 3544 Pid 3540
804     csrss.exe       0x840e96e44300  0x58c   Process 0x1fffff        MsMpEng.exe Pid 3540
804     csrss.exe       0x840e96e20080  0x594   Process 0x1fffff        svchost.exe Pid 3488
804     csrss.exe       0x840e96e3a080  0x598   Thread  0x1fffff        Tid 3492 Pid 3488
804     csrss.exe       0xb7082c548790  0x5a0   Section 0xf0007 -
804     csrss.exe       0x840e97f3aa80  0x5a4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96d421c0  0x5a8   Thread  0x1fffff        Tid 5224 Pid 5220
804     csrss.exe       0xb70817948c50  0x5ac   Section 0xf0007 -
804     csrss.exe       0x840e976a2300  0x5b0   Process 0x1fffff        unsecapp.exe Pid 5220
804     csrss.exe       0xb7082ec290d0  0x5b4   Section 0xf0007 -
804     csrss.exe       0x840e973cb080  0x5b8   Thread  0x1fffff        Tid 4804 Pid 4800
804     csrss.exe       0x840e97ca5d00  0x5bc   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96e08080  0x5c0   Thread  0x1fffff        Tid 3536 Pid 804
804     csrss.exe       0x840e96e3d080  0x5c4   Process 0x1fffff        svchost.exe Pid 3528
804     csrss.exe       0x840e96e1b080  0x5c8   Thread  0x1fffff        Tid 3532 Pid 3528
804     csrss.exe       0x840e96dd26e0  0x5cc   Event   0x1f0003        -
804     csrss.exe       0x840e96d1d540  0x5d0   IoCompletion    0x1f0003        -
804     csrss.exe       0x840e98f19080  0x5d4   Thread  0x1fffff        Tid 9568 Pid 3864
804     csrss.exe       0xb70817947530  0x5d8   Section 0xf0007 -
804     csrss.exe       0x840e96e10d90  0x5dc   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96e552c0  0x5e0   Process 0x1fffff        svchost.exe Pid 3560
804     csrss.exe       0x840e96e56080  0x5e4   Thread  0x1fffff        Tid 3564 Pid 3560
804     csrss.exe       0x840e96e06d90  0x5e8   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96e7ec20  0x5ec   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96e70530  0x5f0   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96e88370  0x5f4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96e6e790  0x5f8   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96e77080  0x5fc   Process 0x1fffff        svchost.exe Pid 3636
804     csrss.exe       0x840e96ea9380  0x600   Thread  0x1fffff        Tid 3640 Pid 3636
804     csrss.exe       0x840e96e0ecc0  0x604   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96f1d300  0x608   Process 0x1fffff        dasHost.exe Pid 3752
804     csrss.exe       0x840e995680c0  0x60c   Process 0x103800        SgrmBroker.exe Pid 9948
804     csrss.exe       0x840e96f1e080  0x614   Thread  0x1fffff        Tid 3756 Pid 3752
804     csrss.exe       0x840e96fb2080  0x618   Thread  0x1fffff        Tid 3892 Pid 3540
804     csrss.exe       0xb7082c5483d0  0x620   Section 0xf0007 -
804     csrss.exe       0x840e96f6d2c0  0x624   Process 0x1fffff        svchost.exe Pid 3880
804     csrss.exe       0x840e99e240c0  0x628   Process 0x1fffff        ScreenConnect. Pid 6988
804     csrss.exe       0x840e96f0b9f0  0x62c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96f6e080  0x630   Thread  0x1fffff        Tid 3884 Pid 3880
804     csrss.exe       0x840e96f641c0  0x634   Thread  0x1fffff        Tid 3852 Pid 804
804     csrss.exe       0x840e96e0d080  0x638   Process 0x1fffff        svchost.exe Pid 3844
804     csrss.exe       0x840e96f54080  0x63c   Thread  0x1fffff        Tid 3848 Pid 3844
804     csrss.exe       0x840e90b0a680  0x640   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96de8260  0x644   Event   0x1f0003        -
804     csrss.exe       0x840e96ed1340  0x648   IoCompletion    0x1f0003        -
804     csrss.exe       0xb70824558930  0x64c   Section 0xf0007 -
804     csrss.exe       0x840e9715d080  0x650   Process 0x1fffff        Sysmon64.exe Pid 3696
804     csrss.exe       0x840e96e8c2f0  0x654   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9df2d080  0x658   Thread  0x1fffff        Tid 13572 Pid 1000
804     csrss.exe       0x840e96fce080  0x65c   Thread  0x1fffff        Tid 3952 Pid 3844
804     csrss.exe       0x840e96f22a80  0x660   ALPC Port       0x1f0001        -
804     csrss.exe       0xb70830242ef0  0x668   Section 0xf0007 -
804     csrss.exe       0x840e91e15dc0  0x66c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e970f3080  0x670   Process 0x1fffff        svchost.exe Pid 3172
804     csrss.exe       0x840e970ae080  0x674   Thread  0x1fffff        Tid 3136 Pid 3172
804     csrss.exe       0xb708303243b0  0x678   Section 0xf0007 -
804     csrss.exe       0x840e96fb9070  0x67c   ALPC Port       0x1f0001        -
804     csrss.exe       0xb708242447f0  0x680   Section 0xf0007 -
804     csrss.exe       0x840e972491c0  0x684   Process 0x1fffff        svchost.exe Pid 4352
804     csrss.exe       0xb70824558cf0  0x690   Section 0xf0007 -
804     csrss.exe       0xb70807735180  0x694   Section 0xf0007 -
804     csrss.exe       0xb7080a75a2d0  0x698   Section 0xf0007 -
804     csrss.exe       0x840e9712d080  0x69c   Thread  0x1fffff        Tid 3496 Pid 3696
804     csrss.exe       0xb70824548e90  0x6a0   Section 0xf0007 -
804     csrss.exe       0x840e96fd0a80  0x6a4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e97159300  0x6a8   Process 0x1fffff        CSFalconServic Pid 3648
804     csrss.exe       0x840e96e38080  0x6ac   Thread  0x1fffff        Tid 3768 Pid 3648
804     csrss.exe       0xb7082336dad0  0x6b0   Section 0xf0007 -
804     csrss.exe       0x840e9e3f7080  0x6b4   Thread  0x1fffff        Tid 8384 Pid 14028
804     csrss.exe       0x840e9ccec080  0x6b8   Process 0x1fffff        svchost.exe Pid 11776
804     csrss.exe       0x840e971bad20  0x6bc   ALPC Port       0x1f0001        -
804     csrss.exe       0xb708142f1470  0x6c0   Section 0xf0007 -
804     csrss.exe       0x840e973d81c0  0x6c8   Process 0x1fffff        svchost.exe Pid 4800
804     csrss.exe       0x840e9723b080  0x6cc   Thread  0x1fffff        Tid 4356 Pid 4352
804     csrss.exe       0x840e92ca5080  0x6d4   Thread  0x1fffff        Tid 6600 Pid 6604
804     csrss.exe       0xb708142e7e70  0x6d8   Section 0xf0007 -
804     csrss.exe       0x840e971f5a80  0x6dc   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e96f65b60  0x6e0   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e91c3b300  0x6e4   Process 0x1fffff        svchost.exe Pid 8312
804     csrss.exe       0x840e9dc3e040  0x6e8   Thread  0x1fffff        Tid 13552 Pid 10144
804     csrss.exe       0xb7082454beb0  0x6f0   Section 0xf0007 -
804     csrss.exe       0x840e96e88880  0x6f4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e974b4080  0x6f8   Process 0x1fffff        CSFalconContai Pid 324
804     csrss.exe       0x840e97232080  0x6fc   Thread  0x1fffff        Tid 4256 Pid 3648
804     csrss.exe       0x840e99416dc0  0x700   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9783b080  0x704   Process 0x1fffff        CSFalconContai Pid 6604
804     csrss.exe       0x840e98cb8580  0x708   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92307080  0x70c   Thread  0x1fffff        Tid 6620 Pid 6624
804     csrss.exe       0xb708303070d0  0x710   Section 0xf0007 -
804     csrss.exe       0xb7080a947b30  0x714   Section 0xf0007 -
804     csrss.exe       0x840e9f40c340  0x718   Process 0x1fffff        svchost.exe Pid 2800
804     csrss.exe       0x840e9e511080  0x724   Thread  0x1fffff        Tid 15572 Pid 12008
804     csrss.exe       0x840e97014500  0x728   Thread  0x1fffff        Tid 5348 Pid 5340
804     csrss.exe       0xb7082480cad0  0x72c   Section 0xf0007 -
804     csrss.exe       0xb70824813fb0  0x730   Section 0xf0007 -
804     csrss.exe       0xb708248066d0  0x734   Section 0xf0007 -
804     csrss.exe       0xb70830303a70  0x73c   Section 0xf0007 -
804     csrss.exe       0xb708292362f0  0x740   Section 0xf0007 -
804     csrss.exe       0x840e96c25de0  0x744   ALPC Port       0x1f0001        -
804     csrss.exe       0xb708055be640  0x74c   Section 0xf0007 -
804     csrss.exe       0x840e9cc7c080  0x750   Thread  0x1fffff        Tid 8488 Pid 10392
804     csrss.exe       0xb7082ec441f0  0x754   Section 0xf0007 -
804     csrss.exe       0x840e96f2ea80  0x758   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e99a89080  0x75c   Process 0x1fffff        svchost.exe Pid 11756
804     csrss.exe       0xb708142b8850  0x764   Section 0xf0007 -
804     csrss.exe       0xb7080a745150  0x768   Section 0xf0007 -
804     csrss.exe       0xb7080a7665d0  0x76c   Section 0xf0007 -
804     csrss.exe       0xb7080875ba70  0x770   Section 0xf0007 -
804     csrss.exe       0x840e9f3245c0  0x774   Thread  0x1fffff        Tid 14500 Pid 6988
804     csrss.exe       0x840e9ee430c0  0x778   Process 0x1fffff        powershell.exe Pid 13156
804     csrss.exe       0x840e99697070  0x77c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9972c040  0x784   Thread  0x1fffff        Tid 16056 Pid 804
804     csrss.exe       0x840e91c2a080  0x788   Process 0x1fffff        svchost.exe Pid 5340
804     csrss.exe       0x840e9f406080  0x78c   Thread  0x1fffff        Tid 900 Pid 756
804     csrss.exe       0x840e97f7e080  0x794   Process 0x1fffff        svchost.exe Pid 5772
804     csrss.exe       0x840e993202c0  0x798   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e98e59900  0x79c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e978ed080  0x7a0   Process 0x1fffff        cmd.exe Pid 13752
804     csrss.exe       0xb70829239130  0x7a4   Section 0xf0007 -
804     csrss.exe       0x840e97f3a080  0x7a8   Thread  0x1fffff        Tid 3000 Pid 1192
804     csrss.exe       0x840e96f29de0  0x7ac   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e91c36310  0x7b0   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e99d46080  0x7b4   Thread  0x1fffff        Tid 10872 Pid 11596
804     csrss.exe       0x840e977f0080  0x7b8   Thread  0x1fffff        Tid 7084 Pid 3648
804     csrss.exe       0x840e97dcab80  0x7bc   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e97f80080  0x7c0   Process 0x1fffff        AggregatorHost Pid 5804
804     csrss.exe       0xb70824543850  0x7c8   Section 0xf0007 -
804     csrss.exe       0xb7080a769cd0  0x7cc   Section 0xf0007 -
804     csrss.exe       0xb70824546870  0x7d0   Section 0xf0007 -
804     csrss.exe       0x840e91c06080  0x7d4   Thread  0x1fffff        Tid 8028 Pid 1552
804     csrss.exe       0x840ea17e9080  0x7d8   Process 0x1fffff        svchost.exe Pid 10392
804     csrss.exe       0x840e90b062d0  0x7dc   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9741d2d0  0x7e0   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e97be6080  0x7e4   Thread  0x1fffff        Tid 16080 Pid 4672
804     csrss.exe       0xb70817949970  0x7ec   Section 0xf0007 -
804     csrss.exe       0x840e971fa080  0x7f0   Thread  0x1fffff        Tid 4192 Pid 3648
804     csrss.exe       0x840e974b7080  0x7f8   Process 0x1fffff        CSFalconContai Pid 5980
804     csrss.exe       0x840e974020c0  0x7fc   Thread  0x1fffff        Tid 5984 Pid 5980
804     csrss.exe       0x840e97add1c0  0x804   Process 0x1fffff        svchost.exe Pid 6020
804     csrss.exe       0x840e97acd080  0x808   Thread  0x1fffff        Tid 6024 Pid 6020
804     csrss.exe       0x840e9c0ee080  0x80c   Thread  0x1fffff        Tid 10736 Pid 7964
804     csrss.exe       0xb70824250410  0x810   Section 0xf0007 -
804     csrss.exe       0x840ea7c570c0  0x814   Thread  0x1fffff        Tid 9156 Pid 11408
804     csrss.exe       0x840e97ad4ce0  0x818   ALPC Port       0x1f0001        -
804     csrss.exe       0xb70823b8ba70  0x81c   Section 0xf0007 -
804     csrss.exe       0xb70830559a10  0x820   Section 0xf0007 -
804     csrss.exe       0x840e975ad080  0x824   Process 0x1fffff        CSFalconContai Pid 1192
804     csrss.exe       0x840e9eec7080  0x828   Thread  0x1fffff        Tid 14076 Pid 10224
804     csrss.exe       0xb7080a7645f0  0x82c   Section 0xf0007 -
804     csrss.exe       0x840e976c8080  0x834   Thread  0x1fffff        Tid 5400 Pid 3696
804     csrss.exe       0x840e9752ace0  0x838   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e99380070  0x83c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e91ba00c0  0x840   Process 0x1fffff        svchost.exe Pid 7220
804     csrss.exe       0x840e92cdf2d0  0x84c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92e3a0c0  0x850   Thread  0x1fffff        Tid 412 Pid 1852
804     csrss.exe       0xb70824247630  0x854   Section 0xf0007 -
804     csrss.exe       0x840e9741ba80  0x85c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e90bb1ae0  0x860   Event   0x1f0003        -
804     csrss.exe       0x840e90de2080  0x874   Process 0x1fffff        svchost.exe Pid 1480
804     csrss.exe       0xb708142e61b0  0x878   Section 0xf0007 -
804     csrss.exe       0x840e91ae8480  0x87c   Thread  0x1fffff        Tid 1492 Pid 1480
804     csrss.exe       0xb708142ba5b0  0x888   Section 0xf0007 -
804     csrss.exe       0xb70824544070  0x890   Section 0xf0007 -
804     csrss.exe       0xb7083030a0f0  0x894   Section 0xf0007 -
804     csrss.exe       0xb70823b8d0f0  0x898   Section 0xf0007 -
804     csrss.exe       0xb708236bcb10  0x89c   Section 0xf0007 -
804     csrss.exe       0x840e98be3080  0x8a0   Thread  0x1fffff        Tid 3956 Pid 10224
804     csrss.exe       0x840e96d56340  0x8a4   Process 0x1fffff        SenseTVM.exe Pid 1552
804     csrss.exe       0x840e9e6f3080  0x8a8   Process 0x1fffff        excel-update.e Pid 4672
804     csrss.exe       0x840e970b3080  0x8ac   Thread  0x1fffff        Tid 9260 Pid 10224
804     csrss.exe       0x840e98c63080  0x8b0   Thread  0x1fffff        Tid 8788 Pid 7220
804     csrss.exe       0x840e98e742c0  0x8b4   Process 0x1fffff        svchost.exe Pid 8292
804     csrss.exe       0x840e97586080  0x8b8   Thread  0x1fffff        Tid 1112 Pid 6356
804     csrss.exe       0x840e982cc810  0x8bc   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9b03b080  0x8c0   Thread  0x1fffff        Tid 12884 Pid 11296
804     csrss.exe       0xb708142d08b0  0x8c4   Section 0xf0007 -
804     csrss.exe       0x840e98dbf2c0  0x8c8   Process 0x1fffff        svchost.exe Pid 8688
804     csrss.exe       0x840e96f6a080  0x8cc   Thread  0x1fffff        Tid 13696 Pid 10144
804     csrss.exe       0x840e9c215080  0x8d0   Process 0x1fffff        conhost.exe Pid 1600
804     csrss.exe       0xb7080a94bf50  0x8d4   Section 0xf0007 -
804     csrss.exe       0x840e9f2f2080  0x8d8   Thread  0x1fffff        Tid 7376 Pid 2800
804     csrss.exe       0x840e995cd0c0  0x8e0   Process 0x1fffff        svchost.exe Pid 6356
804     csrss.exe       0x840e9856b2c0  0x8e4   Process 0x1fffff        svchost.exe Pid 7780
804     csrss.exe       0x840e98b1e080  0x8e8   Thread  0x1fffff        Tid 7784 Pid 7780
804     csrss.exe       0x840e98b1ea80  0x8ec   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e99cec080  0x8f0   Thread  0x1fffff        Tid 7616 Pid 9444
804     csrss.exe       0x840e9b7bda80  0x8f4   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e92e07080  0x8fc   Thread  0x1fffff        Tid 9400 Pid 12392
804     csrss.exe       0xb70824546690  0x908   Section 0xf0007 -
804     csrss.exe       0x840ea08f1080  0x910   Process 0x1fffff        svchost.exe Pid 3444
804     csrss.exe       0x840e99d81dd0  0x918   ALPC Port       0x1f0001        -
804     csrss.exe       0xb7080a507530  0x91c   Section 0xf0007 -
804     csrss.exe       0x840e99607080  0x920   Thread  0x1fffff        Tid 11760 Pid 11756
804     csrss.exe       0xb70830544110  0x924   Section 0xf0007 -
804     csrss.exe       0x840e97be9a80  0x92c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e99b83a10  0x930   ALPC Port       0x1f0001        -
804     csrss.exe       0x840ea0dc8d20  0x938   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9952b1b0  0x93c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e966d9300  0x940   Process 0x1fffff        svchost.exe Pid 8912
804     csrss.exe       0x840e98db8080  0x944   Process 0x1fffff        MsSense.exe Pid 10224
804     csrss.exe       0x840e98c5c2c0  0x948   ALPC Port       0x1f0001        -
804     csrss.exe       0xb70824245470  0x950   Section 0xf0007 -
804     csrss.exe       0x840e97e3d080  0x954   Thread  0x101c00        Tid 3732 Pid 9948
804     csrss.exe       0x840e99d4a380  0x958   Thread  0x1fffff        Tid 13288 Pid 11776
804     csrss.exe       0x840e997450c0  0x95c   Process 0x1fffff        svchost.exe Pid 3052
804     csrss.exe       0x840e98ff32c0  0x960   Process 0x1fffff        SecurityHealth Pid 8808
804     csrss.exe       0x840e9f4e6080  0x964   Thread  0x1fffff        Tid 9296 Pid 15508
804     csrss.exe       0x840e98f14080  0x968   Thread  0x1fffff        Tid 8296 Pid 8292
804     csrss.exe       0x840e99066080  0x96c   Thread  0x1fffff        Tid 8812 Pid 8808
804     csrss.exe       0x840e98fc3080  0x970   Thread  0x1fffff        Tid 8972 Pid 8808
804     csrss.exe       0x840e923d9080  0x974   Thread  0x1fffff        Tid 636 Pid 548
804     csrss.exe       0xb70824556630  0x978   Section 0xf0007 -
804     csrss.exe       0x840ea8d6a080  0x97c   Process 0x1fffff        OpenHandleColl Pid 11408
804     csrss.exe       0x840e99179440  0x980   Thread  0x1fffff        Tid 8916 Pid 8912
804     csrss.exe       0xb708302455b0  0x984   Section 0xf0007 -
804     csrss.exe       0x840e92ce3080  0x988   Thread  0x1fffff        Tid 5636 Pid 756
804     csrss.exe       0x840e9744e430  0x98c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9c0cb080  0x994   Process 0x1fffff        splunk-winevtl Pid 11256
804     csrss.exe       0x840e99667dd0  0x998   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9f3460c0  0x9a0   Process 0x1fffff        svchost.exe Pid 11596
804     csrss.exe       0x840e9c208080  0x9a4   Thread  0x1fffff        Tid 10976 Pid 1600
804     csrss.exe       0x840e91ab8ad0  0x9a8   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e976bc080  0x9ac   Thread  0x1fffff        Tid 6224 Pid 3648
804     csrss.exe       0x840e970ed080  0x9b8   Thread  0x1fffff        Tid 10096 Pid 3052
804     csrss.exe       0x840e92fb4080  0x9bc   Thread  0x1fffff        Tid 14020 Pid 11296
804     csrss.exe       0xb708292512d0  0x9c0   Section 0xf0007 -
804     csrss.exe       0xb708142e35f0  0x9c4   Section 0xf0007 -
804     csrss.exe       0xb708241d5ab0  0x9cc   Section 0xf0007 -
804     csrss.exe       0x840e9e9aa080  0x9d0   Thread  0x1fffff        Tid 3068 Pid 11740
804     csrss.exe       0xb70830246e10  0x9d8   Section 0xf0007 -
804     csrss.exe       0x840e9e29cb40  0x9dc   ALPC Port       0x1f0001        -
804     csrss.exe       0x840ea5d2f080  0x9e4   Process 0x1fffff        svchost.exe Pid 9444
804     csrss.exe       0x840e98ed2080  0x9ec   Thread  0x1fffff        Tid 8456 Pid 13620
804     csrss.exe       0x840e996fc440  0x9f0   Thread  0x1fffff        Tid 9024 Pid 10224
804     csrss.exe       0x840e9844f080  0x9f4   Thread  0x1fffff        Tid 10228 Pid 10224
804     csrss.exe       0x840e96f0e7f0  0x9f8   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e998519c0  0x9fc   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9bee60c0  0xa00   Process 0x1fffff        svchost.exe Pid 12440
804     csrss.exe       0xb7082d904830  0xa04   Section 0xf0007 -
804     csrss.exe       0x840e91abc080  0xa08   Thread  0x1fffff        Tid 1088 Pid 5340
804     csrss.exe       0x840e96d40080  0xa10   Thread  0x1fffff        Tid 5900 Pid 12440
804     csrss.exe       0x840e985e9080  0xa18   Thread  0x1fffff        Tid 720 Pid 7728
804     csrss.exe       0x840e97cb2810  0xa20   ALPC Port       0x1f0001        -
804     csrss.exe       0xb708241dad30  0xa24   Section 0xf0007 -
804     csrss.exe       0xb708292380f0  0xa30   Section 0xf0007 -
804     csrss.exe       0x840e98c4f5b0  0xa34   ALPC Port       0x1f0001        -
804     csrss.exe       0xb70811b467d0  0xa38   Section 0xf0007 -
804     csrss.exe       0xb708241db2d0  0xa3c   Section 0xf0007 -
804     csrss.exe       0x840e9e5a11c0  0xa40   Process 0x1fffff        uhssvc.exe Pid 11740
804     csrss.exe       0xb708241deb10  0xa44   Section 0xf0007 -
804     csrss.exe       0xb7080a768e70  0xa4c   Section 0xf0007 -
804     csrss.exe       0x840e9cba3080  0xa50   Process 0x1fffff        WmiPrvSE.exe Pid 7964
804     csrss.exe       0x840e9dc3dcb0  0xa54   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9f28b080  0xa58   Process 0x1fffff        WmiPrvSE.exe Pid 14028
804     csrss.exe       0x840e9f4e4a80  0xa5c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e994dc080  0xa60   Thread  0x1fffff        Tid 15004 Pid 10352
804     csrss.exe       0x840e9b2df080  0xa68   Process 0x1fffff        MoUsoCoreWorke Pid 10144
804     csrss.exe       0xb70833e31370  0xa78   Section 0xf0007 -
804     csrss.exe       0xb7080a74f8d0  0xa7c   Section 0xf0007 -
804     csrss.exe       0xb708233b3ad0  0xa80   Section 0xf0007 -
804     csrss.exe       0xb7080a5032f0  0xa84   Section 0xf0007 -
804     csrss.exe       0x840e9c1b2080  0xa8c   Thread  0x1fffff        Tid 11540 Pid 11740
804     csrss.exe       0xb70824813650  0xa94   Section 0xf0007 -
804     csrss.exe       0x840e99581080  0xa98   Process 0x1fffff        svchost.exe Pid 15508
804     csrss.exe       0xb70833e263d0  0xa9c   Section 0xf0007 -
804     csrss.exe       0xb708142bb2d0  0xaa8   Section 0xf0007 -
804     csrss.exe       0xb70823378e30  0xab0   Section 0xf0007 -
804     csrss.exe       0x840ea15df080  0xac4   Thread  0x1fffff        Tid 12088 Pid 3444
804     csrss.exe       0xb708087541d0  0xacc   Section 0xf0007 -
804     csrss.exe       0xb70824802df0  0xad0   Section 0xf0007 -
804     csrss.exe       0xb70829237a10  0xad4   Section 0xf0007 -
804     csrss.exe       0x840e97e59080  0xad8   Process 0x1fffff        conhost.exe Pid 12392
804     csrss.exe       0xb70830562390  0xae4   Section 0xf0007 -
804     csrss.exe       0xb70814b43350  0xaec   Section 0xf0007 -
804     csrss.exe       0x840e9978b070  0xaf0   ALPC Port       0x1f0001        -
804     csrss.exe       0xb70811b49110  0xaf8   Section 0xf0007 -
804     csrss.exe       0x840e9ea815c0  0xafc   Thread  0x1fffff        Tid 10032 Pid 13620
804     csrss.exe       0xb7080a76d1f0  0xb04   Section 0xf0007 -
804     csrss.exe       0xb7080a744bb0  0xb08   Section 0xf0007 -
804     csrss.exe       0xb7083030fa50  0xb10   Section 0xf0007 -
804     csrss.exe       0x840e99304080  0xb14   Thread  0x1fffff        Tid 1884 Pid 7728
804     csrss.exe       0xb70817927190  0xb18   Section 0xf0007 -
804     csrss.exe       0x840e9e4de080  0xb1c   Thread  0x1fffff        Tid 5656 Pid 14128
804     csrss.exe       0xb7080874b990  0xb20   Section 0xf0007 -
804     csrss.exe       0xb7081794d610  0xb24   Section 0xf0007 -
804     csrss.exe       0x840ea7061080  0xb28   Process 0x1fffff        svchost.exe Pid 16024
804     csrss.exe       0x840e918e90c0  0xb2c   Process 0x1fffff        svchost.exe Pid 14128
804     csrss.exe       0xb7081792c9b0  0xb30   Section 0xf0007 -
804     csrss.exe       0xb70830305f50  0xb34   Section 0xf0007 -
804     csrss.exe       0xb70833e24530  0xb38   Section 0xf0007 -
804     csrss.exe       0x840e984599c0  0xb44   ALPC Port       0x1f0001        -
804     csrss.exe       0xb7081794cdf0  0xb48   Section 0xf0007 -
804     csrss.exe       0xb7080a943c10  0xb54   Section 0xf0007 -
804     csrss.exe       0x840e9ce19080  0xb58   Thread  0x1fffff        Tid 14664 Pid 7256
804     csrss.exe       0xb708303048d0  0xb5c   Section 0xf0007 -
804     csrss.exe       0x840e985dd7c0  0xb68   ALPC Port       0x1f0001        -
804     csrss.exe       0xb7082ec25610  0xb70   Section 0xf0007 -
804     csrss.exe       0x840e973dcdf0  0xb74   ALPC Port       0x1f0001        -
804     csrss.exe       0xb7080a743f30  0xb7c   Section 0xf0007 -
804     csrss.exe       0xb708142f3e50  0xb80   Section 0xf0007 -
804     csrss.exe       0x840e9db37080  0xb84   Thread  0x1fffff        Tid 13776 Pid 1000
804     csrss.exe       0x840e97e2f080  0xb88   Thread  0x1fffff        Tid 14164 Pid 1600
804     csrss.exe       0x840e9bdf1810  0xb8c   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9e5e1080  0xb90   Thread  0x1fffff        Tid 12408 Pid 13532
804     csrss.exe       0xb7081ecdec50  0xb94   Section 0xf0007 -
804     csrss.exe       0x840e974b8080  0xb98   Thread  0x1fffff        Tid 4440 Pid 12392
804     csrss.exe       0x840e9e5d6900  0xba0   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9e5540c0  0xba4   Thread  0x1fffff        Tid 5084 Pid 756
804     csrss.exe       0x840e99111080  0xba8   Thread  0x1fffff        Tid 15556 Pid 10352
804     csrss.exe       0xb70833e24670  0xbb0   Section 0xf0007 -
804     csrss.exe       0xb70833e31910  0xbb4   Section 0xf0007 -
804     csrss.exe       0x840e9dabc940  0xbb8   IoCompletion    0x1f0003        -
804     csrss.exe       0x840e968c9070  0xbbc   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9f3df080  0xbc0   Process 0x1fffff        conhost.exe Pid 7256
804     csrss.exe       0x840ea08f3080  0xbc4   Process 0x1fffff        svchost.exe Pid 12008
804     csrss.exe       0x840e91e16b50  0xbc8   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e990ee070  0xbcc   ALPC Port       0x1f0001        -
804     csrss.exe       0x840e9cc4edd0  0xbd4   ALPC Port       0x1f0001        -
804     csrss.exe       0xb708236d3590  0xbe0   Section 0xf0007 -
804     csrss.exe       0xb708233aa610  0xbe8   Section 0xf0007 -
804     csrss.exe       0x840e9e253340  0xbec   Process 0x1fffff        conhost.exe Pid 13532
804     csrss.exe       0x840e9e24a080  0xbf0   Thread  0x1fffff        Tid 8404 Pid 11256
804     csrss.exe       0x840e97c640c0  0xbf8   Process 0x1fffff        powershell.exe Pid 10352
880     wininit.exe     0x840e92296460  0x4     Event   0x1f0003        -
880     wininit.exe     0x840e92296ce0  0x8     Event   0x1f0003        -
880     wininit.exe     0x840e90c62730  0xc     WaitCompletionPacket    0x1     -
880     wininit.exe     0x840e920fc780  0x10    IoCompletion    0x1f0003        -
880     wininit.exe     0x840e9225dd70  0x14    TpWorkerFactory 0xf00ff -
880     wininit.exe     0x840e919d4b40  0x18    IRTimer 0x100002        -
880     wininit.exe     0x840e90c635d0  0x1c    WaitCompletionPacket    0x1     -
880     wininit.exe     0x840e919d55e0  0x20    IRTimer 0x100002        -
880     wininit.exe     0x840e90c628d0  0x24    WaitCompletionPacket    0x1     -
880     wininit.exe     0x840e920fb520  0x28    EtwRegistration 0x804   -
880     wininit.exe     0x840e920fb600  0x2c    EtwRegistration 0x804   -
880     wininit.exe     0x840e920fb8a0  0x30    EtwRegistration 0x804   -
880     wininit.exe     0x840e92296ae0  0x34    Event   0x1f0003        -
880     wininit.exe     0x840e92296b60  0x38    Event   0x1f0003        -
880     wininit.exe     0x840e92282b00  0x3c    File    0x100020        \Device\HarddiskVolume3\Windows\System32
880     wininit.exe     0x840e920fc080  0x40    EtwRegistration 0x804   -
880     wininit.exe     0x840e921ef850  0x44    Mutant  0x1f0001        SM0:880:304:WilStaging_02
880     wininit.exe     0x840e91f6d530  0x48    ALPC Port       0x1f0001        -
880     wininit.exe     0xb7080435d660  0x4c    Directory       0xf     BaseNamedObjects
880     wininit.exe     0x840e91fb0fd0  0x50    Semaphore       0x1f0003        SM0:880:304:WilStaging_02_p0
880     wininit.exe     0x840e91fb0530  0x54    Semaphore       0x1f0003        SM0:880:304:WilStaging_02_p0h
880     wininit.exe     0x840e920fccc0  0x58    EtwRegistration 0x804   -
880     wininit.exe     0x840e920fcda0  0x5c    EtwRegistration 0x804   -
880     wininit.exe     0x840e920fce80  0x60    EtwRegistration 0x804   -
880     wininit.exe     0x840e920fcf80  0x64    IoCompletion    0x1f0003        -
880     wininit.exe     0x840e922b8d70  0x68    TpWorkerFactory 0xf00ff -
880     wininit.exe     0x840e919d56f0  0x6c    IRTimer 0x100002        -
880     wininit.exe     0x840e90c62180  0x70    WaitCompletionPacket    0x1     -
880     wininit.exe     0x840e922c8b40  0x74    IRTimer 0x100002        -
880     wininit.exe     0x840e90c62f50  0x78    WaitCompletionPacket    0x1     -
880     wininit.exe     0x840e911970c0  0x7c    EtwRegistration 0x804   -
880     wininit.exe     0x840e92296de0  0x80    Event   0x1f0003        -
880     wininit.exe     0xb708042b9e70  0x84    Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER
880     wininit.exe     0xb708042b94e0  0x88    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NLS\SORTING\VERSIONS
880     wininit.exe     0x840e91198400  0x8c    EtwRegistration 0x804   -
880     wininit.exe     0x840e92296c60  0x90    Semaphore       0x100003        -
880     wininit.exe     0x840e911985c0  0x94    EtwRegistration 0x804   -
880     wininit.exe     0x840e922970e0  0x98    Semaphore       0x100003        -
880     wininit.exe     0x840e92281b60  0x9c    File    0x100003        \Device\KsecDD
880     wininit.exe     0x840e91197520  0xa0    EtwRegistration 0x804   -
880     wininit.exe     0x840e922d7400  0xa4    EtwRegistration 0x804   -
880     wininit.exe     0x840e922d66e0  0xa8    EtwRegistration 0x804   -
880     wininit.exe     0x840e922d67c0  0xac    EtwRegistration 0x804   -
880     wininit.exe     0xb708042bc5c0  0xb0    Key     0xf003f MACHINE
880     wininit.exe     0x840e922d7a20  0xb4    EtwRegistration 0x804   -
880     wininit.exe     0x840e922ca180  0xb8    Event   0x1f0003        FirstWinlogonCheck
880     wininit.exe     0x840e922ca2c0  0xbc    Event   0x1f0003        WinlogonLogoff
880     wininit.exe     0xb708042bc070  0xc0    Key     0x9     MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\IMAGE FILE EXECUTION OPTIONS
880     wininit.exe     0x840e922973e0  0xc4    Event   0x1f0003        -
880     wininit.exe     0x840e922d6d00  0xc8    EtwRegistration 0x804   -
880     wininit.exe     0x840e922d6b40  0xcc    EtwRegistration 0x804   -
880     wininit.exe     0x840e922d6600  0xd0    EtwRegistration 0x804   -
880     wininit.exe     0x840e92297160  0xd4    Event   0x1f0003        -
880     wininit.exe     0x840e9228d080  0xd8    Thread  0x1fffff        Tid 884 Pid 880
880     wininit.exe     0x840e922974e0  0xdc    Event   0x1f0003        -
880     wininit.exe     0x840e922b9da0  0xe0    ALPC Port       0x1f0001        WMsgKRpc01A9590
880     wininit.exe     0x840e92297fe0  0xe4    Event   0x1f0003        -
880     wininit.exe     0x840e90c631c0  0xe8    WaitCompletionPacket    0x1     -
880     wininit.exe     0x840e922d74e0  0xec    EtwRegistration 0x804   -
880     wininit.exe     0x840e922d7160  0xf0    EtwRegistration 0x804   -
880     wininit.exe     0x840e922d6c20  0xf4    EtwRegistration 0x804   -
880     wininit.exe     0x840e922d6ec0  0xf8    EtwRegistration 0x804   -
880     wininit.exe     0x840e92298ae0  0xfc    Semaphore       0x100003        -
880     wininit.exe     0x840e92298d60  0x100   Event   0x1f0003        -
880     wininit.exe     0x840e92282330  0x104   File    0x12019f        \Device\KsecDD
880     wininit.exe     0x840e922d7320  0x108   EtwRegistration 0x804   -
880     wininit.exe     0x840e922d6440  0x10c   EtwRegistration 0x804   -
880     wininit.exe     0x840e922d6520  0x110   EtwRegistration 0x804   -
880     wininit.exe     0x840e922d6360  0x114   EtwRegistration 0x804   -
880     wininit.exe     0x840e922979e0  0x118   Semaphore       0x100003        -
880     wininit.exe     0x840e922d7f80  0x11c   IoCompletion    0x1f0003        -
880     wininit.exe     0x840e9229ac60  0x120   Event   0x1f0003        -
880     wininit.exe     0x840e922d7e80  0x128   EtwRegistration 0x804   -
880     wininit.exe     0x840e90c849b0  0x130   ALPC Port       0x1f0001        -
880     wininit.exe     0x840e920d4920  0x134   WindowStation   0xf037f WinSta0
880     wininit.exe     0x840e911f22c0  0x138   Desktop 0xf01ff Winlogon
880     wininit.exe     0x840e922f54d0  0x13c   File    0x100001        \Device\HarddiskVolume3\Windows\System32\en-US\user32.dll.mui
880     wininit.exe     0x840e920d4920  0x140   WindowStation   0xf037f WinSta0
880     wininit.exe     0x840e92298fe0  0x144   Event   0x1f0003        -
880     wininit.exe     0xb708042bb190  0x148   Key     0x20019 USER\.DEFAULT\CONTROL PANEL\INTERNATIONAL
880     wininit.exe     0x840e922fde60  0x14c   Desktop 0xf01ff Disconnect
880     wininit.exe     0x840e922febf0  0x150   Desktop 0xf01ff Default
880     wininit.exe     0x840e922f4d00  0x154   File    0x100000        \Device\NamedPipe\InitShutdown
880     wininit.exe     0x840e922f4530  0x158   File    0x12019f        \Device\NamedPipe\InitShutdown
880     wininit.exe     0x840e922f49e0  0x15c   File    0x12019f        \Device\NamedPipe\InitShutdown
880     wininit.exe     0x840e92c0cd60  0x160   EtwRegistration 0x804   -
880     wininit.exe     0x840e92c0bda0  0x164   EtwRegistration 0x804   -
880     wininit.exe     0x840e92c0cac0  0x168   EtwRegistration 0x804   -
880     wininit.exe     0x840e9229aee0  0x16c   Event   0x1f0003        -
880     wininit.exe     0x840e922e5da0  0x170   ALPC Port       0x1f0001        WindowsShutdown
880     wininit.exe     0x840e9230b100  0x174   Process 0x1fffff        services.exe Pid 1000
880     wininit.exe     0x840e92272220  0x178   Event   0x1f0003        UMSServicesStarted
880     wininit.exe     0x840e92349080  0x17c   Process 0x1fffff        lsass.exe Pid 548
880     wininit.exe     0x840e91bc8b20  0x180   WaitCompletionPacket    0x1     -
880     wininit.exe     0x840e91bca790  0x188   WaitCompletionPacket    0x1     -
880     wininit.exe     0x840e922701a0  0x18c   EtwRegistration 0x804   -
880     wininit.exe     0xb708044d7830  0x190   Key     0x8     USER\.DEFAULT\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION
880     wininit.exe     0xb70804814d80  0x1ac   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NLS\SORTING\IDS
880     wininit.exe     0x840e92c9fa80  0x1b4   ALPC Port       0x1f0001        -
880     wininit.exe     0x840e923cca80  0x1c0   ALPC Port       0x1f0001        -
880     wininit.exe     0x840e923a91e0  0x1c4   Event   0x1f0003        -
880     wininit.exe     0x840e923af960  0x1c8   Event   0x1f0003        -
880     wininit.exe     0xb70804815a40  0x1cc   Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS\SOFTWARE\MICROSOFT
880     wininit.exe     0xb708048173c0  0x1d0   Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS
880     wininit.exe     0xb70804816e70  0x1d4   Key     0x20019 MACHINE
880     wininit.exe     0x840e923a9260  0x1d8   Event   0x1f0003        -
880     wininit.exe     0xb70804815930  0x1dc   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\OLE
880     wininit.exe     0x840e923e9940  0x1e0   EtwRegistration 0x804   -
880     wininit.exe     0x840e923e9b00  0x1e4   EtwRegistration 0x804   -
880     wininit.exe     0x840e923e80c0  0x1e8   EtwRegistration 0x804   -
880     wininit.exe     0x840e923a9ce0  0x1ec   Event   0x1f0003        -
880     wininit.exe     0x840e923a9f60  0x1f0   Event   0x1f0003        -
880     wininit.exe     0x840e923aa060  0x1f4   Event   0x1f0003        -
880     wininit.exe     0x840e923aa0e0  0x1f8   Event   0x1f0003        -
880     wininit.exe     0x840e923a9360  0x1fc   Event   0x1f0003        -
880     wininit.exe     0x840e923a9760  0x200   Event   0x1f0003        -
880     wininit.exe     0x840e923e86e0  0x204   EtwRegistration 0x804   -
880     wininit.exe     0x840e923e81a0  0x208   EtwRegistration 0x804   -
880     wininit.exe     0x840e923e8360  0x20c   EtwRegistration 0x804   -
880     wininit.exe     0x840e923e8a60  0x210   EtwRegistration 0x804   -
880     wininit.exe     0x840e923e8520  0x214   EtwRegistration 0x804   -
880     wininit.exe     0x840e923e8600  0x218   EtwRegistration 0x804   -
880     wininit.exe     0x840e923e8b40  0x21c   EtwRegistration 0x804   -
880     wininit.exe     0x840e923a9660  0x220   Semaphore       0x100003        -
880     wininit.exe     0x840e923a98e0  0x224   Semaphore       0x100003        -
880     wininit.exe     0x840e923a92e0  0x228   Semaphore       0x100003        -
880     wininit.exe     0x840e923a93e0  0x22c   Semaphore       0x100003        -
880     wininit.exe     0x840e923a94e0  0x230   Semaphore       0x100003        -
880     wininit.exe     0x840e923a96e0  0x234   Semaphore       0x100003        -
880     wininit.exe     0x840e923a9a60  0x238   Semaphore       0x100003        -
880     wininit.exe     0x840e923a9160  0x23c   Semaphore       0x100003        -
880     wininit.exe     0x840e923e9da0  0x240   EtwRegistration 0x804   -
880     wininit.exe     0x840e923e9cc0  0x244   EtwRegistration 0x804   -
880     wininit.exe     0x840e923eb540  0x24c   EtwRegistration 0x804   -
880     wininit.exe     0x840e923eb1c0  0x250   EtwRegistration 0x804   -
880     wininit.exe     0xb708048154f0  0x254   Key     0xf003f USER
880     wininit.exe     0x840e92379ea0  0x258   File    0x100001        \Device\CNG
880     wininit.exe     0x840e923eb8c0  0x25c   EtwRegistration 0x804   -
880     wininit.exe     0x840e923ebe00  0x260   EtwRegistration 0x804   -
880     wininit.exe     0x840e923ebee0  0x264   EtwRegistration 0x804   -
880     wininit.exe     0x840e923eb9a0  0x268   EtwRegistration 0x804   -
880     wininit.exe     0x840e923a9c60  0x26c   Event   0x1f0003        -
880     wininit.exe     0x840e923182a0  0x270   EtwRegistration 0x804   -
880     wininit.exe     0x840e9237b480  0x274   File    0x100080        \Device\Nsi
880     wininit.exe     0xb70804813c80  0x278   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS\INTERFACES
880     wininit.exe     0xb708048163d0  0x27c   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP6\PARAMETERS\INTERFACES
880     wininit.exe     0x840e923beb40  0x280   EtwRegistration 0x804   -
880     wininit.exe     0x840e923be7c0  0x288   EtwRegistration 0x804   -
880     wininit.exe     0x840e923be6e0  0x28c   EtwRegistration 0x804   -
880     wininit.exe     0x840e923bf080  0x290   EtwRegistration 0x804   -
880     wininit.exe     0x840e923be8a0  0x294   EtwRegistration 0x804   -
880     wininit.exe     0x840e92367b30  0x298   Mutant  0x1f0001        SM0:880:120:WilError_03
880     wininit.exe     0x840e92339880  0x29c   Semaphore       0x1f0003        SM0:880:120:WilError_03_p0
880     wininit.exe     0x840e92338f20  0x2a0   Semaphore       0x1f0003        SM0:880:120:WilError_03_p0h
880     wininit.exe     0x840e923afce0  0x2c4   Event   0x1f0003        -
880     wininit.exe     0xb70804818f60  0x2c8   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\SERVICES\WINSOCK2\PARAMETERS\PROTOCOL_CATALOG9
880     wininit.exe     0x840e92c0d7e0  0x2cc   EtwRegistration 0x804   -
880     wininit.exe     0xb70804a940d0  0x2d0   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\SERVICES\WINSOCK2\PARAMETERS\NAMESPACE_CATALOG5
880     wininit.exe     0x840e92c64b10  0x2d4   File    0x16019f        \Device\Afd\Endpoint
880     wininit.exe     0x840e92c62bd0  0x2d8   File    0x16019f        \Device\Afd\Endpoint
880     wininit.exe     0x840e92c66280  0x2dc   File    0x160089        \Device\NamedPipe\Winsock2\CatalogChangeListener-370-0
880     wininit.exe     0x840e92c676d0  0x2e8   File    0x16019f        \Device\Afd\Endpoint
880     wininit.exe     0x840e92c66be0  0x2ec   File    0x16019f        \Device\Afd\Endpoint
888     csrss.exe       0x840e922963e0  0x4     Event   0x1f0003        -
888     csrss.exe       0x840e92296be0  0x8     Event   0x1f0003        -
888     csrss.exe       0x840e90c629a0  0xc     WaitCompletionPacket    0x1     -
888     csrss.exe       0x840e920fbd00  0x10    IoCompletion    0x1f0003        -
888     csrss.exe       0x840e9226ed70  0x14    TpWorkerFactory 0xf00ff -
888     csrss.exe       0x840e919d44e0  0x18    IRTimer 0x100002        -
888     csrss.exe       0x840e90c620b0  0x1c    WaitCompletionPacket    0x1     -
888     csrss.exe       0x840e919d41b0  0x20    IRTimer 0x100002        -
888     csrss.exe       0x840e90c63770  0x24    WaitCompletionPacket    0x1     -
888     csrss.exe       0x840e920fb6e0  0x28    EtwRegistration 0x804   -
888     csrss.exe       0x840e920fc320  0x2c    EtwRegistration 0x804   -
888     csrss.exe       0x840e920fba60  0x30    EtwRegistration 0x804   -
888     csrss.exe       0x840e92296660  0x34    Event   0x1f0003        -
888     csrss.exe       0x840e92296d60  0x38    Event   0x1f0003        -
888     csrss.exe       0x840e92283140  0x3c    File    0x100020        \Device\HarddiskVolume3\Windows\System32
888     csrss.exe       0x840e920fbe00  0x40    IoCompletion    0x1f0003        -
888     csrss.exe       0x840e90c93bf0  0x44    TpWorkerFactory 0xf00ff -
888     csrss.exe       0x840e919d4e70  0x48    IRTimer 0x100002        -
888     csrss.exe       0x840e90c62a70  0x4c    WaitCompletionPacket    0x1     -
888     csrss.exe       0x840e919d42c0  0x50    IRTimer 0x100002        -
888     csrss.exe       0x840e90c62c10  0x54    WaitCompletionPacket    0x1     -
888     csrss.exe       0x840e920fc400  0x58    EtwRegistration 0x804   -
888     csrss.exe       0xb7080435e740  0x5c    Directory       0xf000f BNOLINKS
888     csrss.exe       0xb70804352ba0  0x60    SymbolicLink    0xf0001 1
888     csrss.exe       0xb7080435d840  0x64    Directory       0xf000f 1
888     csrss.exe       0xb7080435dc00  0x68    Directory       0xf000f DosDevices
888     csrss.exe       0xb7080435dde0  0x6c    Directory       0xf000f Windows
888     csrss.exe       0x840e9724e7f0  0x70    ALPC Port       0x1f0001        -
888     csrss.exe       0x840e922b7de0  0x74    ALPC Port       0x1f0001        ApiPort
888     csrss.exe       0x840e922b8080  0x78    Thread  0x1fffff        Tid 904 Pid 888
888     csrss.exe       0xb708043fd1a0  0x7c    Directory       0xf000f AppContainerNamedObjects
888     csrss.exe       0xb70801731870  0x80    Section 0xf001f SharedSection
888     csrss.exe       0xb7080435e1a0  0x84    Directory       0xf000f BaseNamedObjects
888     csrss.exe       0xb708042b8820  0x88    Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER
888     csrss.exe       0xb708043538b0  0x8c    SymbolicLink    0xf0001 Global
888     csrss.exe       0xb708043528e0  0x90    SymbolicLink    0xf0001 Local
888     csrss.exe       0xb70804352990  0x94    SymbolicLink    0xf0001 AppContainerNamedObjects
888     csrss.exe       0xb70804353960  0x98    SymbolicLink    0xf0001 Session
888     csrss.exe       0xb708043fc840  0x9c    Directory       0xf000f Restricted
888     csrss.exe       0x840e92281520  0xa0    File    0x1f01ff        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects
888     csrss.exe       0x840e922ca4a0  0xa4    Semaphore       0x1f0003        SM0:888:304:WilStaging_02_p0h
888     csrss.exe       0x840e911984e0  0xa8    EtwRegistration 0x804   -
888     csrss.exe       0xb708042b83e0  0xac    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NLS\SORTING\VERSIONS
888     csrss.exe       0xb7080435e1a0  0xb0    Directory       0xf     BaseNamedObjects
888     csrss.exe       0x840e922d7080  0xb4    EtwRegistration 0x804   -
888     csrss.exe       0x840e922cb300  0xb8    Semaphore       0x1f0003        SM0:888:304:WilStaging_02_p0
888     csrss.exe       0x840e91198940  0xbc    EtwRegistration 0x804   -
888     csrss.exe       0x840e921f0bd0  0xc0    Mutant  0x1f0001        SM0:888:304:WilStaging_02
888     csrss.exe       0x840e91198a20  0xc4    EtwRegistration 0x804   -
888     csrss.exe       0x840e91197a60  0xc8    EtwRegistration 0x804   -
888     csrss.exe       0x840e922e4080  0xcc    Thread  0x1fffff        Tid 964 Pid 888
888     csrss.exe       0x840e918ab710  0xd0    Event   0x100000        TermSrvReadyEvent
888     csrss.exe       0x840e922e6080  0xd4    Thread  0x1fffff        Tid 972 Pid 888
888     csrss.exe       0x840e922e5080  0xd8    Thread  0x1fffff        Tid 968 Pid 888
888     csrss.exe       0x840e922e3080  0xdc    Thread  0x1fffff        Tid 960 Pid 888
888     csrss.exe       0xb708042ba6f0  0xe0    Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\WINDOWS\CURRENTVERSION\SIDEBYSIDE
888     csrss.exe       0x840e9230d080  0xe4    Thread  0x1fffff        Tid 72 Pid 888
888     csrss.exe       0x840e92310070  0xe8    RawInputManager 0x3     -
888     csrss.exe       0x840e922d7b00  0xec    EtwRegistration 0x804   -
888     csrss.exe       0x840e922d61a0  0xf0    EtwRegistration 0x804   -
888     csrss.exe       0xb708042bb7f0  0xf4    Key     0x9     MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\IMAGE FILE EXECUTION OPTIONS
888     csrss.exe       0xb7080a326200  0xf8    Key     0x20019 USER\S-1-5-21-3294271487-4292050549-1661813568-1001\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\FONTS
888     csrss.exe       0x840e922d7be0  0xfc    EtwRegistration 0x804   -
888     csrss.exe       0xb708042bbe50  0x100   Key     0xf003f MACHINE
888     csrss.exe       0x840e922967e0  0x104   Event   0x1f0003        -
888     csrss.exe       0x840e90c62ce0  0x108   WaitCompletionPacket    0x1     -
888     csrss.exe       0xb70801668b20  0x10c   Key     0x20019 MACHINE
888     csrss.exe       0xb70801667c40  0x110   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\OLE
888     csrss.exe       0x840e92296260  0x114   Event   0x1f0003        -
888     csrss.exe       0xb708042b9700  0x118   Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS\SOFTWARE\MICROSOFT
888     csrss.exe       0xb70801669e40  0x11c   Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS
888     csrss.exe       0x840e922961e0  0x120   Event   0x1f0003        -
888     csrss.exe       0x840e922d7860  0x124   EtwRegistration 0x804   -
888     csrss.exe       0x840e922d76a0  0x128   EtwRegistration 0x804   -
888     csrss.exe       0x840e922d7940  0x12c   EtwRegistration 0x804   -
888     csrss.exe       0x840e922962e0  0x130   Event   0x1f0003        -
888     csrss.exe       0x840e922968e0  0x134   Event   0x1f0003        -
888     csrss.exe       0x840e92296960  0x138   Event   0x1f0003        -
888     csrss.exe       0x840e92296360  0x13c   Event   0x1f0003        -
888     csrss.exe       0x840e92297c60  0x140   Event   0x1f0003        -
888     csrss.exe       0x840e92297d60  0x144   Event   0x1f0003        -
888     csrss.exe       0x840e922d6280  0x148   EtwRegistration 0x804   -
888     csrss.exe       0x840e922d68a0  0x14c   EtwRegistration 0x804   -
888     csrss.exe       0x840e922d7780  0x150   EtwRegistration 0x804   -
888     csrss.exe       0x840e922d6de0  0x154   EtwRegistration 0x804   -
888     csrss.exe       0x840e922d6980  0x158   EtwRegistration 0x804   -
888     csrss.exe       0x840e922d60c0  0x15c   EtwRegistration 0x804   -
888     csrss.exe       0x840e922d6a60  0x160   EtwRegistration 0x804   -
888     csrss.exe       0x840e922824c0  0x164   File    0x120089        \Device\DeviceApi\CMApi
888     csrss.exe       0x840e922d7240  0x168   EtwRegistration 0x804   -
888     csrss.exe       0x840e922d75c0  0x16c   EtwRegistration 0x804   -
888     csrss.exe       0x840e922976e0  0x170   Event   0x1f0003        -
888     csrss.exe       0x840e92297460  0x174   Event   0x1f0003        -
888     csrss.exe       0x840e922ca540  0x178   Event   0x1f0003        EventRitExited
888     csrss.exe       0x840e92297be0  0x17c   Event   0x1f0003        -
888     csrss.exe       0x840e92297660  0x180   Event   0x1f0003        -
888     csrss.exe       0x840e92297ce0  0x184   Event   0x1f0003        -
888     csrss.exe       0x840e92297760  0x188   Event   0x1f0003        -
888     csrss.exe       0xb708042bc180  0x18c   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\FONTS
888     csrss.exe       0xb7080a325a90  0x190   Key     0x20019 USER\S-1-5-21-3294271487-4292050549-1661813568-1001\CONTROL PANEL\INTERNATIONAL
888     csrss.exe       0x840e922dfda0  0x194   ALPC Port       0x1f0001        SbApiPort
888     csrss.exe       0x840e92cdec50  0x198   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e97b05080  0x19c   Thread  0x1fffff        Tid 9548 Pid 7800
888     csrss.exe       0x840e922eb700  0x1a0   Event   0x1f0003        ScNetDrvMsg
888     csrss.exe       0x840e922e7080  0x1a4   Thread  0x1fffff        Tid 976 Pid 888
888     csrss.exe       0xb708042b6b70  0x1a8   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\PRIORITYCONTROL
888     csrss.exe       0x840e91ff4580  0x1ac   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e92013dc0  0x1b0   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e9229b060  0x1b4   Event   0x1f0003        -
888     csrss.exe       0x840e922ed440  0x1b8   IoCompletion    0x1f0003        -
888     csrss.exe       0x840e92305080  0x1bc   Process 0x1fffff        winlogon.exe Pid 980
888     csrss.exe       0x840e92306080  0x1c0   Thread  0x1fffff        Tid 984 Pid 980
888     csrss.exe       0x840e90c4d2d0  0x1c4   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e9228a080  0x1c8   Thread  0x1fffff        Tid 1008 Pid 888
888     csrss.exe       0x840e92285080  0x1cc   Thread  0x1fffff        Tid 1020 Pid 888
888     csrss.exe       0x840e9229b1e0  0x1d0   Event   0x1f0003        -
888     csrss.exe       0x840e92270c40  0x1d4   IoCompletion    0x1f0003        -
888     csrss.exe       0x840e922ebd40  0x1d8   Event   0x1f0003        {773F1B9A-35B9-4E95-83A0-A210F2DE3B37}-running
888     csrss.exe       0x840e922ebe80  0x1dc   Event   0x1f0003        {773F1B9A-35B9-4E95-83A0-A210F2DE3B37}-request
888     csrss.exe       0x840e9229b8e0  0x1e0   Event   0x1f0003        -
888     csrss.exe       0x840e92270280  0x1e4   IoCompletion    0x1f0003        -
888     csrss.exe       0x840e9229bb60  0x1e8   Event   0x1f0003        -
888     csrss.exe       0x840e9229bc60  0x1ec   Event   0x1f0003        -
888     csrss.exe       0x840e922fb5b0  0x1f0   Timer   0x1f0003        -
888     csrss.exe       0x840e922fc9f0  0x1f4   Timer   0x1f0003        -
888     csrss.exe       0x840e9229b560  0x1f8   Event   0x1f0003        -
888     csrss.exe       0x840e9229b2e0  0x1fc   Event   0x1f0003        -
888     csrss.exe       0x840e9229b660  0x200   Event   0x1f0003        -
888     csrss.exe       0x840e9229bce0  0x204   Event   0x1f0003        -
888     csrss.exe       0x840e92304bc0  0x208   RawInputManager 0x3     -
888     csrss.exe       0x840e9230e070  0x20c   RawInputManager 0x3     -
888     csrss.exe       0x840e9230f070  0x210   RawInputManager 0x3     -
888     csrss.exe       0x840e922ecec0  0x214   Event   0x1f0003        WinSta0_DesktopSwitch
888     csrss.exe       0x840e9229c4e0  0x218   Event   0x1f0003        -
888     csrss.exe       0x840e9229cee0  0x21c   Event   0x1f0003        -
888     csrss.exe       0x840e922fb760  0x220   Timer   0x1f0003        -
888     csrss.exe       0x840e9229c560  0x224   Event   0x1f0003        -
888     csrss.exe       0x840e9229cd60  0x228   Event   0x1f0003        -
888     csrss.exe       0x840e9229c660  0x22c   Event   0x1f0003        -
888     csrss.exe       0x840e9201d070  0x230   RawInputManager 0x3     -
888     csrss.exe       0x840e97836080  0x234   Thread  0x1fffff        Tid 5320 Pid 5328
888     csrss.exe       0x840e978c0300  0x238   Process 0x1fffff        explorer.exe Pid 5328
888     csrss.exe       0x840e90d761c0  0x23c   Thread  0x1fffff        Tid 6304 Pid 1244
888     csrss.exe       0x840e91f131c0  0x240   Process 0x1fffff        Sysmon64.exe Pid 1244
888     csrss.exe       0xb7080a957490  0x244   Section 0xf0007 -
888     csrss.exe       0x840e9845bac0  0x248   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e97f3c080  0x24c   Thread  0x1fffff        Tid 7124 Pid 7108
888     csrss.exe       0x840e96c5a080  0x258   Process 0x1fffff        svchost.exe Pid 7108
888     csrss.exe       0x840e96e0f080  0x260   Process 0x1fffff        sihost.exe Pid 6908
888     csrss.exe       0x840e98ae5080  0x268   Thread  0x1fffff        Tid 6720 Pid 8120
888     csrss.exe       0x840e923b48e0  0x26c   Event   0x1f0003        -
888     csrss.exe       0x840e920d54a0  0x270   WindowStation   0xf0003 WinSta0
888     csrss.exe       0x840e923f41c0  0x274   Process 0x1fffff        fontdrvhost.ex Pid 800
888     csrss.exe       0x840e923f1080  0x278   Thread  0x1fffff        Tid 796 Pid 800
888     csrss.exe       0x840e923c1700  0x27c   EtwRegistration 0x804   -
888     csrss.exe       0x840e923aa6e0  0x280   Event   0x1f0003        -
888     csrss.exe       0x840e923c31a0  0x284   Mutant  0x1f0001        -
888     csrss.exe       0x840e97802ee0  0x288   Event   0x1f0003        -
888     csrss.exe       0x840e923c1540  0x28c   EtwRegistration 0x804   -
888     csrss.exe       0x840e923c1620  0x290   EtwRegistration 0x804   -
888     csrss.exe       0x840e923c0680  0x294   IoCompletion    0x1f0003        -
888     csrss.exe       0x840e9238a340  0x298   File    0x100001        \Device\CNG
888     csrss.exe       0x840e923e5ce0  0x29c   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e92cd6780  0x2a0   IoCompletion    0x1f0003        -
888     csrss.exe       0x840e994d1080  0x2a4   Process 0x1fffff        LogonUI.exe Pid 10680
888     csrss.exe       0x840e976c3080  0x2a8   Thread  0x1fffff        Tid 5128 Pid 10680
888     csrss.exe       0xb7080a943670  0x2ac   Section 0xf0007 -
888     csrss.exe       0x840e96eab2d0  0x2b0   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e971b1080  0x2b8   Thread  0x1fffff        Tid 6872 Pid 6908
888     csrss.exe       0x840e92ce5080  0x2bc   Thread  0x1fffff        Tid 1212 Pid 888
888     csrss.exe       0x840e92cfa140  0x2c0   Process 0x1fffff        dwm.exe Pid 1204
888     csrss.exe       0x840e92ce6080  0x2c4   Thread  0x1fffff        Tid 1208 Pid 1204
888     csrss.exe       0x840e92e5f660  0x2c8   Event   0x1f0003        -
888     csrss.exe       0x840e92e92440  0x2cc   IoCompletion    0x1f0003        -
888     csrss.exe       0x840e974b9080  0x2d4   Thread  0x1fffff        Tid 2432 Pid 5328
888     csrss.exe       0x840e98c83080  0x2e4   Thread  0x1fffff        Tid 6744 Pid 8120
888     csrss.exe       0x840e92cfc790  0x2e8   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e92ce8080  0x2f0   Thread  0x1fffff        Tid 1188 Pid 980
888     csrss.exe       0x840e97815760  0x308   Event   0x1f0003        -
888     csrss.exe       0x840e97648800  0x30c   IoCompletion    0x1f0003        -
888     csrss.exe       0x840e97ab6e60  0x310   File    0x100001        \Device\HarddiskVolume3\Windows\System32\en-US\winsrv.dll.mui
888     csrss.exe       0x840e998450c0  0x314   Process 0x1fffff        SearchApp.exe Pid 4236
888     csrss.exe       0x840e990f7080  0x31c   Thread  0x1fffff        Tid 9124 Pid 9016
888     csrss.exe       0x840e90ad3080  0x320   Process 0x1fffff        svchost.exe Pid 7152
888     csrss.exe       0x840e9845d080  0x324   Thread  0x1fffff        Tid 7148 Pid 7152
888     csrss.exe       0x840e91e77d80  0x328   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e99387080  0x32c   Process 0x1fffff        StartMenuExper Pid 7408
888     csrss.exe       0x840e98bba080  0x330   Process 0x1fffff        SearchApp.exe Pid 7932
888     csrss.exe       0x840e99372080  0x334   Thread  0x1fffff        Tid 7412 Pid 7408
888     csrss.exe       0x840e97e11080  0x338   Process 0x1fffff        TextInputHost. Pid 1256
888     csrss.exe       0x840e99306080  0x33c   Thread  0x1fffff        Tid 7544 Pid 7408
888     csrss.exe       0x840e98b8e340  0x340   Process 0x1fffff        RuntimeBroker. Pid 7612
888     csrss.exe       0x840e91d7d080  0x344   Thread  0x1fffff        Tid 7936 Pid 7932
888     csrss.exe       0x840e91db7340  0x350   Process 0x1fffff        taskhostw.exe Pid 4428
888     csrss.exe       0x840e91f14080  0x354   Thread  0x1fffff        Tid 4424 Pid 4428
888     csrss.exe       0x840e91dba9f0  0x358   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e98444d80  0x360   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e90c19340  0x364   Process 0x1fffff        svchost.exe Pid 3604
888     csrss.exe       0x840e99ca2080  0x36c   Thread  0x1fffff        Tid 2792 Pid 10680
888     csrss.exe       0x840e91a41080  0x370   Thread  0x1fffff        Tid 6436 Pid 5328
888     csrss.exe       0x840e98c61080  0x380   Thread  0x1fffff        Tid 1264 Pid 1256
888     csrss.exe       0x840e91abf080  0x390   Thread  0x1fffff        Tid 2612 Pid 5328
888     csrss.exe       0xb7080a758d90  0x394   Section 0xf0007 -
888     csrss.exe       0x840e97eda310  0x3a4   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e90be0080  0x3ac   Thread  0x1fffff        Tid 3608 Pid 3604
888     csrss.exe       0x840e90b21660  0x3b4   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e99306a80  0x3c0   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e9eb950c0  0x3c4   Process 0x1fffff        CloudNotificat Pid 14972
888     csrss.exe       0x840e98040080  0x3c8   Thread  0x1fffff        Tid 6424 Pid 5328
888     csrss.exe       0x840e98dd2080  0x3e0   Thread  0x1fffff        Tid 2948 Pid 1256
888     csrss.exe       0x840e98aef080  0x3e4   Process 0x1fffff        RuntimeBroker. Pid 7684
888     csrss.exe       0x840e91baa080  0x3e8   Thread  0x1fffff        Tid 7688 Pid 7684
888     csrss.exe       0x840e90d78b80  0x3ec   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e98bd2d60  0x3fc   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e99071080  0x404   Thread  0x1fffff        Tid 8084 Pid 4836
888     csrss.exe       0x840e90e55080  0x410   Thread  0x1fffff        Tid 7440 Pid 5328
888     csrss.exe       0x840e98bd6240  0x41c   Process 0x1fffff        RuntimeBroker. Pid 8120
888     csrss.exe       0x840e98063080  0x420   Thread  0x1fffff        Tid 8124 Pid 8120
888     csrss.exe       0x840e9935c580  0x424   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e98c6c080  0x428   Thread  0x1fffff        Tid 5032 Pid 5328
888     csrss.exe       0x840e98bbf550  0x42c   ALPC Port       0x1f0001        -
888     csrss.exe       0xb7082c576690  0x434   Section 0xf0007 -
888     csrss.exe       0x840e90e57080  0x438   Thread  0x1fffff        Tid 4760 Pid 5328
888     csrss.exe       0x840e98c450c0  0x44c   Process 0x1fffff        dllhost.exe Pid 7800
888     csrss.exe       0x840e972e4d90  0x450   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e98033a80  0x458   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e985e1900  0x45c   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e99857920  0x460   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e91a0d080  0x464   Thread  0x1fffff        Tid 7668 Pid 5328
888     csrss.exe       0x840e92de8080  0x468   Thread  0x1fffff        Tid 3436 Pid 7612
888     csrss.exe       0x840e91a61ad0  0x46c   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e999f46e0  0x484   Semaphore       0x100003        -
888     csrss.exe       0x840e9958a0c0  0x48c   Process 0x1fffff        msedge.exe Pid 9128
888     csrss.exe       0x840e98fb61c0  0x490   Process 0x1fffff        dllhost.exe Pid 8472
888     csrss.exe       0x840e9769e080  0x494   Thread  0x1fffff        Tid 8476 Pid 8472
888     csrss.exe       0x840e98034ce0  0x498   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e98bc4080  0x49c   Thread  0x1fffff        Tid 8500 Pid 8472
888     csrss.exe       0x840e99528080  0x4a0   Thread  0x1fffff        Tid 3932 Pid 704
888     csrss.exe       0x840e98f11580  0x4ac   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e90bd51c0  0x4b0   Thread  0x1fffff        Tid 3016 Pid 5328
888     csrss.exe       0x840e990f4080  0x4b4   Thread  0x1fffff        Tid 9132 Pid 9128
888     csrss.exe       0xb708142c4330  0x4b8   Section 0xf0007 -
888     csrss.exe       0x840e90c17080  0x4c0   Thread  0x1fffff        Tid 2956 Pid 5328
888     csrss.exe       0x840e990c3dc0  0x4c4   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e98dc11c0  0x4cc   Process 0x1fffff        SecurityHealth Pid 8736
888     csrss.exe       0x840e98fee080  0x4d0   Thread  0x1fffff        Tid 8740 Pid 8736
888     csrss.exe       0x840e99010310  0x4d4   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e91e25dc0  0x4dc   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e9db77080  0x4e4   Process 0x1fffff        taskhostw.exe Pid 15784
888     csrss.exe       0x840e98fe2e60  0x4e8   Event   0x1f0003        -
888     csrss.exe       0x840e90de9080  0x4ec   Process 0x1fffff        FileCoAuth.exe Pid 8592
888     csrss.exe       0x840e966c0080  0x4f0   Thread  0x1fffff        Tid 9844 Pid 10020
888     csrss.exe       0x840e98f07080  0x500   Process 0x1fffff        msedge.exe Pid 9172
888     csrss.exe       0x840e9682f080  0x504   Process 0x1fffff        OneDrive.exe Pid 3320
888     csrss.exe       0x840e98ab4080  0x508   Thread  0x1fffff        Tid 9176 Pid 9172
888     csrss.exe       0xb708241c60b0  0x510   Section 0xf0007 -
888     csrss.exe       0x840e9904cc80  0x514   IoCompletion    0x1f0003        -
888     csrss.exe       0x840e9662b080  0x518   Thread  0x1fffff        Tid 3652 Pid 888
888     csrss.exe       0xb7083054a5b0  0x52c   Section 0xf0007 -
888     csrss.exe       0x840e98bc2080  0x530   Thread  0x1fffff        Tid 3248 Pid 3832
888     csrss.exe       0x840e996e91c0  0x534   Process 0x1fffff        msedge.exe Pid 9016
888     csrss.exe       0x840e92cda9e0  0x53c   EtwRegistration 0x804   -
888     csrss.exe       0x840e98332080  0x548   Thread  0x1fffff        Tid 8240 Pid 9172
888     csrss.exe       0x840e9832a1c0  0x550   Process 0x1fffff        msedge.exe Pid 3832
888     csrss.exe       0x840e98f28080  0x554   Thread  0x1fffff        Tid 3576 Pid 3832
888     csrss.exe       0x840e9951c1c0  0x558   Process 0x1fffff        msedge.exe Pid 8732
888     csrss.exe       0x840e99521080  0x55c   Thread  0x1fffff        Tid 8728 Pid 8732
888     csrss.exe       0x840e9954bdc0  0x560   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e92cdbe00  0x568   EtwRegistration 0x804   -
888     csrss.exe       0x840e91e24dc0  0x56c   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e999f5160  0x570   Semaphore       0x100003        -
888     csrss.exe       0x840e91e7f540  0x578   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e92f74080  0x57c   Thread  0x1fffff        Tid 9892 Pid 9888
888     csrss.exe       0x840e91ba8070  0x580   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e9977ddc0  0x584   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e92cda900  0x588   EtwRegistration 0x804   -
888     csrss.exe       0x840e974b2080  0x58c   Process 0x1fffff        UserOOBEBroker Pid 9620
888     csrss.exe       0x840e98ae3080  0x590   Thread  0x1fffff        Tid 9464 Pid 9172
888     csrss.exe       0xb7080a747d10  0x598   Section 0xf0007 -
888     csrss.exe       0x840e98be5080  0x5a0   Thread  0x1fffff        Tid 12608 Pid 14972
888     csrss.exe       0x840e98e6a2c0  0x5a4   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e999f4c60  0x5a8   Event   0x1f0003        -
888     csrss.exe       0xb70818405e10  0x5ac   Section 0xf0007 -
888     csrss.exe       0x840e998251c0  0x5b4   Process 0x1fffff        msedge.exe Pid 9728
888     csrss.exe       0xb7083023ef30  0x5bc   Section 0xf0007 -
888     csrss.exe       0x840e96789080  0x5c0   Thread  0x1fffff        Tid 9732 Pid 9728
888     csrss.exe       0x840e98db4080  0x5c8   Process 0x1fffff        msedge.exe Pid 9888
888     csrss.exe       0x840e9969e080  0x5d0   Process 0x1fffff        svchost.exe Pid 4156
888     csrss.exe       0x840e99b5ec90  0x5e4   File    0x100003        \Device\KsecDD
888     csrss.exe       0x840e972e4080  0x5ec   Thread  0x1fffff        Tid 3936 Pid 4156
888     csrss.exe       0x840e98bdf080  0x5f4   Process 0x1fffff        SystemSettings Pid 10020
888     csrss.exe       0x840e96f51070  0x5fc   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e971f2080  0x600   Thread  0x1fffff        Tid 3424 Pid 4836
888     csrss.exe       0x840e98446080  0x608   Thread  0x1fffff        Tid 2456 Pid 8592
888     csrss.exe       0x840e91995080  0x60c   Thread  0x1fffff        Tid 9744 Pid 10020
888     csrss.exe       0x840e98c48080  0x610   Process 0x1fffff        ApplicationFra Pid 4836
888     csrss.exe       0x840e90e54080  0x614   Thread  0x1fffff        Tid 5936 Pid 5328
888     csrss.exe       0xb708238a5eb0  0x618   Section 0xf0007 -
888     csrss.exe       0x840e977e7080  0x628   Thread  0x1fffff        Tid 6552 Pid 3320
888     csrss.exe       0x840e91a66080  0x630   Thread  0x1fffff        Tid 4856 Pid 5328
888     csrss.exe       0x840e9910e080  0x634   Thread  0x1fffff        Tid 1972 Pid 3320
888     csrss.exe       0x840e99180080  0x64c   Thread  0x1fffff        Tid 9644 Pid 9620
888     csrss.exe       0x840e96fda080  0x6a8   Thread  0x1fffff        Tid 14460 Pid 15784
888     csrss.exe       0x840e99059080  0x6b4   Thread  0x1fffff        Tid 5840 Pid 7800
888     csrss.exe       0x840e99d85cc0  0x6bc   ALPC Port       0x1f0001        -
888     csrss.exe       0xb7083054c630  0x6d4   Section 0xf0007 -
888     csrss.exe       0x840e992c56c0  0x6d8   IoCompletion    0x1f0003        -
888     csrss.exe       0x840e9954e080  0x6dc   Thread  0x1fffff        Tid 6772 Pid 4236
888     csrss.exe       0x840e982d0a60  0x6ec   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e99371080  0x700   Thread  0x1fffff        Tid 6028 Pid 1832
888     csrss.exe       0x840e982ee1c0  0x710   Thread  0x1fffff        Tid 2916 Pid 5328
888     csrss.exe       0x840e98e4b080  0x718   Thread  0x1fffff        Tid 7876 Pid 2492
888     csrss.exe       0x840e90c11070  0x724   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e990c4a80  0x734   ALPC Port       0x1f0001        -
888     csrss.exe       0xb70813226070  0x73c   Section 0xf0007 -
888     csrss.exe       0x840e97ac6080  0x75c   Thread  0x1fffff        Tid 2884 Pid 2492
888     csrss.exe       0x840e98c56d30  0x768   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e96cf0080  0x780   Thread  0x1fffff        Tid 8388 Pid 5328
888     csrss.exe       0x840e99c2d7e0  0x78c   Event   0x1f0003        -
888     csrss.exe       0xb7081723c700  0x790   Key     0x20019 USER\.DEFAULT\CONTROL PANEL\INTERNATIONAL
888     csrss.exe       0x840e9a943d60  0x794   Event   0x1f0003        -
888     csrss.exe       0x840e98d89080  0x7ac   Thread  0x1fffff        Tid 4120 Pid 4236
888     csrss.exe       0x840e9688f080  0x7b8   Thread  0x1fffff        Tid 3364 Pid 704
888     csrss.exe       0x840e97ea9080  0x7c8   Process 0x1fffff        RuntimeBroker. Pid 1832
888     csrss.exe       0x840e996fa080  0x7cc   Thread  0x1fffff        Tid 9640 Pid 5328
888     csrss.exe       0x840e983179a0  0x7dc   ALPC Port       0x1f0001        -
888     csrss.exe       0x840e99840080  0x7e4   Process 0x1fffff        dllhost.exe Pid 2492
888     csrss.exe       0x840e98df3080  0x7ec   Process 0x1fffff        ShellExperienc Pid 704
888     csrss.exe       0xb70808750c10  0x7f8   Section 0xf0007 -
888     csrss.exe       0xb70824545f10  0x8f8   Section 0xf0007 -
888     csrss.exe       0xb70823891690  0x964   Section 0xf0007 -
888     csrss.exe       0xb708142c00f0  0xa78   Section 0xf0007 -
888     csrss.exe       0xb7082337d9d0  0xacc   Section 0xf0007 -
888     csrss.exe       0xb708233817b0  0xb34   Section 0xf0007 -
888     csrss.exe       0xb7080a748e90  0xb38   Section 0xf0007 -
888     csrss.exe       0xb7083030cd50  0xb44   Section 0xf0007 -
888     csrss.exe       0xb7082481a450  0xb58   Section 0xf0007 -
888     csrss.exe       0xb70830316cb0  0xb7c   Section 0xf0007 -
888     csrss.exe       0xb7082388ceb0  0xba8   Section 0xf0007 -
888     csrss.exe       0xb70830306090  0xbac   Section 0xf0007 -
888     csrss.exe       0xb7082336cbd0  0xbb0   Section 0xf0007 -
888     csrss.exe       0xb708233a9850  0xbb8   Section 0xf0007 -
888     csrss.exe       0xb70830571a70  0xbc0   Section 0xf0007 -
888     csrss.exe       0xb70824246b90  0xbc8   Section 0xf0007 -
888     csrss.exe       0xb708211147d0  0xbd4   Section 0xf0007 -
888     csrss.exe       0xb708233ae170  0xbec   Section 0xf0007 -
888     csrss.exe       0xb708233883d0  0xbf0   Section 0xf0007 -
980     winlogon.exe    0x840e9229a2e0  0x4     Event   0x1f0003        -
980     winlogon.exe    0x840e9229aae0  0x8     Event   0x1f0003        -
980     winlogon.exe    0x840e90ac76a0  0xc     WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e922ed600  0x10    IoCompletion    0x1f0003        -
980     winlogon.exe    0x840e92305d70  0x14    TpWorkerFactory 0xf00ff -
980     winlogon.exe    0x840e922c9800  0x18    IRTimer 0x100002        -
980     winlogon.exe    0x840e90ac6800  0x1c    WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e922c8920  0x20    IRTimer 0x100002        -
980     winlogon.exe    0x840e90ac7840  0x24    WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e922ee940  0x28    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ed0c0  0x2c    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922edd00  0x30    EtwRegistration 0x804   -
980     winlogon.exe    0xb708012fede0  0x34    Directory       0x3     KnownDlls
980     winlogon.exe    0x840e9229afe0  0x38    Event   0x1f0003        -
980     winlogon.exe    0x840e9229b0e0  0x3c    Event   0x1f0003        -
980     winlogon.exe    0x840e922f3bd0  0x40    File    0x100020        \Device\HarddiskVolume3\Windows\System32
980     winlogon.exe    0x840e922ee780  0x44    EtwRegistration 0x804   -
980     winlogon.exe    0x840e921f1710  0x48    Mutant  0x1f0001        SM0:980:304:WilStaging_02
980     winlogon.exe    0x840e90c4d070  0x4c    ALPC Port       0x1f0001        -
980     winlogon.exe    0xb7080435e1a0  0x50    Directory       0xf     BaseNamedObjects
980     winlogon.exe    0x840e922ecc40  0x54    Semaphore       0x1f0003        SM0:980:304:WilStaging_02_p0
980     winlogon.exe    0x840e922eb8e0  0x58    Semaphore       0x1f0003        SM0:980:304:WilStaging_02_p0h
980     winlogon.exe    0x840e922ed1a0  0x5c    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ee400  0x60    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922eea20  0x64    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ed8c0  0x68    IoCompletion    0x1f0003        -
980     winlogon.exe    0x840e922e6d70  0x6c    TpWorkerFactory 0xf00ff -
980     winlogon.exe    0x840e922c80a0  0x70    IRTimer 0x100002        -
980     winlogon.exe    0x840e90ac69a0  0x74    WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e922c9910  0x78    IRTimer 0x100002        -
980     winlogon.exe    0x840e90ac6c10  0x7c    WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e922eeda0  0x80    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922eda60  0x84    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922edb40  0x88    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922eee80  0x8c    EtwRegistration 0x804   -
980     winlogon.exe    0xb708042bbd40  0x90    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NLS\SORTING\VERSIONS
980     winlogon.exe    0x840e922eeb00  0x94    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922eebe0  0x98    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ee320  0x9c    EtwRegistration 0x804   -
980     winlogon.exe    0x840e9229a260  0xa0    Semaphore       0x100003        -
980     winlogon.exe    0x840e9229ab60  0xa4    Semaphore       0x100003        -
980     winlogon.exe    0x840e9229a860  0xa8    Event   0x1f0003        -
980     winlogon.exe    0x840e922f46c0  0xac    File    0x100003        \Device\KsecDD
980     winlogon.exe    0x840e922ed280  0xb0    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922edc20  0xb4    EtwRegistration 0x804   -
980     winlogon.exe    0xb708042bc7e0  0xb8    Key     0xf003f MACHINE
980     winlogon.exe    0x840e9229ad60  0xbc    Event   0x1f0003        -
980     winlogon.exe    0x840e90ac6250  0xc0    WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e9229ace0  0xc4    Event   0x1f0003        -
980     winlogon.exe    0xb708042bdc10  0xc8    Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\OLE
980     winlogon.exe    0xb708042bdb00  0xcc    Key     0x20019 MACHINE
980     winlogon.exe    0xb708042bcf50  0xd0    Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS\SOFTWARE\MICROSOFT
980     winlogon.exe    0xb708042bc8f0  0xd4    Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS
980     winlogon.exe    0x840e9229a660  0xd8    Event   0x1f0003        -
980     winlogon.exe    0x840e922ee240  0xdc    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ee160  0xe0    EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ee5c0  0xe4    EtwRegistration 0x804   -
980     winlogon.exe    0x840e9229a560  0xe8    Event   0x1f0003        -
980     winlogon.exe    0x840e9229a160  0xec    Event   0x1f0003        -
980     winlogon.exe    0x840e9229a1e0  0xf0    Event   0x1f0003        -
980     winlogon.exe    0x840e9229a360  0xf4    Event   0x1f0003        -
980     winlogon.exe    0x840e9229a8e0  0xf8    Event   0x1f0003        -
980     winlogon.exe    0x840e9229a460  0xfc    Event   0x1f0003        -
980     winlogon.exe    0x840e922ee4e0  0x100   EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ed6e0  0x104   EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ee6a0  0x108   EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ee860  0x10c   EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ed360  0x110   EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ed520  0x114   EtwRegistration 0x804   -
980     winlogon.exe    0x840e922ed7c0  0x118   EtwRegistration 0x804   -
980     winlogon.exe    0xb708042bdd20  0x11c   Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER
980     winlogon.exe    0x840e922eef60  0x120   EtwRegistration 0x804   -
980     winlogon.exe    0x840e920fc860  0x124   EtwRegistration 0x804   -
980     winlogon.exe    0x840e9229a9e0  0x128   Semaphore       0x100003        -
980     winlogon.exe    0x840e9229a7e0  0x12c   Semaphore       0x100003        -
980     winlogon.exe    0x840e920fb0c0  0x130   EtwRegistration 0x804   -
980     winlogon.exe    0x840e920fb280  0x134   EtwRegistration 0x804   -
980     winlogon.exe    0x840e920fb360  0x138   EtwRegistration 0x804   -
980     winlogon.exe    0x840e920fb440  0x13c   EtwRegistration 0x804   -
980     winlogon.exe    0x840e920fb980  0x140   EtwRegistration 0x804   -
980     winlogon.exe    0xb708042bcd30  0x144   Key     0xf003f MACHINE\SOFTWARE\CLASSES
980     winlogon.exe    0xb708042bb080  0x148   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\WINLOGON
980     winlogon.exe    0x840e9229aa60  0x14c   Event   0x1f0003        -
980     winlogon.exe    0x840e9229b860  0x150   Event   0x1f0003        -
980     winlogon.exe    0x840e9229be60  0x154   Event   0x1f0003        -
980     winlogon.exe    0xb708042bd170  0x158   Key     0x9     MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\IMAGE FILE EXECUTION OPTIONS
980     winlogon.exe    0x840e922ed980  0x15c   EtwRegistration 0x804   -
980     winlogon.exe    0x840e920fc5c0  0x160   EtwRegistration 0x804   -
980     winlogon.exe    0x840e922eecc0  0x164   EtwRegistration 0x804   -
980     winlogon.exe    0x840e9229bfe0  0x168   Event   0x1f0003        -
980     winlogon.exe    0x840e92306080  0x16c   Thread  0x1fffff        Tid 984 Pid 980
980     winlogon.exe    0x840e922ca2c0  0x170   Event   0x1f0003        WinlogonLogoff
980     winlogon.exe    0x840e90ac6320  0x174   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e9229bee0  0x178   Event   0x1f0003        -
980     winlogon.exe    0x840e9230cda0  0x17c   ALPC Port       0x1f0001        WMsgKRpc01AF781
980     winlogon.exe    0x840e920fc160  0x180   EtwRegistration 0x804   -
980     winlogon.exe    0x840e9229c0e0  0x184   Event   0x1f0003        -
980     winlogon.exe    0x840e92271a40  0x188   IoCompletion    0x1f0003        -
980     winlogon.exe    0x840e92271400  0x18c   EtwRegistration 0x804   -
980     winlogon.exe    0x840e922fe810  0x190   Desktop 0xf01ff Disconnect
980     winlogon.exe    0x840e92273d00  0x194   Event   0x1f0003        BootShellComplete
980     winlogon.exe    0x840e9c711d60  0x19c   Semaphore       0x100003        -
980     winlogon.exe    0x840e922b9a80  0x1a0   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e920d54a0  0x1a4   WindowStation   0xf037f WinSta0
980     winlogon.exe    0x840e922fede0  0x1a8   Desktop 0xf01ff Winlogon
980     winlogon.exe    0xb7080ec3d910  0x1ac   Key     0xf003f USER\S-1-5-21-3294271487-4292050549-1661813568-1001
980     winlogon.exe    0x840e920d54a0  0x1b0   WindowStation   0xf037f WinSta0
980     winlogon.exe    0x840e9229bbe0  0x1b4   Event   0x1f0003        -
980     winlogon.exe    0xb708044d6c80  0x1b8   Key     0x20019 USER\.DEFAULT\CONTROL PANEL\INTERNATIONAL
980     winlogon.exe    0x840e922fd0d0  0x1bc   Desktop 0xf01ff Default
980     winlogon.exe    0x840e92271be0  0x1c0   EtwRegistration 0x804   -
980     winlogon.exe    0xb70804a96710  0x1c4   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NLS\SORTING\IDS
980     winlogon.exe    0x840e92cd72c0  0x1c8   EtwRegistration 0x804   -
980     winlogon.exe    0xb708049f1330  0x1d0   Section 0x4     Theme4128594183
980     winlogon.exe    0x840e92cd6e60  0x1d4   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92cfc2d0  0x1d8   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e91981d00  0x1dc   EtwRegistration 0x804   -
980     winlogon.exe    0x840e918ab710  0x1e0   Event   0x100000        TermSrvReadyEvent
980     winlogon.exe    0xb708049f0bb0  0x1e4   Section 0x4     Theme2228036042
980     winlogon.exe    0x840e923e96a0  0x1e8   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923a8960  0x1ec   Semaphore       0x100003        -
980     winlogon.exe    0x840e923a90e0  0x1f0   Semaphore       0x100003        -
980     winlogon.exe    0x840e923cba80  0x1f4   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e923e9a20  0x1f8   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92cdd320  0x1fc   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e923a81e0  0x200   Semaphore       0x100003        -
980     winlogon.exe    0x840e923a8360  0x204   Semaphore       0x100003        -
980     winlogon.exe    0x840e923a8160  0x208   Semaphore       0x100003        -
980     winlogon.exe    0x840e923a88e0  0x20c   Semaphore       0x100003        -
980     winlogon.exe    0x840e923a84e0  0x210   Semaphore       0x100003        -
980     winlogon.exe    0x840e923a8560  0x214   Semaphore       0x100003        -
980     winlogon.exe    0x840e923a8660  0x218   Semaphore       0x100003        -
980     winlogon.exe    0x840e923a9fe0  0x21c   Semaphore       0x100003        -
980     winlogon.exe    0x840e923e8440  0x220   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923e94e0  0x224   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92c4e170  0x228   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e923eaf20  0x22c   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923ea4a0  0x230   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923ea900  0x234   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923799f0  0x238   File    0x100001        \Device\CNG
980     winlogon.exe    0xb70804814610  0x23c   Key     0xf003f USER
980     winlogon.exe    0x840e923ea820  0x240   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923eaac0  0x244   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923eaba0  0x248   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923a7360  0x24c   Event   0x1f0003        -
980     winlogon.exe    0x840e92315b40  0x250   EtwRegistration 0x804   -
980     winlogon.exe    0x840e9237ab20  0x254   File    0x100080        \Device\Nsi
980     winlogon.exe    0xb708048130d0  0x258   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS\INTERFACES
980     winlogon.exe    0xb708048131e0  0x25c   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP6\PARAMETERS\INTERFACES
980     winlogon.exe    0x840e923eba80  0x260   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923ebb60  0x264   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923ebc40  0x268   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923ebd20  0x26c   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923bf320  0x270   EtwRegistration 0x804   -
980     winlogon.exe    0x840e923be980  0x274   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92368a30  0x278   Mutant  0x1f0001        SM0:980:120:WilError_03
980     winlogon.exe    0x840e92339920  0x27c   Semaphore       0x1f0003        SM0:980:120:WilError_03_p0
980     winlogon.exe    0x840e92339100  0x280   Semaphore       0x1f0003        SM0:980:120:WilError_03_p0h
980     winlogon.exe    0x840e92db6de0  0x288   Event   0x1f0003        -
980     winlogon.exe    0xb70804816b40  0x28c   Key     0x8     USER\.DEFAULT\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION
980     winlogon.exe    0x840e923bfbe0  0x290   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92cd6ca0  0x294   EtwRegistration 0x804   -
980     winlogon.exe    0xb708049ef0b0  0x298   Section 0x4     ThemeSection
980     winlogon.exe    0x840e92cd6bc0  0x29c   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92cd7640  0x2a0   EtwRegistration 0x804   -
980     winlogon.exe    0xb70804a9a180  0x2a4   Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\WINLOGON\NOTIFICATIONS\COMPONENTS\GPCLIENT
980     winlogon.exe    0x840e92d13d80  0x2a8   File    0x100001        \Device\KsecDD
980     winlogon.exe    0x840e92cd5dc0  0x2ac   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92cd8d00  0x2b0   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92db6de0  0x2b4   Event   0x2     -
980     winlogon.exe    0x840e92cd9160  0x2b8   EtwRegistration 0x804   -
980     winlogon.exe    0x840e96d198e0  0x2bc   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92cd8980  0x2c0   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92cd7aa0  0x2c4   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92cd7900  0x2c8   IoCompletion    0x1f0003        -
980     winlogon.exe    0x840e92cdc0a0  0x2cc   TpWorkerFactory 0xf00ff -
980     winlogon.exe    0x840e92cd2920  0x2d0   IRTimer 0x100002        -
980     winlogon.exe    0x840e92c4e7f0  0x2d4   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e92cd2b40  0x2d8   IRTimer 0x100002        -
980     winlogon.exe    0x840e92c4f0e0  0x2dc   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e96dd2160  0x2e0   Event   0x1f0003        -
980     winlogon.exe    0xb70805fae060  0x2e4   Token   0xf01ff -
980     winlogon.exe    0xb70804b66630  0x2e8   Token   0xb     -
980     winlogon.exe    0x840e99c1fde0  0x2ec   Semaphore       0x100003        -
980     winlogon.exe    0x840e96d19480  0x2f0   EtwRegistration 0x804   -
980     winlogon.exe    0xb70804b67630  0x2f4   Token   0x2f    -
980     winlogon.exe    0x840e9c36aae0  0x2f8   File    0x100001        \Device\HarddiskVolume3\Windows\System32\en-US\winlogon.exe.mui
980     winlogon.exe    0x840e92db79e0  0x2fc   Event   0x1f0003        -
980     winlogon.exe    0xb708061ba840  0x300   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\SERVICES\CRYPT32
980     winlogon.exe    0x840e922fd0d0  0x308   Desktop 0xf01ff Default
980     winlogon.exe    0x840e92dc6570  0x30c   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e92cfa140  0x310   Process 0x1fffff        dwm.exe Pid 1204
980     winlogon.exe    0x840e96de0160  0x318   Event   0x1f0003        -
980     winlogon.exe    0x840e9711e6e0  0x31c   Event   0x1f0003        -
980     winlogon.exe    0x840e92cd2700  0x320   IRTimer 0x1f0003        -
980     winlogon.exe    0x840e92c4f420  0x324   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e92db7460  0x32c   Event   0x1f0003        -
980     winlogon.exe    0x840e92db7fe0  0x330   Event   0x1f0003        -
980     winlogon.exe    0x840e92db7be0  0x334   Event   0x1f0003        -
980     winlogon.exe    0x840e92db7ae0  0x338   Event   0x1f0003        -
980     winlogon.exe    0x840e92db7c60  0x33c   Event   0x1f0003        -
980     winlogon.exe    0x840e92db8060  0x340   Event   0x1f0003        -
980     winlogon.exe    0x840e92db7860  0x344   Event   0x1f0003        -
980     winlogon.exe    0x840e92ce8080  0x348   Thread  0x1fffff        Tid 1188 Pid 980
980     winlogon.exe    0x840e92db7160  0x34c   Event   0x1f0003        -
980     winlogon.exe    0x840e92cdac80  0x350   IoCompletion    0x1f0003        -
980     winlogon.exe    0x840e92cd9cc0  0x354   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92e37e80  0x358   EtwRegistration 0x804   -
980     winlogon.exe    0x840e92cd95c0  0x35c   EtwRegistration 0x804   -
980     winlogon.exe    0xb70804a9ac20  0x360   Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\WINLOGON\NOTIFICATIONS\COMPONENTS\PROFILES
980     winlogon.exe    0xb70804a9af50  0x364   Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\WINLOGON\NOTIFICATIONS\COMPONENTS\SENS
980     winlogon.exe    0xb70804a9b4a0  0x368   Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\WINLOGON\NOTIFICATIONS\COMPONENTS\SESSIONENV
980     winlogon.exe    0xb70804a9b170  0x36c   Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\WINLOGON\NOTIFICATIONS\COMPONENTS\TERMSRV
980     winlogon.exe    0x840e92e402d0  0x370   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e92cdaba0  0x374   EtwRegistration 0x804   -
980     winlogon.exe    0x840e97cda080  0x378   File    0x100001        \Device\HarddiskVolume3\Windows\System32\en-US\user32.dll.mui
980     winlogon.exe    0x840e92e6aa80  0x37c   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e969a2b30  0x380   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e966bfce0  0x384   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e96660460  0x388   Event   0x1f0003        ThemesStartEvent
980     winlogon.exe    0x840e966a65c0  0x38c   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e9666d690  0x390   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e966bea80  0x398   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e96d19ac0  0x39c   IoCompletion    0x1f0003        -
980     winlogon.exe    0x840e96d1a280  0x3a0   EtwRegistration 0x804   -
980     winlogon.exe    0x840e96d19800  0x3a4   EtwRegistration 0x804   -
980     winlogon.exe    0x840e96dc7d80  0x3a8   TpWorkerFactory 0xf00ff -
980     winlogon.exe    0x840e96c6ae20  0x3ac   IRTimer 0x100002        -
980     winlogon.exe    0x840e969a4530  0x3b0   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e96c6a6b0  0x3b4   IRTimer 0x100002        -
980     winlogon.exe    0x840e969a3c40  0x3b8   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e96d1b400  0x3bc   EtwRegistration 0x804   -
980     winlogon.exe    0x840e96d1bcc0  0x3c0   EtwRegistration 0x804   -
980     winlogon.exe    0x840e97fb9a60  0x3c4   Semaphore       0x100003        -
980     winlogon.exe    0x840e975ac790  0x3c8   ALPC Port       0x1f0001        -
980     winlogon.exe    0xb70807a58060  0x3cc   Token   0xe     -
980     winlogon.exe    0x840e97fb9be0  0x3d0   Semaphore       0x100003        -
980     winlogon.exe    0x840e96863a80  0x3d4   ALPC Port       0x1f0001        -
980     winlogon.exe    0x840e9711e360  0x3d8   Event   0x1f0003        -
980     winlogon.exe    0xb708015635f0  0x3dc   Token   0xf01ff -
980     winlogon.exe    0xb70805fae770  0x3e0   Token   0xe     -
980     winlogon.exe    0x840e97818f60  0x3e8   Event   0x1f0003        -
980     winlogon.exe    0x840e966b9600  0x3ec   ALPC Port       0x1f0001        -
980     winlogon.exe    0xb70805fae770  0x3f0   Token   0xe     -
980     winlogon.exe    0xb7080770a770  0x3f4   Token   0xe     -
980     winlogon.exe    0x840e981a0990  0x3f8   WaitCompletionPacket    0x1     -
980     winlogon.exe    0xb7080ec3e4c0  0x3fc   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\WINDOWS\CURRENTVERSION\POLICIES\SYSTEM
980     winlogon.exe    0x840e9755d580  0x404   Event   0x1f0003        ShellDesktopSwitchEvent
980     winlogon.exe    0x840e97141c00  0x408   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e97568660  0x40c   Event   0x1f0003        0000000000144079_WlballoonKerberosNotificationEventName
980     winlogon.exe    0x840e97568700  0x410   Event   0x1f0003        0000000000143ff6_WlballoonKerberosNotificationEventName
980     winlogon.exe    0x840e97c4e6e0  0x414   Event   0x1f0003        -
980     winlogon.exe    0x840e97c4f4e0  0x418   Event   0x1f0003        -
980     winlogon.exe    0xb7080a325cb0  0x41c   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NETWORKPROVIDER\HWORDER
980     winlogon.exe    0xb7080a323890  0x420   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NETWORKPROVIDER\PROVIDERORDER
980     winlogon.exe    0x840e97c515e0  0x424   Semaphore       0x100003        -
980     winlogon.exe    0x840e97c51e60  0x428   Semaphore       0x100003        -
980     winlogon.exe    0x840e9764a230  0x42c   EtwRegistration 0x804   -
980     winlogon.exe    0x840e9817df70  0x430   WaitCompletionPacket    0x1     -
980     winlogon.exe    0xb708121b60d0  0x434   Key     0xf003f USER\S-1-5-21-3294271487-4292050549-1661813568-1001
980     winlogon.exe    0x840e994d1080  0x438   Process 0x1fffff        LogonUI.exe Pid 10680
980     winlogon.exe    0x840e97568a20  0x43c   Event   0x1f0003        0000000000144079_WlballoonKerberosCloudPasswordExpired
980     winlogon.exe    0x840e97568ac0  0x440   Event   0x1f0003        0000000000143ff6_WlballoonKerberosCloudPasswordExpired
980     winlogon.exe    0x840e981a1280  0x444   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e981a1aa0  0x448   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e97dc2b70  0x44c   Event   0x1f0003        0000000000144079_WlballoonNTLMNotificationEventName
980     winlogon.exe    0x840e97dc36b0  0x450   Event   0x1f0003        0000000000143ff6_WlballoonNTLMNotificationEventName
980     winlogon.exe    0x840e981a1de0  0x454   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e981a1760  0x458   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e97dc3d90  0x45c   Event   0x1f0003        0000000000144079_WlballoonSmartCardUnlockNotificationEventName
980     winlogon.exe    0x840e97dc3a70  0x460   Event   0x1f0003        0000000000143ff6_WlballoonSmartCardUnlockNotificationEventName
980     winlogon.exe    0x840e981a1b70  0x464   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e981a1900  0x468   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e97dc3b10  0x46c   Event   0x1f0003        0000000000144079_WlballoonAlternateCredsNotificationEventName
980     winlogon.exe    0x840e97dc2ad0  0x470   Event   0x1f0003        0000000000143ff6_WlballoonAlternateCredsNotificationEventName
980     winlogon.exe    0x840e981a1eb0  0x474   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e981a1f80  0x478   WaitCompletionPacket    0x1     -
980     winlogon.exe    0x840e99abf090  0x47c   EtwRegistration 0x804   -
980     winlogon.exe    0x840e9c7116e0  0x480   Semaphore       0x100003        -
980     winlogon.exe    0x840e99c1ebe0  0x488   Semaphore       0x100003        -
980     winlogon.exe    0x840e99550a40  0x49c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9229e060  0x4     Event   0x1f0003        -
1000    services.exe    0x840e9229d4e0  0x8     Event   0x1f0003        -
1000    services.exe    0x840e91bca520  0xc     WaitCompletionPacket    0x1     -
1000    services.exe    0x840e922715c0  0x10    IoCompletion    0x1f0003        -
1000    services.exe    0x840e9230fd70  0x14    TpWorkerFactory 0xf00ff -
1000    services.exe    0x840e922c9090  0x18    IRTimer 0x100002        -
1000    services.exe    0x840e91bca5f0  0x1c    WaitCompletionPacket    0x1     -
1000    services.exe    0x840e922c8c50  0x20    IRTimer 0x100002        -
1000    services.exe    0x840e91bcad40  0x24    WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92270360  0x28    EtwRegistration 0x804   -
1000    services.exe    0x840e92270b40  0x2c    EtwRegistration 0x804   -
1000    services.exe    0x840e92270520  0x30    EtwRegistration 0x804   -
1000    services.exe    0x840e9229de60  0x34    Event   0x1f0003        -
1000    services.exe    0x840e9229d260  0x38    Event   0x1f0003        -
1000    services.exe    0x840e922f6dd0  0x3c    File    0x100020        \Device\HarddiskVolume3\Windows\System32
1000    services.exe    0x840e92270de0  0x40    EtwRegistration 0x804   -
1000    services.exe    0x840e921f26d0  0x44    Mutant  0x1f0001        SM0:1000:304:WilStaging_02
1000    services.exe    0x840e922baa80  0x48    ALPC Port       0x1f0001        -
1000    services.exe    0xb7080435d660  0x4c    Directory       0xf     BaseNamedObjects
1000    services.exe    0x840e92272860  0x50    Semaphore       0x1f0003        SM0:1000:304:WilStaging_02_p0
1000    services.exe    0x840e92272f40  0x54    Semaphore       0x1f0003        SM0:1000:304:WilStaging_02_p0h
1000    services.exe    0x840e92270d00  0x58    EtwRegistration 0x804   -
1000    services.exe    0x840e922706e0  0x5c    EtwRegistration 0x804   -
1000    services.exe    0x840e92270600  0x60    EtwRegistration 0x804   -
1000    services.exe    0x840e922700c0  0x64    IoCompletion    0x1f0003        -
1000    services.exe    0x840e9230ed70  0x68    TpWorkerFactory 0xf00ff -
1000    services.exe    0x840e922c9a20  0x6c    IRTimer 0x100002        -
1000    services.exe    0x840e91bc9dd0  0x70    WaitCompletionPacket    0x1     -
1000    services.exe    0x840e922c9e60  0x74    IRTimer 0x100002        -
1000    services.exe    0x840e91bcb220  0x78    WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92271160  0x7c    EtwRegistration 0x804   -
1000    services.exe    0x840e9234ad70  0x80    TpWorkerFactory 0xf00ff -
1000    services.exe    0xb708044d6ea0  0x84    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NLS\SORTING\VERSIONS
1000    services.exe    0x840e92271b00  0x88    EtwRegistration 0x804   -
1000    services.exe    0x840e92271240  0x8c    EtwRegistration 0x804   -
1000    services.exe    0x840e9229d1e0  0x90    Semaphore       0x100003        -
1000    services.exe    0x840e9229d6e0  0x94    Semaphore       0x100003        -
1000    services.exe    0x840e9229d360  0x98    Event   0x1f0003        -
1000    services.exe    0x840e922f7be0  0x9c    File    0x100003        \Device\KsecDD
1000    services.exe    0x840e922707c0  0xa0    EtwRegistration 0x804   -
1000    services.exe    0x840e92271320  0xa4    EtwRegistration 0x804   -
1000    services.exe    0x840e92270ec0  0xa8    EtwRegistration 0x804   -
1000    services.exe    0x840e92270980  0xac    EtwRegistration 0x804   -
1000    services.exe    0x840e922fade0  0xb0    File    0x120089        \Device\DeviceApi\CMApi
1000    services.exe    0xb708044d6b70  0xb4    Key     0xf003f MACHINE
1000    services.exe    0x840e92271940  0xb8    EtwRegistration 0x804   -
1000    services.exe    0x840e92271780  0xbc    EtwRegistration 0x804   -
1000    services.exe    0x840e92271860  0xc0    EtwRegistration 0x804   -
1000    services.exe    0x840e92271da0  0xc4    EtwRegistration 0x804   -
1000    services.exe    0x840e92271e80  0xc8    EtwRegistration 0x804   -
1000    services.exe    0x840e92271f60  0xcc    EtwRegistration 0x804   -
1000    services.exe    0x840e92271cc0  0xd0    EtwRegistration 0x804   -
1000    services.exe    0xb708044ae060  0xd4    Token   0x2a    -
1000    services.exe    0x840e92273da0  0xd8    Event   0x1f0003        SC_AutoStartComplete
1000    services.exe    0x840e9229d460  0xdc    Event   0x1f0003        -
1000    services.exe    0x840e91bca930  0xe0    WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9229d560  0xe4    Event   0x1f0003        -
1000    services.exe    0xb708044d71d0  0xe8    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\NETWORKPROVIDER\ORDER
1000    services.exe    0x840e922743e0  0xec    Event   0x1f0003        SC_AutoStartPhase1Done
1000    services.exe    0x840e92274340  0xf0    Event   0x1f0003        SvcctrlStartEvent_A3752DX
1000    services.exe    0x840e91982b00  0xf4    IoCompletion    0x1f0003        -
1000    services.exe    0x840e922c9b30  0xf8    IRTimer 0x100002        -
1000    services.exe    0x840e91bca2b0  0xfc    WaitCompletionPacket    0x1     -
1000    services.exe    0x840e922c81b0  0x100   IRTimer 0x100002        -
1000    services.exe    0x840e91bcb630  0x104   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9229d5e0  0x108   Semaphore       0x100003        -
1000    services.exe    0x840e9229d660  0x10c   Semaphore       0x100003        -
1000    services.exe    0x840e9229d760  0x110   Semaphore       0x100003        -
1000    services.exe    0x840e9229d7e0  0x114   Semaphore       0x100003        -
1000    services.exe    0xb708044d6510  0x118   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\SERVICEGROUPORDER
1000    services.exe    0x840e92318380  0x11c   EtwRegistration 0x804   -
1000    services.exe    0xb708044d72e0  0x120   Key     0x2001d MACHINE\SYSTEM\CONTROLSET001\SERVICES
1000    services.exe    0x840e92375210  0x124   File    0x100001        \Device\HarddiskVolume3\Windows\ServiceState
1000    services.exe    0x840e923a6fe0  0x128   Semaphore       0x100003        -
1000    services.exe    0x840e92318460  0x12c   EtwRegistration 0x804   -
1000    services.exe    0x840e923a5c60  0x130   Event   0x1f0003        -
1000    services.exe    0x840e923a5560  0x134   Event   0x1f0003        -
1000    services.exe    0x840e923a5ce0  0x138   Event   0x1f0003        -
1000    services.exe    0x840e923a5fe0  0x13c   Event   0x1f0003        -
1000    services.exe    0x840e92316cc0  0x140   EtwRegistration 0x804   -
1000    services.exe    0xb708044dc7e0  0x144   Key     0x9     MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\IMAGE FILE EXECUTION OPTIONS
1000    services.exe    0x840e923187e0  0x148   EtwRegistration 0x804   -
1000    services.exe    0x840e92317200  0x14c   EtwRegistration 0x804   -
1000    services.exe    0xb70809771060  0x150   Token   0xf01ff -
1000    services.exe    0x840e9f00ff50  0x154   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70810d67870  0x158   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e9239b590  0x15c   ALPC Port       0x1f0001        ntsvcs
1000    services.exe    0x840e92376340  0x160   File    0x100000        \Device\NamedPipe\ntsvcs
1000    services.exe    0x840e92376980  0x164   File    0x12019f        \Device\NamedPipe\ntsvcs
1000    services.exe    0x840e92375e90  0x168   File    0x12019f        \Device\NamedPipe\ntsvcs
1000    services.exe    0x840e923a6760  0x16c   Semaphore       0x100003        -
1000    services.exe    0x840e92316e80  0x170   EtwRegistration 0x804   -
1000    services.exe    0xb70804814830  0x174   Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER
1000    services.exe    0x840e923a64e0  0x178   Event   0x1f0003        -
1000    services.exe    0xb708048151c0  0x17c   Key     0x20019 USER\.DEFAULT\CONTROL PANEL\INTERNATIONAL
1000    services.exe    0xb70808d9f060  0x180   Token   0xf01ff -
1000    services.exe    0xb70813385060  0x184   Token   0xf01ff -
1000    services.exe    0x840e923756c0  0x188   File    0x12019f        \Device\KsecDD
1000    services.exe    0x840e923a1c50  0x18c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92374a40  0x190   File    0x12019f        \Device\NamedPipe\scerpc
1000    services.exe    0x840e92374720  0x194   File    0x100000        \Device\NamedPipe\scerpc
1000    services.exe    0x840e923764d0  0x198   File    0x12019f        \Device\NamedPipe\scerpc
1000    services.exe    0x840e923a6f60  0x19c   Semaphore       0x100003        -
1000    services.exe    0x840e99c6e080  0x1a0   Process 0x1fffff        svchost.exe Pid 4636
1000    services.exe    0x840e92318b60  0x1a4   EtwRegistration 0x804   -
1000    services.exe    0x840e923a6a60  0x1a8   Semaphore       0x100003        -
1000    services.exe    0x840e92318c40  0x1ac   EtwRegistration 0x804   -
1000    services.exe    0x840e92318ee0  0x1b0   EtwRegistration 0x804   -
1000    services.exe    0x840e92318a80  0x1b4   EtwRegistration 0x804   -
1000    services.exe    0x840e90c06a80  0x1bc   ALPC Port       0x1f0001        -
1000    services.exe    0x840e923e8ec0  0x1c0   EtwRegistration 0x804   -
1000    services.exe    0x840e92318e00  0x1c4   EtwRegistration 0x804   -
1000    services.exe    0x840e923e9be0  0x1c8   EtwRegistration 0x804   -
1000    services.exe    0x840e923189a0  0x1cc   EtwRegistration 0x804   -
1000    services.exe    0x840e923e8d00  0x1d0   EtwRegistration 0x804   -
1000    services.exe    0x840e923e9080  0x1d4   EtwRegistration 0x804   -
1000    services.exe    0x840e923e9780  0x1d8   EtwRegistration 0x804   -
1000    services.exe    0x840e923e9160  0x1dc   EtwRegistration 0x804   -
1000    services.exe    0x840e923e8de0  0x1e0   EtwRegistration 0x804   -
1000    services.exe    0x840e923a72e0  0x1e4   Semaphore       0x1f0003        -
1000    services.exe    0x840e923a7e60  0x1e8   Event   0x1f0003        -
1000    services.exe    0x840e923a7ae0  0x1ec   Event   0x1f0003        -
1000    services.exe    0x840e923a7fe0  0x1f0   Event   0x1f0003        -
1000    services.exe    0x840e923a7460  0x1f4   Event   0x1f0003        -
1000    services.exe    0xb708045fa910  0x1f8   Directory       0xf000f WUDF
1000    services.exe    0x840e923e8c40  0x1fc   IoCompletion    0x1f0003        -
1000    services.exe    0x840e923c8310  0x200   TpWorkerFactory 0xf00ff -
1000    services.exe    0x840e9182cd60  0x204   IRTimer 0x100002        -
1000    services.exe    0x840e90c642d0  0x208   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9182db30  0x20c   IRTimer 0x100002        -
1000    services.exe    0x840e90ac64c0  0x210   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e923a7960  0x214   Event   0x1f0003        -
1000    services.exe    0x840e923a7560  0x218   Event   0x1f0003        -
1000    services.exe    0x840e923a7de0  0x21c   Event   0x1f0003        -
1000    services.exe    0x840e923a79e0  0x220   Event   0x1f0003        -
1000    services.exe    0x840e922fbac0  0x224   Timer   0x1f0003        -
1000    services.exe    0x840e923a7ee0  0x228   Event   0x1f0003        -
1000    services.exe    0x840e9239b090  0x22c   ALPC Port       0x1f0001        ProcessManagement
1000    services.exe    0x840e923a80e0  0x230   Event   0x1f0003        -
1000    services.exe    0x840e923d3080  0x234   Thread  0x1fffff        Tid 668 Pid 1000
1000    services.exe    0x840e923a1530  0x238   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9c902e60  0x23c   Event   0x1f0003        -
1000    services.exe    0x840e990b23c0  0x240   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92cab2c0  0x244   Process 0x1fffff        svchost.exe Pid 1124
1000    services.exe    0xb70807c3c060  0x248   Token   0xf01ff -
1000    services.exe    0xb70816c11970  0x24c   Token   0xf01ff -
1000    services.exe    0x840e923e9240  0x250   EtwRegistration 0x804   -
1000    services.exe    0x840e923e95c0  0x254   EtwRegistration 0x804   -
1000    services.exe    0x840e92db6160  0x258   Event   0x1f0003        -
1000    services.exe    0xb70804a9a4b0  0x25c   Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS
1000    services.exe    0x840e923de2c0  0x260   Process 0x1fffff        svchost.exe Pid 756
1000    services.exe    0x840e923afe60  0x264   Event   0x1f0003        -
1000    services.exe    0x840e923793b0  0x268   File    0x100001        \Device\CNG
1000    services.exe    0x840e9bba8780  0x26c   File    0x100001        \Device\KsecDD
1000    services.exe    0x840e9746d260  0x270   Event   0x1f0003        -
1000    services.exe    0xb7080484e060  0x274   Token   0xf01ff -
1000    services.exe    0x840e923d7d20  0x278   ALPC Port       0x1f0001        -
1000    services.exe    0xb70804813730  0x27c   Key     0xf003f USER
1000    services.exe    0x840e923a8f60  0x280   Event   0x1f0003        -
1000    services.exe    0x840e923eb380  0x284   EtwRegistration 0x804   -
1000    services.exe    0x840ea5d2f080  0x288   Process 0x1fffff        svchost.exe Pid 9444
1000    services.exe    0xb7080976a730  0x28c   Token   0xf01ff -
1000    services.exe    0x840e91bce890  0x290   WaitCompletionPacket    0x1     -
1000    services.exe    0xb708070f1730  0x294   Token   0xf01ff -
1000    services.exe    0x840e92c11ec0  0x298   IoCompletion    0x1f0003        -
1000    services.exe    0x840e9237a990  0x29c   File    0x100001        \Device\HarddiskVolume3\Windows\System32\en-US\services.exe.mui
1000    services.exe    0x840e923e2a80  0x2a0   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92c4b3f0  0x2a4   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e923e3a80  0x2a8   ALPC Port       0x1f0001        -
1000    services.exe    0x840e923c6b60  0x2ac   TpWorkerFactory 0xf00ff -
1000    services.exe    0x840e923914d0  0x2b0   IRTimer 0x100002        -
1000    services.exe    0x840e92c4b0b0  0x2b4   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e923912b0  0x2b8   IRTimer 0x100002        -
1000    services.exe    0x840e92c4b180  0x2bc   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92cd8ec0  0x2c0   EtwRegistration 0x804   -
1000    services.exe    0xb708049b1830  0x2c4   Token   0xf01ff -
1000    services.exe    0x840e92ca5a80  0x2c8   ALPC Port       0x1f0001        -
1000    services.exe    0xb708044dca00  0x2cc   Key     0x2001f USER\S-1-5-20
1000    services.exe    0x840e92db6be0  0x2d0   Event   0x1f0003        -
1000    services.exe    0xb70804aa9830  0x2d4   Token   0xf01ff -
1000    services.exe    0xb70804a98800  0x2d8   Key     0x20019 MACHINE
1000    services.exe    0x840e92c9a340  0x2dc   Process 0x1fffff        svchost.exe Pid 1076
1000    services.exe    0x840e92db6ce0  0x2e0   Event   0x1f0003        -
1000    services.exe    0x840e923ae0e0  0x2e4   Event   0x1f0003        -
1000    services.exe    0x840e92c4d880  0x2e8   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92cd7c60  0x2ec   EtwRegistration 0x804   -
1000    services.exe    0x840e92cd8de0  0x2f0   EtwRegistration 0x804   -
1000    services.exe    0x840e92db6e60  0x2f4   Event   0x1f0003        -
1000    services.exe    0x840e92cae2d0  0x2f8   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92db6f60  0x2fc   Event   0x1f0003        -
1000    services.exe    0x840e92caedb0  0x300   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92db67e0  0x304   Event   0x1f0003        -
1000    services.exe    0x840e92db6fe0  0x308   Event   0x1f0003        -
1000    services.exe    0x840e92db6660  0x30c   Event   0x1f0003        -
1000    services.exe    0xb70804a996e0  0x310   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\OLE
1000    services.exe    0xb70804a99900  0x314   Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS\SOFTWARE\MICROSOFT
1000    services.exe    0x840e92cd8600  0x318   EtwRegistration 0x804   -
1000    services.exe    0x840e92cd9320  0x31c   EtwRegistration 0x804   -
1000    services.exe    0x840e92cd9240  0x320   EtwRegistration 0x804   -
1000    services.exe    0x840e92cd8360  0x324   EtwRegistration 0x804   -
1000    services.exe    0x840e92cd8c20  0x328   EtwRegistration 0x804   -
1000    services.exe    0x840e92cd7d40  0x32c   EtwRegistration 0x804   -
1000    services.exe    0x840e92cd86e0  0x330   EtwRegistration 0x804   -
1000    services.exe    0x840e92cd9400  0x334   EtwRegistration 0x804   -
1000    services.exe    0x840e92cf42d0  0x338   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92e07ce0  0x33c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92cf3530  0x340   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92c53860  0x344   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92e28a80  0x34c   ALPC Port       0x1f0001        -
1000    services.exe    0x840ea0286c20  0x350   File    0x16019f        \Device\Afd\Endpoint
1000    services.exe    0xb70804ebd0a0  0x354   Token   0xf01ff -
1000    services.exe    0x840e99fcec60  0x358   Event   0x1f0003        -
1000    services.exe    0xb70804f8f060  0x35c   Token   0xf01ff -
1000    services.exe    0x840e918e90c0  0x360   Process 0x1fffff        svchost.exe Pid 14128
1000    services.exe    0x840eaa185d20  0x364   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70804c160d0  0x368   Key     0x8     USER\.DEFAULT\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION
1000    services.exe    0x840e92c554d0  0x36c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e981a3570  0x370   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70816b11770  0x374   Token   0xf01ff -
1000    services.exe    0x840e9819f610  0x378   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e97141650  0x37c   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70804d2b3f0  0x380   Token   0xf01ff -
1000    services.exe    0xb708050fa730  0x384   Token   0xf01ff -
1000    services.exe    0x840e92f23080  0x390   Process 0x1fffff        svchost.exe Pid 1776
1000    services.exe    0x840e92def080  0x394   Process 0x1fffff        svchost.exe Pid 1292
1000    services.exe    0xb70804d50820  0x398   Token   0xf01ff -
1000    services.exe    0x840e92dba660  0x39c   Event   0x1f0003        -
1000    services.exe    0xb70804c192c0  0x3a0   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e92e56860  0x3a4   Event   0x1f0003        -
1000    services.exe    0xb70813589550  0x3a8   Key     0x2001f USER\S-1-5-20
1000    services.exe    0x840e90bbfe60  0x3ac   Event   0x1f0003        -
1000    services.exe    0xb70804f790a0  0x3b0   Token   0xf01ff -
1000    services.exe    0x840e92e3aac0  0x3b4   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92f09a80  0x3b8   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9844a9f0  0x3bc   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92d2e240  0x3c0   Process 0x1fffff        svchost.exe Pid 1320
1000    services.exe    0x840e978cd870  0x3c4   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92dbd0e0  0x3c8   Event   0x1f0003        -
1000    services.exe    0xb70804d62060  0x3cc   Token   0xf01ff -
1000    services.exe    0x840e9968d080  0x3d0   Process 0x1fffff        splunkd.exe Pid 13620
1000    services.exe    0xb70804c1a2b0  0x3d4   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e92e48080  0x3d8   Process 0x1fffff        svchost.exe Pid 1524
1000    services.exe    0x840e92d48080  0x3dc   Process 0x1fffff        svchost.exe Pid 1364
1000    services.exe    0x840e97fc96e0  0x3e0   Event   0x1f0003        -
1000    services.exe    0x840e9c9269e0  0x3e4   Event   0x1f0003        -
1000    services.exe    0x840e9ba3d270  0x3e8   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92dbe660  0x3ec   Event   0x1f0003        -
1000    services.exe    0xb70813591800  0x3f0   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e92df6a80  0x3f4   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92c51980  0x3f8   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e98b31a70  0x3fc   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70804e24830  0x404   Token   0xf01ff -
1000    services.exe    0xb70804c17830  0x408   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e92edbb90  0x40c   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70804c1a5e0  0x410   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e92e537e0  0x414   Event   0x1f0003        -
1000    services.exe    0xb70804fd5910  0x418   Key     0x2001f USER\S-1-5-19
1000    services.exe    0xb70804ebd8b0  0x41c   Token   0xf01ff -
1000    services.exe    0xb70804e32060  0x420   Token   0xf01ff -
1000    services.exe    0x840e92e04300  0x424   Process 0x1fffff        svchost.exe Pid 1408
1000    services.exe    0x840e97f39cb0  0x428   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96c5a080  0x42c   Process 0x1fffff        svchost.exe Pid 7108
1000    services.exe    0xb7081f8a2970  0x430   Token   0xf01ff -
1000    services.exe    0x840e9689a080  0x434   Process 0x1fffff        svchost.exe Pid 2472
1000    services.exe    0x840e92e30080  0x438   Process 0x1fffff        svchost.exe Pid 1392
1000    services.exe    0x840e92db7660  0x440   Event   0x1f0003        -
1000    services.exe    0x840e92dc10e0  0x444   Event   0x1f0003        -
1000    services.exe    0x840e92edb1d0  0x448   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e90ad3080  0x44c   Process 0x1fffff        svchost.exe Pid 7152
1000    services.exe    0x840e993fbd30  0x450   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92e2ca80  0x454   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92c54080  0x458   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e91c2a080  0x45c   Process 0x1fffff        svchost.exe Pid 5340
1000    services.exe    0x840e92c536c0  0x460   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92e732c0  0x464   Process 0x1fffff        svchost.exe Pid 1580
1000    services.exe    0xb70804fcf420  0x468   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e91a68d20  0x46c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9234aa80  0x470   ALPC Port       0x1f0001        -
1000    services.exe    0xb708053ae730  0x474   Token   0xf01ff -
1000    services.exe    0x840e90ac6730  0x478   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92c58c10  0x47c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92ee6220  0x480   WaitCompletionPacket    0x1     -
1000    services.exe    0xb708056c3280  0x484   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\SERVICES\SAMSS
1000    services.exe    0xb70804c21460  0x488   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e92399a10  0x48c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e97dc72c0  0x490   Process 0x1fffff        SearchIndexer. Pid 1852
1000    services.exe    0x840e9f3fc690  0x494   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92eac340  0x498   Process 0x1fffff        svchost.exe Pid 1612
1000    services.exe    0xb708053083d0  0x49c   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e92eaa2c0  0x4a0   Process 0x1fffff        svchost.exe Pid 1604
1000    services.exe    0x840e92e587e0  0x4a4   Event   0x1f0003        -
1000    services.exe    0x840e923f7dc0  0x4a8   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92e546e0  0x4ac   Event   0x1f0003        -
1000    services.exe    0x840e92fdb070  0x4b0   ALPC Port       0x1f0001        -
1000    services.exe    0xb708051ac0a0  0x4b4   Token   0xf01ff -
1000    services.exe    0x840e92f04340  0x4b8   Process 0x1fffff        svchost.exe Pid 1652
1000    services.exe    0x840e92f202c0  0x4bc   Process 0x1fffff        svchost.exe Pid 1764
1000    services.exe    0x840e92e5cae0  0x4c0   Event   0x1f0003        -
1000    services.exe    0x840e92e61d60  0x4c4   Event   0x1f0003        -
1000    services.exe    0x840e92ea8d90  0x4c8   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92eb4ce0  0x4cc   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92edcbd0  0x4d0   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92e619e0  0x4d4   Event   0x1f0003        -
1000    services.exe    0xb70804fd7a00  0x4d8   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e92fc3a80  0x4dc   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92f68e60  0x4e0   Event   0x1f0003        -
1000    services.exe    0x840e92fc7080  0x4e4   Process 0x1fffff        svchost.exe Pid 1988
1000    services.exe    0x840e91c3b300  0x4e8   Process 0x1fffff        svchost.exe Pid 8312
1000    services.exe    0x840e99c28160  0x4ec   Event   0x1f0003        -
1000    services.exe    0x840e92f27080  0x4f0   Process 0x1fffff        svchost.exe Pid 1808
1000    services.exe    0x840e968c22c0  0x4f4   Process 0x1fffff        svchost.exe Pid 2504
1000    services.exe    0x840e92f4fde0  0x4f8   Event   0x1f0003        -
1000    services.exe    0x840e96769240  0x4fc   Process 0x1fffff        svchost.exe Pid 2160
1000    services.exe    0x840e96670ea0  0x500   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92fda070  0x504   ALPC Port       0x1f0001        -
1000    services.exe    0xb708108e9890  0x508   Token   0xf01ff -
1000    services.exe    0x840e92f28ce0  0x50c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92ee86b0  0x510   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9b997980  0x514   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92f4f8e0  0x518   Event   0x1f0003        -
1000    services.exe    0x840e98a882c0  0x520   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92ee27a0  0x524   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9677bb60  0x528   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92fc5300  0x52c   Process 0x1fffff        svchost.exe Pid 1976
1000    services.exe    0xb7080571a7d0  0x530   Token   0xf01ff -
1000    services.exe    0x840e966a6820  0x534   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92f5a7e0  0x538   Event   0x1f0003        -
1000    services.exe    0x840e966d62c0  0x53c   Process 0x1fffff        svchost.exe Pid 2092
1000    services.exe    0x840e92f6dfe0  0x540   Event   0x1f0003        -
1000    services.exe    0x840e92fc0a80  0x544   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9666a430  0x548   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96668210  0x54c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92fc6080  0x550   Process 0x1fffff        svchost.exe Pid 1964
1000    services.exe    0x840e9905a2d0  0x554   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92f5bd60  0x558   Event   0x1f0003        -
1000    services.exe    0x840e96805f30  0x55c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92f5a1e0  0x560   Event   0x1f0003        -
1000    services.exe    0x840e97e8c080  0x568   Process 0x1fffff        svchost.exe Pid 3864
1000    services.exe    0x840e97809c60  0x56c   Semaphore       0x100003        -
1000    services.exe    0xb708055eb060  0x570   Token   0xf01ff -
1000    services.exe    0xb70804c17610  0x574   Key     0x2001f USER\S-1-5-20
1000    services.exe    0xb70805446060  0x578   Token   0xf01ff -
1000    services.exe    0x840e96668a30  0x57c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96be7360  0x580   Event   0x1f0003        -
1000    services.exe    0x840e92fbb0c0  0x584   Process 0x1fffff        svchost.exe Pid 604
1000    services.exe    0x840e92f5ea60  0x588   Event   0x1f0003        -
1000    services.exe    0x840e96668480  0x58c   WaitCompletionPacket    0x1     -
1000    services.exe    0xb7080564f7d0  0x590   Token   0xf01ff -
1000    services.exe    0x840e92fc3ce0  0x594   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92f5d460  0x598   Event   0x1f0003        -
1000    services.exe    0x840e96788ce0  0x59c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e966d7740  0x5a0   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9666d420  0x5a4   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70805311bc0  0x5a8   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e96c97a80  0x5ac   ALPC Port       0x1f0001        -
1000    services.exe    0xb708056c6580  0x5b0   Key     0x2001f USER\S-1-5-20
1000    services.exe    0xb708056860a0  0x5b4   Token   0xf01ff -
1000    services.exe    0x840e97142b70  0x5b8   WaitCompletionPacket    0x1     -
1000    services.exe    0xb708056bfb40  0x5bc   Key     0x2001f USER\S-1-5-19
1000    services.exe    0xb708056970a0  0x5c0   Token   0xf01ff -
1000    services.exe    0xb7080530d490  0x5c4   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e96794080  0x5c8   Process 0x1fffff        svchost.exe Pid 2276
1000    services.exe    0xb70804f77060  0x5cc   Token   0xf01ff -
1000    services.exe    0xb708073f9970  0x5d0   Token   0xf01ff -
1000    services.exe    0x840e9678ace0  0x5d4   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96672970  0x5d8   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96787ce0  0x5dc   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9676b300  0x5e0   Process 0x1fffff        svchost.exe Pid 2180
1000    services.exe    0x840e96892de0  0x5e4   ALPC Port       0x1f0001        -
1000    services.exe    0x840e967cd760  0x5e8   Event   0x1f0003        -
1000    services.exe    0x840e9676e080  0x5ec   Process 0x1fffff        svchost.exe Pid 2168
1000    services.exe    0x840e96671380  0x5f0   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e92f68b60  0x5f4   Event   0x1f0003        -
1000    services.exe    0xb70805b983d0  0x5f8   Token   0xf01ff -
1000    services.exe    0xb708081ba570  0x5fc   Token   0xf01ff -
1000    services.exe    0xb70805b9c6f0  0x600   Token   0xf01ff -
1000    services.exe    0xb708056c8670  0x604   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e96862080  0x608   Process 0x1fffff        svchost.exe Pid 2376
1000    services.exe    0x840e967d6460  0x60c   Event   0x1f0003        -
1000    services.exe    0xb70805ccb060  0x610   Token   0xf01ff -
1000    services.exe    0x840e96e77080  0x614   Process 0x1fffff        svchost.exe Pid 3636
1000    services.exe    0x840e96b0dce0  0x618   ALPC Port       0x1f0001        -
1000    services.exe    0x840e97f7e080  0x61c   Process 0x1fffff        svchost.exe Pid 5772
1000    services.exe    0x840e967cf3e0  0x620   Event   0x1f0003        -
1000    services.exe    0x840e96860080  0x624   Process 0x1fffff        svchost.exe Pid 2396
1000    services.exe    0xb70805bc9880  0x628   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e96864380  0x62c   Process 0x1fffff        svchost.exe Pid 2384
1000    services.exe    0xb70805c020a0  0x630   Token   0xf01ff -
1000    services.exe    0x840e91bcb700  0x634   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e967d10e0  0x638   Event   0x1f0003        -
1000    services.exe    0xb70805bc5150  0x63c   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e968c00c0  0x640   Process 0x1fffff        svchost.exe Pid 2496
1000    services.exe    0x840e967d0860  0x644   Event   0x1f0003        -
1000    services.exe    0x840ea08f1080  0x648   Process 0x1fffff        svchost.exe Pid 3444
1000    services.exe    0x840e968635c0  0x64c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9680dab0  0x650   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e967d5560  0x654   Event   0x1f0003        -
1000    services.exe    0x840e96884ce0  0x658   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9680fda0  0x65c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e967e72e0  0x660   Event   0x1f0003        -
1000    services.exe    0x840e96813dd0  0x664   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96bc00c0  0x668   Process 0x1fffff        svchost.exe Pid 2848
1000    services.exe    0x840e967d62e0  0x66c   Event   0x1f0003        -
1000    services.exe    0xb70806407920  0x670   Token   0xf01ff -
1000    services.exe    0xb708073563a0  0x674   Token   0xf01ff -
1000    services.exe    0xb7080605f7f0  0x678   Token   0xf01ff -
1000    services.exe    0xb70807057870  0x67c   Token   0xf01ff -
1000    services.exe    0xb70805d3f060  0x680   Token   0xf01ff -
1000    services.exe    0xb70805bcd730  0x684   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e91bc3df0  0x688   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96c582c0  0x68c   Process 0x1fffff        svchost.exe Pid 3028
1000    services.exe    0xb708061a2360  0x690   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e9ac37680  0x694   WaitCompletionPacket    0x1     -
1000    services.exe    0xb708061ae7f0  0x698   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e96b0b850  0x69c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e90986540  0x6a0   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9680fcd0  0x6a4   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96c23280  0x6a8   Process 0x1fffff        spoolsv.exe Pid 2904
1000    services.exe    0x840e91bc9270  0x6ac   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96be16e0  0x6b0   Event   0x1f0003        -
1000    services.exe    0xb70807211060  0x6b4   Token   0xf01ff -
1000    services.exe    0x840e96bba530  0x6b8   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96b0c070  0x6bc   ALPC Port       0x1f0001        -
1000    services.exe    0xb708061b0e30  0x6c0   Key     0x2001f USER\S-1-5-20
1000    services.exe    0x840e969999c0  0x6c4   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e91bcba40  0x6c8   WaitCompletionPacket    0x1     -
1000    services.exe    0x840ea47c52b0  0x6cc   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e91c47070  0x6d0   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96e042c0  0x6d4   Process 0x1fffff        svchost.exe Pid 3376
1000    services.exe    0xb708061aa2e0  0x6d8   Key     0x2001f USER\S-1-5-19
1000    services.exe    0xb70807d780a0  0x6dc   Token   0xf01ff -
1000    services.exe    0x840e9c714160  0x6e0   Event   0x1f0003        -
1000    services.exe    0x840e9722ba80  0x6e4   ALPC Port       0x1f0001        -
1000    services.exe    0x840e97a9fda0  0x6e8   File    0x100001        \Device\HarddiskVolume3\Windows\System32\en-US\combase.dll.mui
1000    services.exe    0x840e96997a10  0x6ec   WaitCompletionPacket    0x1     -
1000    services.exe    0xb708075a7770  0x6f0   Token   0xf01ff -
1000    services.exe    0x840e972491c0  0x6f4   Process 0x1fffff        svchost.exe Pid 4352
1000    services.exe    0x840e96e6f070  0x6f8   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96beb760  0x6fc   Event   0x1f0003        -
1000    services.exe    0x840e96c5e1c0  0x700   Process 0x1fffff        svchost.exe Pid 2220
1000    services.exe    0x840e96997bb0  0x704   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96c61080  0x708   Process 0x1fffff        svchost.exe Pid 2216
1000    services.exe    0x840e96942ce0  0x70c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96beb8e0  0x710   Event   0x1f0003        -
1000    services.exe    0xb70807710730  0x714   Token   0xf01ff -
1000    services.exe    0x840e96bf8a60  0x718   Event   0x1f0003        -
1000    services.exe    0x840e96cdd1c0  0x71c   Process 0x1fffff        svchost.exe Pid 740
1000    services.exe    0x840e92eeea80  0x720   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96beeee0  0x724   Event   0x1f0003        -
1000    services.exe    0x840e96c94d60  0x728   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96d9f080  0x72c   Process 0x1fffff        svchost.exe Pid 3308
1000    services.exe    0x840e96997870  0x730   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e90de2080  0x734   Process 0x1fffff        svchost.exe Pid 1480
1000    services.exe    0xb70807c19730  0x738   Token   0xf01ff -
1000    services.exe    0xb70808cc0970  0x73c   Token   0xf01ff -
1000    services.exe    0x840e981816b0  0x740   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e968127e0  0x744   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9680b3b0  0x74c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e90b0d080  0x750   Process 0x1fffff        svchost.exe Pid 9452
1000    services.exe    0x840e97c3b0d0  0x754   File    0x16019f        \Device\Afd\Endpoint
1000    services.exe    0x840e97f240b0  0x758   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96f0c790  0x75c   ALPC Port       0x1f0001        -
1000    services.exe    0xb70808573060  0x764   Token   0xf01ff -
1000    services.exe    0x840e9b023dd0  0x768   ALPC Port       0x1f0001        -
1000    services.exe    0x840e97112460  0x76c   Event   0x1f0003        -
1000    services.exe    0x840e97151640  0x770   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e97c5f860  0x774   Event   0x1f0003        -
1000    services.exe    0x840e97c3b260  0x778   File    0x160089        \Device\NamedPipe\Winsock2\CatalogChangeListener-3e8-0
1000    services.exe    0x840ea5a101e0  0x77c   Event   0x1f0003        -
1000    services.exe    0xb70817f37060  0x780   Token   0xf01ff -
1000    services.exe    0x840e96f6d2c0  0x784   Process 0x1fffff        svchost.exe Pid 3880
1000    services.exe    0x840e97684a70  0x788   EtwRegistration 0x804   -
1000    services.exe    0x840e96de53e0  0x78c   Event   0x1f0003        -
1000    services.exe    0x840e97683b90  0x790   EtwRegistration 0x804   -
1000    services.exe    0x840e96e0d080  0x794   Process 0x1fffff        svchost.exe Pid 3844
1000    services.exe    0x840e97685090  0x798   EtwRegistration 0x804   -
1000    services.exe    0x840e970c1f60  0x79c   EtwRegistration 0x804   -
1000    services.exe    0x840ea17e9080  0x7a0   Process 0x1fffff        svchost.exe Pid 10392
1000    services.exe    0x840e96dcf4e0  0x7a4   Event   0x1f0003        -
1000    services.exe    0x840e96e822c0  0x7a8   ALPC Port       0x1f0001        -
1000    services.exe    0xb708078685f0  0x7ac   Token   0xf01ff -
1000    services.exe    0xb708061b6ee0  0x7b0   Key     0x2001f USER\S-1-5-20
1000    services.exe    0x840e96d5da80  0x7b4   ALPC Port       0x1f0001        -
1000    services.exe    0xb70807b135b0  0x7b8   Token   0xf01ff -
1000    services.exe    0x840e96fbb530  0x7bc   ALPC Port       0x1f0001        -
1000    services.exe    0xb70807ba3f00  0x7c0   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e97134720  0x7c4   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96da72c0  0x7c8   Process 0x1fffff        svchost.exe Pid 3324
1000    services.exe    0x840e96817440  0x7cc   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9b99a970  0x7d0   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96da9300  0x7d4   Process 0x1fffff        svchost.exe Pid 3332
1000    services.exe    0x840e96bfc460  0x7d8   Event   0x1f0003        -
1000    services.exe    0x840e98cb42d0  0x7dc   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96bfc8e0  0x7e0   Event   0x1f0003        -
1000    services.exe    0x840e978d6840  0x7e4   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9c84cde0  0x7e8   Event   0x1f0003        -
1000    services.exe    0x840e91d912e0  0x7ec   Event   0x1f0003        -
1000    services.exe    0xb7080786c060  0x7f0   Token   0xf01ff -
1000    services.exe    0xb708061b8970  0x7f4   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e92ca4310  0x7f8   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96dc4080  0x7fc   Process 0x1fffff        svchost.exe Pid 3352
1000    services.exe    0x840e91ba00c0  0x804   Process 0x1fffff        svchost.exe Pid 7220
1000    services.exe    0x840e96bfc4e0  0x808   Event   0x1f0003        -
1000    services.exe    0x840e90dd21b0  0x810   ALPC Port       0x1f0001        -
1000    services.exe    0x840e98bb2520  0x814   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96e709f0  0x81c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e97453be0  0x820   Event   0x1f0003        -
1000    services.exe    0xb70809349570  0x824   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\SERVICES\WINSOCK2\PARAMETERS\PROTOCOL_CATALOG9
1000    services.exe    0x840e90bb8b60  0x828   Event   0x1f0003        -
1000    services.exe    0x840e96dc5dc0  0x82c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9b98fe00  0x830   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96e19de0  0x834   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96eb2d30  0x838   ALPC Port       0x1f0001        -
1000    services.exe    0x840e97c4e3e0  0x83c   Event   0x1f0003        -
1000    services.exe    0x840e978d7c90  0x840   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70807710060  0x844   Token   0xf01ff -
1000    services.exe    0x840e91165730  0x848   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9699f8d0  0x84c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96db7a80  0x850   ALPC Port       0x1f0001        -
1000    services.exe    0x840e91bcf660  0x854   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70807d16060  0x858   Token   0xf01ff -
1000    services.exe    0x840e97f438d0  0x85c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e90794070  0x860   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96e350c0  0x864   Process 0x1fffff        svchost.exe Pid 3480
1000    services.exe    0xb708079dc060  0x868   Token   0xf01ff -
1000    services.exe    0xb708061bcc60  0x86c   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e96f0c070  0x870   ALPC Port       0x1f0001        -
1000    services.exe    0xb708078ae600  0x874   Token   0xf01ff -
1000    services.exe    0x840e96806dd0  0x878   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e996cf080  0x87c   Process 0x1fffff        svchost.exe Pid 9320
1000    services.exe    0x840e96dcfee0  0x880   Event   0x1f0003        -
1000    services.exe    0x840e9680a1d0  0x884   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96e20080  0x888   Process 0x1fffff        svchost.exe Pid 3488
1000    services.exe    0x840e96dd45e0  0x88c   Event   0x1f0003        -
1000    services.exe    0x840ea535efe0  0x890   Event   0x1f0003        -
1000    services.exe    0x840e97c3ac20  0x894   File    0x16019f        \Device\Afd\Endpoint
1000    services.exe    0x840eaa1aeb30  0x898   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e976838f0  0x89c   EtwRegistration 0x804   -
1000    services.exe    0xb70820bad890  0x8a0   Token   0xf01ff -
1000    services.exe    0x840e96e1aa80  0x8a4   ALPC Port       0x1f0001        -
1000    services.exe    0x840e97b96070  0x8a8   ALPC Port       0x1f0001        -
1000    services.exe    0xb7080786b6b0  0x8ac   Token   0xf01ff -
1000    services.exe    0x840e90c60180  0x8b0   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9bee60c0  0x8b8   Process 0x1fffff        svchost.exe Pid 12440
1000    services.exe    0xb7080871dc70  0x8bc   Key     0x2001f USER\S-1-5-20
1000    services.exe    0xb70807810970  0x8c4   Token   0xf01ff -
1000    services.exe    0x840e978dcf70  0x8c8   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70807babb50  0x8cc   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e96fcaa80  0x8d4   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9f40c340  0x8d8   Process 0x1fffff        svchost.exe Pid 2800
1000    services.exe    0x840e91bcaad0  0x8dc   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e90c60e80  0x8e0   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e96e3d080  0x8e4   Process 0x1fffff        svchost.exe Pid 3528
1000    services.exe    0x840e96e44300  0x8e8   Process 0x1fffff        MsMpEng.exe Pid 3540
1000    services.exe    0xb70807944060  0x8ec   Token   0xf01ff -
1000    services.exe    0x840e96de50e0  0x8f0   Event   0x1f0003        -
1000    services.exe    0x840e96dd48e0  0x8f4   Event   0x1f0003        -
1000    services.exe    0x840e96dc8a80  0x8f8   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96fd5790  0x8fc   ALPC Port       0x1f0001        -
1000    services.exe    0x840e970c1860  0x900   EtwRegistration 0x804   -
1000    services.exe    0x840e96dd4760  0x904   Event   0x1f0003        -
1000    services.exe    0x840e96ddbc60  0x908   Event   0x1f0003        -
1000    services.exe    0x840e969a5cc0  0x90c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e971089e0  0x910   Event   0x1f0003        -
1000    services.exe    0x840e9f3460c0  0x914   Process 0x1fffff        svchost.exe Pid 11596
1000    services.exe    0x840e90c19340  0x918   Process 0x1fffff        svchost.exe Pid 3604
1000    services.exe    0xb708079dc870  0x91c   Token   0xf01ff -
1000    services.exe    0x840e970f3080  0x920   Process 0x1fffff        svchost.exe Pid 3172
1000    services.exe    0x840e97103e60  0x924   Event   0x1f0003        -
1000    services.exe    0x840e96e552c0  0x928   Process 0x1fffff        svchost.exe Pid 3560
1000    services.exe    0x840e96e719f0  0x92c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e96dd6e60  0x930   Event   0x1f0003        -
1000    services.exe    0x840e969a68f0  0x934   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70807b91cf0  0x938   Key     0x2001f USER\S-1-5-19
1000    services.exe    0x840e9f78f5b0  0x940   EtwRegistration 0x804   -
1000    services.exe    0x840e97159300  0x944   Process 0x1fffff        CSFalconServic Pid 3648
1000    services.exe    0x840e9715d080  0x948   Process 0x1fffff        Sysmon64.exe Pid 3696
1000    services.exe    0x840e97149920  0x94c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e970c1320  0x950   EtwRegistration 0x804   -
1000    services.exe    0x840e97108960  0x954   Event   0x1f0003        -
1000    services.exe    0x840e971c6070  0x95c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e9ac1a1b0  0x960   WaitCompletionPacket    0x1     -
1000    services.exe    0xb70808480060  0x964   Token   0xf01ff -
1000    services.exe    0x840e90986130  0x968   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e9713ad80  0x96c   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e97452860  0x974   Event   0x1f0003        -
1000    services.exe    0x840e96f21a80  0x97c   ALPC Port       0x1f0001        -
1000    services.exe    0x840e97228a80  0x980   ALPC Port       0x1f0001        -
1000    services.exe    0x840e97142de0  0x984   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e99703860  0x988   Event   0x1f0003        -
1000    services.exe    0xb70807e6d060  0x98c   Token   0xf01ff -
1000    services.exe    0x840e995fba80  0x994   ALPC Port       0x1f0001        -
1000    services.exe    0x840e92e23070  0x998   ALPC Port       0x1f0001        -
1000    services.exe    0x840e97fb5760  0x99c   Event   0x1f0003        -
1000    services.exe    0x840e9714f4f0  0x9a0   WaitCompletionPacket    0x1     -
1000    services.exe    0x840e973d81c0  0x9a4   Process 0x1fffff        svchost.exe Pid 4800
1000    services.exe    0x840e9ccec080  0x9a8   Process 0x1fffff        svchost.exe Pid 11776
1000    services.exe    0x840e97121ee0  0x9ac   Event   0x1f0003        -
1000    services.exe    0xb70809642470  0x9b0   Token   0xf01ff -
1000    services.exe    0x840e97809fe0  0x9b4   Semaphore       0x100003        -
1000    services.exe    0x840e978098e0  0x9b8   Semaphore       0x100003        -
1000    services.exe    0x840e97809760  0x9bc   Semaphore       0x100003        -Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\princ\anaconda3\Scripts\vol.exe\__main__.py", line 7, in <module>
    sys.exit(main())
             ~~~~^^
  File "C:\Users\princ\anaconda3\Lib\site-packages\volatility3\cli\__init__.py", line 927, in main
    CommandLine().run()
    ~~~~~~~~~~~~~~~~~^^
  File "C:\Users\princ\anaconda3\Lib\site-packages\volatility3\cli\__init__.py", line 515, in run
    renderer.render(grid)
    ~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\princ\anaconda3\Lib\site-packages\volatility3\cli\text_renderer.py", line 330, in render
    grid.populate(visitor, outfd)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^
  File "C:\Users\princ\anaconda3\Lib\site-packages\volatility3\framework\renderers\__init__.py", line 317, in populate
    for level, item in self._generator:
                       ^^^^^^^^^^^^^^^
  File "C:\Users\princ\anaconda3\Lib\site-packages\volatility3\framework\plugins\windows\handles.py", line 380, in _generator
    format_hints.Hex(entry.Body.vol.offset),
                     ^^^^^^^^^^
  File "C:\Users\princ\anaconda3\Lib\site-packages\volatility3\framework\objects\__init__.py", line 974, in __getattr__
    relative_offset, template = self.vol.members[attr]
                                ^^^^^^^^^^^^^^^^
  File "C:\Users\princ\anaconda3\Lib\site-packages\volatility3\framework\interfaces\objects.py", line 34, in __getattr__
    return self._dict[attr]
           ~~~~~~~~~~^^^^^^
  File "C:\Users\princ\anaconda3\Lib\collections\__init__.py", line 1018, in __getitem__
    pass
KeyboardInterrupt
(base) PS C:\Users\princ\Labs\iveep> vol -q -f iveep.mem windows.handles
Volatility 3 Framework 2.26.2

PID     Process Offset  HandleValue     Type    GrantedAccess   Name

4       System  0x840e90261040  0x4     Process 0x1fffff        System Pid 4
4       System  0x840e902d1140  0x8     Thread  0x1fffff        Tid 28 Pid 4
4       System  0xb707fbd9db40  0xc     Key     0x10    MACHINE\SYSTEM\CONTROLSET001\CONTROL\NETWORKPROVIDER\PROVIDERORDER
4       System  0x840e9028b320  0x10    Mutant  0x1f0001        BcdSyncMutant
4       System  0xb707fb855a40  0x14    Directory       0xf000f GLOBAL??
4       System  0xb707fb8993c0  0x18    Directory       0xf000f -
4       System  0x840e902ac960  0x1c    Partition       0x1f0003        MemoryPartition0
4       System  0xb707fb899680  0x20    Directory       0xf000f KernelObjects
4       System  0x840e90291620  0x24    Event   0x1f0003        LowPagedPoolCondition
4       System  0x840e90291b20  0x28    Event   0x1f0003        HighPagedPoolCondition
4       System  0x840e90291720  0x2c    Event   0x1f0003        LowNonPagedPoolCondition
4       System  0x840e902916a0  0x30    Event   0x1f0003        HighNonPagedPoolCondition
4       System  0x840e90291ba0  0x34    Event   0x1f0003        LowMemoryCondition
4       System  0x840e902917a0  0x38    Event   0x1f0003        HighMemoryCondition
4       System  0x840e90291ea0  0x3c    Event   0x1f0003        LowCommitCondition
4       System  0x840e902919a0  0x40    Event   0x1f0003        HighCommitCondition
4       System  0x840e902913a0  0x44    Event   0x1f0003        MaximumCommitCondition
4       System  0x840e90291c20  0x48    Event   0x1f0003        MemoryErrors
4       System  0x840e902914a0  0x4c    Event   0x1f0003        PhysicalMemoryChange
4       System  0x840e902f4040  0x50    Thread  0x1fffff        Tid 92 Pid 4
4       System  0x840e9037c040  0x54    Process 0x1fffff        Registry Pid 108
4       System  0xb707fb859e20  0x58    Key     0x0     -
4       System  0xb708014ded80  0x5c    SymbolicLink    0xf0001 ACPI#GenuineIntel_-_Intel64_Family_15_Model_107_-_QEMU_Virtual_CPU_version_2.5+#_0#{97fadb10-4e33-40ae-359c-8bef029dbdd0}
4       System  0xb707fb934930  0x60    Key     0x2001f MACHINE\SYSTEM\CONTROLSET001\CONTROL\HIVELIST
4       System  0xb707fbd77600  0x64    Key     0x10    MACHINE\SYSTEM\CONTROLSET001\CONTROL\NETWORKPROVIDER\ORDER
4       System  0x840e90c66380  0x68    File    0x12019f        \Device\clfs\Device\HarddiskVolume3\$Extend\$RmMetadata\$TxfLog\$TxfLog
4       System  0xb707fb9343e0  0x6c    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NOTIFICATIONS
4       System  0xb707fbd6e850  0x70    SymbolicLink    0xf0001 PCI#VEN_8086&DEV_2922&SUBSYS_11001AF4&REV_02#3&11583659&0&FA#{2accfe60-c130-11d2-b082-00a0c91efb8b}
4       System  0xb707fb934c60  0x74    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001
4       System  0xb707fb9c02d0  0x78    Key     0xf003f MACHINE\SYSTEM\RNG
4       System  0xb707fb934d70  0x7c    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\LEAPSECONDINFORMATION
4       System  0xb707fbd6e070  0x80    SymbolicLink    0xf0001 ROOT#SYSTEM#0002#{3ad09657-7cd7-47ff-afb0-0763b78d95ea}
4       System  0xb707fb934820  0x84    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER\MEMORY MANAGEMENT\PREFETCHPARAMETERS
4       System  0xb707fb934a40  0x88    Key     0x2001f MACHINE\SYSTEM\SETUP
4       System  0x840e902cadb0  0x8c    ALPC Port       0x1f0001        PowerPort
4       System  0x840e976a2300  0x90    Process 0x1fffff        unsecapp.exe Pid 5220
4       System  0x840e9037cd80  0x94    ALPC Port       0x1f0001        PowerMonitorPort
4       System  0xb707fb9350a0  0x98    Key     0x11    MACHINE\SYSTEM\CONTROLSET001\CONTROL\POWER
4       System  0xb707fb921720  0x9c    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001
4       System  0xb707fb9c0a40  0xa0    Key     0xf003f MACHINE\SYSTEM\DRIVERDATABASE
4       System  0xb707fb921900  0xa4    Key     0xf003f MACHINE\SYSTEM
4       System  0xb707fb921270  0xa8    Key     0xf003f USER
4       System  0x840e902a7c30  0xac    Event   0x1f0003        -
4       System  0xb707fb9c1c50  0xb0    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\ENUM
4       System  0xb707fb9c1700  0xb4    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\DEVICECLASSES
4       System  0xb707fb9d4460  0xb8    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\DEVICECLASSES
4       System  0xb707fb9c13d0  0xbc    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\DEVICECONTAINERS
4       System  0xb707fb9c1a30  0xc0    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\CLASS
4       System  0xb707fb9c1b40  0xc4    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\DEVICEPANELS
4       System  0xb707fb9344f0  0xc8    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\SERVICES
4       System  0xb707fb9c1e70  0xcc    Key     0xf003f MACHINE\SYSTEM\HARDWARECONFIG
4       System  0x840e918c5be0  0xd0    File    0x12019f        \Device\Tcp
4       System  0xb707fb9d4cd0  0xd4    Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\HARDWARE PROFILES
4       System  0x840e918c5900  0xd8    File    0x12019f        \Device\Tcp
4       System  0xb707fb9d5db0  0xdc    Key     0x10    MACHINE\SYSTEM\CONTROLSET001\CONTROL\STORPORT
4       System  0x840e918c5d50  0xe0    File    0x12019f        \Device\Tcp
4       System  0xb707fb9c00b0  0xe4    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\WMI\SECURITY
4       System  0xb708014de7e0  0xe8    SymbolicLink    0xf0001 ACPI#GenuineIntel_-_Intel64_Family_15_Model_107_-_QEMU_Virtual_CPU_version_2.5+#_3#{97fadb10-4e33-40ae-359c-8bef029dbdd0}
4       System  0xb707fb9bd330  0xec    Token   0xe     -
4       System  0xb707fb9d5810  0xf0    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\SERVICES\MUP
4       System  0x840e90c660a0  0xf4    File    0x1     \Device\HarddiskVolume3\$Extend\$RmMetadata\$Txf:$I30:$INDEX_ALLOCATION
4       System  0x840e918c5620  0xf8    File    0x12019f        \Device\Tcp
4       System  0xb707fb9c9e80  0xfc    SymbolicLink    0xf0001 ACPI#FixedButton#2&daba3ff&1#{4afa3d53-74a7-11d0-be5e-00a0c9062857}
4       System  0x840e918c4370  0x100   File    0x12019f        \Device\Tcp
4       System  0xb707fbce51c0  0x104   Key     0xf003f MACHINE\SYSTEM\HARDWARECONFIG\{899E4A84-D601-4DAE-9F20-82D9D827BE81}\PRODUCTIDS
4       System  0x840e918c4ef0  0x108   File    0x12019f        \Device\Tcp
4       System  0x840e918c5060  0x10c   File    0x12019f        \Device\Tcp
4       System  0x840e918c5340  0x110   File    0x12019f        \Device\Tcp
4       System  0x840e918c4200  0x114   File    0x12019f        \Device\Tcp
4       System  0x840e918c54b0  0x118   File    0x12019f        \Device\Tcp
4       System  0x840e918c3510  0x11c   File    0x12019f        \Device\Tcp
4       System  0xb707fbc7bef0  0x120   SymbolicLink    0xf0001 HID#VID_0627&PID_0001#6&38621d22&0&0000#{378de44c-56ef-11d1-bc8c-00a0c91405dd}
4       System  0xb707fbc7b830  0x124   SymbolicLink    0xf0001 HID#VID_0627&PID_0001#6&38621d22&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}
4       System  0x840e918c6e90  0x128   File    0x12019f        \Device\Tcp
4       System  0x840e918c6a40  0x12c   File    0x12019f        \Device\Tcp
4       System  0x840e918c8140  0x130   File    0x12019f        \Device\Tcp
4       System  0x840e918c68d0  0x134   File    0x12019f        \Device\Tcp
4       System  0xb708016277e0  0x138   SymbolicLink    0xf0001 USB#VID_0627&PID_0001#28754-0000:00:1d.7-1#{a5dcbf10-6530-11d2-901f-00c04fb951ed}
4       System  0xb7080425b130  0x13c   Section 0xf001f Win32kCrossSessionGlobals
4       System  0xb708016251a0  0x140   SymbolicLink    0xf0001 USB#ROOT_HUB20#4&2146ae0&0#{f18a0e88-c30c-11d0-8815-00a0c906bed8}
4       System  0xb70801624780  0x144   SymbolicLink    0xf0001 USB#ROOT_HUB#4&1222a604&0#{f18a0e88-c30c-11d0-8815-00a0c906bed8}
4       System  0xb70801625230  0x148   SymbolicLink    0xf0001 USB#ROOT_HUB#4&8919e06&0#{f18a0e88-c30c-11d0-8815-00a0c906bed8}
4       System  0xb708014dec60  0x14c   SymbolicLink    0xf0001 USB#ROOT_HUB#4&2d76e1ea&0#{f18a0e88-c30c-11d0-8815-00a0c906bed8}
4       System  0xb70801624810  0x150   SymbolicLink    0xf0001 USB#ROOT_HUB#4&cddaa6d&0#{f18a0e88-c30c-11d0-8815-00a0c906bed8}
4       System  0x840e92f585e0  0x154   Event   0x1f0003        -
4       System  0xb70801308670  0x158   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL
4       System  0xb708014de990  0x15c   SymbolicLink    0xf0001 USB#ROOT_HUB#4&a49778e&0#{f18a0e88-c30c-11d0-8815-00a0c906bed8}
4       System  0x840e90b4cc30  0x160   FilterConnectionPort    0x1f0001        CS-HashCommsPort
4       System  0x840e90b4cad0  0x164   FilterConnectionPort    0x1f0001        CS-00000001
4       System  0xb707fe20f910  0x168   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\WDI\SCENARIOS\{08D9111A-EE53-4F7C-AA92-B87A349630ED}
4       System  0x840e98079910  0x16c   CoreMessaging   0xf0000 -
4       System  0x840e9c942e60  0x170   Event   0x1f0003        -
4       System  0x840e96f1d300  0x174   Process 0x102a  dasHost.exe Pid 3752
4       System  0xb708014ddca0  0x178   SymbolicLink    0xf0001 ROOT#UMBUS#0000#{65a9a6cf-64cd-480b-843e-32c86e1ba19f}
4       System  0x840e97648b80  0x17c   IoCompletion    0x1f0003        -
4       System  0x840e966d62c0  0x180   Process 0x0     svchost.exe Pid 2092
4       System  0xb707fbdd7a00  0x184   SymbolicLink    0xf0001 STORAGE#Volume#{badcb557-79f6-11ef-ba71-806e6f6e6963}#0000000006500000#{7f108a28-9833-4b3b-b780-2c6b5fa5c062}
4       System  0xb707fbd1ee80  0x188   Key     0xf003f MACHINE\HARDWARE\DEVICEMAP\SCSI\SCSI PORT 0\SCSI BUS 0
4       System  0x840e90992b60  0x18c   TmTm    0xf003f -
4       System  0x840e90c67070  0x190   File    0x12019f        \Device\clfsKtmLog
4       System  0x840e90c67bf0  0x194   File    0x13019f        \Device\clfs\Device\HarddiskVolume3\$Extend\$RmMetadata\$TxfLog\$TxfLog
4       System  0xb707fbd6d0b0  0x198   SymbolicLink    0xf0001 ROOT#vdrvroot#0000#{2e34d650-5819-42ca-84ae-d30803bae505}
4       System  0xb707fbd894e0  0x19c   Key     0xf003f MACHINE\HARDWARE\DEVICEMAP\SCSI\SCSI PORT 0\SCSI BUS 2
4       System  0xb707fbd6d890  0x1a0   SymbolicLink    0xf0001 ROOT#spaceport#0000#{53f5630e-b6bf-11d0-94f2-00a0c91efb8b}
4       System  0x840e908a46a0  0x1a4   ALPC Port       0x1f0001        PdcPort
4       System  0xb707fbd89810  0x1a8   Key     0x8     MACHINE\SYSTEM\CONTROLSET001\CONTROL\POWER\PROFILE\EVENTS\{54533251-82BE-4824-96C1-47B60B740D00}\{0DA965DC-8FCF-4C0B-8EFE-8DD5E7BC959A}\{7E01ADEF-81E6-4E1B-8075-56F373584694}
4       System  0xb707fbd895f0  0x1ac   Key     0x8     MACHINE\SYSTEM\CONTROLSET001\CONTROL\POWER\PROFILE\EVENTS\{54533251-82BE-4824-96C1-47B60B740D00}\{0AABB002-A307-447E-9B81-1D819DF6C6D0}\{CE74AA52-A71A-4036-BEEF-B6C411010E28}
4       System  0xb707fbd89920  0x1b0   Key     0x8     MACHINE\SYSTEM\CONTROLSET001\CONTROL\POWER\PROFILE\EVENTS\{54533251-82BE-4824-96C1-47B60B740D00}\{8BC6262C-C026-411D-AE3B-7E2F70811A13}\{C072EEBB-1955-4FA9-B4BA-421E96E1D674}
4       System  0xb707fbd89700  0x1b4   Key     0x8     MACHINE\SYSTEM\CONTROLSET001\CONTROL\POWER\PROFILE\EVENTS\{54533251-82BE-4824-96C1-47B60B740D00}\{D4140C81-EBBA-4E60-8561-6918290359CD}\{35037BB4-9528-481D-8CB2-8FCC63A9DD81}
4       System  0xb707fbd880b0  0x1b8   Key     0x8     MACHINE\SYSTEM\CONTROLSET001\CONTROL\POWER\PROFILE\EVENTS\{54533251-82BE-4824-96C1-47B60B740D00}\{EE1E4F72-E368-46B1-B3C6-5048B11C2DBD}\{9C1F0DBA-33E9-43AF-9EDA-A607AA5139DA}
4       System  0x840e908a5040  0x1bc   Thread  0x1fffff        Tid 292 Pid 4
4       System  0xb707fbd6d260  0x1c0   SymbolicLink    0xf0001 ROOT#spaceport#0000#{ef66a56f-88d1-4cd8-98c4-49faf57ad8af}
4       System  0xb707fbd89d60  0x1c4   Key     0xf003f MACHINE\HARDWARE\DEVICEMAP\SCSI\SCSI PORT 0\SCSI BUS 3
4       System  0xb707fbd882d0  0x1c8   Key     0xf003f MACHINE\HARDWARE\DEVICEMAP\SCSI\SCSI PORT 0\SCSI BUS 4
4       System  0xb707fbd6ef10  0x1cc   SymbolicLink    0xf0001 ROOT#volmgr#0000#{53f5630e-b6bf-11d0-94f2-00a0c91efb8b}
4       System  0xb707fbd88600  0x1d0   Key     0xf003f MACHINE\HARDWARE\DEVICEMAP\SCSI\SCSI PORT 0\SCSI BUS 1
4       System  0xb707fbd9d4e0  0x1d4   Key     0xf003f MACHINE\HARDWARE\DEVICEMAP\SCSI\SCSI PORT 0\SCSI BUS 5
4       System  0x840e918c30c0  0x1d8   File    0x12019f        \Device\Tcp
4       System  0xb707fbd9c710  0x1dc   Key     0x10    MACHINE\SYSTEM\CONTROLSET001\CONTROL\FILESYSTEM
4       System  0x840e90991d00  0x1e0   TmRm    0x1f007f        -
4       System  0x840e90993050  0x1e4   TmRm    0x1f007f        -
4       System  0x840e90924600  0x1e8   FilterConnectionPort    0x1f0001        MicrosoftMalwareProtectionControlPortWD
4       System  0x840e90924290  0x1ec   FilterConnectionPort    0x1f0001        MicrosoftMalwareProtectionPortWD
4       System  0x840e90924080  0x1f0   FilterConnectionPort    0x1f0001        MicrosoftMalwareProtectionVeryLowIoPortWD
4       System  0x840e909241e0  0x1f4   FilterConnectionPort    0x1f0001        MicrosoftMalwareProtectionRemoteIoPortWD
4       System  0x840e909230b0  0x1f8   FilterConnectionPort    0x1f0001        MicrosoftMalwareProtectionAsyncPortWD
4       System  0xb707fb9d5630  0x1fc   Key     0x10    MACHINE\SYSTEM\CONTROLSET001\CONTROL\FILESYSTEM
4       System  0xb707fbd9d700  0x200   Key     0x10    MACHINE\SYSTEM\CONTROLSET001\POLICIES
4       System  0x840e90261040  0x204   Process 0x1440  System Pid 4
4       System  0xb707fbdd7580  0x208   SymbolicLink    0xf0001 STORAGE#Volume#{badcb557-79f6-11ef-ba71-806e6f6e6963}#0000000FE0B00000#{53f5630d-b6bf-11d0-94f2-00a0c91efb8b}
4       System  0xb707fbd9d1b0  0x20c   Key     0xb     MACHINE\SYSTEM\CONTROLSET001\SERVICES\NDIS\IFTYPES\71
4       System  0xb707fbd9c4f0  0x210   Key     0xb     MACHINE\SYSTEM\CONTROLSET001\SERVICES\NDIS\IFTYPES\24
4       System  0xb707fbdd7df0  0x214   SymbolicLink    0xf0001 STORAGE#Volume#{badcb557-79f6-11ef-ba71-806e6f6e6963}#0000000007500000#{53f5630d-b6bf-11d0-94f2-00a0c91efb8b}
4       System  0xb708016249c0  0x218   SymbolicLink    0xf0001 USB#ROOT_HUB20#4&313a3917&0#{f18a0e88-c30c-11d0-8815-00a0c906bed8}
4       System  0xb707fb851720  0x21c   Token   0xf01ff -
4       System  0xb707fbdd8640  0x220   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\SERVICES\IORATE
4       System  0x840e918c78a0  0x224   File    0x12019f        \Device\Tcp
4       System  0xb707fb927760  0x228   Directory       0xf000f Harddisk0
4       System  0x840e910b4040  0x22c   Process 0x1fffff        smss.exe Pid 536
4       System  0x840e90bac040  0x230   Thread  0x1fffff        Tid 328 Pid 4
4       System  0xb707fbd9c1c0  0x234   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\POWER\ENERGYESTIMATION\STORAGE\HDD\IDLESTATE
4       System  0xb707fbdd6380  0x238   SymbolicLink    0xf0001 SCSI#Disk&Ven_QEMU&Prod_HARDDISK#4&35424867&0&000000#{53f56307-b6bf-11d0-94f2-00a0c91efb8b}
4       System  0xb707fbdd78e0  0x23c   SymbolicLink    0xf0001 STORAGE#Volume#{badcb557-79f6-11ef-ba71-806e6f6e6963}#0000000000100000#{53f5630d-b6bf-11d0-94f2-00a0c91efb8b}
4       System  0x840e90bad040  0x240   Thread  0x1fffff        Tid 332 Pid 4
4       System  0xb707fbdd9360  0x244   Key     0x10    MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER\MEMORY MANAGEMENT
4       System  0x840e90bfa7d0  0x248   File    0x12019f        \Device\HarddiskVolume3\$Extend\$RmMetadata\$TxfLog\$TxfLog.blf
4       System  0x840e90a0c350  0x24c   File    0x12019f        \Device\HarddiskVolume3\$Extend\$RmMetadata\$TxfLog\$TxfLogContainer00000000000000000002
4       System  0x840e90bfbd60  0x250   File    0x12019f        \Device\HarddiskVolume3\$Extend\$RmMetadata\$TxfLog\$TxfLogContainer00000000000000000015
4       System  0x840e90bfa4f0  0x254   File    0x13019f        \Device\clfs\Device\HarddiskVolume3\$Extend\$RmMetadata\$TxfLog\$TxfLog
4       System  0x840e90c66210  0x258   File    0x12019f        \Device\clfsTxfLog
4       System  0x840e90c88050  0x25c   TmRm    0x1f007f        -
4       System  0x840e90c8f8d0  0x260   TmTm    0xf003f -
4       System  0x840e90c87b10  0x264   TmRm    0x12007f        -
4       System  0xb708014de870  0x26c   SymbolicLink    0xf0001 ACPI#GenuineIntel_-_Intel64_Family_15_Model_107_-_QEMU_Virtual_CPU_version_2.5+#_3#{dbe4373d-3c81-40cb-ace4-e0e5d05f0c9f}
4       System  0xb708014821c0  0x270   Key     0x20000 MACHINE\SYSTEM\CONTROLSET001\CONTROL\CLASS\{4D36E97D-E325-11CE-BFC1-08002BE10318}\0013
4       System  0x840e97159300  0x274   Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e973488a0  0x278   FilterCommunicationPort 0x1f0001        -
4       System  0xb708042b6400  0x27c   Key     0x10    MACHINE\SOFTWARE\MICROSOFT\WINDOWS\CURRENTVERSION\APPMODELUNLOCK
4       System  0xb70801482c60  0x280   Key     0x10    MACHINE\SYSTEM\CROWDSTRIKE\DEVICE CONTROL POLICY
4       System  0xb70801624f60  0x284   SymbolicLink    0xf0001 USB#ROOT_HUB#4&b25d0e5&0#{f18a0e88-c30c-11d0-8815-00a0c906bed8}
4       System  0xb708014dce00  0x288   SymbolicLink    0xf0001 ACPI#QEMUVGID#2&daba3ff&1#{3ff2c92b-6598-4e60-8e1c-0ccf4927e319}
4       System  0xb7080130fc90  0x28c   Key     0x20000 MACHINE\SYSTEM\CONTROLSET001\CONTROL\CLASS\{4D36E97D-E325-11CE-BFC1-08002BE10318}\0013\SETTINGS
4       System  0xb707fbd881c0  0x290   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\HVSI
4       System  0xb708014ddf70  0x294   SymbolicLink    0xf0001 PCI#VEN_8086&DEV_2938&SUBSYS_11001AF4&REV_03#3&11583659&0&D1#{3abf6f2d-71c4-462a-8a92-1e6861e6af27}
4       System  0xb707fe266960  0x298   Directory       0xf000f cimfs
4       System  0xb708014de2d0  0x29c   SymbolicLink    0xf0001 PCI#VEN_8086&DEV_2934&SUBSYS_11001AF4&REV_03#3&11583659&0&E8#{3abf6f2d-71c4-462a-8a92-1e6861e6af27}
4       System  0xb708014de750  0x2a0   SymbolicLink    0xf0001 ACPI#GenuineIntel_-_Intel64_Family_15_Model_107_-_QEMU_Virtual_CPU_version_2.5+#_2#{dbe4373d-3c81-40cb-ace4-e0e5d05f0c9f}
4       System  0xb70800bfe8c0  0x2a4   Section 0x6     -
4       System  0xb707fb9c11b0  0x2a8   Key     0x2001f MACHINE\SYSTEM\CONTROLSET001\SERVICES\VWIFIFLT\STATE\PARAMETERS\VWIFISETTINGS
4       System  0xb708014dea20  0x2ac   SymbolicLink    0xf0001 ACPI#GenuineIntel_-_Intel64_Family_15_Model_107_-_QEMU_Virtual_CPU_version_2.5+#_1#{97fadb10-4e33-40ae-359c-8bef029dbdd0}
4       System  0xb708014dee10  0x2b0   SymbolicLink    0xf0001 ACPI#GenuineIntel_-_Intel64_Family_15_Model_107_-_QEMU_Virtual_CPU_version_2.5+#_2#{97fadb10-4e33-40ae-359c-8bef029dbdd0}
4       System  0xb708014dd430  0x2b4   SymbolicLink    0xf0001 ACPI#PNP0303#4&22f5829e&0#{884b96c3-56ef-11d1-bc8c-00a0c91405dd}
4       System  0xb708014de6c0  0x2b8   SymbolicLink    0xf0001 ACPI#GenuineIntel_-_Intel64_Family_15_Model_107_-_QEMU_Virtual_CPU_version_2.5+#_0#{dbe4373d-3c81-40cb-ace4-e0e5d05f0c9f}
4       System  0xb707fbd1f810  0x2bc   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\SERVICES\BAM\STATE\USERSETTINGS
4       System  0xb707fbce6a30  0x2c0   Key     0x10    MACHINE\SYSTEM\CONTROLSET001\CONTROL\LSA
4       System  0x840e90f57750  0x2c4   File    0x120116        \Device\Mup
4       System  0x840e910b4040  0x2c8   Process 0x102a  smss.exe Pid 536
4       System  0xb708014defc0  0x2cc   SymbolicLink    0xf0001 SCSI#CdRom&Ven_QEMU&Prod_QEMU_DVD-ROM#4&35424867&0&010000#{53f5630d-b6bf-11d0-94f2-00a0c91efb8b}
4       System  0x840e910b4040  0x2d0   Process 0x102a  smss.exe Pid 536
4       System  0xb708014debd0  0x2d4   SymbolicLink    0xf0001 SCSI#CdRom&Ven_QEMU&Prod_QEMU_DVD-ROM#4&35424867&0&010000#{53f56308-b6bf-11d0-94f2-00a0c91efb8b}
4       System  0x840e910a3d30  0x2d8   ALPC Port       0x1f0001        SeRmCommandPort
4       System  0xb708014dcfb0  0x2dc   SymbolicLink    0xf0001 PCI#VEN_8086&DEV_2937&SUBSYS_11001AF4&REV_03#3&11583659&0&D0#{3abf6f2d-71c4-462a-8a92-1e6861e6af27}
4       System  0xb7080130d620  0x2e0   Key     0x20000 MACHINE\SYSTEM\CONTROLSET001\CONTROL\CLASS\{4D36E97D-E325-11CE-BFC1-08002BE10318}\0013
4       System  0x840e91bd30a0  0x2e4   Timer   0x1f0003        -
4       System  0xb7080130e7f0  0x2e8   Key     0x10    MACHINE\SYSTEM\CONTROLSET001\CONTROL\FILESYSTEM
4       System  0xb707fe20ec50  0x2ec   Key     0x10    MACHINE\SYSTEM\CONTROLSET001\SERVICES\SWENUM\NOTIFY
4       System  0x840e90795040  0x2f0   Thread  0x1fffff        Tid 444 Pid 4
4       System  0x840e9078a040  0x2f4   Thread  0x1fffff        Tid 388 Pid 4
4       System  0x840e96809cf0  0x2f8   WaitCompletionPacket    0x1     -
4       System  0x840e90cc83c0  0x2fc   Thread  0x1fffff        Tid 352 Pid 4
4       System  0x840e9e902340  0x300   Process 0x1fffff        msedge.exe Pid 5808
4       System  0x840e9078f040  0x304   Thread  0x1fffff        Tid 420 Pid 4
4       System  0x840e90790040  0x308   Thread  0x1fffff        Tid 424 Pid 4
4       System  0x840e90791040  0x30c   Thread  0x1fffff        Tid 428 Pid 4
4       System  0x840e90792040  0x310   Thread  0x1fffff        Tid 432 Pid 4
4       System  0x840e98fe3be0  0x314   Event   0x1f0003        -
4       System  0x840e91f561e0  0x318   Event   0x1f0003        -
4       System  0xb708014dd5e0  0x31c   SymbolicLink    0xf0001 PCI#VEN_8086&DEV_2935&SUBSYS_11001AF4&REV_03#3&11583659&0&E9#{3abf6f2d-71c4-462a-8a92-1e6861e6af27}
4       System  0xb7080130d710  0x320   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL
4       System  0xb708014deb40  0x324   SymbolicLink    0xf0001 ACPI#GenuineIntel_-_Intel64_Family_15_Model_107_-_QEMU_Virtual_CPU_version_2.5+#_1#{dbe4373d-3c81-40cb-ace4-e0e5d05f0c9f}
4       System  0x840e91099540  0x328   Thread  0x1fffff        Tid 520 Pid 4
4       System  0xb7080130dda0  0x32c   Key     0x11    MACHINE\SYSTEM\CONTROLSET001\CONTROL\LSA
4       System  0xb707fbd9d0a0  0x330   Key     0x10    MACHINE\SYSTEM\CONTROLSET001\SERVICES\DFSC\PARAMETERS
4       System  0xb7080130e160  0x334   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL
4       System  0xb708014dda60  0x338   SymbolicLink    0xf0001 PCI#VEN_8086&DEV_2936&SUBSYS_11001AF4&REV_03#3&11583659&0&EA#{3abf6f2d-71c4-462a-8a92-1e6861e6af27}
4       System  0xb7080130e430  0x33c   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL
4       System  0xb707fbd9d2c0  0x340   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\{7746D80F-97E0-4E26-9543-26B41FC22F79}\{A25AE4F2-1B96-4CED-8007-AA30E9B1A218}
4       System  0xb7080130dcb0  0x344   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\POWER
4       System  0xb707fbce6d60  0x348   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\{7746D80F-97E0-4E26-9543-26B41FC22F79}\{D73E01AC-F5A0-4D80-928B-33C1920C38BA}
4       System  0xb707fbce5a40  0x34c   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\{7746D80F-97E0-4E26-9543-26B41FC22F79}\{59AEE675-B203-4D61-9A1F-04518A20F359}
4       System  0xb707fe20ee70  0x350   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\{7746D80F-97E0-4E26-9543-26B41FC22F79}\{FB9F5B62-B48B-45F5-8586-E514958C92E2}
4       System  0xb707fbd9dc50  0x354   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\CONTROL\{7746D80F-97E0-4E26-9543-26B41FC22F79}\{221601AB-48C7-4970-B0EC-96E66F578407}
4       System  0x840e91150e60  0x358   File    0x12019f        \Device\Tcp
4       System  0xb70801482b50  0x35c   Key     0x11    MACHINE\SYSTEM\CONTROLSET001\CONTROL\KERNELVELOCITY
4       System  0xb708014de510  0x360   SymbolicLink    0xf0001 ROOT#SYSTEM#0000#{3c0d501a-140b-11d1-b40f-00a0c9223196}
4       System  0xb708014dd700  0x364   SymbolicLink    0xf0001 PCI#VEN_8086&DEV_2939&SUBSYS_11001AF4&REV_03#3&11583659&0&D2#{3abf6f2d-71c4-462a-8a92-1e6861e6af27}
4       System  0x840e90fc0b30  0x368   Event   0x1f0003        -
4       System  0xb708014decf0  0x36c   SymbolicLink    0xf0001 ACPI#PNP0F13#4&22f5829e&0#{378de44c-56ef-11d1-bc8c-00a0c91405dd}
4       System  0xb708014dd4c0  0x370   SymbolicLink    0xf0001 ROOT#SYSTEM#0001#{bf0513dc-034c-4507-926c-a631aa80dc49}
4       System  0xb70801482710  0x374   Key     0x20000 MACHINE\SYSTEM\CONTROLSET001\CONTROL\CLASS\{4D36E97D-E325-11CE-BFC1-08002BE10318}\0013\SETTINGS
4       System  0x840e918c7a10  0x378   File    0x12019f        \Device\Tcp
4       System  0xb708014dd820  0x37c   SymbolicLink    0xf0001 PCI#VEN_8086&DEV_293C&SUBSYS_11001AF4&REV_03#3&11583659&0&D7#{3abf6f2d-71c4-462a-8a92-1e6861e6af27}
4       System  0x840e918c8700  0x380   File    0x12019f        \Device\Tcp
4       System  0x840e91049560  0x384   Event   0x2     EFSInitEvent
4       System  0x840e918c7730  0x388   File    0x12019f        \Device\Tcp
4       System  0xb708014ddd30  0x38c   SymbolicLink    0xf0001 ROOT#SYSTEM#0000#{4747b320-62ce-11cf-a5d6-28db04c10000}
4       System  0x840e918c61a0  0x390   File    0x12019f        \Device\Tcp
4       System  0xb708014823e0  0x394   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\AUDIO\HDAUDIOFLAGS
4       System  0xb708014de3f0  0x398   SymbolicLink    0xf0001 PCI#VEN_8086&DEV_100E&SUBSYS_11001AF4&REV_03#5&2490727a&0&9008F0#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0x840e91fd0c40  0x39c   File    0x12008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\Microsoft-Windows-Rdp-Graphics-RdpIdd-Trace.etl
4       System  0xb708014de5a0  0x3a0   SymbolicLink    0xf0001 PCI#VEN_8086&DEV_100E&SUBSYS_11001AF4&REV_03#5&2490727a&0&9008F0#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0xb708014ddc10  0x3a4   SymbolicLink    0xf0001 PCI#VEN_8086&DEV_293A&SUBSYS_11001AF4&REV_03#3&11583659&0&EF#{3abf6f2d-71c4-462a-8a92-1e6861e6af27}
4       System  0x840e91151e30  0x3a8   File    0x100001        \Device\KsecDD
4       System  0x840e91049560  0x3ac   Event   0x2     EFSInitEvent
4       System  0x840e918c44e0  0x3b0   File    0x12019f        \Device\Udp
4       System  0x840e91150cf0  0x3b4   File    0x100001        \Device\KsecDD
4       System  0x840e918c6480  0x3b8   File    0x12019f        \Device\Tcp
4       System  0x840e918c3230  0x3bc   File    0x12019f        \Device\Udp
4       System  0x840e918c6bb0  0x3c0   File    0x12019f        \Device\Tcp
4       System  0x840e918c37f0  0x3c4   File    0x12019f        \Device\Tcp
4       System  0x840e918c47c0  0x3c8   File    0x12019f        \Device\Tcp
4       System  0x840e918c4650  0x3cc   File    0x12019f        \Device\Tcp
4       System  0x840e918c33a0  0x3d0   File    0x12019f        \Device\Tcp
4       System  0x840e918c5a70  0x3d4   File    0x12019f        \Device\Tcp
4       System  0x840e918c4c10  0x3d8   File    0x12019f        \Device\Tcp
4       System  0x840e918c3960  0x3dc   File    0x12019f        \Device\Tcp
4       System  0x840e918c4aa0  0x3e0   File    0x12019f        \Device\Tcp
4       System  0x840e918c51d0  0x3e4   File    0x12019f        \Device\Tcp
4       System  0x840e918c4090  0x3e8   File    0x12019f        \Device\Tcp
4       System  0x840e918c3db0  0x3ec   File    0x12019f        \Device\Tcp
4       System  0x840e918c3c40  0x3f0   File    0x12019f        \Device\Tcp
4       System  0x840e918c3680  0x3f4   File    0x12019f        \Device\Tcp
4       System  0x840e918c4d80  0x3f8   File    0x12019f        \Device\Tcp
4       System  0x840e918c4930  0x3fc   File    0x12019f        \Device\Tcp
4       System  0x840e918c82b0  0x404   File    0x12019f        \Device\Tcp
4       System  0x840e918c7170  0x408   File    0x12019f        \Device\Tcp
4       System  0x840e918c8420  0x40c   File    0x12019f        \Device\Tcp
4       System  0x840e918c72e0  0x410   File    0x12019f        \Device\Tcp
4       System  0x840e918c6310  0x414   File    0x12019f        \Device\Tcp
4       System  0x840e918c65f0  0x418   File    0x12019f        \Device\Tcp
4       System  0x840e918c8590  0x41c   File    0x12019f        \Device\Tcp
4       System  0x840e918c8870  0x420   File    0x12019f        \Device\Tcp
4       System  0x840e918c7b80  0x424   File    0x12019f        \Device\Tcp
4       System  0x840e918c89e0  0x428   File    0x12019f        \Device\Tcp
4       System  0x840e918c8b50  0x42c   File    0x12019f        \Device\Tcp
4       System  0x840e918c7cf0  0x430   File    0x12019f        \Device\Tcp
4       System  0x840e918c5ec0  0x434   File    0x12019f        \Device\Tcp
4       System  0x840e910b4040  0x438   Process 0x102a  smss.exe Pid 536
4       System  0xb70801482930  0x43c   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL
4       System  0x840e91fd6b20  0x440   File    0x12019f        \Device\HarddiskVolume3\Windows\System32\config\TxR\{53b39e3e-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000002.regtrans-ms
4       System  0x840e91fb3d50  0x444   File    0x20003 \Device\HarddiskVolume1\EFI\Microsoft\Boot\BCD
4       System  0x840e92f584e0  0x448   Event   0x1f0003        -
4       System  0x840e91bd3400  0x44c   Timer   0x1f0003        -
4       System  0x840e92f587e0  0x450   Event   0x1f0003        -
4       System  0x840e92f58b60  0x454   Event   0x1f0003        -
4       System  0x840e92f58a60  0x458   Event   0x1f0003        -
4       System  0x840e91bd4d50  0x45c   Timer   0x2     -
4       System  0x840e92f58360  0x460   Event   0x1f0003        -
4       System  0x840e92f583e0  0x464   Event   0x1f0003        -
4       System  0x840e91db7340  0x468   Process 0x1fffff        taskhostw.exe Pid 4428
4       System  0x840e99fd6e60  0x46c   Event   0x1f0003        -
4       System  0x840e98a46cf0  0x470   IoCompletionReserve     0xf0003 -
4       System  0x840e9229c4e0  0x474   Event   0x1f0003        -
4       System  0x840e922fb760  0x478   Timer   0x2     -
4       System  0x840e99c32d60  0x47c   Event   0x1f0003        -
4       System  0x840e9229c560  0x480   Event   0x1f0003        -
4       System  0x840e9229cd60  0x484   Event   0x1f0003        -
4       System  0x840e9229c660  0x488   Event   0x1f0003        -
4       System  0x840e90e10410  0x48c   Session 0x1     Session2
4       System  0x840e9230f070  0x490   RawInputManager 0x3     -
4       System  0x840e9229b060  0x498   Event   0x1f0003        -
4       System  0x840e922ed440  0x49c   IoCompletion    0x1f0003        -
4       System  0x840e90ac6a70  0x4a0   WaitCompletionPacket    0x1     -
4       System  0x840e9238e350  0x4a4   File    0x12019f        \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM.blf
4       System  0x840e9228c080  0x4a8   Process 0x0     wininit.exe Pid 880
4       System  0xb708044e18c0  0x4ac   Token   0xf01ff -
4       System  0x840e923f61c0  0x4b0   Process 0x102a  fontdrvhost.ex Pid 892
4       System  0xb70804905040  0x4b4   Token   0xe     -
4       System  0xb708045f8750  0x4b8   Directory       0xf000f 00000000-0001cbf7
4       System  0x840e92298760  0x4bc   Event   0x1f0003        -
4       System  0x840e92298460  0x4c0   Event   0x1f0003        -
4       System  0x840e911dc180  0x4c4   Timer   0x2     -
4       System  0x840e922982e0  0x4c8   Event   0x1f0003        -
4       System  0x840e922985e0  0x4cc   Event   0x1f0003        -
4       System  0x840e922983e0  0x4d0   Event   0x1f0003        -
4       System  0x840e922981e0  0x4d4   Event   0x1f0003        -
4       System  0x840e911dc180  0x4d8   Timer   0x1f0003        -
4       System  0x840e911dc330  0x4dc   Timer   0x1f0003        -
4       System  0x840e922ede00  0x4e0   IoCompletion    0x1f0003        -
4       System  0x840e90ac75d0  0x4e4   WaitCompletionPacket    0x1     -
4       System  0x840e92298460  0x4e8   Event   0x1f0003        -
4       System  0x840e92298960  0x4ec   Event   0x1f0003        -
4       System  0x840e90c65310  0x4f0   WaitCompletionPacket    0x1     -
4       System  0x840e92298560  0x4f4   Event   0x1f0003        -
4       System  0x840e92305080  0x4f8   Process 0x102a  winlogon.exe Pid 980
4       System  0x840e922d7dc0  0x4fc   IoCompletion    0x1f0003        -
4       System  0x840e9228c080  0x500   Process 0x102a  wininit.exe Pid 880
4       System  0x840e91fd1930  0x504   File    0x12019f        \Device\HarddiskVolume3\Windows\System32\config\TxR\{53b39e3e-18c4-11ea-a811-000d3aa4692b}.TM.blf
4       System  0x840e91fd63f0  0x508   File    0x12019f        \Device\HarddiskVolume3\Windows\System32\config\TxR\{53b39e3e-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000001.regtrans-ms
4       System  0x840e9229c4e0  0x50c   Event   0x1f0003        -
4       System  0x840e91fd42e0  0x510   File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTSenseEventLog.etl
4       System  0x840e91f56260  0x514   Event   0x1f0003        -
4       System  0xb7080174e3e0  0x518   Section 0x17    -
4       System  0xb70801666a30  0x51c   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL
4       System  0xb708016662c0  0x520   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL
4       System  0x840e91fb46b0  0x524   File    0x20003 \Device\HarddiskVolume3\Windows\System32\config\SOFTWARE
4       System  0xb70801669180  0x528   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL
4       System  0xb70801667800  0x52c   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL
4       System  0x840e9112d560  0x530   File    0x12019f        \Device\HarddiskVolume4\$Extend\$RmMetadata\$TxfLog\$TxfLog.blf
4       System  0x840e9112e0e0  0x534   File    0x12019f        \Device\HarddiskVolume4\$Extend\$RmMetadata\$TxfLog\$TxfLogContainer00000000000000000001
4       System  0x840e9112e6a0  0x538   File    0x12019f        \Device\HarddiskVolume4\$Extend\$RmMetadata\$TxfLog\$TxfLogContainer00000000000000000002
4       System  0x840e9112c140  0x53c   File    0x13019f        \Device\clfs\Device\HarddiskVolume4\$Extend\$RmMetadata\$TxfLog\$TxfLog
4       System  0x840e9112eaf0  0x540   File    0x12019f        \Device\clfsTxfLog
4       System  0x840e91fd07f0  0x544   File    0x1     \Device\HarddiskVolume4\$Extend\$RmMetadata\$Txf:$I30:$INDEX_ALLOCATION
4       System  0x840e91fd2900  0x548   File    0x12019f        \Device\clfs\Device\HarddiskVolume4\$Extend\$RmMetadata\$TxfLog\$TxfLog
4       System  0x840e91fd17c0  0x54c   File    0x13019f        \Device\clfs\Device\HarddiskVolume4\$Extend\$RmMetadata\$TxfLog\$TxfLog
4       System  0x840e91fd0510  0x550   File    0x12019f        \Device\clfsKtmLog
4       System  0x840e91f62050  0x554   TmRm    0x1f007f        -
4       System  0x840e91f9f8d0  0x558   TmTm    0xf003f -
4       System  0x840e91fa6940  0x55c   TmRm    0x12007f        -
4       System  0xb708061f47b0  0x560   Directory       0xf000f Http
4       System  0xb707fb8858e0  0x564   Directory       0x1     Device
4       System  0x840e91fb4840  0x568   File    0x12019f        \Device\HarddiskVolume3\Windows\bootstat.dat
4       System  0x840e91fd0960  0x56c   File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTEventlog-Security.etl
4       System  0xb708042b8930  0x570   Key     0x2     MACHINE\SYSTEM\CONTROLSET001\CONTROL\CLASS\{4D36E968-E325-11CE-BFC1-08002BE10318}\0000
4       System  0x840e91fd52b0  0x574   File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTSenseAuditLogger.etl
4       System  0x840e9b9b1bf0  0x578   WaitCompletionPacket    0x1     -
4       System  0x840e91fd35f0  0x57c   File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTUBPM.etl
4       System  0xb70811b4c310  0x580   Section 0x17    -
4       System  0x840e91fd0680  0x584   File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTEventLog-Application.etl
4       System  0xb70807b5e5d0  0x588   Token   0xe     -
4       System  0x840e91fd2d50  0x58c   File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTDefenderAuditLogger.etl
4       System  0x840e91fd31a0  0x590   File    0x12008b        \Device\HarddiskVolume3\Windows\System32\WDI\LogFiles\WdiContextLog.etl.002
4       System  0x840e91fd1200  0x594   File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTEventLog-Microsoft-Windows-Sysmon-Operational.etl
4       System  0x840e91fd59e0  0x598   File    0x12008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RadioMgr.etl
4       System  0x840e91fd0ad0  0x59c   File    0x12008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\LwtNetLog.etl
4       System  0x840e91fd2620  0x5a0   File    0x12008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\NtfsLog.etl
4       System  0x840e91fd2060  0x5a4   File    0x12008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\NetCore.etl
4       System  0x840e91fd2a70  0x5a8   File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTDiagLog.etl
4       System  0x840e91fd38d0  0x5ac   File    0x12008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\Wifi.etl
4       System  0x840e91fd2790  0x5b0   File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTDefenderApiLogger.etl
4       System  0x840e91f57fe0  0x5b4   Event   0x1f0003        -
4       System  0x840e91fd00c0  0x5b8   File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTEventLog-System.etl
4       System  0x840e91fb51a0  0x5bc   File    0x20003 \Device\HarddiskVolume3\Windows\System32\config\SYSTEM
4       System  0x840e91fb6780  0x5c0   File    0x120089        \Device\HarddiskVolume3\Windows\System32\en-US\win32kbase.sys.mui
4       System  0x840e91f579e0  0x5c4   Event   0x1f0003        -
4       System  0x840e91fb3260  0x5c8   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\SYSTEM.LOG1
4       System  0x840e91fb4390  0x5cc   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\SYSTEM.LOG2
4       System  0xb707fbc73340  0x5d0   Section 0x17    -
4       System  0x840e91fb6140  0x5d4   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\SOFTWARE.LOG1
4       System  0x840e91fb4b60  0x5d8   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\SOFTWARE.LOG2
4       System  0xb708042b72e0  0x5dc   Key     0xf003f MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\NOTIFICATIONS
4       System  0x840e91fb33f0  0x5e0   File    0x3     \Device\HarddiskVolume1\EFI\Microsoft\Boot\BCD.LOG
4       System  0xb708042b7940  0x5e4   Key     0xf003f MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\NOTIFICATIONS\DATA
4       System  0x840e90badc70  0x5e8   TmRm    0x1f007f        -
4       System  0x840e91fd6840  0x5ec   File    0x12019f        \Device\clfs\SystemRoot\System32\Config\TxR\{53b39e3e-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0x840e91fd69b0  0x5f0   File    0x12019f        \Device\clfs\SystemRoot\System32\Config\TxR\{53b39e3e-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0xb7080174f9c0  0x5f4   Section 0x17    -
4       System  0x840e90c81d30  0x5f8   TmRm    0x1f007f        -
4       System  0x840e90f4f640  0x5fc   TmTm    0xf003f -
4       System  0x840e92017080  0x600   Thread  0x1fffff        Tid 832 Pid 4
4       System  0xb707fb8858e0  0x604   Directory       0x1     Device
4       System  0xb708043510d0  0x608   SymbolicLink    0xf0001 PCI#VEN_1234&DEV_1111&SUBSYS_11001AF4&REV_02#3&11583659&0&08#{1ca05180-a699-450a-9a0c-de4fbe3ddd89}
4       System  0xb70804351650  0x60c   SymbolicLink    0xf0001 PCI#VEN_1234&DEV_1111&SUBSYS_11001AF4&REV_02#3&11583659&0&08#{5b45201d-f2f2-4f3b-85bb-30ff1f953599}
4       System  0x840e91fb38a0  0x610   File    0x100003        \Device\HarddiskVolume3\hiberfil.sys
4       System  0x840e91fb4520  0x614   File    0x140003        \Device\HarddiskVolume3\pagefile.sys
4       System  0x840e91fc9580  0x618   Thread  0x1fffff        Tid 792 Pid 4
4       System  0x840e917ff570  0x61c   Session 0x1     Session0
4       System  0x840e920fc500  0x620   IoCompletion    0x1f0003        -
4       System  0x840e90c60c10  0x624   WaitCompletionPacket    0x1     -
4       System  0xb708042b60d0  0x628   Key     0x9     MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\IMAGE FILE EXECUTION OPTIONS
4       System  0x840e922fb400  0x62c   Timer   0x1f0003        -
4       System  0x840e91f44510  0x630   File    0x150003        \Device\HarddiskVolume3\DumpStack.log.tmp
4       System  0x840e92297de0  0x634   Event   0x1f0003        -
4       System  0x840e911dc4e0  0x638   Timer   0x1f0003        -
4       System  0x840e921650c0  0x63c   Process 0x102a  csrss.exe Pid 804
4       System  0x840e91fb3a30  0x640   File    0x140003        \Device\HarddiskVolume3\swapfile.sys
4       System  0x840e921650c0  0x644   Process 0x1fffff        csrss.exe Pid 804
4       System  0x840e91f56760  0x648   Event   0x1f0003        -
4       System  0x840e91f563e0  0x64c   Event   0x1f0003        -
4       System  0x840e91fb3580  0x650   File    0x20003 \Device\HarddiskVolume3\Windows\System32\config\SAM
4       System  0xb708042b8b50  0x654   Key     0x2     MACHINE\SYSTEM\CONTROLSET001\CONTROL\CLASS\{4D36E97D-E325-11CE-BFC1-08002BE10318}\0034
4       System  0x840e921650c0  0x658   Process 0x102a  csrss.exe Pid 804
4       System  0x840e91fb4200  0x65c   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\SAM.LOG1
4       System  0x840e91fb5b00  0x660   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\SAM.LOG2
4       System  0x840e920f6390  0x664   File    0x20003 \Device\HarddiskVolume3\Windows\System32\config\DEFAULT
4       System  0x840e91f58360  0x668   Event   0x1f0003        -
4       System  0x840e920f8140  0x66c   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\DEFAULT.LOG1
4       System  0xb707fb8c8150  0x670   Section 0xf001f PhysicalMemory
4       System  0x840e922fb760  0x674   Timer   0x1f0003        -
4       System  0x840e91f64240  0x678   Thread  0x1fffff        Tid 836 Pid 4
4       System  0x840e920f5260  0x67c   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\DEFAULT.LOG2
4       System  0xb70801750460  0x680   Section 0x17    -
4       System  0xb708043531d0  0x684   SymbolicLink    0xf0001 ROOT#BasicRender#0000#{1ca05180-a699-450a-9a0c-de4fbe3ddd89}
4       System  0x840e92def080  0x688   Process 0x1fffff        svchost.exe Pid 1292
4       System  0xb708017bdee0  0x68c   SymbolicLink    0xf0001 DISPLAY#Default_Monitor#4&27b1e55b&0&UID0#{10910c20-0c64-4172-9409-add3064c0cad}
4       System  0xb708017bd790  0x690   SymbolicLink    0xf0001 DISPLAY#Default_Monitor#4&27b1e55b&0&UID0#{866519b5-3f07-4c97-b7df-24c5d8a8ccb8}
4       System  0x840e9228c080  0x694   Process 0x1fffff        wininit.exe Pid 880
4       System  0x840e921650c0  0x698   Process 0x40    csrss.exe Pid 804
4       System  0xb708042ba3c0  0x69c   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\CLASS\{4D36E96E-E325-11CE-BFC1-08002BE10318}\0000
4       System  0xb70804351bd0  0x6a0   SymbolicLink    0xf0001 ROOT#BasicDisplay#0000#{5b45201d-f2f2-4f3b-85bb-30ff1f953599}
4       System  0x840e91fa9080  0x6a4   Thread  0x1fffff        Tid 840 Pid 4
4       System  0x840e921650c0  0x6a8   Process 0x40    csrss.exe Pid 804
4       System  0x840e92297560  0x6ac   Event   0x1f0003        -
4       System  0x840e9029aba0  0x6b0   Timer   0x1f0003        -
4       System  0x840e92291140  0x6b4   Process 0x102a  csrss.exe Pid 888
4       System  0xb708044a54b0  0x6b8   Token   0xe     -
4       System  0xb708043f00e0  0x6bc   SymbolicLink    0xf0001 DISPLAY#Default_Monitor#4&27b1e55b&0&UID0#{e6f07b5f-ee97-4a90-b076-33f57bf4eaa7}
4       System  0xb708044225f0  0x6c0   Token   0xf01ff -
4       System  0xb7080435e380  0x6c4   Directory       0x4     WindowStations
4       System  0x840e9230b100  0x6c8   Process 0x102a  services.exe Pid 1000
4       System  0x840e9238d6d0  0x6cc   File    0x12008b        \Device\HarddiskVolume3\Windows\System32\SleepStudy\UserNotPresentSession.etl
4       System  0x840e923f1a70  0x6d0   TmTm    0xf003f -
4       System  0x840e92298d60  0x6d4   Event   0x1f0003        -
4       System  0x840e9227d5d0  0x6d8   Event   0x1f0003        UniqueInteractiveSessionIdEvent
4       System  0x840e923de2c0  0x6dc   Process 0x102a  svchost.exe Pid 756
4       System  0x840e90c63b80  0x6e0   WaitCompletionPacket    0x1     -
4       System  0x840e917ffdb0  0x6e4   Session 0x1     Session1
4       System  0x840e98fe0ae0  0x6e8   Event   0x1f0003        -
4       System  0x840e92296f60  0x6ec   Event   0x1f0003        -
4       System  0x840e92295360  0x6f0   Event   0x1f0003        -
4       System  0x840e922969e0  0x6f4   Event   0x1f0003        -
4       System  0x840e92296560  0x6f8   Event   0x1f0003        -
4       System  0x840e92291140  0x6fc   Process 0x102a  csrss.exe Pid 888
4       System  0x840e92291140  0x700   Process 0x1fffff        csrss.exe Pid 888
4       System  0xb7080446f870  0x704   Token   0xe     -
4       System  0x840e922fb250  0x708   Timer   0x1f0003        -
4       System  0x840e902995b0  0x70c   Timer   0x1f0003        -
4       System  0x840e9029a9f0  0x710   Timer   0x1f0003        -
4       System  0x840e9228c080  0x714   Process 0x102a  wininit.exe Pid 880
4       System  0xb7080132d6f0  0x718   Key     0x3001f MACHINE\SYSTEM\CROWDSTRIKE\{A123C27E-7DF1-4EAF-A2D0-1112802C7C79}
4       System  0xb708043fd380  0x71c   Directory       0x4     WindowStations
4       System  0x840e90e10410  0x720   Session 0x0     Session2
4       System  0x840e92299ee0  0x724   Event   0x1f0003        -
4       System  0x840e90c84750  0x728   ALPC Port       0x1f0001        -
4       System  0x840e922d7f80  0x72c   IoCompletion    0x1f0003        -
4       System  0x840e922d7cc0  0x730   IoCompletion    0x1f0003        -
4       System  0x840e922dc080  0x734   Thread  0x1fffff        Tid 940 Pid 4
4       System  0x840e922999e0  0x738   Event   0x1f0003        -
4       System  0x840e9229a0e0  0x73c   Event   0x1f0003        -
4       System  0x840e92299e60  0x740   Event   0x1f0003        -
4       System  0x840e92299ae0  0x744   Event   0x1f0003        -
4       System  0x840e92299460  0x748   Event   0x1f0003        -
4       System  0x840e92299de0  0x74c   Event   0x1f0003        -
4       System  0x840e92299660  0x750   Event   0x1f0003        -
4       System  0x840e92299ee0  0x754   Event   0x1f0003        -
4       System  0x840e902995b0  0x758   Timer   0x2     -
4       System  0x840e92299160  0x75c   Event   0x1f0003        -
4       System  0x840e922991e0  0x760   Event   0x1f0003        -
4       System  0x840e922fc840  0x764   Timer   0x1f0003        -
4       System  0x840e922998e0  0x768   Event   0x1f0003        -
4       System  0x840e92299360  0x76c   Event   0x1f0003        -
4       System  0x840e922994e0  0x770   Event   0x1f0003        -
4       System  0x840e92299160  0x774   Event   0x1f0003        -
4       System  0x840e922fc840  0x778   Timer   0x2     -
4       System  0x840e922fde60  0x77c   Desktop 0xf0003 Disconnect
4       System  0x840e9952c080  0x780   Process 0x102a  csrss.exe Pid 10732
4       System  0x840e98e8f460  0x784   Event   0x1f0003        -
4       System  0x840e92310070  0x788   RawInputManager 0x3     -
4       System  0x840e9229b1e0  0x78c   Event   0x1f0003        -
4       System  0x840e9c364620  0x790   File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee2a-852e-11ef-ba7c-bc2411845bc2}.TM
4       System  0x840e98fe8760  0x794   Event   0x1f0003        -
4       System  0x840e92305080  0x798   Process 0x102a  winlogon.exe Pid 980
4       System  0x840e92305080  0x79c   Process 0x1fffff        winlogon.exe Pid 980
4       System  0x840e9230b100  0x7a0   Process 0x1fffff        services.exe Pid 1000
4       System  0x840e92270c40  0x7a4   IoCompletion    0x1f0003        -
4       System  0x840e9229cee0  0x7a8   Event   0x1f0003        -
4       System  0x840e9229c0e0  0x7ac   Event   0x1f0003        -
4       System  0x840e92016070  0x7b0   ALPC Port       0x1f0001        -
4       System  0x840e92271a40  0x7b4   IoCompletion    0x1f0003        -
4       System  0x840e90ac6ce0  0x7b8   WaitCompletionPacket    0x1     -
4       System  0x840e90ac6db0  0x7bc   WaitCompletionPacket    0x1     -
4       System  0x840e922ecb00  0x7c0   Event   0x1f0003        EventShutDownCSRSS
4       System  0x840e9229b8e0  0x7c4   Event   0x1f0003        -
4       System  0x840e92270280  0x7c8   IoCompletion    0x1f0003        -
4       System  0x840e90ac63f0  0x7cc   WaitCompletionPacket    0x1     -
4       System  0x840e9229bb60  0x7d0   Event   0x1f0003        -
4       System  0x840e9229bc60  0x7d4   Event   0x1f0003        -
4       System  0x840e922fb5b0  0x7d8   Timer   0x1f0003        -
4       System  0x840e922fc9f0  0x7dc   Timer   0x1f0003        -
4       System  0x840e9229b560  0x7e0   Event   0x1f0003        -
4       System  0x840e9229b2e0  0x7e4   Event   0x1f0003        -
4       System  0x840e9229b660  0x7e8   Event   0x1f0003        -
4       System  0x840e9229bce0  0x7ec   Event   0x1f0003        -
4       System  0x840e9229bb60  0x7f0   Event   0x1f0003        -
4       System  0x840e922fb5b0  0x7f4   Timer   0x2     -
4       System  0x840e9ca43f00  0x7f8   IoCompletion    0x1f0003        -
4       System  0x840e9230e070  0x7fc   RawInputManager 0x3     -
4       System  0x840e91bd4ba0  0x804   Timer   0x1f0003        -
4       System  0x840e91bd4840  0x808   Timer   0x1f0003        -
4       System  0x840e92f58ce0  0x80c   Event   0x1f0003        -
4       System  0x840e91bd4d50  0x810   Timer   0x1f0003        -
4       System  0x840e92f58360  0x814   Event   0x1f0003        -
4       System  0x840e922fe810  0x818   Desktop 0xf0003 Disconnect
4       System  0x840e91bd4690  0x81c   Timer   0x1f0003        -
4       System  0x840e9b5e6510  0x820   File    0x100001        \Device\00000020
4       System  0x840e90e10410  0x824   Session 0x0     Session2
4       System  0x840e92349080  0x828   Process 0x102a  lsass.exe Pid 548
4       System  0x840e92c9a340  0x82c   Process 0x102a  svchost.exe Pid 1076
4       System  0x840e923f41c0  0x830   Process 0x102a  fontdrvhost.ex Pid 800
4       System  0x840e92c9a340  0x834   Process 0x1fffff        svchost.exe Pid 1076
4       System  0xb708045fa370  0x838   Directory       0xf000f 00000000-0001cc57
4       System  0x840e9229da60  0x83c   Event   0x1f0003        -
4       System  0x840e92271080  0x840   IoCompletion    0x1f0003        -
4       System  0x840e91bc8a50  0x844   WaitCompletionPacket    0x1     -
4       System  0x840e92349080  0x848   Process 0x1028  lsass.exe Pid 548
4       System  0x840e9230b100  0x84c   Process 0x102a  services.exe Pid 1000
4       System  0x840e9c121080  0x850   Process 0x102a  RuntimeBroker. Pid 10764
4       System  0x840e92313a60  0x854   ALPC Port       0x1f0001        -
4       System  0x840e92310a60  0x858   ALPC Port       0x1f0001        -
4       System  0x840e98d0c6c0  0x85c   IoCompletion    0x1f0003        -
4       System  0x840e9230b100  0x860   Process 0x102a  services.exe Pid 1000
4       System  0x840e9230b100  0x864   Process 0x102a  services.exe Pid 1000
4       System  0x840e92349080  0x868   Process 0x102a  lsass.exe Pid 548
4       System  0x840e92349080  0x870   Process 0x102a  lsass.exe Pid 548
4       System  0x840e9230b100  0x874   Process 0x102a  services.exe Pid 1000
4       System  0xb708044f7e00  0x878   Section 0x17    -
4       System  0x840e98d2bc40  0x87c   IoCompletion    0x1f0003        -
4       System  0x840e92349080  0x880   Process 0x1fffff        lsass.exe Pid 548
4       System  0x840e92d2e240  0x884   Process 0x102a  svchost.exe Pid 1320
4       System  0x840e99f4f3c0  0x888   IoCompletion    0x1f0003        -
4       System  0x840e9231f7f0  0x88c   File    0x20003 \Device\HarddiskVolume3\Windows\System32\config\SECURITY
4       System  0xb70804cb0160  0x890   Section 0x17    -
4       System  0xb70804aafdd0  0x894   Directory       0xf000f 00000000-00022c45
4       System  0x840e9228c080  0x898   Process 0x0     wininit.exe Pid 880
4       System  0x840e9231da40  0x89c   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\SECURITY.LOG1
4       System  0x840e9231f980  0x8a0   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\SECURITY.LOG2
4       System  0x840e92e04300  0x8a4   Process 0x102a  svchost.exe Pid 1408
4       System  0x840e99088b20  0x8a8   WaitCompletionPacket    0x1     -
4       System  0x840e92c517e0  0x8ac   WaitCompletionPacket    0x1     -
4       System  0x840e923de2c0  0x8b0   Process 0x102a  svchost.exe Pid 756
4       System  0x840e92def080  0x8b4   Process 0x102a  svchost.exe Pid 1292
4       System  0x840e92f36d80  0x8b8   IoCompletion    0x1f0003        -
4       System  0x840e92dc03e0  0x8bc   Event   0x1f0003        -
4       System  0xb70804a24060  0x8c0   Token   0xf01ff -
4       System  0x840e923de2c0  0x8c4   Process 0x102a  svchost.exe Pid 756
4       System  0x840e9238cd70  0x8c8   File    0x3     \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\NTUSER.DAT.LOG2
4       System  0x840e9237bde0  0x8cc   File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-95739096-486727260-2033287795-3853587803-1685597119-444378811-2746676523
4       System  0x840e9238b470  0x8d0   File    0x12019f        \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000002.regtrans-ms
4       System  0x840e92c9a340  0x8d4   Process 0x102a  svchost.exe Pid 1076
4       System  0xb708044f37d0  0x8d8   SymbolicLink    0xf0001 Local
4       System  0x840e9238d9f0  0x8dc   File    0x20003 \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\NTUSER.DAT
4       System  0x840e92cab2c0  0x8e0   Process 0x102a  svchost.exe Pid 1124
4       System  0x840e92d2e240  0x8e4   Process 0x1fffff        svchost.exe Pid 1320
4       System  0xb7080442edc0  0x8e8   Section 0xf001f -
4       System  0x840e92349080  0x8ec   Process 0x1478  lsass.exe Pid 548
4       System  0x840e92307d80  0x8f0   ALPC Port       0x1f0001        -
4       System  0x840e92349080  0x8f4   Process 0x102a  lsass.exe Pid 548
4       System  0xb7080442b040  0x8f8   Section 0xf001f -
4       System  0x840e9238f160  0x8fc   File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0x840e923de2c0  0x900   Process 0x102a  svchost.exe Pid 756
4       System  0xb708045f8570  0x904   Directory       0xf     RPC Control
4       System  0xb70804816c50  0x908   Key     0x9     MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER\QUOTA SYSTEM
4       System  0xb708044f2d80  0x90c   SymbolicLink    0xf0001 Global
4       System  0x840e923e4ce0  0x910   ALPC Port       0x1f0001        -
4       System  0xb7080442b440  0x914   Section 0xf001f -
4       System  0x840e923de2c0  0x918   Process 0x1fffff        svchost.exe Pid 756
4       System  0x840e923de2c0  0x91c   Process 0x102a  svchost.exe Pid 756
4       System  0xb708045f7670  0x920   Directory       0xf     S-1-15-2-95739096-486727260-2033287795-3853587803-1685597119-444378811-2746676523
4       System  0xb708044f2e30  0x924   SymbolicLink    0xf0001 Session
4       System  0x840e923f41c0  0x928   Process 0x1fffff        fontdrvhost.ex Pid 800
4       System  0x840e923de2c0  0x92c   Process 0x102a  svchost.exe Pid 756
4       System  0x840e923aa6e0  0x930   Event   0x1f0003        -
4       System  0xb708045fa550  0x934   Directory       0xf     S-1-15-2-95739096-486727260-2033287795-3853587803-1685597119-444378811-2746676523
4       System  0xb708045f9bf0  0x938   Directory       0xf     RPC Control
4       System  0xb708044f3a90  0x93c   SymbolicLink    0xf0001 Local
4       System  0xb708044f3300  0x940   SymbolicLink    0xf0001 Global
4       System  0xb708044f31a0  0x944   SymbolicLink    0xf0001 Session
4       System  0x840e920f5ee0  0x948   File    0x1f0006        \Device\NamedPipe\Sessions\0\AppContainerNamedObjects\S-1-15-2-95739096-486727260-2033287795-3853587803-1685597119-444378811-2746676523
4       System  0x840e923f61c0  0x94c   Process 0x102a  fontdrvhost.ex Pid 892
4       System  0x840e92353a80  0x950   TmRm    0x1f007f        -
4       System  0x840e923c0680  0x954   IoCompletion    0x1f0003        -
4       System  0xb7080442cac0  0x958   Section 0xf001f -
4       System  0x840e923c1f00  0x95c   IoCompletion    0x1f0003        -
4       System  0x840e91bcee40  0x960   WaitCompletionPacket    0x1     -
4       System  0x840e923f61c0  0x964   Process 0x1fffff        fontdrvhost.ex Pid 892
4       System  0x840e91bcfa70  0x968   WaitCompletionPacket    0x1     -
4       System  0x840e923dc730  0x96c   ALPC Port       0x1f0001        -
4       System  0x840e92c54560  0x970   WaitCompletionPacket    0x1     -
4       System  0x840e92352a80  0x974   TmRm    0x1f007f        -
4       System  0x840e923ab960  0x978   Event   0x1f0003        -
4       System  0x840e923f41c0  0x97c   Process 0x102a  fontdrvhost.ex Pid 800
4       System  0xb7080442e7c0  0x980   Section 0xf001f -
4       System  0x840e9f2d83e0  0x984   Event   0x1f0003        -
4       System  0x840e9238dea0  0x988   File    0x3     \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\NTUSER.DAT.LOG1
4       System  0x840e9238fac0  0x98c   File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0xb708048d2c20  0x990   Section 0x17    -
4       System  0x840e9238b2e0  0x994   File    0x12019f        \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000001.regtrans-ms
4       System  0x840e92cf49f0  0x998   ALPC Port       0x1f0001        -
4       System  0x840e92c9a340  0x99c   Process 0x102a  svchost.exe Pid 1076
4       System  0x840e92cdd7e0  0x9a0   ALPC Port       0x1f0001        -
4       System  0xb708045f90b0  0x9a4   Directory       0xf000f 00000000-000003e4
4       System  0x840e92cab2c0  0x9a8   Process 0x102a  svchost.exe Pid 1124
4       System  0x840e92caa070  0x9ac   ALPC Port       0x1f0001        -
4       System  0x840e92d51ac0  0x9b0   IoCompletion    0x1f0003        -
4       System  0x840e974b2080  0x9b4   Process 0x1fffff        UserOOBEBroker Pid 9620
4       System  0x840e92305080  0x9b8   Process 0x102a  winlogon.exe Pid 980
4       System  0x840e923de2c0  0x9bc   Process 0x102a  svchost.exe Pid 756
4       System  0x840e923de2c0  0x9c0   Process 0x102a  svchost.exe Pid 756
4       System  0x840e92cab2c0  0x9c4   Process 0x1fffff        svchost.exe Pid 1124
4       System  0x840e98db8080  0x9c8   Process 0x1028  MsSense.exe Pid 10224
4       System  0x840e92cf5070  0x9cc   ALPC Port       0x1f0001        -
4       System  0x840e923de2c0  0x9d0   Process 0x102a  svchost.exe Pid 756
4       System  0x840e923b48e0  0x9d4   Event   0x1f0003        -
4       System  0x840e92c4d3a0  0x9d8   WaitCompletionPacket    0x1     -
4       System  0x840e92cd6780  0x9dc   IoCompletion    0x1f0003        -
4       System  0x840e92f61260  0x9e0   Event   0x1f0003        -
4       System  0x840e966414c0  0x9e4   IoCompletion    0x1f0003        -
4       System  0x840e966690b0  0x9e8   WaitCompletionPacket    0x1     -
4       System  0xb70804a96a40  0x9ec   Key     0xf003f MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\VOLATILENOTIFICATIONS
4       System  0x840e92db7160  0x9f0   Event   0x1f0003        -
4       System  0x840e92cdac80  0x9f4   IoCompletion    0x1f0003        -
4       System  0x840e92c4ee70  0x9f8   WaitCompletionPacket    0x1     -
4       System  0x840e98ff32c0  0x9fc   Process 0x0     SecurityHealth Pid 8808
4       System  0x840e9c862fe0  0xa00   Event   0x1f0003        -
4       System  0x840e92cfa140  0xa04   Process 0x102a  dwm.exe Pid 1204
4       System  0x840e98b506d0  0xa08   WaitCompletionPacket    0x1     -
4       System  0x840e9b8fecc0  0xa0c   Timer   0x1f0003        -
4       System  0x840e92cfa140  0xa10   Process 0x1fffff        dwm.exe Pid 1204
4       System  0x840e92d19cd0  0xa14   File    0x20003 \Device\HarddiskVolume3\Windows\System32\config\BBI
4       System  0x840e92d19500  0xa18   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\BBI.LOG2
4       System  0x840e92cfa140  0xa1c   Process 0x102a  dwm.exe Pid 1204
4       System  0x840e92d1cbb0  0xa20   File    0x12019f        \Device\HarddiskVolume3\Windows\ServiceProfiles\LocalService\NTUSER.DAT{e0ec8a14-2a3a-11eb-ba6e-d2afaee3e4d2}.TMContainer00000000000000000001.regtrans-ms
4       System  0x840e92d1c0c0  0xa24   File    0x12019f        \Device\HarddiskVolume3\Windows\ServiceProfiles\LocalService\NTUSER.DAT{e0ec8a14-2a3a-11eb-ba6e-d2afaee3e4d2}.TM.blf
4       System  0x840e96f1d300  0xa28   Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e98db4080  0xa2c   Process 0x102a  msedge.exe Pid 9888
4       System  0x840e92c51d90  0xa30   WaitCompletionPacket    0x1     -
4       System  0x840e978164e0  0xa34   Event   0x1f0003        -
4       System  0x840e92d1a180  0xa38   File    0x3     \Device\HarddiskVolume3\Windows\System32\config\BBI.LOG1
4       System  0x840e92dd2a80  0xa3c   ALPC Port       0x1f0001        -
4       System  0x840e974557e0  0xa40   Event   0x1f0003        -
4       System  0x840e923de2c0  0xa44   Process 0x1028  svchost.exe Pid 756
4       System  0x840e92d1e190  0xa48   File    0x12019f        \Device\HarddiskVolume3\Windows\ServiceProfiles\LocalService\NTUSER.DAT{e0ec8a14-2a3a-11eb-ba6e-d2afaee3e4d2}.TMContainer00000000000000000002.regtrans-ms
4       System  0x840e92d1a4a0  0xa4c   File    0x20003 \Device\HarddiskVolume3\Windows\ServiceProfiles\LocalService\NTUSER.DAT
4       System  0x840e9783b080  0xa50   Process 0x102a  CSFalconContai Pid 6604
4       System  0x840e92d1b760  0xa54   File    0x3     \Device\HarddiskVolume3\Windows\ServiceProfiles\LocalService\NTUSER.DAT.LOG2
4       System  0x840e96c5a080  0xa58   Process 0x0     svchost.exe Pid 7108
4       System  0x840e90ad3080  0xa5c   Process 0x0     svchost.exe Pid 7152
4       System  0x840e978c0300  0xa60   Process 0x0     explorer.exe Pid 5328
4       System  0x840e96f1d300  0xa64   Process 0x0     dasHost.exe Pid 3752
4       System  0x840e92def080  0xa68   Process 0x102a  svchost.exe Pid 1292
4       System  0x840e92dcfd50  0xa6c   TmRm    0x1f007f        -
4       System  0x840e99f770c0  0xa70   IoCompletion    0x1f0003        -
4       System  0x840e99715760  0xa74   Event   0x1f0003        -
4       System  0xb70804cb1560  0xa78   Section 0x17    -
4       System  0x840e97582080  0xa7c   Process 0x102a  msedge.exe Pid 16344
4       System  0x840e96ab1590  0xa80   File    0x12008b        \Device\HarddiskVolume3\ProgramData\Microsoft\Windows Defender Advanced Threat Protection\Trace\SenseIRTraceLogger133862713365250000.etl
4       System  0x840e90261040  0xa84   Process 0x0     System Pid 4
4       System  0x840e9ac0f300  0xa88   WaitCompletionPacket    0x1     -
4       System  0x840e9c8966e0  0xa8c   Event   0x1f0003        -
4       System  0x840e92e30080  0xa94   Process 0x102a  svchost.exe Pid 1392
4       System  0x840e92d1ec80  0xa98   File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Windows\ServiceProfiles\LocalService\NTUSER.DAT{e0ec8a14-2a3a-11eb-ba6e-d2afaee3e4d2}.TM
4       System  0x840e92d1d510  0xa9c   File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Windows\ServiceProfiles\LocalService\NTUSER.DAT{e0ec8a14-2a3a-11eb-ba6e-d2afaee3e4d2}.TM
4       System  0x840e92d1b2b0  0xaa0   File    0x3     \Device\HarddiskVolume3\Windows\ServiceProfiles\LocalService\NTUSER.DAT.LOG1
4       System  0x840e92de8a80  0xaa4   ALPC Port       0x1f0001        -
4       System  0x840e92dbbf60  0xaa8   Event   0x1f0003        -
4       System  0x840e92dfa240  0xaac   IoCompletion    0x1f0003        -
4       System  0x840e9c3606c0  0xab4   File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee2a-852e-11ef-ba7c-bc2411845bc2}.TM.blf
4       System  0x840e923f7420  0xab8   TmTm    0xf003f -
4       System  0x840e92dcfa80  0xabc   TmRm    0x1f007f        -
4       System  0xb70804aae930  0xac0   Directory       0xf000f 00000000-000003e5
4       System  0x840e92dbea60  0xac4   Event   0x1f0003        -
4       System  0x840e91da7ee0  0xac8   Event   0x1f0003        -
4       System  0x840e92dbf560  0xacc   Event   0x1f0003        -
4       System  0x840e92d4ff80  0xad0   IoCompletion    0x1f0003        -
4       System  0x840e92c51cc0  0xad4   WaitCompletionPacket    0x1     -
4       System  0x840e92d2e240  0xad8   Process 0x102a  svchost.exe Pid 1320
4       System  0x840e92d50b00  0xadc   IoCompletion    0x1f0003        -
4       System  0x840e99fd9be0  0xae4   Event   0x1f0003        -
4       System  0x840e92e732c0  0xae8   Process 0x1fffff        svchost.exe Pid 1580
4       System  0x840e9ac40a60  0xaec   WaitCompletionPacket    0x1     -
4       System  0x840e92d48080  0xaf4   Process 0x102a  svchost.exe Pid 1364
4       System  0x840e92e732c0  0xaf8   Process 0x102a  svchost.exe Pid 1580
4       System  0x840e92d48080  0xafc   Process 0x1fffff        svchost.exe Pid 1364
4       System  0x840e92e59f60  0xb00   Event   0x1f0003        -
4       System  0x840e92e48080  0xb04   Process 0x1028  svchost.exe Pid 1524
4       System  0x840e92e04300  0xb08   Process 0x1fffff        svchost.exe Pid 1408
4       System  0x840e92d55d40  0xb0c   IoCompletion    0x1f0003        -
4       System  0x840e92e30080  0xb10   Process 0x1fffff        svchost.exe Pid 1392
4       System  0x840e92d48080  0xb14   Process 0x102a  svchost.exe Pid 1364
4       System  0x840e92c54220  0xb18   WaitCompletionPacket    0x1     -
4       System  0x840e92def080  0xb1c   Process 0x102a  svchost.exe Pid 1292
4       System  0x840e92e04300  0xb20   Process 0x102a  svchost.exe Pid 1408
4       System  0x840e92e368c0  0xb24   IoCompletion    0x1f0003        -
4       System  0x840e92e536e0  0xb28   Event   0x1f0003        -
4       System  0x840e92eac340  0xb30   Process 0x102a  svchost.exe Pid 1612
4       System  0x840e92e48080  0xb34   Process 0x102a  svchost.exe Pid 1524
4       System  0x840e92dc32e0  0xb38   Event   0x1f0003        -
4       System  0x840e92dc1ce0  0xb3c   Event   0x1f0003        -
4       System  0x840e92e30080  0xb40   Process 0x102a  svchost.exe Pid 1392
4       System  0x840e92e34e80  0xb44   IoCompletion    0x1f0003        -
4       System  0x840e92c52ea0  0xb48   WaitCompletionPacket    0x1     -
4       System  0x840e92e8f500  0xb50   IoCompletion    0x1f0003        -
4       System  0x840e92c53930  0xb54   WaitCompletionPacket    0x1     -
4       System  0x840e92c55e90  0xb58   WaitCompletionPacket    0x1     -
4       System  0x840e92e38c80  0xb5c   IoCompletion    0x1f0003        -
4       System  0x840e92e48080  0xb60   Process 0x1fffff        svchost.exe Pid 1524
4       System  0x840e92dc4ee0  0xb64   Event   0x1f0003        -
4       System  0x840e96c5e1c0  0xb68   Process 0x102a  svchost.exe Pid 2220
4       System  0x840e92e59160  0xb6c   Event   0x1f0003        -
4       System  0x840e911653f0  0xb70   WaitCompletionPacket    0x1     -
4       System  0x840e92e8c940  0xb74   IoCompletion    0x1f0003        -
4       System  0x840e92eac340  0xb78   Process 0x1fffff        svchost.exe Pid 1612
4       System  0xb70804de5830  0xb7c   Token   0xf01ff -
4       System  0x840e92cfa140  0xb80   Process 0x102a  dwm.exe Pid 1204
4       System  0x840e92f31600  0xb84   IoCompletion    0x1f0003        -
4       System  0x840e92e48080  0xb88   Process 0x102a  svchost.exe Pid 1524
4       System  0x840e92e8bb40  0xb8c   IoCompletion    0x1f0003        -
4       System  0x840e92f23080  0xb90   Process 0x102a  svchost.exe Pid 1776
4       System  0x840e92e57de0  0xb94   Event   0x1f0003        -
4       System  0x840e92eac340  0xb98   Process 0x102a  svchost.exe Pid 1612
4       System  0x840e92e732c0  0xb9c   Process 0x102a  svchost.exe Pid 1580
4       System  0x840e92edc7c0  0xba0   WaitCompletionPacket    0x1     -
4       System  0x840e90c643a0  0xba4   WaitCompletionPacket    0x1     -
4       System  0x840e92e732c0  0xba8   Process 0x102a  svchost.exe Pid 1580
4       System  0x840e92e86f00  0xbac   IoCompletion    0x1f0003        -
4       System  0x840e92e5b560  0xbb0   Event   0x1f0003        -
4       System  0xb70804ee1060  0xbb4   Token   0xf01ff -
4       System  0x840e92e5b760  0xbb8   Event   0x1f0003        -
4       System  0x840e92e90680  0xbbc   IoCompletion    0x1f0003        -
4       System  0x840e92e8aa80  0xbc0   IoCompletion    0x1f0003        -
4       System  0x840e92e58960  0xbc4   Event   0x1f0003        -
4       System  0x840e92c57e40  0xbc8   WaitCompletionPacket    0x1     -
4       System  0x840e92eafa80  0xbcc   ALPC Port       0x1f0001        -
4       System  0x840e92eaa2c0  0xbd0   Process 0x102a  svchost.exe Pid 1604
4       System  0x840e92cc6bd0  0xbd4   CoreMessaging   0xf0000 -
4       System  0x840e92eaa2c0  0xbd8   Process 0x1fffff        svchost.exe Pid 1604
4       System  0x840e92e0cd20  0xbdc   ALPC Port       0x1f0001        [CoreMsgK]-{8f45e070-ff60-11ef-ba82-bc2411703d8b}
4       System  0x840e91a824e0  0xbe0   CoreMessaging   0xf0000 -
4       System  0x840e92cc7200  0xbe4   CoreMessaging   0xf0000 -
4       System  0x840e92eaa2c0  0xbe8   Process 0x102a  svchost.exe Pid 1604
4       System  0x840e92c58250  0xbec   WaitCompletionPacket    0x1     -
4       System  0x840e92eb0ce0  0xbf0   ALPC Port       0x1f0001        -
4       System  0x840e92cc7290  0xbf4   CoreMessaging   0xf0000 -
4       System  0x840e9170de50  0xbf8   WaitCompletionPacket    0x1     -
4       System  0x840e92f27080  0xbfc   Process 0x1fffff        svchost.exe Pid 1808
4       System  0x840e92cc6360  0xc04   CoreMessaging   0xf0000 -
4       System  0x840e92f04340  0xc08   Process 0x1fffff        svchost.exe Pid 1652
4       System  0x840e9c8bccf0  0xc0c   File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee29-852e-11ef-ba7c-bc2411845bc2}.TxR.blf
4       System  0x840e92e5e360  0xc10   Event   0x1f0003        -
4       System  0x840e92f04340  0xc14   Process 0x102a  svchost.exe Pid 1652
4       System  0x840e92f04340  0xc18   Process 0x102a  svchost.exe Pid 1652
4       System  0x840e92edd0b0  0xc1c   WaitCompletionPacket    0x1     -
4       System  0x840e92e93240  0xc20   IoCompletion    0x1f0003        -
4       System  0x840e92e732c0  0xc24   Process 0x102a  svchost.exe Pid 1580
4       System  0x840e9c932f60  0xc2c   Event   0x1f0003        -
4       System  0x840e96860080  0xc30   Process 0x102a  svchost.exe Pid 2396
4       System  0x840e92f23080  0xc34   Process 0x102a  svchost.exe Pid 1776
4       System  0x840e92e5fd60  0xc38   Event   0x1f0003        -
4       System  0x840e92e926c0  0xc3c   IoCompletion    0x1f0003        -
4       System  0x840e92edde80  0xc40   WaitCompletionPacket    0x1     -
4       System  0x840e92e5f660  0xc44   Event   0x1f0003        -
4       System  0x840e92e92440  0xc48   IoCompletion    0x1f0003        -
4       System  0x840e92edd8d0  0xc4c   WaitCompletionPacket    0x1     -
4       System  0x840e92cc73b0  0xc50   CoreMessaging   0xf0000 -
4       System  0x840e92f202c0  0xc54   Process 0x102a  svchost.exe Pid 1764
4       System  0x840e92cc7320  0xc58   CoreMessaging   0xf0000 -
4       System  0x840e92cc65a0  0xc5c   CoreMessaging   0xf0000 -
4       System  0x840e92cc6ab0  0xc60   CoreMessaging   0xf0000 -
4       System  0x840e92f202c0  0xc64   Process 0x1fffff        svchost.exe Pid 1764
4       System  0x840e99fdcb60  0xc68   Event   0x1f0003        -
4       System  0x840e968109d0  0xc6c   WaitCompletionPacket    0x1     -
4       System  0x840e92ee07f0  0xc70   WaitCompletionPacket    0x1     -
4       System  0x840e92f23080  0xc74   Process 0x1fffff        svchost.exe Pid 1776
4       System  0xb7080131dbb0  0xc78   Key     0x2001f MACHINE\SOFTWARE\MICROSOFT\WINDOWS\CURRENTVERSION\GROUP POLICY\STATE\MACHINE
4       System  0x840e92f4e360  0xc7c   Event   0x1f0003        -
4       System  0x840e92f202c0  0xc80   Process 0x102a  svchost.exe Pid 1764
4       System  0x840e96860080  0xc84   Process 0x102a  svchost.exe Pid 2396
4       System  0x840e92fbb0c0  0xc88   Process 0x102a  svchost.exe Pid 604
4       System  0xb70809310730  0xc8c   Token   0xf01ff -
4       System  0x840e91a82370  0xc90   FilterConnectionPort    0x1f0001        CLDMSGPORT
4       System  0x840e92ef4a80  0xc98   ALPC Port       0x1f0001        -
4       System  0xb70804fdcf00  0xc9c   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\SERVICES\WCIFS\INSTANCES\WCIFS INSTANCE
4       System  0x840e92c49100  0xca4   WaitCompletionPacket    0x1     -
4       System  0x840e966f82e0  0xca8   Event   0x1f0003        -
4       System  0x840e92f82080  0xcac   File    0x3     \Device\HarddiskVolume3
4       System  0x840ea17e9080  0xcb0   Process 0x102a  svchost.exe Pid 10392
4       System  0x840e9ac29f30  0xcb4   WaitCompletionPacket    0x1     -
4       System  0x840e91a82420  0xcb8   FilterConnectionPort    0x1f0001        WcifsPort
4       System  0x840e92f53760  0xcbc   Event   0x1f0003        -
4       System  0x840e91a83810  0xcc0   FilterConnectionPort    0x1f0001        storqosfltport
4       System  0x840e92fbb0c0  0xcc4   Process 0x102a  svchost.exe Pid 604
4       System  0x840e92f533e0  0xcc8   Event   0x1f0003        -
4       System  0x840e92fdc070  0xccc   ALPC Port       0x1f0001        -
4       System  0x840e92f27080  0xcd0   Process 0x102a  svchost.exe Pid 1808
4       System  0x840e92f5fa60  0xcd4   Event   0x1f0003        -
4       System  0x840e92f54f60  0xcd8   Event   0x1f0003        -
4       System  0x840e92f580e0  0xcdc   Event   0x1f0003        -
4       System  0x840e92e1b0a0  0xce0   Timer   0x2     -
4       System  0x840e92fbb0c0  0xce4   Process 0x1fffff        svchost.exe Pid 604
4       System  0x840e92f35080  0xce8   IoCompletion    0x1f0003        -
4       System  0x840e92ee49c0  0xcec   WaitCompletionPacket    0x1     -
4       System  0x840e92f23080  0xcf4   Process 0x102a  svchost.exe Pid 1776
4       System  0x840e92fdb2d0  0xcf8   ALPC Port       0x1f0001        -
4       System  0x840e92e732c0  0xcfc   Process 0x102a  svchost.exe Pid 1580
4       System  0x840e92fc7080  0xd00   Process 0x1fffff        svchost.exe Pid 1988
4       System  0x840e92f27080  0xd04   Process 0x102a  svchost.exe Pid 1808
4       System  0x840e966f7140  0xd08   Event   0x1f0003        -
4       System  0x840e92f56660  0xd0c   Event   0x1f0003        -
4       System  0x840e92ee5860  0xd10   WaitCompletionPacket    0x1     -
4       System  0x840e92e732c0  0xd14   Process 0x102a  svchost.exe Pid 1580
4       System  0x840e92fc6080  0xd18   Process 0x1fffff        svchost.exe Pid 1964
4       System  0x840e92fc5300  0xd1c   Process 0x102a  svchost.exe Pid 1976
4       System  0x840e92cc7710  0xd20   CoreMessaging   0xf0000 -
4       System  0xb70804f40060  0xd24   Token   0xf01ff -
4       System  0x840e92e1b250  0xd28   Timer   0x2     -
4       System  0x840e92f25a80  0xd2c   ALPC Port       0x1f0001        -
4       System  0x840e92f23080  0xd30   Process 0x102a  svchost.exe Pid 1776
4       System  0x840e92f54fe0  0xd34   Event   0x1f0003        -
4       System  0x840e9663bf00  0xd38   IoCompletion    0x1f0003        -
4       System  0x840e92fc6080  0xd3c   Process 0x102a  svchost.exe Pid 1964
4       System  0x840e92f574e0  0xd40   Event   0x1f0003        -
4       System  0x840e91bd3ac0  0xd44   Timer   0x2     -
4       System  0x840e92f57860  0xd48   Event   0x1f0003        -
4       System  0x840e92f577e0  0xd4c   Event   0x1f0003        -
4       System  0x840e92f57260  0xd50   Event   0x1f0003        -
4       System  0x840e92f572e0  0xd54   Event   0x1f0003        -
4       System  0x840e91bd3c70  0xd58   Timer   0x2     -
4       System  0x840e92f58160  0xd5c   Event   0x1f0003        -
4       System  0x840e92f57be0  0xd60   Event   0x1f0003        -
4       System  0x840e92f57fe0  0xd64   Event   0x1f0003        -
4       System  0x840e91bd4180  0xd68   Timer   0x2     -
4       System  0x840e91bd4330  0xd6c   Timer   0x2     -
4       System  0x840e92f57560  0xd70   Event   0x1f0003        -
4       System  0x840e92cc74d0  0xd74   CoreMessaging   0xf0000 -
4       System  0x840e92cc75f0  0xd78   CoreMessaging   0xf0000 -
4       System  0x840e91a83ad0  0xd7c   FilterConnectionPort    0x1f0001        BindFltPort
4       System  0x840e92cc8400  0xd80   CoreMessaging   0xf0000 -
4       System  0x840e92cc77a0  0xd84   CoreMessaging   0xf0000 -
4       System  0x840e92cc81c0  0xd88   CoreMessaging   0xf0000 -
4       System  0x840e92cc7ef0  0xd8c   CoreMessaging   0xf0000 -
4       System  0x840e92cc7830  0xd90   CoreMessaging   0xf0000 -
4       System  0x840e92cc7f80  0xd94   CoreMessaging   0xf0000 -
4       System  0x840e92cc7a70  0xd98   CoreMessaging   0xf0000 -
4       System  0x840e92cc8250  0xd9c   CoreMessaging   0xf0000 -
4       System  0x840e92f3a3c0  0xda0   IoCompletion    0x1f0003        -
4       System  0x840e92ee7e90  0xda4   WaitCompletionPacket    0x1     -
4       System  0x840e92ee93b0  0xda8   WaitCompletionPacket    0x1     -
4       System  0x840e92ee9960  0xdac   WaitCompletionPacket    0x1     -
4       System  0x840e92ee9480  0xdb0   WaitCompletionPacket    0x1     -
4       System  0x840e92ee9550  0xdb4   WaitCompletionPacket    0x1     -
4       System  0x840e92ee89f0  0xdb8   WaitCompletionPacket    0x1     -
4       System  0x840e92ee9620  0xdbc   WaitCompletionPacket    0x1     -
4       System  0x840e92ee8100  0xdc0   WaitCompletionPacket    0x1     -
4       System  0x840e92ee8c60  0xdc4   WaitCompletionPacket    0x1     -
4       System  0x840e92ee96f0  0xdc8   WaitCompletionPacket    0x1     -
4       System  0x840e92ee8ed0  0xdcc   WaitCompletionPacket    0x1     -
4       System  0x840e92ee97c0  0xdd0   WaitCompletionPacket    0x1     -
4       System  0x840e92ee8920  0xdd4   WaitCompletionPacket    0x1     -
4       System  0x840e92ee9a30  0xdd8   WaitCompletionPacket    0x1     -
4       System  0x840e92ee8d30  0xddc   WaitCompletionPacket    0x1     -
4       System  0x840ea08f1080  0xde0   Process 0x102a  svchost.exe Pid 3444
4       System  0x840e92f58ce0  0xde4   Event   0x1f0003        -
4       System  0x840e9ccec080  0xde8   Process 0x102a  svchost.exe Pid 11776
4       System  0x840e92f5fae0  0xdec   Event   0x1f0003        -
4       System  0x840e9c5ac2e0  0xdf4   Event   0x1f0003        -
4       System  0x840e9c368240  0xdf8   File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee2a-852e-11ef-ba7c-bc2411845bc2}.TM
4       System  0x840e96667d30  0xdfc   WaitCompletionPacket    0x1     -
4       System  0x840e92fc5300  0xe00   Process 0x102a  svchost.exe Pid 1976
4       System  0x840e966711e0  0xe04   WaitCompletionPacket    0x1     -
4       System  0x840e966682e0  0xe08   WaitCompletionPacket    0x1     -
4       System  0x840e92fbb0c0  0xe0c   Process 0x102a  svchost.exe Pid 604
4       System  0x840e92fc5300  0xe18   Process 0x1fffff        svchost.exe Pid 1976
4       System  0x840e92fc7080  0xe1c   Process 0x102a  svchost.exe Pid 1988
4       System  0x840e92fc6080  0xe20   Process 0x102a  svchost.exe Pid 1964
4       System  0x840e9663b740  0xe24   IoCompletion    0x1f0003        -
4       System  0x840e9832a1c0  0xe28   Process 0x102a  msedge.exe Pid 3832
4       System  0x840e92fc7080  0xe2c   Process 0x102a  svchost.exe Pid 1988
4       System  0x840e92f5e1e0  0xe30   Event   0x1f0003        -
4       System  0x840e96667780  0xe34   WaitCompletionPacket    0x1     -
4       System  0x840e9663b900  0xe38   IoCompletion    0x1f0003        -
4       System  0x840e966eb8c0  0xe3c   IoCompletion    0x1f0003        -
4       System  0x840e9676b300  0xe44   Process 0x1fffff        svchost.exe Pid 2180
4       System  0x840e92f6bf60  0xe48   Event   0x1f0003        -
4       System  0x840e96671e10  0xe4c   WaitCompletionPacket    0x1     -
4       System  0x840e966eb1c0  0xe50   IoCompletion    0x1f0003        -
4       System  0x840e92f6b260  0xe54   Event   0x1f0003        -
4       System  0x840e9c896fe0  0xe60   Event   0x1f0003        -
4       System  0x840e9676b300  0xe64   Process 0x102a  svchost.exe Pid 2180
4       System  0x840e98c48080  0xe68   Process 0x40    ApplicationFra Pid 4836
4       System  0x840e99da3080  0xe6c   Process 0x102a  RuntimeBroker. Pid 4464
4       System  0x840e98184ab0  0xe70   WaitCompletionPacket    0x1     -
4       System  0x840ea70570c0  0xe74   Process 0x102a  msedge.exe Pid 1336
4       System  0x840e966ab530  0xe78   ALPC Port       0x1f0001        -
4       System  0x840e92fbb0c0  0xe7c   Process 0x102a  svchost.exe Pid 604
4       System  0x840e92fbb0c0  0xe80   Process 0x102a  svchost.exe Pid 604
4       System  0x840e92f609e0  0xe84   Event   0x1f0003        -
4       System  0x840e9663f2c0  0xe88   IoCompletion    0x1f0003        -
4       System  0x840e9666a0f0  0xe8c   WaitCompletionPacket    0x1     -
4       System  0x840e92f6bce0  0xe90   Event   0x1f0003        -
4       System  0x840e966b1080  0xe94   Process 0x1fffff        MemCompression Pid 1924
4       System  0x840e9676b300  0xe98   Process 0x102a  svchost.exe Pid 2180
4       System  0x840e981a41a0  0xe9c   WaitCompletionPacket    0x1     -
4       System  0x840e9229d0e0  0xea0   Event   0x1f0003        -
4       System  0x840e96640180  0xea4   IoCompletion    0x1f0003        -
4       System  0x840e96669b40  0xea8   WaitCompletionPacket    0x1     -
4       System  0x840e92cfa140  0xeac   Process 0x102a  dwm.exe Pid 1204
4       System  0x840e92cfa140  0xeb0   Process 0x40    dwm.exe Pid 1204
4       System  0x840e966d62c0  0xeb4   Process 0x102a  svchost.exe Pid 2092
4       System  0x840e92fc6080  0xeb8   Process 0x102a  svchost.exe Pid 1964
4       System  0x840e966eaf00  0xebc   IoCompletion    0x1f0003        -
4       System  0x840e96671ad0  0xec0   WaitCompletionPacket    0x1     -
4       System  0x840e92f61f60  0xec4   Event   0x1f0003        -
4       System  0x840e9676e080  0xec8   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9665eac0  0xecc   Event   0x1f0003        DwmComposedEvent_1
4       System  0x840e92fbb0c0  0xed0   Process 0x102a  svchost.exe Pid 604
4       System  0x840e966d62c0  0xed4   Process 0x102a  svchost.exe Pid 2092
4       System  0x840e9676e080  0xed8   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e96769240  0xedc   Process 0x102a  svchost.exe Pid 2160
4       System  0x840e966d62c0  0xee0   Process 0x1fffff        svchost.exe Pid 2092
4       System  0x840e92f641e0  0xee4   Event   0x1f0003        -
4       System  0x840e966e39c0  0xee8   IoCompletion    0x1f0003        -
4       System  0x840e9666ad20  0xeec   WaitCompletionPacket    0x1     -
4       System  0x840e96769240  0xef0   Process 0x102a  svchost.exe Pid 2160
4       System  0x840e9676e080  0xef4   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9676e080  0xef8   Process 0x1fffff        svchost.exe Pid 2168
4       System  0x840e96769240  0xefc   Process 0x1fffff        svchost.exe Pid 2160
4       System  0xb7081bb44c00  0xf00   Directory       0xf000f 00000000-0262ee0b
4       System  0x840e92f6bd60  0xf04   Event   0x1f0003        -
4       System  0x840e966ea2c0  0xf08   IoCompletion    0x1f0003        -
4       System  0x840e9676e080  0xf0c   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e96671c70  0xf10   WaitCompletionPacket    0x1     -
4       System  0xb708053faec0  0xf14   SymbolicLink    0xf0001 SWD#MMDEVAPI#MicrosoftGSWavetableSynth#{6dc23320-ab33-4ce4-80d4-bbb3ebbf2814}
4       System  0x840e990b41d0  0xf18   WaitCompletionPacket    0x1     -
4       System  0x840e97c48460  0xf20   Event   0x1f0003        -
4       System  0x840e96794080  0xf24   Process 0x102a  svchost.exe Pid 2276
4       System  0x840e992d0dc0  0xf28   IoCompletion    0x1f0003        -
4       System  0x840e9676e080  0xf2c   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e966f0d80  0xf30   IoCompletion    0x1f0003        -
4       System  0x840e96794080  0xf34   Process 0x1fffff        svchost.exe Pid 2276
4       System  0x840e997086e0  0xf38   Event   0x1f0003        -
4       System  0x840e967ccc60  0xf3c   Event   0x1f0003        -
4       System  0x840e96794080  0xf40   Process 0x102a  svchost.exe Pid 2276
4       System  0x840e9676e080  0xf44   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9676e080  0xf48   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9676e080  0xf4c   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9676e080  0xf50   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9676e080  0xf54   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9676e080  0xf58   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9676e080  0xf5c   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e96803de0  0xf60   WaitCompletionPacket    0x1     -
4       System  0x840e96875b00  0xf64   IoCompletion    0x1f0003        -
4       System  0x840e96860080  0xf68   Process 0x102a  svchost.exe Pid 2396
4       System  0xb708056c5bf0  0xf6c   Key     0x2001f MACHINE\SOFTWARE\MICROSOFT\WINDOWS\CURRENTVERSION\GROUP POLICY\STATE\MACHINE
4       System  0x840e967d4ae0  0xf70   Event   0x1f0003        -
4       System  0x840e96774940  0xf74   ALPC Port       0x1f0001        -
4       System  0x840e96862080  0xf78   Process 0x1fffff        svchost.exe Pid 2376
4       System  0x840e92fbb0c0  0xf7c   Process 0x102a  svchost.exe Pid 604
4       System  0x840e96624a80  0xf80   ALPC Port       0x1f0001        -
4       System  0x840e967d4860  0xf84   Event   0x1f0003        -
4       System  0x840e96864380  0xf88   Process 0x102a  svchost.exe Pid 2384
4       System  0x840e96864380  0xf8c   Process 0x1fffff        svchost.exe Pid 2384
4       System  0x840e967f3ce0  0xf90   ALPC Port       0x1f0001        -
4       System  0x840e9680ee30  0xf94   WaitCompletionPacket    0x1     -
4       System  0x840e96873900  0xf98   IoCompletion    0x1f0003        -
4       System  0x840e96794080  0xf9c   Process 0x102a  svchost.exe Pid 2276
4       System  0x840e96860080  0xfa0   Process 0x102a  svchost.exe Pid 2396
4       System  0x840e92ca9df0  0xfa4   ALPC Port       0x1f0001        -
4       System  0x840e96794080  0xfa8   Process 0x102a  svchost.exe Pid 2276
4       System  0x840e968c00c0  0xfac   Process 0x1fffff        svchost.exe Pid 2496
4       System  0x840e969be180  0xfb0   Thread  0x1fffff        Tid 2780 Pid 4
4       System  0x840e96864380  0xfb4   Process 0x102a  svchost.exe Pid 2384
4       System  0x840e923dc970  0xfb8   ALPC Port       0x1f0001        -
4       System  0x840e9689a080  0xfbc   Process 0x1fffff        svchost.exe Pid 2472
4       System  0x840e96860080  0xfc0   Process 0x102a  svchost.exe Pid 2396
4       System  0x840e96860080  0xfc4   Process 0x102a  svchost.exe Pid 2396
4       System  0x840e96862080  0xfc8   Process 0x102a  svchost.exe Pid 2376
4       System  0xb70805c43040  0xfcc   Token   0xe     -
4       System  0x840e96860080  0xfd0   Process 0x1fffff        svchost.exe Pid 2396
4       System  0x840e9689a080  0xfd4   Process 0x102a  svchost.exe Pid 2472
4       System  0x840e96862080  0xfd8   Process 0x102a  svchost.exe Pid 2376
4       System  0x840e96872080  0xfdc   IoCompletion    0x1f0003        -
4       System  0x840e967d4060  0xfe0   Event   0x1f0003        -
4       System  0x840e9680d9e0  0xfe4   WaitCompletionPacket    0x1     -
4       System  0xb708061a28b0  0xfe8   Key     0x2001f MACHINE\SYSTEM\CONTROLSET001\SERVICES\HTTP\PARAMETERS\URLACLINFO
4       System  0x840e968c22c0  0xfec   Process 0x102a  svchost.exe Pid 2504
4       System  0x840e9676e080  0xff0   Process 0x102a  svchost.exe Pid 2168
4       System  0x840e967e7a60  0xff4   Event   0x1f0003        -
4       System  0x840e96860080  0xff8   Process 0x102a  svchost.exe Pid 2396
4       System  0x840e968c22c0  0xffc   Process 0x1fffff        svchost.exe Pid 2504
4       System  0x840e968989f0  0x1004  ALPC Port       0x1f0001        -
4       System  0x840e968c00c0  0x1008  Process 0x102a  svchost.exe Pid 2496
4       System  0x840e9689a080  0x100c  Process 0x102a  svchost.exe Pid 2472
4       System  0x840e96860080  0x1010  Process 0x102a  svchost.exe Pid 2396
4       System  0x840e96bc00c0  0x1014  Process 0x102a  svchost.exe Pid 2848
4       System  0x840e96baf0c0  0x1018  Thread  0x1fffff        Tid 2844 Pid 4
4       System  0x840e96860080  0x101c  Process 0x102a  svchost.exe Pid 2396
4       System  0x840e96897940  0x1020  ALPC Port       0x1f0001        -
4       System  0x840e96860080  0x1024  Process 0x102a  svchost.exe Pid 2396
4       System  0x840e96860080  0x1028  Process 0x102a  svchost.exe Pid 2396
4       System  0x840e96860080  0x102c  Process 0x102a  svchost.exe Pid 2396
4       System  0x840e96860080  0x1030  Process 0x102a  svchost.exe Pid 2396
4       System  0x840e9676e080  0x1034  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e96860080  0x1038  Process 0x102a  svchost.exe Pid 2396
4       System  0x840e968c22c0  0x103c  Process 0x102a  svchost.exe Pid 2504
4       System  0x840e96860080  0x1040  Process 0x102a  svchost.exe Pid 2396
4       System  0xb70805bb8730  0x1044  Token   0xf01ff -
4       System  0x840e97ab7c70  0x1048  File    0x12019f        \Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat{54a96616-79ef-11ef-ba74-bc2411d898d1}.TMContainer00000000000000000001.regtrans-ms
4       System  0x840e978ed080  0x104c  Process 0x102a  cmd.exe Pid 13752
4       System  0x840e968c00c0  0x1050  Process 0x102a  svchost.exe Pid 2496
4       System  0x840e96bc00c0  0x1054  Process 0x102a  svchost.exe Pid 2848
4       System  0x840e9676e080  0x1058  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e92cfa140  0x105c  Process 0x102a  dwm.exe Pid 1204
4       System  0x840e96c23280  0x1060  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e997a81e0  0x1064  Event   0x1f0003        -
4       System  0x840e968ad170  0x1068  File    0x12019f        \Device\HarddiskVolume3\Windows\System32\config\TxR\{53b39e3d-18c4-11ea-a811-000d3aa4692b}.TxR.2.regtrans-ms
4       System  0x840e96997460  0x106c  WaitCompletionPacket    0x1     -
4       System  0x840e96a50bc0  0x1070  File    0x120116        \Device\Mup
4       System  0x840e9c8592e0  0x1074  Event   0x1f0003        -
4       System  0x840e968ad450  0x1078  File    0x12019f        \Device\HarddiskVolume3\Windows\System32\config\TxR\{53b39e3d-18c4-11ea-a811-000d3aa4692b}.TxR.0.regtrans-ms
4       System  0x840e92cfa140  0x107c  Process 0x40    dwm.exe Pid 1204
4       System  0x840e98d0b8c0  0x1080  IoCompletion    0x1f0003        -
4       System  0x840e968ac480  0x1084  File    0x12019f        \Device\HarddiskVolume3\Windows\System32\config\TxR\{53b39e3d-18c4-11ea-a811-000d3aa4692b}.TxR.blf
4       System  0x840e96c5e1c0  0x1088  Process 0x1fffff        svchost.exe Pid 2220
4       System  0x840e968ab900  0x108c  File    0x12019f        \Device\HarddiskVolume3\Windows\System32\config\TxR\{53b39e3d-18c4-11ea-a811-000d3aa4692b}.TxR.1.regtrans-ms
4       System  0x840e96997c80  0x1090  WaitCompletionPacket    0x1     -
4       System  0x840e96bee960  0x1094  Event   0x1f0003        -
4       System  0x840e969c0040  0x1098  Thread  0x100000        Tid 2768 Pid 4
4       System  0x840e92e04300  0x109c  Process 0x102a  svchost.exe Pid 1408
4       System  0x840e96b0d040  0x10a0  Thread  0x1fffff        Tid 2836 Pid 4
4       System  0x840e96da72c0  0x10a4  Process 0x102a  svchost.exe Pid 3324
4       System  0x840e92e04300  0x10ac  Process 0x102a  svchost.exe Pid 1408
4       System  0x840e9bbe60a0  0x10b0  TmEn    0xf001f -
4       System  0x840e96d02280  0x10b4  IoCompletion    0x1f0003        -
4       System  0x840e96864380  0x10b8  Process 0x102a  svchost.exe Pid 2384
4       System  0xb7080132eaa0  0x10bc  Key     0x3001f MACHINE\SYSTEM\CROWDSTRIKE\{A123C27E-7DF1-4EAF-A2D0-1112802C7C79}\{00000000-0000-0000-0000-00000000001C}
4       System  0x840e96b0e040  0x10c4  Thread  0x1fffff        Tid 2840 Pid 4
4       System  0x840e9676e080  0x10c8  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9e920080  0x10cc  Process 0x1fffff        FileCoAuth.exe Pid 7516
4       System  0x840e9f012990  0x10d0  WaitCompletionPacket    0x1     -
4       System  0x840e9ea14080  0x10d4  Process 0x102a  ScreenConnect. Pid 1772
4       System  0x840e968ab790  0x10d8  File    0x13019f        \Device\clfs\SystemRoot\System32\Config\TxR\{53b39e3d-18c4-11ea-a811-000d3aa4692b}.TxR
4       System  0xb7080132f5e0  0x10dc  Key     0x3001f MACHINE\SYSTEM\CROWDSTRIKE\{A123C27E-7DF1-4EAF-A2D0-1112802C7C79}\{00000000-0000-0000-0000-000000000003}
4       System  0x840e966833f0  0x10e0  FilterConnectionPort    0x1f0001        CS-E2eCommsPort
4       System  0xb70801330c60  0x10e4  Key     0x3001f MACHINE\SYSTEM\CROWDSTRIKE\{A123C27E-7DF1-4EAF-A2D0-1112802C7C79}\{00000000-0000-0000-0000-000000000025}
4       System  0xb70801330f30  0x10e8  Key     0x3001f MACHINE\SYSTEM\CROWDSTRIKE\{A123C27E-7DF1-4EAF-A2D0-1112802C7C79}\{00000000-0000-0000-0000-000000000020}
4       System  0xb7080132fc70  0x10ec  Key     0x3001f MACHINE\SYSTEM\CROWDSTRIKE\{A123C27E-7DF1-4EAF-A2D0-1112802C7C79}\{00000000-0000-0000-0000-000000000021}
4       System  0x840e92cfa140  0x10f0  Process 0x40    dwm.exe Pid 1204
4       System  0xb7080132fd60  0x10f4  Key     0x3001f MACHINE\SYSTEM\CROWDSTRIKE\{A123C27E-7DF1-4EAF-A2D0-1112802C7C79}\{00000000-0000-0000-0000-00000000000B}
4       System  0x840e976a2300  0x10f8  Process 0x102a  unsecapp.exe Pid 5220
4       System  0x840e97fb4c60  0x10fc  Event   0x1f0003        -
4       System  0x840e98192af0  0x1100  WaitCompletionPacket    0x1     -
4       System  0x840e96860080  0x1104  Process 0x102a  svchost.exe Pid 2396
4       System  0x840e9ba1cce0  0x1108  WaitCompletionPacket    0x1     -
4       System  0x840e98f691f0  0x110c  IoCompletionReserve     0xf0003 -
4       System  0x840e99cfec00  0x1110  TmTm    0xf003f -
4       System  0x840e9695c480  0x1114  IoCompletion    0x1f0003        -
4       System  0x840e909843f0  0x1118  WaitCompletionPacket    0x1     -
4       System  0x840e96bcd980  0x111c  Event   0x1f0003        -
4       System  0x840e96c23280  0x1120  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e96bc00c0  0x1124  Process 0x1fffff        svchost.exe Pid 2848
4       System  0x840e91bcc0c0  0x1128  WaitCompletionPacket    0x1     -
4       System  0x840e97103ae0  0x112c  Event   0x1f0003        -
4       System  0x840e9695fe40  0x1130  IoCompletion    0x1f0003        -
4       System  0x840e96bde960  0x1134  Event   0x1f0003        -
4       System  0x840e90987170  0x1138  WaitCompletionPacket    0x1     -
4       System  0x840e96be14e0  0x113c  Event   0x1f0003        -
4       System  0x840e96962ac0  0x1140  IoCompletion    0x1f0003        -
4       System  0x840e91bc3f90  0x1144  WaitCompletionPacket    0x1     -
4       System  0x840e96c23280  0x114c  Process 0x1fffff        spoolsv.exe Pid 2904
4       System  0x840e9666e460  0x1150  WaitCompletionPacket    0x1     -
4       System  0x840e96c23280  0x1154  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e96c93040  0x1158  Thread  0x0     Tid 1484 Pid 4
4       System  0x840e96be4360  0x115c  Event   0x1f0003        -
4       System  0x840e9c92c360  0x1160  Event   0x1f0003        -
4       System  0x840e96e0d080  0x1164  Process 0x102a  svchost.exe Pid 3844
4       System  0x840e969677c0  0x1168  IoCompletion    0x1f0003        -
4       System  0x840e96be5860  0x116c  Event   0x1f0003        -
4       System  0x840e9713ef50  0x1170  WaitCompletionPacket    0x1     -
4       System  0x840e96c582c0  0x1174  Process 0x1fffff        svchost.exe Pid 3028
4       System  0x840e96860080  0x1178  Process 0x102a  svchost.exe Pid 2396
4       System  0x840e968c00c0  0x117c  Process 0x102a  svchost.exe Pid 2496
4       System  0x840e96c582c0  0x1180  Process 0x102a  svchost.exe Pid 3028
4       System  0x840e9b9897a0  0x1184  WaitCompletionPacket    0x1     -
4       System  0x840e96967c40  0x1188  IoCompletion    0x1f0003        -
4       System  0x840e91bcce90  0x118c  WaitCompletionPacket    0x1     -
4       System  0x840e96cdd1c0  0x1190  Process 0x1fffff        svchost.exe Pid 740
4       System  0x840e96c5e1c0  0x1194  Process 0x102a  svchost.exe Pid 2220
4       System  0x840e96860080  0x1198  Process 0x102a  svchost.exe Pid 2396
4       System  0xb70806eaf730  0x119c  Token   0xf01ff -
4       System  0x840e96973400  0x11a0  IoCompletion    0x1f0003        -
4       System  0x840e96c582c0  0x11a4  Process 0x102a  svchost.exe Pid 3028
4       System  0x840e96bea0e0  0x11a8  Event   0x1f0003        -
4       System  0x840e9696e1c0  0x11ac  IoCompletion    0x1f0003        -
4       System  0x840e997a7ce0  0x11b0  Event   0x1f0003        -
4       System  0x840e96c61080  0x11b4  Process 0x102a  svchost.exe Pid 2216
4       System  0x840e96c61080  0x11b8  Process 0x102a  svchost.exe Pid 2216
4       System  0x840e9758cd00  0x11bc  ALPC Port       0x1f0001        -
4       System  0x840e96c61080  0x11c0  Process 0x102a  svchost.exe Pid 2216
4       System  0x840e99c83080  0x11c4  Process 0x0     dwm.exe Pid 10628
4       System  0x840e96bec7e0  0x11c8  Event   0x100003        LanmanServerAnnounceEvent
4       System  0x840e9ac22b00  0x11cc  WaitCompletionPacket    0x1     -
4       System  0x840e96c5e1c0  0x11d0  Process 0x102a  svchost.exe Pid 2220
4       System  0x840e96c5e1c0  0x11d4  Process 0x102a  svchost.exe Pid 2220
4       System  0x840e96bec260  0x11d8  Event   0x1f0003        -
4       System  0x840e96cdd1c0  0x11dc  Process 0x102a  svchost.exe Pid 740
4       System  0x840e96a52150  0x11e0  File    0x120116        \Device\Mup
4       System  0x840e96c61080  0x11e4  Process 0x1fffff        svchost.exe Pid 2216
4       System  0x840e96c5e1c0  0x11e8  Process 0x102a  svchost.exe Pid 2220
4       System  0x840e9b1d56b0  0x11ec  TmEn    0xf001f -
4       System  0x840e96cdd1c0  0x11f4  Process 0x102a  svchost.exe Pid 740
4       System  0x840e96f1d300  0x11fc  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e97f191b0  0x1200  ALPC Port       0x1f0001        -
4       System  0x840e96bee1e0  0x1204  Event   0x1f0003        -
4       System  0x840e96d03140  0x1208  IoCompletion    0x1f0003        -
4       System  0x840e96997120  0x120c  WaitCompletionPacket    0x1     -
4       System  0x840e96cdd1c0  0x1210  Process 0x102a  svchost.exe Pid 740
4       System  0x840e9184e810  0x1214  ALPC Port       0x1f0001        -
4       System  0xb708051680a0  0x1218  Token   0xe     -
4       System  0x840e96bf0e60  0x121c  Event   0x1f0003        -
4       System  0x840e96c5e1c0  0x1220  Process 0x102a  svchost.exe Pid 2220
4       System  0x840e96d04b80  0x1224  IoCompletion    0x1f0003        -
4       System  0x840e969987e0  0x1228  WaitCompletionPacket    0x1     -
4       System  0x840e90ad3080  0x122c  Process 0x102a  svchost.exe Pid 7152
4       System  0x840e90de9080  0x1230  Process 0x102a  FileCoAuth.exe Pid 8592
4       System  0x840e96a53400  0x1234  File    0x12019f        \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0xb7080732a5d0  0x123c  Token   0xe     -
4       System  0x840e96c5e1c0  0x1240  Process 0x102a  svchost.exe Pid 2220
4       System  0xb708077b2060  0x1244  Token   0xe     -
4       System  0x840e9971c360  0x1248  Event   0x1f0003        -
4       System  0x840e9eaee0c0  0x124c  Process 0x0     dllhost.exe Pid 10124
4       System  0x840e96d67580  0x1250  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTWFP-IPsec Diagnostics.etl
4       System  0x840e96cb3ec0  0x1254  File    0x12008b        \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\wfp\wfpdiag.etl
4       System  0x840e97dc72c0  0x1258  Process 0x102a  SearchIndexer. Pid 1852
4       System  0xb708051680a0  0x125c  Token   0xe     -
4       System  0x840e96cfb2d0  0x1260  ALPC Port       0x1f0001        -
4       System  0x840e9b9873e0  0x1264  WaitCompletionPacket    0x1     -
4       System  0x840e9746aa60  0x1268  Event   0x1f0003        -
4       System  0x840e9c93d9e0  0x126c  Event   0x1f0003        -
4       System  0xb708077bd060  0x1270  Token   0xe     -
4       System  0xb708077b1060  0x1274  Token   0xe     -
4       System  0x840e9b73a080  0x1278  Process 0x1fffff        svchost.exe Pid 5116
4       System  0x840e97ab8a80  0x127c  File    0x12019f        \Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat{54a96616-79ef-11ef-ba74-bc2411d898d1}.TMContainer00000000000000000002.regtrans-ms
4       System  0x840e990a09d0  0x1280  WaitCompletionPacket    0x1     -
4       System  0x840e9a954160  0x1284  Event   0x1f0003        -
4       System  0x840e9713df10  0x1288  WaitCompletionPacket    0x1     -
4       System  0xb70807340060  0x128c  Token   0xf01ff -
4       System  0x840e9690b290  0x1290  IoCompletionReserve     0xf0003 -
4       System  0x840e97c52060  0x1294  Event   0x1f0003        -
4       System  0x840e96c5e1c0  0x1298  Process 0x102a  svchost.exe Pid 2220
4       System  0xb7080680b630  0x129c  Token   0xe     -
4       System  0xb7080680b060  0x12a0  Token   0xe     -
4       System  0xb708051680a0  0x12a4  Token   0xe     -
4       System  0xb708077b7060  0x12a8  Token   0xe     -
4       System  0xb708077bb060  0x12ac  Token   0xe     -
4       System  0xb708077c4060  0x12b0  Token   0xe     -
4       System  0xb708077be060  0x12b4  Token   0xe     -
4       System  0xb708077bf060  0x12b8  Token   0xe     -
4       System  0xb708077c9060  0x12bc  Token   0xe     -
4       System  0xb708077c7060  0x12c0  Token   0xe     -
4       System  0xb708077af060  0x12c4  Token   0xe     -
4       System  0xb708077cb0a0  0x12c8  Token   0xe     -
4       System  0xb708077cc060  0x12cc  Token   0xe     -
4       System  0xb708077d1060  0x12d0  Token   0xe     -
4       System  0xb708077c5060  0x12d4  Token   0xe     -
4       System  0xb708077ad4b0  0x12d8  Token   0xe     -
4       System  0xb708077bc060  0x12e0  Token   0xe     -
4       System  0xb7080765c060  0x12e4  Token   0xe     -
4       System  0x840e9699cdc0  0x12e8  WaitCompletionPacket    0x1     -
4       System  0x840e9c853460  0x12ec  Event   0x1f0003        -
4       System  0xb708077d6060  0x12f0  Token   0xe     -
4       System  0xb708077d4060  0x12f4  Token   0xe     -
4       System  0x840e96da72c0  0x12f8  Process 0x102a  svchost.exe Pid 3324
4       System  0x840e96c5e1c0  0x12fc  Process 0x1fffff        svchost.exe Pid 2220
4       System  0x840e969a5bf0  0x1300  WaitCompletionPacket    0x1     -
4       System  0x840e96d9f080  0x1304  Process 0x102a  svchost.exe Pid 3308
4       System  0xb708076957c0  0x1308  Key     0x2001f MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS
4       System  0x840e96c5e1c0  0x130c  Process 0x102a  svchost.exe Pid 2220
4       System  0x840e98df3080  0x1310  Process 0x1fffff        ShellExperienc Pid 704
4       System  0x840e9ba091a0  0x1314  WaitCompletionPacket    0x1     -
4       System  0x840e98aef080  0x1318  Process 0x0     RuntimeBroker. Pid 7684
4       System  0x840e96c5e1c0  0x131c  Process 0x102a  svchost.exe Pid 2220
4       System  0x840e96c5e1c0  0x1320  Process 0x102a  svchost.exe Pid 2220
4       System  0x840e9ff06080  0x1324  IoCompletion    0x1f0003        -
4       System  0xb708076956d0  0x1328  Key     0x2001f MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS\INTERFACES\{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e96e0f080  0x1330  Process 0x0     sihost.exe Pid 6908
4       System  0x840e96e44300  0x1334  Process 0x102a  MsMpEng.exe Pid 3540
4       System  0x840e975ad080  0x1338  Process 0x0     CSFalconContai Pid 1192
4       System  0x840e96dc4080  0x133c  Process 0x1fffff        svchost.exe Pid 3352
4       System  0x840e96c5e1c0  0x1340  Process 0x102a  svchost.exe Pid 2220
4       System  0x840e96c61080  0x1344  Process 0x1028  svchost.exe Pid 2216
4       System  0x840e970d34c0  0x1348  IoCompletion    0x1f0003        -
4       System  0x840e96d9f080  0x134c  Process 0x102a  svchost.exe Pid 3308
4       System  0x840e96d6a380  0x1350  File    0x1f01ff        \Device\Ndis
4       System  0x840e96dc4080  0x1354  Process 0x102a  svchost.exe Pid 3352
4       System  0xb70807a82040  0x1358  Token   0xe     -
4       System  0x840e96e0d080  0x135c  Process 0x0     svchost.exe Pid 3844
4       System  0x840e96d9f080  0x1360  Process 0x1fffff        svchost.exe Pid 3308
4       System  0x840e96d153c0  0x1364  IoCompletion    0x1f0003        -
4       System  0x840e96da72c0  0x1368  Process 0x1fffff        svchost.exe Pid 3324
4       System  0x840e96bfd560  0x136c  Event   0x1f0003        -
4       System  0x840e96da9300  0x1370  Process 0x1fffff        svchost.exe Pid 3332
4       System  0xb70808454770  0x1374  Token   0xe     -
4       System  0x840e969a0360  0x1378  WaitCompletionPacket    0x1     -
4       System  0x840e9783b080  0x137c  Process 0x102a  CSFalconContai Pid 6604
4       System  0x840e96da9300  0x1380  Process 0x102a  svchost.exe Pid 3332
4       System  0x840e96e042c0  0x1384  Process 0x1fffff        svchost.exe Pid 3376
4       System  0x840e96dd14e0  0x1388  Event   0x1f0003        -
4       System  0x840e97159300  0x138c  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e96e350c0  0x1390  Process 0x102a  svchost.exe Pid 3480
4       System  0x840e96d1ff40  0x1394  IoCompletion    0x1f0003        -
4       System  0x840e96d1bf80  0x1398  IoCompletion    0x1f0003        -
4       System  0x840e969a4bb0  0x139c  WaitCompletionPacket    0x1     -
4       System  0x840e96dc4080  0x13a0  Process 0x102a  svchost.exe Pid 3352
4       System  0x840e9c9417e0  0x13a4  Event   0x1f0003        -
4       System  0x840e96dd29e0  0x13a8  Event   0x1f0003        -
4       System  0x840e96d17180  0x13ac  IoCompletion    0x1f0003        -
4       System  0x840e96dd6960  0x13b0  Event   0x1f0003        -
4       System  0x840e96da72c0  0x13b4  Process 0x102a  svchost.exe Pid 3324
4       System  0x840e96d193c0  0x13b8  IoCompletion    0x1f0003        -
4       System  0x840e969a3760  0x13bc  WaitCompletionPacket    0x1     -
4       System  0x840e96dd1fe0  0x13c0  Event   0x1f0003        -
4       System  0x840e969a3830  0x13c4  WaitCompletionPacket    0x1     -
4       System  0x840e96e4e140  0x13c8  Thread  0x1fffff        Tid 3520 Pid 4
4       System  0xb70807b99830  0x13cc  Key     0x20019 MACHINE\SOFTWARE\POLICIES\MICROSOFT\WINDOWS\NETWORKPROVIDER\HARDENEDPATHS
4       System  0x840e96da9300  0x13d0  Process 0x102a  svchost.exe Pid 3332
4       System  0x840e9c99ad70  0x13d4  Event   0x1f0003        DwmComposedEvent_1
4       System  0x840e96620960  0x13d8  FilterCommunicationPort 0x1f0001        -
4       System  0xb70807a7e5d0  0x13dc  Token   0xe     -
4       System  0x840e92305080  0x13e0  Process 0x102a  winlogon.exe Pid 980
4       System  0x840e9969e080  0x13e4  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e91bcc8e0  0x13e8  WaitCompletionPacket    0x1     -
4       System  0xb708109a8340  0x13ec  Directory       0xf000f 00000000-0262ee29
4       System  0x840e97152340  0x13f0  WaitCompletionPacket    0x1     -
4       System  0x840e96e042c0  0x13f4  Process 0x102a  svchost.exe Pid 3376
4       System  0x840e96da4040  0x13f8  Thread  0x1fffff        Tid 3456 Pid 4
4       System  0xb70807a7e040  0x13fc  Token   0xe     -
4       System  0x840e969a64e0  0x1404  WaitCompletionPacket    0x1     -
4       System  0x840e96e042c0  0x1408  Process 0x102a  svchost.exe Pid 3376
4       System  0x840e96dc4080  0x140c  Process 0x102a  svchost.exe Pid 3352
4       System  0x840e96dda8e0  0x1410  Event   0x1f0003        -
4       System  0x840e96dd7ee0  0x1414  Event   0x1f0003        -
4       System  0x840e96e44300  0x1418  Process 0x102a  MsMpEng.exe Pid 3540
4       System  0x840e96e3d080  0x141c  Process 0x102a  svchost.exe Pid 3528
4       System  0xb70807a825d0  0x1420  Token   0xe     -
4       System  0x840e96e552c0  0x1424  Process 0x1fffff        svchost.exe Pid 3560
4       System  0x840e96e3d080  0x1428  Process 0x102a  svchost.exe Pid 3528
4       System  0x840e96e44300  0x142c  Process 0x1fffff        MsMpEng.exe Pid 3540
4       System  0x840e972491c0  0x1430  Process 0x102a  svchost.exe Pid 4352
4       System  0x840e96e350c0  0x1434  Process 0x102a  svchost.exe Pid 3480
4       System  0x840e96e20080  0x1438  Process 0x1fffff        svchost.exe Pid 3488
4       System  0xb708061bc1c0  0x143c  Key     0x2001f MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS\INTERFACES\{27021307-ADC3-4967-B1BB-09A0A87E4F50}
4       System  0x840e96f1d300  0x1440  Process 0x1fffff        dasHost.exe Pid 3752
4       System  0x840e96e350c0  0x1444  Process 0x1fffff        svchost.exe Pid 3480
4       System  0xb708081670a0  0x1448  Token   0xf01ff -
4       System  0xb708061bbd80  0x144c  Key     0x2001f MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS\INTERFACES\{E0EC89D8-2A3A-11EB-BA6E-806E6F6E6963}
4       System  0xb708061bf5d0  0x1450  Key     0x2001f MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS\PERSISTENTROUTES
4       System  0x840e96e3d080  0x1454  Process 0x1fffff        svchost.exe Pid 3528
4       System  0xb70807a7d5d0  0x1458  Token   0xe     -
4       System  0x840e98fb61c0  0x145c  Process 0x0     dllhost.exe Pid 8472
4       System  0x840e96e77080  0x1460  Process 0x102a  svchost.exe Pid 3636
4       System  0xb708079cdb10  0x1464  SymbolicLink    0xf0001 SWD#MSRRAS#MS_SSTPMINIPORT#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0x840e96d1d540  0x1468  IoCompletion    0x1f0003        -
4       System  0x840e96d20580  0x146c  IoCompletion    0x1f0003        -
4       System  0x840e96dd26e0  0x1470  Event   0x1f0003        -
4       System  0x840e969a60d0  0x1474  WaitCompletionPacket    0x1     -
4       System  0x840e96e20080  0x1478  Process 0x102a  svchost.exe Pid 3488
4       System  0x840e96e20080  0x147c  Process 0x102a  svchost.exe Pid 3488
4       System  0x840e96dda960  0x1480  Event   0x1f0003        -
4       System  0x840e96e552c0  0x1484  Process 0x102a  svchost.exe Pid 3560
4       System  0x840e96e3d080  0x1488  Process 0x102a  svchost.exe Pid 3528
4       System  0x840e96ebc080  0x148c  IoCompletion    0x1f0003        -
4       System  0x840e969a76c0  0x1490  WaitCompletionPacket    0x1     -
4       System  0x840e96ebbd00  0x1494  IoCompletion    0x1f0003        -
4       System  0x840e90c639e0  0x1498  WaitCompletionPacket    0x1     -
4       System  0x840e96ec6500  0x149c  IoCompletion    0x1f0003        -
4       System  0x840e96dc4080  0x14a0  Process 0x1028  svchost.exe Pid 3352
4       System  0x840e96e552c0  0x14a4  Process 0x102a  svchost.exe Pid 3560
4       System  0xb708079c1990  0x14a8  SymbolicLink    0xf0001 SWD#MSRRAS#MS_L2TPMINIPORT#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0x840e9680ed60  0x14ac  WaitCompletionPacket    0x1     -
4       System  0x840e96e77080  0x14b0  Process 0x102a  svchost.exe Pid 3636
4       System  0x840e96e0d080  0x14b4  Process 0x1fffff        svchost.exe Pid 3844
4       System  0x840e96ddb460  0x14b8  Event   0x1f0003        -
4       System  0x840e96ec0680  0x14bc  IoCompletion    0x1f0003        -
4       System  0x840e96805b20  0x14c0  WaitCompletionPacket    0x1     -
4       System  0xb708079cd600  0x14c4  SymbolicLink    0xf0001 SWD#MSRRAS#MS_SSTPMINIPORT#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0x840e96e77080  0x14c8  Process 0x1fffff        svchost.exe Pid 3636
4       System  0x840e9765d8c0  0x14cc  IoCompletion    0x1f0003        -
4       System  0x840e975ad080  0x14d0  Process 0x102a  CSFalconContai Pid 1192
4       System  0x840e96e350c0  0x14d4  Process 0x102a  svchost.exe Pid 3480
4       System  0x840ea9ed3860  0x14d8  Event   0x1f0003        -
4       System  0xb7081120ad40  0x14dc  SymbolicLink    0xf0001 Session
4       System  0x840e96dde960  0x14e0  Event   0x1f0003        -
4       System  0x840e96f73ed0  0x14e4  File    0x12008b        \Device\HarddiskVolume3\ProgramData\Microsoft\Windows Defender\Support\MpWppTracing-20250312-104004-00000003-ffffffff.bin
4       System  0x840e97d77f80  0x14e8  IoCompletion    0x1f0003        -
4       System  0x840e96e042c0  0x14ec  Process 0x102a  svchost.exe Pid 3376
4       System  0x840e96e44300  0x14f0  Process 0x102a  MsMpEng.exe Pid 3540
4       System  0x840e96f1d300  0x14f4  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e97007040  0x14f8  Thread  0x0     Tid 4040 Pid 4
4       System  0xb708069c7960  0x14fc  SymbolicLink    0xf0001 SWD#MSRRAS#MS_AGILEVPNMINIPORT#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0x840e9ba10290  0x1500  WaitCompletionPacket    0x1     -
4       System  0x840e96e0d080  0x1504  Process 0x102a  svchost.exe Pid 3844
4       System  0xb708078814c0  0x1508  Token   0xf01ff -
4       System  0x840e96de0260  0x150c  Event   0x1f0003        -
4       System  0x840e96de9fe0  0x1510  Event   0x1f0003        -
4       System  0x840e971517e0  0x1514  WaitCompletionPacket    0x1     -
4       System  0x840e970f3080  0x1518  Process 0x102a  svchost.exe Pid 3172
4       System  0x840e96eca180  0x151c  IoCompletion    0x1f0003        -
4       System  0x840e979897d0  0x1520  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\MicrosoftWindows.Client.CBS_1000.19058.1000.0_x64__cw5n1h2txyewy\ActivationStore.dat.LOG1
4       System  0x840e99bcb080  0x1524  Process 0x40    explorer.exe Pid 9852
4       System  0x840e96816810  0x1528  WaitCompletionPacket    0x1     -
4       System  0x840e96f1d300  0x152c  Process 0x102a  dasHost.exe Pid 3752
4       System  0xb708079d0300  0x1530  SymbolicLink    0xf0001 SWD#MSRRAS#MS_NDISWANIP#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0x840e96fdd040  0x1534  Thread  0x1fffff        Tid 3992 Pid 4
4       System  0x840e9676e080  0x1538  Process 0x102a  svchost.exe Pid 2168
4       System  0xb70807b5d5d0  0x153c  Token   0xe     -
4       System  0xb708079cffa0  0x1540  SymbolicLink    0xf0001 SWD#MSRRAS#MS_NDISWANIP#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0x840e96e0d080  0x1544  Process 0x102a  svchost.exe Pid 3844
4       System  0x840e96fde040  0x1548  Thread  0x1fffff        Tid 3996 Pid 4
4       System  0xb708080187d0  0x154c  SymbolicLink    0xf0001 SWD#MSRRAS#MS_NDISWANIPV6#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0x840e96f6d2c0  0x1550  Process 0x102a  svchost.exe Pid 3880
4       System  0x840e96f1d300  0x1554  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e96f6d2c0  0x1558  Process 0x1fffff        svchost.exe Pid 3880
4       System  0x840e9ca64940  0x155c  IoCompletion    0x1f0003        -
4       System  0x840e96f6d2c0  0x1560  Process 0x102a  svchost.exe Pid 3880
4       System  0x840e96de8260  0x1564  Event   0x1f0003        -
4       System  0x840e96ed1340  0x1568  IoCompletion    0x1f0003        -
4       System  0x840e96807380  0x156c  WaitCompletionPacket    0x1     -
4       System  0x840e96de86e0  0x1570  Event   0x1f0003        -
4       System  0x840e96ed1080  0x1574  IoCompletion    0x1f0003        -
4       System  0x840e9680b210  0x1578  WaitCompletionPacket    0x1     -
4       System  0x840e96ed3c40  0x157c  IoCompletion    0x1f0003        -
4       System  0x840e91bc2320  0x1580  WaitCompletionPacket    0x1     -
4       System  0x840e96fdc040  0x1584  Thread  0x1fffff        Tid 3988 Pid 4
4       System  0x840e970b8ac0  0x1588  IoCompletion    0x1f0003        -
4       System  0x840e96fdb040  0x158c  Thread  0x1fffff        Tid 3984 Pid 4
4       System  0x840e96fdf040  0x1590  Thread  0x1fffff        Tid 4000 Pid 4
4       System  0x840e96fe0040  0x1594  Thread  0x1fffff        Tid 4004 Pid 4
4       System  0x840e96fe1040  0x1598  Thread  0x1fffff        Tid 4008 Pid 4
4       System  0x840e96fe2040  0x159c  Thread  0x1fffff        Tid 4012 Pid 4
4       System  0x840e96fe3040  0x15a0  Thread  0x1fffff        Tid 4016 Pid 4
4       System  0x840e96fe4040  0x15a4  Thread  0x1fffff        Tid 4020 Pid 4
4       System  0x840e96fe5040  0x15a8  Thread  0x1fffff        Tid 4024 Pid 4
4       System  0x840e96fe6040  0x15ac  Thread  0x1fffff        Tid 4028 Pid 4
4       System  0x840eab2f08e0  0x15b0  Event   0x1f0003        -
4       System  0x840e9709ba00  0x15b4  Timer   0x2     -
4       System  0xb708069c9580  0x15b8  SymbolicLink    0xf0001 SWD#MSRRAS#MS_AGILEVPNMINIPORT#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0xb708079c1900  0x15bc  SymbolicLink    0xf0001 SWD#MSRRAS#MS_PPPOEMINIPORT#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0xb708079c1d80  0x15c0  SymbolicLink    0xf0001 SWD#MSRRAS#MS_L2TPMINIPORT#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0x840e971062e0  0x15c4  Event   0x1f0003        -
4       System  0x840e970bd600  0x15c8  IoCompletion    0x1f0003        -
4       System  0xb70808018500  0x15cc  SymbolicLink    0xf0001 SWD#MSRRAS#MS_NDISWANIPV6#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0xb708079c23b0  0x15d0  SymbolicLink    0xf0001 SWD#MSRRAS#MS_PPTPMINIPORT#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0xb708079c1630  0x15d4  SymbolicLink    0xf0001 SWD#MSRRAS#MS_PPPOEMINIPORT#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0xb708079c28c0  0x15d8  SymbolicLink    0xf0001 SWD#MSRRAS#MS_PPTPMINIPORT#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0x840e9715d080  0x15dc  Process 0x102a  Sysmon64.exe Pid 3696
4       System  0x840e97159300  0x15e0  Process 0x1fffff        CSFalconServic Pid 3648
4       System  0x840e92def080  0x15e4  Process 0x102a  svchost.exe Pid 1292
4       System  0xb708079c5b60  0x15e8  SymbolicLink    0xf0001 SWD#MSRRAS#MS_NDISWANBH#{ad498944-762f-11d0-8dcb-00c04fc3358c}
4       System  0xb708079c6bb0  0x15ec  SymbolicLink    0xf0001 SWD#MSRRAS#MS_NDISWANBH#{cac88484-7515-4c03-82e6-71a87abac361}
4       System  0x840e97117de0  0x15f0  Event   0x1f0003        -
4       System  0x840e970f3080  0x15f8  Process 0x102a  svchost.exe Pid 3172
4       System  0x840ea3cf5060  0x15fc  Event   0x1f0003        -
4       System  0x840e98ec91c0  0x1600  Process 0x102a  RuntimeBroker. Pid 10968
4       System  0x840e99f5b500  0x1604  IoCompletion    0x1f0003        -
4       System  0x840e97135830  0x1608  WaitCompletionPacket    0x1     -
4       System  0x840e96e0d080  0x160c  Process 0x102a  svchost.exe Pid 3844
4       System  0x840e96dc4080  0x1610  Process 0x102a  svchost.exe Pid 3352
4       System  0x840e971154e0  0x1614  Event   0x1f0003        -
4       System  0x840e971697d0  0x1618  File    0x12019f        \Device\NamedPipe\
4       System  0x840e92ee5450  0x161c  WaitCompletionPacket    0x1     -
4       System  0x840e92def080  0x1624  Process 0x102a  svchost.exe Pid 1292
4       System  0x840e970f3080  0x1628  Process 0x102a  svchost.exe Pid 3172
4       System  0x840e970f3080  0x162c  Process 0x1fffff        svchost.exe Pid 3172
4       System  0x840e97159300  0x1630  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e9676e080  0x1634  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e91ba00c0  0x1638  Process 0x102a  svchost.exe Pid 7220
4       System  0x840e9715d080  0x163c  Process 0x1fffff        Sysmon64.exe Pid 3696
4       System  0x840e97358b80  0x1640  IoCompletion    0x1f0003        -
4       System  0x840e9b9850f0  0x1644  WaitCompletionPacket    0x1     -
4       System  0x840e984fccb0  0x1648  Timer   0x2     -
4       System  0x840e9a14e270  0x164c  File    0x3     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-5-21-3294271487-4292050549-1661813568-1001\SystemAppData\Helium\Cache\b4d622c333fcfdd7_COM15.dat.LOG2
4       System  0x840e9715d080  0x1650  Process 0x102a  Sysmon64.exe Pid 3696
4       System  0x840e970f3080  0x1654  Process 0x102a  svchost.exe Pid 3172
4       System  0x840e97442080  0x1658  Process 0x102a  CSFalconContai Pid 6624
4       System  0x840e97159300  0x165c  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e970c3380  0x1660  IoCompletion    0x1f0003        -
4       System  0x840e9713a7d0  0x1664  WaitCompletionPacket    0x1     -
4       System  0x840e97159300  0x1668  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e971c9ce0  0x166c  ALPC Port       0x1f0001        -
4       System  0x840e96e44300  0x1670  Process 0x102a  MsMpEng.exe Pid 3540
4       System  0x840e97159300  0x1674  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e971b2a80  0x1678  ALPC Port       0x1f0001        -
4       System  0x840e9710b960  0x167c  Event   0x1f0003        -
4       System  0x840e971faa80  0x1680  ALPC Port       0x1f0001        -
4       System  0x840e96e44300  0x1684  Process 0x102a  MsMpEng.exe Pid 3540
4       System  0x840e99bcb080  0x1688  Process 0x102a  explorer.exe Pid 9852
4       System  0x840e9ac2c560  0x168c  WaitCompletionPacket    0x1     -
4       System  0x840e97159300  0x1690  Process 0x102a  CSFalconServic Pid 3648
4       System  0x840e9c709f60  0x1694  Event   0x1f0003        -
4       System  0x840e970ce600  0x1698  IoCompletion    0x1f0003        -
4       System  0x840e971125e0  0x169c  Event   0x1f0003        -
4       System  0x840e9818fe40  0x16a4  WaitCompletionPacket    0x1     -
4       System  0x840e99f51c00  0x16a8  IoCompletion    0x1f0003        -
4       System  0x840e9f3a3c00  0x16ac  IoCompletion    0x1f0003        -
4       System  0x840e972491c0  0x16b0  Process 0x1fffff        svchost.exe Pid 4352
4       System  0x840e96e44300  0x16b4  Process 0x102a  MsMpEng.exe Pid 3540
4       System  0x840e96da72c0  0x16b8  Process 0x102a  svchost.exe Pid 3324
4       System  0x840e973d81c0  0x16bc  Process 0x102a  svchost.exe Pid 4800
4       System  0x840e96621060  0x16c0  FilterCommunicationPort 0x1f0001        -
4       System  0xb70804825040  0x16c4  Token   0xe     -
4       System  0x840e91ba00c0  0x16c8  Process 0x102a  svchost.exe Pid 7220
4       System  0x840e9f02bbc0  0x16cc  WaitCompletionPacket    0x1     -
4       System  0x840e98bba080  0x16d4  Process 0x40    SearchApp.exe Pid 7932
4       System  0x840e97ab29a0  0x16d8  File    0x3     \Device\HarddiskVolume3\Users\admin\ntuser.dat.LOG2
4       System  0x840e970d5540  0x16dc  IoCompletion    0x1f0003        -
4       System  0x840e97141b30  0x16e0  WaitCompletionPacket    0x1     -
4       System  0x840e972491c0  0x16e4  Process 0x102a  svchost.exe Pid 4352
4       System  0x840e9ba0c670  0x16e8  WaitCompletionPacket    0x1     -
4       System  0xb70808920040  0x16ec  Token   0xe     -
4       System  0x840e97115160  0x16f0  Event   0x1f0003        -
4       System  0xb70805679040  0x16f4  Token   0xe     -
4       System  0xb7080893f490  0x16f8  Token   0xe     -
4       System  0x840e91d954e0  0x16fc  Event   0x1f0003        -
4       System  0x840e96d0c540  0x1708  IoCompletion    0x1f0003        -
4       System  0x840e98aef080  0x170c  Process 0x102a  RuntimeBroker. Pid 7684
4       System  0x840e97143bb0  0x1710  WaitCompletionPacket    0x1     -
4       System  0x840e97143ef0  0x1714  WaitCompletionPacket    0x1     -
4       System  0x840e97118a60  0x1718  Event   0x1f0003        -
4       System  0x840e978e8570  0x171c  WaitCompletionPacket    0x1     -
4       System  0x840e9eaee0c0  0x1720  Process 0x102a  dllhost.exe Pid 10124
4       System  0x840e9a62f6c0  0x1724  IoCompletion    0x1f0003        -
4       System  0xb70805679040  0x1728  Token   0xe     -
4       System  0x840e972491c0  0x172c  Process 0x102a  svchost.exe Pid 4352
4       System  0x840e96a515d0  0x1730  File    0x12019f        \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e96c582c0  0x1734  Process 0x102a  svchost.exe Pid 3028
4       System  0x840e97357580  0x1738  IoCompletion    0x1f0003        -
4       System  0x840e973d81c0  0x173c  Process 0x1fffff        svchost.exe Pid 4800
4       System  0x840e97116760  0x1740  Event   0x1f0003        -
4       System  0x840e972491c0  0x1748  Process 0x102a  svchost.exe Pid 4352
4       System  0x840e97142eb0  0x174c  WaitCompletionPacket    0x1     -
4       System  0x840e96f1d300  0x1750  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e91c2a080  0x1754  Process 0x0     svchost.exe Pid 5340
4       System  0x840e971431f0  0x1758  WaitCompletionPacket    0x1     -
4       System  0x840e9735bd40  0x175c  IoCompletion    0x1f0003        -
4       System  0x840e9b372080  0x1760  Process 0x102a  taskhostw.exe Pid 10476
4       System  0x840e972491c0  0x1764  Process 0x102a  svchost.exe Pid 4352
4       System  0x840e98e9e8e0  0x1768  Event   0x1f0003        -
4       System  0x840e96f1d300  0x176c  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e974b4080  0x1770  Process 0x102a  CSFalconContai Pid 324
4       System  0x840e97d56640  0x1774  IoCompletion    0x1f0003        -
4       System  0x840e972491c0  0x1778  Process 0x102a  svchost.exe Pid 4352
4       System  0x840e972491c0  0x177c  Process 0x102a  svchost.exe Pid 4352
4       System  0x840e998450c0  0x1780  Process 0x102a  SearchApp.exe Pid 4236
4       System  0x840e9676e080  0x1784  Process 0x102a  svchost.exe Pid 2168
4       System  0xb708087d37d0  0x1788  Token   0xf01ff -
4       System  0x840e9b98d2f0  0x178c  WaitCompletionPacket    0x1     -
4       System  0x840e972491c0  0x1790  Process 0x102a  svchost.exe Pid 4352
4       System  0xb708088b37b0  0x1794  Token   0xe     -
4       System  0x840e973d81c0  0x1798  Process 0x102a  svchost.exe Pid 4800
4       System  0x840e96f1d300  0x179c  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840ea8d6a080  0x17a0  Process 0x102a  OpenHandleColl Pid 11408
4       System  0x840e98e92f60  0x17a4  Event   0x1f0003        -
4       System  0xb7080893fa20  0x17a8  Token   0xe     -
4       System  0x840e978c0300  0x17b0  Process 0x102a  explorer.exe Pid 5328
4       System  0xb708089308e0  0x17b4  Token   0xe     -
4       System  0x840e99ab94c0  0x17b8  IoCompletion    0x1f0003        -
4       System  0x840e96a51b90  0x17bc  File    0x12019f        \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e96f1d300  0x17c0  Process 0x102a  dasHost.exe Pid 3752
4       System  0xb7081792d630  0x17c4  Section 0x17    -
4       System  0x840e9c441080  0x17c8  Process 0x102a  SearchApp.exe Pid 9092
4       System  0x840e9f392f40  0x17cc  IoCompletion    0x1f0003        -
4       System  0x840e90ad3080  0x17d0  Process 0x102a  svchost.exe Pid 7152
4       System  0x840e9a946de0  0x17d4  Event   0x1f0003        -
4       System  0x840e976a2300  0x17d8  Process 0x102a  unsecapp.exe Pid 5220
4       System  0x840e9a350980  0x17dc  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\windows.immersivecontrolpanel_10.0.2.1000_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG2
4       System  0x840e9c375240  0x17e0  File    0x100001        \Device\00000021
4       System  0x840e9a954be0  0x17e8  Event   0x1f0003        -
4       System  0x840e96c23280  0x17ec  Process 0x0     spoolsv.exe Pid 2904
4       System  0x840e97f80080  0x17f0  Process 0x102a  AggregatorHost Pid 5804
4       System  0x840ea08f3080  0x17f4  Process 0x102a  svchost.exe Pid 12008
4       System  0x840e973d81c0  0x17f8  Process 0x102a  svchost.exe Pid 4800
4       System  0x840e9ba1f990  0x17fc  WaitCompletionPacket    0x1     -
4       System  0x840e9b98ea80  0x1804  WaitCompletionPacket    0x1     -
4       System  0x840e97fafa60  0x1808  Event   0x1f0003        -
4       System  0x840e99078310  0x180c  WaitCompletionPacket    0x1     -
4       System  0x840e97670180  0x1810  IoCompletion    0x1f0003        -
4       System  0x840e97add1c0  0x1814  Process 0x1fffff        svchost.exe Pid 6020
4       System  0x840e96da72c0  0x1818  Process 0x1028  svchost.exe Pid 3324
4       System  0x840e9783b080  0x181c  Process 0x1fffff        CSFalconContai Pid 6604
4       System  0x840e996cf080  0x1820  Process 0x102a  svchost.exe Pid 9320
4       System  0xb70807cbf4b0  0x1824  Token   0xf01ff -
4       System  0xb70805679040  0x182c  Token   0xe     -
4       System  0x840e992c10c0  0x1834  IoCompletion    0x1f0003        -
4       System  0x840e99aba540  0x1838  IoCompletion    0x1f0003        -
4       System  0x840e97442080  0x183c  Process 0x1fffff        CSFalconContai Pid 6624
4       System  0x840e92df89c0  0x1840  IoCompletion    0x1f0003        -
4       System  0x840e9ac0f710  0x1844  WaitCompletionPacket    0x1     -
4       System  0x840e97159300  0x1848  Process 0x1028  CSFalconServic Pid 3648
4       System  0x840e92ee5ad0  0x184c  WaitCompletionPacket    0x1     -
4       System  0x840e96f1d300  0x1850  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e96f1d300  0x1854  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e9b997f30  0x1858  WaitCompletionPacket    0x1     -
4       System  0x840e9818d4d0  0x185c  WaitCompletionPacket    0x1     -
4       System  0x840e91db7340  0x1860  Process 0x0     taskhostw.exe Pid 4428
4       System  0x840e9cf21180  0x1864  IoCompletion    0x1f0003        -
4       System  0x840e9907f670  0x1868  WaitCompletionPacket    0x1     -
4       System  0x840e9c121080  0x186c  Process 0x1fffff        RuntimeBroker. Pid 10764
4       System  0x840e97456060  0x1870  Event   0x1f0003        -
4       System  0x840e97ab90c0  0x1874  File    0x12019f        \Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat{54a96616-79ef-11ef-ba74-bc2411d898d1}.TM.blf
4       System  0x840e98193240  0x1878  WaitCompletionPacket    0x1     -
4       System  0x840e98d05440  0x187c  IoCompletion    0x1f0003        -
4       System  0x840e97442080  0x1880  Process 0x102a  CSFalconContai Pid 6624
4       System  0x840e9769f080  0x1884  Process 0x102a  winlogon.exe Pid 2804
4       System  0x840e97362100  0x1888  IoCompletion    0x1f0003        -
4       System  0x840e98b5b170  0x188c  WaitCompletionPacket    0x1     -
4       System  0x840e96c23280  0x1890  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e90bbdfe0  0x1894  Event   0x1f0003        -
4       System  0x840e97647f40  0x1898  IoCompletion    0x1f0003        -
4       System  0x840e9c895fe0  0x18a4  Event   0x1f0003        -
4       System  0x840e98a3a240  0x18a8  IoCompletion    0x1f0003        -
4       System  0x840e97c4fa60  0x18ac  Event   0x1f0003        -
4       System  0x840e97108660  0x18b0  Event   0x1f0003        -
4       System  0x840e97b5c750  0x18b4  File    0x12019f        \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e91bcc670  0x18b8  WaitCompletionPacket    0x1     -
4       System  0x840e96f1d300  0x18bc  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e975ad080  0x18c0  Process 0x102a  CSFalconContai Pid 1192
4       System  0x840e96f1d300  0x18c8  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e96f1d300  0x18d0  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e99f508c0  0x18d4  IoCompletion    0x1f0003        -
4       System  0x840e9ac23730  0x18d8  WaitCompletionPacket    0x1     -
4       System  0x840e9ac34c40  0x18dc  WaitCompletionPacket    0x1     -
4       System  0x840e974b4080  0x18e0  Process 0x1fffff        CSFalconContai Pid 324
4       System  0x840e9979e6e0  0x18e4  Event   0x1f0003        -
4       System  0x840e98b374a0  0x18e8  WaitCompletionPacket    0x1     -
4       System  0x840e980e3c80  0x18ec  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTcs-etw-stp.etl
4       System  0x840e99c6e080  0x18f0  Process 0x1fffff        svchost.exe Pid 4636
4       System  0x840e98dbf2c0  0x18f8  Process 0x1fffff        svchost.exe Pid 8688
4       System  0x840e999e30e0  0x18fc  Event   0x1f0003        -
4       System  0x840e990373c0  0x1900  IoCompletion    0x1f0003        -
4       System  0x840e99bcb080  0x1908  Process 0x1fffff        explorer.exe Pid 9852
4       System  0x840e97d5cb00  0x190c  IoCompletion    0x1f0003        -
4       System  0x840e9b993950  0x1914  WaitCompletionPacket    0x1     -
4       System  0x840e9b280910  0x1920  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.ShellExperienceHost_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e96c5a080  0x1924  Process 0x102a  svchost.exe Pid 7108
4       System  0x840e97677940  0x1928  IoCompletion    0x1f0003        -
4       System  0x840e9c903fe0  0x192c  Event   0x1f0003        -
4       System  0xb70804826040  0x1930  Token   0xe     -
4       System  0x840e97655740  0x1934  IoCompletion    0x1f0003        -
4       System  0x840e98c1c660  0x1938  Event   0x1f0003        -
4       System  0xb7081e55c8a0  0x193c  Directory       0xf     RPC Control
4       System  0x840e91c2a080  0x1940  Process 0x102a  svchost.exe Pid 5340
4       System  0x840e96d9f080  0x1948  Process 0x102a  svchost.exe Pid 3308
4       System  0x840e90b0d080  0x194c  Process 0x0     svchost.exe Pid 9452
4       System  0x840e96f1d300  0x1950  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e968087d0  0x1954  WaitCompletionPacket    0x1     -
4       System  0x840e90ad3080  0x1958  Process 0x102a  svchost.exe Pid 7152
4       System  0x840e98e742c0  0x195c  Process 0x102a  svchost.exe Pid 8292
4       System  0x840e975c90a0  0x1960  File    0x120089        \Device\HarddiskVolume3\Windows\CSC
4       System  0x840e990382c0  0x1964  IoCompletion    0x1f0003        -
4       System  0xb708106b2c00  0x1968  Directory       0xf000f 00000000-0258d92d
4       System  0x840e9ca96a60  0x196c  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-536077884-713174666-1066051701-3219990555-339840825-1966734348-1611281757
4       System  0x840e96f1d300  0x1970  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e98a3f580  0x1974  IoCompletion    0x1f0003        -
4       System  0x840e9c8be7f0  0x1978  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee29-852e-11ef-ba7c-bc2411845bc2}.TxR.2.regtrans-ms
4       System  0x840e992afc40  0x197c  IoCompletion    0x1f0003        -
4       System  0xb70812e2d060  0x1980  Token   0xf01ff -
4       System  0x840e99ac7f40  0x1984  IoCompletion    0x1f0003        -
4       System  0x840e998450c0  0x1988  Process 0x1fffff        SearchApp.exe Pid 4236
4       System  0x840e91e7c5c0  0x198c  ALPC Port       0x1f0001        -
4       System  0x840e97c4ed60  0x1990  Event   0x1f0003        -
4       System  0xb708109e1c40  0x1994  SymbolicLink    0xf0001 SWD#PRINTENUM#{5921CECD-FF14-4BE5-8254-F33210B5C52D}#{0ecef634-6ef0-472a-8085-5ad023ecbccd}
4       System  0x840e9dacf2c0  0x1998  IoCompletion    0x1f0003        -
4       System  0x840e99a92080  0x199c  Process 0x1fffff        SecurityHealth Pid 12036
4       System  0x840e9bdf4340  0x19a0  Process 0x102a  ScreenConnect. Pid 14756
4       System  0x840e9229bc60  0x19a4  Event   0x1f0003        -
4       System  0x840e990a2230  0x19a8  WaitCompletionPacket    0x1     -
4       System  0x840e97180380  0x19ac  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTSYSMON TRACE.etl
4       System  0x840e9daeba80  0x19b0  IoCompletion    0x1f0003        -
4       System  0x840e92edd180  0x19b4  WaitCompletionPacket    0x1     -
4       System  0x840e98d06080  0x19b8  IoCompletion    0x1f0003        -
4       System  0x840e99096dd0  0x19bc  WaitCompletionPacket    0x1     -
4       System  0x840e9715d080  0x19c0  Process 0x1028  Sysmon64.exe Pid 3696
4       System  0x840e97182770  0x19c4  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTSysmonDnsEtwSession.etl
4       System  0x840e974b7080  0x19c8  Process 0x1fffff        CSFalconContai Pid 5980
4       System  0x840e99094d50  0x19cc  WaitCompletionPacket    0x1     -
4       System  0x840e9daeca40  0x19d0  IoCompletion    0x1f0003        -
4       System  0x840e993526e0  0x19d4  Event   0x1f0003        -
4       System  0x840e975ad080  0x19dc  Process 0x1fffff        CSFalconContai Pid 1192
4       System  0x840e974b7080  0x19e8  Process 0x102a  CSFalconContai Pid 5980
4       System  0x840e9715d080  0x19ec  Process 0x1028  Sysmon64.exe Pid 3696
4       System  0x840e99f79900  0x19f0  IoCompletion    0x1f0003        -
4       System  0x840e96c5a080  0x19f4  Process 0x102a  svchost.exe Pid 7108
4       System  0x840e98a5a1f0  0x19f8  IoCompletionReserve     0xf0003 -
4       System  0x840ea0ec9360  0x19fc  Event   0x1f0003        -
4       System  0x840e9817fa40  0x1a00  WaitCompletionPacket    0x1     -
4       System  0x840e9715d080  0x1a04  Process 0x1fffff        Sysmon64.exe Pid 3696
4       System  0x840e990ad690  0x1a08  WaitCompletionPacket    0x1     -
4       System  0x840e9a351920  0x1a0c  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\windows.immersivecontrolpanel_10.0.2.1000_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat
4       System  0x840e9715d080  0x1a10  Process 0x102a  Sysmon64.exe Pid 3696
4       System  0x840e9715d080  0x1a14  Process 0x102a  Sysmon64.exe Pid 3696
4       System  0x840e96bf6b60  0x1a18  Event   0x1f0003        -
4       System  0xb707fbcb4190  0x1a1c  SymbolicLink    0xf0001 Global
4       System  0x840e9b9b2dd0  0x1a20  WaitCompletionPacket    0x1     -
4       System  0x840e9ac0f230  0x1a24  WaitCompletionPacket    0x1     -
4       System  0x840e9a6280c0  0x1a28  IoCompletion    0x1f0003        -
4       System  0x840e9ee430c0  0x1a2c  Process 0x102a  powershell.exe Pid 13156
4       System  0x840ea47c0990  0x1a30  WaitCompletionPacket    0x1     -
4       System  0x840e97f80080  0x1a38  Process 0x102a  AggregatorHost Pid 5804
4       System  0x840e99bcb080  0x1a3c  Process 0x102a  explorer.exe Pid 9852
4       System  0x840e97d74b00  0x1a40  IoCompletion    0x1f0003        -
4       System  0xb708095b5d80  0x1a44  Directory       0xf000f S-1-15-2-233136611-892874861-1880231551-514580003-522390519-4292070654-795176919
4       System  0x840e999e5b60  0x1a48  Event   0x1f0003        -
4       System  0x840e97121f60  0x1a4c  Event   0x1f0003        -
4       System  0x840e97c36760  0x1a50  File    0x3     \Device\HarddiskVolume3\Windows\appcompat\Programs\Amcache.hve.LOG2
4       System  0x840e97949390  0x1a54  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e980e50a0  0x1a58  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTcs-etw.etl
4       System  0x840e9970dde0  0x1a6c  Event   0x1f0003        -
4       System  0x840e98fe0be0  0x1a70  Event   0x1f0003        -
4       System  0x840e9971d660  0x1a78  Event   0x1f0003        -
4       System  0x840e983e8380  0x1a7c  TmRm    0x1f007f        -
4       System  0x840e90bb8a60  0x1a80  Event   0x1f0003        -
4       System  0x840e992a7e40  0x1a84  IoCompletion    0x1f0003        -
4       System  0x840e97ab3ad0  0x1a8c  File    0x12019f        \Device\HarddiskVolume3\Users\admin\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000002.regtrans-ms
4       System  0x840e9b372080  0x1a90  Process 0x102a  taskhostw.exe Pid 10476
4       System  0x840e9ca3ecc0  0x1a94  IoCompletion    0x1f0003        -
4       System  0x840e9a95c8e0  0x1a98  Event   0x1f0003        -
4       System  0x840e9c250080  0x1a9c  Process 0x102a  SenseIR.exe Pid 7728
4       System  0x840e96d56340  0x1aa0  Process 0x0     SenseTVM.exe Pid 1552
4       System  0x840e96d09280  0x1aa4  IoCompletion    0x1f0003        -
4       System  0xb70810d63be0  0x1aa8  Key     0xf003f MACHINE\SYSTEM\DRIVERDATABASE\DRIVERPACKAGES
4       System  0x840e91d8d7e0  0x1ab0  Event   0x1f0003        -
4       System  0x840e9ac2e850  0x1ab4  WaitCompletionPacket    0x1     -
4       System  0x840e997210e0  0x1ab8  Event   0x1f0003        -
4       System  0x840e90cf27e0  0x1abc  Event   0x1f0003        -
4       System  0x840e90bb4b60  0x1ac0  Event   0x1f0003        -
4       System  0x840e97949500  0x1ac4  File    0x12019f        \Device\Tcp
4       System  0x840ea0ed43e0  0x1ac8  Event   0x1f0003        -
4       System  0x840e9794b8f0  0x1acc  File    0x12019f        \Device\Tcp
4       System  0x840e98ff32c0  0x1ad0  Process 0x102a  SecurityHealth Pid 8808
4       System  0x840e97460c60  0x1ad4  Event   0x1f0003        -
4       System  0x840e99f501c0  0x1ad8  IoCompletion    0x1f0003        -
4       System  0x840e974b7080  0x1adc  Process 0x102a  CSFalconContai Pid 5980
4       System  0x840e9ac374e0  0x1ae0  WaitCompletionPacket    0x1     -
4       System  0x840e978de2f0  0x1ae4  WaitCompletionPacket    0x1     -
4       System  0x840e97949670  0x1ae8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9716a7a0  0x1aec  File    0x180   \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e97add1c0  0x1af0  Process 0x102a  svchost.exe Pid 6020
4       System  0x840e979497e0  0x1af4  File    0x12019f        \Device\Tcp
4       System  0x840e9794b610  0x1af8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794b330  0x1afc  File    0x12019f        \Device\Tcp
4       System  0x840e97949950  0x1b00  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e97949ac0  0x1b04  File    0x12019f        \Device\Tcp
4       System  0x840e9794b780  0x1b08  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e97949c30  0x1b0c  File    0x12019f        \Device\Tcp
4       System  0x840e9794ba60  0x1b10  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794a640  0x1b14  File    0x12019f        \Device\Tcp
4       System  0x840e9794a920  0x1b18  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794a7b0  0x1b1c  File    0x12019f        \Device\Tcp
4       System  0x840e97949da0  0x1b20  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794a080  0x1b24  File    0x12019f        \Device\Tcp
4       System  0x840e97948dd0  0x1b28  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794ac00  0x1b2c  File    0x12019f        \Device\Tcp
4       System  0x840e9794b1c0  0x1b30  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794a4d0  0x1b34  File    0x12019f        \Device\Tcp
4       System  0x840e9794a1f0  0x1b38  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794aa90  0x1b3c  File    0x12019f        \Device\Tcp
4       System  0x840e9794a360  0x1b40  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794b4a0  0x1b44  File    0x12019f        \Device\Tcp
4       System  0x840e9794ad70  0x1b48  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794aee0  0x1b4c  File    0x12019f        \Device\Tcp
4       System  0x840e979490b0  0x1b50  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794c190  0x1b54  File    0x12019f        \Device\Tcp
4       System  0x840e9794ca30  0x1b58  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794c300  0x1b5c  File    0x12019f        \Device\Tcp
4       System  0x840e9794c5e0  0x1b60  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794cd10  0x1b64  File    0x12019f        \Device\Tcp
4       System  0x840e9794e580  0x1b68  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794e6f0  0x1b6c  File    0x12019f        \Device\Tcp
4       System  0x840e9794c470  0x1b70  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794cba0  0x1b74  File    0x12019f        \Device\Tcp
4       System  0x840e9794e410  0x1b78  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794e860  0x1b7c  File    0x12019f        \Device\Tcp
4       System  0x840e9794c750  0x1b80  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794c8c0  0x1b84  File    0x12019f        \Device\Tcp
4       System  0x840e9794ce80  0x1b88  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794d160  0x1b8c  File    0x12019f        \Device\Tcp
4       System  0x840e9794bbd0  0x1b90  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794beb0  0x1b94  File    0x12019f        \Device\Tcp
4       System  0x840e9794d720  0x1b98  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794d2d0  0x1b9c  File    0x12019f        \Device\Tcp
4       System  0x840e9794e130  0x1ba0  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794d890  0x1ba4  File    0x12019f        \Device\Tcp
4       System  0x840e9794d440  0x1ba8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794bd40  0x1bac  File    0x12019f        \Device\Tcp
4       System  0x840e9794d5b0  0x1bb0  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794db70  0x1bb4  File    0x12019f        \Device\Tcp
4       System  0x840e9794da00  0x1bb8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e9794de50  0x1bbc  File    0x12019f        \Device\Tcp
4       System  0x840e9794dce0  0x1bc0  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e979510a0  0x1bc4  File    0x12019f        \Device\Tcp
4       System  0x840e9794e2a0  0x1bc8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e97951210  0x1bcc  File    0x12019f        \Device\Tcp
4       System  0x840e9794ee20  0x1bd0  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e979500d0  0x1bd4  File    0x12019f        \Device\Tcp
4       System  0x840e9794ecb0  0x1bd8  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e97951380  0x1bdc  File    0x12019f        \Device\Tcp
4       System  0x840e97950800  0x1be0  File    0x0     \Device\NetBT_Tcpip_{1EF44EB3-8B26-4D7F-AD02-5E43CF978E68}
4       System  0x840e974b7080  0x1be4  Process 0x102a  CSFalconContai Pid 5980
4       System  0x840e9229ade0  0x1be8  Event   0x1f0003        -
4       System  0x840e97c640c0  0x1bec  Process 0x102a  powershell.exe Pid 10352
4       System  0x840e992bb340  0x1bf0  IoCompletion    0x1f0003        -
4       System  0x840e9737a3c0  0x1bf4  IoCompletion    0x1f0003        -
4       System  0x840e970c5e80  0x1bf8  IoCompletion    0x1f0003        -
4       System  0x840e97add1c0  0x1bfc  Process 0x102a  svchost.exe Pid 6020
4       System  0x840e97aefbd0  0x1c04  TmTm    0xf003f -
4       System  0x840e9c9332e0  0x1c08  Event   0x1f0003        -
4       System  0x840e9daca140  0x1c0c  IoCompletion    0x1f0003        -
4       System  0x840e96c23280  0x1c10  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e99ab9840  0x1c14  IoCompletion    0x1f0003        -
4       System  0x840e981ae900  0x1c18  WaitCompletionPacket    0x1     -
4       System  0x840e99840080  0x1c1c  Process 0x0     dllhost.exe Pid 2492
4       System  0x840e9c4747c0  0x1c20  IoCompletion    0x1f0003        -
4       System  0x840e98c136e0  0x1c24  Event   0x1f0003        -
4       System  0x840e9ba33a80  0x1c28  WaitCompletionPacket    0x1     -
4       System  0x840e98a1d7c0  0x1c2c  IoCompletion    0x1f0003        -
4       System  0xb7082045c5d0  0x1c30  Section 0x17    -
4       System  0x840e96c23280  0x1c34  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e9a34d140  0x1c38  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\windows.immersivecontrolpanel_10.0.2.1000_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG1
4       System  0x840e97c34820  0x1c3c  File    0x3     \Device\HarddiskVolume3\Windows\appcompat\Programs\Amcache.hve.LOG1
4       System  0xb70814b6d330  0x1c40  Section 0x5     -
4       System  0x840e9b998270  0x1c44  WaitCompletionPacket    0x1     -
4       System  0x840e99ac0a00  0x1c48  IoCompletion    0x1f0003        -
4       System  0x840ea534e160  0x1c50  Event   0x1f0003        -
4       System  0xb708044ae060  0x1c54  Token   0xf01ff -
4       System  0x840e97dc72c0  0x1c58  Process 0x102a  SearchIndexer. Pid 1852
4       System  0x840e99aab4c0  0x1c5c  IoCompletion    0x1f0003        -
4       System  0x840ea47c48f0  0x1c60  WaitCompletionPacket    0x1     -
4       System  0x840e97dc72c0  0x1c64  Process 0x1fffff        SearchIndexer. Pid 1852
4       System  0x840e97f80080  0x1c68  Process 0x1fffff        AggregatorHost Pid 5804
4       System  0x840e98c2d9e0  0x1c6c  Event   0x1f0003        -
4       System  0x840e97159300  0x1c74  Process 0x1028  CSFalconServic Pid 3648
4       System  0x840e99e240c0  0x1c78  Process 0x1fffff        ScreenConnect. Pid 6988
4       System  0x840e9cea20e0  0x1c7c  Event   0x1f0003        -
4       System  0x840e993240c0  0x1c80  Process 0x102a  sihost.exe Pid 10384
4       System  0x840ea7061080  0x1c84  Process 0x1fffff        svchost.exe Pid 16024
4       System  0x840e974b4080  0x1c8c  Process 0x102a  CSFalconContai Pid 324
4       System  0x840e92f32940  0x1c90  IoCompletion    0x1f0003        -
4       System  0x840e99e240c0  0x1c94  Process 0x102a  ScreenConnect. Pid 6988
4       System  0x840e97f7e080  0x1c98  Process 0x102a  svchost.exe Pid 5772
4       System  0x840e97ab5d30  0x1c9c  File    0x12019f        \Device\HarddiskVolume3\Users\admin\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM.blf
4       System  0x840e97c4a1e0  0x1ca0  Event   0x1f0003        -
4       System  0xb70805b847b0  0x1ca4  Token   0xe     -
4       System  0x840e9230b100  0x1ca8  Process 0x0     services.exe Pid 1000
4       System  0xb70808820040  0x1cac  Token   0xe     -
4       System  0x840e97d49b40  0x1cb0  IoCompletion    0x1f0003        -
4       System  0x840ea5a198e0  0x1cb4  Event   0x1f0003        -
4       System  0x840e98c271e0  0x1cb8  Event   0x1f0003        -
4       System  0x840e97c52a60  0x1cbc  Event   0x1f0003        -
4       System  0x840e97d4a400  0x1cc8  IoCompletion    0x1f0003        -
4       System  0x840e98bdf080  0x1ccc  Process 0x40    SystemSettings Pid 10020
4       System  0xb708053902d0  0x1cd0  Token   0xe     -
4       System  0x840e978e4470  0x1cd4  WaitCompletionPacket    0x1     -
4       System  0x840e990b2490  0x1cd8  WaitCompletionPacket    0x1     -
4       System  0x840e999e58e0  0x1cdc  Event   0x1f0003        -
4       System  0x840e92cc1090  0x1ce0  IoCompletionReserve     0xf0003 -
4       System  0x840e9760aca0  0x1ce4  ALPC Port       0x1f0001        -
4       System  0x840e96da72c0  0x1ce8  Process 0x102a  svchost.exe Pid 3324
4       System  0x840e980d08e0  0x1cec  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTDiagtrack-Listener.etl
4       System  0x840e91ee7080  0x1cf0  Process 0x102a  Sysmon64.exe Pid 1540
4       System  0x840e9c89a060  0x1cf4  Event   0x1f0003        -
4       System  0x840e97159300  0x1cf8  Process 0x1028  CSFalconServic Pid 3648
4       System  0x840e990ab7b0  0x1cfc  WaitCompletionPacket    0x1     -
4       System  0x840e91d9ace0  0x1d00  Event   0x1f0003        -
4       System  0x840e97fafb60  0x1d04  Event   0x1f0003        -
4       System  0x840e97f7e080  0x1d08  Process 0x102a  svchost.exe Pid 5772
4       System  0x840e9c5aef60  0x1d0c  Event   0x1f0003        -
4       System  0x840e97c2f230  0x1d10  File    0x20003 \Device\HarddiskVolume3\Windows\appcompat\Programs\Amcache.hve
4       System  0x840e9783b080  0x1d14  Process 0x102a  CSFalconContai Pid 6604
4       System  0x840e99bdd080  0x1d18  Process 0x40    SearchApp.exe Pid 9288
4       System  0x840e9907dd40  0x1d1c  WaitCompletionPacket    0x1     -
4       System  0x840e97d55b00  0x1d20  IoCompletion    0x1f0003        -
4       System  0x840e9b7bb080  0x1d24  Process 0x0     StartMenuExper Pid 8
4       System  0xb7080a2764b0  0x1d28  Section 0x17    -
4       System  0x840e9680cdb0  0x1d2c  WaitCompletionPacket    0x1     -
4       System  0x840e974644e0  0x1d30  Event   0x1f0003        -
4       System  0x840e97f7e080  0x1d34  Process 0x1fffff        svchost.exe Pid 5772
4       System  0x840e992b9200  0x1d38  IoCompletion    0x1f0003        -
4       System  0xb70816eba3a0  0x1d3c  Directory       0xf     S-1-15-2-2246530975-808720366-1776470054-230329187-4153223113-3550430174-4193313734
4       System  0x840e978e7600  0x1d40  WaitCompletionPacket    0x1     -
4       System  0x840e9170e740  0x1d44  WaitCompletionPacket    0x1     -
4       System  0x840e97811060  0x1d48  Event   0x1f0003        -
4       System  0x840e97d4c1c0  0x1d4c  IoCompletion    0x1f0003        -
4       System  0x840e975ad080  0x1d50  Process 0x102a  CSFalconContai Pid 1192
4       System  0x840e97349860  0x1d54  FilterCommunicationPort 0x1f0001        -
4       System  0xb7080874e870  0x1d58  Section 0x17    -
4       System  0x840e96f1d300  0x1d5c  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e999dd0e0  0x1d60  Event   0x1f0003        -
4       System  0x840e974b4080  0x1d64  Process 0x102a  CSFalconContai Pid 324
4       System  0x840e99fd2960  0x1d70  Event   0x1f0003        -
4       System  0x840e96f1d300  0x1d74  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840ea08f3080  0x1d7c  Process 0x102a  svchost.exe Pid 12008
4       System  0x840e98d0f980  0x1d80  IoCompletion    0x1f0003        -
4       System  0x840e97348a60  0x1d88  FilterCommunicationPort 0x1f0001        -
4       System  0x840e98c27d60  0x1d8c  Event   0x1f0003        -
4       System  0x840e97ab5ec0  0x1d90  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\admin\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0x840e97159300  0x1d94  Process 0x1028  CSFalconServic Pid 3648
4       System  0x840e91f131c0  0x1d98  Process 0x0     Sysmon64.exe Pid 1244
4       System  0x840e98c48080  0x1d9c  Process 0x102a  ApplicationFra Pid 4836
4       System  0x840e96f1d300  0x1da0  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e96f1d300  0x1da4  Process 0x102a  dasHost.exe Pid 3752
4       System  0x840e97136a10  0x1dac  WaitCompletionPacket    0x1     -
4       System  0x840e97ab3300  0x1db0  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\admin\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0x840e9b98f2a0  0x1db4  WaitCompletionPacket    0x1     -
4       System  0x840e97e0c8a0  0x1db8  TmRm    0x1f007f        -
4       System  0x840e974b2080  0x1dc0  Process 0x102a  UserOOBEBroker Pid 9620
4       System  0x840e9817d8f0  0x1dc4  WaitCompletionPacket    0x1     -
4       System  0x840e995680c0  0x1dc8  Process 0x102a  SgrmBroker.exe Pid 9948
4       System  0x840e978d4550  0x1dcc  WaitCompletionPacket    0x1     -
4       System  0x840e9c906860  0x1dd0  Event   0x1f0003        -
4       System  0x840e99f7bdc0  0x1dd4  IoCompletion    0x1f0003        -
4       System  0x840e97ab3c60  0x1dd8  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat
4       System  0x840e9b9867b0  0x1ddc  WaitCompletionPacket    0x1     -
4       System  0x840e98eb3080  0x1de0  Process 0x102a  conhost.exe Pid 10944
4       System  0x840e9819ef90  0x1de4  WaitCompletionPacket    0x1     -
4       System  0xb70808820040  0x1de8  Token   0xe     -
4       System  0xb70808820040  0x1dec  Token   0xe     -
4       System  0x840e9680d770  0x1df0  WaitCompletionPacket    0x1     -
4       System  0x840e97442080  0x1df4  Process 0x102a  CSFalconContai Pid 6624
4       System  0x840e91cd5cb0  0x1df8  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTcs-etw-sm.etl
4       System  0x840e97d5c780  0x1dfc  IoCompletion    0x1f0003        -
4       System  0x840e96808630  0x1e00  WaitCompletionPacket    0x1     -
4       System  0x840e90bb1ae0  0x1e04  Event   0x1f0003        -
4       System  0x840e97ab16e0  0x1e08  File    0x3     \Device\HarddiskVolume3\Users\admin\ntuser.dat.LOG1
4       System  0x840e981b37d0  0x1e0c  WaitCompletionPacket    0x1     -
4       System  0x840e96d4d350  0x1e10  Timer   0x1f0003        -
4       System  0x840e978e02a0  0x1e14  WaitCompletionPacket    0x1     -
4       System  0x840e96e0f080  0x1e18  Process 0x1fffff        sihost.exe Pid 6908
4       System  0x840e9971f860  0x1e1c  Event   0x1f0003        -
4       System  0x840e96e0f080  0x1e20  Process 0x102a  sihost.exe Pid 6908
4       System  0x840e97988ce0  0x1e24  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\MicrosoftWindows.Client.CBS_1000.19058.1000.0_x64__cw5n1h2txyewy\ActivationStore.dat
4       System  0xb70817928270  0x1e2c  Section 0x5     -
4       System  0x840e97ab5560  0x1e30  File    0x12019f        \Device\HarddiskVolume3\Users\admin\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000001.regtrans-ms
4       System  0x840e97648800  0x1e34  IoCompletion    0x1f0003        -
4       System  0x840e9a637b00  0x1e38  IoCompletion    0x1f0003        -
4       System  0x840e97ab0a60  0x1e3c  File    0x20003 \Device\HarddiskVolume3\Users\admin\NTUSER.DAT
4       System  0xb7080ec3ef60  0x1e40  Key     0xf003f USER\S-1-5-21-3294271487-4292050549-1661813568-1001\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\WINLOGON
4       System  0x840e97815760  0x1e44  Event   0x1f0003        -
4       System  0x840e9c9b2560  0x1e48  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e999f1760  0x1e50  Event   0x1f0003        -
4       System  0x840e91d8e6e0  0x1e54  Event   0x1f0003        -
4       System  0x840e9711e060  0x1e5c  Event   0x1f0003        -
4       System  0x840e9c70f6e0  0x1e60  Event   0x1f0003        -
4       System  0x840e97ab3620  0x1e64  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat.LOG1
4       System  0x840e9764de80  0x1e68  IoCompletion    0x1f0003        -
4       System  0x840e98d8a060  0x1e6c  TmTm    0xf003f -
4       System  0x840e9c236080  0x1e70  Process 0x102a  dllhost.exe Pid 5164
4       System  0x840e97fc5160  0x1e74  Event   0x1f0003        -
4       System  0x840e96c5a080  0x1e78  Process 0x102a  svchost.exe Pid 7108
4       System  0x840e97ab48e0  0x1e7c  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat.LOG2
4       System  0x840e98fe0ce0  0x1e84  Event   0x1f0003        -
4       System  0x840e98aef080  0x1e88  Process 0x1fffff        RuntimeBroker. Pid 7684
4       System  0x840e90ad3080  0x1e8c  Process 0x1fffff        svchost.exe Pid 7152
4       System  0x840e99bcb080  0x1e90  Process 0x0     explorer.exe Pid 9852
4       System  0x840e99016980  0x1e94  IoCompletion    0x1f0003        -
4       System  0x840e97468560  0x1e98  Event   0x1f0003        -
4       System  0x840e98b2cee0  0x1e9c  WaitCompletionPacket    0x1     -
4       System  0xb708087665b0  0x1ea0  Section 0x17    -
4       System  0x840e96771c40  0x1ea4  TmRm    0x1f007f        -
4       System  0x840e97ab6690  0x1ea8  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat{54a96616-79ef-11ef-ba74-bc2411d898d1}.TM
4       System  0x840e97ab6500  0x1eac  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\admin\AppData\Local\Microsoft\Windows\UsrClass.dat{54a96616-79ef-11ef-ba74-bc2411d898d1}.TM
4       System  0x840e97af0c60  0x1eb0  TmRm    0x1f007f        -
4       System  0x840e96f29060  0x1eb4  TmTm    0xf003f -
4       System  0x840e9cf2c5c0  0x1eb8  IoCompletion    0x1f0003        -
4       System  0x840e98b547d0  0x1ebc  WaitCompletionPacket    0x1     -
4       System  0x840e97384f40  0x1ec0  IoCompletion    0x1f0003        -
4       System  0x840ea08f1080  0x1ec4  Process 0x1fffff        svchost.exe Pid 3444
4       System  0xb7081cbe3c50  0x1ec8  SymbolicLink    0xf0001 Session
4       System  0x840e9c924a60  0x1ecc  Event   0x1f0003        -
4       System  0x840e99094c80  0x1ed0  WaitCompletionPacket    0x1     -
4       System  0x840e9714d7b0  0x1ed4  WaitCompletionPacket    0x1     -
4       System  0x840e9ac0f640  0x1ed8  WaitCompletionPacket    0x1     -
4       System  0x840e978c0300  0x1edc  Process 0x1fffff        explorer.exe Pid 5328
4       System  0x840e9ba0fce0  0x1ee0  WaitCompletionPacket    0x1     -
4       System  0x840e9676e080  0x1eec  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9b9856a0  0x1ef0  WaitCompletionPacket    0x1     -
4       System  0x840e981ab770  0x1ef4  WaitCompletionPacket    0x1     -
4       System  0x840e984fdbe0  0x1efc  Timer   0x2     -
4       System  0x840ea184c280  0x1f00  File    0x12008b        \Device\HarddiskVolume3\Windows\System32\SleepStudy\ScreenOn\ScreenOnPowerStudyTraceSession-2025-03-17-11-00-54.etl
4       System  0x840e9a63a940  0x1f04  IoCompletion    0x1f0003        -
4       System  0x840e98b15080  0x1f08  Process 0x102a  fontdrvhost.ex Pid 10704
4       System  0x840e9c89d560  0x1f0c  Event   0x1f0003        -
4       System  0x840e96c23280  0x1f10  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e98c31960  0x1f14  Event   0x1f0003        -
4       System  0x840e99adf4c0  0x1f18  IoCompletion    0x1f0003        -
4       System  0x840e9f444dc0  0x1f1c  File    0x12008b        \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\AppData\Local\Microsoft\Windows\DeliveryOptimization\Logs\dosvc.20250312_220125_696.etl
4       System  0x840e9a628500  0x1f20  IoCompletion    0x1f0003        -
4       System  0x840e9f2dabe0  0x1f24  Event   0x1f0003        -
4       System  0x840e99fcefe0  0x1f28  Event   0x1f0003        -
4       System  0x840e92def080  0x1f2c  Process 0x0     svchost.exe Pid 1292
4       System  0x840e98184c50  0x1f30  WaitCompletionPacket    0x1     -
4       System  0x840e97394b40  0x1f34  IoCompletion    0x1f0003        -
4       System  0x840e92edb2a0  0x1f38  WaitCompletionPacket    0x1     -
4       System  0x840e9901a940  0x1f40  IoCompletion    0x1f0003        -
4       System  0x840e91c2a080  0x1f44  Process 0x102a  svchost.exe Pid 5340
4       System  0x840e96c23280  0x1f48  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e99ca3070  0x1f58  RawInputManager 0x3     -
4       System  0x840e91d9d560  0x1f5c  Event   0x1f0003        -
4       System  0x840e98d1b5c0  0x1f60  IoCompletion    0x1f0003        -
4       System  0x840ea7061080  0x1f64  Process 0x102a  svchost.exe Pid 16024
4       System  0x840e9764f2c0  0x1f68  IoCompletion    0x1f0003        -
4       System  0x840e923de2c0  0x1f6c  Process 0x102a  svchost.exe Pid 756
4       System  0xb70810d6e6f0  0x1f70  Key     0xf003f MACHINE\SYSTEM\DRIVERDATABASE\DRIVERINFFILES
4       System  0x840e99c6e080  0x1f78  Process 0x102a  svchost.exe Pid 4636
4       System  0x840e96e0f080  0x1f80  Process 0x102a  sihost.exe Pid 6908
4       System  0x840e9817a830  0x1f84  WaitCompletionPacket    0x1     -
4       System  0x840e9746dfe0  0x1f88  Event   0x1f0003        -
4       System  0x840e9a14a260  0x1f8c  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\UserClasses.dat.LOG2
4       System  0x840e98d0df40  0x1f90  IoCompletion    0x1f0003        -
4       System  0x840e9ac11450  0x1f94  WaitCompletionPacket    0x1     -
4       System  0x840e9767c200  0x1f98  IoCompletion    0x1f0003        -
4       System  0x840e96c5a080  0x1f9c  Process 0x1fffff        svchost.exe Pid 7108
4       System  0x840e9769f080  0x1fa0  Process 0x102a  winlogon.exe Pid 2804
4       System  0x840e97818760  0x1fa4  Event   0x1f0003        -
4       System  0xb70809580a40  0x1fa8  Token   0xf01ff -
4       System  0xb70810ca5280  0x1fb0  Directory       0xf000f 00000000-00143ff6
4       System  0x840e96e0f080  0x1fb4  Process 0x102a  sihost.exe Pid 6908
4       System  0x840e9a944860  0x1fb8  Event   0x1f0003        -
4       System  0xb708109b2480  0x1fbc  Directory       0xf000f 00000000-00144079
4       System  0x840e968116d0  0x1fc0  WaitCompletionPacket    0x1     -
4       System  0x840e9c8958e0  0x1fc4  Event   0x1f0003        -
4       System  0x840e999f4660  0x1fc8  Event   0x1f0003        -
4       System  0x840eaa185360  0x1fcc  WaitCompletionPacket    0x1     -
4       System  0x840e978d8a60  0x1fd0  WaitCompletionPacket    0x1     -
4       System  0xb7080a77a2b0  0x1fd8  Section 0x17    -
4       System  0x840e91f131c0  0x1fdc  Process 0x102a  Sysmon64.exe Pid 1244
4       System  0x840e99fd8d60  0x1fe0  Event   0x1f0003        -
4       System  0x840e91f131c0  0x1fe4  Process 0x1fffff        Sysmon64.exe Pid 1244
4       System  0x840e9f00fc10  0x1fec  WaitCompletionPacket    0x1     -
4       System  0x840e97fc9de0  0x1ff0  Event   0x1f0003        -
4       System  0x840e9ca50740  0x1ff4  IoCompletion    0x1f0003        -
4       System  0x840e981a3710  0x1ff8  WaitCompletionPacket    0x1     -
4       System  0xb7081120abe0  0x1ffc  SymbolicLink    0xf0001 Global
4       System  0x840e978c0300  0x2004  Process 0x40    explorer.exe Pid 5328
4       System  0x840e998450c0  0x2008  Process 0x102a  SearchApp.exe Pid 4236
4       System  0x840e9c848460  0x200c  Event   0x1f0003        -
4       System  0x840e98b3e8d0  0x2010  WaitCompletionPacket    0x1     -
4       System  0x840e9cac44e0  0x2014  Event   0x1f0003        -
4       System  0x840e9a949360  0x2018  Event   0x1f0003        -
4       System  0x840e99fd65e0  0x201c  Event   0x1f0003        -
4       System  0x840e9dabb600  0x2020  IoCompletion    0x1f0003        -
4       System  0x840e978c0300  0x2024  Process 0x40    explorer.exe Pid 5328
4       System  0x840e978c0300  0x2028  Process 0x102a  explorer.exe Pid 5328
4       System  0x840e90c611c0  0x202c  WaitCompletionPacket    0x1     -
4       System  0x840e99abccc0  0x2030  IoCompletion    0x1f0003        -
4       System  0x840e9e3d9960  0x2034  Event   0x1f0003        -
4       System  0x840e9971c660  0x2038  Event   0x1f0003        -
4       System  0x840e9ba172b0  0x203c  WaitCompletionPacket    0x1     -
4       System  0x840e91f131c0  0x2040  Process 0x102a  Sysmon64.exe Pid 1244
4       System  0xb7080a744e30  0x2044  Section 0x5     -
4       System  0x840e99029d80  0x2048  IoCompletion    0x1f0003        -
4       System  0x840ea08f3080  0x204c  Process 0x102a  svchost.exe Pid 12008
4       System  0x840e9909fcd0  0x2050  WaitCompletionPacket    0x1     -
4       System  0x840e90ad3080  0x2054  Process 0x102a  svchost.exe Pid 7152
4       System  0x840eab2ef3e0  0x2058  Event   0x1f0003        -
4       System  0x840e98079400  0x205c  CoreMessaging   0xf0000 -
4       System  0x840e98c32ae0  0x2060  Event   0x1f0003        -
4       System  0x840e98b2cd40  0x2064  WaitCompletionPacket    0x1     -
4       System  0x840e9c932fe0  0x2068  Event   0x1f0003        -
4       System  0x840e96c23280  0x206c  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e9a3161f0  0x2070  IoCompletionReserve     0xf0003 -
4       System  0x840e99ba8040  0x2074  Thread  0x1fffff        Tid 5864 Pid 4
4       System  0x840e96959ec0  0x2078  IoCompletion    0x1f0003        -
4       System  0x840e98b2eb50  0x207c  WaitCompletionPacket    0x1     -
4       System  0x840e98e742c0  0x208c  Process 0x102a  svchost.exe Pid 8292
4       System  0x840e9817fcb0  0x2090  WaitCompletionPacket    0x1     -
4       System  0x840e992ac700  0x2094  IoCompletion    0x1f0003        -
4       System  0x840e91d8bc60  0x2098  Event   0x1f0003        -
4       System  0x840e9738c900  0x209c  IoCompletion    0x1f0003        -
4       System  0x840e91c2a080  0x20a0  Process 0x1fffff        svchost.exe Pid 5340
4       System  0x840e9a7886b0  0x20a4  IoCompletionReserve     0xf0003 -
4       System  0x840e91db7340  0x20a8  Process 0x102a  taskhostw.exe Pid 4428
4       System  0x840e91bce2e0  0x20ac  WaitCompletionPacket    0x1     -
4       System  0x840e96c23280  0x20b0  Process 0x102a  spoolsv.exe Pid 2904
4       System  0x840e96bf2f60  0x20b4  Event   0x1f0003        -
4       System  0x840e984fd880  0x20b8  Timer   0x2     -
4       System  0x840e98b39d40  0x20bc  WaitCompletionPacket    0x1     -
4       System  0x840e99f754c0  0x20c0  IoCompletion    0x1f0003        -
4       System  0x840e9738ac00  0x20c8  IoCompletion    0x1f0003        -
4       System  0x840e98d0f440  0x20cc  IoCompletion    0x1f0003        -
4       System  0x840e91db7340  0x20d4  Process 0x102a  taskhostw.exe Pid 4428
4       System  0x840e91d90ce0  0x20d8  Event   0x1f0003        -
4       System  0x840e999ef460  0x20dc  Event   0x1f0003        -
4       System  0x840e9927d630  0x20e0  IoCompletionReserve     0xf0003 -
4       System  0x840e97d80900  0x20e4  IoCompletion    0x1f0003        -
4       System  0x840e995680c0  0x20e8  Process 0x1028  SgrmBroker.exe Pid 9948
4       System  0x840e98d3ab00  0x20ec  IoCompletion    0x1f0003        -
4       System  0x840e96818140  0x20f0  WaitCompletionPacket    0x1     -
4       System  0x840e9c7086e0  0x20f4  Event   0x1f0003        -
4       System  0x840e91da22e0  0x20f8  Event   0x1f0003        -
4       System  0x840e92cfa140  0x20fc  Process 0x40    dwm.exe Pid 1204
4       System  0x840e9c895960  0x2100  Event   0x1f0003        -
4       System  0x840e99042800  0x2104  IoCompletion    0x1f0003        -
4       System  0x840e978082e0  0x2108  Event   0x1f0003        -
4       System  0x840e9695e400  0x210c  IoCompletion    0x1f0003        -
4       System  0x840e9817d680  0x2110  WaitCompletionPacket    0x1     -
4       System  0x840e9736cd80  0x2114  IoCompletion    0x1f0003        -
4       System  0x840e9db77080  0x2118  Process 0x0     taskhostw.exe Pid 15784
4       System  0x840e91da10e0  0x211c  Event   0x1f0003        -
4       System  0x840e992a9980  0x2120  IoCompletion    0x1f0003        -
4       System  0x840e981aa8d0  0x2124  WaitCompletionPacket    0x1     -
4       System  0x840e9a611bc0  0x212c  IoCompletion    0x1f0003        -
4       System  0xb70816ebc560  0x2130  Directory       0xf     RPC Control
4       System  0x840e9c930760  0x2134  Event   0x1f0003        -
4       System  0x840e9a14d140  0x2138  File    0x1     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-5-21-3294271487-4292050549-1661813568-1001\SystemAppData\Helium\Cache\b4d622c333fcfdd7.dat
4       System  0x840e992234b0  0x213c  IoCompletionReserve     0xf0003 -
4       System  0x840e975ca1d0  0x2140  File    0x120089        \Device\HarddiskVolume3\Windows\CSC\v2.0.6\namespace
4       System  0x840e9a146d40  0x2144  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\User.dat.LOG1
4       System  0x840e9909c660  0x2148  WaitCompletionPacket    0x1     -
4       System  0x840e9c8618e0  0x214c  Event   0x1f0003        -
4       System  0x840e981a04b0  0x2150  WaitCompletionPacket    0x1     -
4       System  0x840e97353960  0x2158  FilterCommunicationPort 0x1f0001        -
4       System  0x840e90e977b0  0x215c  Event   0x1f0003        CfProgressEvent
4       System  0x840e992ba7c0  0x2160  IoCompletion    0x1f0003        -
4       System  0x840e999e3c60  0x2164  Event   0x1f0003        -
4       System  0x840e90bc8660  0x2168  Event   0x1f0003        -
4       System  0xb708109e1610  0x216c  SymbolicLink    0xf0001 SWD#PRINTENUM#{A763D421-EDFD-4DAB-8863-40F4DE9FF327}#{0ecef634-6ef0-472a-8085-5ad023ecbccd}
4       System  0x840e90e10410  0x2170  Session 0x0     Session2
4       System  0x840e9745de60  0x2174  Event   0x1f0003        -
4       System  0x840ea3ce7560  0x2178  Event   0x1f0003        -
4       System  0x840e975ca9a0  0x217c  File    0x12019f        \Device\HarddiskVolume3\Windows\CSC\v2.0.6\pq
4       System  0x840e98fe2e60  0x2184  Event   0x1f0003        -
4       System  0x840e98e969e0  0x2188  Event   0x1f0003        -
4       System  0x840e9907bbf0  0x218c  WaitCompletionPacket    0x1     -
4       System  0x840e9908fce0  0x2190  WaitCompletionPacket    0x1     -
4       System  0x840e9ac0f4a0  0x2194  WaitCompletionPacket    0x1     -
4       System  0x840e98fd3d60  0x2198  Event   0x1f0003        -
4       System  0x840e9a651eb0  0x21a0  IoCompletionReserve     0xf0003 -
4       System  0x840e9b9c1630  0x21a8  WaitCompletionPacket    0x1     -
4       System  0x840e96c5a080  0x21ac  Process 0x102a  svchost.exe Pid 7108
4       System  0xb7081bb5b400  0x21b0  Directory       0xf     RPC Control
4       System  0x840e91da7e60  0x21b4  Event   0x1f0003        -
4       System  0xb7081bb5c8a0  0x21bc  Directory       0xf     S-1-15-2-515815643-2845804217-1874292103-218650560-777617685-4287762684-137415000
4       System  0x840e9c714860  0x21c0  Event   0x1f0003        -
4       System  0x840e90ad3080  0x21c4  Process 0x102a  svchost.exe Pid 7152
4       System  0x840e975bb5e0  0x21c8  File    0x100081        \Device\HarddiskVolume3炮☺
4       System  0xb7080a29c390  0x21cc  Section 0x17    -
4       System  0x840e9904fe80  0x21d0  IoCompletion    0x1f0003        -
4       System  0x840e9ca5aa00  0x21d4  IoCompletion    0x1f0003        -
4       System  0x840e9cb526c0  0x21d8  IoCompletion    0x1f0003        -
4       System  0x840e992b5f40  0x21dc  IoCompletion    0x1f0003        -
4       System  0xb70816ebdbe0  0x21e0  Directory       0xf     RPC Control
4       System  0x840e99c157e0  0x21e4  Event   0x1f0003        -
4       System  0x840e9dae0f00  0x21e8  IoCompletion    0x1f0003        -
4       System  0x840e975cacc0  0x21ec  File    0x120089        \Device\HarddiskVolume3\Windows\CSC\v2.0.6
4       System  0x840e975cae50  0x21f0  File    0x12019f        \Device\HarddiskVolume3\Windows\CSC\v2.0.6\temp
4       System  0x840e97e11080  0x21f4  Process 0x102a  TextInputHost. Pid 1256
4       System  0x840e9819bb90  0x21f8  WaitCompletionPacket    0x1     -
4       System  0x840e92eea590  0x21fc  WaitCompletionPacket    0x1     -
4       System  0x840e98d15200  0x2200  IoCompletion    0x1f0003        -
4       System  0x840e97374e00  0x2204  IoCompletion    0x1f0003        -
4       System  0xb708109e1e50  0x2208  SymbolicLink    0xf0001 SWD#PRINTENUM#{AEBD5C3D-F2AF-4E79-8041-0CF8E0AC0AE3}#{0ecef634-6ef0-472a-8085-5ad023ecbccd}
4       System  0x840e98d33240  0x220c  IoCompletion    0x1f0003        -
4       System  0x840e9903c540  0x2210  IoCompletion    0x1f0003        -
4       System  0xb708109e1b90  0x2214  SymbolicLink    0xf0001 SWD#PRINTENUM#{BC835CDC-D582-4230-83CB-5335521321E0}#{0ecef634-6ef0-472a-8085-5ad023ecbccd}
4       System  0x840e98d40980  0x2218  IoCompletion    0x1f0003        -
4       System  0xb708109e04e0  0x221c  SymbolicLink    0xf0001 SWD#PRINTENUM#{CDBBC391-ED2A-41F9-BC46-93FED66AB8AE}#{0ecef634-6ef0-472a-8085-5ad023ecbccd}
4       System  0x840e979bdd50  0x2228  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\MicrosoftWindows.Client.CBS_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e995cd0c0  0x2230  Process 0x102a  svchost.exe Pid 6356
4       System  0x840e9818a820  0x2234  WaitCompletionPacket    0x1     -
4       System  0x840e9a14be80  0x2238  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\UserClasses.dat
4       System  0x840e98c2c660  0x2240  Event   0x1f0003        -
4       System  0x840e98d14400  0x2244  IoCompletion    0x1f0003        -
4       System  0x840e98b38ea0  0x2248  WaitCompletionPacket    0x1     -
4       System  0x840e98d0d080  0x224c  IoCompletion    0x1f0003        -
4       System  0x840e97146ee0  0x2250  WaitCompletionPacket    0x1     -
4       System  0x840e995680c0  0x2254  Process 0x102a  SgrmBroker.exe Pid 9948
4       System  0x840e99f64100  0x2258  IoCompletion    0x1f0003        -
4       System  0x840e97e11080  0x225c  Process 0x1fffff        TextInputHost. Pid 1256
4       System  0x840e98e8ea60  0x2260  Event   0x1f0003        -
4       System  0x840e98a81670  0x2264  IoCompletionReserve     0xf0003 -
4       System  0x840e98d3adc0  0x2268  IoCompletion    0x1f0003        -
4       System  0x840e9969e080  0x2270  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e99fd8ae0  0x2274  Event   0x1f0003        -
4       System  0x840e9d9b1080  0x2278  Process 0x102a  msedge.exe Pid 6408
4       System  0x840e98b37230  0x227c  WaitCompletionPacket    0x1     -
4       System  0x840e968c00c0  0x2288  Process 0x0     svchost.exe Pid 2496
4       System  0x840e97140880  0x228c  WaitCompletionPacket    0x1     -
4       System  0x840e98db8080  0x2290  Process 0x102a  MsSense.exe Pid 10224
4       System  0x840e9e6f3080  0x2294  Process 0x102a  excel-update.e Pid 4672
4       System  0x840e9b8fdd90  0x2298  Timer   0x1f0003        -
4       System  0x840e98e9a560  0x22a4  Event   0x1f0003        -
4       System  0x840e98e89e60  0x22a8  Event   0x1f0003        -
4       System  0x840e98075620  0x22ac  CoreMessaging   0xf0000 -
4       System  0x840e98b388f0  0x22b0  WaitCompletionPacket    0x1     -
4       System  0x840e9ac0a910  0x22b4  WaitCompletionPacket    0x1     -
4       System  0x840e99651080  0x22b8  Process 0x1fffff        conhost.exe Pid 13176
4       System  0x840e9cf2c940  0x22bc  IoCompletion    0x1f0003        -
4       System  0x840e98a23ac0  0x22c0  IoCompletion    0x1f0003        -
4       System  0x840e98181370  0x22c4  WaitCompletionPacket    0x1     -
4       System  0x840e9b99e660  0x22c8  WaitCompletionPacket    0x1     -
4       System  0x840e9b99b9b0  0x22cc  WaitCompletionPacket    0x1     -
4       System  0x840e99084200  0x22d0  WaitCompletionPacket    0x1     -
4       System  0x840e98b8e340  0x22d4  Process 0x102a  RuntimeBroker. Pid 7612
4       System  0x840e9979ca60  0x22d8  Event   0x1f0003        -
4       System  0x840e90ee9dc0  0x22e4  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.blf
4       System  0x840e9f40c340  0x22e8  Process 0x102a  svchost.exe Pid 2800
4       System  0x840e97f80080  0x22ec  Process 0x0     AggregatorHost Pid 5804
4       System  0x840e98b26950  0x22f0  WaitCompletionPacket    0x1     -
4       System  0x840e9b203300  0x22f8  Process 0x102a  KeePass.exe Pid 11628
4       System  0x840e90bb7260  0x22fc  Event   0x1f0003        -
4       System  0x840e992c0d40  0x2300  IoCompletion    0x1f0003        -
4       System  0x840e981b3700  0x2304  WaitCompletionPacket    0x1     -
4       System  0x840e9969e080  0x2308  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e98fb61c0  0x230c  Process 0x102a  dllhost.exe Pid 8472
4       System  0x840e97e11080  0x2310  Process 0x102a  TextInputHost. Pid 1256
4       System  0x840e90bb8460  0x2314  Event   0x1f0003        -
4       System  0x840e992c0740  0x2318  IoCompletion    0x1f0003        -
4       System  0x840e981b4670  0x231c  WaitCompletionPacket    0x1     -
4       System  0x840e98d0f540  0x2328  IoCompletion    0x1f0003        -
4       System  0x840e98c29060  0x232c  Event   0x1f0003        -
4       System  0x840e9ec9c9a0  0x2334  File    0x3     \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\AppData\Local\Microsoft\Windows\DeliveryOptimization\State\dosvcState.dat.LOG1
4       System  0x840e99ab5780  0x2338  IoCompletion    0x1f0003        -
4       System  0x840e98c450c0  0x2340  Process 0x102a  dllhost.exe Pid 7800
4       System  0x840e9a628a40  0x2344  IoCompletion    0x1f0003        -
4       System  0x840e98c18de0  0x2348  Event   0x1f0003        -
4       System  0x840e92cfa140  0x2350  Process 0x0     dwm.exe Pid 1204
4       System  0x840e97159300  0x2354  Process 0x0     CSFalconServic Pid 3648
4       System  0x840e98c289e0  0x2358  Event   0x1f0003        -
4       System  0x840e98d16440  0x235c  IoCompletion    0x1f0003        -
4       System  0x840e9979b0e0  0x2360  Event   0x1f0003        -
4       System  0x840e98b385b0  0x2364  WaitCompletionPacket    0x1     -
4       System  0x840e9676e080  0x2368  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e99387080  0x2370  Process 0x0     StartMenuExper Pid 7408
4       System  0x840e98b45dd0  0x2374  WaitCompletionPacket    0x1     -
4       System  0x840e98d11bc0  0x2378  IoCompletion    0x1f0003        -
4       System  0x840e994d1080  0x237c  Process 0x102a  LogonUI.exe Pid 10680
4       System  0x840e9b9b2680  0x2380  WaitCompletionPacket    0x1     -
4       System  0x840e98e742c0  0x2384  Process 0x0     svchost.exe Pid 8292
4       System  0x840e92fc6080  0x2388  Process 0x0     svchost.exe Pid 1964
4       System  0xb7081304b060  0x238c  Token   0xf01ff -
4       System  0x840e993439e0  0x2390  Event   0x1f0003        -
4       System  0x840e9951c1c0  0x239c  Process 0x0     msedge.exe Pid 8732
4       System  0x840e98e8f0e0  0x23a0  Event   0x1f0003        -
4       System  0x840e970c9900  0x23a4  IoCompletion    0x1f0003        -
4       System  0x840e9d69a690  0x23a8  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\User.dat.LOG2
4       System  0x840e9daf05c0  0x23ac  IoCompletion    0x1f0003        -
4       System  0x840e995ea080  0x23b0  Process 0x102a  OneDrive.exe Pid 11592
4       System  0x840e90de9080  0x23b8  Process 0x1fffff        FileCoAuth.exe Pid 8592
4       System  0x840e98bba080  0x23c0  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e98e742c0  0x23c4  Process 0x102a  svchost.exe Pid 8292
4       System  0xb7081319be60  0x23c8  Directory       0xf     S-1-15-2-3251537155-1984446955-2931258699-841473695-1938553385-924012159-129201922
4       System  0x840e96907690  0x23cc  IoCompletionReserve     0xf0003 -
4       System  0x840e98e96660  0x23d0  Event   0x1f0003        -
4       System  0x840e98177df0  0x23d4  WaitCompletionPacket    0x1     -
4       System  0x840e981a5930  0x23d8  WaitCompletionPacket    0x1     -
4       System  0x840e92c9a340  0x23dc  Process 0x0     svchost.exe Pid 1076
4       System  0x840e995cd0c0  0x23e0  Process 0x0     svchost.exe Pid 6356
4       System  0x840e99adc680  0x23e4  IoCompletion    0x1f0003        -
4       System  0x840e9769f080  0x23e8  Process 0x102a  winlogon.exe Pid 2804
4       System  0x840e990a0420  0x23ec  WaitCompletionPacket    0x1     -
4       System  0x840e97442080  0x23f0  Process 0x0     CSFalconContai Pid 6624
4       System  0x840e992b4b00  0x23f4  IoCompletion    0x1f0003        -
4       System  0x840e9ac3f880  0x23f8  WaitCompletionPacket    0x1     -
4       System  0x840e978c0300  0x23fc  Process 0x40    explorer.exe Pid 5328
4       System  0x840e90c19340  0x2404  Process 0x1fffff        svchost.exe Pid 3604
4       System  0x840e98b15080  0x2408  Process 0x1fffff        fontdrvhost.ex Pid 10704
4       System  0x840e98bba080  0x240c  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e9ac13740  0x2410  WaitCompletionPacket    0x1     -
4       System  0x840e96e552c0  0x2414  Process 0x102a  svchost.exe Pid 3560
4       System  0x840e9ac2a4e0  0x2418  WaitCompletionPacket    0x1     -
4       System  0x840e9a94f4e0  0x241c  Event   0x1f0003        -
4       System  0x840ea0843780  0x2420  IoCompletion    0x1f0003        -
4       System  0x840e9933be60  0x2424  Event   0x1f0003        -
4       System  0x840e98b57210  0x2428  WaitCompletionPacket    0x1     -
4       System  0x840e9968d080  0x242c  Process 0x1fffff        splunkd.exe Pid 13620
4       System  0x840e9832a1c0  0x2430  Process 0x0     msedge.exe Pid 3832
4       System  0x840e996e91c0  0x2434  Process 0x0     msedge.exe Pid 9016
4       System  0x840e98d3a240  0x2438  IoCompletion    0x1f0003        -
4       System  0x840e98d37b00  0x243c  IoCompletion    0x1f0003        -
4       System  0x840e9901d440  0x2440  IoCompletion    0x1f0003        -
4       System  0x840e99338560  0x2444  Event   0x1f0003        -
4       System  0x840e9958a0c0  0x2448  Process 0x0     msedge.exe Pid 9128
4       System  0x840e98b50a10  0x2450  WaitCompletionPacket    0x1     -
4       System  0x840e99f76100  0x2454  IoCompletion    0x1f0003        -
4       System  0x840e9dad8400  0x245c  IoCompletion    0x1f0003        -
4       System  0x840e9969f080  0x2460  Process 0x102a  Microsoft.Shar Pid 5176
4       System  0x840e99f53480  0x2464  IoCompletion    0x1f0003        -
4       System  0x840e92cfa140  0x2468  Process 0x40    dwm.exe Pid 1204
4       System  0x840e9908dc60  0x246c  WaitCompletionPacket    0x1     -
4       System  0x840e9daba640  0x2470  IoCompletion    0x1f0003        -
4       System  0x840e9cf39a80  0x2474  IoCompletion    0x1f0003        -
4       System  0x840e97d58500  0x2478  IoCompletion    0x1f0003        -
4       System  0x840e9ba13c40  0x247c  WaitCompletionPacket    0x1     -
4       System  0xb708156d7990  0x2480  Token   0xf01ff -
4       System  0x840e98d19d40  0x2484  IoCompletion    0x1f0003        -
4       System  0x840e98d18f40  0x2488  IoCompletion    0x1f0003        -
4       System  0x840e98c13a60  0x248c  Event   0x1f0003        -
4       System  0x840e9a945de0  0x2490  Event   0x1f0003        -
4       System  0x840e98a27e00  0x2494  IoCompletion    0x1f0003        -
4       System  0x840e90b0d080  0x2498  Process 0x102a  svchost.exe Pid 9452
4       System  0x840e9ba11610  0x249c  WaitCompletionPacket    0x1     -
4       System  0x840e98c290e0  0x24a0  Event   0x1f0003        -
4       System  0x840e9909b7c0  0x24a4  WaitCompletionPacket    0x1     -
4       System  0x840e98b50390  0x24a8  WaitCompletionPacket    0x1     -
4       System  0x840e990ad0e0  0x24ac  WaitCompletionPacket    0x1     -
4       System  0x840e91ee7080  0x24b0  Process 0x102a  Sysmon64.exe Pid 1540
4       System  0x840e966735a0  0x24b4  WaitCompletionPacket    0x1     -
4       System  0x840e9e3d0760  0x24b8  Event   0x1f0003        -
4       System  0x840e9b9b9910  0x24bc  WaitCompletionPacket    0x1     -
4       System  0x840e978cd940  0x24c0  WaitCompletionPacket    0x1     -
4       System  0x840e9c47d3c0  0x24c4  IoCompletion    0x1f0003        -
4       System  0x840e998251c0  0x24cc  Process 0x0     msedge.exe Pid 9728
4       System  0xb70820e89060  0x24d0  Token   0xf01ff -
4       System  0x840e978d8650  0x24d8  WaitCompletionPacket    0x1     -
4       System  0x840e967e4460  0x24dc  Event   0x1f0003        -
4       System  0x840e9c252080  0x24e0  Process 0x102a  RuntimeBroker. Pid 10648
4       System  0x840e92cfa140  0x24e4  Process 0x102a  dwm.exe Pid 1204
4       System  0x840e9b99b810  0x24e8  WaitCompletionPacket    0x1     -
4       System  0x840e990aac50  0x24ec  WaitCompletionPacket    0x1     -
4       System  0x840e91bc7b00  0x24f0  FilterConnectionPort    0x1f0001        MicrosoftSecFilterControlPort
4       System  0x840e98fe5160  0x24f4  Event   0x1f0003        -
4       System  0x840e970ca9c0  0x24f8  IoCompletion    0x1f0003        -
4       System  0x840e9c89f460  0x24fc  Event   0x1f0003        -
4       System  0x840e923f41c0  0x2500  Process 0x0     fontdrvhost.ex Pid 800
4       System  0x840e90e4d080  0x2504  Thread  0x1fffff        Tid 6536 Pid 4
4       System  0x840e978e50a0  0x2508  WaitCompletionPacket    0x1     -
4       System  0x840e98e86960  0x250c  Event   0x1f0003        -
4       System  0x840e995680c0  0x2510  Process 0x0     SgrmBroker.exe Pid 9948
4       System  0x840e990374c0  0x2514  IoCompletion    0x1f0003        -
4       System  0x840e9e5a11c0  0x2518  Process 0x0     uhssvc.exe Pid 11740
4       System  0x840e91ba00c0  0x251c  Process 0x0     svchost.exe Pid 7220
4       System  0x840ea534e7e0  0x2520  Event   0x1f0003        -
4       System  0x840e96d4d860  0x2524  Timer   0x1f0003        -
4       System  0x840e9dabd900  0x2528  IoCompletion    0x1f0003        -
4       System  0x840e98077e10  0x252c  CoreMessaging   0xf0000 -
4       System  0x840e98c450c0  0x2530  Process 0x102a  dllhost.exe Pid 7800
4       System  0x840e90b0d080  0x2534  Process 0x102a  svchost.exe Pid 9452
4       System  0x840e92291140  0x2538  Process 0x0     csrss.exe Pid 888
4       System  0x840e9c846060  0x253c  Event   0x1f0003        -
4       System  0x840e970c78c0  0x2540  IoCompletion    0x1f0003        -
4       System  0x840e9b99fd20  0x2544  WaitCompletionPacket    0x1     -
4       System  0x840ea9ed2560  0x2548  Event   0x1f0003        -
4       System  0x840e9969e080  0x254c  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e9969e080  0x2550  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e9909b070  0x2554  WaitCompletionPacket    0x1     -
4       System  0x840e9ac234c0  0x2558  WaitCompletionPacket    0x1     -
4       System  0x840e9b7bb080  0x255c  Process 0x102a  StartMenuExper Pid 8
4       System  0x840e9b9b3110  0x2564  WaitCompletionPacket    0x1     -
4       System  0x840e90e10410  0x2568  Session 0x0     Session2
4       System  0x840e96672be0  0x256c  WaitCompletionPacket    0x1     -
4       System  0x840e9c8babf0  0x2570  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.1.regtrans-ms
4       System  0x840e997052e0  0x2574  Event   0x1f0003        -
4       System  0x840e9c9053e0  0x2578  Event   0x1f0003        -
4       System  0x840e9ac34de0  0x257c  WaitCompletionPacket    0x1     -
4       System  0x840e9952c080  0x2580  Process 0x40    csrss.exe Pid 10732
4       System  0x840e98bdf080  0x2584  Process 0x1fffff        SystemSettings Pid 10020
4       System  0x840e98e84860  0x2588  Event   0x1f0003        -
4       System  0x840ea0882780  0x258c  IoCompletion    0x1f0003        -
4       System  0x840e9c252080  0x2590  Process 0x102a  RuntimeBroker. Pid 10648
4       System  0x840e98bdf080  0x2594  Process 0x40    SystemSettings Pid 10020
4       System  0x840e98bba080  0x2598  Process 0x40    SearchApp.exe Pid 7932
4       System  0x840e9c895de0  0x259c  Event   0x1f0003        -
4       System  0x840e98c29b60  0x25a0  Event   0x1f0003        -
4       System  0x840e9909fe70  0x25a4  WaitCompletionPacket    0x1     -
4       System  0x840e92f27080  0x25a8  Process 0x0     svchost.exe Pid 1808
4       System  0x840e993384e0  0x25ac  Event   0x1f0003        -
4       System  0x840e98d113c0  0x25b0  IoCompletion    0x1f0003        -
4       System  0x840e98b44a50  0x25b4  WaitCompletionPacket    0x1     -
4       System  0x840e993fb680  0x25b8  TmRm    0x1f007f        -
4       System  0x840e98185130  0x25bc  WaitCompletionPacket    0x1     -
4       System  0x840e9952c080  0x25c4  Process 0x0     csrss.exe Pid 10732
4       System  0x840e98ec91c0  0x25c8  Process 0x0     RuntimeBroker. Pid 10968
4       System  0x840e9dacb2c0  0x25cc  IoCompletion    0x1f0003        -
4       System  0x840e92ee0990  0x25d4  WaitCompletionPacket    0x1     -
4       System  0x840e92297960  0x25d8  Event   0x1f0003        -
4       System  0x840e9832a1c0  0x25dc  Process 0x102a  msedge.exe Pid 3832
4       System  0x840e90bcff60  0x25e0  Event   0x1f0003        -
4       System  0x840e97145a90  0x25e4  WaitCompletionPacket    0x1     -
4       System  0x840e98d21180  0x25e8  IoCompletion    0x1f0003        -
4       System  0x840e9f3b7d00  0x25ec  IoCompletion    0x1f0003        -
4       System  0x840e9c9adc40  0x25f0  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Photos_2019.19071.12548.0_x64__8wekyb3d8bbwe\ActivationStore.dat.LOG1
4       System  0x840e99ab9140  0x25f8  IoCompletion    0x1f0003        -
4       System  0x840e98a82270  0x25fc  IoCompletionReserve     0xf0003 -
4       System  0x840e98a7ec70  0x2600  IoCompletionReserve     0xf0003 -
4       System  0x840e97e8c080  0x2604  Process 0x102a  svchost.exe Pid 3864
4       System  0xb7081a5c6180  0x2608  SymbolicLink    0xf0001 Session
4       System  0x840e990adaa0  0x260c  WaitCompletionPacket    0x1     -
4       System  0x840e9818d190  0x2610  WaitCompletionPacket    0x1     -
4       System  0x840e9904cc80  0x2614  IoCompletion    0x1f0003        -
4       System  0x840e96f5a080  0x2618  Process 0x1fffff        SenseNdr.exe Pid 15568
4       System  0x840e9907c680  0x261c  WaitCompletionPacket    0x1     -
4       System  0x840e98db8080  0x2620  Process 0x102a  MsSense.exe Pid 10224
4       System  0x840e99ae2b00  0x2624  IoCompletion    0x1f0003        -
4       System  0x840e9b807a30  0x2628  IoCompletionReserve     0xf0003 -
4       System  0x840e9e0b3040  0x262c  Thread  0x1fffff        Tid 6676 Pid 4
4       System  0x840e98bcace0  0x2630  ALPC Port       0x1f0001        -
4       System  0x840e978c0300  0x2634  Process 0x40    explorer.exe Pid 5328
4       System  0x840e98db4080  0x2638  Process 0x0     msedge.exe Pid 9888
4       System  0x840e98f07080  0x263c  Process 0x102a  msedge.exe Pid 9172
4       System  0x840e98b3b5a0  0x2640  WaitCompletionPacket    0x1     -
4       System  0x840e98fe2c60  0x2644  Event   0x1f0003        -
4       System  0x840e90c63df0  0x2648  WaitCompletionPacket    0x1     -
4       System  0x840e98d201c0  0x264c  IoCompletion    0x1f0003        -
4       System  0x840e98c2fc60  0x2650  Event   0x1f0003        -
4       System  0x840e9230b100  0x265c  Process 0x0     services.exe Pid 1000
4       System  0x840e990246c0  0x2660  IoCompletion    0x1f0003        -
4       System  0x840e98e861e0  0x2664  Event   0x1f0003        -
4       System  0x840e997450c0  0x2668  Process 0x102a  svchost.exe Pid 3052
4       System  0x840e92c4fde0  0x2670  WaitCompletionPacket    0x1     -
4       System  0x840e9ea14080  0x2674  Process 0x102a  ScreenConnect. Pid 1772
4       System  0xb70805fad060  0x2678  Token   0xf01ff -
4       System  0x840e91ba00c0  0x267c  Process 0x102a  svchost.exe Pid 7220
4       System  0x840e9c8968e0  0x2680  Event   0x1f0003        -
4       System  0x840e9b160b10  0x2684  File    0x100001        \Device\0000002e
4       System  0x840e98b56e00  0x2688  WaitCompletionPacket    0x1     -
4       System  0x840e90bce7e0  0x268c  Event   0x1f0003        -
4       System  0x840e9ac0e2c0  0x2690  WaitCompletionPacket    0x1     -
4       System  0x840e98c322e0  0x2694  Event   0x1f0003        -
4       System  0x840e9715d080  0x2698  Process 0x0     Sysmon64.exe Pid 3696
4       System  0x840e9f7a8140  0x269c  IoCompletion    0x1f0003        -
4       System  0x840e9229cee0  0x26a0  Event   0x1f0003        -
4       System  0x840e96dd3f60  0x26a4  Event   0x1f0003        -
4       System  0x840e9c8962e0  0x26a8  Event   0x1f0003        -
4       System  0x840e9769f080  0x26ac  Process 0x1fffff        winlogon.exe Pid 2804
4       System  0x840e92cab2c0  0x26b0  Process 0x0     svchost.exe Pid 1124
4       System  0x840e981ae760  0x26b4  WaitCompletionPacket    0x1     -
4       System  0x840e98fd0160  0x26b8  Event   0x1f0003        -
4       System  0x840e98d3d440  0x26bc  IoCompletion    0x1f0003        -
4       System  0x840e92eaa2c0  0x26c4  Process 0x0     svchost.exe Pid 1604
4       System  0x840e9856b2c0  0x26d0  Process 0x0     svchost.exe Pid 7780
4       System  0x840e9682f080  0x26d4  Process 0x40    OneDrive.exe Pid 3320
4       System  0x840e981a5380  0x26d8  WaitCompletionPacket    0x1     -
4       System  0x840e98f07080  0x26dc  Process 0x1fffff        msedge.exe Pid 9172
4       System  0x840e976752c0  0x26e4  IoCompletion    0x1f0003        -
4       System  0x840e971396c0  0x26f0  WaitCompletionPacket    0x1     -
4       System  0x840e9a606d80  0x26f4  IoCompletion    0x1f0003        -
4       System  0x840e90e10410  0x26f8  Session 0x0     Session2
4       System  0x840e98bba080  0x26fc  Process 0x40    SearchApp.exe Pid 7932
4       System  0x840e98e832e0  0x2700  Event   0x1f0003        -
4       System  0xb70834703950  0x2704  Token   0xf01ff -
4       System  0x840e98d3c480  0x2708  IoCompletion    0x1f0003        -
4       System  0x840e98bba080  0x270c  Process 0x0     SearchApp.exe Pid 7932
4       System  0x840e9979eae0  0x2710  Event   0x1f0003        -
4       System  0x840e98b2c6c0  0x2714  WaitCompletionPacket    0x1     -
4       System  0x840e9c8977e0  0x2718  Event   0x1f0003        -
4       System  0x840e99092da0  0x271c  WaitCompletionPacket    0x1     -
4       System  0x840e98e8aae0  0x2720  Event   0x1f0003        -
4       System  0x840e993240c0  0x2724  Process 0x102a  sihost.exe Pid 10384
4       System  0x840e9979cd60  0x2728  Event   0x1f0003        -
4       System  0x840e98d391c0  0x272c  IoCompletion    0x1f0003        -
4       System  0x840e9c52f4e0  0x2730  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0x840ea47d2d40  0x2734  WaitCompletionPacket    0x1     -
4       System  0x840e92f04340  0x2738  Process 0x0     svchost.exe Pid 1652
4       System  0x840e9c47dd40  0x273c  IoCompletion    0x1f0003        -
4       System  0x840e9c84dbe0  0x2740  Event   0x1f0003        -
4       System  0x840e981aac10  0x274c  WaitCompletionPacket    0x1     -
4       System  0x840e9c527ca0  0x2750  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000002.regtrans-ms
4       System  0x840e92e539e0  0x2754  Event   0x100000        -
4       System  0x840e99abf880  0x2758  IoCompletion    0x1f0003        -
4       System  0x840e92eac340  0x275c  Process 0x0     svchost.exe Pid 1612
4       System  0x840e9b985500  0x2760  WaitCompletionPacket    0x1     -
4       System  0x840e978c0300  0x2768  Process 0x102a  explorer.exe Pid 5328
4       System  0x840e96e552c0  0x2770  Process 0x102a  svchost.exe Pid 3560
4       System  0x840e978c0300  0x2774  Process 0x102a  explorer.exe Pid 5328
4       System  0x840e9c92b5e0  0x2778  Event   0x1f0003        -
4       System  0x840e98d19e00  0x277c  IoCompletion    0x1f0003        -
4       System  0x840e90bb8c60  0x2780  Event   0x1f0003        -
4       System  0x840e98c2eb60  0x2784  Event   0x1f0003        -
4       System  0x840e984072b0  0x2788  IoCompletionReserve     0xf0003 -
4       System  0x840e98b601e0  0x2790  WaitCompletionPacket    0x1     -
4       System  0x840e9b7bb080  0x2794  Process 0x1fffff        StartMenuExper Pid 8
4       System  0x840e98b447e0  0x2798  WaitCompletionPacket    0x1     -
4       System  0x840e9956e7d0  0x279c  TmRm    0x1f007f        -
4       System  0x840e978dfc20  0x27a0  WaitCompletionPacket    0x1     -
4       System  0x840e982f2ce0  0x27a4  ALPC Port       0x1f0001        -
4       System  0x840e90c19340  0x27a8  Process 0x102a  svchost.exe Pid 3604
4       System  0x840e90c19340  0x27ac  Process 0x102a  svchost.exe Pid 3604
4       System  0x840e96c61080  0x27b0  Process 0x0     svchost.exe Pid 2216
4       System  0xb70812ba6430  0x27b4  Section 0x17    -
4       System  0x840e96c582c0  0x27b8  Process 0x0     svchost.exe Pid 3028
4       System  0x840e96e552c0  0x27bc  Process 0x102a  svchost.exe Pid 3560
4       System  0x840e998251c0  0x27c0  Process 0x102a  msedge.exe Pid 9728
4       System  0x840e992ccec0  0x27c4  IoCompletion    0x1f0003        -
4       System  0x840e90bc18e0  0x27c8  Event   0x1f0003        -
4       System  0x840e90de2080  0x27cc  Process 0x1fffff        svchost.exe Pid 1480
4       System  0x840e92f23080  0x27d4  Process 0x0     svchost.exe Pid 1776
4       System  0x840e98d0dbc0  0x27d8  IoCompletion    0x1f0003        -
4       System  0x840e9a325a30  0x27dc  IoCompletionReserve     0xf0003 -
4       System  0x840e90bc3560  0x27e0  Event   0x1f0003        -
4       System  0x840e992d0cc0  0x27e4  IoCompletion    0x1f0003        -
4       System  0x840e978d2190  0x27e8  WaitCompletionPacket    0x1     -
4       System  0x840e978ce230  0x27ec  WaitCompletionPacket    0x1     -
4       System  0x840e9933bd60  0x27f0  Event   0x1f0003        -
4       System  0x840e90b0d080  0x27f4  Process 0x102a  svchost.exe Pid 9452
4       System  0x840e9ea14080  0x27f8  Process 0x102a  ScreenConnect. Pid 1772
4       System  0xb7081a5c91a0  0x2804  SymbolicLink    0xf0001 Global
4       System  0x840e978e9270  0x2808  WaitCompletionPacket    0x1     -
4       System  0x840e99f4f740  0x280c  IoCompletion    0x1f0003        -
4       System  0x840e9798adb0  0x2810  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e98bba080  0x2814  Process 0x40    SearchApp.exe Pid 7932
4       System  0x840e98bba080  0x281c  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e99083360  0x2820  WaitCompletionPacket    0x1     -
4       System  0xb7081792d450  0x2824  Section 0x17    -
4       System  0x840e96f6d2c0  0x2828  Process 0x0     svchost.exe Pid 3880
4       System  0x840e992e47c0  0x282c  IoCompletion    0x1f0003        -
4       System  0x840e981b5370  0x2830  WaitCompletionPacket    0x1     -
4       System  0xb7081120ac90  0x2834  SymbolicLink    0xf0001 Local
4       System  0x840ea3ce2f60  0x2838  Event   0x1f0003        -
4       System  0x840e981800c0  0x283c  WaitCompletionPacket    0x1     -
4       System  0x840e90bcb960  0x2840  Event   0x1f0003        -
4       System  0x840e98bdf080  0x2844  Process 0x102a  SystemSettings Pid 10020
4       System  0x840e9681d9d0  0x2848  WaitCompletionPacket    0x1     -
4       System  0xb70821b0e790  0x284c  Section 0x17    -
4       System  0x840e9b9baa20  0x2850  WaitCompletionPacket    0x1     -
4       System  0x840e9c896f60  0x2854  Event   0x1f0003        -
4       System  0x840e990817c0  0x2858  WaitCompletionPacket    0x1     -
4       System  0x840e98182f10  0x285c  WaitCompletionPacket    0x1     -
4       System  0x840e9b9b18b0  0x2860  WaitCompletionPacket    0x1     -
4       System  0x840e978de150  0x2864  WaitCompletionPacket    0x1     -
4       System  0x840e99f66dc0  0x2868  IoCompletion    0x1f0003        -
4       System  0x840e978e43a0  0x286c  WaitCompletionPacket    0x1     -
4       System  0x840e98db4080  0x2870  Process 0x102a  msedge.exe Pid 9888
4       System  0x840e98e93fe0  0x2874  Event   0x1f0003        -
4       System  0x840e992ddc40  0x2878  IoCompletion    0x1f0003        -
4       System  0x840e97ad9cb0  0x287c  ALPC Port       0x1f0001        -
4       System  0x840e90de2080  0x2880  Process 0x102a  svchost.exe Pid 1480
4       System  0x840e99ab85c0  0x2884  IoCompletion    0x1f0003        -
4       System  0x840e996e91c0  0x2888  Process 0x102a  msedge.exe Pid 9016
4       System  0x840e98d0c500  0x288c  IoCompletion    0x1f0003        -
4       System  0x840e98df3080  0x2890  Process 0x0     ShellExperienc Pid 704
4       System  0x840e9b9b7140  0x2894  WaitCompletionPacket    0x1     -
4       System  0xb7081129cb80  0x2898  Directory       0xf     S-1-15-2-515815643-2845804217-1874292103-218650560-777617685-4287762684-137415000
4       System  0x840e9ba0c8e0  0x289c  WaitCompletionPacket    0x1     -
4       System  0x840ea17e9080  0x28a0  Process 0x1fffff        svchost.exe Pid 10392
4       System  0x840e90bc6560  0x28a4  Event   0x1f0003        -
4       System  0x840e992d3c00  0x28a8  IoCompletion    0x1f0003        -
4       System  0x840e97146a00  0x28ac  WaitCompletionPacket    0x1     -
4       System  0x840e9a14f210  0x28b0  File    0x3     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-5-21-3294271487-4292050549-1661813568-1001\SystemAppData\Helium\Cache\b4d622c333fcfdd7_COM15.dat.LOG1
4       System  0x840e98a2ff80  0x28b4  IoCompletion    0x1f0003        -
4       System  0x840e997abee0  0x28b8  Event   0x1f0003        -
4       System  0xb7081129c5e0  0x28bc  Directory       0xf     RPC Control
4       System  0x840e978e3430  0x28c0  WaitCompletionPacket    0x1     -
4       System  0x840e9666fa50  0x28c4  WaitCompletionPacket    0x1     -
4       System  0x840e97654500  0x28c8  IoCompletion    0x1f0003        -
4       System  0x840e90bc9860  0x28cc  Event   0x1f0003        -
4       System  0x840e9c554940  0x28d0  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\UserClasses.dat.LOG2
4       System  0x840e978c0300  0x28d4  Process 0x40    explorer.exe Pid 5328
4       System  0x840e996cf080  0x28d8  Process 0x1fffff        svchost.exe Pid 9320
4       System  0x840e9676e080  0x28dc  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e98582040  0x28e0  Thread  0x1fffff        Tid 15852 Pid 4
4       System  0x840e998251c0  0x28e4  Process 0x102a  msedge.exe Pid 9728
4       System  0x840e98a02b00  0x28e8  IoCompletion    0x1f0003        -
4       System  0x840e992dc580  0x28ec  IoCompletion    0x1f0003        -
4       System  0x840e9933bf60  0x28f0  Event   0x1f0003        -
4       System  0x840e9ba24380  0x28f4  WaitCompletionPacket    0x1     -
4       System  0x840ea70570c0  0x28f8  Process 0x102a  msedge.exe Pid 1336
4       System  0x840e99f67f40  0x28fc  IoCompletion    0x1f0003        -
4       System  0x840e98e92160  0x2900  Event   0x1f0003        -
4       System  0x840e97989e10  0x2904  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\MicrosoftWindows.Client.CBS_1000.19058.1000.0_x64__cw5n1h2txyewy\ActivationStore.dat.LOG2
4       System  0x840e90de2080  0x2908  Process 0x102a  svchost.exe Pid 1480
4       System  0x840e978c0300  0x290c  Process 0x102a  explorer.exe Pid 5328
4       System  0x840e976577c0  0x2910  IoCompletion    0x1f0003        -
4       System  0x840e98b332d0  0x2918  WaitCompletionPacket    0x1     -
4       System  0x840e98a02a40  0x291c  IoCompletion    0x1f0003        -
4       System  0x840e9c862260  0x2920  Event   0x1f0003        -
4       System  0x840e92f202c0  0x2924  Process 0x0     svchost.exe Pid 1764
4       System  0x840e9a14b520  0x2928  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.ShellExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat
4       System  0x840e992cd400  0x292c  IoCompletion    0x1f0003        -
4       System  0x840e9ac0eae0  0x2930  WaitCompletionPacket    0x1     -
4       System  0x840e98df3080  0x2934  Process 0x102a  ShellExperienc Pid 704
4       System  0x840e90bc2b60  0x2938  Event   0x1f0003        -
4       System  0x840e9c9ac3e0  0x293c  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Photos_2019.19071.12548.0_x64__8wekyb3d8bbwe\ActivationStore.dat
4       System  0xb707fe074060  0x2940  Token   0xf01ff -
4       System  0x840e90bc6260  0x2948  Event   0x1f0003        -
4       System  0x840e992d6440  0x294c  IoCompletion    0x1f0003        -
4       System  0x840e978d05f0  0x2950  WaitCompletionPacket    0x1     -
4       System  0x840e98d3f200  0x2954  IoCompletion    0x1f0003        -
4       System  0x840e90de2080  0x295c  Process 0x102a  svchost.exe Pid 1480
4       System  0x840e97c49ae0  0x2960  Event   0x1f0003        -
4       System  0x840e97fc1560  0x2964  Event   0x1f0003        -
4       System  0x840e99038d40  0x2968  IoCompletion    0x1f0003        -
4       System  0xb7081a5a9240  0x296c  Directory       0xf     S-1-15-2-155514346-2573954481-755741238-1654018636-1233331829-3075935687-2861478708
4       System  0x840e99336c60  0x2970  Event   0x1f0003        -
4       System  0x840e90de2080  0x2974  Process 0x102a  svchost.exe Pid 1480
4       System  0x840e98d0cf80  0x2978  IoCompletion    0x1f0003        -
4       System  0x840e9a147830  0x297c  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\User.dat
4       System  0x840e9979af60  0x2980  Event   0x1f0003        -
4       System  0x840e9bcd1bb0  0x2984  IoCompletionReserve     0xf0003 -
4       System  0x840e9cf2c6c0  0x2988  IoCompletion    0x1f0003        -
4       System  0x840e99fd7760  0x298c  Event   0x1f0003        -
4       System  0x840e9ac3d800  0x2994  WaitCompletionPacket    0x1     -
4       System  0x840e98a1dd00  0x2998  IoCompletion    0x1f0003        -
4       System  0x840e9969e080  0x299c  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e97bab850  0x29a0  File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-515815643-2845804217-1874292103-218650560-777617685-4287762684-137415000
4       System  0x840e98fb61c0  0x29a4  Process 0x1fffff        dllhost.exe Pid 8472
4       System  0x840e9979a1e0  0x29a8  Event   0x1f0003        -
4       System  0x840e9ac17c50  0x29ac  WaitCompletionPacket    0x1     -
4       System  0x840e99387080  0x29b0  Process 0x1fffff        StartMenuExper Pid 7408
4       System  0x840e9a625240  0x29b4  IoCompletion    0x1f0003        -
4       System  0x840e98a10780  0x29bc  IoCompletion    0x1f0003        -
4       System  0x840e98a0f980  0x29c0  IoCompletion    0x1f0003        -
4       System  0x840e981a7b50  0x29c4  WaitCompletionPacket    0x1     -
4       System  0x840e98f07080  0x29cc  Process 0x102a  msedge.exe Pid 9172
4       System  0xb7081a5c51b0  0x29d0  SymbolicLink    0xf0001 Global
4       System  0x840e97465360  0x29d4  Event   0x1f0003        -
4       System  0x840e999e81e0  0x29d8  Event   0x1f0003        -
4       System  0x840e9781b160  0x29e0  Event   0x1f0003        -
4       System  0x840e9cb56a00  0x29e4  IoCompletion    0x1f0003        -
4       System  0x840e9908a110  0x29e8  WaitCompletionPacket    0x1     -
4       System  0x840e98c18460  0x29ec  Event   0x1f0003        -
4       System  0x840e9927e8f0  0x29f0  IoCompletionReserve     0xf0003 -
4       System  0x840e99387080  0x29f4  Process 0x102a  StartMenuExper Pid 7408
4       System  0x840e99ab8780  0x29f8  IoCompletion    0x1f0003        -
4       System  0x840e9e902340  0x29fc  Process 0x102a  msedge.exe Pid 5808
4       System  0x840e9c937260  0x2a00  Event   0x1f0003        -
4       System  0x840e9798a450  0x2a04  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Search_1.14.15.19041_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat
4       System  0x840e91bcf3f0  0x2a08  WaitCompletionPacket    0x1     -
4       System  0x840e97381300  0x2a0c  IoCompletion    0x1f0003        -
4       System  0x840e99f51b00  0x2a10  IoCompletion    0x1f0003        -
4       System  0x840e91dde840  0x2a14  Timer   0x2     -
4       System  0x840e96d4b340  0x2a18  Timer   0x1f0003        -
4       System  0x840e97d73980  0x2a1c  IoCompletion    0x1f0003        -
4       System  0x840e99aae780  0x2a20  IoCompletion    0x1f0003        -
4       System  0x840e92c52dd0  0x2a24  WaitCompletionPacket    0x1     -
4       System  0x840e99036940  0x2a28  IoCompletion    0x1f0003        -
4       System  0x840e985d8080  0x2a2c  Process 0x40    ShellExperienc Pid 7588
4       System  0x840e976a2300  0x2a30  Process 0x0     unsecapp.exe Pid 5220
4       System  0x840e91ee7080  0x2a34  Process 0x0     Sysmon64.exe Pid 1540
4       System  0xb708106add40  0x2a38  Directory       0xf000f 00000000-0258f3ec
4       System  0x840e99fd6a60  0x2a3c  Event   0x1f0003        -
4       System  0x840e981af2c0  0x2a40  WaitCompletionPacket    0x1     -
4       System  0x840e99a89080  0x2a48  Process 0x102a  svchost.exe Pid 11756
4       System  0x840e98f07080  0x2a4c  Process 0x102a  msedge.exe Pid 9172
4       System  0x840e91d8a360  0x2a50  Event   0x1f0003        -
4       System  0x840e978c0300  0x2a54  Process 0x40    explorer.exe Pid 5328
4       System  0x840e98bd6240  0x2a58  Process 0x1fffff        RuntimeBroker. Pid 8120
4       System  0x840e97a764a0  0x2a5c  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.StartMenuExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat
4       System  0x840e90de9080  0x2a60  Process 0x0     FileCoAuth.exe Pid 8592
4       System  0x840e9903f1c0  0x2a6c  IoCompletion    0x1f0003        -
4       System  0x840e9c716260  0x2a74  Event   0x1f0003        -
4       System  0x840e99fcd9e0  0x2a78  Event   0x1f0003        -
4       System  0xb70811698bc0  0x2a7c  Directory       0xf     RPC Control
4       System  0x840e98b26a20  0x2a80  WaitCompletionPacket    0x1     -
4       System  0x840e9681d4f0  0x2a84  WaitCompletionPacket    0x1     -
4       System  0x840e9c71c3e0  0x2a88  Event   0x1f0003        -
4       System  0x840e923de2c0  0x2a8c  Process 0x0     svchost.exe Pid 756
4       System  0x840e97a77da0  0x2a90  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.StartMenuExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG1
4       System  0x840e98191360  0x2a94  WaitCompletionPacket    0x1     -
4       System  0x840e98c1ade0  0x2a98  Event   0x1f0003        -
4       System  0x840e9c545a60  0x2a9c  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-95739096-486727260-2033287795-3853587803-1685597119-444378811-2746676523
4       System  0x840e9798d970  0x2aa0  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e96f7e920  0x2aa4  File    0x12008b        \Device\HarddiskVolume3\ProgramData\Microsoft\Windows Security Health\Logs\SHS-03122025-104154-7-7f-19041.1.amd64fre.vb_release.191206-1406.etl
4       System  0x840e98b3f500  0x2aac  WaitCompletionPacket    0x1     -
4       System  0x840e97e9daa0  0x2ab0  CoreMessaging   0xf0000 -
4       System  0x840e9934bce0  0x2ab4  Event   0x1f0003        -
4       System  0x840e9a60e200  0x2ab8  IoCompletion    0x1f0003        -
4       System  0x840e9783b080  0x2ac0  Process 0x0     CSFalconContai Pid 6604
4       System  0x840e980783b0  0x2ac4  CoreMessaging   0xf0000 -
4       System  0x840e967def60  0x2ac8  Event   0x1f0003        -
4       System  0x840e96904810  0x2acc  IoCompletionReserve     0xf0003 -
4       System  0x840e9676b300  0x2ad0  Process 0x0     svchost.exe Pid 2180
4       System  0x840e9904eb40  0x2ad4  IoCompletion    0x1f0003        -
4       System  0x840e9904cbc0  0x2ad8  IoCompletion    0x1f0003        -
4       System  0x840e99da3080  0x2adc  Process 0x102a  RuntimeBroker. Pid 4464
4       System  0x840e9037c040  0x2ae0  Process 0x0     Registry Pid 108
4       System  0x840e97146380  0x2ae4  WaitCompletionPacket    0x1     -
4       System  0x840e9969e080  0x2ae8  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e98a16a80  0x2aec  IoCompletion    0x1f0003        -
4       System  0x840e98b28f80  0x2af4  WaitCompletionPacket    0x1     -
4       System  0x840e9ec9fa60  0x2af8  File    0x3     \Device\HarddiskVolume3\Windows\ServiceProfiles\NetworkService\AppData\Local\Microsoft\Windows\DeliveryOptimization\State\dosvcState.dat.LOG2
4       System  0x840e98bba080  0x2afc  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e9a62b600  0x2b00  IoCompletion    0x1f0003        -
4       System  0x840e99fdb0e0  0x2b04  Event   0x1f0003        -
4       System  0x840e9c36f8e0  0x2b08  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e99083840  0x2b0c  WaitCompletionPacket    0x1     -
4       System  0x840e96c5e1c0  0x2b10  Process 0x0     svchost.exe Pid 2220
4       System  0x840e98b8e340  0x2b14  Process 0x0     RuntimeBroker. Pid 7612
4       System  0x840e9934d4e0  0x2b18  Event   0x1f0003        -
4       System  0x840e9b6cc4b0  0x2b28  IoCompletionReserve     0xf0003 -
4       System  0x840e9c5a7ee0  0x2b2c  Event   0x1f0003        -
4       System  0x840e9dabaf00  0x2b34  IoCompletion    0x1f0003        -
4       System  0x840e97a75820  0x2b38  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.StartMenuExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG2
4       System  0x840e98ff32c0  0x2b44  Process 0x102a  SecurityHealth Pid 8808
4       System  0x840e99b263e0  0x2b48  File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-155514346-2573954481-755741238-1654018636-1233331829-3075935687-2861478708
4       System  0x840e9c8b69f0  0x2b4c  File    0x12008b        \Device\HarddiskVolume3\Windows\Logs\WindowsUpdate\WindowsUpdate.20250313.155518.080.4.etl
4       System  0x840e9b739080  0x2b50  Process 0x0     Microsoft.Phot Pid 11148
4       System  0x840e999ef260  0x2b58  Event   0x1f0003        -
4       System  0x840e99043540  0x2b5c  IoCompletion    0x1f0003        -
4       System  0x840e98e742c0  0x2b64  Process 0x1fffff        svchost.exe Pid 8292
4       System  0x840e98c13de0  0x2b68  Event   0x1f0003        -
4       System  0x840e9798bd50  0x2b6c  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.Search_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e998450c0  0x2b70  Process 0x0     SearchApp.exe Pid 4236
4       System  0x840e98bba080  0x2b74  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e9ba08300  0x2b78  WaitCompletionPacket    0x1     -
4       System  0x840e990ab880  0x2b7c  WaitCompletionPacket    0x1     -
4       System  0x840e90b0d080  0x2b80  Process 0x102a  svchost.exe Pid 9452
4       System  0x840e974b2080  0x2b84  Process 0x0     UserOOBEBroker Pid 9620
4       System  0x840e99387080  0x2b88  Process 0x102a  StartMenuExper Pid 7408
4       System  0x840e98ea0360  0x2b8c  Event   0x1f0003        -
4       System  0x840e98b37710  0x2b90  WaitCompletionPacket    0x1     -
4       System  0x840e97136530  0x2b94  WaitCompletionPacket    0x1     -
4       System  0x840e98b291f0  0x2b98  WaitCompletionPacket    0x1     -
4       System  0x840e9952c080  0x2ba4  Process 0x1fffff        csrss.exe Pid 10732
4       System  0x840e98b56ed0  0x2ba8  WaitCompletionPacket    0x1     -
4       System  0x840e9b992360  0x2bac  WaitCompletionPacket    0x1     -
4       System  0xb708087bb5b0  0x2bb4  Section 0x5     -
4       System  0x840e99037300  0x2bb8  IoCompletion    0x1f0003        -
4       System  0xb70812d09fd0  0x2bbc  SymbolicLink    0xf0001 Session
4       System  0x840e99fd97e0  0x2bc0  Event   0x1f0003        -
4       System  0x840e9817ed40  0x2bc4  WaitCompletionPacket    0x1     -
4       System  0x840e9b372080  0x2bc8  Process 0x1fffff        taskhostw.exe Pid 10476
4       System  0x840e9909f650  0x2bcc  WaitCompletionPacket    0x1     -
4       System  0x840e9c5554a0  0x2bd0  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-2246530975-808720366-1776470054-230329187-4153223113-3550430174-4193313734
4       System  0x840e97b09080  0x2bd4  Process 0x102a  powershell.exe Pid 15172
4       System  0x840e92e30080  0x2bd8  Process 0x0     svchost.exe Pid 1392
4       System  0x840e97387f80  0x2bdc  IoCompletion    0x1f0003        -
4       System  0xb7081121aaf0  0x2be0  SymbolicLink    0xf0001 Global
4       System  0x840e97e9e0d0  0x2be4  CoreMessaging   0xf0000 -
4       System  0x840e97f7e080  0x2be8  Process 0x0     svchost.exe Pid 5772
4       System  0x840e98c48080  0x2bec  Process 0x1fffff        ApplicationFra Pid 4836
4       System  0x840e990a8bd0  0x2bf0  WaitCompletionPacket    0x1     -
4       System  0x840e997b7de0  0x2bf4  Event   0x1f0003        -
4       System  0x840e97e9e9d0  0x2bf8  CoreMessaging   0xf0000 -
4       System  0x840e9909d1c0  0x2bfc  WaitCompletionPacket    0x1     -
4       System  0x840e996e91c0  0x2c08  Process 0x1fffff        msedge.exe Pid 9016
4       System  0xb70816ebda00  0x2c0c  Directory       0xf     S-1-15-2-155514346-2573954481-755741238-1654018636-1233331829-3075935687-2861478708
4       System  0x840e923de2c0  0x2c10  Process 0x102a  svchost.exe Pid 756
4       System  0x840e98bd6240  0x2c14  Process 0x102a  RuntimeBroker. Pid 8120
4       System  0x840e921650c0  0x2c1c  Process 0x0     csrss.exe Pid 804
4       System  0x840e9b99fdf0  0x2c20  WaitCompletionPacket    0x1     -
4       System  0xb708106afd20  0x2c24  Directory       0xf     S-1-15-2-95739096-486727260-2033287795-3853587803-1685597119-444378811-2746676523
4       System  0x840e9c8632e0  0x2c2c  Event   0x1f0003        -
4       System  0x840e9909e130  0x2c34  WaitCompletionPacket    0x1     -
4       System  0x840e99014680  0x2c38  IoCompletion    0x1f0003        -
4       System  0x840e9a148640  0x2c3c  File    0x1     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-5-21-3294271487-4292050549-1661813568-1001\SystemAppData\Helium\Cache\b4d622c333fcfdd7.dat
4       System  0x840e98189160  0x2c40  WaitCompletionPacket    0x1     -
4       System  0x840e9a652870  0x2c44  IoCompletionReserve     0xf0003 -
4       System  0x840e9c896f60  0x2c48  Event   0x1f0003        -
4       System  0x840e9908b630  0x2c4c  WaitCompletionPacket    0x1     -
4       System  0x840e98aef080  0x2c50  Process 0x102a  RuntimeBroker. Pid 7684
4       System  0x840e9ac0ea10  0x2c54  WaitCompletionPacket    0x1     -
4       System  0x840e9969e080  0x2c58  Process 0x102a  svchost.exe Pid 4156
4       System  0xb70811872430  0x2c5c  SymbolicLink    0xf0001 Global
4       System  0x840e99fcc960  0x2c60  Event   0x1f0003        -
4       System  0x840e990eddc0  0x2c64  ALPC Port       0x1f0001        -
4       System  0x840e9a014b70  0x2c68  IoCompletionReserve     0xf0003 -
4       System  0x840e98d2d580  0x2c6c  IoCompletion    0x1f0003        -
4       System  0x840e97989960  0x2c70  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Search_1.14.15.19041_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG1
4       System  0x840e97139e10  0x2c78  WaitCompletionPacket    0x1     -
4       System  0xb7080879c7f0  0x2c7c  Section 0x5     -
4       System  0x840e97d55e80  0x2c80  IoCompletion    0x1f0003        -
4       System  0x840e9ccec080  0x2c84  Process 0x102a  svchost.exe Pid 11776
4       System  0x840e9713fb80  0x2c88  WaitCompletionPacket    0x1     -
4       System  0x840e9bbc9120  0x2c8c  File    0x1     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-12-1-419028606-1323772725-945580434-2117921032\SystemAppData\Helium\Cache\b4d622c333fcfdd7.dat
4       System  0x840e99387080  0x2c90  Process 0x102a  StartMenuExper Pid 7408
4       System  0x840e99adfa00  0x2c94  IoCompletion    0x1f0003        -
4       System  0x840e99bcb080  0x2c98  Process 0x102a  explorer.exe Pid 9852
4       System  0x840e99474080  0x2c9c  Process 0x102a  PSEXESVC.exe Pid 11296
4       System  0x840e99f696c0  0x2ca0  IoCompletion    0x1f0003        -
4       System  0x840e99783440  0x2ca4  Thread  0x0     Tid 6556 Pid 4
4       System  0x840e9a9574e0  0x2ca8  Event   0x1f0003        -
4       System  0x840e98b38340  0x2cac  WaitCompletionPacket    0x1     -
4       System  0x840e9a624540  0x2cb0  IoCompletion    0x1f0003        -
4       System  0xb70811218ec0  0x2cb4  SymbolicLink    0xf0001 Session
4       System  0x840e968c22c0  0x2cb8  Process 0x0     svchost.exe Pid 2504
4       System  0x840e9ba0cb50  0x2cc4  WaitCompletionPacket    0x1     -
4       System  0x840e990835d0  0x2cc8  WaitCompletionPacket    0x1     -
4       System  0x840e9958a0c0  0x2ccc  Process 0x102a  msedge.exe Pid 9128
4       System  0x840e92349080  0x2cd0  Process 0x102a  lsass.exe Pid 548
4       System  0x840e98c17c60  0x2cd4  Event   0x1f0003        -
4       System  0x840e98dbf2c0  0x2cd8  Process 0x102a  svchost.exe Pid 8688
4       System  0x840e966d9300  0x2cdc  Process 0x1fffff        svchost.exe Pid 8912
4       System  0x840e997b44e0  0x2ce0  Event   0x1f0003        -
4       System  0x840e9798a2c0  0x2ce4  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.Search_1.14.15.19041_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG2
4       System  0x840e9689a080  0x2ce8  Process 0x0     svchost.exe Pid 2472
4       System  0x840e910b4040  0x2cec  Process 0x0     smss.exe Pid 536
4       System  0x840e9c9be1e0  0x2cf0  File    0x1f0006        \Device\NamedPipe\Sessions\2\AppContainerNamedObjects\S-1-15-2-515815643-2845804217-1874292103-218650560-777617685-4287762684-137415000
4       System  0x840e9a14aa30  0x2cf4  File    0x20003 \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-5-21-3294271487-4292050549-1661813568-1001\SystemAppData\Helium\Cache\b4d622c333fcfdd7_COM15.dat
4       System  0x840e98c48080  0x2cf8  Process 0x0     ApplicationFra Pid 4836
4       System  0x840e9682f080  0x2cfc  Process 0x0     OneDrive.exe Pid 3320
4       System  0x840e9682f080  0x2d00  Process 0x1fffff        OneDrive.exe Pid 3320
4       System  0x840e9cf352c0  0x2d04  IoCompletion    0x1f0003        -
4       System  0x840e98bba080  0x2d08  Process 0x102a  SearchApp.exe Pid 7932
4       System  0x840e998450c0  0x2d0c  Process 0x40    SearchApp.exe Pid 4236
4       System  0x840e971385b0  0x2d14  WaitCompletionPacket    0x1     -
4       System  0x840e9cb38260  0x2d18  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\MicrosoftWindows.Client.CBS_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e98b297a0  0x2d1c  WaitCompletionPacket    0x1     -
4       System  0x840e98b3a220  0x2d20  WaitCompletionPacket    0x1     -
4       System  0x840e997a7ae0  0x2d24  Event   0x1f0003        -
4       System  0x840e98b15080  0x2d28  Process 0x102a  fontdrvhost.ex Pid 10704
4       System  0x840e98df3080  0x2d2c  Process 0x102a  ShellExperienc Pid 704
4       System  0x840e9ac24ec0  0x2d30  WaitCompletionPacket    0x1     -
4       System  0x840e9ac422c0  0x2d34  WaitCompletionPacket    0x1     -
4       System  0x840e99084540  0x2d38  WaitCompletionPacket    0x1     -
4       System  0x840e9ac2fbd0  0x2d40  WaitCompletionPacket    0x1     -
4       System  0x840e99ab78c0  0x2d44  IoCompletion    0x1f0003        -
4       System  0x840e999edee0  0x2d48  Event   0x1f0003        -
4       System  0x840e99c83080  0x2d4c  Process 0x102a  dwm.exe Pid 10628
4       System  0x840e979be390  0x2d54  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\MicrosoftWindows.Client.CBS_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e98b51a50  0x2d5c  WaitCompletionPacket    0x1     -
4       System  0x840e97391dc0  0x2d60  IoCompletion    0x1f0003        -
4       System  0x840e979911b0  0x2d64  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\MicrosoftWindows.Client.CBS_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e9937dd20  0x2d68  ALPC Port       0x1f0001        -
4       System  0x840e98192060  0x2d6c  WaitCompletionPacket    0x1     -
4       System  0x840e9b98a8b0  0x2d70  WaitCompletionPacket    0x1     -
4       System  0x840e98b5a950  0x2d74  WaitCompletionPacket    0x1     -
4       System  0x840ea534dae0  0x2d78  Event   0x1f0003        -
4       System  0x840e97582080  0x2d7c  Process 0x102a  msedge.exe Pid 16344
4       System  0x840e990a0b70  0x2d80  WaitCompletionPacket    0x1     -
4       System  0x840e923de2c0  0x2d84  Process 0x102a  svchost.exe Pid 756
4       System  0x840e9bee60c0  0x2d8c  Process 0x102a  svchost.exe Pid 12440
4       System  0x840e99093280  0x2d90  WaitCompletionPacket    0x1     -
4       System  0x840e996cf080  0x2d94  Process 0x102a  svchost.exe Pid 9320
4       System  0x840e978e12e0  0x2d98  WaitCompletionPacket    0x1     -
4       System  0x840e98fb61c0  0x2d9c  Process 0x102a  dllhost.exe Pid 8472
4       System  0x840e996cf080  0x2da0  Process 0x102a  svchost.exe Pid 9320
4       System  0x840e91d8ed60  0x2da4  Event   0x1f0003        -
4       System  0x840e90c13ac0  0x2da8  ALPC Port       0x1f0001        -
4       System  0x840e98d0c0c0  0x2dac  IoCompletion    0x1f0003        -
4       System  0x840e9a149db0  0x2db0  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\UserClasses.dat.LOG1
4       System  0x840e98db4080  0x2db4  Process 0x102a  msedge.exe Pid 9888
4       System  0x840e96d0c700  0x2db8  IoCompletion    0x1f0003        -
4       System  0x840e998450c0  0x2dbc  Process 0x102a  SearchApp.exe Pid 4236
4       System  0x840e9a14c4c0  0x2dc0  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.ShellExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG1
4       System  0x840e9c8961e0  0x2dc8  Event   0x1f0003        -
4       System  0x840e9856b2c0  0x2dcc  Process 0x102a  svchost.exe Pid 7780
4       System  0x840e98d0b540  0x2dd0  IoCompletion    0x1f0003        -
4       System  0x840e9a325d30  0x2dd4  IoCompletionReserve     0xf0003 -
4       System  0x840ea3ce2be0  0x2dd8  Event   0x1f0003        -
4       System  0x840e99ac8b80  0x2ddc  IoCompletion    0x1f0003        -
4       System  0x840e9951c1c0  0x2de0  Process 0x102a  msedge.exe Pid 8732
4       System  0x840e99075b40  0x2de4  WaitCompletionPacket    0x1     -
4       System  0x840e990ad4f0  0x2de8  WaitCompletionPacket    0x1     -
4       System  0x840e9908bf20  0x2dec  WaitCompletionPacket    0x1     -
4       System  0x840e99041b00  0x2df0  IoCompletion    0x1f0003        -
4       System  0x840e9b7bb080  0x2df4  Process 0x102a  StartMenuExper Pid 8
4       System  0x840e98bd6240  0x2df8  Process 0x102a  RuntimeBroker. Pid 8120
4       System  0x840e990aea10  0x2dfc  WaitCompletionPacket    0x1     -
4       System  0x840e96f38680  0x2e04  CoreMessaging   0xf0000 -
4       System  0x840e92fc5300  0x2e08  Process 0x0     svchost.exe Pid 1976
4       System  0x840e98b36a10  0x2e0c  WaitCompletionPacket    0x1     -
4       System  0x840e98fd2f60  0x2e10  Event   0x1f0003        -
4       System  0xb70811219de0  0x2e14  SymbolicLink    0xf0001 Local
4       System  0x840e97152ea0  0x2e1c  WaitCompletionPacket    0x1     -
4       System  0x840e96d4a920  0x2e20  Timer   0x1f0003        -
4       System  0x840e99f82940  0x2e24  IoCompletion    0x1f0003        -
4       System  0x840e9e463cc0  0x2e28  TmEn    0xf001f -
4       System  0x840e98e88960  0x2e2c  Event   0x1f0003        -
4       System  0x840e9ac2ed30  0x2e34  WaitCompletionPacket    0x1     -
4       System  0x840e98b50c80  0x2e38  WaitCompletionPacket    0x1     -
4       System  0x840e99f6b3c0  0x2e3c  IoCompletion    0x1f0003        -
4       System  0x840ea8d6a080  0x2e44  Process 0x1fffff        OpenHandleColl Pid 11408
4       System  0x840e99051a80  0x2e48  IoCompletion    0x1f0003        -
4       System  0xb70811697cc0  0x2e4c  Directory       0xf     S-1-15-2-536077884-713174666-1066051701-3219990555-339840825-1966734348-1611281757
4       System  0x840e90e10410  0x2e50  Session 0x0     Session2
4       System  0x840e98a30080  0x2e54  IoCompletion    0x1f0003        -
4       System  0x840e90e10410  0x2e58  Session 0x0     Session2
4       System  0x840e98c1fc60  0x2e5c  Event   0x1f0003        -
4       System  0x840e9969e080  0x2e60  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e9ba0dac0  0x2e64  WaitCompletionPacket    0x1     -
4       System  0x840e992c4d40  0x2e6c  IoCompletion    0x1f0003        -
4       System  0xb7080a748a30  0x2e70  Section 0x17    -
4       System  0x840e97d586c0  0x2e74  IoCompletion    0x1f0003        -
4       System  0x840e90bb8160  0x2e78  Event   0x1f0003        -
4       System  0x840e9713a490  0x2e7c  WaitCompletionPacket    0x1     -
4       System  0x840e90bcb760  0x2e80  Event   0x1f0003        -
4       System  0x840e99f69340  0x2e84  IoCompletion    0x1f0003        -
4       System  0x840e99b30b10  0x2e88  File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-3251537155-1984446955-2931258699-841473695-1938553385-924012159-129201922
4       System  0x840e9e3cbae0  0x2e8c  Event   0x1f0003        -
4       System  0x840e9856b2c0  0x2e90  Process 0x102a  svchost.exe Pid 7780
4       System  0x840e99089d00  0x2e98  WaitCompletionPacket    0x1     -
4       System  0x840e91da7560  0x2e9c  Event   0x1f0003        -
4       System  0x840e992e1880  0x2ea0  IoCompletion    0x1f0003        -
4       System  0x840e97e11080  0x2ea4  Process 0x40    TextInputHost. Pid 1256
4       System  0x840e99026800  0x2ea8  IoCompletion    0x1f0003        -
4       System  0x840e92f36a00  0x2eac  IoCompletion    0x1f0003        -
4       System  0x840e98e949e0  0x2eb0  Event   0x1f0003        -
4       System  0x840e9a60f440  0x2eb8  IoCompletion    0x1f0003        -
4       System  0x840e98b319a0  0x2ebc  WaitCompletionPacket    0x1     -
4       System  0x840e9c895c60  0x2ec4  Event   0x1f0003        -
4       System  0x840e98bba080  0x2ecc  Process 0x1fffff        SearchApp.exe Pid 7932
4       System  0x840e975e71e0  0x2ed0  File    0x20003 \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat
4       System  0x840e91bc24c0  0x2ed4  WaitCompletionPacket    0x1     -
4       System  0x840e9dad3c40  0x2ed8  IoCompletion    0x1f0003        -
4       System  0x840e99036780  0x2edc  IoCompletion    0x1f0003        -
4       System  0xb70813b9c0c0  0x2ee0  Directory       0xf     S-1-15-2-2226957697-3030467180-2301525-4248967783-2024719031-2325529081-2915787518
4       System  0x840e98a128c0  0x2ee4  IoCompletion    0x1f0003        -
4       System  0x840e9856b2c0  0x2ee8  Process 0x102a  svchost.exe Pid 7780
4       System  0x840e96958900  0x2eec  IoCompletion    0x1f0003        -
4       System  0x840e98b8e340  0x2ef0  Process 0x102a  RuntimeBroker. Pid 7612
4       System  0x840e973d81c0  0x2ef8  Process 0x0     svchost.exe Pid 4800
4       System  0x840e92fc7080  0x2efc  Process 0x0     svchost.exe Pid 1988
4       System  0x840e9856b2c0  0x2f00  Process 0x102a  svchost.exe Pid 7780
4       System  0x840e978e99c0  0x2f04  WaitCompletionPacket    0x1     -
4       System  0x840e99971f30  0x2f08  IoCompletionReserve     0xf0003 -
4       System  0x840e97e59080  0x2f0c  Process 0x102a  conhost.exe Pid 12392
4       System  0x840e98e87a60  0x2f14  Event   0x1f0003        -
4       System  0x840e99f80c80  0x2f20  IoCompletion    0x1f0003        -
4       System  0x840e9c523ee0  0x2f24  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat
4       System  0x840e997a44e0  0x2f28  Event   0x1f0003        -
4       System  0x840e918e90c0  0x2f2c  Process 0x0     svchost.exe Pid 14128
4       System  0x840e9ac3db40  0x2f30  WaitCompletionPacket    0x1     -
4       System  0x840e9db77080  0x2f34  Process 0x102a  taskhostw.exe Pid 15784
4       System  0x840e92e04300  0x2f38  Process 0x0     svchost.exe Pid 1408
4       System  0x840e99aaf740  0x2f3c  IoCompletion    0x1f0003        -
4       System  0x840e9856b2c0  0x2f40  Process 0x1fffff        svchost.exe Pid 7780
4       System  0x840e970c8880  0x2f44  IoCompletion    0x1f0003        -
4       System  0x840e998450c0  0x2f48  Process 0x102a  SearchApp.exe Pid 4236
4       System  0x840e9856b2c0  0x2f4c  Process 0x102a  svchost.exe Pid 7780
4       System  0xb708087979d0  0x2f54  Section 0x17    -
4       System  0x840e98aef080  0x2f58  Process 0x102a  RuntimeBroker. Pid 7684
4       System  0x840e9856b2c0  0x2f5c  Process 0x102a  svchost.exe Pid 7780
4       System  0x840e97baf090  0x2f60  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG1
4       System  0x840e98b30ca0  0x2f68  WaitCompletionPacket    0x1     -
4       System  0x840e993240c0  0x2f6c  Process 0x0     sihost.exe Pid 10384
4       System  0x840e9c856160  0x2f74  Event   0x1f0003        -
4       System  0x840e98b8e340  0x2f78  Process 0x1fffff        RuntimeBroker. Pid 7612
4       System  0x840e90de9080  0x2f7c  Process 0x102a  FileCoAuth.exe Pid 8592
4       System  0x840e99474080  0x2f80  Process 0x1fffff        PSEXESVC.exe Pid 11296
4       System  0xb7081a5c65a0  0x2f84  SymbolicLink    0xf0001 Local
4       System  0x840e966b1080  0x2f88  Process 0x0     MemCompression Pid 1924
4       System  0x840e9ceb5160  0x2f8c  Event   0x1f0003        -
4       System  0xb70822a63a60  0x2f90  Token   0xe     -
4       System  0x840e98a2a740  0x2f94  IoCompletion    0x1f0003        -
4       System  0x840e98fe5d60  0x2f98  Event   0x1f0003        -
4       System  0x840ea0ed2360  0x2f9c  Event   0x1f0003        -
4       System  0x840e9934dc60  0x2fa0  Event   0x1f0003        -
4       System  0x840e9769f080  0x2fa4  Process 0x0     winlogon.exe Pid 2804
4       System  0x840e9bdf4340  0x2fa8  Process 0x102a  ScreenConnect. Pid 14756
4       System  0x840e96a0a7b0  0x2fac  File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-536077884-713174666-1066051701-3219990555-339840825-1966734348-1611281757
4       System  0x840e91d977e0  0x2fb0  Event   0x1f0003        -
4       System  0x840e98dbf2c0  0x2fb4  Process 0x102a  svchost.exe Pid 8688
4       System  0x840e97a65b00  0x2fc0  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.Windows.StartMenuExperienceHost_cw5n1h2txyewy\Settings\settings.dat.LOG2
4       System  0x840e999ec460  0x2fc4  Event   0x1f0003        -
4       System  0x840e9f40c340  0x2fc8  Process 0x1fffff        svchost.exe Pid 2800
4       System  0x840e974b4080  0x2fcc  Process 0x0     CSFalconContai Pid 324
4       System  0x840e98c24a60  0x2fd0  Event   0x1f0003        -
4       System  0x840e99078990  0x2fd4  WaitCompletionPacket    0x1     -
4       System  0x840e99f697c0  0x2fd8  IoCompletion    0x1f0003        -
4       System  0x840e9745b760  0x2fdc  Event   0x1f0003        -
4       System  0x840e98e8e060  0x2fe0  Event   0x1f0003        -
4       System  0x840e9c71db60  0x2fe8  Event   0x1f0003        -
4       System  0x840e97656100  0x2fec  IoCompletion    0x1f0003        -
4       System  0x840e96da9300  0x2ff4  Process 0x0     svchost.exe Pid 3332
4       System  0x840e990b0680  0x2ff8  WaitCompletionPacket    0x1     -
4       System  0x840e990b0820  0x2ffc  WaitCompletionPacket    0x1     -
4       System  0x840e9f02adf0  0x3004  WaitCompletionPacket    0x1     -
4       System  0x840e997b1860  0x300c  Event   0x1f0003        -
4       System  0x840e99aad180  0x3010  IoCompletion    0x1f0003        -
4       System  0x840e99f82780  0x3014  IoCompletion    0x1f0003        -
4       System  0x840e98a11680  0x3018  IoCompletion    0x1f0003        -
4       System  0x840e98dc11c0  0x301c  Process 0x102a  SecurityHealth Pid 8736
4       System  0x840e9979f2e0  0x3020  Event   0x1f0003        -
4       System  0x840e90e10410  0x3024  Session 0x0     Session2
4       System  0xb7081319baa0  0x3028  Directory       0xf     RPC Control
4       System  0x840e9a94a460  0x302c  Event   0x1f0003        -
4       System  0x840e997b1c60  0x3030  Event   0x1f0003        -
4       System  0x840e997176e0  0x3034  Event   0x1f0003        -
4       System  0x840e9958a0c0  0x3038  Process 0x1fffff        msedge.exe Pid 9128
4       System  0x840e9979f560  0x303c  Event   0x1f0003        -
4       System  0x840e97142760  0x3040  WaitCompletionPacket    0x1     -
4       System  0x840e9666ee20  0x3044  WaitCompletionPacket    0x1     -
4       System  0x840e98d0a840  0x3048  IoCompletion    0x1f0003        -
4       System  0x840e98e92260  0x304c  Event   0x1f0003        -
4       System  0x840e98e85d60  0x3050  Event   0x1f0003        -
4       System  0x840e90bc5560  0x3054  Event   0x1f0003        -
4       System  0x840e9ac28ae0  0x3058  WaitCompletionPacket    0x1     -
4       System  0x840e99aa7140  0x305c  IoCompletion    0x1f0003        -
4       System  0x840e99bcb080  0x3060  Process 0x102a  explorer.exe Pid 9852
4       System  0x840e9958a0c0  0x3064  Process 0x102a  msedge.exe Pid 9128
4       System  0x840e99976eb0  0x3068  IoCompletionReserve     0xf0003 -
4       System  0x840e9a95cbe0  0x3070  Event   0x1f0003        -
4       System  0x840e9e99e080  0x3074  Process 0x102a  dllhost.exe Pid 7884
4       System  0x840e98f8b570  0x3078  IoCompletionReserve     0xf0003 -
4       System  0x840e999f60e0  0x307c  Event   0x1f0003        -
4       System  0x840e990799d0  0x3080  WaitCompletionPacket    0x1     -
4       System  0x840e997a2660  0x3084  Event   0x1f0003        -
4       System  0x840e985cee20  0x3088  FilterCommunicationPort 0x1f0001        -
4       System  0x840e98c2cce0  0x308c  Event   0x1f0003        -
4       System  0x840e98d0de80  0x3090  IoCompletion    0x1f0003        -
4       System  0x840e9907d2b0  0x3094  WaitCompletionPacket    0x1     -
4       System  0x840e990760f0  0x3098  WaitCompletionPacket    0x1     -
4       System  0x840e98d0c600  0x309c  IoCompletion    0x1f0003        -
4       System  0x840e971473c0  0x30a0  WaitCompletionPacket    0x1     -
4       System  0x840e98d28340  0x30a4  IoCompletion    0x1f0003        -
4       System  0x840e978e3500  0x30a8  WaitCompletionPacket    0x1     -
4       System  0x840e98d255c0  0x30ac  IoCompletion    0x1f0003        -
4       System  0x840e9951c1c0  0x30b0  Process 0x1fffff        msedge.exe Pid 8732
4       System  0x840e96794080  0x30b4  Process 0x0     svchost.exe Pid 2276
4       System  0x840e98ff32c0  0x30b8  Process 0x1fffff        SecurityHealth Pid 8808
4       System  0x840e98184430  0x30bc  WaitCompletionPacket    0x1     -
4       System  0x840e99f76700  0x30c0  IoCompletion    0x1f0003        -
4       System  0x840e990502c0  0x30c4  IoCompletion    0x1f0003        -
4       System  0x840e9c858d60  0x30c8  Event   0x1f0003        -
4       System  0x840e9904d1c0  0x30cc  IoCompletion    0x1f0003        -
4       System  0x840e9907be60  0x30d0  WaitCompletionPacket    0x1     -
4       System  0x840e98fd93e0  0x30d4  Event   0x1f0003        -
4       System  0x840e99f61d40  0x30d8  IoCompletion    0x1f0003        -
4       System  0x840e98fdd0e0  0x30dc  Event   0x1f0003        -
4       System  0x840e99aa4a00  0x30e4  IoCompletion    0x1f0003        -
4       System  0x840e99030a00  0x30e8  IoCompletion    0x1f0003        -
4       System  0x840e98dc11c0  0x30ec  Process 0x102a  SecurityHealth Pid 8736
4       System  0x840e92fbb0c0  0x30f0  Process 0x0     svchost.exe Pid 604
4       System  0x840e997aa260  0x30f4  Event   0x1f0003        -
4       System  0x840e99fd09e0  0x30f8  Event   0x1f0003        -
4       System  0x840e981809b0  0x30fc  WaitCompletionPacket    0x1     -
4       System  0x840e9979cde0  0x3100  Event   0x1f0003        -
4       System  0x840e9b73a080  0x3104  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e98a30a00  0x3108  IoCompletion    0x1f0003        -
4       System  0x840e98fd5060  0x310c  Event   0x1f0003        -
4       System  0x840e99095980  0x3110  WaitCompletionPacket    0x1     -
4       System  0x840e9979cae0  0x3114  Event   0x1f0003        -
4       System  0x840e99d66a20  0x3118  ALPC Port       0x1f0001        -
4       System  0x840e98dc11c0  0x311c  Process 0x1fffff        SecurityHealth Pid 8736
4       System  0x840e9f0024c0  0x3120  WaitCompletionPacket    0x1     -
4       System  0x840e9ba0f180  0x3128  WaitCompletionPacket    0x1     -
4       System  0x840e9902c680  0x312c  IoCompletion    0x1f0003        -
4       System  0x840e97add1c0  0x3130  Process 0x102a  svchost.exe Pid 6020
4       System  0x840e96e3d080  0x3134  Process 0x0     svchost.exe Pid 3528
4       System  0x840e98180c20  0x3138  WaitCompletionPacket    0x1     -
4       System  0x840ea46df1e0  0x313c  Event   0x1f0003        -
4       System  0x840e98a0e3c0  0x3140  IoCompletion    0x1f0003        -
4       System  0x840e97b71740  0x3144  IoCompletionReserve     0xf0003 -
4       System  0x840e9682f080  0x3148  Process 0x102a  OneDrive.exe Pid 3320
4       System  0x840e98fe9ce0  0x314c  Event   0x1f0003        -
4       System  0x840e9a603bc0  0x3150  IoCompletion    0x1f0003        -
4       System  0x840e96864380  0x3154  Process 0x0     svchost.exe Pid 2384
4       System  0x840e9f7a7e80  0x3158  IoCompletion    0x1f0003        -
4       System  0x840e9817e6c0  0x315c  WaitCompletionPacket    0x1     -
4       System  0x840e9a61e6c0  0x3160  IoCompletion    0x1f0003        -
4       System  0x840e9832a1c0  0x3164  Process 0x1fffff        msedge.exe Pid 3832
4       System  0x840e976562c0  0x3168  IoCompletion    0x1f0003        -
4       System  0x840e9951c1c0  0x316c  Process 0x102a  msedge.exe Pid 8732
4       System  0x840e99c2bae0  0x3170  Event   0x1f0003        -
4       System  0x840e98b63100  0x3174  WaitCompletionPacket    0x1     -
4       System  0x840e996e91c0  0x3178  Process 0x102a  msedge.exe Pid 9016
4       System  0x840e997b0660  0x317c  Event   0x1f0003        -
4       System  0x840e978e47b0  0x3180  WaitCompletionPacket    0x1     -
4       System  0x840e9bdf4340  0x3184  Process 0x0     ScreenConnect. Pid 14756
4       System  0x840e96e552c0  0x3188  Process 0x0     svchost.exe Pid 3560
4       System  0x840e96d9f080  0x318c  Process 0x0     svchost.exe Pid 3308
4       System  0x840e997ae1e0  0x3190  Event   0x1f0003        -
4       System  0x840e98ff32c0  0x3194  Process 0x102a  SecurityHealth Pid 8808
4       System  0x840e9ba113a0  0x3198  WaitCompletionPacket    0x1     -
4       System  0x840e99031480  0x319c  IoCompletion    0x1f0003        -
4       System  0x840e9c85fa60  0x31a0  Event   0x1f0003        -
4       System  0x840e98fd45e0  0x31a4  Event   0x1f0003        -
4       System  0x840e98dbf2c0  0x31a8  Process 0x102a  svchost.exe Pid 8688
4       System  0xb70818ebd310  0x31ac  Token   0xf01ff -
4       System  0x840e99077e30  0x31b4  WaitCompletionPacket    0x1     -
4       System  0x840e99ad29c0  0x31b8  IoCompletion    0x1f0003        -
4       System  0x840e9b9b96a0  0x31bc  WaitCompletionPacket    0x1     -
4       System  0x840e99092e70  0x31c0  WaitCompletionPacket    0x1     -
4       System  0x840e990422c0  0x31c4  IoCompletion    0x1f0003        -
4       System  0x840e998251c0  0x31c8  Process 0x102a  msedge.exe Pid 9728
4       System  0x840e9ceb47e0  0x31cc  Event   0x1f0003        -
4       System  0x840e99702d60  0x31d0  Event   0x1f0003        -
4       System  0x840e98fd03e0  0x31d4  Event   0x1f0003        -
4       System  0x840e98f07080  0x31d8  Process 0x102a  msedge.exe Pid 9172
4       System  0x840e98e8fb60  0x31dc  Event   0x1f0003        -
4       System  0x840ea87f9f40  0x31e0  IoCompletion    0x1f0003        -
4       System  0x840e990afb20  0x31e4  WaitCompletionPacket    0x1     -
4       System  0x840e91ba00c0  0x31e8  Process 0x102a  svchost.exe Pid 7220
4       System  0x840e96bf8660  0x31ec  Event   0x1f0003        -
4       System  0x840e9904df00  0x31f0  IoCompletion    0x1f0003        -
4       System  0x840e96d1a8c0  0x31f4  IoCompletion    0x1f0003        -
4       System  0x840e990af570  0x31f8  WaitCompletionPacket    0x1     -
4       System  0x840e9979a3e0  0x31fc  Event   0x1f0003        -
4       System  0x840e99aae5c0  0x3200  IoCompletion    0x1f0003        -
4       System  0x840e99f578c0  0x3204  IoCompletion    0x1f0003        -
4       System  0x840e9979aa60  0x3208  Event   0x1f0003        -
4       System  0x840e9dae9e80  0x320c  IoCompletion    0x1f0003        -
4       System  0x840e9713b9b0  0x3210  WaitCompletionPacket    0x1     -
4       System  0x840e9979a560  0x3214  Event   0x1f0003        -
4       System  0xb70812d09e70  0x3218  SymbolicLink    0xf0001 Local
4       System  0x840e996e91c0  0x321c  Process 0x102a  msedge.exe Pid 9016
4       System  0x840e9951c1c0  0x3220  Process 0x102a  msedge.exe Pid 8732
4       System  0xb70812d09840  0x3224  SymbolicLink    0xf0001 Global
4       System  0x840e97148260  0x3228  WaitCompletionPacket    0x1     -
4       System  0x840e92349080  0x322c  Process 0x0     lsass.exe Pid 548
4       System  0x840e9c45e540  0x3234  IoCompletion    0x1f0003        -
4       System  0x840e999dca60  0x3238  Event   0x1f0003        -
4       System  0x840e994d1080  0x323c  Process 0x102a  LogonUI.exe Pid 10680
4       System  0x840e98db8080  0x3240  Process 0x1fffff        MsSense.exe Pid 10224
4       System  0x840e9979c2e0  0x3244  Event   0x1f0003        -
4       System  0x840e92ee4c30  0x3248  WaitCompletionPacket    0x1     -
4       System  0x840e997a0560  0x324c  Event   0x1f0003        -
4       System  0x840e9676e080  0x3250  Process 0x0     svchost.exe Pid 2168
4       System  0x840e9909ad30  0x3254  WaitCompletionPacket    0x1     -
4       System  0x840e997a6260  0x3258  Event   0x1f0003        -
4       System  0x840e96999270  0x325c  WaitCompletionPacket    0x1     -
4       System  0x840e96da72c0  0x3260  Process 0x0     svchost.exe Pid 3324
4       System  0x840e9745f7e0  0x3264  Event   0x1f0003        -
4       System  0x840e99aaf200  0x3268  IoCompletion    0x1f0003        -
4       System  0x840e9817f700  0x326c  WaitCompletionPacket    0x1     -
4       System  0x840e98b5e090  0x3270  WaitCompletionPacket    0x1     -
4       System  0x840e98a1edc0  0x3274  IoCompletion    0x1f0003        -
4       System  0x840e96956c00  0x3278  IoCompletion    0x1f0003        -
4       System  0x840e96e042c0  0x327c  Process 0x0     svchost.exe Pid 3376
4       System  0x840e98db4080  0x3280  Process 0x1fffff        msedge.exe Pid 9888
4       System  0xb707fbcbaac0  0x3284  SymbolicLink    0xf0001 Session
4       System  0x840e99aa6440  0x3288  IoCompletion    0x1f0003        -
4       System  0x840e9bdf4340  0x328c  Process 0x1fffff        ScreenConnect. Pid 14756
4       System  0x840e99aa4c80  0x3290  IoCompletion    0x1f0003        -
4       System  0x840e91bc2a70  0x3294  WaitCompletionPacket    0x1     -
4       System  0x840e923f61c0  0x329c  Process 0x0     fontdrvhost.ex Pid 892
4       System  0x840e9c70bb60  0x32a0  Event   0x1f0003        -
4       System  0x840e980787a0  0x32a4  CoreMessaging   0xf0000 -
4       System  0x840e9b993c90  0x32a8  WaitCompletionPacket    0x1     -
4       System  0x840e96806a90  0x32ac  WaitCompletionPacket    0x1     -
4       System  0x840e997a7c60  0x32b0  Event   0x1f0003        -
4       System  0x840e92d4f440  0x32b8  IoCompletion    0x1f0003        -
4       System  0x840e99ad3e00  0x32bc  IoCompletion    0x1f0003        -
4       System  0x840e9a9443e0  0x32c0  Event   0x1f0003        -
4       System  0x840e92d48080  0x32c4  Process 0x0     svchost.exe Pid 1364
4       System  0x840e98d17f80  0x32c8  IoCompletion    0x1f0003        -
4       System  0x840e92c4fd10  0x32cc  WaitCompletionPacket    0x1     -
4       System  0x840e99f4ba00  0x32d0  IoCompletion    0x1f0003        -
4       System  0x840e9ac0a0f0  0x32d4  WaitCompletionPacket    0x1     -
4       System  0x840e98b48260  0x32d8  WaitCompletionPacket    0x1     -
4       System  0x840e91ba00c0  0x32dc  Process 0x1fffff        svchost.exe Pid 7220
4       System  0x840e98bdf080  0x32e0  Process 0x102a  SystemSettings Pid 10020
4       System  0x840e996cf080  0x32e4  Process 0x102a  svchost.exe Pid 9320
4       System  0x840e91f131c0  0x32e8  Process 0x102a  Sysmon64.exe Pid 1244
4       System  0x840e998251c0  0x32f0  Process 0x1fffff        msedge.exe Pid 9728
4       System  0x840e99aac640  0x32f4  IoCompletion    0x1f0003        -
4       System  0x840e9933d360  0x32f8  Event   0x1f0003        -
4       System  0x840e9f38b840  0x3300  IoCompletion    0x1f0003        -
4       System  0x840e999e2de0  0x3304  Event   0x1f0003        -
4       System  0x840e98bdf080  0x3308  Process 0x102a  SystemSettings Pid 10020
4       System  0x840e9ac13330  0x330c  WaitCompletionPacket    0x1     -
4       System  0x840e96dc4080  0x3310  Process 0x0     svchost.exe Pid 3352
4       System  0x840e997aab60  0x3314  Event   0x1f0003        -
4       System  0x840e92e48080  0x331c  Process 0x0     svchost.exe Pid 1524
4       System  0x840e97468b60  0x3320  Event   0x1f0003        -
4       System  0x840e999de660  0x3324  Event   0x1f0003        -
4       System  0x840e9a94e9e0  0x3328  Event   0x1f0003        -
4       System  0x840e9c121080  0x332c  Process 0x102a  RuntimeBroker. Pid 10764
4       System  0x840e9c8bb670  0x3330  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.2.regtrans-ms
4       System  0x840e9764e840  0x3334  IoCompletion    0x1f0003        -
4       System  0x840e990a1d50  0x3338  WaitCompletionPacket    0x1     -
4       System  0x840e98bdf080  0x333c  Process 0x102a  SystemSettings Pid 10020
4       System  0x840e98eb3080  0x3340  Process 0x0     conhost.exe Pid 10944
4       System  0x840e96860080  0x3344  Process 0x0     svchost.exe Pid 2396
4       System  0x840e97d58e80  0x3348  IoCompletion    0x1f0003        -
4       System  0x840e98db8080  0x334c  Process 0x102a  MsSense.exe Pid 10224
4       System  0x840e96769240  0x3350  Process 0x0     svchost.exe Pid 2160
4       System  0x840e96cdd1c0  0x3358  Process 0x0     svchost.exe Pid 740
4       System  0x840e99c29ce0  0x335c  Event   0x1f0003        -
4       System  0x840e98db8080  0x3364  Process 0x0     MsSense.exe Pid 10224
4       System  0x840e96bc00c0  0x3368  Process 0x0     svchost.exe Pid 2848
4       System  0x840e99aaa940  0x336c  IoCompletion    0x1f0003        -
4       System  0x840e990999b0  0x3370  WaitCompletionPacket    0x1     -
4       System  0x840e99aa3dc0  0x3374  IoCompletion    0x1f0003        -
4       System  0x840e90de2080  0x3378  Process 0x0     svchost.exe Pid 1480
4       System  0x840e90c19340  0x337c  Process 0x0     svchost.exe Pid 3604
4       System  0x840e97add1c0  0x3380  Process 0x0     svchost.exe Pid 6020
4       System  0x840e9b996d50  0x3384  WaitCompletionPacket    0x1     -
4       System  0x840e974b7080  0x3388  Process 0x0     CSFalconContai Pid 5980
4       System  0x840e999e0a60  0x338c  Event   0x1f0003        -
4       System  0x840e99720ce0  0x3390  Event   0x1f0003        -
4       System  0x840e99c83080  0x3394  Process 0x102a  dwm.exe Pid 10628
4       System  0x840e97e8c080  0x3398  Process 0x102a  svchost.exe Pid 3864
4       System  0x840e99fd62e0  0x33a0  Event   0x1f0003        -
4       System  0x840e98079880  0x33a4  CoreMessaging   0xf0000 -
4       System  0x840e96df8c70  0x33a8  Desktop 0xf0003 Disconnect
4       System  0x840e9682f080  0x33b0  Process 0x102a  OneDrive.exe Pid 3320
4       System  0x840ea08f3080  0x33b4  Process 0x102a  svchost.exe Pid 12008
4       System  0x840e99513a60  0x33b8  ALPC Port       0x1f0001        -
4       System  0x840e9933eb60  0x33bc  Event   0x1f0003        -
4       System  0x840e9ba21460  0x33c0  WaitCompletionPacket    0x1     -
4       System  0x840e997196e0  0x33c4  Event   0x1f0003        -
4       System  0x840e9676e080  0x33c8  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e990dc5c0  0x33cc  TmRm    0x1f007f        -
4       System  0x840e98db8080  0x33d0  Process 0x102a  MsSense.exe Pid 10224
4       System  0x840e98a26e40  0x33d4  IoCompletion    0x1f0003        -
4       System  0x840e97dc72c0  0x33d8  Process 0x0     SearchIndexer. Pid 1852
4       System  0x840e96862080  0x33dc  Process 0x0     svchost.exe Pid 2376
4       System  0x840e98b29e20  0x33e0  WaitCompletionPacket    0x1     -
4       System  0x840e98c450c0  0x33e4  Process 0x0     dllhost.exe Pid 7800
4       System  0x840e9676e080  0x33e8  Process 0x102a  svchost.exe Pid 2168
4       System  0x840e9682f080  0x33ec  Process 0x102a  OneDrive.exe Pid 3320
4       System  0x840e994d1080  0x33f0  Process 0x0     LogonUI.exe Pid 10680
4       System  0x840e92d2e240  0x33f4  Process 0x0     svchost.exe Pid 1320
4       System  0x840e98187b70  0x33f8  WaitCompletionPacket    0x1     -
4       System  0x840e98bce2b0  0x33fc  TmEn    0xf001f -
4       System  0x840e98dc11c0  0x3404  Process 0x0     SecurityHealth Pid 8736
4       System  0x840e92e732c0  0x3408  Process 0x0     svchost.exe Pid 1580
4       System  0xb7081a44b5e0  0x340c  Key     0xf003f USER\S-1-12-1-419028606-1323772725-945580434-2117921032\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\WINLOGON
4       System  0x840e96e350c0  0x3410  Process 0x0     svchost.exe Pid 3480
4       System  0x840e96e20080  0x3414  Process 0x0     svchost.exe Pid 3488
4       System  0x840e966d9300  0x3418  Process 0x0     svchost.exe Pid 8912
4       System  0x840e9ca6b240  0x341c  IoCompletion    0x1f0003        -
4       System  0x840e92305080  0x3420  Process 0x0     winlogon.exe Pid 980
4       System  0x840e995cd0c0  0x3424  Process 0x1fffff        svchost.exe Pid 6356
4       System  0x840e978e24c0  0x3428  WaitCompletionPacket    0x1     -
4       System  0x840e98bd6240  0x342c  Process 0x0     RuntimeBroker. Pid 8120
4       System  0x840e97e11080  0x3430  Process 0x0     TextInputHost. Pid 1256
4       System  0x840e96e44300  0x3434  Process 0x0     MsMpEng.exe Pid 3540
4       System  0xb7080878d570  0x3438  Section 0x17    -
4       System  0x840e970ce700  0x3440  IoCompletion    0x1f0003        -
4       System  0xb707fbcba960  0x3444  SymbolicLink    0xf0001 Local
4       System  0x840e96e77080  0x3448  Process 0x0     svchost.exe Pid 3636
4       System  0x840e972491c0  0x344c  Process 0x0     svchost.exe Pid 4352
4       System  0x840e98f07080  0x3450  Process 0x0     msedge.exe Pid 9172
4       System  0x840e9969e080  0x3454  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e98dbf2c0  0x3458  Process 0x0     svchost.exe Pid 8688
4       System  0x840e9e99e080  0x345c  Process 0x1fffff        dllhost.exe Pid 7884
4       System  0x840e996cf080  0x3460  Process 0x0     svchost.exe Pid 9320
4       System  0x840e9b9afde0  0x3468  WaitCompletionPacket    0x1     -
4       System  0x840ea3cf4760  0x3470  Event   0x1f0003        -
4       System  0x840e995680c0  0x3474  Process 0x1fffff        SgrmBroker.exe Pid 9948
4       System  0x840e970f3080  0x3478  Process 0x0     svchost.exe Pid 3172
4       System  0x840e99840080  0x347c  Process 0x1fffff        dllhost.exe Pid 2492
4       System  0x840e9f2d8ae0  0x3480  Event   0x1f0003        -
4       System  0x840e9933af60  0x3484  Event   0x1f0003        -
4       System  0x840e98db8080  0x3488  Process 0x1028  MsSense.exe Pid 10224
4       System  0x840e98b3cd30  0x348c  WaitCompletionPacket    0x1     -
4       System  0x840e9f016ea0  0x3490  WaitCompletionPacket    0x1     -
4       System  0x840e92ee45b0  0x3494  WaitCompletionPacket    0x1     -
4       System  0x840e981a6700  0x34a0  WaitCompletionPacket    0x1     -
4       System  0x840e99448990  0x34a4  ALPC Port       0x1f0001        -
4       System  0x840e98b3c5e0  0x34ac  WaitCompletionPacket    0x1     -
4       System  0x840e97681600  0x34b0  IoCompletion    0x1f0003        -
4       System  0x840e9f39a800  0x34b4  IoCompletion    0x1f0003        -
4       System  0x840e995cd0c0  0x34b8  Process 0x102a  svchost.exe Pid 6356
4       System  0x840e966d9300  0x34bc  Process 0x102a  svchost.exe Pid 8912
4       System  0x840e9b1a1240  0x34c0  Process 0x1fffff        RuntimeBroker. Pid 12140
4       System  0x840e9ca424c0  0x34c4  IoCompletion    0x1f0003        -
4       System  0x840e995680c0  0x34c8  Process 0x0     SgrmBroker.exe Pid 9948
4       System  0x840e98e94460  0x34cc  Event   0x1f0003        -
4       System  0x840ea70570c0  0x34d0  Process 0x0     msedge.exe Pid 1336
4       System  0x840e995680c0  0x34d4  Process 0x102a  SgrmBroker.exe Pid 9948
4       System  0x840e995680c0  0x34d8  Process 0x102a  SgrmBroker.exe Pid 9948
4       System  0x840e91c3b300  0x34e0  Process 0x102a  svchost.exe Pid 8312
4       System  0x840e99dbfb30  0x34e4  File    0x13008b        \Device\HarddiskVolume3\Windows\System32\LogFiles\WMI\RtBackup\EtwRTSgrmEtwSession.etl
4       System  0x840e9969e080  0x34e8  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e995ea080  0x34f0  Process 0x0     OneDrive.exe Pid 11592
4       System  0x840e99c201e0  0x34f4  Event   0x1f0003        -
4       System  0x840e9b9b25b0  0x34f8  WaitCompletionPacket    0x1     -
4       System  0xb70823a39930  0x3500  SymbolicLink    0xf0001 Global
4       System  0x840e9bce9070  0x3504  RawInputManager 0x3     -
4       System  0x840e966d9300  0x3508  Process 0x102a  svchost.exe Pid 8912
4       System  0x840e9971fce0  0x3510  Event   0x1f0003        -
4       System  0x840e990a65a0  0x3514  WaitCompletionPacket    0x1     -
4       System  0x840e9d699990  0x3518  File    0x1     \Device\HarddiskVolume3\ProgramData\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\S-1-12-1-419028606-1323772725-945580434-2117921032\SystemAppData\Helium\Cache\b4d622c333fcfdd7.dat
4       System  0xb70812b05280  0x351c  Directory       0xf     S-1-15-2-1910091885-1573563583-1104941280-2418270861-3411158377-2822700936-2990310272
4       System  0x840e99581080  0x3520  Process 0x102a  svchost.exe Pid 15508
4       System  0x840e9b98acc0  0x3524  WaitCompletionPacket    0x1     -
4       System  0x840e9c8647e0  0x352c  Event   0x1f0003        -
4       System  0x840e9a61d440  0x3530  IoCompletion    0x1f0003        -
4       System  0x840e997450c0  0x3534  Process 0x1fffff        svchost.exe Pid 3052
4       System  0x840e9969e080  0x3538  Process 0x1fffff        svchost.exe Pid 4156
4       System  0x840e997450c0  0x353c  Process 0x0     svchost.exe Pid 3052
4       System  0x840e99f57980  0x3540  IoCompletion    0x1f0003        -
4       System  0x840e9969e080  0x3544  Process 0x102a  svchost.exe Pid 4156
4       System  0x840e98184290  0x3548  WaitCompletionPacket    0x1     -
4       System  0x840e997450c0  0x354c  Process 0x1fffff        svchost.exe Pid 3052
4       System  0x840e97ea9080  0x3550  Process 0x1fffff        RuntimeBroker. Pid 1832
4       System  0x840e966d9300  0x3554  Process 0x0     svchost.exe Pid 8912
4       System  0x840e9a625400  0x3558  IoCompletion    0x1f0003        -
4       System  0x840e99086350  0x355c  WaitCompletionPacket    0x1     -
4       System  0x840e922da080  0x3564  Thread  0x1fffff        Tid 3748 Pid 4
4       System  0x840e9979cf60  0x356c  Event   0x1f0003        -
4       System  0x840e9a61e880  0x3570  IoCompletion    0x1f0003        -
4       System  0x840e997450c0  0x3574  Process 0x102a  svchost.exe Pid 3052
4       System  0x840e98b28eb0  0x3578  WaitCompletionPacket    0x1     -
4       System  0x840e9a625880  0x357c  IoCompletion    0x1f0003        -
4       System  0x840e99707160  0x3580  Event   0x1f0003        -
4       System  0x840e9969e080  0x3584  Process 0x0     svchost.exe Pid 4156
4       System  0xb70829130960  0x3588  Token   0xf01ff -
4       System  0x840e96f5a080  0x358c  Process 0x0     SenseNdr.exe Pid 15568
4       System  0x840e9830b530  0x3590  ALPC Port       0x1f0001        -
4       System  0x840e91d8afe0  0x3594  Event   0x1f0003        -
4       System  0x840e9ac0e940  0x3598  WaitCompletionPacket    0x1     -
4       System  0x840e98183320  0x359c  WaitCompletionPacket    0x1     -
4       System  0x840e99712d60  0x35a4  Event   0x1f0003        -
4       System  0x840e96a5d5d0  0x35a8  IoCompletionReserve     0xf0003 -
4       System  0x840e9970ee60  0x35ac  Event   0x1f0003        -
4       System  0x840e99f49440  0x35b0  IoCompletion    0x1f0003        -
4       System  0x840e90c049f0  0x35b4  TmRm    0x1f007f        -
4       System  0x840e966d9300  0x35b8  Process 0x102a  svchost.exe Pid 8912
4       System  0x840e9e3df5e0  0x35bc  Event   0x1f0003        -
4       System  0x840ea9ecef60  0x35c0  Event   0x1f0003        -
4       System  0x840e9f3d1040  0x35c4  Thread  0x1fffff        Tid 5156 Pid 4
4       System  0x840e96bfbae0  0x35cc  Event   0x1f0003        -
4       System  0x840e92ee3640  0x35d4  WaitCompletionPacket    0x1     -
4       System  0x840e90e10410  0x35d8  Session 0x0     Session2
4       System  0x840e990a3d00  0x35dc  WaitCompletionPacket    0x1     -
4       System  0x840e9907f810  0x35e0  WaitCompletionPacket    0x1     -
4       System  0x840e91c3b300  0x35e4  Process 0x102a  svchost.exe Pid 8312
4       System  0x840e970d1480  0x35ec  IoCompletion    0x1f0003        -
4       System  0x840e99a92080  0x35f0  Process 0x0     SecurityHealth Pid 12036
4       System  0x840ea08f3080  0x35f4  Process 0x102a  svchost.exe Pid 12008
4       System  0x840ea08f3080  0x35f8  Process 0x1fffff        svchost.exe Pid 12008
4       System  0x840ea5362d60  0x35fc  Event   0x1f0003        -
4       System  0x840e9e3d0ee0  0x3604  Event   0x1f0003        -
4       System  0x840e9807ab10  0x3608  CoreMessaging   0xf0000 -
4       System  0x840e99fd75e0  0x360c  Event   0x1f0003        -
4       System  0x840e979ea310  0x3610  ALPC Port       0x1f0001        -
4       System  0x840e9ac0f3d0  0x3614  WaitCompletionPacket    0x1     -
4       System  0x840e9907e8a0  0x361c  WaitCompletionPacket    0x1     -
4       System  0x840e99fc7de0  0x3620  Event   0x1f0003        -
4       System  0x840e96818d70  0x3624  WaitCompletionPacket    0x1     -
4       System  0x840e90e10410  0x3628  Session 0x0     Session2
4       System  0x840e99847070  0x362c  RawInputManager 0x3     -
4       System  0x840e9f388f40  0x3630  IoCompletion    0x1f0003        -
4       System  0x840e981886d0  0x3638  WaitCompletionPacket    0x1     -
4       System  0x840e9909f0a0  0x363c  WaitCompletionPacket    0x1     -
4       System  0x840e99fd6960  0x3640  Event   0x1f0003        -
4       System  0x840e97ea9080  0x3644  Process 0x0     RuntimeBroker. Pid 1832
4       System  0x840e9c852460  0x3648  Event   0x1f0003        -
4       System  0x840e9ac30a70  0x364c  WaitCompletionPacket    0x1     -
4       System  0x840e98a0fb40  0x3650  IoCompletion    0x1f0003        -
4       System  0x840e98a27700  0x3654  IoCompletion    0x1f0003        -
4       System  0x840e970d0c80  0x3658  IoCompletion    0x1f0003        -
4       System  0xb70823891230  0x365c  Section 0x5     -
4       System  0x840e97c46c60  0x3660  Event   0x1f0003        -
4       System  0xb707fbcbacd0  0x3664  SymbolicLink    0xf0001 Global
4       System  0x840e9c9a9800  0x3668  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.UI.Xaml.2.0_2.1810.18004.0_x64__8wekyb3d8bbwe\ActivationStore.dat.LOG2
4       System  0x840e91da7360  0x366c  Event   0x1f0003        -
4       System  0x840e97211cc0  0x3670  IoCompletionReserve     0xf0003 -
4       System  0x840e91b3c0e0  0x3674  Timer   0x1f0003        -
4       System  0x840e98a0a300  0x3678  IoCompletion    0x1f0003        -
4       System  0x840e9a1476a0  0x367c  File    0x3     \Device\HarddiskVolume3\Users\admin\AppData\Local\Packages\Microsoft.SkypeApp_kzf8qxf38zg5c\SystemAppData\Helium\User.dat.LOG2
4       System  0x840e9a625dc0  0x3680  IoCompletion    0x1f0003        -
4       System  0x840e990a11f0  0x3684  WaitCompletionPacket    0x1     -
4       System  0xb707fbcb43a0  0x3688  SymbolicLink    0xf0001 Local
4       System  0x840e99c2c5e0  0x368c  Event   0x1f0003        -
4       System  0x840e97810a60  0x3694  Event   0x1f0003        -
4       System  0x840e96d56340  0x3698  Process 0x1fffff        SenseTVM.exe Pid 1552
4       System  0x840e9c9a87c0  0x369c  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.UI.Xaml.2.0_2.1810.18004.0_x64__8wekyb3d8bbwe\ActivationStore.dat.LOG1
4       System  0x840e99f6a940  0x36ac  IoCompletion    0x1f0003        -
4       System  0x840e9902a380  0x36b4  IoCompletion    0x1f0003        -
4       System  0x840e9933ea60  0x36b8  Event   0x1f0003        -
4       System  0x840e96956280  0x36bc  IoCompletion    0x1f0003        -
4       System  0x840e99f6a4c0  0x36c0  IoCompletion    0x1f0003        -
4       System  0x840e967df060  0x36c8  Event   0x1f0003        -
4       System  0x840e98192470  0x36cc  WaitCompletionPacket    0x1     -
4       System  0x840e9ac109c0  0x36d0  WaitCompletionPacket    0x1     -
4       System  0x840ea3cf7260  0x36d4  Event   0x1f0003        -
4       System  0x840e9db77080  0x36d8  Process 0x102a  taskhostw.exe Pid 15784
4       System  0x840e9a944de0  0x36e0  Event   0x1f0003        -
4       System  0x840e995ea080  0x36e4  Process 0x1fffff        OneDrive.exe Pid 11592
4       System  0x840e99fd2760  0x36e8  Event   0x1f0003        -
4       System  0x840e990a52f0  0x36f0  WaitCompletionPacket    0x1     -
4       System  0x840e9a615f00  0x36f4  IoCompletion    0x1f0003        -
4       System  0x840e99658510  0x36f8  ALPC Port       0x1f0001        -
4       System  0x840e9ba042d0  0x36fc  WaitCompletionPacket    0x1     -
4       System  0x840e98d887c0  0x3700  ALPC Port       0x1f0001        -
4       System  0x840e9dad0980  0x3704  IoCompletion    0x1f0003        -
4       System  0x840ea534d560  0x3708  Event   0x1f0003        -
4       System  0x840ea085e440  0x370c  IoCompletion    0x1f0003        -
4       System  0xb7081cbe29c0  0x3710  SymbolicLink    0xf0001 Global
4       System  0x840e985d8080  0x3714  Process 0x1fffff        ShellExperienc Pid 7588
4       System  0x840e99f6e680  0x3718  IoCompletion    0x1f0003        -
4       System  0x840e9ac0f980  0x371c  WaitCompletionPacket    0x1     -
4       System  0x840e973867c0  0x3724  IoCompletion    0x1f0003        -
4       System  0x840e99335b60  0x3728  Event   0x1f0003        -
4       System  0x840e98d04b80  0x372c  IoCompletion    0x1f0003        -
4       System  0x840e99093350  0x373c  WaitCompletionPacket    0x1     -
4       System  0x840e9c5a8d60  0x3740  Event   0x1f0003        -
4       System  0x840e98b50ef0  0x3744  WaitCompletionPacket    0x1     -
4       System  0x840e9b9ba200  0x3748  WaitCompletionPacket    0x1     -
4       System  0x840e97d72900  0x374c  IoCompletion    0x1f0003        -
4       System  0x840e9a629080  0x3750  IoCompletion    0x1f0003        -
4       System  0x840e9c52f820  0x3754  File    0x12019f        \Device\clfs\Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM
4       System  0x840e97d72800  0x375c  IoCompletion    0x1f0003        -
4       System  0x840e99a92080  0x3760  Process 0x102a  SecurityHealth Pid 12036
4       System  0xb7081a5c8d80  0x3768  SymbolicLink    0xf0001 Session
4       System  0x840e90e10410  0x3770  Session 0x0     Session2
4       System  0x840e9cf27700  0x3774  IoCompletion    0x1f0003        -
4       System  0x840ea5357060  0x377c  Event   0x1f0003        -
4       System  0x840e9cf159c0  0x3784  IoCompletion    0x1f0003        -
4       System  0x840e98e90660  0x3788  Event   0x1f0003        -
4       System  0x840e98304a80  0x378c  ALPC Port       0x1f0001        -
4       System  0x840e9842baf0  0x3790  IoCompletionReserve     0xf0003 -
4       System  0x840e99c83080  0x3798  Process 0x1fffff        dwm.exe Pid 10628
4       System  0x840e974b2080  0x379c  Process 0x102a  UserOOBEBroker Pid 9620
4       System  0x840e99716f60  0x37a0  Event   0x1f0003        -
4       System  0x840e980788c0  0x37a4  CoreMessaging   0xf0000 -
4       System  0x840e9f28b080  0x37a8  Process 0x1fffff        WmiPrvSE.exe Pid 14028
4       System  0x840e99f67740  0x37ac  IoCompletion    0x1f0003        -
4       System  0x840e99c83080  0x37b0  Process 0x102a  dwm.exe Pid 10628
4       System  0x840e999d9560  0x37b4  Event   0x1f0003        -
4       System  0x840e98c48080  0x37b8  Process 0x102a  ApplicationFra Pid 4836
4       System  0x840e9a628c00  0x37bc  IoCompletion    0x1f0003        -
4       System  0x840e98bdf080  0x37c0  Process 0x0     SystemSettings Pid 10020
4       System  0x840e9c0cb080  0x37c4  Process 0x0     splunk-winevtl Pid 11256
4       System  0x840e999f5f60  0x37c8  Event   0x1f0003        -
4       System  0x840e98b51980  0x37d0  WaitCompletionPacket    0x1     -
4       System  0x840e9682f080  0x37d4  Process 0x102a  OneDrive.exe Pid 3320
4       System  0x840ea9ed28e0  0x37d8  Event   0x1f0003        -
4       System  0xb70809062060  0x37dc  Token   0xf01ff -
4       System  0x840e99f76480  0x37e0  IoCompletion    0x1f0003        -
4       System  0x840e9ba171e0  0x37e4  WaitCompletionPacket    0x1     -
4       System  0x840e990b28a0  0x37e8  WaitCompletionPacket    0x1     -
4       System  0x840e99f707c0  0x37ec  IoCompletion    0x1f0003        -
4       System  0x840e997162e0  0x37f0  Event   0x1f0003        -
4       System  0x840e997ade60  0x37f4  Event   0x1f0003        -
4       System  0x840e985cb7e0  0x37f8  FilterCommunicationPort 0x1f0001        -
4       System  0x840e9952c080  0x37fc  Process 0x102a  csrss.exe Pid 10732
4       System  0x840e990a5970  0x3804  WaitCompletionPacket    0x1     -
4       System  0x840e9c8964e0  0x3808  Event   0x1f0003        -
4       System  0x840e9ba2f160  0x3814  WaitCompletionPacket    0x1     -
4       System  0x840e90ee7240  0x3818  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.0.regtrans-ms
4       System  0x840e99fc6560  0x3820  Event   0x1f0003        -
4       System  0x840e9a94c160  0x3828  Event   0x1f0003        -
4       System  0x840ea3cdf7e0  0x382c  Event   0x1f0003        -
4       System  0x840e96956440  0x3830  IoCompletion    0x1f0003        -
4       System  0x840e96d56340  0x3834  Process 0x102a  SenseTVM.exe Pid 1552
4       System  0x840e98a2f280  0x3838  IoCompletion    0x1f0003        -
4       System  0x840e9c441080  0x3844  Process 0x102a  SearchApp.exe Pid 9092
4       System  0x840e99e240c0  0x3848  Process 0x102a  ScreenConnect. Pid 6988
4       System  0x840e97e8c080  0x384c  Process 0x0     svchost.exe Pid 3864
4       System  0x840e9ba24520  0x3850  WaitCompletionPacket    0x1     -
4       System  0x840e99c83080  0x3858  Process 0x40    dwm.exe Pid 10628
4       System  0x840e9c8bd2f0  0x3860  File    0x13019f        \Device\clfs\Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee29-852e-11ef-ba7c-bc2411845bc2}.TxR
4       System  0x840e990ab470  0x3864  WaitCompletionPacket    0x1     -
4       System  0x840e90b0d080  0x3868  Process 0x102a  svchost.exe Pid 9452
4       System  0x840e9a943460  0x386c  Event   0x1f0003        -
4       System  0x840e9908c330  0x3870  WaitCompletionPacket    0x1     -
4       System  0x840e9b5df1b0  0x3874  File    0x3     \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat.LOG1
4       System  0x840e9ac1af80  0x3878  WaitCompletionPacket    0x1     -
4       System  0x840e9b9afb70  0x387c  WaitCompletionPacket    0x1     -
4       System  0x840e9c8a4060  0x3880  Event   0x1f0003        -
4       System  0x840e97e9e940  0x3888  CoreMessaging   0xf0000 -
4       System  0x840e9b2df080  0x388c  Process 0x102a  MoUsoCoreWorke Pid 10144
4       System  0x840e99f61640  0x3890  IoCompletion    0x1f0003        -
4       System  0x840e97e9dfb0  0x3898  CoreMessaging   0xf0000 -
4       System  0x840e990a6cf0  0x38a0  WaitCompletionPacket    0x1     -
4       System  0x840e98a06bc0  0x38a4  IoCompletion    0x1f0003        -
4       System  0x840e9ac098d0  0x38a8  WaitCompletionPacket    0x1     -
4       System  0x840ea0869180  0x38ac  IoCompletion    0x1f0003        -
4       System  0x840e99f776c0  0x38b0  IoCompletion    0x1f0003        -
4       System  0x840ea534e4e0  0x38b4  Event   0x1f0003        -
4       System  0x840e98df3080  0x38b8  Process 0x102a  ShellExperienc Pid 704
4       System  0xb70801532630  0x38bc  Token   0xf01ff -
4       System  0x840e91c3b300  0x38c0  Process 0x0     svchost.exe Pid 8312
4       System  0x840e99f799c0  0x38c4  IoCompletion    0x1f0003        -
4       System  0x840e9b372080  0x38cc  Process 0x0     taskhostw.exe Pid 10476
4       System  0x840e9cf2fdc0  0x38d0  IoCompletion    0x1f0003        -
4       System  0x840e994d1080  0x38d4  Process 0x102a  LogonUI.exe Pid 10680
4       System  0x840e9ea14080  0x38d8  Process 0x1fffff        ScreenConnect. Pid 1772
4       System  0x840e9b739080  0x38dc  Process 0x102a  Microsoft.Phot Pid 11148
4       System  0x840e9c713560  0x38e0  Event   0x1f0003        -
4       System  0x840e9c7120e0  0x38e4  Event   0x1f0003        -
4       System  0xb70829d62a70  0x38e8  Token   0xf01ff -
4       System  0x840e9c5277c0  0x38ec  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000001.regtrans-ms
4       System  0x840e9e99e080  0x38f0  Process 0x102a  dllhost.exe Pid 7884
4       System  0x840e9a6294c0  0x38f4  IoCompletion    0x1f0003        -
4       System  0x840e9a617a40  0x38f8  IoCompletion    0x1f0003        -
4       System  0x840e9a61c800  0x38fc  IoCompletion    0x1f0003        -
4       System  0x840e9f3460c0  0x3900  Process 0x0     svchost.exe Pid 11596
4       System  0x840e98df3080  0x3904  Process 0x40    ShellExperienc Pid 704
4       System  0x840e9daca840  0x390c  IoCompletion    0x1f0003        -
4       System  0x840e976767c0  0x3910  IoCompletion    0x1f0003        -
4       System  0x840e9c47c940  0x3914  IoCompletion    0x1f0003        -
4       System  0x840e91c3b300  0x3918  Process 0x1fffff        svchost.exe Pid 8312
4       System  0x840e9ac23590  0x391c  WaitCompletionPacket    0x1     -
4       System  0x840e98e83ce0  0x3920  Event   0x1f0003        -
4       System  0xb7081792d770  0x3924  Section 0x5     -
4       System  0x840e99fd7d60  0x3928  Event   0x1f0003        -
4       System  0x840ea08666c0  0x392c  IoCompletion    0x1f0003        -
4       System  0x840e9f327080  0x3930  Process 0x0     ApplicationFra Pid 7492
4       System  0x840e97e9e8b0  0x3934  CoreMessaging   0xf0000 -
4       System  0x840e98079520  0x393c  CoreMessaging   0xf0000 -
4       System  0x840e9c707b60  0x3940  Event   0x1f0003        -
4       System  0x840e9856b2c0  0x3944  Process 0x102a  svchost.exe Pid 7780
4       System  0x840ea085eec0  0x3948  IoCompletion    0x1f0003        -
4       System  0x840e99bcb080  0x394c  Process 0x40    explorer.exe Pid 9852
4       System  0x840e99e240c0  0x3950  Process 0x102a  ScreenConnect. Pid 6988
4       System  0x840e99ac04c0  0x3954  IoCompletion    0x1f0003        -
4       System  0x840e9a94db60  0x3958  Event   0x1f0003        -
4       System  0x840e98b24800  0x3960  WaitCompletionPacket    0x1     -
4       System  0x840e9708cdf0  0x3964  ALPC Port       0x1f0001        -
4       System  0x840e99ae0900  0x3968  IoCompletion    0x1f0003        -
4       System  0x840e9ac2e2a0  0x396c  WaitCompletionPacket    0x1     -
4       System  0x840e9c456140  0x3970  IoCompletion    0x1f0003        -
4       System  0x840e98a21180  0x3974  IoCompletion    0x1f0003        -
4       System  0x840e98b3b4d0  0x3978  WaitCompletionPacket    0x1     -
4       System  0x840e91bc3770  0x3980  WaitCompletionPacket    0x1     -
4       System  0x840e999ede60  0x3988  Event   0x1f0003        -
4       System  0x840e98f962b0  0x398c  IoCompletionReserve     0xf0003 -
4       System  0x840e99fd3fe0  0x3990  Event   0x1f0003        -
4       System  0xb7081a787190  0x3994  Section 0x5     -
4       System  0x840e9b73a080  0x3998  Process 0x102a  svchost.exe Pid 5116
4       System  0x840e99840080  0x39a0  Process 0x102a  dllhost.exe Pid 2492
4       System  0x840e998450c0  0x39a4  Process 0x40    SearchApp.exe Pid 4236
4       System  0x840e9b6cd9b0  0x39a8  IoCompletionReserve     0xf0003 -
4       System  0x840e97360f80  0x39ac  IoCompletion    0x1f0003        -
4       System  0x840e99fd9560  0x39b0  Event   0x1f0003        -
4       System  0x840e970d2c00  0x39b4  IoCompletion    0x1f0003        -
4       System  0x840e99e51e70  0x39bc  IoCompletionReserve     0xf0003 -
4       System  0x840e9b805270  0x39c8  IoCompletionReserve     0xf0003 -
4       System  0x840e99738af0  0x39cc  ALPC Port       0x1f0001        -
4       System  0x840e9dabc940  0x39d0  IoCompletion    0x1f0003        -
4       System  0x840e98b3bcf0  0x39d4  WaitCompletionPacket    0x1     -
4       System  0xb7081792db30  0x39e0  Section 0x17    -
4       System  0x840e99fca660  0x39e4  Event   0x1f0003        -
4       System  0x840e9ba16410  0x39e8  WaitCompletionPacket    0x1     -
4       System  0x840e98b15080  0x39ec  Process 0x0     fontdrvhost.ex Pid 10704
4       System  0x840e993449e0  0x39f0  Event   0x1f0003        -
4       System  0x840e9907b7e0  0x39f4  WaitCompletionPacket    0x1     -
4       System  0x840e9a623900  0x39f8  IoCompletion    0x1f0003        -
4       System  0x840e998450c0  0x39fc  Process 0x102a  SearchApp.exe Pid 4236
4       System  0x840e9b98cba0  0x3a08  WaitCompletionPacket    0x1     -
4       System  0x840e97d56900  0x3a0c  IoCompletion    0x1f0003        -
4       System  0x840e9c5b50e0  0x3a14  Event   0x1f0003        -
4       System  0x840e9711ee60  0x3a18  Event   0x1f0003        -
4       System  0x840e9ba12310  0x3a1c  WaitCompletionPacket    0x1     -
4       System  0x840e9ba0fc10  0x3a24  WaitCompletionPacket    0x1     -
4       System  0x840e99acef40  0x3a28  IoCompletion    0x1f0003        -
4       System  0x840e9c35d460  0x3a30  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Microsoft\Windows\UsrClass.dat{d5baee2a-852e-11ef-ba7c-bc2411845bc2}.TMContainer00000000000000000001.regtrans-ms
4       System  0x840e9ac2bba0  0x3a34  WaitCompletionPacket    0x1     -
4       System  0x840e9ac0f8b0  0x3a38  WaitCompletionPacket    0x1     -
4       System  0x840e99fd75e0  0x3a3c  Event   0x1f0003        -
4       System  0x840e9a20b650  0x3a40  File    0x1f0006        \Device\NamedPipe\Sessions\1\AppContainerNamedObjects\S-1-15-2-2246530975-808720366-1776470054-230329187-4153223113-3550430174-4193313734
4       System  0xb7081c69be60  0x3a44  Directory       0xf     S-1-15-2-157618707-224758843-3162413466-2249835351-834486866-1672254014-2610752905
4       System  0x840e9c70b760  0x3a48  Event   0x1f0003        -
4       System  0xb7081b046a00  0x3a50  Directory       0xf     RPC Control
4       System  0x840e99840080  0x3a54  Process 0x102a  dllhost.exe Pid 2492
4       System  0x840e99c83080  0x3a58  Process 0x40    dwm.exe Pid 10628
4       System  0x840e9968d080  0x3a5c  Process 0x102a  splunkd.exe Pid 13620
4       System  0xb7081a5c8cd0  0x3a64  SymbolicLink    0xf0001 Local
4       System  0x840e9cf0c280  0x3a68  IoCompletion    0x1f0003        -
4       System  0x840e99037c80  0x3a6c  IoCompletion    0x1f0003        -
4       System  0x840e98c160e0  0x3a70  Event   0x1f0003        -
4       System  0x840e9676e080  0x3a74  Process 0x102a  svchost.exe Pid 2168
4       System  0xb7082389a650  0x3a78  Section 0x17    -
4       System  0x840e9cf2e1c0  0x3a7c  IoCompletion    0x1f0003        -
4       System  0x840e9c895e60  0x3a80  Event   0x1f0003        -
4       System  0x840e99c83080  0x3a88  Process 0x102a  dwm.exe Pid 10628
4       System  0x840e91a11080  0x3a8c  Process 0x102a  spoolsv.exe Pid 14536
4       System  0x840e9c458340  0x3a94  IoCompletion    0x1f0003        -
4       System  0x840e99da3080  0x3a98  Process 0x0     RuntimeBroker. Pid 4464
4       System  0x840e9a14a710  0x3aa4  File    0x1     \Device\HarddiskVolume3\ProgramData\Microsoft\Windows\AppRepository\Packages\Microsoft.Windows.ShellExperienceHost_10.0.19041.4239_neutral_neutral_cw5n1h2txyewy\ActivationStore.dat.LOG2
4       System  0x840e9b99a7d0  0x3aa8  WaitCompletionPacket    0x1     -
4       System  0x840ea3cf83e0  0x3aac  Event   0x1f0003        -
4       System  0x840e9c896160  0x3ab0  Event   0x1f0003        -
4       System  0x840e96696790  0x3abc  IoCompletionReserve     0xf0003 -
4       System  0x840e9c528ce0  0x3ac0  File    0x12019f        \Device\HarddiskVolume3\Users\IanVeep\NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM.blf
4       System  0x840e91e919b0  0x3acc  IoCompletionReserve     0xf0003 -
4       System  0x840e99fd5560  0x3ad0  Event   0x1f0003        -
4       System  0x840e9b9a3bb0  0x3ad4  WaitCompletionPacket    0x1     -
4       System  0xb7082389ab50  0x3ad8  Section 0x17    -
4       System  0x840e9a60cdc0  0x3adc  IoCompletion    0x1f0003        -
4       System  0x840e98fe4b60  0x3ae0  Event   0x1f0003        -
4       System  0x840e92e48080  0x3af0  Process 0x1028  svchost.exe Pid 1524
4       System  0xb708109dc7b0  0x3af8  SymbolicLink    0xf0001 Session
4       System  0x840e98b57a30  0x3afc  WaitCompletionPacket    0x1     -
4       System  0x840e97d59900  0x3b00  IoCompletion    0x1f0003        -
4       System  0x840e9934a860  0x3b08  Event   0x1f0003        -
4       System  0x840e90b0d080  0x3b0c  Process 0x1fffff        svchost.exe Pid 9452
4       System  0x840e9a630e80  0x3b1c  IoCompletion    0x1f0003        -
4       System  0x840e9b9a2c40  0x3b24  WaitCompletionPacket    0x1     -
4       System  0x840e99451a80  0x3b2c  ALPC Port       0x1f0001        -
4       System  0x840e9f7bf880  0x3b30  IoCompletion    0x1f0003        -
4       System  0x840e97e8c080  0x3b34  Process 0x102a  svchost.exe Pid 3864
4       System  0x840e91ee7080  0x3b38  Process 0x1fffff        Sysmon64.exe Pid 1540
4       System  0x840e9a948e60  0x3b3c  Event   0x1f0003        -
4       System  0x840e96817850  0x3b40  WaitCompletionPacket    0x1     -
4       System  0x840e9f327080  0x3b44  Process 0x102a  ApplicationFra Pid 7492
4       System  0xb7081f175120  0x3b48  Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\PEERDIST\SERVICE
4       System  0x840e9f2d7c60  0x3b4c  Event   0x1f0003        -
4       System  0x840e9c856a60  0x3b50  Event   0x1f0003        -
4       System  0x840e9f7ac200  0x3b54  IoCompletion    0x1f0003        -
4       System  0x840e92f5a760  0x3b58  Event   0x1f0003        -
4       System  0x840e9daf5100  0x3b5c  IoCompletion    0x1f0003        -
4       System  0x840e98078710  0x3b60  CoreMessaging   0xf0000 -
4       System  0x840e99fdb6e0  0x3b64  Event   0x1f0003        -
4       System  0x840e99840080  0x3b68  Process 0x102a  dllhost.exe Pid 2492
4       System  0x840ea9ecf0e0  0x3b6c  Event   0x1f0003        -
4       System  0x840e9709ba00  0x3b70  Timer   0x1f0003        -
4       System  0x840e9d8809e0  0x3b74  File    0x20003 \Device\HarddiskVolume3\Users\IanVeep\AppData\Local\Packages\Microsoft.Windows.ShellExperienceHost_cw5n1h2txyewy\Settings

```

Notes: This output is a lot so we wil use the`--pid=6899` flag to retrieve output relating to our suspicious `screenconnect` activity

`vol -f iveep.mem windows.handles --pid=6988`

```
Volatility 3 Framework 2.26.2

PID     Process Offset  HandleValue     Type    GrantedAccess   Name

6988    ScreenConnect.  0x840e9c84bfe0  0x4     Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c84d960  0x8     Event   0x1f0003        -
6988    ScreenConnect.  0x840e9ba0dc60  0xc     WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9f7bc400  0x10    IoCompletion    0x1f0003        -
6988    ScreenConnect.  0x840e98e68ae0  0x14    TpWorkerFactory 0xf00ff -
6988    ScreenConnect.  0x840eaa1edb20  0x18    IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba0fe80  0x1c    WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840eaa1eca20  0x20    IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba0f730  0x24    WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9f7bd9f0  0x28    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bd670  0x2c    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bc4f0  0x30    EtwRegistration 0x804   -
6988    ScreenConnect.  0xb708012fede0  0x34    Directory       0x3     KnownDlls
6988    ScreenConnect.  0x840e9c84dd60  0x38    Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c84dfe0  0x3c    Event   0x1f0003        -
6988    ScreenConnect.  0x840ea1843d40  0x40    File    0x100020        \Device\HarddiskVolume3\Windows
6988    ScreenConnect.  0x840e9c848a60  0x44    Event   0x1f0003        -
6988    ScreenConnect.  0xb708015ff920  0x48    Directory       0x3     KnownDlls32
6988    ScreenConnect.  0x840e9c84d260  0x4c    Event   0x1f0003        -
6988    ScreenConnect.  0x840e9ba0f320  0x50    WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9f7bc780  0x54    IoCompletion    0x1f0003        -
6988    ScreenConnect.  0x840e9df29d80  0x58    TpWorkerFactory 0xf00ff -
6988    ScreenConnect.  0x840e972e93e0  0x5c    IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba0f3f0  0x60    WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e972e9e80  0x64    IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba0f800  0x68    WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9f7bdad0  0x6c    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bd750  0x70    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bc870  0x74    EtwRegistration 0x804   -
6988    ScreenConnect.  0xb708015ff920  0x78    Directory       0x3     KnownDlls32
6988    ScreenConnect.  0x840e9c84d3e0  0x7c    Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c84d2e0  0x80    Event   0x1f0003        -
6988    ScreenConnect.  0x840ea1845740  0x84    File    0x100020        \Device\HarddiskVolume3\Windows\SysWOW64
6988    ScreenConnect.  0x840e9f7bd130  0x88    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9a78fa50  0x8c    Mutant  0x1f0001        SM0:6988:168:WilStaging_02
6988    ScreenConnect.  0x840e9724dd90  0x90    ALPC Port       0x1f0001        -
6988    ScreenConnect.  0xb7080435d660  0x94    Directory       0xf     BaseNamedObjects
6988    ScreenConnect.  0x840e9e7389b0  0x98    Semaphore       0x1f0003        SM0:6988:168:WilStaging_02_p0
6988    ScreenConnect.  0x840e9f7bccd0  0x9c    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bd830  0xa0    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bc950  0xa4    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bdd70  0xa8    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bca30  0xac    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bdbb0  0xb0    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bcb10  0xb4    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bd910  0xb8    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bd200  0xbc    IoCompletion    0x1f0003        -
6988    ScreenConnect.  0x840e9f7bcdb0  0xc0    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e976309f0  0xc4    TpWorkerFactory 0xf00ff -
6988    ScreenConnect.  0x840e972ea920  0xc8    IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba0f8d0  0xcc    WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e972edc20  0xd0    IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba10360  0xd4    WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0xb7082d62d3b0  0xd8    Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NLS\SORTING\VERSIONS
6988    ScreenConnect.  0x840e9f7bce90  0xdc    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bc250  0xe0    EtwRegistration 0x804   -
6988    ScreenConnect.  0xb7082d629b60  0xe4    Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\SESSION MANAGER
6988    ScreenConnect.  0x840e9f7bdc90  0xe8    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9e73abf0  0xec    Mutant  0x1f0001        -
6988    ScreenConnect.  0x840e9f7bcf70  0xf0    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bf430  0xf4    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bee10  0xf8    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bf7b0  0xfc    EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7be1d0  0x100   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7be9b0  0x104   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9c898fe0  0x10c   Semaphore       0x100003        -
6988    ScreenConnect.  0x840e9c8985e0  0x110   Semaphore       0x100003        -
6988    ScreenConnect.  0x840e9c8990e0  0x114   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea18431e0  0x118   File    0x100003        \Device\KsecDD
6988    ScreenConnect.  0x840e9f7be0f0  0x11c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bf6d0  0x120   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bf190  0x124   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7be2b0  0x128   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7be390  0x12c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9ee163a0  0x130   File    0x120089        \Device\HarddiskVolume3\Program Files (x86)\ScreenConnect Client (a17d7bba0dc91adf)\ScreenConnect.Windows.dll
6988    ScreenConnect.  0x840e9c89a0e0  0x134   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9ba0fa70  0x138   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0xb7082d62e180  0x13c   Key     0x20019 MACHINE
6988    ScreenConnect.  0xb7082d62ef50  0x140   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\OLE
6988    ScreenConnect.  0x840e9c899ae0  0x144   Event   0x1f0003        -
6988    ScreenConnect.  0xb7082d630490  0x148   Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS\SOFTWARE\MICROSOFT
6988    ScreenConnect.  0xb7082d630af0  0x14c   Key     0x20019 USER\.DEFAULT\SOFTWARE\CLASSES\LOCAL SETTINGS
6988    ScreenConnect.  0x840e9c89af60  0x150   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9f7bf0b0  0x154   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7be7f0  0x158   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bf270  0x15c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9c89a260  0x160   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c89a9e0  0x164   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c89a5e0  0x168   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c89ac60  0x16c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c89b560  0x170   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c89dbe0  0x174   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9f7bea90  0x178   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bed30  0x17c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bde50  0x180   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bec50  0x184   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7be470  0x188   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7be550  0x18c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bdf30  0x190   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb70831f081c0  0x194   Key     0xf003f USER\.DEFAULT
6988    ScreenConnect.  0x840e9f7c0d90  0x198   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9b2ec830  0x19c   IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba117b0  0x1a0   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9c89f2e0  0x1a4   Event   0x1f0003        -
6988    ScreenConnect.  0xb7082d62f7d0  0x1a8   Key     0x1     MACHINE\SYSTEM\CONTROLSET001\CONTROL\NLS\CUSTOMLOCALE
6988    ScreenConnect.  0x840e9ba11bc0  0x1ac   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e98c93a30  0x1b0   TpWorkerFactory 0xf00ff -
6988    ScreenConnect.  0x840e9f7c0cc0  0x1b4   IoCompletion    0x1f0003        -
6988    ScreenConnect.  0x840e9c8a57e0  0x1b8   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9b2ece90  0x1bc   IRTimer 0x100002        -
6988    ScreenConnect.  0x840e99f7da90  0x1c0   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9b39ec90  0x1c4   Mutant  0x1f0001        -
6988    ScreenConnect.  0x840ea3cf4760  0x1c8   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9f7bf510  0x1cc   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bf5f0  0x1d0   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7beb70  0x1d4   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb7082d626750  0x1d8   Key     0x9     MACHINE\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION\IMAGE FILE EXECUTION OPTIONS
6988    ScreenConnect.  0x840e9c8a20e0  0x1dc   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9f7bf880  0x1e0   IoCompletion    0x1f0003        -
6988    ScreenConnect.  0x840e920dd8e0  0x1e4   WindowStation   0xf016e Service-0x0-3e7$
6988    ScreenConnect.  0x840e922fe430  0x1e8   Desktop 0xf00cf Default
6988    ScreenConnect.  0x840e920dd8e0  0x1ec   WindowStation   0xf016e Service-0x0-3e7$
6988    ScreenConnect.  0x840e9f7be630  0x1f0   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bf970  0x1f4   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7be710  0x1f8   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7be8d0  0x1fc   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7beef0  0x200   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c1100  0x204   IoCompletion    0x1f0003        -
6988    ScreenConnect.  0x840e9c441d70  0x208   TpWorkerFactory 0xf00ff -
6988    ScreenConnect.  0x840e9b2ec500  0x20c   IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba109e0  0x210   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9b2ec720  0x214   IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba12240  0x218   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9c8a1360  0x21c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9f7c0bd0  0x220   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bff90  0x224   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c11f0  0x228   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9c8a3de0  0x22c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9f3245c0  0x230   Thread  0x1fffff        Tid 14500 Pid 6988
6988    ScreenConnect.  0x840e9f7c0770  0x234   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c0a10  0x238   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e99735070  0x23c   ALPC Port       0x1f0001        -
6988    ScreenConnect.  0x840ea184f820  0x240   File    0x100001        \Device\CNG
6988    ScreenConnect.  0x840e9b39d9f0  0x244   Event   0x1f0003        6988Stop
6988    ScreenConnect.  0x840e9b39dc70  0x248   Event   0x1f0003        6988SessionChange
6988    ScreenConnect.  0x840e9b39e170  0x24c   Event   0x1f0003        6988Shutdown
6988    ScreenConnect.  0x840ea3ce1460  0x250   Event   0x1f0003        -
6988    ScreenConnect.  0x840e99d58080  0x254   Thread  0x1fffff        Tid 14724 Pid 6988
6988    ScreenConnect.  0x840e9f7c0300  0x258   IoCompletion    0x1f0003        -
6988    ScreenConnect.  0xb70831f050e0  0x25c   Key     0x20019 MACHINE\SOFTWARE\WOW6432NODE\MICROSOFT\.NETFRAMEWORK
6988    ScreenConnect.  0xb70831f093d0  0x260   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\CONTROL\NLS\SORTING\IDS
6988    ScreenConnect.  0x840ea3cf4ce0  0x264   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf4d60  0x268   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf54e0  0x26c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9f7c05b0  0x270   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c04d0  0x274   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bfc10  0x278   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bfcf0  0x27c   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb7082ef91920  0x280   Directory       0xf000f -
6988    ScreenConnect.  0xb70818885f50  0x284   Section 0xf0007 Cor_Private_IPCBlock_v4_6988
6988    ScreenConnect.  0xb7081888a810  0x288   Section 0xf0007 Cor_SxSPublic_IPCBlock
6988    ScreenConnect.  0x840ea3cf59e0  0x28c   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf5f60  0x290   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9b3a1ad0  0x294   Mutant  0x1f0001        -
6988    ScreenConnect.  0x840ea3cf5b60  0x298   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf5fe0  0x29c   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf57e0  0x2a0   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf53e0  0x2a4   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf5960  0x2a8   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf5ce0  0x2ac   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf5d60  0x2b0   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf5ae0  0x2b4   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf5de0  0x2b8   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf5ee0  0x2bc   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9b3a2b10  0x2c0   Mutant  0x1f0001        -
6988    ScreenConnect.  0x840e9975f5c0  0x2c4   Thread  0x1fffff        Tid 7708 Pid 6988
6988    ScreenConnect.  0x840e99d58080  0x2c8   Thread  0x1fffff        Tid 14724 Pid 6988
6988    ScreenConnect.  0x840ea3cf6060  0x2cc   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf60e0  0x2d0   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf5560  0x2d4   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf5660  0x2d8   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf68e0  0x2dc   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf7060  0x2e0   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6360  0x2e4   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6660  0x2e8   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6e60  0x2ec   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf70e0  0x2f0   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf7160  0x2f4   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6c60  0x2f8   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6ae0  0x2fc   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6b60  0x300   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf61e0  0x304   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6260  0x308   Event   0x1f0003        -
6988    ScreenConnect.  0x840e90291ba0  0x30c   Event   0x100001        LowMemoryCondition
6988    ScreenConnect.  0x840ea3cf65e0  0x310   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf62e0  0x314   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6ce0  0x318   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6460  0x31c   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6de0  0x320   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf6960  0x324   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf74e0  0x328   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf75e0  0x32c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e99407080  0x330   Thread  0x1fffff        Tid 5036 Pid 6988
6988    ScreenConnect.  0x840ea3cf8060  0x334   Event   0x1f0003        -
6988    ScreenConnect.  0xb70831f09810  0x338   Key     0x20019 USER\.DEFAULT\CONTROL PANEL\INTERNATIONAL
6988    ScreenConnect.  0x840ea3cf51e0  0x33c   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf7fe0  0x340   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf7c60  0x344   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf7ce0  0x348   Event   0x1f0003        -
6988    ScreenConnect.  0x840e99407080  0x34c   Thread  0x1fffff        Tid 5036 Pid 6988
6988    ScreenConnect.  0x840e9b3a2ef0  0x354   Event   0x1f0003        CPFATE_6988_v4.0.30319
6988    ScreenConnect.  0x840ea3cf7760  0x358   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf7460  0x35c   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf7860  0x360   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf7660  0x364   Event   0x1f0003        -
6988    ScreenConnect.  0xb70831f1bb30  0x368   Key     0xf003f MACHINE
6988    ScreenConnect.  0xb70831f1bc40  0x36c   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\FUSION\PUBLISHERPOLICY\DEFAULT
6988    ScreenConnect.  0xb7081804a2f0  0x370   Key     0x20019 MACHINE\SOFTWARE\WOW6432NODE\MICROSOFT\INTERNET EXPLORER\SECURITY
6988    ScreenConnect.  0x840e9f7c0f50  0x374   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840ea0603a40  0x378   File    0x120089        \Device\HarddiskVolume3\Program Files (x86)\ScreenConnect Client (a17d7bba0dc91adf)\ScreenConnect.ClientService.dll
6988    ScreenConnect.  0xb708142c11d0  0x37c   Section 0xf0005 -
6988    ScreenConnect.  0x840e9f7c0230  0x380   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c1490  0x384   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840ea0606b00  0x388   File    0x100001        \Device\KsecDD
6988    ScreenConnect.  0xb70818042e10  0x38c   Key     0x2001f USER\.DEFAULT\SOFTWARE\MICROSOFT\WINDOWS\CURRENTVERSION\INTERNET SETTINGS\ZONEMAP
6988    ScreenConnect.  0x840e97fb1be0  0x390   Semaphore       0x100003        -
6988    ScreenConnect.  0x840e97fb0560  0x394   Semaphore       0x100003        -
6988    ScreenConnect.  0x840e97fb0c60  0x398   Semaphore       0x100003        -
6988    ScreenConnect.  0x840e97fb2160  0x39c   Semaphore       0x100003        -
6988    ScreenConnect.  0x840e9f7bfdd0  0x3a0   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb70818048ca0  0x3a4   Key     0x20019 MACHINE\SOFTWARE\WOW6432NODE\MICROSOFT\WINDOWS\CURRENTVERSION\INTERNET SETTINGS\ZONEMAP
6988    ScreenConnect.  0x840e9f7c0850  0x3a8   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c0690  0x3ac   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c0e70  0x3b0   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c03f0  0x3b4   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb70818040e30  0x3b8   Key     0x1     MACHINE\SOFTWARE\WOW6432NODE\MICROSOFT\INTERNET EXPLORER\MAIN\FEATURECONTROL
6988    ScreenConnect.  0xb7081803e190  0x3bc   Key     0x20019 MACHINE\SOFTWARE\POLICIES
6988    ScreenConnect.  0xb70818047a90  0x3c0   Key     0x20019 USER\.DEFAULT\SOFTWARE\POLICIES
6988    ScreenConnect.  0xb70818048860  0x3c4   Key     0x20019 USER\.DEFAULT\SOFTWARE
6988    ScreenConnect.  0xb7081804a1e0  0x3c8   Key     0x20019 MACHINE\SOFTWARE\WOW6432NODE
6988    ScreenConnect.  0xb70818049c90  0x3cc   Key     0x20019 MACHINE\SOFTWARE\WOW6432NODE\MICROSOFT\INTERNET EXPLORER\MAIN
6988    ScreenConnect.  0xb7081804bc70  0x3d0   Key     0x20019 MACHINE\SOFTWARE\POLICIES\MICROSOFT\WINDOWS\CURRENTVERSION\INTERNET SETTINGS
6988    ScreenConnect.  0xb7081804a730  0x3d4   Key     0x20019 USER\.DEFAULT\SOFTWARE\MICROSOFT\WINDOWS\CURRENTVERSION\INTERNET SETTINGS
6988    ScreenConnect.  0xb7081804aa60  0x3d8   Key     0x20019 MACHINE\SOFTWARE\WOW6432NODE\MICROSOFT\WINDOWS\CURRENTVERSION\INTERNET SETTINGS
6988    ScreenConnect.  0x840e9721d260  0x3dc   Mutant  0x1f0001        ZonesLockedCacheCounterMutex
6988    ScreenConnect.  0x840e9721c8a0  0x3e0   Mutant  0x1f0001        ZonesCacheCounterMutex
6988    ScreenConnect.  0x840e97ae5520  0x3e8   ALPC Port       0x1f0001        -
6988    ScreenConnect.  0x840e97fb1c60  0x3ec   Semaphore       0x100003        -
6988    ScreenConnect.  0x840e97fb1560  0x3f0   Semaphore       0x100003        -
6988    ScreenConnect.  0x840e9f7c0070  0x3f4   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb70807935e90  0x3fc   Section 0xf0007 UrlZonesSM_IVEEP$
6988    ScreenConnect.  0x840e9f7c12d0  0x40c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bfeb0  0x410   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9a63d350  0x414   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e92f36910  0x418   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e99f7db70  0x41c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e99f7dd30  0x420   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c13b0  0x424   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c1570  0x428   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bfa50  0x42c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7bfb30  0x430   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c2990  0x434   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c3090  0x438   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c1b90  0x43c   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb70804944fb0  0x440   Section 0x6     windows_shell_global_counters
6988    ScreenConnect.  0x840e9f7c1ab0  0x444   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c1d50  0x448   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb7080a4073f0  0x44c   Key     0x1     USER\.DEFAULT\SOFTWARE\MICROSOFT\WINDOWS\CURRENTVERSION\EXPLORER
6988    ScreenConnect.  0x840e9f7c2450  0x450   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840ea9ed3d60  0x454   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea0615500  0x458   File    0x120089        \Device\HarddiskVolume3\Windows\assembly\GAC_MSIL\CrowdStrike.Sensor.ScriptControl\7.21.19205.0__02b60967b1f68187\CrowdStrike.Sensor.ScriptControl.dll
6988    ScreenConnect.  0x840eab2dfde0  0x45c   Semaphore       0x100003        -
6988    ScreenConnect.  0x840e9f7c2370  0x464   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c28b0  0x468   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840ea060b900  0x46c   File    0x120089        \Device\HarddiskVolume3\Program Files (x86)\ScreenConnect Client (a17d7bba0dc91adf)\ScreenConnect.Core.dll
6988    ScreenConnect.  0x840e9ee1cba0  0x470   File    0x120089        \Device\HarddiskVolume3\Windows\Microsoft.NET\assembly\GAC_MSIL\System.Windows.Forms\v4.0_4.0.0.0__b77a5c561934e089\System.Windows.Forms.dll
6988    ScreenConnect.  0xb708142c4510  0x474   Section 0xf0005 -
6988    ScreenConnect.  0xb708142d3f10  0x478   Section 0xf0005 -
6988    ScreenConnect.  0x840e9f94f820  0x47c   File    0x100001        \Device\HarddiskVolume3\Windows\System32\en-US\winnlsres.dll.mui
6988    ScreenConnect.  0xb7082ec2cc30  0x480   Section 0xf0005 -
6988    ScreenConnect.  0x840e9e303700  0x484   File    0x120089        \Device\HarddiskVolume3\Windows\Microsoft.NET\assembly\GAC_MSIL\System.Security\v4.0_4.0.0.0__b03f5f7f11d50a3a\System.Security.dll
6988    ScreenConnect.  0xb7082ec2dc70  0x488   Section 0xf0005 -
6988    ScreenConnect.  0x840e9bcb1c50  0x48c   Mutant  0x1f0001        SM0:6988:64:WilError_03
6988    ScreenConnect.  0x840e9cfabfb0  0x490   Semaphore       0x1f0003        SM0:6988:64:WilError_03_p0
6988    ScreenConnect.  0x840e9f7892d0  0x494   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840ea0a5b4a0  0x498   File    0x120089        \Device\HarddiskVolume3\Windows\Microsoft.NET\assembly\GAC_MSIL\System.Drawing\v4.0_4.0.0.0__b03f5f7f11d50a3a\System.Drawing.dll
6988    ScreenConnect.  0x840eab2e24e0  0x49c   Semaphore       0x100003        -
6988    ScreenConnect.  0x840e9f7c1c70  0x4a0   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c1e30  0x4a4   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c20d0  0x4a8   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c2290  0x4ac   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c2ed0  0x4b0   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c1810  0x4b4   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c1f10  0x4b8   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c2530  0x4bc   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c18f0  0x4c0   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7c2610  0x4c4   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb70832f21be0  0x4c8   Key     0xf003f USER
6988    ScreenConnect.  0xb7082ec26fb0  0x4cc   Section 0xf0005 -
6988    ScreenConnect.  0x840e9f78f690  0x4d4   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9c8a6a60  0x4d8   Event   0x1f0003        -
6988    ScreenConnect.  0xb70821115450  0x4dc   Section 0xf0005 -
6988    ScreenConnect.  0xb7082186b070  0x4e0   Section 0xf0005 -
6988    ScreenConnect.  0x840e9c8a70e0  0x4e4   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9f4602e0  0x4e8   File    0x120089        \Device\HarddiskVolume3\Program Files (x86)\ScreenConnect Client (a17d7bba0dc91adf)\ScreenConnect.Client.dll
6988    ScreenConnect.  0x840e9f784410  0x4f0   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f78f850  0x4f4   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f793bb0  0x4f8   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f793910  0x4fc   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7932f0  0x500   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f7925d0  0x504   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb708130060d0  0x508   Key     0x20019 MACHINE\SOFTWARE\POLICIES\MICROSOFT
6988    ScreenConnect.  0xb7081300a1a0  0x50c   Key     0x20019 MACHINE\SOFTWARE\POLICIES\MICROSOFT\WINDOWS DEFENDER\POLICY MANAGER
6988    ScreenConnect.  0xb7081300e6b0  0x510   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\WINDOWS DEFENDER
6988    ScreenConnect.  0xb70813012340  0x514   Key     0x20019 MACHINE\SOFTWARE\MICROSOFT\WINDOWS DEFENDER
6988    ScreenConnect.  0x840e9c846ce0  0x518   Event   0x1f0003        -
6988    ScreenConnect.  0xb70813012ab0  0x51c   Key     0x10    MACHINE\SOFTWARE\POLICIES\MICROSOFT\WINDOWS DEFENDER\POLICY MANAGER
6988    ScreenConnect.  0x840e9ba1aed0  0x520   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9ba1c0b0  0x524   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9f794100  0x528   IoCompletion    0x1f0003        -
6988    ScreenConnect.  0x840e994207b0  0x52c   TpWorkerFactory 0xf00ff -
6988    ScreenConnect.  0x840e995a2190  0x530   IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba1a9f0  0x534   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e995a3e40  0x538   IRTimer 0x100002        -
6988    ScreenConnect.  0x840e9ba1b890  0x53c   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9c846ee0  0x540   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9ba13760  0x544   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0xb70813012bc0  0x548   Key     0x10    MACHINE\SOFTWARE\MICROSOFT\WINDOWS DEFENDER
6988    ScreenConnect.  0x840e9c846de0  0x54c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c846de0  0x550   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c847ae0  0x554   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9944e080  0x558   Thread  0x1fffff        Tid 16220 Pid 6988
6988    ScreenConnect.  0x840e9f095c90  0x55c   Mutant  0x1f0001        -
6988    ScreenConnect.  0x840ea1845dc0  0x560   File    0x100001        \Device\HarddiskVolume3\Windows\System32\en-US\KernelBase.dll.mui
6988    ScreenConnect.  0x840e9c8a6ce0  0x568   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a68e0  0x56c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9f796230  0x570   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9c8a6de0  0x574   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a69e0  0x578   Event   0x1f0003        -
6988    ScreenConnect.  0x840e978bd080  0x57c   Thread  0x1fffff        Tid 2968 Pid 6988
6988    ScreenConnect.  0x840e9c8a6be0  0x580   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a5be0  0x584   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a59e0  0x588   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a58e0  0x58c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e918ab710  0x590   Event   0x100000        TermSrvReadyEvent
6988    ScreenConnect.  0x840e9f796150  0x594   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9a63ab10  0x598   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9b39d9f0  0x59c   Event   0x100002        6988Stop
6988    ScreenConnect.  0x840e9b39dc70  0x5a0   Event   0x100002        6988SessionChange
6988    ScreenConnect.  0x840e9b39e170  0x5a4   Event   0x100002        6988Shutdown
6988    ScreenConnect.  0x840e9c8a67e0  0x5a8   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a65e0  0x5ac   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a62e0  0x5b0   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a6ae0  0x5b4   Event   0x1f0003        -
6988    ScreenConnect.  0x840e99308080  0x5b8   Thread  0x1fffff        Tid 11612 Pid 6988
6988    ScreenConnect.  0x840e9c8a6b60  0x5bc   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a6360  0x5c0   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a63e0  0x5c4   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a6460  0x5c8   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a7be0  0x5cc   Event   0x1f0003        -
6988    ScreenConnect.  0x840e96ea8080  0x5d0   Thread  0x1fffff        Tid 7796 Pid 6988
6988    ScreenConnect.  0x840e9c8a7760  0x5d4   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a7ce0  0x5d8   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a7ee0  0x5dc   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a71e0  0x5e0   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a8be0  0x5e4   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a8ce0  0x5e8   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8a9ce0  0x5ec   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8acce0  0x5f0   Event   0x1f0003        -
6988    ScreenConnect.  0x840e99bf8080  0x5f4   Thread  0x1fffff        Tid 15028 Pid 6988
6988    ScreenConnect.  0x840e9944e080  0x5f8   Thread  0x1fffff        Tid 16220 Pid 6988
6988    ScreenConnect.  0x840ea3cf49e0  0x5fc   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9dfed2c0  0x600   Thread  0x1fffff        Tid 12524 Pid 6988
6988    ScreenConnect.  0x840ea3ce2960  0x604   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3ce1960  0x608   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3ce1860  0x60c   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3ce0a60  0x610   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea1852c20  0x614   File    0x120089        \Device\NamedPipe\6ff59a85-bc37-4cd4-35da-621d3b6cfc5d
6988    ScreenConnect.  0x840e9a63b9f0  0x618   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9a63a6b0  0x61c   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb7082d631ae0  0x620   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS\INTERFACES
6988    ScreenConnect.  0x840e96ea8080  0x624   Thread  0x1fffff        Tid 7796 Pid 6988
6988    ScreenConnect.  0x840e9a6272b0  0x62c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9a6269f0  0x630   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e978bd080  0x638   Thread  0x1fffff        Tid 2968 Pid 6988
6988    ScreenConnect.  0x840e9c8b0360  0x63c   Event   0x1f0003        -
6988    ScreenConnect.  0xb7082d62f170  0x640   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\SERVICES\WINSOCK2\PARAMETERS\PROTOCOL_CATALOG9
6988    ScreenConnect.  0x840e9c8b1660  0x644   Event   0x1f0003        -
6988    ScreenConnect.  0xb7082d631150  0x648   Key     0xf003f MACHINE\SYSTEM\CONTROLSET001\SERVICES\WINSOCK2\PARAMETERS\NAMESPACE_CATALOG5
6988    ScreenConnect.  0x840e9a63b830  0x64c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840ea3ce0260  0x650   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9a63f8f0  0x654   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840ea3ce22e0  0x658   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea9ed8460  0x65c   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea1851560  0x660   File    0x16019f        \Device\Afd\Endpoint
6988    ScreenConnect.  0x840e9f7852f0  0x664   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9a60c6d0  0x668   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9a60e110  0x66c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9745e860  0x670   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c8adee0  0x674   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9745e260  0x678   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea3cf7960  0x67c   Semaphore       0x1f0003        -
6988    ScreenConnect.  0x840e9f785910  0x680   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e9f784b10  0x68c   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e99d4dab0  0x690   ALPC Port       0x1f0001        -
6988    ScreenConnect.  0x840ea3cf7ee0  0x694   Event   0x1f0003        -
6988    ScreenConnect.  0x840e978bd080  0x698   Thread  0x1fffff        Tid 2968 Pid 6988
6988    ScreenConnect.  0xb7082d631bf0  0x69c   Key     0x20019 MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP6\PARAMETERS\INTERFACES
6988    ScreenConnect.  0x840ea18513c0  0x6a0   File    0x100080        \Device\Nsi
6988    ScreenConnect.  0x840e92d4e2a0  0x6a4   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840ea9ed4b60  0x6a8   Event   0x1f0003        -
6988    ScreenConnect.  0x840e98cf3080  0x6ac   Thread  0x1fffff        Tid 11384 Pid 6988
6988    ScreenConnect.  0x840e92d4fa40  0x6b0   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e97e60df0  0x6b8   ALPC Port       0x1f0001        -
6988    ScreenConnect.  0x840e99471080  0x6bc   Thread  0x1fffff        Tid 14268 Pid 6988
6988    ScreenConnect.  0x840e98cf3080  0x6c0   Thread  0x1fffff        Tid 11384 Pid 6988
6988    ScreenConnect.  0x840e9f785750  0x6c4   EtwRegistration 0x804   -
6988    ScreenConnect.  0xb7081ad18d80  0x6c8   Key     0x8     USER\.DEFAULT\SOFTWARE\MICROSOFT\WINDOWS NT\CURRENTVERSION
6988    ScreenConnect.  0x840e9a63fb90  0x6d0   EtwRegistration 0x804   -
6988    ScreenConnect.  0x840e91a7b070  0x6d8   ALPC Port       0x1f0001        -
6988    ScreenConnect.  0x840ea46e31e0  0x6dc   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea46e3ae0  0x6e0   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea46e38e0  0x6e4   Event   0x1f0003        -
6988    ScreenConnect.  0x840e98b40c90  0x6e8   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840ea46e3f60  0x6ec   Event   0x1f0003        -
6988    ScreenConnect.  0x840e98b40540  0x6f0   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840ea46e3a60  0x6f4   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea46e3260  0x6f8   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea46e3360  0x6fc   Event   0x1f0003        -
6988    ScreenConnect.  0x840e98b40610  0x700   WaitCompletionPacket    0x1     -
6988    ScreenConnect.  0x840e9745da60  0x710   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9745ea60  0x71c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e98cf3080  0x720   Thread  0x1fffff        Tid 11384 Pid 6988
6988    ScreenConnect.  0x840ea9ed9160  0x724   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c907e60  0x734   Event   0x1f0003        -
6988    ScreenConnect.  0x840e97f3d5c0  0x744   Thread  0x1fffff        Tid 15476 Pid 6988
6988    ScreenConnect.  0x840ea119eba0  0x74c   File    0x120196        \Device\NamedPipe\e3755dea-7bde-4deb-b26f-47bb2c89b003ServerRead
6988    ScreenConnect.  0x840ea119ce60  0x750   File    0x120089        \Device\NamedPipe\e3755dea-7bde-4deb-b26f-47bb2c89b003ServerWrite
6988    ScreenConnect.  0x840e9c9063e0  0x754   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c906560  0x758   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c9065e0  0x75c   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea46e3d60  0x794   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9fe48800  0x79c   File    0x120196        \Device\NamedPipe\9980373c-98d8-4227-a0e6-05d642de2ec1ServerRead
6988    ScreenConnect.  0x840e9fe47e40  0x7a0   File    0x120089        \Device\NamedPipe\9980373c-98d8-4227-a0e6-05d642de2ec1ServerWrite
6988    ScreenConnect.  0x840ea46e3560  0x7a4   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea46e3be0  0x7a8   Event   0x1f0003        -
6988    ScreenConnect.  0x840ea46e4060  0x7ac   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9f336080  0x7b0   Thread  0x1fffff        Tid 7640 Pid 6988
6988    ScreenConnect.  0x840ea9ed46e0  0x7b4   Event   0x1f0003        -
6988    ScreenConnect.  0x840e99308080  0x7b8   Thread  0x1fffff        Tid 11612 Pid 6988
6988    ScreenConnect.  0x840e974302c0  0x7c8   ALPC Port       0x1f0001        -
6988    ScreenConnect.  0x840ea53551e0  0x80c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c845a60  0x814   Event   0x1f0003        -
6988    ScreenConnect.  0x840e995e9080  0x81c   Thread  0x1fffff        Tid 10104 Pid 6988
6988    ScreenConnect.  0x840e9c5ba6e0  0x820   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c5bb6e0  0x824   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c5bb560  0x828   Event   0x1f0003        -
6988    ScreenConnect.  0x840e9c5bb7e0  0x82c   Event   0x1f0003        -
6988    ScreenConnect.  0x840e99b91080  0x830   Thread  0x1fffff        Tid 15588 Pid 6988
```

The output analysis of PID 6899 from the `windows.handles` command indicates that the process has access to a range of system files and directories, including registry keys and executable files, suggesting it may be interacting with critical system components, which indicates potentially malicious or unauthorized behavior.

#### Privileges
Since screenconnect has access to a range of system files and operations, it is important to further investigate its privileges and permissions using the `windows.privs` command to determine its level of access and potential security risks.

`vol -f iveep.mem windows.privs`

```
Volatility 3 Framework 2.26.2

PID     Process Value   Privilege       Attributes      Description

4       System  2       SeCreateTokenPrivilege  Present Create a token object
4       System  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
4       System  4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
4       System  5       SeIncreaseQuotaPrivilege        Present Increase quotas
4       System  6       SeMachineAccountPrivilege               Add workstations to the domain
4       System  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
4       System  8       SeSecurityPrivilege     Present Manage auditing and security log
4       System  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
4       System  10      SeLoadDriverPrivilege   Present Load and unload device drivers
4       System  11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
4       System  12      SeSystemtimePrivilege   Present Change the system time
4       System  13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
4       System  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
4       System  15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
4       System  16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
4       System  17      SeBackupPrivilege       Present Backup files and directories
4       System  18      SeRestorePrivilege      Present Restore files and directories
4       System  19      SeShutdownPrivilege     Present Shut down the system
4       System  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
4       System  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
4       System  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
4       System  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4       System  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4       System  25      SeUndockPrivilege       Present Remove computer from docking station
4       System  26      SeSyncAgentPrivilege            Synch directory service data
4       System  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4       System  28      SeManageVolumePrivilege Present Manage the files on a volume
4       System  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
4       System  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
4       System  31      SeTrustedCredManAccessPrivilege Present Access Credential Manager as a trusted caller
4       System  32      SeRelabelPrivilege      Present Modify the mandatory integrity level of an object
4       System  33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
4       System  34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
4       System  35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
4       System  36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
108     Registry        2       SeCreateTokenPrivilege          Create a token object
108     Registry        3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
108     Registry        4       SeLockMemoryPrivilege           Lock pages in memory
108     Registry        5       SeIncreaseQuotaPrivilege                Increase quotas
108     Registry        6       SeMachineAccountPrivilege               Add workstations to the domain
108     Registry        7       SeTcbPrivilege          Act as part of the operating system
108     Registry        8       SeSecurityPrivilege             Manage auditing and security log
108     Registry        9       SeTakeOwnershipPrivilege                Take ownership of files/objects
108     Registry        10      SeLoadDriverPrivilege           Load and unload device drivers
108     Registry        11      SeSystemProfilePrivilege                Profile system performance
108     Registry        12      SeSystemtimePrivilege           Change the system time
108     Registry        13      SeProfileSingleProcessPrivilege         Profile a single process
108     Registry        14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
108     Registry        15      SeCreatePagefilePrivilege               Create a pagefile
108     Registry        16      SeCreatePermanentPrivilege              Create permanent shared objects
108     Registry        17      SeBackupPrivilege               Backup files and directories
108     Registry        18      SeRestorePrivilege              Restore files and directories
108     Registry        19      SeShutdownPrivilege             Shut down the system
108     Registry        20      SeDebugPrivilege                Debug programs
108     Registry        21      SeAuditPrivilege                Generate security audits
108     Registry        22      SeSystemEnvironmentPrivilege            Edit firmware environment values
108     Registry        23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
108     Registry        24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
108     Registry        25      SeUndockPrivilege               Remove computer from docking station
108     Registry        26      SeSyncAgentPrivilege            Synch directory service data
108     Registry        27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
108     Registry        28      SeManageVolumePrivilege         Manage the files on a volume
108     Registry        29      SeImpersonatePrivilege          Impersonate a client after authentication
108     Registry        30      SeCreateGlobalPrivilege         Create global objects
108     Registry        31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
108     Registry        32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
108     Registry        33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
108     Registry        34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
108     Registry        35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
108     Registry        36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
536     smss.exe        2       SeCreateTokenPrivilege  Present Create a token object
536     smss.exe        3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
536     smss.exe        4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
536     smss.exe        5       SeIncreaseQuotaPrivilege        Present Increase quotas
536     smss.exe        6       SeMachineAccountPrivilege               Add workstations to the domain
536     smss.exe        7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
536     smss.exe        8       SeSecurityPrivilege     Present Manage auditing and security log
536     smss.exe        9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
536     smss.exe        10      SeLoadDriverPrivilege   Present Load and unload device drivers
536     smss.exe        11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
536     smss.exe        12      SeSystemtimePrivilege   Present Change the system time
536     smss.exe        13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
536     smss.exe        14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
536     smss.exe        15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
536     smss.exe        16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
536     smss.exe        17      SeBackupPrivilege       Present Backup files and directories
536     smss.exe        18      SeRestorePrivilege      Present Restore files and directories
536     smss.exe        19      SeShutdownPrivilege     Present Shut down the system
536     smss.exe        20      SeDebugPrivilege        Present,Enabled,Default Debug programs
536     smss.exe        21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
536     smss.exe        22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
536     smss.exe        23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
536     smss.exe        24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
536     smss.exe        25      SeUndockPrivilege       Present Remove computer from docking station
536     smss.exe        26      SeSyncAgentPrivilege            Synch directory service data
536     smss.exe        27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
536     smss.exe        28      SeManageVolumePrivilege Present Manage the files on a volume
536     smss.exe        29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
536     smss.exe        30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
536     smss.exe        31      SeTrustedCredManAccessPrivilege Present Access Credential Manager as a trusted caller
536     smss.exe        32      SeRelabelPrivilege      Present Modify the mandatory integrity level of an object
536     smss.exe        33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
536     smss.exe        34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
536     smss.exe        35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
536     smss.exe        36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
804     csrss.exe       2       SeCreateTokenPrivilege  Present Create a token object
804     csrss.exe       3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
804     csrss.exe       4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
804     csrss.exe       5       SeIncreaseQuotaPrivilege        Present Increase quotas
804     csrss.exe       6       SeMachineAccountPrivilege               Add workstations to the domain
804     csrss.exe       7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
804     csrss.exe       8       SeSecurityPrivilege     Present Manage auditing and security log
804     csrss.exe       9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
804     csrss.exe       10      SeLoadDriverPrivilege   Present Load and unload device drivers
804     csrss.exe       11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
804     csrss.exe       12      SeSystemtimePrivilege   Present Change the system time
804     csrss.exe       13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
804     csrss.exe       14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
804     csrss.exe       15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
804     csrss.exe       16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
804     csrss.exe       17      SeBackupPrivilege       Present Backup files and directories
804     csrss.exe       18      SeRestorePrivilege      Present Restore files and directories
804     csrss.exe       19      SeShutdownPrivilege     Present Shut down the system
804     csrss.exe       20      SeDebugPrivilege        Present,Enabled,Default Debug programs
804     csrss.exe       21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
804     csrss.exe       22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
804     csrss.exe       23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
804     csrss.exe       24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
804     csrss.exe       25      SeUndockPrivilege       Present Remove computer from docking station
804     csrss.exe       26      SeSyncAgentPrivilege            Synch directory service data
804     csrss.exe       27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
804     csrss.exe       28      SeManageVolumePrivilege Present Manage the files on a volume
804     csrss.exe       29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
804     csrss.exe       30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
804     csrss.exe       31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
804     csrss.exe       32      SeRelabelPrivilege      Present Modify the mandatory integrity level of an object
804     csrss.exe       33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
804     csrss.exe       34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
804     csrss.exe       35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
804     csrss.exe       36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
880     wininit.exe     2       SeCreateTokenPrivilege          Create a token object
880     wininit.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
880     wininit.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
880     wininit.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
880     wininit.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
880     wininit.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
880     wininit.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
880     wininit.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
880     wininit.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
880     wininit.exe     11      SeSystemProfilePrivilege        Default Profile system performance
880     wininit.exe     12      SeSystemtimePrivilege           Change the system time
880     wininit.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
880     wininit.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
880     wininit.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
880     wininit.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
880     wininit.exe     17      SeBackupPrivilege       Present Backup files and directories
880     wininit.exe     18      SeRestorePrivilege      Present Restore files and directories
880     wininit.exe     19      SeShutdownPrivilege     Present Shut down the system
880     wininit.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
880     wininit.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
880     wininit.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
880     wininit.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
880     wininit.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
880     wininit.exe     25      SeUndockPrivilege       Present Remove computer from docking station
880     wininit.exe     26      SeSyncAgentPrivilege            Synch directory service data
880     wininit.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
880     wininit.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
880     wininit.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
880     wininit.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
880     wininit.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
880     wininit.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
880     wininit.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
880     wininit.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
880     wininit.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
880     wininit.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
888     csrss.exe       2       SeCreateTokenPrivilege  Present Create a token object
888     csrss.exe       3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
888     csrss.exe       4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
888     csrss.exe       5       SeIncreaseQuotaPrivilege        Present Increase quotas
888     csrss.exe       6       SeMachineAccountPrivilege               Add workstations to the domain
888     csrss.exe       7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
888     csrss.exe       8       SeSecurityPrivilege     Present Manage auditing and security log
888     csrss.exe       9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
888     csrss.exe       10      SeLoadDriverPrivilege   Present Load and unload device drivers
888     csrss.exe       11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
888     csrss.exe       12      SeSystemtimePrivilege   Present Change the system time
888     csrss.exe       13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
888     csrss.exe       14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
888     csrss.exe       15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
888     csrss.exe       16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
888     csrss.exe       17      SeBackupPrivilege       Present Backup files and directories
888     csrss.exe       18      SeRestorePrivilege      Present Restore files and directories
888     csrss.exe       19      SeShutdownPrivilege     Present Shut down the system
888     csrss.exe       20      SeDebugPrivilege        Present,Enabled,Default Debug programs
888     csrss.exe       21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
888     csrss.exe       22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
888     csrss.exe       23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
888     csrss.exe       24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
888     csrss.exe       25      SeUndockPrivilege       Present Remove computer from docking station
888     csrss.exe       26      SeSyncAgentPrivilege            Synch directory service data
888     csrss.exe       27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
888     csrss.exe       28      SeManageVolumePrivilege Present Manage the files on a volume
888     csrss.exe       29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
888     csrss.exe       30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
888     csrss.exe       31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
888     csrss.exe       32      SeRelabelPrivilege      Present Modify the mandatory integrity level of an object
888     csrss.exe       33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
888     csrss.exe       34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
888     csrss.exe       35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
888     csrss.exe       36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
980     winlogon.exe    2       SeCreateTokenPrivilege          Create a token object
980     winlogon.exe    3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
980     winlogon.exe    4       SeLockMemoryPrivilege   Default Lock pages in memory
980     winlogon.exe    5       SeIncreaseQuotaPrivilege        Present Increase quotas
980     winlogon.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
980     winlogon.exe    7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
980     winlogon.exe    8       SeSecurityPrivilege     Present Manage auditing and security log
980     winlogon.exe    9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
980     winlogon.exe    10      SeLoadDriverPrivilege   Present Load and unload device drivers
980     winlogon.exe    11      SeSystemProfilePrivilege        Default Profile system performance
980     winlogon.exe    12      SeSystemtimePrivilege           Change the system time
980     winlogon.exe    13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
980     winlogon.exe    14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
980     winlogon.exe    15      SeCreatePagefilePrivilege       Default Create a pagefile
980     winlogon.exe    16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
980     winlogon.exe    17      SeBackupPrivilege       Present Backup files and directories
980     winlogon.exe    18      SeRestorePrivilege      Present Restore files and directories
980     winlogon.exe    19      SeShutdownPrivilege     Present Shut down the system
980     winlogon.exe    20      SeDebugPrivilege        Present,Enabled,Default Debug programs
980     winlogon.exe    21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
980     winlogon.exe    22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
980     winlogon.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
980     winlogon.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
980     winlogon.exe    25      SeUndockPrivilege       Present Remove computer from docking station
980     winlogon.exe    26      SeSyncAgentPrivilege            Synch directory service data
980     winlogon.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
980     winlogon.exe    28      SeManageVolumePrivilege Present Manage the files on a volume
980     winlogon.exe    29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
980     winlogon.exe    30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
980     winlogon.exe    31      SeTrustedCredManAccessPrivilege Present Access Credential Manager as a trusted caller
980     winlogon.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
980     winlogon.exe    33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
980     winlogon.exe    34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
980     winlogon.exe    35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
980     winlogon.exe    36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
1000    services.exe    2       SeCreateTokenPrivilege          Create a token object
1000    services.exe    3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1000    services.exe    4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
1000    services.exe    5       SeIncreaseQuotaPrivilege        Present Increase quotas
1000    services.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
1000    services.exe    7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
1000    services.exe    8       SeSecurityPrivilege     Present Manage auditing and security log
1000    services.exe    9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
1000    services.exe    10      SeLoadDriverPrivilege   Present Load and unload device drivers
1000    services.exe    11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
1000    services.exe    12      SeSystemtimePrivilege   Present Change the system time
1000    services.exe    13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
1000    services.exe    14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
1000    services.exe    15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
1000    services.exe    16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
1000    services.exe    17      SeBackupPrivilege       Present Backup files and directories
1000    services.exe    18      SeRestorePrivilege      Present Restore files and directories
1000    services.exe    19      SeShutdownPrivilege     Present Shut down the system
1000    services.exe    20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1000    services.exe    21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
1000    services.exe    22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
1000    services.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1000    services.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1000    services.exe    25      SeUndockPrivilege       Present Remove computer from docking station
1000    services.exe    26      SeSyncAgentPrivilege            Synch directory service data
1000    services.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1000    services.exe    28      SeManageVolumePrivilege Present Manage the files on a volume
1000    services.exe    29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1000    services.exe    30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1000    services.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1000    services.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1000    services.exe    33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
1000    services.exe    34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
1000    services.exe    35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
1000    services.exe    36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
548     lsass.exe       2       SeCreateTokenPrivilege  Present,Enabled Create a token object
548     lsass.exe       3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
548     lsass.exe       4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
548     lsass.exe       5       SeIncreaseQuotaPrivilege        Present Increase quotas
548     lsass.exe       6       SeMachineAccountPrivilege               Add workstations to the domain
548     lsass.exe       7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
548     lsass.exe       8       SeSecurityPrivilege     Present Manage auditing and security log
548     lsass.exe       9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
548     lsass.exe       10      SeLoadDriverPrivilege   Present Load and unload device drivers
548     lsass.exe       11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
548     lsass.exe       12      SeSystemtimePrivilege   Present Change the system time
548     lsass.exe       13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
548     lsass.exe       14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
548     lsass.exe       15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
548     lsass.exe       16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
548     lsass.exe       17      SeBackupPrivilege       Present Backup files and directories
548     lsass.exe       18      SeRestorePrivilege      Present Restore files and directories
548     lsass.exe       19      SeShutdownPrivilege     Present Shut down the system
548     lsass.exe       20      SeDebugPrivilege        Present,Enabled,Default Debug programs
548     lsass.exe       21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
548     lsass.exe       22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
548     lsass.exe       23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
548     lsass.exe       24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
548     lsass.exe       25      SeUndockPrivilege       Present Remove computer from docking station
548     lsass.exe       26      SeSyncAgentPrivilege            Synch directory service data
548     lsass.exe       27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
548     lsass.exe       28      SeManageVolumePrivilege Present Manage the files on a volume
548     lsass.exe       29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
548     lsass.exe       30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
548     lsass.exe       31      SeTrustedCredManAccessPrivilege Present Access Credential Manager as a trusted caller
548     lsass.exe       32      SeRelabelPrivilege      Present Modify the mandatory integrity level of an object
548     lsass.exe       33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
548     lsass.exe       34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
548     lsass.exe       35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
548     lsass.exe       36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
756     svchost.exe     2       SeCreateTokenPrivilege          Create a token object
756     svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
756     svchost.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
756     svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
756     svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
756     svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
756     svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
756     svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
756     svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
756     svchost.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
756     svchost.exe     12      SeSystemtimePrivilege   Present Change the system time
756     svchost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
756     svchost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
756     svchost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
756     svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
756     svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
756     svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
756     svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
756     svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
756     svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
756     svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
756     svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
756     svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
756     svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
756     svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
756     svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
756     svchost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
756     svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
756     svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
756     svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
756     svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
756     svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
756     svchost.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
756     svchost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
756     svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
800     fontdrvhost.ex  2       SeCreateTokenPrivilege          Create a token object
800     fontdrvhost.ex  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
800     fontdrvhost.ex  4       SeLockMemoryPrivilege           Lock pages in memory
800     fontdrvhost.ex  5       SeIncreaseQuotaPrivilege                Increase quotas
800     fontdrvhost.ex  6       SeMachineAccountPrivilege               Add workstations to the domain
800     fontdrvhost.ex  7       SeTcbPrivilege          Act as part of the operating system
800     fontdrvhost.ex  8       SeSecurityPrivilege             Manage auditing and security log
800     fontdrvhost.ex  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
800     fontdrvhost.ex  10      SeLoadDriverPrivilege           Load and unload device drivers
800     fontdrvhost.ex  11      SeSystemProfilePrivilege                Profile system performance
800     fontdrvhost.ex  12      SeSystemtimePrivilege           Change the system time
800     fontdrvhost.ex  13      SeProfileSingleProcessPrivilege         Profile a single process
800     fontdrvhost.ex  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
800     fontdrvhost.ex  15      SeCreatePagefilePrivilege               Create a pagefile
800     fontdrvhost.ex  16      SeCreatePermanentPrivilege              Create permanent shared objects
800     fontdrvhost.ex  17      SeBackupPrivilege               Backup files and directories
800     fontdrvhost.ex  18      SeRestorePrivilege              Restore files and directories
800     fontdrvhost.ex  19      SeShutdownPrivilege             Shut down the system
800     fontdrvhost.ex  20      SeDebugPrivilege                Debug programs
800     fontdrvhost.ex  21      SeAuditPrivilege                Generate security audits
800     fontdrvhost.ex  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
800     fontdrvhost.ex  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
800     fontdrvhost.ex  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
800     fontdrvhost.ex  25      SeUndockPrivilege               Remove computer from docking station
800     fontdrvhost.ex  26      SeSyncAgentPrivilege            Synch directory service data
800     fontdrvhost.ex  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
800     fontdrvhost.ex  28      SeManageVolumePrivilege         Manage the files on a volume
800     fontdrvhost.ex  29      SeImpersonatePrivilege          Impersonate a client after authentication
800     fontdrvhost.ex  30      SeCreateGlobalPrivilege         Create global objects
800     fontdrvhost.ex  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
800     fontdrvhost.ex  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
800     fontdrvhost.ex  33      SeIncreaseWorkingSetPrivilege   Present,Enabled Allocate more memory for user applications
800     fontdrvhost.ex  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
800     fontdrvhost.ex  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
800     fontdrvhost.ex  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
892     fontdrvhost.ex  2       SeCreateTokenPrivilege          Create a token object
892     fontdrvhost.ex  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
892     fontdrvhost.ex  4       SeLockMemoryPrivilege           Lock pages in memory
892     fontdrvhost.ex  5       SeIncreaseQuotaPrivilege                Increase quotas
892     fontdrvhost.ex  6       SeMachineAccountPrivilege               Add workstations to the domain
892     fontdrvhost.ex  7       SeTcbPrivilege          Act as part of the operating system
892     fontdrvhost.ex  8       SeSecurityPrivilege             Manage auditing and security log
892     fontdrvhost.ex  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
892     fontdrvhost.ex  10      SeLoadDriverPrivilege           Load and unload device drivers
892     fontdrvhost.ex  11      SeSystemProfilePrivilege                Profile system performance
892     fontdrvhost.ex  12      SeSystemtimePrivilege           Change the system time
892     fontdrvhost.ex  13      SeProfileSingleProcessPrivilege         Profile a single process
892     fontdrvhost.ex  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
892     fontdrvhost.ex  15      SeCreatePagefilePrivilege               Create a pagefile
892     fontdrvhost.ex  16      SeCreatePermanentPrivilege              Create permanent shared objects
892     fontdrvhost.ex  17      SeBackupPrivilege               Backup files and directories
892     fontdrvhost.ex  18      SeRestorePrivilege              Restore files and directories
892     fontdrvhost.ex  19      SeShutdownPrivilege             Shut down the system
892     fontdrvhost.ex  20      SeDebugPrivilege                Debug programs
892     fontdrvhost.ex  21      SeAuditPrivilege                Generate security audits
892     fontdrvhost.ex  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
892     fontdrvhost.ex  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
892     fontdrvhost.ex  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
892     fontdrvhost.ex  25      SeUndockPrivilege               Remove computer from docking station
892     fontdrvhost.ex  26      SeSyncAgentPrivilege            Synch directory service data
892     fontdrvhost.ex  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
892     fontdrvhost.ex  28      SeManageVolumePrivilege         Manage the files on a volume
892     fontdrvhost.ex  29      SeImpersonatePrivilege          Impersonate a client after authentication
892     fontdrvhost.ex  30      SeCreateGlobalPrivilege         Create global objects
892     fontdrvhost.ex  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
892     fontdrvhost.ex  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
892     fontdrvhost.ex  33      SeIncreaseWorkingSetPrivilege   Present,Enabled Allocate more memory for user applications
892     fontdrvhost.ex  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
892     fontdrvhost.ex  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
892     fontdrvhost.ex  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1076    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1076    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1076    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1076    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1076    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1076    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1076    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1076    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1076    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1076    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1076    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1076    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1076    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1076    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1076    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1076    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1076    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1076    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1076    svchost.exe     20      SeDebugPrivilege                Debug programs
1076    svchost.exe     21      SeAuditPrivilege                Generate security audits
1076    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1076    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1076    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1076    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1076    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1076    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1076    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1076    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1076    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1076    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1076    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1076    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1076    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1076    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1076    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1124    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1124    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1124    svchost.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
1124    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
1124    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1124    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
1124    svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
1124    svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
1124    svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
1124    svchost.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
1124    svchost.exe     12      SeSystemtimePrivilege   Present Change the system time
1124    svchost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
1124    svchost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
1124    svchost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
1124    svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
1124    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
1124    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
1124    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
1124    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1124    svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
1124    svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
1124    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1124    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1124    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
1124    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1124    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1124    svchost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
1124    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1124    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1124    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1124    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1124    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
1124    svchost.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
1124    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
1124    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
1204    dwm.exe 2       SeCreateTokenPrivilege          Create a token object
1204    dwm.exe 3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1204    dwm.exe 4       SeLockMemoryPrivilege           Lock pages in memory
1204    dwm.exe 5       SeIncreaseQuotaPrivilege                Increase quotas
1204    dwm.exe 6       SeMachineAccountPrivilege               Add workstations to the domain
1204    dwm.exe 7       SeTcbPrivilege          Act as part of the operating system
1204    dwm.exe 8       SeSecurityPrivilege             Manage auditing and security log
1204    dwm.exe 9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1204    dwm.exe 10      SeLoadDriverPrivilege           Load and unload device drivers
1204    dwm.exe 11      SeSystemProfilePrivilege                Profile system performance
1204    dwm.exe 12      SeSystemtimePrivilege           Change the system time
1204    dwm.exe 13      SeProfileSingleProcessPrivilege         Profile a single process
1204    dwm.exe 14      SeIncreaseBasePriorityPrivilege Present,Enabled Increase scheduling priority
1204    dwm.exe 15      SeCreatePagefilePrivilege               Create a pagefile
1204    dwm.exe 16      SeCreatePermanentPrivilege              Create permanent shared objects
1204    dwm.exe 17      SeBackupPrivilege               Backup files and directories
1204    dwm.exe 18      SeRestorePrivilege              Restore files and directories
1204    dwm.exe 19      SeShutdownPrivilege             Shut down the system
1204    dwm.exe 20      SeDebugPrivilege                Debug programs
1204    dwm.exe 21      SeAuditPrivilege                Generate security audits
1204    dwm.exe 22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1204    dwm.exe 23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1204    dwm.exe 24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1204    dwm.exe 25      SeUndockPrivilege               Remove computer from docking station
1204    dwm.exe 26      SeSyncAgentPrivilege            Synch directory service data
1204    dwm.exe 27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1204    dwm.exe 28      SeManageVolumePrivilege         Manage the files on a volume
1204    dwm.exe 29      SeImpersonatePrivilege          Impersonate a client after authentication
1204    dwm.exe 30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1204    dwm.exe 31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1204    dwm.exe 32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1204    dwm.exe 33      SeIncreaseWorkingSetPrivilege   Present,Enabled Allocate more memory for user applications
1204    dwm.exe 34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1204    dwm.exe 35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1204    dwm.exe 36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1292    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1292    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1292    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
1292    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1292    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1292    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
1292    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1292    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1292    svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
1292    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
1292    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1292    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
1292    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
1292    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
1292    svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
1292    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1292    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1292    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1292    svchost.exe     20      SeDebugPrivilege        Default Debug programs
1292    svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
1292    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1292    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1292    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1292    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1292    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1292    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1292    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1292    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1292    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
1292    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1292    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1292    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
1292    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
1292    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
1292    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
1320    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1320    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1320    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1320    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1320    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1320    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1320    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1320    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1320    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1320    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1320    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1320    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1320    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1320    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1320    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1320    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1320    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1320    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1320    svchost.exe     20      SeDebugPrivilege                Debug programs
1320    svchost.exe     21      SeAuditPrivilege                Generate security audits
1320    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1320    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1320    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1320    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1320    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1320    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1320    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1320    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
1320    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1320    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1320    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1320    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1320    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1320    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1320    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1364    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1364    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1364    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1364    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
1364    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1364    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1364    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1364    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1364    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1364    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1364    svchost.exe     12      SeSystemtimePrivilege   Present Change the system time
1364    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1364    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1364    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1364    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1364    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1364    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1364    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
1364    svchost.exe     20      SeDebugPrivilege                Debug programs
1364    svchost.exe     21      SeAuditPrivilege        Present Generate security audits
1364    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1364    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1364    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1364    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
1364    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1364    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1364    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1364    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1364    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1364    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1364    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1364    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
1364    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
1364    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1364    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1392    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1392    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1392    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1392    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1392    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1392    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1392    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1392    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1392    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1392    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1392    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1392    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1392    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1392    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1392    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1392    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1392    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1392    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1392    svchost.exe     20      SeDebugPrivilege                Debug programs
1392    svchost.exe     21      SeAuditPrivilege                Generate security audits
1392    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1392    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1392    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1392    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1392    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1392    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1392    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1392    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
1392    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1392    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1392    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1392    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1392    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1392    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1392    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1408    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1408    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1408    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
1408    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1408    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1408    svchost.exe     7       SeTcbPrivilege  Default Act as part of the operating system
1408    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1408    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1408    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1408    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
1408    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1408    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
1408    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
1408    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
1408    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
1408    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1408    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1408    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1408    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1408    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
1408    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1408    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1408    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1408    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1408    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1408    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1408    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1408    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1408    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1408    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1408    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1408    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
1408    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
1408    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
1408    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
1524    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1524    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1524    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1524    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1524    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1524    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1524    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1524    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1524    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1524    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1524    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1524    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1524    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1524    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1524    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1524    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1524    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1524    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1524    svchost.exe     20      SeDebugPrivilege                Debug programs
1524    svchost.exe     21      SeAuditPrivilege        Present Generate security audits
1524    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1524    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1524    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1524    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1524    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1524    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1524    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1524    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1524    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
1524    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1524    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1524    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1524    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1524    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1524    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1580    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1580    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1580    svchost.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
1580    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
1580    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1580    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
1580    svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
1580    svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
1580    svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
1580    svchost.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
1580    svchost.exe     12      SeSystemtimePrivilege   Present Change the system time
1580    svchost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
1580    svchost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
1580    svchost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
1580    svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
1580    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
1580    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
1580    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
1580    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1580    svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
1580    svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
1580    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1580    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1580    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
1580    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1580    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1580    svchost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
1580    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1580    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1580    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1580    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1580    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
1580    svchost.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
1580    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
1580    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
1604    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1604    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1604    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
1604    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1604    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1604    svchost.exe     7       SeTcbPrivilege  Default Act as part of the operating system
1604    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1604    svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
1604    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1604    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
1604    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1604    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
1604    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
1604    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
1604    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
1604    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
1604    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
1604    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1604    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1604    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
1604    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1604    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1604    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1604    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1604    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1604    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1604    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1604    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1604    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
1604    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1604    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1604    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
1604    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
1604    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
1604    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
1612    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1612    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1612    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1612    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1612    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1612    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1612    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1612    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1612    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1612    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1612    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1612    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1612    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1612    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1612    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1612    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1612    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1612    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1612    svchost.exe     20      SeDebugPrivilege                Debug programs
1612    svchost.exe     21      SeAuditPrivilege                Generate security audits
1612    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1612    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1612    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1612    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1612    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1612    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1612    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1612    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1612    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1612    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1612    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1612    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1612    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1612    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1612    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1652    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1652    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1652    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1652    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1652    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1652    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1652    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1652    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1652    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1652    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1652    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1652    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1652    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1652    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1652    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1652    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1652    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1652    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1652    svchost.exe     20      SeDebugPrivilege                Debug programs
1652    svchost.exe     21      SeAuditPrivilege                Generate security audits
1652    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1652    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1652    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1652    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1652    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1652    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1652    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1652    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1652    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1652    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1652    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1652    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1652    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1652    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1652    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1764    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1764    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1764    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
1764    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1764    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1764    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
1764    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1764    svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
1764    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1764    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
1764    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1764    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
1764    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
1764    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
1764    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
1764    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
1764    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
1764    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1764    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1764    svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
1764    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1764    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1764    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1764    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1764    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1764    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1764    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1764    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1764    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
1764    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1764    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1764    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
1764    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
1764    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
1764    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
1776    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1776    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1776    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1776    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1776    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1776    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1776    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1776    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1776    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1776    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1776    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1776    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1776    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1776    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1776    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1776    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1776    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1776    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1776    svchost.exe     20      SeDebugPrivilege                Debug programs
1776    svchost.exe     21      SeAuditPrivilege                Generate security audits
1776    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1776    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1776    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1776    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1776    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1776    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1776    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1776    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1776    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1776    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1776    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1776    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1776    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1776    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1776    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1808    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1808    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1808    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1808    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1808    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1808    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1808    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1808    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1808    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1808    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1808    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1808    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1808    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1808    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1808    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1808    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1808    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1808    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1808    svchost.exe     20      SeDebugPrivilege                Debug programs
1808    svchost.exe     21      SeAuditPrivilege                Generate security audits
1808    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1808    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1808    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1808    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1808    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1808    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1808    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1808    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1808    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
1808    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1808    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1808    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1808    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1808    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1808    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1964    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1964    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1964    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1964    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1964    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1964    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1964    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1964    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1964    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1964    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1964    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1964    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1964    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1964    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1964    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1964    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1964    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1964    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1964    svchost.exe     20      SeDebugPrivilege                Debug programs
1964    svchost.exe     21      SeAuditPrivilege                Generate security audits
1964    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1964    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1964    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1964    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1964    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1964    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1964    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1964    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1964    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
1964    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1964    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1964    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1964    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1964    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1964    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1976    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1976    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1976    svchost.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
1976    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1976    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1976    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
1976    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1976    svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
1976    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1976    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
1976    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1976    svchost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
1976    svchost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
1976    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
1976    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
1976    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1976    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1976    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1976    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1976    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
1976    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1976    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1976    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1976    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1976    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1976    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1976    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1976    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
1976    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
1976    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1976    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1976    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
1976    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
1976    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
1976    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
1988    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1988    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1988    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
1988    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1988    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1988    svchost.exe     7       SeTcbPrivilege  Default Act as part of the operating system
1988    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1988    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1988    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1988    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
1988    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1988    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
1988    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
1988    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
1988    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
1988    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1988    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1988    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1988    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1988    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
1988    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1988    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1988    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1988    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1988    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1988    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1988    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1988    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1988    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
1988    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1988    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1988    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
1988    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
1988    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
1988    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
604     svchost.exe     2       SeCreateTokenPrivilege          Create a token object
604     svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
604     svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
604     svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
604     svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
604     svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
604     svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
604     svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
604     svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
604     svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
604     svchost.exe     12      SeSystemtimePrivilege           Change the system time
604     svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
604     svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
604     svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
604     svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
604     svchost.exe     17      SeBackupPrivilege               Backup files and directories
604     svchost.exe     18      SeRestorePrivilege              Restore files and directories
604     svchost.exe     19      SeShutdownPrivilege             Shut down the system
604     svchost.exe     20      SeDebugPrivilege                Debug programs
604     svchost.exe     21      SeAuditPrivilege        Present,Enabled Generate security audits
604     svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
604     svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
604     svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
604     svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
604     svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
604     svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
604     svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
604     svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
604     svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
604     svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
604     svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
604     svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
604     svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
604     svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
604     svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1924    MemCompression  2       SeCreateTokenPrivilege  Present Create a token object
1924    MemCompression  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1924    MemCompression  4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
1924    MemCompression  5       SeIncreaseQuotaPrivilege        Present Increase quotas
1924    MemCompression  6       SeMachineAccountPrivilege               Add workstations to the domain
1924    MemCompression  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
1924    MemCompression  8       SeSecurityPrivilege     Present Manage auditing and security log
1924    MemCompression  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
1924    MemCompression  10      SeLoadDriverPrivilege   Present Load and unload device drivers
1924    MemCompression  11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
1924    MemCompression  12      SeSystemtimePrivilege   Present Change the system time
1924    MemCompression  13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
1924    MemCompression  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
1924    MemCompression  15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
1924    MemCompression  16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
1924    MemCompression  17      SeBackupPrivilege       Present Backup files and directories
1924    MemCompression  18      SeRestorePrivilege      Present Restore files and directories
1924    MemCompression  19      SeShutdownPrivilege     Present Shut down the system
1924    MemCompression  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1924    MemCompression  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
1924    MemCompression  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
1924    MemCompression  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1924    MemCompression  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1924    MemCompression  25      SeUndockPrivilege       Present Remove computer from docking station
1924    MemCompression  26      SeSyncAgentPrivilege            Synch directory service data
1924    MemCompression  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1924    MemCompression  28      SeManageVolumePrivilege Present Manage the files on a volume
1924    MemCompression  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1924    MemCompression  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1924    MemCompression  31      SeTrustedCredManAccessPrivilege Present Access Credential Manager as a trusted caller
1924    MemCompression  32      SeRelabelPrivilege      Present Modify the mandatory integrity level of an object
1924    MemCompression  33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
1924    MemCompression  34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
1924    MemCompression  35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
1924    MemCompression  36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
2092    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2092    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2092    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
2092    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2092    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2092    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
2092    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2092    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2092    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2092    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
2092    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2092    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
2092    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
2092    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
2092    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
2092    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2092    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2092    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2092    svchost.exe     20      SeDebugPrivilege        Default Debug programs
2092    svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
2092    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2092    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2092    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2092    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2092    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2092    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2092    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2092    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2092    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
2092    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2092    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2092    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
2092    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
2092    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
2092    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
2160    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2160    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2160    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2160    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2160    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2160    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2160    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2160    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2160    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2160    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2160    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2160    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2160    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2160    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2160    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2160    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2160    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2160    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2160    svchost.exe     20      SeDebugPrivilege                Debug programs
2160    svchost.exe     21      SeAuditPrivilege        Present Generate security audits
2160    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2160    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2160    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2160    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2160    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2160    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2160    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2160    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2160    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
2160    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2160    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2160    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
2160    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2160    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2160    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2168    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2168    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2168    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2168    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2168    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2168    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2168    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2168    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2168    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2168    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2168    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2168    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2168    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2168    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2168    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2168    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2168    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2168    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2168    svchost.exe     20      SeDebugPrivilege                Debug programs
2168    svchost.exe     21      SeAuditPrivilege                Generate security audits
2168    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2168    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2168    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2168    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2168    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2168    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2168    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2168    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2168    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
2168    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2168    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2168    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
2168    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2168    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2168    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2180    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2180    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2180    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
2180    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2180    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2180    svchost.exe     7       SeTcbPrivilege  Default Act as part of the operating system
2180    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2180    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2180    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2180    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
2180    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2180    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
2180    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
2180    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
2180    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
2180    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2180    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2180    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2180    svchost.exe     20      SeDebugPrivilege        Default Debug programs
2180    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
2180    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2180    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2180    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2180    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2180    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2180    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2180    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2180    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
2180    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
2180    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2180    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2180    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
2180    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
2180    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
2180    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
2276    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2276    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2276    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2276    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2276    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2276    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2276    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2276    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2276    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2276    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2276    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2276    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2276    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2276    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2276    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2276    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2276    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2276    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2276    svchost.exe     20      SeDebugPrivilege                Debug programs
2276    svchost.exe     21      SeAuditPrivilege                Generate security audits
2276    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2276    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2276    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2276    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2276    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2276    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2276    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2276    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2276    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
2276    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2276    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2276    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
2276    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2276    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2276    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2376    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2376    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2376    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2376    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2376    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2376    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2376    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2376    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2376    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2376    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2376    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2376    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2376    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2376    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2376    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2376    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2376    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2376    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2376    svchost.exe     20      SeDebugPrivilege                Debug programs
2376    svchost.exe     21      SeAuditPrivilege                Generate security audits
2376    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2376    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2376    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2376    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2376    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2376    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2376    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2376    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
2376    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
2376    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2376    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2376    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
2376    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2376    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2376    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2384    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2384    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
2384    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2384    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
2384    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2384    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2384    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2384    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2384    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2384    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2384    svchost.exe     12      SeSystemtimePrivilege   Present Change the system time
2384    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2384    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2384    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2384    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2384    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2384    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2384    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
2384    svchost.exe     20      SeDebugPrivilege                Debug programs
2384    svchost.exe     21      SeAuditPrivilege        Present Generate security audits
2384    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2384    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2384    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2384    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
2384    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2384    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2384    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2384    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2384    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
2384    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2384    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2384    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
2384    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
2384    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2384    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2396    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2396    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2396    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2396    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2396    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2396    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2396    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2396    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2396    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2396    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2396    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2396    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2396    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2396    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2396    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2396    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2396    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2396    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2396    svchost.exe     20      SeDebugPrivilege                Debug programs
2396    svchost.exe     21      SeAuditPrivilege                Generate security audits
2396    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2396    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2396    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2396    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2396    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2396    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2396    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2396    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2396    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
2396    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2396    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2396    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
2396    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2396    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2396    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2472    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2472    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2472    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2472    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2472    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2472    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2472    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2472    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2472    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2472    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2472    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2472    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2472    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2472    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2472    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2472    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2472    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2472    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2472    svchost.exe     20      SeDebugPrivilege                Debug programs
2472    svchost.exe     21      SeAuditPrivilege                Generate security audits
2472    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2472    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2472    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2472    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2472    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2472    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2472    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2472    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2472    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
2472    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2472    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2472    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
2472    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2472    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2472    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2496    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2496    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2496    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2496    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2496    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2496    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2496    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2496    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2496    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2496    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2496    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2496    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2496    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2496    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2496    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2496    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2496    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2496    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2496    svchost.exe     20      SeDebugPrivilege                Debug programs
2496    svchost.exe     21      SeAuditPrivilege                Generate security audits
2496    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2496    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2496    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2496    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2496    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2496    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2496    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2496    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
2496    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
2496    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2496    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2496    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
2496    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2496    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2496    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2504    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2504    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2504    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
2504    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2504    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2504    svchost.exe     7       SeTcbPrivilege  Default Act as part of the operating system
2504    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2504    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2504    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2504    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
2504    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2504    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
2504    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
2504    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
2504    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
2504    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2504    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2504    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2504    svchost.exe     20      SeDebugPrivilege        Default Debug programs
2504    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
2504    svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
2504    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2504    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2504    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2504    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2504    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2504    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2504    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2504    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
2504    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2504    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2504    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
2504    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
2504    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
2504    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
2848    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2848    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2848    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2848    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2848    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2848    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2848    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2848    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2848    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2848    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2848    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2848    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2848    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2848    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2848    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2848    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2848    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2848    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2848    svchost.exe     20      SeDebugPrivilege                Debug programs
2848    svchost.exe     21      SeAuditPrivilege                Generate security audits
2848    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2848    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2848    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2848    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2848    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2848    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2848    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2848    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2848    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
2848    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2848    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2848    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
2848    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2848    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2848    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2904    spoolsv.exe     2       SeCreateTokenPrivilege          Create a token object
2904    spoolsv.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
2904    spoolsv.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
2904    spoolsv.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2904    spoolsv.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2904    spoolsv.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
2904    spoolsv.exe     8       SeSecurityPrivilege             Manage auditing and security log
2904    spoolsv.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2904    spoolsv.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2904    spoolsv.exe     11      SeSystemProfilePrivilege        Default Profile system performance
2904    spoolsv.exe     12      SeSystemtimePrivilege           Change the system time
2904    spoolsv.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
2904    spoolsv.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
2904    spoolsv.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
2904    spoolsv.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
2904    spoolsv.exe     17      SeBackupPrivilege               Backup files and directories
2904    spoolsv.exe     18      SeRestorePrivilege              Restore files and directories
2904    spoolsv.exe     19      SeShutdownPrivilege             Shut down the system
2904    spoolsv.exe     20      SeDebugPrivilege        Default Debug programs
2904    spoolsv.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
2904    spoolsv.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2904    spoolsv.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2904    spoolsv.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2904    spoolsv.exe     25      SeUndockPrivilege               Remove computer from docking station
2904    spoolsv.exe     26      SeSyncAgentPrivilege            Synch directory service data
2904    spoolsv.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2904    spoolsv.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2904    spoolsv.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2904    spoolsv.exe     30      SeCreateGlobalPrivilege Default Create global objects
2904    spoolsv.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2904    spoolsv.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2904    spoolsv.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
2904    spoolsv.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
2904    spoolsv.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
2904    spoolsv.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3028    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3028    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
3028    svchost.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
3028    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
3028    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3028    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3028    svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
3028    svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
3028    svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
3028    svchost.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
3028    svchost.exe     12      SeSystemtimePrivilege   Present Change the system time
3028    svchost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
3028    svchost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
3028    svchost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
3028    svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
3028    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
3028    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
3028    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
3028    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
3028    svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
3028    svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
3028    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3028    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3028    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
3028    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3028    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3028    svchost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
3028    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3028    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3028    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3028    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3028    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
3028    svchost.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
3028    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
3028    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
2220    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2220    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2220    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2220    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2220    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2220    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2220    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2220    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2220    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2220    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2220    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2220    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2220    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2220    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2220    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2220    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2220    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2220    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2220    svchost.exe     20      SeDebugPrivilege                Debug programs
2220    svchost.exe     21      SeAuditPrivilege                Generate security audits
2220    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2220    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2220    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2220    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2220    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2220    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2220    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2220    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
2220    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
2220    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2220    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2220    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
2220    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2220    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2220    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2216    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2216    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
2216    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2216    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
2216    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2216    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2216    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2216    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2216    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2216    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2216    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2216    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2216    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2216    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2216    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2216    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2216    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2216    svchost.exe     19      SeShutdownPrivilege             Shut down the system
2216    svchost.exe     20      SeDebugPrivilege                Debug programs
2216    svchost.exe     21      SeAuditPrivilege        Present,Enabled Generate security audits
2216    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2216    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2216    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2216    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
2216    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2216    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2216    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2216    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2216    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
2216    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2216    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2216    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
2216    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2216    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2216    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
740     svchost.exe     2       SeCreateTokenPrivilege          Create a token object
740     svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
740     svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
740     svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
740     svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
740     svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
740     svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
740     svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
740     svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
740     svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
740     svchost.exe     12      SeSystemtimePrivilege           Change the system time
740     svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
740     svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
740     svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
740     svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
740     svchost.exe     17      SeBackupPrivilege               Backup files and directories
740     svchost.exe     18      SeRestorePrivilege              Restore files and directories
740     svchost.exe     19      SeShutdownPrivilege             Shut down the system
740     svchost.exe     20      SeDebugPrivilege                Debug programs
740     svchost.exe     21      SeAuditPrivilege        Present,Enabled Generate security audits
740     svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
740     svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
740     svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
740     svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
740     svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
740     svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
740     svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
740     svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
740     svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
740     svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
740     svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
740     svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
740     svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
740     svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
740     svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3308    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3308    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3308    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
3308    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3308    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3308    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
3308    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3308    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3308    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3308    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
3308    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3308    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
3308    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3308    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
3308    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
3308    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3308    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3308    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3308    svchost.exe     20      SeDebugPrivilege                Debug programs
3308    svchost.exe     21      SeAuditPrivilege                Generate security audits
3308    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3308    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3308    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3308    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3308    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3308    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3308    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3308    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3308    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3308    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3308    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3308    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
3308    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
3308    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3308    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3324    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3324    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
3324    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3324    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3324    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3324    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3324    svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
3324    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3324    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3324    svchost.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
3324    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3324    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3324    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
3324    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3324    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
3324    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3324    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3324    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3324    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
3324    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
3324    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3324    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3324    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3324    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3324    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3324    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3324    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3324    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3324    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3324    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3324    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3324    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3324    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3324    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3324    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3332    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3332    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
3332    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3332    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3332    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3332    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3332    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3332    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3332    svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
3332    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
3332    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3332    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3332    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
3332    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3332    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
3332    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3332    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
3332    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3332    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
3332    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
3332    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3332    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3332    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3332    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3332    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3332    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3332    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3332    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3332    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
3332    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3332    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3332    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3332    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3332    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3332    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3352    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3352    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
3352    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
3352    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3352    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3352    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
3352    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3352    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3352    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3352    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
3352    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3352    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
3352    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3352    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
3352    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
3352    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3352    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3352    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3352    svchost.exe     20      SeDebugPrivilege                Debug programs
3352    svchost.exe     21      SeAuditPrivilege                Generate security audits
3352    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3352    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3352    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3352    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3352    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3352    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3352    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3352    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3352    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3352    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3352    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3352    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
3352    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
3352    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3352    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3376    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3376    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3376    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3376    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3376    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3376    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3376    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3376    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3376    svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
3376    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
3376    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3376    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3376    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
3376    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3376    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
3376    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3376    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3376    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3376    svchost.exe     20      SeDebugPrivilege        Default Debug programs
3376    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
3376    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3376    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3376    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3376    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3376    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3376    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3376    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3376    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3376    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3376    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3376    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3376    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3376    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3376    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3376    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3480    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3480    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3480    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
3480    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3480    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3480    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
3480    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3480    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3480    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3480    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
3480    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3480    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
3480    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3480    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
3480    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
3480    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3480    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3480    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3480    svchost.exe     20      SeDebugPrivilege                Debug programs
3480    svchost.exe     21      SeAuditPrivilege                Generate security audits
3480    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3480    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3480    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3480    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3480    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3480    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3480    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3480    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3480    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
3480    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3480    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3480    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
3480    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
3480    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3480    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3488    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3488    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3488    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3488    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3488    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3488    svchost.exe     7       SeTcbPrivilege  Default Act as part of the operating system
3488    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3488    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3488    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3488    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
3488    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3488    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3488    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
3488    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3488    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
3488    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3488    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3488    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3488    svchost.exe     20      SeDebugPrivilege        Default Debug programs
3488    svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
3488    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3488    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3488    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3488    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3488    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3488    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3488    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3488    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3488    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
3488    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3488    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3488    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3488    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3488    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3488    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3528    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3528    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3528    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3528    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3528    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3528    svchost.exe     7       SeTcbPrivilege  Default Act as part of the operating system
3528    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3528    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3528    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3528    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
3528    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3528    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3528    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
3528    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3528    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
3528    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3528    svchost.exe     18      SeRestorePrivilege      Present,Enabled Restore files and directories
3528    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3528    svchost.exe     20      SeDebugPrivilege        Default Debug programs
3528    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
3528    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3528    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3528    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3528    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3528    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3528    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3528    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3528    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3528    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
3528    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3528    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3528    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3528    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3528    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3528    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3540    MsMpEng.exe     2       SeCreateTokenPrivilege          Create a token object
3540    MsMpEng.exe     3       SeAssignPrimaryTokenPrivilege   Present,Enabled Replace a process-level token
3540    MsMpEng.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3540    MsMpEng.exe     5       SeIncreaseQuotaPrivilege        Present,Enabled Increase quotas
3540    MsMpEng.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3540    MsMpEng.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3540    MsMpEng.exe     8       SeSecurityPrivilege     Present,Enabled Manage auditing and security log
3540    MsMpEng.exe     9       SeTakeOwnershipPrivilege        Present,Enabled Take ownership of files/objects
3540    MsMpEng.exe     10      SeLoadDriverPrivilege   Present,Enabled Load and unload device drivers
3540    MsMpEng.exe     11      SeSystemProfilePrivilege        Default Profile system performance
3540    MsMpEng.exe     12      SeSystemtimePrivilege           Change the system time
3540    MsMpEng.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3540    MsMpEng.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
3540    MsMpEng.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3540    MsMpEng.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
3540    MsMpEng.exe     17      SeBackupPrivilege       Present,Enabled Backup files and directories
3540    MsMpEng.exe     18      SeRestorePrivilege      Present,Enabled Restore files and directories
3540    MsMpEng.exe     19      SeShutdownPrivilege     Present,Enabled Shut down the system
3540    MsMpEng.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
3540    MsMpEng.exe     21      SeAuditPrivilege        Default Generate security audits
3540    MsMpEng.exe     22      SeSystemEnvironmentPrivilege    Present,Enabled Edit firmware environment values
3540    MsMpEng.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3540    MsMpEng.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3540    MsMpEng.exe     25      SeUndockPrivilege               Remove computer from docking station
3540    MsMpEng.exe     26      SeSyncAgentPrivilege            Synch directory service data
3540    MsMpEng.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3540    MsMpEng.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3540    MsMpEng.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3540    MsMpEng.exe     30      SeCreateGlobalPrivilege Default Create global objects
3540    MsMpEng.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3540    MsMpEng.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3540    MsMpEng.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3540    MsMpEng.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3540    MsMpEng.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3540    MsMpEng.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3560    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3560    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3560    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3560    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3560    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3560    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3560    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3560    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3560    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3560    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
3560    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3560    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3560    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
3560    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3560    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
3560    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3560    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3560    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3560    svchost.exe     20      SeDebugPrivilege        Default Debug programs
3560    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
3560    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3560    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3560    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3560    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3560    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3560    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3560    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3560    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3560    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3560    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3560    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3560    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3560    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3560    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3560    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3636    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3636    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3636    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
3636    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3636    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3636    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
3636    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3636    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3636    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3636    svchost.exe     11      SeSystemProfilePrivilege        Present Profile system performance
3636    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3636    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
3636    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3636    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
3636    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
3636    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3636    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3636    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3636    svchost.exe     20      SeDebugPrivilege                Debug programs
3636    svchost.exe     21      SeAuditPrivilege                Generate security audits
3636    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3636    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3636    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3636    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3636    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3636    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3636    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3636    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3636    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
3636    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3636    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3636    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
3636    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
3636    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3636    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3752    dasHost.exe     2       SeCreateTokenPrivilege          Create a token object
3752    dasHost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
3752    dasHost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
3752    dasHost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
3752    dasHost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3752    dasHost.exe     7       SeTcbPrivilege          Act as part of the operating system
3752    dasHost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3752    dasHost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3752    dasHost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3752    dasHost.exe     11      SeSystemProfilePrivilege                Profile system performance
3752    dasHost.exe     12      SeSystemtimePrivilege   Present Change the system time
3752    dasHost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
3752    dasHost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3752    dasHost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
3752    dasHost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
3752    dasHost.exe     17      SeBackupPrivilege               Backup files and directories
3752    dasHost.exe     18      SeRestorePrivilege              Restore files and directories
3752    dasHost.exe     19      SeShutdownPrivilege     Present Shut down the system
3752    dasHost.exe     20      SeDebugPrivilege                Debug programs
3752    dasHost.exe     21      SeAuditPrivilege        Present Generate security audits
3752    dasHost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3752    dasHost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3752    dasHost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3752    dasHost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
3752    dasHost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3752    dasHost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3752    dasHost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3752    dasHost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3752    dasHost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3752    dasHost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3752    dasHost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3752    dasHost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
3752    dasHost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
3752    dasHost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3752    dasHost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3844    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3844    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
3844    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3844    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
3844    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3844    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3844    svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
3844    svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
3844    svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
3844    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
3844    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3844    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3844    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
3844    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3844    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
3844    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
3844    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
3844    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3844    svchost.exe     20      SeDebugPrivilege        Default Debug programs
3844    svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
3844    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3844    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3844    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3844    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3844    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3844    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3844    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3844    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3844    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3844    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3844    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3844    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3844    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3844    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3844    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3880    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3880    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
3880    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3880    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3880    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3880    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3880    svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
3880    svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
3880    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3880    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
3880    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3880    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3880    svchost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
3880    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3880    svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
3880    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
3880    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
3880    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3880    svchost.exe     20      SeDebugPrivilege        Default Debug programs
3880    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
3880    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3880    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3880    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3880    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3880    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3880    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3880    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3880    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3880    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3880    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3880    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3880    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3880    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3880    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3880    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3172    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3172    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3172    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
3172    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3172    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3172    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
3172    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3172    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3172    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3172    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
3172    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3172    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
3172    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3172    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
3172    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
3172    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3172    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3172    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3172    svchost.exe     20      SeDebugPrivilege                Debug programs
3172    svchost.exe     21      SeAuditPrivilege                Generate security audits
3172    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3172    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3172    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3172    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3172    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3172    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3172    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3172    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
3172    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3172    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3172    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3172    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
3172    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
3172    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3172    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3648    CSFalconServic  2       SeCreateTokenPrivilege          Create a token object
3648    CSFalconServic  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
3648    CSFalconServic  4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
3648    CSFalconServic  5       SeIncreaseQuotaPrivilege        Present Increase quotas
3648    CSFalconServic  6       SeMachineAccountPrivilege               Add workstations to the domain
3648    CSFalconServic  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3648    CSFalconServic  8       SeSecurityPrivilege     Present Manage auditing and security log
3648    CSFalconServic  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
3648    CSFalconServic  10      SeLoadDriverPrivilege   Present Load and unload device drivers
3648    CSFalconServic  11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
3648    CSFalconServic  12      SeSystemtimePrivilege   Present Change the system time
3648    CSFalconServic  13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
3648    CSFalconServic  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
3648    CSFalconServic  15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
3648    CSFalconServic  16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
3648    CSFalconServic  17      SeBackupPrivilege       Present Backup files and directories
3648    CSFalconServic  18      SeRestorePrivilege      Present Restore files and directories
3648    CSFalconServic  19      SeShutdownPrivilege     Present Shut down the system
3648    CSFalconServic  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
3648    CSFalconServic  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
3648    CSFalconServic  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
3648    CSFalconServic  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3648    CSFalconServic  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3648    CSFalconServic  25      SeUndockPrivilege       Present Remove computer from docking station
3648    CSFalconServic  26      SeSyncAgentPrivilege            Synch directory service data
3648    CSFalconServic  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3648    CSFalconServic  28      SeManageVolumePrivilege Present Manage the files on a volume
3648    CSFalconServic  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3648    CSFalconServic  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3648    CSFalconServic  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3648    CSFalconServic  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3648    CSFalconServic  33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
3648    CSFalconServic  34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
3648    CSFalconServic  35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
3648    CSFalconServic  36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
3696    Sysmon64.exe    2       SeCreateTokenPrivilege          Create a token object
3696    Sysmon64.exe    3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
3696    Sysmon64.exe    4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
3696    Sysmon64.exe    5       SeIncreaseQuotaPrivilege        Present Increase quotas
3696    Sysmon64.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
3696    Sysmon64.exe    7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3696    Sysmon64.exe    8       SeSecurityPrivilege     Present,Enabled Manage auditing and security log
3696    Sysmon64.exe    9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
3696    Sysmon64.exe    10      SeLoadDriverPrivilege   Present Load and unload device drivers
3696    Sysmon64.exe    11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
3696    Sysmon64.exe    12      SeSystemtimePrivilege   Present Change the system time
3696    Sysmon64.exe    13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
3696    Sysmon64.exe    14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
3696    Sysmon64.exe    15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
3696    Sysmon64.exe    16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
3696    Sysmon64.exe    17      SeBackupPrivilege       Present Backup files and directories
3696    Sysmon64.exe    18      SeRestorePrivilege      Present Restore files and directories
3696    Sysmon64.exe    19      SeShutdownPrivilege     Present Shut down the system
3696    Sysmon64.exe    20      SeDebugPrivilege        Present,Enabled,Default Debug programs
3696    Sysmon64.exe    21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
3696    Sysmon64.exe    22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
3696    Sysmon64.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3696    Sysmon64.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3696    Sysmon64.exe    25      SeUndockPrivilege       Present Remove computer from docking station
3696    Sysmon64.exe    26      SeSyncAgentPrivilege            Synch directory service data
3696    Sysmon64.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3696    Sysmon64.exe    28      SeManageVolumePrivilege Present Manage the files on a volume
3696    Sysmon64.exe    29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3696    Sysmon64.exe    30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3696    Sysmon64.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3696    Sysmon64.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3696    Sysmon64.exe    33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
3696    Sysmon64.exe    34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
3696    Sysmon64.exe    35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
3696    Sysmon64.exe    36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
4352    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
4352    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
4352    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
4352    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
4352    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
4352    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
4352    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
4352    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4352    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
4352    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
4352    svchost.exe     12      SeSystemtimePrivilege   Present Change the system time
4352    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
4352    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4352    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
4352    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
4352    svchost.exe     17      SeBackupPrivilege               Backup files and directories
4352    svchost.exe     18      SeRestorePrivilege              Restore files and directories
4352    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
4352    svchost.exe     20      SeDebugPrivilege                Debug programs
4352    svchost.exe     21      SeAuditPrivilege        Present Generate security audits
4352    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4352    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4352    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4352    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
4352    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
4352    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4352    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
4352    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
4352    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
4352    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4352    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4352    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
4352    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
4352    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4352    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
4800    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
4800    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
4800    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
4800    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
4800    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
4800    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
4800    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
4800    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4800    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
4800    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
4800    svchost.exe     12      SeSystemtimePrivilege           Change the system time
4800    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
4800    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4800    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
4800    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
4800    svchost.exe     17      SeBackupPrivilege               Backup files and directories
4800    svchost.exe     18      SeRestorePrivilege              Restore files and directories
4800    svchost.exe     19      SeShutdownPrivilege             Shut down the system
4800    svchost.exe     20      SeDebugPrivilege                Debug programs
4800    svchost.exe     21      SeAuditPrivilege        Present,Enabled Generate security audits
4800    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4800    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4800    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4800    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
4800    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
4800    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4800    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
4800    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
4800    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
4800    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4800    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4800    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
4800    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
4800    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4800    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
5220    unsecapp.exe    2       SeCreateTokenPrivilege          Create a token object
5220    unsecapp.exe    3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
5220    unsecapp.exe    4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
5220    unsecapp.exe    5       SeIncreaseQuotaPrivilege        Present Increase quotas
5220    unsecapp.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
5220    unsecapp.exe    7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
5220    unsecapp.exe    8       SeSecurityPrivilege     Present Manage auditing and security log
5220    unsecapp.exe    9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
5220    unsecapp.exe    10      SeLoadDriverPrivilege   Present Load and unload device drivers
5220    unsecapp.exe    11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
5220    unsecapp.exe    12      SeSystemtimePrivilege   Present Change the system time
5220    unsecapp.exe    13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
5220    unsecapp.exe    14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
5220    unsecapp.exe    15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
5220    unsecapp.exe    16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
5220    unsecapp.exe    17      SeBackupPrivilege       Present Backup files and directories
5220    unsecapp.exe    18      SeRestorePrivilege      Present Restore files and directories
5220    unsecapp.exe    19      SeShutdownPrivilege     Present Shut down the system
5220    unsecapp.exe    20      SeDebugPrivilege        Present,Enabled,Default Debug programs
5220    unsecapp.exe    21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
5220    unsecapp.exe    22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
5220    unsecapp.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
5220    unsecapp.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5220    unsecapp.exe    25      SeUndockPrivilege       Present Remove computer from docking station
5220    unsecapp.exe    26      SeSyncAgentPrivilege            Synch directory service data
5220    unsecapp.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5220    unsecapp.exe    28      SeManageVolumePrivilege Present Manage the files on a volume
5220    unsecapp.exe    29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
5220    unsecapp.exe    30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
5220    unsecapp.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5220    unsecapp.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5220    unsecapp.exe    33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
5220    unsecapp.exe    34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
5220    unsecapp.exe    35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
5220    unsecapp.exe    36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
5980    CSFalconContai  2       SeCreateTokenPrivilege          Create a token object
5980    CSFalconContai  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
5980    CSFalconContai  4       SeLockMemoryPrivilege           Lock pages in memory
5980    CSFalconContai  5       SeIncreaseQuotaPrivilege                Increase quotas
5980    CSFalconContai  6       SeMachineAccountPrivilege               Add workstations to the domain
5980    CSFalconContai  7       SeTcbPrivilege          Act as part of the operating system
5980    CSFalconContai  8       SeSecurityPrivilege             Manage auditing and security log
5980    CSFalconContai  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
5980    CSFalconContai  10      SeLoadDriverPrivilege           Load and unload device drivers
5980    CSFalconContai  11      SeSystemProfilePrivilege                Profile system performance
5980    CSFalconContai  12      SeSystemtimePrivilege           Change the system time
5980    CSFalconContai  13      SeProfileSingleProcessPrivilege         Profile a single process
5980    CSFalconContai  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
5980    CSFalconContai  15      SeCreatePagefilePrivilege               Create a pagefile
5980    CSFalconContai  16      SeCreatePermanentPrivilege              Create permanent shared objects
5980    CSFalconContai  17      SeBackupPrivilege               Backup files and directories
5980    CSFalconContai  18      SeRestorePrivilege              Restore files and directories
5980    CSFalconContai  19      SeShutdownPrivilege             Shut down the system
5980    CSFalconContai  20      SeDebugPrivilege                Debug programs
5980    CSFalconContai  21      SeAuditPrivilege                Generate security audits
5980    CSFalconContai  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5980    CSFalconContai  23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
5980    CSFalconContai  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5980    CSFalconContai  25      SeUndockPrivilege               Remove computer from docking station
5980    CSFalconContai  26      SeSyncAgentPrivilege            Synch directory service data
5980    CSFalconContai  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5980    CSFalconContai  28      SeManageVolumePrivilege         Manage the files on a volume
5980    CSFalconContai  29      SeImpersonatePrivilege          Impersonate a client after authentication
5980    CSFalconContai  30      SeCreateGlobalPrivilege         Create global objects
5980    CSFalconContai  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5980    CSFalconContai  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5980    CSFalconContai  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
5980    CSFalconContai  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
5980    CSFalconContai  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
5980    CSFalconContai  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
6020    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
6020    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
6020    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
6020    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
6020    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
6020    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
6020    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
6020    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
6020    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
6020    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
6020    svchost.exe     12      SeSystemtimePrivilege           Change the system time
6020    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
6020    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
6020    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
6020    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
6020    svchost.exe     17      SeBackupPrivilege               Backup files and directories
6020    svchost.exe     18      SeRestorePrivilege              Restore files and directories
6020    svchost.exe     19      SeShutdownPrivilege             Shut down the system
6020    svchost.exe     20      SeDebugPrivilege                Debug programs
6020    svchost.exe     21      SeAuditPrivilege                Generate security audits
6020    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
6020    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
6020    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
6020    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
6020    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
6020    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
6020    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
6020    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
6020    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
6020    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
6020    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
6020    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
6020    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
6020    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
6020    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1852    SearchIndexer.  2       SeCreateTokenPrivilege          Create a token object
1852    SearchIndexer.  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1852    SearchIndexer.  4       SeLockMemoryPrivilege   Default Lock pages in memory
1852    SearchIndexer.  5       SeIncreaseQuotaPrivilege        Present Increase quotas
1852    SearchIndexer.  6       SeMachineAccountPrivilege               Add workstations to the domain
1852    SearchIndexer.  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
1852    SearchIndexer.  8       SeSecurityPrivilege             Manage auditing and security log
1852    SearchIndexer.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1852    SearchIndexer.  10      SeLoadDriverPrivilege           Load and unload device drivers
1852    SearchIndexer.  11      SeSystemProfilePrivilege        Default Profile system performance
1852    SearchIndexer.  12      SeSystemtimePrivilege           Change the system time
1852    SearchIndexer.  13      SeProfileSingleProcessPrivilege Default Profile a single process
1852    SearchIndexer.  14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
1852    SearchIndexer.  15      SeCreatePagefilePrivilege       Default Create a pagefile
1852    SearchIndexer.  16      SeCreatePermanentPrivilege      Default Create permanent shared objects
1852    SearchIndexer.  17      SeBackupPrivilege               Backup files and directories
1852    SearchIndexer.  18      SeRestorePrivilege              Restore files and directories
1852    SearchIndexer.  19      SeShutdownPrivilege             Shut down the system
1852    SearchIndexer.  20      SeDebugPrivilege        Default Debug programs
1852    SearchIndexer.  21      SeAuditPrivilege        Default Generate security audits
1852    SearchIndexer.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1852    SearchIndexer.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1852    SearchIndexer.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1852    SearchIndexer.  25      SeUndockPrivilege               Remove computer from docking station
1852    SearchIndexer.  26      SeSyncAgentPrivilege            Synch directory service data
1852    SearchIndexer.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1852    SearchIndexer.  28      SeManageVolumePrivilege Present Manage the files on a volume
1852    SearchIndexer.  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1852    SearchIndexer.  30      SeCreateGlobalPrivilege Default Create global objects
1852    SearchIndexer.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1852    SearchIndexer.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1852    SearchIndexer.  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
1852    SearchIndexer.  34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
1852    SearchIndexer.  35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
1852    SearchIndexer.  36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
5772    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
5772    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
5772    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
5772    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
5772    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
5772    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
5772    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
5772    svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
5772    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
5772    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
5772    svchost.exe     12      SeSystemtimePrivilege           Change the system time
5772    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
5772    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
5772    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
5772    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
5772    svchost.exe     17      SeBackupPrivilege               Backup files and directories
5772    svchost.exe     18      SeRestorePrivilege              Restore files and directories
5772    svchost.exe     19      SeShutdownPrivilege             Shut down the system
5772    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
5772    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
5772    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5772    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
5772    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5772    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
5772    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
5772    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5772    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
5772    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
5772    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
5772    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5772    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5772    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
5772    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
5772    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
5772    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
5804    AggregatorHost  2       SeCreateTokenPrivilege          Create a token object
5804    AggregatorHost  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
5804    AggregatorHost  4       SeLockMemoryPrivilege   Default Lock pages in memory
5804    AggregatorHost  5       SeIncreaseQuotaPrivilege                Increase quotas
5804    AggregatorHost  6       SeMachineAccountPrivilege               Add workstations to the domain
5804    AggregatorHost  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
5804    AggregatorHost  8       SeSecurityPrivilege     Present Manage auditing and security log
5804    AggregatorHost  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
5804    AggregatorHost  10      SeLoadDriverPrivilege           Load and unload device drivers
5804    AggregatorHost  11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
5804    AggregatorHost  12      SeSystemtimePrivilege           Change the system time
5804    AggregatorHost  13      SeProfileSingleProcessPrivilege Default Profile a single process
5804    AggregatorHost  14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
5804    AggregatorHost  15      SeCreatePagefilePrivilege       Default Create a pagefile
5804    AggregatorHost  16      SeCreatePermanentPrivilege      Default Create permanent shared objects
5804    AggregatorHost  17      SeBackupPrivilege               Backup files and directories
5804    AggregatorHost  18      SeRestorePrivilege              Restore files and directories
5804    AggregatorHost  19      SeShutdownPrivilege             Shut down the system
5804    AggregatorHost  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
5804    AggregatorHost  21      SeAuditPrivilege        Default Generate security audits
5804    AggregatorHost  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5804    AggregatorHost  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
5804    AggregatorHost  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5804    AggregatorHost  25      SeUndockPrivilege               Remove computer from docking station
5804    AggregatorHost  26      SeSyncAgentPrivilege            Synch directory service data
5804    AggregatorHost  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5804    AggregatorHost  28      SeManageVolumePrivilege         Manage the files on a volume
5804    AggregatorHost  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
5804    AggregatorHost  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
5804    AggregatorHost  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5804    AggregatorHost  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5804    AggregatorHost  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
5804    AggregatorHost  34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
5804    AggregatorHost  35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
5804    AggregatorHost  36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
324     CSFalconContai  2       SeCreateTokenPrivilege          Create a token object
324     CSFalconContai  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
324     CSFalconContai  4       SeLockMemoryPrivilege           Lock pages in memory
324     CSFalconContai  5       SeIncreaseQuotaPrivilege                Increase quotas
324     CSFalconContai  6       SeMachineAccountPrivilege               Add workstations to the domain
324     CSFalconContai  7       SeTcbPrivilege          Act as part of the operating system
324     CSFalconContai  8       SeSecurityPrivilege             Manage auditing and security log
324     CSFalconContai  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
324     CSFalconContai  10      SeLoadDriverPrivilege           Load and unload device drivers
324     CSFalconContai  11      SeSystemProfilePrivilege                Profile system performance
324     CSFalconContai  12      SeSystemtimePrivilege           Change the system time
324     CSFalconContai  13      SeProfileSingleProcessPrivilege         Profile a single process
324     CSFalconContai  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
324     CSFalconContai  15      SeCreatePagefilePrivilege               Create a pagefile
324     CSFalconContai  16      SeCreatePermanentPrivilege              Create permanent shared objects
324     CSFalconContai  17      SeBackupPrivilege               Backup files and directories
324     CSFalconContai  18      SeRestorePrivilege              Restore files and directories
324     CSFalconContai  19      SeShutdownPrivilege             Shut down the system
324     CSFalconContai  20      SeDebugPrivilege                Debug programs
324     CSFalconContai  21      SeAuditPrivilege                Generate security audits
324     CSFalconContai  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
324     CSFalconContai  23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
324     CSFalconContai  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
324     CSFalconContai  25      SeUndockPrivilege               Remove computer from docking station
324     CSFalconContai  26      SeSyncAgentPrivilege            Synch directory service data
324     CSFalconContai  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
324     CSFalconContai  28      SeManageVolumePrivilege         Manage the files on a volume
324     CSFalconContai  29      SeImpersonatePrivilege          Impersonate a client after authentication
324     CSFalconContai  30      SeCreateGlobalPrivilege         Create global objects
324     CSFalconContai  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
324     CSFalconContai  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
324     CSFalconContai  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
324     CSFalconContai  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
324     CSFalconContai  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
324     CSFalconContai  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1192    CSFalconContai  2       SeCreateTokenPrivilege          Create a token object
1192    CSFalconContai  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1192    CSFalconContai  4       SeLockMemoryPrivilege           Lock pages in memory
1192    CSFalconContai  5       SeIncreaseQuotaPrivilege                Increase quotas
1192    CSFalconContai  6       SeMachineAccountPrivilege               Add workstations to the domain
1192    CSFalconContai  7       SeTcbPrivilege          Act as part of the operating system
1192    CSFalconContai  8       SeSecurityPrivilege             Manage auditing and security log
1192    CSFalconContai  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1192    CSFalconContai  10      SeLoadDriverPrivilege           Load and unload device drivers
1192    CSFalconContai  11      SeSystemProfilePrivilege                Profile system performance
1192    CSFalconContai  12      SeSystemtimePrivilege           Change the system time
1192    CSFalconContai  13      SeProfileSingleProcessPrivilege         Profile a single process
1192    CSFalconContai  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1192    CSFalconContai  15      SeCreatePagefilePrivilege               Create a pagefile
1192    CSFalconContai  16      SeCreatePermanentPrivilege              Create permanent shared objects
1192    CSFalconContai  17      SeBackupPrivilege               Backup files and directories
1192    CSFalconContai  18      SeRestorePrivilege              Restore files and directories
1192    CSFalconContai  19      SeShutdownPrivilege             Shut down the system
1192    CSFalconContai  20      SeDebugPrivilege                Debug programs
1192    CSFalconContai  21      SeAuditPrivilege                Generate security audits
1192    CSFalconContai  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1192    CSFalconContai  23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
1192    CSFalconContai  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1192    CSFalconContai  25      SeUndockPrivilege               Remove computer from docking station
1192    CSFalconContai  26      SeSyncAgentPrivilege            Synch directory service data
1192    CSFalconContai  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1192    CSFalconContai  28      SeManageVolumePrivilege         Manage the files on a volume
1192    CSFalconContai  29      SeImpersonatePrivilege          Impersonate a client after authentication
1192    CSFalconContai  30      SeCreateGlobalPrivilege         Create global objects
1192    CSFalconContai  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1192    CSFalconContai  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1192    CSFalconContai  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
1192    CSFalconContai  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1192    CSFalconContai  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1192    CSFalconContai  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
6604    CSFalconContai  2       SeCreateTokenPrivilege          Create a token object
6604    CSFalconContai  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
6604    CSFalconContai  4       SeLockMemoryPrivilege           Lock pages in memory
6604    CSFalconContai  5       SeIncreaseQuotaPrivilege                Increase quotas
6604    CSFalconContai  6       SeMachineAccountPrivilege               Add workstations to the domain
6604    CSFalconContai  7       SeTcbPrivilege          Act as part of the operating system
6604    CSFalconContai  8       SeSecurityPrivilege             Manage auditing and security log
6604    CSFalconContai  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
6604    CSFalconContai  10      SeLoadDriverPrivilege           Load and unload device drivers
6604    CSFalconContai  11      SeSystemProfilePrivilege                Profile system performance
6604    CSFalconContai  12      SeSystemtimePrivilege           Change the system time
6604    CSFalconContai  13      SeProfileSingleProcessPrivilege         Profile a single process
6604    CSFalconContai  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
6604    CSFalconContai  15      SeCreatePagefilePrivilege               Create a pagefile
6604    CSFalconContai  16      SeCreatePermanentPrivilege              Create permanent shared objects
6604    CSFalconContai  17      SeBackupPrivilege               Backup files and directories
6604    CSFalconContai  18      SeRestorePrivilege              Restore files and directories
6604    CSFalconContai  19      SeShutdownPrivilege             Shut down the system
6604    CSFalconContai  20      SeDebugPrivilege                Debug programs
6604    CSFalconContai  21      SeAuditPrivilege                Generate security audits
6604    CSFalconContai  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
6604    CSFalconContai  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
6604    CSFalconContai  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
6604    CSFalconContai  25      SeUndockPrivilege               Remove computer from docking station
6604    CSFalconContai  26      SeSyncAgentPrivilege            Synch directory service data
6604    CSFalconContai  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
6604    CSFalconContai  28      SeManageVolumePrivilege         Manage the files on a volume
6604    CSFalconContai  29      SeImpersonatePrivilege          Impersonate a client after authentication
6604    CSFalconContai  30      SeCreateGlobalPrivilege         Create global objects
6604    CSFalconContai  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
6604    CSFalconContai  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
6604    CSFalconContai  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
6604    CSFalconContai  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
6604    CSFalconContai  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
6604    CSFalconContai  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
6624    CSFalconContai  2       SeCreateTokenPrivilege          Create a token object
6624    CSFalconContai  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
6624    CSFalconContai  4       SeLockMemoryPrivilege           Lock pages in memory
6624    CSFalconContai  5       SeIncreaseQuotaPrivilege                Increase quotas
6624    CSFalconContai  6       SeMachineAccountPrivilege               Add workstations to the domain
6624    CSFalconContai  7       SeTcbPrivilege          Act as part of the operating system
6624    CSFalconContai  8       SeSecurityPrivilege             Manage auditing and security log
6624    CSFalconContai  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
6624    CSFalconContai  10      SeLoadDriverPrivilege           Load and unload device drivers
6624    CSFalconContai  11      SeSystemProfilePrivilege                Profile system performance
6624    CSFalconContai  12      SeSystemtimePrivilege           Change the system time
6624    CSFalconContai  13      SeProfileSingleProcessPrivilege         Profile a single process
6624    CSFalconContai  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
6624    CSFalconContai  15      SeCreatePagefilePrivilege               Create a pagefile
6624    CSFalconContai  16      SeCreatePermanentPrivilege              Create permanent shared objects
6624    CSFalconContai  17      SeBackupPrivilege               Backup files and directories
6624    CSFalconContai  18      SeRestorePrivilege              Restore files and directories
6624    CSFalconContai  19      SeShutdownPrivilege             Shut down the system
6624    CSFalconContai  20      SeDebugPrivilege                Debug programs
6624    CSFalconContai  21      SeAuditPrivilege                Generate security audits
6624    CSFalconContai  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
6624    CSFalconContai  23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
6624    CSFalconContai  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
6624    CSFalconContai  25      SeUndockPrivilege               Remove computer from docking station
6624    CSFalconContai  26      SeSyncAgentPrivilege            Synch directory service data
6624    CSFalconContai  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
6624    CSFalconContai  28      SeManageVolumePrivilege         Manage the files on a volume
6624    CSFalconContai  29      SeImpersonatePrivilege          Impersonate a client after authentication
6624    CSFalconContai  30      SeCreateGlobalPrivilege         Create global objects
6624    CSFalconContai  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
6624    CSFalconContai  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
6624    CSFalconContai  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
6624    CSFalconContai  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
6624    CSFalconContai  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
6624    CSFalconContai  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
6908    sihost.exe      2       SeCreateTokenPrivilege          Create a token object
6908    sihost.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
6908    sihost.exe      4       SeLockMemoryPrivilege           Lock pages in memory
6908    sihost.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
6908    sihost.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
6908    sihost.exe      7       SeTcbPrivilege          Act as part of the operating system
6908    sihost.exe      8       SeSecurityPrivilege             Manage auditing and security log
6908    sihost.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
6908    sihost.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
6908    sihost.exe      11      SeSystemProfilePrivilege                Profile system performance
6908    sihost.exe      12      SeSystemtimePrivilege           Change the system time
6908    sihost.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
6908    sihost.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
6908    sihost.exe      15      SeCreatePagefilePrivilege               Create a pagefile
6908    sihost.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
6908    sihost.exe      17      SeBackupPrivilege               Backup files and directories
6908    sihost.exe      18      SeRestorePrivilege              Restore files and directories
6908    sihost.exe      19      SeShutdownPrivilege     Present Shut down the system
6908    sihost.exe      20      SeDebugPrivilege                Debug programs
6908    sihost.exe      21      SeAuditPrivilege                Generate security audits
6908    sihost.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
6908    sihost.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
6908    sihost.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
6908    sihost.exe      25      SeUndockPrivilege       Present Remove computer from docking station
6908    sihost.exe      26      SeSyncAgentPrivilege            Synch directory service data
6908    sihost.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
6908    sihost.exe      28      SeManageVolumePrivilege         Manage the files on a volume
6908    sihost.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
6908    sihost.exe      30      SeCreateGlobalPrivilege Default Create global objects
6908    sihost.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
6908    sihost.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
6908    sihost.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
6908    sihost.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
6908    sihost.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
6908    sihost.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7108    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
7108    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7108    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
7108    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
7108    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
7108    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
7108    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
7108    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7108    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
7108    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
7108    svchost.exe     12      SeSystemtimePrivilege           Change the system time
7108    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
7108    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7108    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
7108    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
7108    svchost.exe     17      SeBackupPrivilege               Backup files and directories
7108    svchost.exe     18      SeRestorePrivilege              Restore files and directories
7108    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
7108    svchost.exe     20      SeDebugPrivilege                Debug programs
7108    svchost.exe     21      SeAuditPrivilege                Generate security audits
7108    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7108    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7108    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7108    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
7108    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
7108    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7108    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
7108    svchost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
7108    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
7108    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7108    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7108    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7108    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7108    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7108    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7152    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
7152    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7152    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
7152    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
7152    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
7152    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
7152    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
7152    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7152    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
7152    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
7152    svchost.exe     12      SeSystemtimePrivilege           Change the system time
7152    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
7152    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7152    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
7152    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
7152    svchost.exe     17      SeBackupPrivilege               Backup files and directories
7152    svchost.exe     18      SeRestorePrivilege              Restore files and directories
7152    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
7152    svchost.exe     20      SeDebugPrivilege                Debug programs
7152    svchost.exe     21      SeAuditPrivilege                Generate security audits
7152    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7152    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7152    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7152    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
7152    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
7152    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7152    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
7152    svchost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
7152    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
7152    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7152    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7152    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7152    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7152    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7152    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7156    userinit.exe    2       SeCreateTokenPrivilege          Create a token object
7156    userinit.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7156    userinit.exe    4       SeLockMemoryPrivilege           Lock pages in memory
7156    userinit.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
7156    userinit.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
7156    userinit.exe    7       SeTcbPrivilege          Act as part of the operating system
7156    userinit.exe    8       SeSecurityPrivilege             Manage auditing and security log
7156    userinit.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7156    userinit.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
7156    userinit.exe    11      SeSystemProfilePrivilege                Profile system performance
7156    userinit.exe    12      SeSystemtimePrivilege           Change the system time
7156    userinit.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
7156    userinit.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7156    userinit.exe    15      SeCreatePagefilePrivilege               Create a pagefile
7156    userinit.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
7156    userinit.exe    17      SeBackupPrivilege               Backup files and directories
7156    userinit.exe    18      SeRestorePrivilege              Restore files and directories
7156    userinit.exe    19      SeShutdownPrivilege     Present Shut down the system
7156    userinit.exe    20      SeDebugPrivilege                Debug programs
7156    userinit.exe    21      SeAuditPrivilege                Generate security audits
7156    userinit.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7156    userinit.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7156    userinit.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7156    userinit.exe    25      SeUndockPrivilege       Present Remove computer from docking station
7156    userinit.exe    26      SeSyncAgentPrivilege            Synch directory service data
7156    userinit.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7156    userinit.exe    28      SeManageVolumePrivilege         Manage the files on a volume
7156    userinit.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
7156    userinit.exe    30      SeCreateGlobalPrivilege Default Create global objects
7156    userinit.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7156    userinit.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7156    userinit.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7156    userinit.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7156    userinit.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7156    userinit.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
5328    explorer.exe    2       SeCreateTokenPrivilege          Create a token object
5328    explorer.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
5328    explorer.exe    4       SeLockMemoryPrivilege           Lock pages in memory
5328    explorer.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
5328    explorer.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
5328    explorer.exe    7       SeTcbPrivilege          Act as part of the operating system
5328    explorer.exe    8       SeSecurityPrivilege             Manage auditing and security log
5328    explorer.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
5328    explorer.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
5328    explorer.exe    11      SeSystemProfilePrivilege                Profile system performance
5328    explorer.exe    12      SeSystemtimePrivilege           Change the system time
5328    explorer.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
5328    explorer.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
5328    explorer.exe    15      SeCreatePagefilePrivilege               Create a pagefile
5328    explorer.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
5328    explorer.exe    17      SeBackupPrivilege               Backup files and directories
5328    explorer.exe    18      SeRestorePrivilege              Restore files and directories
5328    explorer.exe    19      SeShutdownPrivilege     Present Shut down the system
5328    explorer.exe    20      SeDebugPrivilege                Debug programs
5328    explorer.exe    21      SeAuditPrivilege                Generate security audits
5328    explorer.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5328    explorer.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
5328    explorer.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5328    explorer.exe    25      SeUndockPrivilege       Present Remove computer from docking station
5328    explorer.exe    26      SeSyncAgentPrivilege            Synch directory service data
5328    explorer.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5328    explorer.exe    28      SeManageVolumePrivilege         Manage the files on a volume
5328    explorer.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
5328    explorer.exe    30      SeCreateGlobalPrivilege Default Create global objects
5328    explorer.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5328    explorer.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5328    explorer.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
5328    explorer.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
5328    explorer.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
5328    explorer.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1244    Sysmon64.exe    2       SeCreateTokenPrivilege          Create a token object
1244    Sysmon64.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1244    Sysmon64.exe    4       SeLockMemoryPrivilege           Lock pages in memory
1244    Sysmon64.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
1244    Sysmon64.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
1244    Sysmon64.exe    7       SeTcbPrivilege          Act as part of the operating system
1244    Sysmon64.exe    8       SeSecurityPrivilege             Manage auditing and security log
1244    Sysmon64.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1244    Sysmon64.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
1244    Sysmon64.exe    11      SeSystemProfilePrivilege                Profile system performance
1244    Sysmon64.exe    12      SeSystemtimePrivilege           Change the system time
1244    Sysmon64.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
1244    Sysmon64.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1244    Sysmon64.exe    15      SeCreatePagefilePrivilege               Create a pagefile
1244    Sysmon64.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
1244    Sysmon64.exe    17      SeBackupPrivilege               Backup files and directories
1244    Sysmon64.exe    18      SeRestorePrivilege              Restore files and directories
1244    Sysmon64.exe    19      SeShutdownPrivilege     Present Shut down the system
1244    Sysmon64.exe    20      SeDebugPrivilege                Debug programs
1244    Sysmon64.exe    21      SeAuditPrivilege                Generate security audits
1244    Sysmon64.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1244    Sysmon64.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1244    Sysmon64.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1244    Sysmon64.exe    25      SeUndockPrivilege       Present Remove computer from docking station
1244    Sysmon64.exe    26      SeSyncAgentPrivilege            Synch directory service data
1244    Sysmon64.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1244    Sysmon64.exe    28      SeManageVolumePrivilege         Manage the files on a volume
1244    Sysmon64.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
1244    Sysmon64.exe    30      SeCreateGlobalPrivilege Default Create global objects
1244    Sysmon64.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1244    Sysmon64.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1244    Sysmon64.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
1244    Sysmon64.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
1244    Sysmon64.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1244    Sysmon64.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
4428    taskhostw.exe   2       SeCreateTokenPrivilege          Create a token object
4428    taskhostw.exe   3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
4428    taskhostw.exe   4       SeLockMemoryPrivilege           Lock pages in memory
4428    taskhostw.exe   5       SeIncreaseQuotaPrivilege                Increase quotas
4428    taskhostw.exe   6       SeMachineAccountPrivilege               Add workstations to the domain
4428    taskhostw.exe   7       SeTcbPrivilege          Act as part of the operating system
4428    taskhostw.exe   8       SeSecurityPrivilege             Manage auditing and security log
4428    taskhostw.exe   9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4428    taskhostw.exe   10      SeLoadDriverPrivilege           Load and unload device drivers
4428    taskhostw.exe   11      SeSystemProfilePrivilege                Profile system performance
4428    taskhostw.exe   12      SeSystemtimePrivilege           Change the system time
4428    taskhostw.exe   13      SeProfileSingleProcessPrivilege         Profile a single process
4428    taskhostw.exe   14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4428    taskhostw.exe   15      SeCreatePagefilePrivilege               Create a pagefile
4428    taskhostw.exe   16      SeCreatePermanentPrivilege              Create permanent shared objects
4428    taskhostw.exe   17      SeBackupPrivilege               Backup files and directories
4428    taskhostw.exe   18      SeRestorePrivilege              Restore files and directories
4428    taskhostw.exe   19      SeShutdownPrivilege     Present Shut down the system
4428    taskhostw.exe   20      SeDebugPrivilege                Debug programs
4428    taskhostw.exe   21      SeAuditPrivilege                Generate security audits
4428    taskhostw.exe   22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4428    taskhostw.exe   23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4428    taskhostw.exe   24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4428    taskhostw.exe   25      SeUndockPrivilege       Present Remove computer from docking station
4428    taskhostw.exe   26      SeSyncAgentPrivilege            Synch directory service data
4428    taskhostw.exe   27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4428    taskhostw.exe   28      SeManageVolumePrivilege         Manage the files on a volume
4428    taskhostw.exe   29      SeImpersonatePrivilege          Impersonate a client after authentication
4428    taskhostw.exe   30      SeCreateGlobalPrivilege Default Create global objects
4428    taskhostw.exe   31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4428    taskhostw.exe   32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4428    taskhostw.exe   33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
4428    taskhostw.exe   34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
4428    taskhostw.exe   35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4428    taskhostw.exe   36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
4732    conhost.exe     2       SeCreateTokenPrivilege          Create a token object
4732    conhost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
4732    conhost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
4732    conhost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
4732    conhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
4732    conhost.exe     7       SeTcbPrivilege          Act as part of the operating system
4732    conhost.exe     8       SeSecurityPrivilege             Manage auditing and security log
4732    conhost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4732    conhost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
4732    conhost.exe     11      SeSystemProfilePrivilege                Profile system performance
4732    conhost.exe     12      SeSystemtimePrivilege           Change the system time
4732    conhost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
4732    conhost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4732    conhost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
4732    conhost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
4732    conhost.exe     17      SeBackupPrivilege               Backup files and directories
4732    conhost.exe     18      SeRestorePrivilege              Restore files and directories
4732    conhost.exe     19      SeShutdownPrivilege     Present Shut down the system
4732    conhost.exe     20      SeDebugPrivilege                Debug programs
4732    conhost.exe     21      SeAuditPrivilege                Generate security audits
4732    conhost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4732    conhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4732    conhost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4732    conhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
4732    conhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
4732    conhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4732    conhost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
4732    conhost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
4732    conhost.exe     30      SeCreateGlobalPrivilege Default Create global objects
4732    conhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4732    conhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4732    conhost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
4732    conhost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
4732    conhost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4732    conhost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
5340    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
5340    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
5340    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
5340    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
5340    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
5340    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
5340    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
5340    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
5340    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
5340    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
5340    svchost.exe     12      SeSystemtimePrivilege           Change the system time
5340    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
5340    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
5340    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
5340    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
5340    svchost.exe     17      SeBackupPrivilege               Backup files and directories
5340    svchost.exe     18      SeRestorePrivilege              Restore files and directories
5340    svchost.exe     19      SeShutdownPrivilege             Shut down the system
5340    svchost.exe     20      SeDebugPrivilege        Default Debug programs
5340    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
5340    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5340    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
5340    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5340    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
5340    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
5340    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5340    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
5340    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
5340    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
5340    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5340    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5340    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
5340    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
5340    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
5340    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
1480    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
1480    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1480    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
1480    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
1480    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1480    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
1480    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
1480    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1480    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
1480    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
1480    svchost.exe     12      SeSystemtimePrivilege           Change the system time
1480    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
1480    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1480    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
1480    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
1480    svchost.exe     17      SeBackupPrivilege               Backup files and directories
1480    svchost.exe     18      SeRestorePrivilege              Restore files and directories
1480    svchost.exe     19      SeShutdownPrivilege             Shut down the system
1480    svchost.exe     20      SeDebugPrivilege                Debug programs
1480    svchost.exe     21      SeAuditPrivilege                Generate security audits
1480    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1480    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1480    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1480    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
1480    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1480    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1480    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
1480    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1480    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
1480    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1480    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1480    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1480    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1480    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1480    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3604    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3604    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3604    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
3604    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3604    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3604    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
3604    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3604    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3604    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3604    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
3604    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3604    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
3604    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3604    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
3604    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
3604    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3604    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3604    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
3604    svchost.exe     20      SeDebugPrivilege                Debug programs
3604    svchost.exe     21      SeAuditPrivilege                Generate security audits
3604    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3604    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3604    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3604    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
3604    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3604    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3604    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3604    svchost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
3604    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
3604    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3604    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3604    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
3604    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
3604    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3604    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7408    StartMenuExper  2       SeCreateTokenPrivilege          Create a token object
7408    StartMenuExper  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7408    StartMenuExper  4       SeLockMemoryPrivilege           Lock pages in memory
7408    StartMenuExper  5       SeIncreaseQuotaPrivilege                Increase quotas
7408    StartMenuExper  6       SeMachineAccountPrivilege               Add workstations to the domain
7408    StartMenuExper  7       SeTcbPrivilege          Act as part of the operating system
7408    StartMenuExper  8       SeSecurityPrivilege             Manage auditing and security log
7408    StartMenuExper  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7408    StartMenuExper  10      SeLoadDriverPrivilege           Load and unload device drivers
7408    StartMenuExper  11      SeSystemProfilePrivilege                Profile system performance
7408    StartMenuExper  12      SeSystemtimePrivilege           Change the system time
7408    StartMenuExper  13      SeProfileSingleProcessPrivilege         Profile a single process
7408    StartMenuExper  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7408    StartMenuExper  15      SeCreatePagefilePrivilege               Create a pagefile
7408    StartMenuExper  16      SeCreatePermanentPrivilege              Create permanent shared objects
7408    StartMenuExper  17      SeBackupPrivilege               Backup files and directories
7408    StartMenuExper  18      SeRestorePrivilege              Restore files and directories
7408    StartMenuExper  19      SeShutdownPrivilege             Shut down the system
7408    StartMenuExper  20      SeDebugPrivilege                Debug programs
7408    StartMenuExper  21      SeAuditPrivilege                Generate security audits
7408    StartMenuExper  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7408    StartMenuExper  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7408    StartMenuExper  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7408    StartMenuExper  25      SeUndockPrivilege               Remove computer from docking station
7408    StartMenuExper  26      SeSyncAgentPrivilege            Synch directory service data
7408    StartMenuExper  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7408    StartMenuExper  28      SeManageVolumePrivilege         Manage the files on a volume
7408    StartMenuExper  29      SeImpersonatePrivilege          Impersonate a client after authentication
7408    StartMenuExper  30      SeCreateGlobalPrivilege         Create global objects
7408    StartMenuExper  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7408    StartMenuExper  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7408    StartMenuExper  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7408    StartMenuExper  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
7408    StartMenuExper  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7408    StartMenuExper  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7684    RuntimeBroker.  2       SeCreateTokenPrivilege          Create a token object
7684    RuntimeBroker.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7684    RuntimeBroker.  4       SeLockMemoryPrivilege           Lock pages in memory
7684    RuntimeBroker.  5       SeIncreaseQuotaPrivilege                Increase quotas
7684    RuntimeBroker.  6       SeMachineAccountPrivilege               Add workstations to the domain
7684    RuntimeBroker.  7       SeTcbPrivilege          Act as part of the operating system
7684    RuntimeBroker.  8       SeSecurityPrivilege             Manage auditing and security log
7684    RuntimeBroker.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7684    RuntimeBroker.  10      SeLoadDriverPrivilege           Load and unload device drivers
7684    RuntimeBroker.  11      SeSystemProfilePrivilege                Profile system performance
7684    RuntimeBroker.  12      SeSystemtimePrivilege           Change the system time
7684    RuntimeBroker.  13      SeProfileSingleProcessPrivilege         Profile a single process
7684    RuntimeBroker.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7684    RuntimeBroker.  15      SeCreatePagefilePrivilege               Create a pagefile
7684    RuntimeBroker.  16      SeCreatePermanentPrivilege              Create permanent shared objects
7684    RuntimeBroker.  17      SeBackupPrivilege               Backup files and directories
7684    RuntimeBroker.  18      SeRestorePrivilege              Restore files and directories
7684    RuntimeBroker.  19      SeShutdownPrivilege     Present Shut down the system
7684    RuntimeBroker.  20      SeDebugPrivilege                Debug programs
7684    RuntimeBroker.  21      SeAuditPrivilege                Generate security audits
7684    RuntimeBroker.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7684    RuntimeBroker.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7684    RuntimeBroker.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7684    RuntimeBroker.  25      SeUndockPrivilege       Present Remove computer from docking station
7684    RuntimeBroker.  26      SeSyncAgentPrivilege            Synch directory service data
7684    RuntimeBroker.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7684    RuntimeBroker.  28      SeManageVolumePrivilege         Manage the files on a volume
7684    RuntimeBroker.  29      SeImpersonatePrivilege          Impersonate a client after authentication
7684    RuntimeBroker.  30      SeCreateGlobalPrivilege Default Create global objects
7684    RuntimeBroker.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7684    RuntimeBroker.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7684    RuntimeBroker.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7684    RuntimeBroker.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7684    RuntimeBroker.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7684    RuntimeBroker.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7780    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
7780    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
7780    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
7780    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
7780    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
7780    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
7780    svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
7780    svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
7780    svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
7780    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
7780    svchost.exe     12      SeSystemtimePrivilege           Change the system time
7780    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
7780    svchost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
7780    svchost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
7780    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
7780    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
7780    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
7780    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
7780    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
7780    svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
7780    svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
7780    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7780    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7780    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
7780    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
7780    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7780    svchost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
7780    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
7780    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
7780    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7780    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7780    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
7780    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
7780    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
7780    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
7932    SearchApp.exe   2       SeCreateTokenPrivilege          Create a token object
7932    SearchApp.exe   3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7932    SearchApp.exe   4       SeLockMemoryPrivilege           Lock pages in memory
7932    SearchApp.exe   5       SeIncreaseQuotaPrivilege                Increase quotas
7932    SearchApp.exe   6       SeMachineAccountPrivilege               Add workstations to the domain
7932    SearchApp.exe   7       SeTcbPrivilege          Act as part of the operating system
7932    SearchApp.exe   8       SeSecurityPrivilege             Manage auditing and security log
7932    SearchApp.exe   9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7932    SearchApp.exe   10      SeLoadDriverPrivilege           Load and unload device drivers
7932    SearchApp.exe   11      SeSystemProfilePrivilege                Profile system performance
7932    SearchApp.exe   12      SeSystemtimePrivilege           Change the system time
7932    SearchApp.exe   13      SeProfileSingleProcessPrivilege         Profile a single process
7932    SearchApp.exe   14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7932    SearchApp.exe   15      SeCreatePagefilePrivilege               Create a pagefile
7932    SearchApp.exe   16      SeCreatePermanentPrivilege              Create permanent shared objects
7932    SearchApp.exe   17      SeBackupPrivilege               Backup files and directories
7932    SearchApp.exe   18      SeRestorePrivilege              Restore files and directories
7932    SearchApp.exe   19      SeShutdownPrivilege             Shut down the system
7932    SearchApp.exe   20      SeDebugPrivilege                Debug programs
7932    SearchApp.exe   21      SeAuditPrivilege                Generate security audits
7932    SearchApp.exe   22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7932    SearchApp.exe   23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7932    SearchApp.exe   24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7932    SearchApp.exe   25      SeUndockPrivilege               Remove computer from docking station
7932    SearchApp.exe   26      SeSyncAgentPrivilege            Synch directory service data
7932    SearchApp.exe   27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7932    SearchApp.exe   28      SeManageVolumePrivilege         Manage the files on a volume
7932    SearchApp.exe   29      SeImpersonatePrivilege          Impersonate a client after authentication
7932    SearchApp.exe   30      SeCreateGlobalPrivilege         Create global objects
7932    SearchApp.exe   31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7932    SearchApp.exe   32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7932    SearchApp.exe   33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7932    SearchApp.exe   34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
7932    SearchApp.exe   35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7932    SearchApp.exe   36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
8120    RuntimeBroker.  2       SeCreateTokenPrivilege          Create a token object
8120    RuntimeBroker.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8120    RuntimeBroker.  4       SeLockMemoryPrivilege           Lock pages in memory
8120    RuntimeBroker.  5       SeIncreaseQuotaPrivilege                Increase quotas
8120    RuntimeBroker.  6       SeMachineAccountPrivilege               Add workstations to the domain
8120    RuntimeBroker.  7       SeTcbPrivilege          Act as part of the operating system
8120    RuntimeBroker.  8       SeSecurityPrivilege             Manage auditing and security log
8120    RuntimeBroker.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8120    RuntimeBroker.  10      SeLoadDriverPrivilege           Load and unload device drivers
8120    RuntimeBroker.  11      SeSystemProfilePrivilege                Profile system performance
8120    RuntimeBroker.  12      SeSystemtimePrivilege           Change the system time
8120    RuntimeBroker.  13      SeProfileSingleProcessPrivilege         Profile a single process
8120    RuntimeBroker.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
8120    RuntimeBroker.  15      SeCreatePagefilePrivilege               Create a pagefile
8120    RuntimeBroker.  16      SeCreatePermanentPrivilege              Create permanent shared objects
8120    RuntimeBroker.  17      SeBackupPrivilege               Backup files and directories
8120    RuntimeBroker.  18      SeRestorePrivilege              Restore files and directories
8120    RuntimeBroker.  19      SeShutdownPrivilege     Present Shut down the system
8120    RuntimeBroker.  20      SeDebugPrivilege                Debug programs
8120    RuntimeBroker.  21      SeAuditPrivilege                Generate security audits
8120    RuntimeBroker.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8120    RuntimeBroker.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8120    RuntimeBroker.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8120    RuntimeBroker.  25      SeUndockPrivilege       Present Remove computer from docking station
8120    RuntimeBroker.  26      SeSyncAgentPrivilege            Synch directory service data
8120    RuntimeBroker.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8120    RuntimeBroker.  28      SeManageVolumePrivilege         Manage the files on a volume
8120    RuntimeBroker.  29      SeImpersonatePrivilege          Impersonate a client after authentication
8120    RuntimeBroker.  30      SeCreateGlobalPrivilege Default Create global objects
8120    RuntimeBroker.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8120    RuntimeBroker.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8120    RuntimeBroker.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
8120    RuntimeBroker.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
8120    RuntimeBroker.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
8120    RuntimeBroker.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1256    TextInputHost.  2       SeCreateTokenPrivilege          Create a token object
1256    TextInputHost.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1256    TextInputHost.  4       SeLockMemoryPrivilege           Lock pages in memory
1256    TextInputHost.  5       SeIncreaseQuotaPrivilege                Increase quotas
1256    TextInputHost.  6       SeMachineAccountPrivilege               Add workstations to the domain
1256    TextInputHost.  7       SeTcbPrivilege          Act as part of the operating system
1256    TextInputHost.  8       SeSecurityPrivilege             Manage auditing and security log
1256    TextInputHost.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1256    TextInputHost.  10      SeLoadDriverPrivilege           Load and unload device drivers
1256    TextInputHost.  11      SeSystemProfilePrivilege                Profile system performance
1256    TextInputHost.  12      SeSystemtimePrivilege           Change the system time
1256    TextInputHost.  13      SeProfileSingleProcessPrivilege         Profile a single process
1256    TextInputHost.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1256    TextInputHost.  15      SeCreatePagefilePrivilege               Create a pagefile
1256    TextInputHost.  16      SeCreatePermanentPrivilege              Create permanent shared objects
1256    TextInputHost.  17      SeBackupPrivilege               Backup files and directories
1256    TextInputHost.  18      SeRestorePrivilege              Restore files and directories
1256    TextInputHost.  19      SeShutdownPrivilege     Present Shut down the system
1256    TextInputHost.  20      SeDebugPrivilege                Debug programs
1256    TextInputHost.  21      SeAuditPrivilege                Generate security audits
1256    TextInputHost.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1256    TextInputHost.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1256    TextInputHost.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1256    TextInputHost.  25      SeUndockPrivilege       Present Remove computer from docking station
1256    TextInputHost.  26      SeSyncAgentPrivilege            Synch directory service data
1256    TextInputHost.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1256    TextInputHost.  28      SeManageVolumePrivilege         Manage the files on a volume
1256    TextInputHost.  29      SeImpersonatePrivilege          Impersonate a client after authentication
1256    TextInputHost.  30      SeCreateGlobalPrivilege Default Create global objects
1256    TextInputHost.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1256    TextInputHost.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1256    TextInputHost.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
1256    TextInputHost.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
1256    TextInputHost.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1256    TextInputHost.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7612    RuntimeBroker.  2       SeCreateTokenPrivilege          Create a token object
7612    RuntimeBroker.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7612    RuntimeBroker.  4       SeLockMemoryPrivilege           Lock pages in memory
7612    RuntimeBroker.  5       SeIncreaseQuotaPrivilege                Increase quotas
7612    RuntimeBroker.  6       SeMachineAccountPrivilege               Add workstations to the domain
7612    RuntimeBroker.  7       SeTcbPrivilege          Act as part of the operating system
7612    RuntimeBroker.  8       SeSecurityPrivilege             Manage auditing and security log
7612    RuntimeBroker.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7612    RuntimeBroker.  10      SeLoadDriverPrivilege           Load and unload device drivers
7612    RuntimeBroker.  11      SeSystemProfilePrivilege                Profile system performance
7612    RuntimeBroker.  12      SeSystemtimePrivilege           Change the system time
7612    RuntimeBroker.  13      SeProfileSingleProcessPrivilege         Profile a single process
7612    RuntimeBroker.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7612    RuntimeBroker.  15      SeCreatePagefilePrivilege               Create a pagefile
7612    RuntimeBroker.  16      SeCreatePermanentPrivilege              Create permanent shared objects
7612    RuntimeBroker.  17      SeBackupPrivilege               Backup files and directories
7612    RuntimeBroker.  18      SeRestorePrivilege              Restore files and directories
7612    RuntimeBroker.  19      SeShutdownPrivilege     Present Shut down the system
7612    RuntimeBroker.  20      SeDebugPrivilege                Debug programs
7612    RuntimeBroker.  21      SeAuditPrivilege                Generate security audits
7612    RuntimeBroker.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7612    RuntimeBroker.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7612    RuntimeBroker.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7612    RuntimeBroker.  25      SeUndockPrivilege       Present Remove computer from docking station
7612    RuntimeBroker.  26      SeSyncAgentPrivilege            Synch directory service data
7612    RuntimeBroker.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7612    RuntimeBroker.  28      SeManageVolumePrivilege         Manage the files on a volume
7612    RuntimeBroker.  29      SeImpersonatePrivilege          Impersonate a client after authentication
7612    RuntimeBroker.  30      SeCreateGlobalPrivilege Default Create global objects
7612    RuntimeBroker.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7612    RuntimeBroker.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7612    RuntimeBroker.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7612    RuntimeBroker.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7612    RuntimeBroker.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7612    RuntimeBroker.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
8292    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
8292    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8292    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
8292    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
8292    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
8292    svchost.exe     7       SeTcbPrivilege  Default Act as part of the operating system
8292    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
8292    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8292    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
8292    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
8292    svchost.exe     12      SeSystemtimePrivilege           Change the system time
8292    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
8292    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
8292    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
8292    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
8292    svchost.exe     17      SeBackupPrivilege               Backup files and directories
8292    svchost.exe     18      SeRestorePrivilege              Restore files and directories
8292    svchost.exe     19      SeShutdownPrivilege             Shut down the system
8292    svchost.exe     20      SeDebugPrivilege        Default Debug programs
8292    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
8292    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8292    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8292    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8292    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
8292    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
8292    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8292    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
8292    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
8292    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
8292    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8292    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8292    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
8292    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
8292    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
8292    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
8472    dllhost.exe     2       SeCreateTokenPrivilege          Create a token object
8472    dllhost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8472    dllhost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
8472    dllhost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
8472    dllhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
8472    dllhost.exe     7       SeTcbPrivilege          Act as part of the operating system
8472    dllhost.exe     8       SeSecurityPrivilege             Manage auditing and security log
8472    dllhost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8472    dllhost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
8472    dllhost.exe     11      SeSystemProfilePrivilege                Profile system performance
8472    dllhost.exe     12      SeSystemtimePrivilege           Change the system time
8472    dllhost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
8472    dllhost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
8472    dllhost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
8472    dllhost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
8472    dllhost.exe     17      SeBackupPrivilege               Backup files and directories
8472    dllhost.exe     18      SeRestorePrivilege              Restore files and directories
8472    dllhost.exe     19      SeShutdownPrivilege     Present Shut down the system
8472    dllhost.exe     20      SeDebugPrivilege                Debug programs
8472    dllhost.exe     21      SeAuditPrivilege                Generate security audits
8472    dllhost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8472    dllhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8472    dllhost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8472    dllhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
8472    dllhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
8472    dllhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8472    dllhost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
8472    dllhost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
8472    dllhost.exe     30      SeCreateGlobalPrivilege Default Create global objects
8472    dllhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8472    dllhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8472    dllhost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
8472    dllhost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
8472    dllhost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
8472    dllhost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
8688    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
8688    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
8688    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
8688    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
8688    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
8688    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
8688    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
8688    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8688    svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
8688    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
8688    svchost.exe     12      SeSystemtimePrivilege           Change the system time
8688    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
8688    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
8688    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
8688    svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
8688    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
8688    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
8688    svchost.exe     19      SeShutdownPrivilege             Shut down the system
8688    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
8688    svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
8688    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8688    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8688    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8688    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
8688    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
8688    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8688    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
8688    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
8688    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
8688    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8688    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8688    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
8688    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
8688    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
8688    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
8736    SecurityHealth  2       SeCreateTokenPrivilege          Create a token object
8736    SecurityHealth  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8736    SecurityHealth  4       SeLockMemoryPrivilege           Lock pages in memory
8736    SecurityHealth  5       SeIncreaseQuotaPrivilege                Increase quotas
8736    SecurityHealth  6       SeMachineAccountPrivilege               Add workstations to the domain
8736    SecurityHealth  7       SeTcbPrivilege          Act as part of the operating system
8736    SecurityHealth  8       SeSecurityPrivilege             Manage auditing and security log
8736    SecurityHealth  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8736    SecurityHealth  10      SeLoadDriverPrivilege           Load and unload device drivers
8736    SecurityHealth  11      SeSystemProfilePrivilege                Profile system performance
8736    SecurityHealth  12      SeSystemtimePrivilege           Change the system time
8736    SecurityHealth  13      SeProfileSingleProcessPrivilege         Profile a single process
8736    SecurityHealth  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
8736    SecurityHealth  15      SeCreatePagefilePrivilege               Create a pagefile
8736    SecurityHealth  16      SeCreatePermanentPrivilege              Create permanent shared objects
8736    SecurityHealth  17      SeBackupPrivilege               Backup files and directories
8736    SecurityHealth  18      SeRestorePrivilege              Restore files and directories
8736    SecurityHealth  19      SeShutdownPrivilege     Present Shut down the system
8736    SecurityHealth  20      SeDebugPrivilege                Debug programs
8736    SecurityHealth  21      SeAuditPrivilege                Generate security audits
8736    SecurityHealth  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8736    SecurityHealth  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8736    SecurityHealth  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8736    SecurityHealth  25      SeUndockPrivilege       Present Remove computer from docking station
8736    SecurityHealth  26      SeSyncAgentPrivilege            Synch directory service data
8736    SecurityHealth  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8736    SecurityHealth  28      SeManageVolumePrivilege         Manage the files on a volume
8736    SecurityHealth  29      SeImpersonatePrivilege          Impersonate a client after authentication
8736    SecurityHealth  30      SeCreateGlobalPrivilege Default Create global objects
8736    SecurityHealth  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8736    SecurityHealth  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8736    SecurityHealth  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
8736    SecurityHealth  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
8736    SecurityHealth  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
8736    SecurityHealth  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
8808    SecurityHealth  2       SeCreateTokenPrivilege          Create a token object
8808    SecurityHealth  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
8808    SecurityHealth  4       SeLockMemoryPrivilege   Default Lock pages in memory
8808    SecurityHealth  5       SeIncreaseQuotaPrivilege                Increase quotas
8808    SecurityHealth  6       SeMachineAccountPrivilege               Add workstations to the domain
8808    SecurityHealth  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
8808    SecurityHealth  8       SeSecurityPrivilege     Present Manage auditing and security log
8808    SecurityHealth  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8808    SecurityHealth  10      SeLoadDriverPrivilege           Load and unload device drivers
8808    SecurityHealth  11      SeSystemProfilePrivilege        Default Profile system performance
8808    SecurityHealth  12      SeSystemtimePrivilege           Change the system time
8808    SecurityHealth  13      SeProfileSingleProcessPrivilege Default Profile a single process
8808    SecurityHealth  14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
8808    SecurityHealth  15      SeCreatePagefilePrivilege       Default Create a pagefile
8808    SecurityHealth  16      SeCreatePermanentPrivilege      Default Create permanent shared objects
8808    SecurityHealth  17      SeBackupPrivilege       Present Backup files and directories
8808    SecurityHealth  18      SeRestorePrivilege      Present Restore files and directories
8808    SecurityHealth  19      SeShutdownPrivilege     Present Shut down the system
8808    SecurityHealth  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
8808    SecurityHealth  21      SeAuditPrivilege        Default Generate security audits
8808    SecurityHealth  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
8808    SecurityHealth  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8808    SecurityHealth  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8808    SecurityHealth  25      SeUndockPrivilege               Remove computer from docking station
8808    SecurityHealth  26      SeSyncAgentPrivilege            Synch directory service data
8808    SecurityHealth  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8808    SecurityHealth  28      SeManageVolumePrivilege         Manage the files on a volume
8808    SecurityHealth  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
8808    SecurityHealth  30      SeCreateGlobalPrivilege Default Create global objects
8808    SecurityHealth  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8808    SecurityHealth  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8808    SecurityHealth  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
8808    SecurityHealth  34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
8808    SecurityHealth  35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
8808    SecurityHealth  36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
9116    OneDrive.exe    2       SeCreateTokenPrivilege          Create a token object
9116    OneDrive.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9116    OneDrive.exe    4       SeLockMemoryPrivilege           Lock pages in memory
9116    OneDrive.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
9116    OneDrive.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
9116    OneDrive.exe    7       SeTcbPrivilege          Act as part of the operating system
9116    OneDrive.exe    8       SeSecurityPrivilege             Manage auditing and security log
9116    OneDrive.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9116    OneDrive.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
9116    OneDrive.exe    11      SeSystemProfilePrivilege                Profile system performance
9116    OneDrive.exe    12      SeSystemtimePrivilege           Change the system time
9116    OneDrive.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
9116    OneDrive.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9116    OneDrive.exe    15      SeCreatePagefilePrivilege               Create a pagefile
9116    OneDrive.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
9116    OneDrive.exe    17      SeBackupPrivilege               Backup files and directories
9116    OneDrive.exe    18      SeRestorePrivilege              Restore files and directories
9116    OneDrive.exe    19      SeShutdownPrivilege     Present Shut down the system
9116    OneDrive.exe    20      SeDebugPrivilege                Debug programs
9116    OneDrive.exe    21      SeAuditPrivilege                Generate security audits
9116    OneDrive.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9116    OneDrive.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9116    OneDrive.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9116    OneDrive.exe    25      SeUndockPrivilege       Present Remove computer from docking station
9116    OneDrive.exe    26      SeSyncAgentPrivilege            Synch directory service data
9116    OneDrive.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9116    OneDrive.exe    28      SeManageVolumePrivilege         Manage the files on a volume
9116    OneDrive.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
9116    OneDrive.exe    30      SeCreateGlobalPrivilege Default Create global objects
9116    OneDrive.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9116    OneDrive.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9116    OneDrive.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
9116    OneDrive.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
9116    OneDrive.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9116    OneDrive.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9172    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
9172    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9172    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
9172    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
9172    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
9172    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
9172    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
9172    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9172    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
9172    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
9172    msedge.exe      12      SeSystemtimePrivilege           Change the system time
9172    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
9172    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9172    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
9172    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
9172    msedge.exe      17      SeBackupPrivilege               Backup files and directories
9172    msedge.exe      18      SeRestorePrivilege              Restore files and directories
9172    msedge.exe      19      SeShutdownPrivilege     Present Shut down the system
9172    msedge.exe      20      SeDebugPrivilege                Debug programs
9172    msedge.exe      21      SeAuditPrivilege                Generate security audits
9172    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9172    msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9172    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9172    msedge.exe      25      SeUndockPrivilege       Present Remove computer from docking station
9172    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
9172    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9172    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
9172    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
9172    msedge.exe      30      SeCreateGlobalPrivilege Default Create global objects
9172    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9172    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9172    msedge.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
9172    msedge.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
9172    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9172    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3832    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
3832    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3832    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
3832    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
3832    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
3832    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
3832    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
3832    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3832    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
3832    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
3832    msedge.exe      12      SeSystemtimePrivilege           Change the system time
3832    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
3832    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3832    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
3832    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
3832    msedge.exe      17      SeBackupPrivilege               Backup files and directories
3832    msedge.exe      18      SeRestorePrivilege              Restore files and directories
3832    msedge.exe      19      SeShutdownPrivilege     Present Shut down the system
3832    msedge.exe      20      SeDebugPrivilege                Debug programs
3832    msedge.exe      21      SeAuditPrivilege                Generate security audits
3832    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3832    msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3832    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3832    msedge.exe      25      SeUndockPrivilege       Present Remove computer from docking station
3832    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
3832    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3832    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
3832    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
3832    msedge.exe      30      SeCreateGlobalPrivilege Default Create global objects
3832    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3832    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3832    msedge.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
3832    msedge.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
3832    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3832    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
8732    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
8732    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8732    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
8732    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
8732    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
8732    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
8732    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
8732    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8732    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
8732    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
8732    msedge.exe      12      SeSystemtimePrivilege           Change the system time
8732    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
8732    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
8732    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
8732    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
8732    msedge.exe      17      SeBackupPrivilege               Backup files and directories
8732    msedge.exe      18      SeRestorePrivilege              Restore files and directories
8732    msedge.exe      19      SeShutdownPrivilege             Shut down the system
8732    msedge.exe      20      SeDebugPrivilege                Debug programs
8732    msedge.exe      21      SeAuditPrivilege                Generate security audits
8732    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8732    msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8732    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8732    msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
8732    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
8732    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8732    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
8732    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
8732    msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
8732    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8732    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8732    msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
8732    msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
8732    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
8732    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9016    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
9016    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9016    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
9016    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
9016    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
9016    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
9016    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
9016    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9016    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
9016    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
9016    msedge.exe      12      SeSystemtimePrivilege           Change the system time
9016    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
9016    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9016    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
9016    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
9016    msedge.exe      17      SeBackupPrivilege               Backup files and directories
9016    msedge.exe      18      SeRestorePrivilege              Restore files and directories
9016    msedge.exe      19      SeShutdownPrivilege     Present Shut down the system
9016    msedge.exe      20      SeDebugPrivilege                Debug programs
9016    msedge.exe      21      SeAuditPrivilege                Generate security audits
9016    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9016    msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9016    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9016    msedge.exe      25      SeUndockPrivilege       Present Remove computer from docking station
9016    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
9016    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9016    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
9016    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
9016    msedge.exe      30      SeCreateGlobalPrivilege Default Create global objects
9016    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9016    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9016    msedge.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
9016    msedge.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
9016    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9016    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9128    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
9128    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9128    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
9128    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
9128    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
9128    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
9128    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
9128    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9128    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
9128    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
9128    msedge.exe      12      SeSystemtimePrivilege           Change the system time
9128    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
9128    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9128    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
9128    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
9128    msedge.exe      17      SeBackupPrivilege               Backup files and directories
9128    msedge.exe      18      SeRestorePrivilege              Restore files and directories
9128    msedge.exe      19      SeShutdownPrivilege             Shut down the system
9128    msedge.exe      20      SeDebugPrivilege                Debug programs
9128    msedge.exe      21      SeAuditPrivilege                Generate security audits
9128    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9128    msedge.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
9128    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9128    msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
9128    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
9128    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9128    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
9128    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
9128    msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
9128    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9128    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9128    msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
9128    msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
9128    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9128    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9728    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
9728    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9728    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
9728    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
9728    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
9728    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
9728    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
9728    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9728    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
9728    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
9728    msedge.exe      12      SeSystemtimePrivilege           Change the system time
9728    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
9728    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9728    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
9728    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
9728    msedge.exe      17      SeBackupPrivilege               Backup files and directories
9728    msedge.exe      18      SeRestorePrivilege              Restore files and directories
9728    msedge.exe      19      SeShutdownPrivilege             Shut down the system
9728    msedge.exe      20      SeDebugPrivilege                Debug programs
9728    msedge.exe      21      SeAuditPrivilege                Generate security audits
9728    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9728    msedge.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
9728    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9728    msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
9728    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
9728    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9728    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
9728    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
9728    msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
9728    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9728    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9728    msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
9728    msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
9728    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9728    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9888    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
9888    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9888    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
9888    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
9888    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
9888    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
9888    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
9888    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9888    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
9888    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
9888    msedge.exe      12      SeSystemtimePrivilege           Change the system time
9888    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
9888    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9888    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
9888    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
9888    msedge.exe      17      SeBackupPrivilege               Backup files and directories
9888    msedge.exe      18      SeRestorePrivilege              Restore files and directories
9888    msedge.exe      19      SeShutdownPrivilege             Shut down the system
9888    msedge.exe      20      SeDebugPrivilege                Debug programs
9888    msedge.exe      21      SeAuditPrivilege                Generate security audits
9888    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9888    msedge.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
9888    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9888    msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
9888    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
9888    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9888    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
9888    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
9888    msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
9888    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9888    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9888    msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
9888    msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
9888    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9888    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10224   MsSense.exe     2       SeCreateTokenPrivilege          Create a token object
10224   MsSense.exe     3       SeAssignPrimaryTokenPrivilege   Present,Enabled Replace a process-level token
10224   MsSense.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
10224   MsSense.exe     5       SeIncreaseQuotaPrivilege        Present,Enabled Increase quotas
10224   MsSense.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
10224   MsSense.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
10224   MsSense.exe     8       SeSecurityPrivilege     Present,Enabled Manage auditing and security log
10224   MsSense.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
10224   MsSense.exe     10      SeLoadDriverPrivilege   Present,Enabled Load and unload device drivers
10224   MsSense.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
10224   MsSense.exe     12      SeSystemtimePrivilege   Present Change the system time
10224   MsSense.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
10224   MsSense.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
10224   MsSense.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
10224   MsSense.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
10224   MsSense.exe     17      SeBackupPrivilege       Present,Enabled Backup files and directories
10224   MsSense.exe     18      SeRestorePrivilege      Present,Enabled Restore files and directories
10224   MsSense.exe     19      SeShutdownPrivilege     Present Shut down the system
10224   MsSense.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
10224   MsSense.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
10224   MsSense.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
10224   MsSense.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10224   MsSense.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10224   MsSense.exe     25      SeUndockPrivilege       Present Remove computer from docking station
10224   MsSense.exe     26      SeSyncAgentPrivilege            Synch directory service data
10224   MsSense.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10224   MsSense.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
10224   MsSense.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
10224   MsSense.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
10224   MsSense.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10224   MsSense.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10224   MsSense.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
10224   MsSense.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
10224   MsSense.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
10224   MsSense.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
9948    SgrmBroker.exe  2       SeCreateTokenPrivilege          Create a token object
9948    SgrmBroker.exe  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9948    SgrmBroker.exe  4       SeLockMemoryPrivilege   Default Lock pages in memory
9948    SgrmBroker.exe  5       SeIncreaseQuotaPrivilege                Increase quotas
9948    SgrmBroker.exe  6       SeMachineAccountPrivilege               Add workstations to the domain
9948    SgrmBroker.exe  7       SeTcbPrivilege  Default Act as part of the operating system
9948    SgrmBroker.exe  8       SeSecurityPrivilege             Manage auditing and security log
9948    SgrmBroker.exe  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9948    SgrmBroker.exe  10      SeLoadDriverPrivilege           Load and unload device drivers
9948    SgrmBroker.exe  11      SeSystemProfilePrivilege        Default Profile system performance
9948    SgrmBroker.exe  12      SeSystemtimePrivilege           Change the system time
9948    SgrmBroker.exe  13      SeProfileSingleProcessPrivilege Default Profile a single process
9948    SgrmBroker.exe  14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
9948    SgrmBroker.exe  15      SeCreatePagefilePrivilege       Default Create a pagefile
9948    SgrmBroker.exe  16      SeCreatePermanentPrivilege      Default Create permanent shared objects
9948    SgrmBroker.exe  17      SeBackupPrivilege               Backup files and directories
9948    SgrmBroker.exe  18      SeRestorePrivilege              Restore files and directories
9948    SgrmBroker.exe  19      SeShutdownPrivilege             Shut down the system
9948    SgrmBroker.exe  20      SeDebugPrivilege        Default Debug programs
9948    SgrmBroker.exe  21      SeAuditPrivilege        Default Generate security audits
9948    SgrmBroker.exe  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9948    SgrmBroker.exe  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9948    SgrmBroker.exe  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9948    SgrmBroker.exe  25      SeUndockPrivilege               Remove computer from docking station
9948    SgrmBroker.exe  26      SeSyncAgentPrivilege            Synch directory service data
9948    SgrmBroker.exe  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9948    SgrmBroker.exe  28      SeManageVolumePrivilege         Manage the files on a volume
9948    SgrmBroker.exe  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
9948    SgrmBroker.exe  30      SeCreateGlobalPrivilege Default Create global objects
9948    SgrmBroker.exe  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9948    SgrmBroker.exe  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9948    SgrmBroker.exe  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
9948    SgrmBroker.exe  34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
9948    SgrmBroker.exe  35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
9948    SgrmBroker.exe  36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
6356    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
6356    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
6356    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
6356    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
6356    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
6356    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
6356    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
6356    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
6356    svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
6356    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
6356    svchost.exe     12      SeSystemtimePrivilege           Change the system time
6356    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
6356    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
6356    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
6356    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
6356    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
6356    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
6356    svchost.exe     19      SeShutdownPrivilege             Shut down the system
6356    svchost.exe     20      SeDebugPrivilege        Default Debug programs
6356    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
6356    svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
6356    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
6356    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
6356    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
6356    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
6356    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
6356    svchost.exe     28      SeManageVolumePrivilege Present,Enabled Manage the files on a volume
6356    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
6356    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
6356    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
6356    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
6356    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
6356    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
6356    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
6356    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3052    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3052    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
3052    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
3052    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3052    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3052    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
3052    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3052    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3052    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3052    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
3052    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3052    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
3052    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3052    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
3052    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
3052    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3052    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3052    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3052    svchost.exe     20      SeDebugPrivilege                Debug programs
3052    svchost.exe     21      SeAuditPrivilege        Present,Enabled Generate security audits
3052    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3052    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3052    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3052    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3052    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3052    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3052    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3052    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3052    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3052    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3052    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3052    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
3052    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
3052    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3052    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
8912    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
8912    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8912    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
8912    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
8912    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
8912    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
8912    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
8912    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8912    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
8912    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
8912    svchost.exe     12      SeSystemtimePrivilege           Change the system time
8912    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
8912    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
8912    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
8912    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
8912    svchost.exe     17      SeBackupPrivilege               Backup files and directories
8912    svchost.exe     18      SeRestorePrivilege              Restore files and directories
8912    svchost.exe     19      SeShutdownPrivilege             Shut down the system
8912    svchost.exe     20      SeDebugPrivilege                Debug programs
8912    svchost.exe     21      SeAuditPrivilege                Generate security audits
8912    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8912    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8912    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8912    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
8912    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
8912    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8912    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
8912    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
8912    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
8912    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8912    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8912    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
8912    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
8912    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
8912    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
4156    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
4156    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
4156    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
4156    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
4156    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
4156    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
4156    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
4156    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4156    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
4156    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
4156    svchost.exe     12      SeSystemtimePrivilege           Change the system time
4156    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
4156    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4156    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
4156    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
4156    svchost.exe     17      SeBackupPrivilege               Backup files and directories
4156    svchost.exe     18      SeRestorePrivilege              Restore files and directories
4156    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
4156    svchost.exe     20      SeDebugPrivilege                Debug programs
4156    svchost.exe     21      SeAuditPrivilege                Generate security audits
4156    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4156    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4156    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4156    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
4156    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
4156    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4156    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
4156    svchost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
4156    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
4156    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4156    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4156    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
4156    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
4156    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4156    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7220    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
7220    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
7220    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
7220    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
7220    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
7220    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
7220    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
7220    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7220    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
7220    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
7220    svchost.exe     12      SeSystemtimePrivilege           Change the system time
7220    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
7220    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
7220    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
7220    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
7220    svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
7220    svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
7220    svchost.exe     19      SeShutdownPrivilege             Shut down the system
7220    svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
7220    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
7220    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7220    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7220    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7220    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
7220    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
7220    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7220    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
7220    svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
7220    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
7220    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7220    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7220    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
7220    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
7220    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
7220    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
8772    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
8772    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8772    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
8772    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
8772    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
8772    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
8772    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
8772    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8772    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
8772    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
8772    svchost.exe     12      SeSystemtimePrivilege           Change the system time
8772    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
8772    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
8772    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
8772    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
8772    svchost.exe     17      SeBackupPrivilege               Backup files and directories
8772    svchost.exe     18      SeRestorePrivilege              Restore files and directories
8772    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
8772    svchost.exe     20      SeDebugPrivilege        Default Debug programs
8772    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
8772    svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
8772    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8772    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8772    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
8772    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
8772    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8772    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
8772    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
8772    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
8772    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8772    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8772    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
8772    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
8772    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
8772    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
3320    OneDrive.exe    2       SeCreateTokenPrivilege          Create a token object
3320    OneDrive.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3320    OneDrive.exe    4       SeLockMemoryPrivilege           Lock pages in memory
3320    OneDrive.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
3320    OneDrive.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
3320    OneDrive.exe    7       SeTcbPrivilege          Act as part of the operating system
3320    OneDrive.exe    8       SeSecurityPrivilege             Manage auditing and security log
3320    OneDrive.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3320    OneDrive.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
3320    OneDrive.exe    11      SeSystemProfilePrivilege                Profile system performance
3320    OneDrive.exe    12      SeSystemtimePrivilege           Change the system time
3320    OneDrive.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
3320    OneDrive.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3320    OneDrive.exe    15      SeCreatePagefilePrivilege               Create a pagefile
3320    OneDrive.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
3320    OneDrive.exe    17      SeBackupPrivilege               Backup files and directories
3320    OneDrive.exe    18      SeRestorePrivilege              Restore files and directories
3320    OneDrive.exe    19      SeShutdownPrivilege     Present Shut down the system
3320    OneDrive.exe    20      SeDebugPrivilege                Debug programs
3320    OneDrive.exe    21      SeAuditPrivilege                Generate security audits
3320    OneDrive.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3320    OneDrive.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3320    OneDrive.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3320    OneDrive.exe    25      SeUndockPrivilege       Present Remove computer from docking station
3320    OneDrive.exe    26      SeSyncAgentPrivilege            Synch directory service data
3320    OneDrive.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3320    OneDrive.exe    28      SeManageVolumePrivilege         Manage the files on a volume
3320    OneDrive.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
3320    OneDrive.exe    30      SeCreateGlobalPrivilege Default Create global objects
3320    OneDrive.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3320    OneDrive.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3320    OneDrive.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
3320    OneDrive.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
3320    OneDrive.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3320    OneDrive.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10020   SystemSettings  2       SeCreateTokenPrivilege          Create a token object
10020   SystemSettings  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10020   SystemSettings  4       SeLockMemoryPrivilege           Lock pages in memory
10020   SystemSettings  5       SeIncreaseQuotaPrivilege                Increase quotas
10020   SystemSettings  6       SeMachineAccountPrivilege               Add workstations to the domain
10020   SystemSettings  7       SeTcbPrivilege          Act as part of the operating system
10020   SystemSettings  8       SeSecurityPrivilege             Manage auditing and security log
10020   SystemSettings  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10020   SystemSettings  10      SeLoadDriverPrivilege           Load and unload device drivers
10020   SystemSettings  11      SeSystemProfilePrivilege                Profile system performance
10020   SystemSettings  12      SeSystemtimePrivilege           Change the system time
10020   SystemSettings  13      SeProfileSingleProcessPrivilege         Profile a single process
10020   SystemSettings  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
10020   SystemSettings  15      SeCreatePagefilePrivilege               Create a pagefile
10020   SystemSettings  16      SeCreatePermanentPrivilege              Create permanent shared objects
10020   SystemSettings  17      SeBackupPrivilege               Backup files and directories
10020   SystemSettings  18      SeRestorePrivilege              Restore files and directories
10020   SystemSettings  19      SeShutdownPrivilege     Present Shut down the system
10020   SystemSettings  20      SeDebugPrivilege                Debug programs
10020   SystemSettings  21      SeAuditPrivilege                Generate security audits
10020   SystemSettings  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
10020   SystemSettings  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10020   SystemSettings  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10020   SystemSettings  25      SeUndockPrivilege       Present Remove computer from docking station
10020   SystemSettings  26      SeSyncAgentPrivilege            Synch directory service data
10020   SystemSettings  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10020   SystemSettings  28      SeManageVolumePrivilege         Manage the files on a volume
10020   SystemSettings  29      SeImpersonatePrivilege          Impersonate a client after authentication
10020   SystemSettings  30      SeCreateGlobalPrivilege Default Create global objects
10020   SystemSettings  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10020   SystemSettings  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10020   SystemSettings  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
10020   SystemSettings  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
10020   SystemSettings  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
10020   SystemSettings  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
4836    ApplicationFra  2       SeCreateTokenPrivilege          Create a token object
4836    ApplicationFra  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
4836    ApplicationFra  4       SeLockMemoryPrivilege           Lock pages in memory
4836    ApplicationFra  5       SeIncreaseQuotaPrivilege                Increase quotas
4836    ApplicationFra  6       SeMachineAccountPrivilege               Add workstations to the domain
4836    ApplicationFra  7       SeTcbPrivilege          Act as part of the operating system
4836    ApplicationFra  8       SeSecurityPrivilege             Manage auditing and security log
4836    ApplicationFra  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4836    ApplicationFra  10      SeLoadDriverPrivilege           Load and unload device drivers
4836    ApplicationFra  11      SeSystemProfilePrivilege                Profile system performance
4836    ApplicationFra  12      SeSystemtimePrivilege           Change the system time
4836    ApplicationFra  13      SeProfileSingleProcessPrivilege         Profile a single process
4836    ApplicationFra  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4836    ApplicationFra  15      SeCreatePagefilePrivilege               Create a pagefile
4836    ApplicationFra  16      SeCreatePermanentPrivilege              Create permanent shared objects
4836    ApplicationFra  17      SeBackupPrivilege               Backup files and directories
4836    ApplicationFra  18      SeRestorePrivilege              Restore files and directories
4836    ApplicationFra  19      SeShutdownPrivilege     Present Shut down the system
4836    ApplicationFra  20      SeDebugPrivilege                Debug programs
4836    ApplicationFra  21      SeAuditPrivilege                Generate security audits
4836    ApplicationFra  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4836    ApplicationFra  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4836    ApplicationFra  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4836    ApplicationFra  25      SeUndockPrivilege       Present Remove computer from docking station
4836    ApplicationFra  26      SeSyncAgentPrivilege            Synch directory service data
4836    ApplicationFra  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4836    ApplicationFra  28      SeManageVolumePrivilege         Manage the files on a volume
4836    ApplicationFra  29      SeImpersonatePrivilege          Impersonate a client after authentication
4836    ApplicationFra  30      SeCreateGlobalPrivilege Default Create global objects
4836    ApplicationFra  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4836    ApplicationFra  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4836    ApplicationFra  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
4836    ApplicationFra  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
4836    ApplicationFra  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4836    ApplicationFra  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9620    UserOOBEBroker  2       SeCreateTokenPrivilege          Create a token object
9620    UserOOBEBroker  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9620    UserOOBEBroker  4       SeLockMemoryPrivilege           Lock pages in memory
9620    UserOOBEBroker  5       SeIncreaseQuotaPrivilege                Increase quotas
9620    UserOOBEBroker  6       SeMachineAccountPrivilege               Add workstations to the domain
9620    UserOOBEBroker  7       SeTcbPrivilege          Act as part of the operating system
9620    UserOOBEBroker  8       SeSecurityPrivilege             Manage auditing and security log
9620    UserOOBEBroker  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9620    UserOOBEBroker  10      SeLoadDriverPrivilege           Load and unload device drivers
9620    UserOOBEBroker  11      SeSystemProfilePrivilege                Profile system performance
9620    UserOOBEBroker  12      SeSystemtimePrivilege           Change the system time
9620    UserOOBEBroker  13      SeProfileSingleProcessPrivilege         Profile a single process
9620    UserOOBEBroker  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9620    UserOOBEBroker  15      SeCreatePagefilePrivilege               Create a pagefile
9620    UserOOBEBroker  16      SeCreatePermanentPrivilege              Create permanent shared objects
9620    UserOOBEBroker  17      SeBackupPrivilege               Backup files and directories
9620    UserOOBEBroker  18      SeRestorePrivilege              Restore files and directories
9620    UserOOBEBroker  19      SeShutdownPrivilege     Present Shut down the system
9620    UserOOBEBroker  20      SeDebugPrivilege                Debug programs
9620    UserOOBEBroker  21      SeAuditPrivilege                Generate security audits
9620    UserOOBEBroker  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9620    UserOOBEBroker  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9620    UserOOBEBroker  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9620    UserOOBEBroker  25      SeUndockPrivilege       Present Remove computer from docking station
9620    UserOOBEBroker  26      SeSyncAgentPrivilege            Synch directory service data
9620    UserOOBEBroker  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9620    UserOOBEBroker  28      SeManageVolumePrivilege         Manage the files on a volume
9620    UserOOBEBroker  29      SeImpersonatePrivilege          Impersonate a client after authentication
9620    UserOOBEBroker  30      SeCreateGlobalPrivilege Default Create global objects
9620    UserOOBEBroker  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9620    UserOOBEBroker  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9620    UserOOBEBroker  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
9620    UserOOBEBroker  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
9620    UserOOBEBroker  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9620    UserOOBEBroker  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
8592    FileCoAuth.exe  2       SeCreateTokenPrivilege          Create a token object
8592    FileCoAuth.exe  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8592    FileCoAuth.exe  4       SeLockMemoryPrivilege           Lock pages in memory
8592    FileCoAuth.exe  5       SeIncreaseQuotaPrivilege                Increase quotas
8592    FileCoAuth.exe  6       SeMachineAccountPrivilege               Add workstations to the domain
8592    FileCoAuth.exe  7       SeTcbPrivilege          Act as part of the operating system
8592    FileCoAuth.exe  8       SeSecurityPrivilege             Manage auditing and security log
8592    FileCoAuth.exe  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8592    FileCoAuth.exe  10      SeLoadDriverPrivilege           Load and unload device drivers
8592    FileCoAuth.exe  11      SeSystemProfilePrivilege                Profile system performance
8592    FileCoAuth.exe  12      SeSystemtimePrivilege           Change the system time
8592    FileCoAuth.exe  13      SeProfileSingleProcessPrivilege         Profile a single process
8592    FileCoAuth.exe  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
8592    FileCoAuth.exe  15      SeCreatePagefilePrivilege               Create a pagefile
8592    FileCoAuth.exe  16      SeCreatePermanentPrivilege              Create permanent shared objects
8592    FileCoAuth.exe  17      SeBackupPrivilege               Backup files and directories
8592    FileCoAuth.exe  18      SeRestorePrivilege              Restore files and directories
8592    FileCoAuth.exe  19      SeShutdownPrivilege     Present Shut down the system
8592    FileCoAuth.exe  20      SeDebugPrivilege                Debug programs
8592    FileCoAuth.exe  21      SeAuditPrivilege                Generate security audits
8592    FileCoAuth.exe  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8592    FileCoAuth.exe  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8592    FileCoAuth.exe  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8592    FileCoAuth.exe  25      SeUndockPrivilege       Present Remove computer from docking station
8592    FileCoAuth.exe  26      SeSyncAgentPrivilege            Synch directory service data
8592    FileCoAuth.exe  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8592    FileCoAuth.exe  28      SeManageVolumePrivilege         Manage the files on a volume
8592    FileCoAuth.exe  29      SeImpersonatePrivilege          Impersonate a client after authentication
8592    FileCoAuth.exe  30      SeCreateGlobalPrivilege Default Create global objects
8592    FileCoAuth.exe  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8592    FileCoAuth.exe  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8592    FileCoAuth.exe  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
8592    FileCoAuth.exe  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
8592    FileCoAuth.exe  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
8592    FileCoAuth.exe  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
8312    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
8312    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8312    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
8312    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
8312    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
8312    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
8312    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
8312    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8312    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
8312    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
8312    svchost.exe     12      SeSystemtimePrivilege           Change the system time
8312    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
8312    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
8312    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
8312    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
8312    svchost.exe     17      SeBackupPrivilege               Backup files and directories
8312    svchost.exe     18      SeRestorePrivilege              Restore files and directories
8312    svchost.exe     19      SeShutdownPrivilege             Shut down the system
8312    svchost.exe     20      SeDebugPrivilege                Debug programs
8312    svchost.exe     21      SeAuditPrivilege                Generate security audits
8312    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8312    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8312    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8312    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
8312    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
8312    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8312    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
8312    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
8312    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
8312    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8312    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8312    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
8312    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
8312    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
8312    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
4236    SearchApp.exe   2       SeCreateTokenPrivilege          Create a token object
4236    SearchApp.exe   3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
4236    SearchApp.exe   4       SeLockMemoryPrivilege           Lock pages in memory
4236    SearchApp.exe   5       SeIncreaseQuotaPrivilege                Increase quotas
4236    SearchApp.exe   6       SeMachineAccountPrivilege               Add workstations to the domain
4236    SearchApp.exe   7       SeTcbPrivilege          Act as part of the operating system
4236    SearchApp.exe   8       SeSecurityPrivilege             Manage auditing and security log
4236    SearchApp.exe   9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4236    SearchApp.exe   10      SeLoadDriverPrivilege           Load and unload device drivers
4236    SearchApp.exe   11      SeSystemProfilePrivilege                Profile system performance
4236    SearchApp.exe   12      SeSystemtimePrivilege           Change the system time
4236    SearchApp.exe   13      SeProfileSingleProcessPrivilege         Profile a single process
4236    SearchApp.exe   14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4236    SearchApp.exe   15      SeCreatePagefilePrivilege               Create a pagefile
4236    SearchApp.exe   16      SeCreatePermanentPrivilege              Create permanent shared objects
4236    SearchApp.exe   17      SeBackupPrivilege               Backup files and directories
4236    SearchApp.exe   18      SeRestorePrivilege              Restore files and directories
4236    SearchApp.exe   19      SeShutdownPrivilege             Shut down the system
4236    SearchApp.exe   20      SeDebugPrivilege                Debug programs
4236    SearchApp.exe   21      SeAuditPrivilege                Generate security audits
4236    SearchApp.exe   22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4236    SearchApp.exe   23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4236    SearchApp.exe   24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4236    SearchApp.exe   25      SeUndockPrivilege               Remove computer from docking station
4236    SearchApp.exe   26      SeSyncAgentPrivilege            Synch directory service data
4236    SearchApp.exe   27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4236    SearchApp.exe   28      SeManageVolumePrivilege         Manage the files on a volume
4236    SearchApp.exe   29      SeImpersonatePrivilege          Impersonate a client after authentication
4236    SearchApp.exe   30      SeCreateGlobalPrivilege         Create global objects
4236    SearchApp.exe   31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4236    SearchApp.exe   32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4236    SearchApp.exe   33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
4236    SearchApp.exe   34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
4236    SearchApp.exe   35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4236    SearchApp.exe   36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
704     ShellExperienc  2       SeCreateTokenPrivilege          Create a token object
704     ShellExperienc  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
704     ShellExperienc  4       SeLockMemoryPrivilege           Lock pages in memory
704     ShellExperienc  5       SeIncreaseQuotaPrivilege                Increase quotas
704     ShellExperienc  6       SeMachineAccountPrivilege               Add workstations to the domain
704     ShellExperienc  7       SeTcbPrivilege          Act as part of the operating system
704     ShellExperienc  8       SeSecurityPrivilege             Manage auditing and security log
704     ShellExperienc  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
704     ShellExperienc  10      SeLoadDriverPrivilege           Load and unload device drivers
704     ShellExperienc  11      SeSystemProfilePrivilege                Profile system performance
704     ShellExperienc  12      SeSystemtimePrivilege           Change the system time
704     ShellExperienc  13      SeProfileSingleProcessPrivilege         Profile a single process
704     ShellExperienc  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
704     ShellExperienc  15      SeCreatePagefilePrivilege               Create a pagefile
704     ShellExperienc  16      SeCreatePermanentPrivilege              Create permanent shared objects
704     ShellExperienc  17      SeBackupPrivilege               Backup files and directories
704     ShellExperienc  18      SeRestorePrivilege              Restore files and directories
704     ShellExperienc  19      SeShutdownPrivilege             Shut down the system
704     ShellExperienc  20      SeDebugPrivilege                Debug programs
704     ShellExperienc  21      SeAuditPrivilege                Generate security audits
704     ShellExperienc  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
704     ShellExperienc  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
704     ShellExperienc  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
704     ShellExperienc  25      SeUndockPrivilege               Remove computer from docking station
704     ShellExperienc  26      SeSyncAgentPrivilege            Synch directory service data
704     ShellExperienc  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
704     ShellExperienc  28      SeManageVolumePrivilege         Manage the files on a volume
704     ShellExperienc  29      SeImpersonatePrivilege          Impersonate a client after authentication
704     ShellExperienc  30      SeCreateGlobalPrivilege         Create global objects
704     ShellExperienc  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
704     ShellExperienc  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
704     ShellExperienc  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
704     ShellExperienc  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
704     ShellExperienc  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
704     ShellExperienc  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1832    RuntimeBroker.  2       SeCreateTokenPrivilege          Create a token object
1832    RuntimeBroker.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1832    RuntimeBroker.  4       SeLockMemoryPrivilege           Lock pages in memory
1832    RuntimeBroker.  5       SeIncreaseQuotaPrivilege                Increase quotas
1832    RuntimeBroker.  6       SeMachineAccountPrivilege               Add workstations to the domain
1832    RuntimeBroker.  7       SeTcbPrivilege          Act as part of the operating system
1832    RuntimeBroker.  8       SeSecurityPrivilege             Manage auditing and security log
1832    RuntimeBroker.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1832    RuntimeBroker.  10      SeLoadDriverPrivilege           Load and unload device drivers
1832    RuntimeBroker.  11      SeSystemProfilePrivilege                Profile system performance
1832    RuntimeBroker.  12      SeSystemtimePrivilege           Change the system time
1832    RuntimeBroker.  13      SeProfileSingleProcessPrivilege         Profile a single process
1832    RuntimeBroker.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1832    RuntimeBroker.  15      SeCreatePagefilePrivilege               Create a pagefile
1832    RuntimeBroker.  16      SeCreatePermanentPrivilege              Create permanent shared objects
1832    RuntimeBroker.  17      SeBackupPrivilege               Backup files and directories
1832    RuntimeBroker.  18      SeRestorePrivilege              Restore files and directories
1832    RuntimeBroker.  19      SeShutdownPrivilege     Present Shut down the system
1832    RuntimeBroker.  20      SeDebugPrivilege                Debug programs
1832    RuntimeBroker.  21      SeAuditPrivilege                Generate security audits
1832    RuntimeBroker.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1832    RuntimeBroker.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1832    RuntimeBroker.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1832    RuntimeBroker.  25      SeUndockPrivilege       Present Remove computer from docking station
1832    RuntimeBroker.  26      SeSyncAgentPrivilege            Synch directory service data
1832    RuntimeBroker.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1832    RuntimeBroker.  28      SeManageVolumePrivilege         Manage the files on a volume
1832    RuntimeBroker.  29      SeImpersonatePrivilege          Impersonate a client after authentication
1832    RuntimeBroker.  30      SeCreateGlobalPrivilege Default Create global objects
1832    RuntimeBroker.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1832    RuntimeBroker.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1832    RuntimeBroker.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
1832    RuntimeBroker.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
1832    RuntimeBroker.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1832    RuntimeBroker.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2492    dllhost.exe     2       SeCreateTokenPrivilege          Create a token object
2492    dllhost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2492    dllhost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2492    dllhost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
2492    dllhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2492    dllhost.exe     7       SeTcbPrivilege          Act as part of the operating system
2492    dllhost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2492    dllhost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2492    dllhost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2492    dllhost.exe     11      SeSystemProfilePrivilege                Profile system performance
2492    dllhost.exe     12      SeSystemtimePrivilege           Change the system time
2492    dllhost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2492    dllhost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2492    dllhost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2492    dllhost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2492    dllhost.exe     17      SeBackupPrivilege               Backup files and directories
2492    dllhost.exe     18      SeRestorePrivilege              Restore files and directories
2492    dllhost.exe     19      SeShutdownPrivilege     Present Shut down the system
2492    dllhost.exe     20      SeDebugPrivilege                Debug programs
2492    dllhost.exe     21      SeAuditPrivilege                Generate security audits
2492    dllhost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2492    dllhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2492    dllhost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2492    dllhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
2492    dllhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2492    dllhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2492    dllhost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2492    dllhost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
2492    dllhost.exe     30      SeCreateGlobalPrivilege Default Create global objects
2492    dllhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2492    dllhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2492    dllhost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
2492    dllhost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
2492    dllhost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2492    dllhost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3864    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3864    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3864    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3864    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3864    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3864    svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
3864    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3864    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3864    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3864    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
3864    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3864    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3864    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
3864    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3864    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
3864    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3864    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3864    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3864    svchost.exe     20      SeDebugPrivilege        Default Debug programs
3864    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
3864    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3864    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3864    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3864    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3864    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3864    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3864    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3864    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3864    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
3864    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3864    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3864    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3864    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3864    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3864    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
1552    SenseTVM.exe    2       SeCreateTokenPrivilege          Create a token object
1552    SenseTVM.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1552    SenseTVM.exe    4       SeLockMemoryPrivilege           Lock pages in memory
1552    SenseTVM.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
1552    SenseTVM.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
1552    SenseTVM.exe    7       SeTcbPrivilege          Act as part of the operating system
1552    SenseTVM.exe    8       SeSecurityPrivilege             Manage auditing and security log
1552    SenseTVM.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1552    SenseTVM.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
1552    SenseTVM.exe    11      SeSystemProfilePrivilege                Profile system performance
1552    SenseTVM.exe    12      SeSystemtimePrivilege           Change the system time
1552    SenseTVM.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
1552    SenseTVM.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1552    SenseTVM.exe    15      SeCreatePagefilePrivilege               Create a pagefile
1552    SenseTVM.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
1552    SenseTVM.exe    17      SeBackupPrivilege               Backup files and directories
1552    SenseTVM.exe    18      SeRestorePrivilege              Restore files and directories
1552    SenseTVM.exe    19      SeShutdownPrivilege             Shut down the system
1552    SenseTVM.exe    20      SeDebugPrivilege                Debug programs
1552    SenseTVM.exe    21      SeAuditPrivilege                Generate security audits
1552    SenseTVM.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1552    SenseTVM.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1552    SenseTVM.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1552    SenseTVM.exe    25      SeUndockPrivilege               Remove computer from docking station
1552    SenseTVM.exe    26      SeSyncAgentPrivilege            Synch directory service data
1552    SenseTVM.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1552    SenseTVM.exe    28      SeManageVolumePrivilege         Manage the files on a volume
1552    SenseTVM.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
1552    SenseTVM.exe    30      SeCreateGlobalPrivilege         Create global objects
1552    SenseTVM.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1552    SenseTVM.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1552    SenseTVM.exe    33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
1552    SenseTVM.exe    34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
1552    SenseTVM.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1552    SenseTVM.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7800    dllhost.exe     2       SeCreateTokenPrivilege          Create a token object
7800    dllhost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7800    dllhost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
7800    dllhost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
7800    dllhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
7800    dllhost.exe     7       SeTcbPrivilege          Act as part of the operating system
7800    dllhost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
7800    dllhost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
7800    dllhost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
7800    dllhost.exe     11      SeSystemProfilePrivilege        Present Profile system performance
7800    dllhost.exe     12      SeSystemtimePrivilege   Present Change the system time
7800    dllhost.exe     13      SeProfileSingleProcessPrivilege Present Profile a single process
7800    dllhost.exe     14      SeIncreaseBasePriorityPrivilege Present Increase scheduling priority
7800    dllhost.exe     15      SeCreatePagefilePrivilege       Present Create a pagefile
7800    dllhost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
7800    dllhost.exe     17      SeBackupPrivilege       Present Backup files and directories
7800    dllhost.exe     18      SeRestorePrivilege      Present Restore files and directories
7800    dllhost.exe     19      SeShutdownPrivilege     Present Shut down the system
7800    dllhost.exe     20      SeDebugPrivilege        Present Debug programs
7800    dllhost.exe     21      SeAuditPrivilege                Generate security audits
7800    dllhost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
7800    dllhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7800    dllhost.exe     24      SeRemoteShutdownPrivilege       Present Force shutdown from a remote system
7800    dllhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
7800    dllhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
7800    dllhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7800    dllhost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
7800    dllhost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
7800    dllhost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
7800    dllhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7800    dllhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7800    dllhost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7800    dllhost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7800    dllhost.exe     35      SeCreateSymbolicLinkPrivilege   Present Required to create a symbolic link
7800    dllhost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present Obtain an impersonation token for another user in the same session.
8260    chrome.exe      2       SeCreateTokenPrivilege          Create a token object
8260    chrome.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8260    chrome.exe      4       SeLockMemoryPrivilege           Lock pages in memory
8260    chrome.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
8260    chrome.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
8260    chrome.exe      7       SeTcbPrivilege          Act as part of the operating system
8260    chrome.exe      8       SeSecurityPrivilege             Manage auditing and security log
8260    chrome.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8260    chrome.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
8260    chrome.exe      11      SeSystemProfilePrivilege                Profile system performance
8260    chrome.exe      12      SeSystemtimePrivilege           Change the system time
8260    chrome.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
8260    chrome.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
8260    chrome.exe      15      SeCreatePagefilePrivilege               Create a pagefile
8260    chrome.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
8260    chrome.exe      17      SeBackupPrivilege               Backup files and directories
8260    chrome.exe      18      SeRestorePrivilege              Restore files and directories
8260    chrome.exe      19      SeShutdownPrivilege             Shut down the system
8260    chrome.exe      20      SeDebugPrivilege                Debug programs
8260    chrome.exe      21      SeAuditPrivilege                Generate security audits
8260    chrome.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8260    chrome.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
8260    chrome.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8260    chrome.exe      25      SeUndockPrivilege               Remove computer from docking station
8260    chrome.exe      26      SeSyncAgentPrivilege            Synch directory service data
8260    chrome.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8260    chrome.exe      28      SeManageVolumePrivilege         Manage the files on a volume
8260    chrome.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
8260    chrome.exe      30      SeCreateGlobalPrivilege         Create global objects
8260    chrome.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8260    chrome.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8260    chrome.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
8260    chrome.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
8260    chrome.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
8260    chrome.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7868    chrome.exe      2       SeCreateTokenPrivilege          Create a token object
7868    chrome.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7868    chrome.exe      4       SeLockMemoryPrivilege           Lock pages in memory
7868    chrome.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
7868    chrome.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
7868    chrome.exe      7       SeTcbPrivilege          Act as part of the operating system
7868    chrome.exe      8       SeSecurityPrivilege             Manage auditing and security log
7868    chrome.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7868    chrome.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
7868    chrome.exe      11      SeSystemProfilePrivilege                Profile system performance
7868    chrome.exe      12      SeSystemtimePrivilege           Change the system time
7868    chrome.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
7868    chrome.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7868    chrome.exe      15      SeCreatePagefilePrivilege               Create a pagefile
7868    chrome.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
7868    chrome.exe      17      SeBackupPrivilege               Backup files and directories
7868    chrome.exe      18      SeRestorePrivilege              Restore files and directories
7868    chrome.exe      19      SeShutdownPrivilege             Shut down the system
7868    chrome.exe      20      SeDebugPrivilege                Debug programs
7868    chrome.exe      21      SeAuditPrivilege                Generate security audits
7868    chrome.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7868    chrome.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
7868    chrome.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7868    chrome.exe      25      SeUndockPrivilege               Remove computer from docking station
7868    chrome.exe      26      SeSyncAgentPrivilege            Synch directory service data
7868    chrome.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7868    chrome.exe      28      SeManageVolumePrivilege         Manage the files on a volume
7868    chrome.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
7868    chrome.exe      30      SeCreateGlobalPrivilege         Create global objects
7868    chrome.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7868    chrome.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7868    chrome.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
7868    chrome.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
7868    chrome.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7868    chrome.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3240    chrome.exe      2       SeCreateTokenPrivilege          Create a token object
3240    chrome.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3240    chrome.exe      4       SeLockMemoryPrivilege           Lock pages in memory
3240    chrome.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
3240    chrome.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
3240    chrome.exe      7       SeTcbPrivilege          Act as part of the operating system
3240    chrome.exe      8       SeSecurityPrivilege             Manage auditing and security log
3240    chrome.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3240    chrome.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
3240    chrome.exe      11      SeSystemProfilePrivilege                Profile system performance
3240    chrome.exe      12      SeSystemtimePrivilege           Change the system time
3240    chrome.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
3240    chrome.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
3240    chrome.exe      15      SeCreatePagefilePrivilege               Create a pagefile
3240    chrome.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
3240    chrome.exe      17      SeBackupPrivilege               Backup files and directories
3240    chrome.exe      18      SeRestorePrivilege              Restore files and directories
3240    chrome.exe      19      SeShutdownPrivilege             Shut down the system
3240    chrome.exe      20      SeDebugPrivilege                Debug programs
3240    chrome.exe      21      SeAuditPrivilege                Generate security audits
3240    chrome.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3240    chrome.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
3240    chrome.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3240    chrome.exe      25      SeUndockPrivilege               Remove computer from docking station
3240    chrome.exe      26      SeSyncAgentPrivilege            Synch directory service data
3240    chrome.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3240    chrome.exe      28      SeManageVolumePrivilege         Manage the files on a volume
3240    chrome.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
3240    chrome.exe      30      SeCreateGlobalPrivilege         Create global objects
3240    chrome.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3240    chrome.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3240    chrome.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
3240    chrome.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
3240    chrome.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
3240    chrome.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2516    chrome.exe      2       SeCreateTokenPrivilege          Create a token object
2516    chrome.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
2516    chrome.exe      4       SeLockMemoryPrivilege           Lock pages in memory
2516    chrome.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
2516    chrome.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
2516    chrome.exe      7       SeTcbPrivilege          Act as part of the operating system
2516    chrome.exe      8       SeSecurityPrivilege             Manage auditing and security log
2516    chrome.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2516    chrome.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
2516    chrome.exe      11      SeSystemProfilePrivilege                Profile system performance
2516    chrome.exe      12      SeSystemtimePrivilege           Change the system time
2516    chrome.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
2516    chrome.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2516    chrome.exe      15      SeCreatePagefilePrivilege               Create a pagefile
2516    chrome.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
2516    chrome.exe      17      SeBackupPrivilege               Backup files and directories
2516    chrome.exe      18      SeRestorePrivilege              Restore files and directories
2516    chrome.exe      19      SeShutdownPrivilege             Shut down the system
2516    chrome.exe      20      SeDebugPrivilege                Debug programs
2516    chrome.exe      21      SeAuditPrivilege                Generate security audits
2516    chrome.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2516    chrome.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
2516    chrome.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2516    chrome.exe      25      SeUndockPrivilege               Remove computer from docking station
2516    chrome.exe      26      SeSyncAgentPrivilege            Synch directory service data
2516    chrome.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2516    chrome.exe      28      SeManageVolumePrivilege         Manage the files on a volume
2516    chrome.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
2516    chrome.exe      30      SeCreateGlobalPrivilege         Create global objects
2516    chrome.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2516    chrome.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2516    chrome.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
2516    chrome.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
2516    chrome.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2516    chrome.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
8792    chrome.exe      2       SeCreateTokenPrivilege          Create a token object
8792    chrome.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8792    chrome.exe      4       SeLockMemoryPrivilege           Lock pages in memory
8792    chrome.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
8792    chrome.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
8792    chrome.exe      7       SeTcbPrivilege          Act as part of the operating system
8792    chrome.exe      8       SeSecurityPrivilege             Manage auditing and security log
8792    chrome.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8792    chrome.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
8792    chrome.exe      11      SeSystemProfilePrivilege                Profile system performance
8792    chrome.exe      12      SeSystemtimePrivilege           Change the system time
8792    chrome.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
8792    chrome.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
8792    chrome.exe      15      SeCreatePagefilePrivilege               Create a pagefile
8792    chrome.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
8792    chrome.exe      17      SeBackupPrivilege               Backup files and directories
8792    chrome.exe      18      SeRestorePrivilege              Restore files and directories
8792    chrome.exe      19      SeShutdownPrivilege             Shut down the system
8792    chrome.exe      20      SeDebugPrivilege                Debug programs
8792    chrome.exe      21      SeAuditPrivilege                Generate security audits
8792    chrome.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8792    chrome.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
8792    chrome.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8792    chrome.exe      25      SeUndockPrivilege               Remove computer from docking station
8792    chrome.exe      26      SeSyncAgentPrivilege            Synch directory service data
8792    chrome.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8792    chrome.exe      28      SeManageVolumePrivilege         Manage the files on a volume
8792    chrome.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
8792    chrome.exe      30      SeCreateGlobalPrivilege         Create global objects
8792    chrome.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8792    chrome.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8792    chrome.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
8792    chrome.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
8792    chrome.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
8792    chrome.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
6056    chrome.exe      2       SeCreateTokenPrivilege          Create a token object
6056    chrome.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
6056    chrome.exe      4       SeLockMemoryPrivilege           Lock pages in memory
6056    chrome.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
6056    chrome.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
6056    chrome.exe      7       SeTcbPrivilege          Act as part of the operating system
6056    chrome.exe      8       SeSecurityPrivilege             Manage auditing and security log
6056    chrome.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
6056    chrome.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
6056    chrome.exe      11      SeSystemProfilePrivilege                Profile system performance
6056    chrome.exe      12      SeSystemtimePrivilege           Change the system time
6056    chrome.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
6056    chrome.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
6056    chrome.exe      15      SeCreatePagefilePrivilege               Create a pagefile
6056    chrome.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
6056    chrome.exe      17      SeBackupPrivilege               Backup files and directories
6056    chrome.exe      18      SeRestorePrivilege              Restore files and directories
6056    chrome.exe      19      SeShutdownPrivilege             Shut down the system
6056    chrome.exe      20      SeDebugPrivilege                Debug programs
6056    chrome.exe      21      SeAuditPrivilege                Generate security audits
6056    chrome.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
6056    chrome.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
6056    chrome.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
6056    chrome.exe      25      SeUndockPrivilege               Remove computer from docking station
6056    chrome.exe      26      SeSyncAgentPrivilege            Synch directory service data
6056    chrome.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
6056    chrome.exe      28      SeManageVolumePrivilege         Manage the files on a volume
6056    chrome.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
6056    chrome.exe      30      SeCreateGlobalPrivilege         Create global objects
6056    chrome.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
6056    chrome.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
6056    chrome.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
6056    chrome.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
6056    chrome.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
6056    chrome.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
4720    chrome.exe      2       SeCreateTokenPrivilege          Create a token object
4720    chrome.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
4720    chrome.exe      4       SeLockMemoryPrivilege           Lock pages in memory
4720    chrome.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
4720    chrome.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
4720    chrome.exe      7       SeTcbPrivilege          Act as part of the operating system
4720    chrome.exe      8       SeSecurityPrivilege             Manage auditing and security log
4720    chrome.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4720    chrome.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
4720    chrome.exe      11      SeSystemProfilePrivilege                Profile system performance
4720    chrome.exe      12      SeSystemtimePrivilege           Change the system time
4720    chrome.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
4720    chrome.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4720    chrome.exe      15      SeCreatePagefilePrivilege               Create a pagefile
4720    chrome.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
4720    chrome.exe      17      SeBackupPrivilege               Backup files and directories
4720    chrome.exe      18      SeRestorePrivilege              Restore files and directories
4720    chrome.exe      19      SeShutdownPrivilege             Shut down the system
4720    chrome.exe      20      SeDebugPrivilege                Debug programs
4720    chrome.exe      21      SeAuditPrivilege                Generate security audits
4720    chrome.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4720    chrome.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
4720    chrome.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4720    chrome.exe      25      SeUndockPrivilege               Remove computer from docking station
4720    chrome.exe      26      SeSyncAgentPrivilege            Synch directory service data
4720    chrome.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4720    chrome.exe      28      SeManageVolumePrivilege         Manage the files on a volume
4720    chrome.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
4720    chrome.exe      30      SeCreateGlobalPrivilege         Create global objects
4720    chrome.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4720    chrome.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4720    chrome.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
4720    chrome.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
4720    chrome.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4720    chrome.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10680   LogonUI.exe     2       SeCreateTokenPrivilege          Create a token object
10680   LogonUI.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10680   LogonUI.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
10680   LogonUI.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
10680   LogonUI.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
10680   LogonUI.exe     7       SeTcbPrivilege  Default Act as part of the operating system
10680   LogonUI.exe     8       SeSecurityPrivilege             Manage auditing and security log
10680   LogonUI.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10680   LogonUI.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
10680   LogonUI.exe     11      SeSystemProfilePrivilege        Default Profile system performance
10680   LogonUI.exe     12      SeSystemtimePrivilege           Change the system time
10680   LogonUI.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
10680   LogonUI.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
10680   LogonUI.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
10680   LogonUI.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
10680   LogonUI.exe     17      SeBackupPrivilege               Backup files and directories
10680   LogonUI.exe     18      SeRestorePrivilege              Restore files and directories
10680   LogonUI.exe     19      SeShutdownPrivilege     Present Shut down the system
10680   LogonUI.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
10680   LogonUI.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
10680   LogonUI.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
10680   LogonUI.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10680   LogonUI.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10680   LogonUI.exe     25      SeUndockPrivilege               Remove computer from docking station
10680   LogonUI.exe     26      SeSyncAgentPrivilege            Synch directory service data
10680   LogonUI.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10680   LogonUI.exe     28      SeManageVolumePrivilege         Manage the files on a volume
10680   LogonUI.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
10680   LogonUI.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
10680   LogonUI.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10680   LogonUI.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10680   LogonUI.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
10680   LogonUI.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
10680   LogonUI.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
10680   LogonUI.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
10732   csrss.exe       2       SeCreateTokenPrivilege  Present Create a token object
10732   csrss.exe       3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
10732   csrss.exe       4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
10732   csrss.exe       5       SeIncreaseQuotaPrivilege        Present Increase quotas
10732   csrss.exe       6       SeMachineAccountPrivilege               Add workstations to the domain
10732   csrss.exe       7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
10732   csrss.exe       8       SeSecurityPrivilege     Present Manage auditing and security log
10732   csrss.exe       9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
10732   csrss.exe       10      SeLoadDriverPrivilege   Present Load and unload device drivers
10732   csrss.exe       11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
10732   csrss.exe       12      SeSystemtimePrivilege   Present Change the system time
10732   csrss.exe       13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
10732   csrss.exe       14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
10732   csrss.exe       15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
10732   csrss.exe       16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
10732   csrss.exe       17      SeBackupPrivilege       Present Backup files and directories
10732   csrss.exe       18      SeRestorePrivilege      Present Restore files and directories
10732   csrss.exe       19      SeShutdownPrivilege     Present Shut down the system
10732   csrss.exe       20      SeDebugPrivilege        Present,Enabled,Default Debug programs
10732   csrss.exe       21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
10732   csrss.exe       22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
10732   csrss.exe       23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10732   csrss.exe       24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10732   csrss.exe       25      SeUndockPrivilege       Present Remove computer from docking station
10732   csrss.exe       26      SeSyncAgentPrivilege            Synch directory service data
10732   csrss.exe       27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10732   csrss.exe       28      SeManageVolumePrivilege Present Manage the files on a volume
10732   csrss.exe       29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
10732   csrss.exe       30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
10732   csrss.exe       31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10732   csrss.exe       32      SeRelabelPrivilege      Present Modify the mandatory integrity level of an object
10732   csrss.exe       33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
10732   csrss.exe       34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
10732   csrss.exe       35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
10732   csrss.exe       36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
2804    winlogon.exe    2       SeCreateTokenPrivilege          Create a token object
2804    winlogon.exe    3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
2804    winlogon.exe    4       SeLockMemoryPrivilege   Default Lock pages in memory
2804    winlogon.exe    5       SeIncreaseQuotaPrivilege        Present Increase quotas
2804    winlogon.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
2804    winlogon.exe    7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
2804    winlogon.exe    8       SeSecurityPrivilege     Present Manage auditing and security log
2804    winlogon.exe    9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
2804    winlogon.exe    10      SeLoadDriverPrivilege   Present Load and unload device drivers
2804    winlogon.exe    11      SeSystemProfilePrivilege        Default Profile system performance
2804    winlogon.exe    12      SeSystemtimePrivilege           Change the system time
2804    winlogon.exe    13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
2804    winlogon.exe    14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
2804    winlogon.exe    15      SeCreatePagefilePrivilege       Default Create a pagefile
2804    winlogon.exe    16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
2804    winlogon.exe    17      SeBackupPrivilege       Present Backup files and directories
2804    winlogon.exe    18      SeRestorePrivilege      Present Restore files and directories
2804    winlogon.exe    19      SeShutdownPrivilege     Present Shut down the system
2804    winlogon.exe    20      SeDebugPrivilege        Present,Enabled,Default Debug programs
2804    winlogon.exe    21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
2804    winlogon.exe    22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
2804    winlogon.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2804    winlogon.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2804    winlogon.exe    25      SeUndockPrivilege       Present Remove computer from docking station
2804    winlogon.exe    26      SeSyncAgentPrivilege            Synch directory service data
2804    winlogon.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2804    winlogon.exe    28      SeManageVolumePrivilege Present Manage the files on a volume
2804    winlogon.exe    29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2804    winlogon.exe    30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
2804    winlogon.exe    31      SeTrustedCredManAccessPrivilege Present Access Credential Manager as a trusted caller
2804    winlogon.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2804    winlogon.exe    33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
2804    winlogon.exe    34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
2804    winlogon.exe    35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
2804    winlogon.exe    36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
10628   dwm.exe 2       SeCreateTokenPrivilege          Create a token object
10628   dwm.exe 3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10628   dwm.exe 4       SeLockMemoryPrivilege           Lock pages in memory
10628   dwm.exe 5       SeIncreaseQuotaPrivilege                Increase quotas
10628   dwm.exe 6       SeMachineAccountPrivilege               Add workstations to the domain
10628   dwm.exe 7       SeTcbPrivilege          Act as part of the operating system
10628   dwm.exe 8       SeSecurityPrivilege             Manage auditing and security log
10628   dwm.exe 9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10628   dwm.exe 10      SeLoadDriverPrivilege           Load and unload device drivers
10628   dwm.exe 11      SeSystemProfilePrivilege                Profile system performance
10628   dwm.exe 12      SeSystemtimePrivilege           Change the system time
10628   dwm.exe 13      SeProfileSingleProcessPrivilege         Profile a single process
10628   dwm.exe 14      SeIncreaseBasePriorityPrivilege Present,Enabled Increase scheduling priority
10628   dwm.exe 15      SeCreatePagefilePrivilege               Create a pagefile
10628   dwm.exe 16      SeCreatePermanentPrivilege              Create permanent shared objects
10628   dwm.exe 17      SeBackupPrivilege               Backup files and directories
10628   dwm.exe 18      SeRestorePrivilege              Restore files and directories
10628   dwm.exe 19      SeShutdownPrivilege             Shut down the system
10628   dwm.exe 20      SeDebugPrivilege                Debug programs
10628   dwm.exe 21      SeAuditPrivilege                Generate security audits
10628   dwm.exe 22      SeSystemEnvironmentPrivilege            Edit firmware environment values
10628   dwm.exe 23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10628   dwm.exe 24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10628   dwm.exe 25      SeUndockPrivilege               Remove computer from docking station
10628   dwm.exe 26      SeSyncAgentPrivilege            Synch directory service data
10628   dwm.exe 27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10628   dwm.exe 28      SeManageVolumePrivilege         Manage the files on a volume
10628   dwm.exe 29      SeImpersonatePrivilege          Impersonate a client after authentication
10628   dwm.exe 30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
10628   dwm.exe 31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10628   dwm.exe 32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10628   dwm.exe 33      SeIncreaseWorkingSetPrivilege   Present,Enabled Allocate more memory for user applications
10628   dwm.exe 34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
10628   dwm.exe 35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
10628   dwm.exe 36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10704   fontdrvhost.ex  2       SeCreateTokenPrivilege          Create a token object
10704   fontdrvhost.ex  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10704   fontdrvhost.ex  4       SeLockMemoryPrivilege           Lock pages in memory
10704   fontdrvhost.ex  5       SeIncreaseQuotaPrivilege                Increase quotas
10704   fontdrvhost.ex  6       SeMachineAccountPrivilege               Add workstations to the domain
10704   fontdrvhost.ex  7       SeTcbPrivilege          Act as part of the operating system
10704   fontdrvhost.ex  8       SeSecurityPrivilege             Manage auditing and security log
10704   fontdrvhost.ex  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10704   fontdrvhost.ex  10      SeLoadDriverPrivilege           Load and unload device drivers
10704   fontdrvhost.ex  11      SeSystemProfilePrivilege                Profile system performance
10704   fontdrvhost.ex  12      SeSystemtimePrivilege           Change the system time
10704   fontdrvhost.ex  13      SeProfileSingleProcessPrivilege         Profile a single process
10704   fontdrvhost.ex  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
10704   fontdrvhost.ex  15      SeCreatePagefilePrivilege               Create a pagefile
10704   fontdrvhost.ex  16      SeCreatePermanentPrivilege              Create permanent shared objects
10704   fontdrvhost.ex  17      SeBackupPrivilege               Backup files and directories
10704   fontdrvhost.ex  18      SeRestorePrivilege              Restore files and directories
10704   fontdrvhost.ex  19      SeShutdownPrivilege             Shut down the system
10704   fontdrvhost.ex  20      SeDebugPrivilege                Debug programs
10704   fontdrvhost.ex  21      SeAuditPrivilege                Generate security audits
10704   fontdrvhost.ex  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
10704   fontdrvhost.ex  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10704   fontdrvhost.ex  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10704   fontdrvhost.ex  25      SeUndockPrivilege               Remove computer from docking station
10704   fontdrvhost.ex  26      SeSyncAgentPrivilege            Synch directory service data
10704   fontdrvhost.ex  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10704   fontdrvhost.ex  28      SeManageVolumePrivilege         Manage the files on a volume
10704   fontdrvhost.ex  29      SeImpersonatePrivilege          Impersonate a client after authentication
10704   fontdrvhost.ex  30      SeCreateGlobalPrivilege         Create global objects
10704   fontdrvhost.ex  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10704   fontdrvhost.ex  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10704   fontdrvhost.ex  33      SeIncreaseWorkingSetPrivilege   Present,Enabled Allocate more memory for user applications
10704   fontdrvhost.ex  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
10704   fontdrvhost.ex  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
10704   fontdrvhost.ex  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10384   sihost.exe      2       SeCreateTokenPrivilege          Create a token object
10384   sihost.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10384   sihost.exe      4       SeLockMemoryPrivilege           Lock pages in memory
10384   sihost.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
10384   sihost.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
10384   sihost.exe      7       SeTcbPrivilege          Act as part of the operating system
10384   sihost.exe      8       SeSecurityPrivilege             Manage auditing and security log
10384   sihost.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10384   sihost.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
10384   sihost.exe      11      SeSystemProfilePrivilege                Profile system performance
10384   sihost.exe      12      SeSystemtimePrivilege           Change the system time
10384   sihost.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
10384   sihost.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
10384   sihost.exe      15      SeCreatePagefilePrivilege               Create a pagefile
10384   sihost.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
10384   sihost.exe      17      SeBackupPrivilege               Backup files and directories
10384   sihost.exe      18      SeRestorePrivilege              Restore files and directories
10384   sihost.exe      19      SeShutdownPrivilege     Present Shut down the system
10384   sihost.exe      20      SeDebugPrivilege                Debug programs
10384   sihost.exe      21      SeAuditPrivilege                Generate security audits
10384   sihost.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
10384   sihost.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10384   sihost.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10384   sihost.exe      25      SeUndockPrivilege       Present Remove computer from docking station
10384   sihost.exe      26      SeSyncAgentPrivilege            Synch directory service data
10384   sihost.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10384   sihost.exe      28      SeManageVolumePrivilege         Manage the files on a volume
10384   sihost.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
10384   sihost.exe      30      SeCreateGlobalPrivilege Default Create global objects
10384   sihost.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10384   sihost.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10384   sihost.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
10384   sihost.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
10384   sihost.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
10384   sihost.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9320    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
9320    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9320    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
9320    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
9320    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
9320    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
9320    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
9320    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9320    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
9320    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
9320    svchost.exe     12      SeSystemtimePrivilege           Change the system time
9320    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
9320    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9320    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
9320    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
9320    svchost.exe     17      SeBackupPrivilege               Backup files and directories
9320    svchost.exe     18      SeRestorePrivilege              Restore files and directories
9320    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
9320    svchost.exe     20      SeDebugPrivilege                Debug programs
9320    svchost.exe     21      SeAuditPrivilege                Generate security audits
9320    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9320    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9320    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9320    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
9320    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
9320    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9320    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
9320    svchost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
9320    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
9320    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9320    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9320    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
9320    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
9320    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9320    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9452    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
9452    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9452    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
9452    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
9452    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
9452    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
9452    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
9452    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9452    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
9452    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
9452    svchost.exe     12      SeSystemtimePrivilege           Change the system time
9452    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
9452    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9452    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
9452    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
9452    svchost.exe     17      SeBackupPrivilege               Backup files and directories
9452    svchost.exe     18      SeRestorePrivilege              Restore files and directories
9452    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
9452    svchost.exe     20      SeDebugPrivilege                Debug programs
9452    svchost.exe     21      SeAuditPrivilege                Generate security audits
9452    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9452    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9452    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9452    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
9452    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
9452    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9452    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
9452    svchost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
9452    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
9452    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9452    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9452    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
9452    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
9452    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9452    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10476   taskhostw.exe   2       SeCreateTokenPrivilege          Create a token object
10476   taskhostw.exe   3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10476   taskhostw.exe   4       SeLockMemoryPrivilege           Lock pages in memory
10476   taskhostw.exe   5       SeIncreaseQuotaPrivilege                Increase quotas
10476   taskhostw.exe   6       SeMachineAccountPrivilege               Add workstations to the domain
10476   taskhostw.exe   7       SeTcbPrivilege          Act as part of the operating system
10476   taskhostw.exe   8       SeSecurityPrivilege             Manage auditing and security log
10476   taskhostw.exe   9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10476   taskhostw.exe   10      SeLoadDriverPrivilege           Load and unload device drivers
10476   taskhostw.exe   11      SeSystemProfilePrivilege                Profile system performance
10476   taskhostw.exe   12      SeSystemtimePrivilege           Change the system time
10476   taskhostw.exe   13      SeProfileSingleProcessPrivilege         Profile a single process
10476   taskhostw.exe   14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
10476   taskhostw.exe   15      SeCreatePagefilePrivilege               Create a pagefile
10476   taskhostw.exe   16      SeCreatePermanentPrivilege              Create permanent shared objects
10476   taskhostw.exe   17      SeBackupPrivilege               Backup files and directories
10476   taskhostw.exe   18      SeRestorePrivilege              Restore files and directories
10476   taskhostw.exe   19      SeShutdownPrivilege     Present Shut down the system
10476   taskhostw.exe   20      SeDebugPrivilege                Debug programs
10476   taskhostw.exe   21      SeAuditPrivilege                Generate security audits
10476   taskhostw.exe   22      SeSystemEnvironmentPrivilege            Edit firmware environment values
10476   taskhostw.exe   23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10476   taskhostw.exe   24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10476   taskhostw.exe   25      SeUndockPrivilege       Present Remove computer from docking station
10476   taskhostw.exe   26      SeSyncAgentPrivilege            Synch directory service data
10476   taskhostw.exe   27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10476   taskhostw.exe   28      SeManageVolumePrivilege         Manage the files on a volume
10476   taskhostw.exe   29      SeImpersonatePrivilege          Impersonate a client after authentication
10476   taskhostw.exe   30      SeCreateGlobalPrivilege Default Create global objects
10476   taskhostw.exe   31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10476   taskhostw.exe   32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10476   taskhostw.exe   33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
10476   taskhostw.exe   34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
10476   taskhostw.exe   35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
10476   taskhostw.exe   36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9940    userinit.exe    2       SeCreateTokenPrivilege          Create a token object
9940    userinit.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9940    userinit.exe    4       SeLockMemoryPrivilege           Lock pages in memory
9940    userinit.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
9940    userinit.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
9940    userinit.exe    7       SeTcbPrivilege          Act as part of the operating system
9940    userinit.exe    8       SeSecurityPrivilege             Manage auditing and security log
9940    userinit.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9940    userinit.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
9940    userinit.exe    11      SeSystemProfilePrivilege                Profile system performance
9940    userinit.exe    12      SeSystemtimePrivilege           Change the system time
9940    userinit.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
9940    userinit.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9940    userinit.exe    15      SeCreatePagefilePrivilege               Create a pagefile
9940    userinit.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
9940    userinit.exe    17      SeBackupPrivilege               Backup files and directories
9940    userinit.exe    18      SeRestorePrivilege              Restore files and directories
9940    userinit.exe    19      SeShutdownPrivilege     Present Shut down the system
9940    userinit.exe    20      SeDebugPrivilege                Debug programs
9940    userinit.exe    21      SeAuditPrivilege                Generate security audits
9940    userinit.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9940    userinit.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9940    userinit.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9940    userinit.exe    25      SeUndockPrivilege       Present Remove computer from docking station
9940    userinit.exe    26      SeSyncAgentPrivilege            Synch directory service data
9940    userinit.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9940    userinit.exe    28      SeManageVolumePrivilege         Manage the files on a volume
9940    userinit.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
9940    userinit.exe    30      SeCreateGlobalPrivilege Default Create global objects
9940    userinit.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9940    userinit.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9940    userinit.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
9940    userinit.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
9940    userinit.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9940    userinit.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9852    explorer.exe    2       SeCreateTokenPrivilege          Create a token object
9852    explorer.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9852    explorer.exe    4       SeLockMemoryPrivilege           Lock pages in memory
9852    explorer.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
9852    explorer.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
9852    explorer.exe    7       SeTcbPrivilege          Act as part of the operating system
9852    explorer.exe    8       SeSecurityPrivilege             Manage auditing and security log
9852    explorer.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9852    explorer.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
9852    explorer.exe    11      SeSystemProfilePrivilege                Profile system performance
9852    explorer.exe    12      SeSystemtimePrivilege           Change the system time
9852    explorer.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
9852    explorer.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9852    explorer.exe    15      SeCreatePagefilePrivilege               Create a pagefile
9852    explorer.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
9852    explorer.exe    17      SeBackupPrivilege               Backup files and directories
9852    explorer.exe    18      SeRestorePrivilege              Restore files and directories
9852    explorer.exe    19      SeShutdownPrivilege     Present Shut down the system
9852    explorer.exe    20      SeDebugPrivilege                Debug programs
9852    explorer.exe    21      SeAuditPrivilege                Generate security audits
9852    explorer.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9852    explorer.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9852    explorer.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9852    explorer.exe    25      SeUndockPrivilege       Present Remove computer from docking station
9852    explorer.exe    26      SeSyncAgentPrivilege            Synch directory service data
9852    explorer.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9852    explorer.exe    28      SeManageVolumePrivilege         Manage the files on a volume
9852    explorer.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
9852    explorer.exe    30      SeCreateGlobalPrivilege Default Create global objects
9852    explorer.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9852    explorer.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9852    explorer.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
9852    explorer.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
9852    explorer.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9852    explorer.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1540    Sysmon64.exe    2       SeCreateTokenPrivilege          Create a token object
1540    Sysmon64.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1540    Sysmon64.exe    4       SeLockMemoryPrivilege           Lock pages in memory
1540    Sysmon64.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
1540    Sysmon64.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
1540    Sysmon64.exe    7       SeTcbPrivilege          Act as part of the operating system
1540    Sysmon64.exe    8       SeSecurityPrivilege             Manage auditing and security log
1540    Sysmon64.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1540    Sysmon64.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
1540    Sysmon64.exe    11      SeSystemProfilePrivilege                Profile system performance
1540    Sysmon64.exe    12      SeSystemtimePrivilege           Change the system time
1540    Sysmon64.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
1540    Sysmon64.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1540    Sysmon64.exe    15      SeCreatePagefilePrivilege               Create a pagefile
1540    Sysmon64.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
1540    Sysmon64.exe    17      SeBackupPrivilege               Backup files and directories
1540    Sysmon64.exe    18      SeRestorePrivilege              Restore files and directories
1540    Sysmon64.exe    19      SeShutdownPrivilege     Present Shut down the system
1540    Sysmon64.exe    20      SeDebugPrivilege                Debug programs
1540    Sysmon64.exe    21      SeAuditPrivilege                Generate security audits
1540    Sysmon64.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1540    Sysmon64.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1540    Sysmon64.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1540    Sysmon64.exe    25      SeUndockPrivilege       Present Remove computer from docking station
1540    Sysmon64.exe    26      SeSyncAgentPrivilege            Synch directory service data
1540    Sysmon64.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1540    Sysmon64.exe    28      SeManageVolumePrivilege         Manage the files on a volume
1540    Sysmon64.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
1540    Sysmon64.exe    30      SeCreateGlobalPrivilege Default Create global objects
1540    Sysmon64.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1540    Sysmon64.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1540    Sysmon64.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
1540    Sysmon64.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
1540    Sysmon64.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1540    Sysmon64.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10944   conhost.exe     2       SeCreateTokenPrivilege          Create a token object
10944   conhost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10944   conhost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
10944   conhost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
10944   conhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
10944   conhost.exe     7       SeTcbPrivilege          Act as part of the operating system
10944   conhost.exe     8       SeSecurityPrivilege             Manage auditing and security log
10944   conhost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10944   conhost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
10944   conhost.exe     11      SeSystemProfilePrivilege                Profile system performance
10944   conhost.exe     12      SeSystemtimePrivilege           Change the system time
10944   conhost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
10944   conhost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
10944   conhost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
10944   conhost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
10944   conhost.exe     17      SeBackupPrivilege               Backup files and directories
10944   conhost.exe     18      SeRestorePrivilege              Restore files and directories
10944   conhost.exe     19      SeShutdownPrivilege     Present Shut down the system
10944   conhost.exe     20      SeDebugPrivilege                Debug programs
10944   conhost.exe     21      SeAuditPrivilege                Generate security audits
10944   conhost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
10944   conhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10944   conhost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10944   conhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
10944   conhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
10944   conhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10944   conhost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
10944   conhost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
10944   conhost.exe     30      SeCreateGlobalPrivilege Default Create global objects
10944   conhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10944   conhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10944   conhost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
10944   conhost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
10944   conhost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
10944   conhost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
4636    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
4636    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
4636    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
4636    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
4636    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
4636    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
4636    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
4636    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4636    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
4636    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
4636    svchost.exe     12      SeSystemtimePrivilege           Change the system time
4636    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
4636    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4636    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
4636    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
4636    svchost.exe     17      SeBackupPrivilege               Backup files and directories
4636    svchost.exe     18      SeRestorePrivilege              Restore files and directories
4636    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
4636    svchost.exe     20      SeDebugPrivilege                Debug programs
4636    svchost.exe     21      SeAuditPrivilege                Generate security audits
4636    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4636    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4636    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4636    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
4636    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
4636    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4636    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
4636    svchost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
4636    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
4636    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4636    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4636    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
4636    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
4636    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4636    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
8       StartMenuExper  2       SeCreateTokenPrivilege          Create a token object
8       StartMenuExper  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
8       StartMenuExper  4       SeLockMemoryPrivilege           Lock pages in memory
8       StartMenuExper  5       SeIncreaseQuotaPrivilege                Increase quotas
8       StartMenuExper  6       SeMachineAccountPrivilege               Add workstations to the domain
8       StartMenuExper  7       SeTcbPrivilege          Act as part of the operating system
8       StartMenuExper  8       SeSecurityPrivilege             Manage auditing and security log
8       StartMenuExper  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
8       StartMenuExper  10      SeLoadDriverPrivilege           Load and unload device drivers
8       StartMenuExper  11      SeSystemProfilePrivilege                Profile system performance
8       StartMenuExper  12      SeSystemtimePrivilege           Change the system time
8       StartMenuExper  13      SeProfileSingleProcessPrivilege         Profile a single process
8       StartMenuExper  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
8       StartMenuExper  15      SeCreatePagefilePrivilege               Create a pagefile
8       StartMenuExper  16      SeCreatePermanentPrivilege              Create permanent shared objects
8       StartMenuExper  17      SeBackupPrivilege               Backup files and directories
8       StartMenuExper  18      SeRestorePrivilege              Restore files and directories
8       StartMenuExper  19      SeShutdownPrivilege             Shut down the system
8       StartMenuExper  20      SeDebugPrivilege                Debug programs
8       StartMenuExper  21      SeAuditPrivilege                Generate security audits
8       StartMenuExper  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
8       StartMenuExper  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
8       StartMenuExper  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
8       StartMenuExper  25      SeUndockPrivilege               Remove computer from docking station
8       StartMenuExper  26      SeSyncAgentPrivilege            Synch directory service data
8       StartMenuExper  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
8       StartMenuExper  28      SeManageVolumePrivilege         Manage the files on a volume
8       StartMenuExper  29      SeImpersonatePrivilege          Impersonate a client after authentication
8       StartMenuExper  30      SeCreateGlobalPrivilege         Create global objects
8       StartMenuExper  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
8       StartMenuExper  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
8       StartMenuExper  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
8       StartMenuExper  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
8       StartMenuExper  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
8       StartMenuExper  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10764   RuntimeBroker.  2       SeCreateTokenPrivilege          Create a token object
10764   RuntimeBroker.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10764   RuntimeBroker.  4       SeLockMemoryPrivilege           Lock pages in memory
10764   RuntimeBroker.  5       SeIncreaseQuotaPrivilege                Increase quotas
10764   RuntimeBroker.  6       SeMachineAccountPrivilege               Add workstations to the domain
10764   RuntimeBroker.  7       SeTcbPrivilege          Act as part of the operating system
10764   RuntimeBroker.  8       SeSecurityPrivilege             Manage auditing and security log
10764   RuntimeBroker.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10764   RuntimeBroker.  10      SeLoadDriverPrivilege           Load and unload device drivers
10764   RuntimeBroker.  11      SeSystemProfilePrivilege                Profile system performance
10764   RuntimeBroker.  12      SeSystemtimePrivilege           Change the system time
10764   RuntimeBroker.  13      SeProfileSingleProcessPrivilege         Profile a single process
10764   RuntimeBroker.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
10764   RuntimeBroker.  15      SeCreatePagefilePrivilege               Create a pagefile
10764   RuntimeBroker.  16      SeCreatePermanentPrivilege              Create permanent shared objects
10764   RuntimeBroker.  17      SeBackupPrivilege               Backup files and directories
10764   RuntimeBroker.  18      SeRestorePrivilege              Restore files and directories
10764   RuntimeBroker.  19      SeShutdownPrivilege     Present Shut down the system
10764   RuntimeBroker.  20      SeDebugPrivilege                Debug programs
10764   RuntimeBroker.  21      SeAuditPrivilege                Generate security audits
10764   RuntimeBroker.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
10764   RuntimeBroker.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10764   RuntimeBroker.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10764   RuntimeBroker.  25      SeUndockPrivilege       Present Remove computer from docking station
10764   RuntimeBroker.  26      SeSyncAgentPrivilege            Synch directory service data
10764   RuntimeBroker.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10764   RuntimeBroker.  28      SeManageVolumePrivilege         Manage the files on a volume
10764   RuntimeBroker.  29      SeImpersonatePrivilege          Impersonate a client after authentication
10764   RuntimeBroker.  30      SeCreateGlobalPrivilege Default Create global objects
10764   RuntimeBroker.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10764   RuntimeBroker.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10764   RuntimeBroker.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
10764   RuntimeBroker.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
10764   RuntimeBroker.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
10764   RuntimeBroker.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9288    SearchApp.exe   2       SeCreateTokenPrivilege          Create a token object
9288    SearchApp.exe   3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9288    SearchApp.exe   4       SeLockMemoryPrivilege           Lock pages in memory
9288    SearchApp.exe   5       SeIncreaseQuotaPrivilege                Increase quotas
9288    SearchApp.exe   6       SeMachineAccountPrivilege               Add workstations to the domain
9288    SearchApp.exe   7       SeTcbPrivilege          Act as part of the operating system
9288    SearchApp.exe   8       SeSecurityPrivilege             Manage auditing and security log
9288    SearchApp.exe   9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9288    SearchApp.exe   10      SeLoadDriverPrivilege           Load and unload device drivers
9288    SearchApp.exe   11      SeSystemProfilePrivilege                Profile system performance
9288    SearchApp.exe   12      SeSystemtimePrivilege           Change the system time
9288    SearchApp.exe   13      SeProfileSingleProcessPrivilege         Profile a single process
9288    SearchApp.exe   14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9288    SearchApp.exe   15      SeCreatePagefilePrivilege               Create a pagefile
9288    SearchApp.exe   16      SeCreatePermanentPrivilege              Create permanent shared objects
9288    SearchApp.exe   17      SeBackupPrivilege               Backup files and directories
9288    SearchApp.exe   18      SeRestorePrivilege              Restore files and directories
9288    SearchApp.exe   19      SeShutdownPrivilege             Shut down the system
9288    SearchApp.exe   20      SeDebugPrivilege                Debug programs
9288    SearchApp.exe   21      SeAuditPrivilege                Generate security audits
9288    SearchApp.exe   22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9288    SearchApp.exe   23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9288    SearchApp.exe   24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9288    SearchApp.exe   25      SeUndockPrivilege               Remove computer from docking station
9288    SearchApp.exe   26      SeSyncAgentPrivilege            Synch directory service data
9288    SearchApp.exe   27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9288    SearchApp.exe   28      SeManageVolumePrivilege         Manage the files on a volume
9288    SearchApp.exe   29      SeImpersonatePrivilege          Impersonate a client after authentication
9288    SearchApp.exe   30      SeCreateGlobalPrivilege         Create global objects
9288    SearchApp.exe   31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9288    SearchApp.exe   32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9288    SearchApp.exe   33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
9288    SearchApp.exe   34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
9288    SearchApp.exe   35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9288    SearchApp.exe   36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10968   RuntimeBroker.  2       SeCreateTokenPrivilege          Create a token object
10968   RuntimeBroker.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10968   RuntimeBroker.  4       SeLockMemoryPrivilege           Lock pages in memory
10968   RuntimeBroker.  5       SeIncreaseQuotaPrivilege                Increase quotas
10968   RuntimeBroker.  6       SeMachineAccountPrivilege               Add workstations to the domain
10968   RuntimeBroker.  7       SeTcbPrivilege          Act as part of the operating system
10968   RuntimeBroker.  8       SeSecurityPrivilege             Manage auditing and security log
10968   RuntimeBroker.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10968   RuntimeBroker.  10      SeLoadDriverPrivilege           Load and unload device drivers
10968   RuntimeBroker.  11      SeSystemProfilePrivilege                Profile system performance
10968   RuntimeBroker.  12      SeSystemtimePrivilege           Change the system time
10968   RuntimeBroker.  13      SeProfileSingleProcessPrivilege         Profile a single process
10968   RuntimeBroker.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
10968   RuntimeBroker.  15      SeCreatePagefilePrivilege               Create a pagefile
10968   RuntimeBroker.  16      SeCreatePermanentPrivilege              Create permanent shared objects
10968   RuntimeBroker.  17      SeBackupPrivilege               Backup files and directories
10968   RuntimeBroker.  18      SeRestorePrivilege              Restore files and directories
10968   RuntimeBroker.  19      SeShutdownPrivilege     Present Shut down the system
10968   RuntimeBroker.  20      SeDebugPrivilege                Debug programs
10968   RuntimeBroker.  21      SeAuditPrivilege                Generate security audits
10968   RuntimeBroker.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
10968   RuntimeBroker.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10968   RuntimeBroker.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10968   RuntimeBroker.  25      SeUndockPrivilege       Present Remove computer from docking station
10968   RuntimeBroker.  26      SeSyncAgentPrivilege            Synch directory service data
10968   RuntimeBroker.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10968   RuntimeBroker.  28      SeManageVolumePrivilege         Manage the files on a volume
10968   RuntimeBroker.  29      SeImpersonatePrivilege          Impersonate a client after authentication
10968   RuntimeBroker.  30      SeCreateGlobalPrivilege Default Create global objects
10968   RuntimeBroker.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10968   RuntimeBroker.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10968   RuntimeBroker.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
10968   RuntimeBroker.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
10968   RuntimeBroker.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
10968   RuntimeBroker.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
5164    dllhost.exe     2       SeCreateTokenPrivilege          Create a token object
5164    dllhost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
5164    dllhost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
5164    dllhost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
5164    dllhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
5164    dllhost.exe     7       SeTcbPrivilege          Act as part of the operating system
5164    dllhost.exe     8       SeSecurityPrivilege             Manage auditing and security log
5164    dllhost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
5164    dllhost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
5164    dllhost.exe     11      SeSystemProfilePrivilege                Profile system performance
5164    dllhost.exe     12      SeSystemtimePrivilege           Change the system time
5164    dllhost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
5164    dllhost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
5164    dllhost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
5164    dllhost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
5164    dllhost.exe     17      SeBackupPrivilege               Backup files and directories
5164    dllhost.exe     18      SeRestorePrivilege              Restore files and directories
5164    dllhost.exe     19      SeShutdownPrivilege     Present Shut down the system
5164    dllhost.exe     20      SeDebugPrivilege                Debug programs
5164    dllhost.exe     21      SeAuditPrivilege                Generate security audits
5164    dllhost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5164    dllhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
5164    dllhost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5164    dllhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
5164    dllhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
5164    dllhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5164    dllhost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
5164    dllhost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
5164    dllhost.exe     30      SeCreateGlobalPrivilege Default Create global objects
5164    dllhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5164    dllhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5164    dllhost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
5164    dllhost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
5164    dllhost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
5164    dllhost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
11756   svchost.exe     2       SeCreateTokenPrivilege          Create a token object
11756   svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
11756   svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
11756   svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
11756   svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
11756   svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
11756   svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
11756   svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
11756   svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
11756   svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
11756   svchost.exe     12      SeSystemtimePrivilege           Change the system time
11756   svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
11756   svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
11756   svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
11756   svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
11756   svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
11756   svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
11756   svchost.exe     19      SeShutdownPrivilege     Present,Enabled Shut down the system
11756   svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
11756   svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
11756   svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
11756   svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
11756   svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
11756   svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
11756   svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
11756   svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
11756   svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
11756   svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
11756   svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
11756   svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
11756   svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
11756   svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
11756   svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
11756   svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
11756   svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
12188   ctfmon.exe      2       SeCreateTokenPrivilege          Create a token object
12188   ctfmon.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
12188   ctfmon.exe      4       SeLockMemoryPrivilege           Lock pages in memory
12188   ctfmon.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
12188   ctfmon.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
12188   ctfmon.exe      7       SeTcbPrivilege          Act as part of the operating system
12188   ctfmon.exe      8       SeSecurityPrivilege             Manage auditing and security log
12188   ctfmon.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
12188   ctfmon.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
12188   ctfmon.exe      11      SeSystemProfilePrivilege                Profile system performance
12188   ctfmon.exe      12      SeSystemtimePrivilege           Change the system time
12188   ctfmon.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
12188   ctfmon.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
12188   ctfmon.exe      15      SeCreatePagefilePrivilege               Create a pagefile
12188   ctfmon.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
12188   ctfmon.exe      17      SeBackupPrivilege               Backup files and directories
12188   ctfmon.exe      18      SeRestorePrivilege              Restore files and directories
12188   ctfmon.exe      19      SeShutdownPrivilege     Present Shut down the system
12188   ctfmon.exe      20      SeDebugPrivilege                Debug programs
12188   ctfmon.exe      21      SeAuditPrivilege                Generate security audits
12188   ctfmon.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
12188   ctfmon.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
12188   ctfmon.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
12188   ctfmon.exe      25      SeUndockPrivilege       Present Remove computer from docking station
12188   ctfmon.exe      26      SeSyncAgentPrivilege            Synch directory service data
12188   ctfmon.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
12188   ctfmon.exe      28      SeManageVolumePrivilege         Manage the files on a volume
12188   ctfmon.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
12188   ctfmon.exe      30      SeCreateGlobalPrivilege Default Create global objects
12188   ctfmon.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
12188   ctfmon.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
12188   ctfmon.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
12188   ctfmon.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
12188   ctfmon.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
12188   ctfmon.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
11628   KeePass.exe     2       SeCreateTokenPrivilege          Create a token object
11628   KeePass.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
11628   KeePass.exe     4       SeLockMemoryPrivilege           Lock pages in memory
11628   KeePass.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
11628   KeePass.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
11628   KeePass.exe     7       SeTcbPrivilege          Act as part of the operating system
11628   KeePass.exe     8       SeSecurityPrivilege             Manage auditing and security log
11628   KeePass.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
11628   KeePass.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
11628   KeePass.exe     11      SeSystemProfilePrivilege                Profile system performance
11628   KeePass.exe     12      SeSystemtimePrivilege           Change the system time
11628   KeePass.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
11628   KeePass.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
11628   KeePass.exe     15      SeCreatePagefilePrivilege               Create a pagefile
11628   KeePass.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
11628   KeePass.exe     17      SeBackupPrivilege               Backup files and directories
11628   KeePass.exe     18      SeRestorePrivilege              Restore files and directories
11628   KeePass.exe     19      SeShutdownPrivilege     Present Shut down the system
11628   KeePass.exe     20      SeDebugPrivilege                Debug programs
11628   KeePass.exe     21      SeAuditPrivilege                Generate security audits
11628   KeePass.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
11628   KeePass.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
11628   KeePass.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
11628   KeePass.exe     25      SeUndockPrivilege       Present Remove computer from docking station
11628   KeePass.exe     26      SeSyncAgentPrivilege            Synch directory service data
11628   KeePass.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
11628   KeePass.exe     28      SeManageVolumePrivilege         Manage the files on a volume
11628   KeePass.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
11628   KeePass.exe     30      SeCreateGlobalPrivilege Default Create global objects
11628   KeePass.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
11628   KeePass.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
11628   KeePass.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
11628   KeePass.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
11628   KeePass.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
11628   KeePass.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7964    WmiPrvSE.exe    2       SeCreateTokenPrivilege          Create a token object
7964    WmiPrvSE.exe    3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
7964    WmiPrvSE.exe    4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
7964    WmiPrvSE.exe    5       SeIncreaseQuotaPrivilege        Present Increase quotas
7964    WmiPrvSE.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
7964    WmiPrvSE.exe    7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
7964    WmiPrvSE.exe    8       SeSecurityPrivilege     Present Manage auditing and security log
7964    WmiPrvSE.exe    9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
7964    WmiPrvSE.exe    10      SeLoadDriverPrivilege   Present Load and unload device drivers
7964    WmiPrvSE.exe    11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
7964    WmiPrvSE.exe    12      SeSystemtimePrivilege   Present Change the system time
7964    WmiPrvSE.exe    13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
7964    WmiPrvSE.exe    14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
7964    WmiPrvSE.exe    15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
7964    WmiPrvSE.exe    16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
7964    WmiPrvSE.exe    17      SeBackupPrivilege       Present Backup files and directories
7964    WmiPrvSE.exe    18      SeRestorePrivilege      Present Restore files and directories
7964    WmiPrvSE.exe    19      SeShutdownPrivilege     Present Shut down the system
7964    WmiPrvSE.exe    20      SeDebugPrivilege        Present,Enabled,Default Debug programs
7964    WmiPrvSE.exe    21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
7964    WmiPrvSE.exe    22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
7964    WmiPrvSE.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7964    WmiPrvSE.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7964    WmiPrvSE.exe    25      SeUndockPrivilege       Present Remove computer from docking station
7964    WmiPrvSE.exe    26      SeSyncAgentPrivilege            Synch directory service data
7964    WmiPrvSE.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7964    WmiPrvSE.exe    28      SeManageVolumePrivilege Present Manage the files on a volume
7964    WmiPrvSE.exe    29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
7964    WmiPrvSE.exe    30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
7964    WmiPrvSE.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7964    WmiPrvSE.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7964    WmiPrvSE.exe    33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
7964    WmiPrvSE.exe    34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
7964    WmiPrvSE.exe    35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
7964    WmiPrvSE.exe    36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
12140   RuntimeBroker.  2       SeCreateTokenPrivilege          Create a token object
12140   RuntimeBroker.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
12140   RuntimeBroker.  4       SeLockMemoryPrivilege           Lock pages in memory
12140   RuntimeBroker.  5       SeIncreaseQuotaPrivilege                Increase quotas
12140   RuntimeBroker.  6       SeMachineAccountPrivilege               Add workstations to the domain
12140   RuntimeBroker.  7       SeTcbPrivilege          Act as part of the operating system
12140   RuntimeBroker.  8       SeSecurityPrivilege             Manage auditing and security log
12140   RuntimeBroker.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
12140   RuntimeBroker.  10      SeLoadDriverPrivilege           Load and unload device drivers
12140   RuntimeBroker.  11      SeSystemProfilePrivilege                Profile system performance
12140   RuntimeBroker.  12      SeSystemtimePrivilege           Change the system time
12140   RuntimeBroker.  13      SeProfileSingleProcessPrivilege         Profile a single process
12140   RuntimeBroker.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
12140   RuntimeBroker.  15      SeCreatePagefilePrivilege               Create a pagefile
12140   RuntimeBroker.  16      SeCreatePermanentPrivilege              Create permanent shared objects
12140   RuntimeBroker.  17      SeBackupPrivilege               Backup files and directories
12140   RuntimeBroker.  18      SeRestorePrivilege              Restore files and directories
12140   RuntimeBroker.  19      SeShutdownPrivilege     Present Shut down the system
12140   RuntimeBroker.  20      SeDebugPrivilege                Debug programs
12140   RuntimeBroker.  21      SeAuditPrivilege                Generate security audits
12140   RuntimeBroker.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
12140   RuntimeBroker.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
12140   RuntimeBroker.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
12140   RuntimeBroker.  25      SeUndockPrivilege       Present Remove computer from docking station
12140   RuntimeBroker.  26      SeSyncAgentPrivilege            Synch directory service data
12140   RuntimeBroker.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
12140   RuntimeBroker.  28      SeManageVolumePrivilege         Manage the files on a volume
12140   RuntimeBroker.  29      SeImpersonatePrivilege          Impersonate a client after authentication
12140   RuntimeBroker.  30      SeCreateGlobalPrivilege Default Create global objects
12140   RuntimeBroker.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
12140   RuntimeBroker.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
12140   RuntimeBroker.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
12140   RuntimeBroker.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
12140   RuntimeBroker.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
12140   RuntimeBroker.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
12036   SecurityHealth  2       SeCreateTokenPrivilege          Create a token object
12036   SecurityHealth  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
12036   SecurityHealth  4       SeLockMemoryPrivilege           Lock pages in memory
12036   SecurityHealth  5       SeIncreaseQuotaPrivilege                Increase quotas
12036   SecurityHealth  6       SeMachineAccountPrivilege               Add workstations to the domain
12036   SecurityHealth  7       SeTcbPrivilege          Act as part of the operating system
12036   SecurityHealth  8       SeSecurityPrivilege             Manage auditing and security log
12036   SecurityHealth  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
12036   SecurityHealth  10      SeLoadDriverPrivilege           Load and unload device drivers
12036   SecurityHealth  11      SeSystemProfilePrivilege                Profile system performance
12036   SecurityHealth  12      SeSystemtimePrivilege           Change the system time
12036   SecurityHealth  13      SeProfileSingleProcessPrivilege         Profile a single process
12036   SecurityHealth  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
12036   SecurityHealth  15      SeCreatePagefilePrivilege               Create a pagefile
12036   SecurityHealth  16      SeCreatePermanentPrivilege              Create permanent shared objects
12036   SecurityHealth  17      SeBackupPrivilege               Backup files and directories
12036   SecurityHealth  18      SeRestorePrivilege              Restore files and directories
12036   SecurityHealth  19      SeShutdownPrivilege     Present Shut down the system
12036   SecurityHealth  20      SeDebugPrivilege                Debug programs
12036   SecurityHealth  21      SeAuditPrivilege                Generate security audits
12036   SecurityHealth  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
12036   SecurityHealth  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
12036   SecurityHealth  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
12036   SecurityHealth  25      SeUndockPrivilege       Present Remove computer from docking station
12036   SecurityHealth  26      SeSyncAgentPrivilege            Synch directory service data
12036   SecurityHealth  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
12036   SecurityHealth  28      SeManageVolumePrivilege         Manage the files on a volume
12036   SecurityHealth  29      SeImpersonatePrivilege          Impersonate a client after authentication
12036   SecurityHealth  30      SeCreateGlobalPrivilege Default Create global objects
12036   SecurityHealth  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
12036   SecurityHealth  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
12036   SecurityHealth  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
12036   SecurityHealth  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
12036   SecurityHealth  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
12036   SecurityHealth  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
12252   OneDrive.exe    2       SeCreateTokenPrivilege          Create a token object
12252   OneDrive.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
12252   OneDrive.exe    4       SeLockMemoryPrivilege           Lock pages in memory
12252   OneDrive.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
12252   OneDrive.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
12252   OneDrive.exe    7       SeTcbPrivilege          Act as part of the operating system
12252   OneDrive.exe    8       SeSecurityPrivilege             Manage auditing and security log
12252   OneDrive.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
12252   OneDrive.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
12252   OneDrive.exe    11      SeSystemProfilePrivilege                Profile system performance
12252   OneDrive.exe    12      SeSystemtimePrivilege           Change the system time
12252   OneDrive.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
12252   OneDrive.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
12252   OneDrive.exe    15      SeCreatePagefilePrivilege               Create a pagefile
12252   OneDrive.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
12252   OneDrive.exe    17      SeBackupPrivilege               Backup files and directories
12252   OneDrive.exe    18      SeRestorePrivilege              Restore files and directories
12252   OneDrive.exe    19      SeShutdownPrivilege     Present Shut down the system
12252   OneDrive.exe    20      SeDebugPrivilege                Debug programs
12252   OneDrive.exe    21      SeAuditPrivilege                Generate security audits
12252   OneDrive.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
12252   OneDrive.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
12252   OneDrive.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
12252   OneDrive.exe    25      SeUndockPrivilege       Present Remove computer from docking station
12252   OneDrive.exe    26      SeSyncAgentPrivilege            Synch directory service data
12252   OneDrive.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
12252   OneDrive.exe    28      SeManageVolumePrivilege         Manage the files on a volume
12252   OneDrive.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
12252   OneDrive.exe    30      SeCreateGlobalPrivilege Default Create global objects
12252   OneDrive.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
12252   OneDrive.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
12252   OneDrive.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
12252   OneDrive.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
12252   OneDrive.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
12252   OneDrive.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7484    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
7484    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7484    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
7484    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
7484    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
7484    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
7484    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
7484    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7484    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
7484    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
7484    msedge.exe      12      SeSystemtimePrivilege           Change the system time
7484    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
7484    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7484    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
7484    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
7484    msedge.exe      17      SeBackupPrivilege               Backup files and directories
7484    msedge.exe      18      SeRestorePrivilege              Restore files and directories
7484    msedge.exe      19      SeShutdownPrivilege     Present Shut down the system
7484    msedge.exe      20      SeDebugPrivilege                Debug programs
7484    msedge.exe      21      SeAuditPrivilege                Generate security audits
7484    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7484    msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7484    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7484    msedge.exe      25      SeUndockPrivilege       Present Remove computer from docking station
7484    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
7484    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7484    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
7484    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
7484    msedge.exe      30      SeCreateGlobalPrivilege Default Create global objects
7484    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7484    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7484    msedge.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7484    msedge.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7484    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7484    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
12740   TextInputHost.  2       SeCreateTokenPrivilege          Create a token object
12740   TextInputHost.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
12740   TextInputHost.  4       SeLockMemoryPrivilege           Lock pages in memory
12740   TextInputHost.  5       SeIncreaseQuotaPrivilege                Increase quotas
12740   TextInputHost.  6       SeMachineAccountPrivilege               Add workstations to the domain
12740   TextInputHost.  7       SeTcbPrivilege          Act as part of the operating system
12740   TextInputHost.  8       SeSecurityPrivilege             Manage auditing and security log
12740   TextInputHost.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
12740   TextInputHost.  10      SeLoadDriverPrivilege           Load and unload device drivers
12740   TextInputHost.  11      SeSystemProfilePrivilege                Profile system performance
12740   TextInputHost.  12      SeSystemtimePrivilege           Change the system time
12740   TextInputHost.  13      SeProfileSingleProcessPrivilege         Profile a single process
12740   TextInputHost.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
12740   TextInputHost.  15      SeCreatePagefilePrivilege               Create a pagefile
12740   TextInputHost.  16      SeCreatePermanentPrivilege              Create permanent shared objects
12740   TextInputHost.  17      SeBackupPrivilege               Backup files and directories
12740   TextInputHost.  18      SeRestorePrivilege              Restore files and directories
12740   TextInputHost.  19      SeShutdownPrivilege     Present Shut down the system
12740   TextInputHost.  20      SeDebugPrivilege                Debug programs
12740   TextInputHost.  21      SeAuditPrivilege                Generate security audits
12740   TextInputHost.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
12740   TextInputHost.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
12740   TextInputHost.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
12740   TextInputHost.  25      SeUndockPrivilege       Present Remove computer from docking station
12740   TextInputHost.  26      SeSyncAgentPrivilege            Synch directory service data
12740   TextInputHost.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
12740   TextInputHost.  28      SeManageVolumePrivilege         Manage the files on a volume
12740   TextInputHost.  29      SeImpersonatePrivilege          Impersonate a client after authentication
12740   TextInputHost.  30      SeCreateGlobalPrivilege Default Create global objects
12740   TextInputHost.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
12740   TextInputHost.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
12740   TextInputHost.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
12740   TextInputHost.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
12740   TextInputHost.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
12740   TextInputHost.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
11592   OneDrive.exe    2       SeCreateTokenPrivilege          Create a token object
11592   OneDrive.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
11592   OneDrive.exe    4       SeLockMemoryPrivilege           Lock pages in memory
11592   OneDrive.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
11592   OneDrive.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
11592   OneDrive.exe    7       SeTcbPrivilege          Act as part of the operating system
11592   OneDrive.exe    8       SeSecurityPrivilege             Manage auditing and security log
11592   OneDrive.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
11592   OneDrive.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
11592   OneDrive.exe    11      SeSystemProfilePrivilege                Profile system performance
11592   OneDrive.exe    12      SeSystemtimePrivilege           Change the system time
11592   OneDrive.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
11592   OneDrive.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
11592   OneDrive.exe    15      SeCreatePagefilePrivilege               Create a pagefile
11592   OneDrive.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
11592   OneDrive.exe    17      SeBackupPrivilege               Backup files and directories
11592   OneDrive.exe    18      SeRestorePrivilege              Restore files and directories
11592   OneDrive.exe    19      SeShutdownPrivilege     Present Shut down the system
11592   OneDrive.exe    20      SeDebugPrivilege                Debug programs
11592   OneDrive.exe    21      SeAuditPrivilege                Generate security audits
11592   OneDrive.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
11592   OneDrive.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
11592   OneDrive.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
11592   OneDrive.exe    25      SeUndockPrivilege       Present Remove computer from docking station
11592   OneDrive.exe    26      SeSyncAgentPrivilege            Synch directory service data
11592   OneDrive.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
11592   OneDrive.exe    28      SeManageVolumePrivilege         Manage the files on a volume
11592   OneDrive.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
11592   OneDrive.exe    30      SeCreateGlobalPrivilege Default Create global objects
11592   OneDrive.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
11592   OneDrive.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
11592   OneDrive.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
11592   OneDrive.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
11592   OneDrive.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
11592   OneDrive.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
5116    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
5116    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
5116    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
5116    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
5116    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
5116    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
5116    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
5116    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
5116    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
5116    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
5116    svchost.exe     12      SeSystemtimePrivilege           Change the system time
5116    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
5116    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
5116    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
5116    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
5116    svchost.exe     17      SeBackupPrivilege               Backup files and directories
5116    svchost.exe     18      SeRestorePrivilege              Restore files and directories
5116    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
5116    svchost.exe     20      SeDebugPrivilege                Debug programs
5116    svchost.exe     21      SeAuditPrivilege                Generate security audits
5116    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5116    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
5116    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5116    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
5116    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
5116    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5116    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
5116    svchost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
5116    svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
5116    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5116    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5116    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
5116    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
5116    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
5116    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7492    ApplicationFra  2       SeCreateTokenPrivilege          Create a token object
7492    ApplicationFra  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7492    ApplicationFra  4       SeLockMemoryPrivilege           Lock pages in memory
7492    ApplicationFra  5       SeIncreaseQuotaPrivilege                Increase quotas
7492    ApplicationFra  6       SeMachineAccountPrivilege               Add workstations to the domain
7492    ApplicationFra  7       SeTcbPrivilege          Act as part of the operating system
7492    ApplicationFra  8       SeSecurityPrivilege             Manage auditing and security log
7492    ApplicationFra  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7492    ApplicationFra  10      SeLoadDriverPrivilege           Load and unload device drivers
7492    ApplicationFra  11      SeSystemProfilePrivilege                Profile system performance
7492    ApplicationFra  12      SeSystemtimePrivilege           Change the system time
7492    ApplicationFra  13      SeProfileSingleProcessPrivilege         Profile a single process
7492    ApplicationFra  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7492    ApplicationFra  15      SeCreatePagefilePrivilege               Create a pagefile
7492    ApplicationFra  16      SeCreatePermanentPrivilege              Create permanent shared objects
7492    ApplicationFra  17      SeBackupPrivilege               Backup files and directories
7492    ApplicationFra  18      SeRestorePrivilege              Restore files and directories
7492    ApplicationFra  19      SeShutdownPrivilege     Present Shut down the system
7492    ApplicationFra  20      SeDebugPrivilege                Debug programs
7492    ApplicationFra  21      SeAuditPrivilege                Generate security audits
7492    ApplicationFra  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7492    ApplicationFra  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7492    ApplicationFra  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7492    ApplicationFra  25      SeUndockPrivilege       Present Remove computer from docking station
7492    ApplicationFra  26      SeSyncAgentPrivilege            Synch directory service data
7492    ApplicationFra  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7492    ApplicationFra  28      SeManageVolumePrivilege         Manage the files on a volume
7492    ApplicationFra  29      SeImpersonatePrivilege          Impersonate a client after authentication
7492    ApplicationFra  30      SeCreateGlobalPrivilege Default Create global objects
7492    ApplicationFra  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7492    ApplicationFra  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7492    ApplicationFra  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7492    ApplicationFra  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7492    ApplicationFra  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7492    ApplicationFra  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7588    ShellExperienc  2       SeCreateTokenPrivilege          Create a token object
7588    ShellExperienc  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7588    ShellExperienc  4       SeLockMemoryPrivilege           Lock pages in memory
7588    ShellExperienc  5       SeIncreaseQuotaPrivilege                Increase quotas
7588    ShellExperienc  6       SeMachineAccountPrivilege               Add workstations to the domain
7588    ShellExperienc  7       SeTcbPrivilege          Act as part of the operating system
7588    ShellExperienc  8       SeSecurityPrivilege             Manage auditing and security log
7588    ShellExperienc  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7588    ShellExperienc  10      SeLoadDriverPrivilege           Load and unload device drivers
7588    ShellExperienc  11      SeSystemProfilePrivilege                Profile system performance
7588    ShellExperienc  12      SeSystemtimePrivilege           Change the system time
7588    ShellExperienc  13      SeProfileSingleProcessPrivilege         Profile a single process
7588    ShellExperienc  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7588    ShellExperienc  15      SeCreatePagefilePrivilege               Create a pagefile
7588    ShellExperienc  16      SeCreatePermanentPrivilege              Create permanent shared objects
7588    ShellExperienc  17      SeBackupPrivilege               Backup files and directories
7588    ShellExperienc  18      SeRestorePrivilege              Restore files and directories
7588    ShellExperienc  19      SeShutdownPrivilege             Shut down the system
7588    ShellExperienc  20      SeDebugPrivilege                Debug programs
7588    ShellExperienc  21      SeAuditPrivilege                Generate security audits
7588    ShellExperienc  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7588    ShellExperienc  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7588    ShellExperienc  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7588    ShellExperienc  25      SeUndockPrivilege               Remove computer from docking station
7588    ShellExperienc  26      SeSyncAgentPrivilege            Synch directory service data
7588    ShellExperienc  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7588    ShellExperienc  28      SeManageVolumePrivilege         Manage the files on a volume
7588    ShellExperienc  29      SeImpersonatePrivilege          Impersonate a client after authentication
7588    ShellExperienc  30      SeCreateGlobalPrivilege         Create global objects
7588    ShellExperienc  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7588    ShellExperienc  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7588    ShellExperienc  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7588    ShellExperienc  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
7588    ShellExperienc  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7588    ShellExperienc  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
4464    RuntimeBroker.  2       SeCreateTokenPrivilege          Create a token object
4464    RuntimeBroker.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
4464    RuntimeBroker.  4       SeLockMemoryPrivilege           Lock pages in memory
4464    RuntimeBroker.  5       SeIncreaseQuotaPrivilege                Increase quotas
4464    RuntimeBroker.  6       SeMachineAccountPrivilege               Add workstations to the domain
4464    RuntimeBroker.  7       SeTcbPrivilege          Act as part of the operating system
4464    RuntimeBroker.  8       SeSecurityPrivilege             Manage auditing and security log
4464    RuntimeBroker.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4464    RuntimeBroker.  10      SeLoadDriverPrivilege           Load and unload device drivers
4464    RuntimeBroker.  11      SeSystemProfilePrivilege                Profile system performance
4464    RuntimeBroker.  12      SeSystemtimePrivilege           Change the system time
4464    RuntimeBroker.  13      SeProfileSingleProcessPrivilege         Profile a single process
4464    RuntimeBroker.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4464    RuntimeBroker.  15      SeCreatePagefilePrivilege               Create a pagefile
4464    RuntimeBroker.  16      SeCreatePermanentPrivilege              Create permanent shared objects
4464    RuntimeBroker.  17      SeBackupPrivilege               Backup files and directories
4464    RuntimeBroker.  18      SeRestorePrivilege              Restore files and directories
4464    RuntimeBroker.  19      SeShutdownPrivilege     Present Shut down the system
4464    RuntimeBroker.  20      SeDebugPrivilege                Debug programs
4464    RuntimeBroker.  21      SeAuditPrivilege                Generate security audits
4464    RuntimeBroker.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4464    RuntimeBroker.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4464    RuntimeBroker.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4464    RuntimeBroker.  25      SeUndockPrivilege       Present Remove computer from docking station
4464    RuntimeBroker.  26      SeSyncAgentPrivilege            Synch directory service data
4464    RuntimeBroker.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4464    RuntimeBroker.  28      SeManageVolumePrivilege         Manage the files on a volume
4464    RuntimeBroker.  29      SeImpersonatePrivilege          Impersonate a client after authentication
4464    RuntimeBroker.  30      SeCreateGlobalPrivilege Default Create global objects
4464    RuntimeBroker.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4464    RuntimeBroker.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4464    RuntimeBroker.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
4464    RuntimeBroker.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
4464    RuntimeBroker.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4464    RuntimeBroker.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
11148   Microsoft.Phot  2       SeCreateTokenPrivilege          Create a token object
11148   Microsoft.Phot  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
11148   Microsoft.Phot  4       SeLockMemoryPrivilege           Lock pages in memory
11148   Microsoft.Phot  5       SeIncreaseQuotaPrivilege                Increase quotas
11148   Microsoft.Phot  6       SeMachineAccountPrivilege               Add workstations to the domain
11148   Microsoft.Phot  7       SeTcbPrivilege          Act as part of the operating system
11148   Microsoft.Phot  8       SeSecurityPrivilege             Manage auditing and security log
11148   Microsoft.Phot  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
11148   Microsoft.Phot  10      SeLoadDriverPrivilege           Load and unload device drivers
11148   Microsoft.Phot  11      SeSystemProfilePrivilege                Profile system performance
11148   Microsoft.Phot  12      SeSystemtimePrivilege           Change the system time
11148   Microsoft.Phot  13      SeProfileSingleProcessPrivilege         Profile a single process
11148   Microsoft.Phot  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
11148   Microsoft.Phot  15      SeCreatePagefilePrivilege               Create a pagefile
11148   Microsoft.Phot  16      SeCreatePermanentPrivilege              Create permanent shared objects
11148   Microsoft.Phot  17      SeBackupPrivilege               Backup files and directories
11148   Microsoft.Phot  18      SeRestorePrivilege              Restore files and directories
11148   Microsoft.Phot  19      SeShutdownPrivilege             Shut down the system
11148   Microsoft.Phot  20      SeDebugPrivilege                Debug programs
11148   Microsoft.Phot  21      SeAuditPrivilege                Generate security audits
11148   Microsoft.Phot  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
11148   Microsoft.Phot  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
11148   Microsoft.Phot  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
11148   Microsoft.Phot  25      SeUndockPrivilege               Remove computer from docking station
11148   Microsoft.Phot  26      SeSyncAgentPrivilege            Synch directory service data
11148   Microsoft.Phot  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
11148   Microsoft.Phot  28      SeManageVolumePrivilege         Manage the files on a volume
11148   Microsoft.Phot  29      SeImpersonatePrivilege          Impersonate a client after authentication
11148   Microsoft.Phot  30      SeCreateGlobalPrivilege         Create global objects
11148   Microsoft.Phot  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
11148   Microsoft.Phot  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
11148   Microsoft.Phot  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
11148   Microsoft.Phot  34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
11148   Microsoft.Phot  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
11148   Microsoft.Phot  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10648   RuntimeBroker.  2       SeCreateTokenPrivilege          Create a token object
10648   RuntimeBroker.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10648   RuntimeBroker.  4       SeLockMemoryPrivilege           Lock pages in memory
10648   RuntimeBroker.  5       SeIncreaseQuotaPrivilege                Increase quotas
10648   RuntimeBroker.  6       SeMachineAccountPrivilege               Add workstations to the domain
10648   RuntimeBroker.  7       SeTcbPrivilege          Act as part of the operating system
10648   RuntimeBroker.  8       SeSecurityPrivilege             Manage auditing and security log
10648   RuntimeBroker.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10648   RuntimeBroker.  10      SeLoadDriverPrivilege           Load and unload device drivers
10648   RuntimeBroker.  11      SeSystemProfilePrivilege                Profile system performance
10648   RuntimeBroker.  12      SeSystemtimePrivilege           Change the system time
10648   RuntimeBroker.  13      SeProfileSingleProcessPrivilege         Profile a single process
10648   RuntimeBroker.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
10648   RuntimeBroker.  15      SeCreatePagefilePrivilege               Create a pagefile
10648   RuntimeBroker.  16      SeCreatePermanentPrivilege              Create permanent shared objects
10648   RuntimeBroker.  17      SeBackupPrivilege               Backup files and directories
10648   RuntimeBroker.  18      SeRestorePrivilege              Restore files and directories
10648   RuntimeBroker.  19      SeShutdownPrivilege     Present Shut down the system
10648   RuntimeBroker.  20      SeDebugPrivilege                Debug programs
10648   RuntimeBroker.  21      SeAuditPrivilege                Generate security audits
10648   RuntimeBroker.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
10648   RuntimeBroker.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10648   RuntimeBroker.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10648   RuntimeBroker.  25      SeUndockPrivilege       Present Remove computer from docking station
10648   RuntimeBroker.  26      SeSyncAgentPrivilege            Synch directory service data
10648   RuntimeBroker.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10648   RuntimeBroker.  28      SeManageVolumePrivilege         Manage the files on a volume
10648   RuntimeBroker.  29      SeImpersonatePrivilege          Impersonate a client after authentication
10648   RuntimeBroker.  30      SeCreateGlobalPrivilege Default Create global objects
10648   RuntimeBroker.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10648   RuntimeBroker.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10648   RuntimeBroker.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
10648   RuntimeBroker.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
10648   RuntimeBroker.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
10648   RuntimeBroker.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9092    SearchApp.exe   2       SeCreateTokenPrivilege          Create a token object
9092    SearchApp.exe   3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9092    SearchApp.exe   4       SeLockMemoryPrivilege           Lock pages in memory
9092    SearchApp.exe   5       SeIncreaseQuotaPrivilege                Increase quotas
9092    SearchApp.exe   6       SeMachineAccountPrivilege               Add workstations to the domain
9092    SearchApp.exe   7       SeTcbPrivilege          Act as part of the operating system
9092    SearchApp.exe   8       SeSecurityPrivilege             Manage auditing and security log
9092    SearchApp.exe   9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9092    SearchApp.exe   10      SeLoadDriverPrivilege           Load and unload device drivers
9092    SearchApp.exe   11      SeSystemProfilePrivilege                Profile system performance
9092    SearchApp.exe   12      SeSystemtimePrivilege           Change the system time
9092    SearchApp.exe   13      SeProfileSingleProcessPrivilege         Profile a single process
9092    SearchApp.exe   14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9092    SearchApp.exe   15      SeCreatePagefilePrivilege               Create a pagefile
9092    SearchApp.exe   16      SeCreatePermanentPrivilege              Create permanent shared objects
9092    SearchApp.exe   17      SeBackupPrivilege               Backup files and directories
9092    SearchApp.exe   18      SeRestorePrivilege              Restore files and directories
9092    SearchApp.exe   19      SeShutdownPrivilege             Shut down the system
9092    SearchApp.exe   20      SeDebugPrivilege                Debug programs
9092    SearchApp.exe   21      SeAuditPrivilege                Generate security audits
9092    SearchApp.exe   22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9092    SearchApp.exe   23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9092    SearchApp.exe   24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9092    SearchApp.exe   25      SeUndockPrivilege               Remove computer from docking station
9092    SearchApp.exe   26      SeSyncAgentPrivilege            Synch directory service data
9092    SearchApp.exe   27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9092    SearchApp.exe   28      SeManageVolumePrivilege         Manage the files on a volume
9092    SearchApp.exe   29      SeImpersonatePrivilege          Impersonate a client after authentication
9092    SearchApp.exe   30      SeCreateGlobalPrivilege         Create global objects
9092    SearchApp.exe   31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9092    SearchApp.exe   32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9092    SearchApp.exe   33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
9092    SearchApp.exe   34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
9092    SearchApp.exe   35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9092    SearchApp.exe   36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
2800    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
2800    svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
2800    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
2800    svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
2800    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
2800    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
2800    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
2800    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
2800    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
2800    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
2800    svchost.exe     12      SeSystemtimePrivilege           Change the system time
2800    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
2800    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
2800    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
2800    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
2800    svchost.exe     17      SeBackupPrivilege               Backup files and directories
2800    svchost.exe     18      SeRestorePrivilege              Restore files and directories
2800    svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
2800    svchost.exe     20      SeDebugPrivilege                Debug programs
2800    svchost.exe     21      SeAuditPrivilege        Present Generate security audits
2800    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
2800    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
2800    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
2800    svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
2800    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
2800    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
2800    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
2800    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
2800    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
2800    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
2800    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
2800    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
2800    svchost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
2800    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
2800    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
11740   uhssvc.exe      2       SeCreateTokenPrivilege          Create a token object
11740   uhssvc.exe      3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
11740   uhssvc.exe      4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
11740   uhssvc.exe      5       SeIncreaseQuotaPrivilege        Present Increase quotas
11740   uhssvc.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
11740   uhssvc.exe      7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
11740   uhssvc.exe      8       SeSecurityPrivilege     Present Manage auditing and security log
11740   uhssvc.exe      9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
11740   uhssvc.exe      10      SeLoadDriverPrivilege   Present Load and unload device drivers
11740   uhssvc.exe      11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
11740   uhssvc.exe      12      SeSystemtimePrivilege   Present Change the system time
11740   uhssvc.exe      13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
11740   uhssvc.exe      14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
11740   uhssvc.exe      15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
11740   uhssvc.exe      16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
11740   uhssvc.exe      17      SeBackupPrivilege       Present Backup files and directories
11740   uhssvc.exe      18      SeRestorePrivilege      Present Restore files and directories
11740   uhssvc.exe      19      SeShutdownPrivilege     Present Shut down the system
11740   uhssvc.exe      20      SeDebugPrivilege        Present,Enabled,Default Debug programs
11740   uhssvc.exe      21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
11740   uhssvc.exe      22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
11740   uhssvc.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
11740   uhssvc.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
11740   uhssvc.exe      25      SeUndockPrivilege       Present Remove computer from docking station
11740   uhssvc.exe      26      SeSyncAgentPrivilege            Synch directory service data
11740   uhssvc.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
11740   uhssvc.exe      28      SeManageVolumePrivilege Present Manage the files on a volume
11740   uhssvc.exe      29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
11740   uhssvc.exe      30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
11740   uhssvc.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
11740   uhssvc.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
11740   uhssvc.exe      33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
11740   uhssvc.exe      34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
11740   uhssvc.exe      35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
11740   uhssvc.exe      36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
7884    dllhost.exe     2       SeCreateTokenPrivilege          Create a token object
7884    dllhost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7884    dllhost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
7884    dllhost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
7884    dllhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
7884    dllhost.exe     7       SeTcbPrivilege          Act as part of the operating system
7884    dllhost.exe     8       SeSecurityPrivilege             Manage auditing and security log
7884    dllhost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7884    dllhost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
7884    dllhost.exe     11      SeSystemProfilePrivilege                Profile system performance
7884    dllhost.exe     12      SeSystemtimePrivilege           Change the system time
7884    dllhost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
7884    dllhost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7884    dllhost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
7884    dllhost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
7884    dllhost.exe     17      SeBackupPrivilege               Backup files and directories
7884    dllhost.exe     18      SeRestorePrivilege              Restore files and directories
7884    dllhost.exe     19      SeShutdownPrivilege     Present Shut down the system
7884    dllhost.exe     20      SeDebugPrivilege                Debug programs
7884    dllhost.exe     21      SeAuditPrivilege                Generate security audits
7884    dllhost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7884    dllhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7884    dllhost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7884    dllhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
7884    dllhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
7884    dllhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7884    dllhost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
7884    dllhost.exe     29      SeImpersonatePrivilege          Impersonate a client after authentication
7884    dllhost.exe     30      SeCreateGlobalPrivilege Default Create global objects
7884    dllhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7884    dllhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7884    dllhost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7884    dllhost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7884    dllhost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7884    dllhost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
7516    FileCoAuth.exe  2       SeCreateTokenPrivilege          Create a token object
7516    FileCoAuth.exe  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7516    FileCoAuth.exe  4       SeLockMemoryPrivilege           Lock pages in memory
7516    FileCoAuth.exe  5       SeIncreaseQuotaPrivilege                Increase quotas
7516    FileCoAuth.exe  6       SeMachineAccountPrivilege               Add workstations to the domain
7516    FileCoAuth.exe  7       SeTcbPrivilege          Act as part of the operating system
7516    FileCoAuth.exe  8       SeSecurityPrivilege             Manage auditing and security log
7516    FileCoAuth.exe  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7516    FileCoAuth.exe  10      SeLoadDriverPrivilege           Load and unload device drivers
7516    FileCoAuth.exe  11      SeSystemProfilePrivilege                Profile system performance
7516    FileCoAuth.exe  12      SeSystemtimePrivilege           Change the system time
7516    FileCoAuth.exe  13      SeProfileSingleProcessPrivilege         Profile a single process
7516    FileCoAuth.exe  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7516    FileCoAuth.exe  15      SeCreatePagefilePrivilege               Create a pagefile
7516    FileCoAuth.exe  16      SeCreatePermanentPrivilege              Create permanent shared objects
7516    FileCoAuth.exe  17      SeBackupPrivilege               Backup files and directories
7516    FileCoAuth.exe  18      SeRestorePrivilege              Restore files and directories
7516    FileCoAuth.exe  19      SeShutdownPrivilege     Present Shut down the system
7516    FileCoAuth.exe  20      SeDebugPrivilege                Debug programs
7516    FileCoAuth.exe  21      SeAuditPrivilege                Generate security audits
7516    FileCoAuth.exe  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7516    FileCoAuth.exe  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7516    FileCoAuth.exe  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7516    FileCoAuth.exe  25      SeUndockPrivilege       Present Remove computer from docking station
7516    FileCoAuth.exe  26      SeSyncAgentPrivilege            Synch directory service data
7516    FileCoAuth.exe  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7516    FileCoAuth.exe  28      SeManageVolumePrivilege         Manage the files on a volume
7516    FileCoAuth.exe  29      SeImpersonatePrivilege          Impersonate a client after authentication
7516    FileCoAuth.exe  30      SeCreateGlobalPrivilege Default Create global objects
7516    FileCoAuth.exe  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7516    FileCoAuth.exe  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7516    FileCoAuth.exe  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7516    FileCoAuth.exe  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7516    FileCoAuth.exe  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7516    FileCoAuth.exe  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
10124   dllhost.exe     2       SeCreateTokenPrivilege          Create a token object
10124   dllhost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10124   dllhost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
10124   dllhost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
10124   dllhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
10124   dllhost.exe     7       SeTcbPrivilege          Act as part of the operating system
10124   dllhost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
10124   dllhost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
10124   dllhost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
10124   dllhost.exe     11      SeSystemProfilePrivilege        Present Profile system performance
10124   dllhost.exe     12      SeSystemtimePrivilege   Present Change the system time
10124   dllhost.exe     13      SeProfileSingleProcessPrivilege Present Profile a single process
10124   dllhost.exe     14      SeIncreaseBasePriorityPrivilege Present Increase scheduling priority
10124   dllhost.exe     15      SeCreatePagefilePrivilege       Present Create a pagefile
10124   dllhost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
10124   dllhost.exe     17      SeBackupPrivilege       Present Backup files and directories
10124   dllhost.exe     18      SeRestorePrivilege      Present Restore files and directories
10124   dllhost.exe     19      SeShutdownPrivilege     Present Shut down the system
10124   dllhost.exe     20      SeDebugPrivilege        Present Debug programs
10124   dllhost.exe     21      SeAuditPrivilege                Generate security audits
10124   dllhost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
10124   dllhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10124   dllhost.exe     24      SeRemoteShutdownPrivilege       Present Force shutdown from a remote system
10124   dllhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
10124   dllhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
10124   dllhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10124   dllhost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
10124   dllhost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
10124   dllhost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
10124   dllhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10124   dllhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10124   dllhost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
10124   dllhost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
10124   dllhost.exe     35      SeCreateSymbolicLinkPrivilege   Present Required to create a symbolic link
10124   dllhost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present Obtain an impersonation token for another user in the same session.
5824    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
5824    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
5824    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
5824    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
5824    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
5824    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
5824    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
5824    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
5824    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
5824    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
5824    msedge.exe      12      SeSystemtimePrivilege           Change the system time
5824    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
5824    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
5824    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
5824    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
5824    msedge.exe      17      SeBackupPrivilege               Backup files and directories
5824    msedge.exe      18      SeRestorePrivilege              Restore files and directories
5824    msedge.exe      19      SeShutdownPrivilege             Shut down the system
5824    msedge.exe      20      SeDebugPrivilege                Debug programs
5824    msedge.exe      21      SeAuditPrivilege                Generate security audits
5824    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5824    msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
5824    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5824    msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
5824    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
5824    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5824    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
5824    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
5824    msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
5824    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5824    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5824    msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
5824    msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
5824    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
5824    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
4168    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
4168    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
4168    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
4168    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
4168    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
4168    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
4168    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
4168    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
4168    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
4168    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
4168    msedge.exe      12      SeSystemtimePrivilege           Change the system time
4168    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
4168    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
4168    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
4168    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
4168    msedge.exe      17      SeBackupPrivilege               Backup files and directories
4168    msedge.exe      18      SeRestorePrivilege              Restore files and directories
4168    msedge.exe      19      SeShutdownPrivilege             Shut down the system
4168    msedge.exe      20      SeDebugPrivilege                Debug programs
4168    msedge.exe      21      SeAuditPrivilege                Generate security audits
4168    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
4168    msedge.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
4168    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4168    msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
4168    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
4168    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4168    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
4168    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
4168    msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
4168    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4168    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4168    msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
4168    msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
4168    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
4168    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
3444    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
3444    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
3444    svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
3444    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
3444    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
3444    svchost.exe     7       SeTcbPrivilege  Default Act as part of the operating system
3444    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
3444    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
3444    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
3444    svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
3444    svchost.exe     12      SeSystemtimePrivilege           Change the system time
3444    svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
3444    svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
3444    svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
3444    svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
3444    svchost.exe     17      SeBackupPrivilege               Backup files and directories
3444    svchost.exe     18      SeRestorePrivilege              Restore files and directories
3444    svchost.exe     19      SeShutdownPrivilege             Shut down the system
3444    svchost.exe     20      SeDebugPrivilege        Default Debug programs
3444    svchost.exe     21      SeAuditPrivilege        Default Generate security audits
3444    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
3444    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
3444    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
3444    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
3444    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
3444    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
3444    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
3444    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
3444    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
3444    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
3444    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
3444    svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
3444    svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
3444    svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
3444    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
11596   svchost.exe     2       SeCreateTokenPrivilege          Create a token object
11596   svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
11596   svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
11596   svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
11596   svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
11596   svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
11596   svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
11596   svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
11596   svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
11596   svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
11596   svchost.exe     12      SeSystemtimePrivilege           Change the system time
11596   svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
11596   svchost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
11596   svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
11596   svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
11596   svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
11596   svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
11596   svchost.exe     19      SeShutdownPrivilege             Shut down the system
11596   svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
11596   svchost.exe     21      SeAuditPrivilege        Default Generate security audits
11596   svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
11596   svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
11596   svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
11596   svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
11596   svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
11596   svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
11596   svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
11596   svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
11596   svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
11596   svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
11596   svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
11596   svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
11596   svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
11596   svchost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
11596   svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
7188    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
7188    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
7188    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
7188    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
7188    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
7188    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
7188    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
7188    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
7188    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
7188    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
7188    msedge.exe      12      SeSystemtimePrivilege           Change the system time
7188    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
7188    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
7188    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
7188    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
7188    msedge.exe      17      SeBackupPrivilege               Backup files and directories
7188    msedge.exe      18      SeRestorePrivilege              Restore files and directories
7188    msedge.exe      19      SeShutdownPrivilege     Present Shut down the system
7188    msedge.exe      20      SeDebugPrivilege                Debug programs
7188    msedge.exe      21      SeAuditPrivilege                Generate security audits
7188    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
7188    msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7188    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7188    msedge.exe      25      SeUndockPrivilege       Present Remove computer from docking station
7188    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
7188    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7188    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
7188    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
7188    msedge.exe      30      SeCreateGlobalPrivilege Default Create global objects
7188    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7188    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7188    msedge.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
7188    msedge.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
7188    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
7188    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
12780   msedge.exe      2       SeCreateTokenPrivilege          Create a token object
12780   msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
12780   msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
12780   msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
12780   msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
12780   msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
12780   msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
12780   msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
12780   msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
12780   msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
12780   msedge.exe      12      SeSystemtimePrivilege           Change the system time
12780   msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
12780   msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
12780   msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
12780   msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
12780   msedge.exe      17      SeBackupPrivilege               Backup files and directories
12780   msedge.exe      18      SeRestorePrivilege              Restore files and directories
12780   msedge.exe      19      SeShutdownPrivilege             Shut down the system
12780   msedge.exe      20      SeDebugPrivilege                Debug programs
12780   msedge.exe      21      SeAuditPrivilege                Generate security audits
12780   msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
12780   msedge.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
12780   msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
12780   msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
12780   msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
12780   msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
12780   msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
12780   msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
12780   msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
12780   msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
12780   msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
12780   msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
12780   msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
12780   msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
12780   msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
12440   svchost.exe     2       SeCreateTokenPrivilege          Create a token object
12440   svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
12440   svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
12440   svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
12440   svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
12440   svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
12440   svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
12440   svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
12440   svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
12440   svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
12440   svchost.exe     12      SeSystemtimePrivilege           Change the system time
12440   svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
12440   svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
12440   svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
12440   svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
12440   svchost.exe     17      SeBackupPrivilege               Backup files and directories
12440   svchost.exe     18      SeRestorePrivilege              Restore files and directories
12440   svchost.exe     19      SeShutdownPrivilege             Shut down the system
12440   svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
12440   svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
12440   svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
12440   svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
12440   svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
12440   svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
12440   svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
12440   svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
12440   svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
12440   svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
12440   svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
12440   svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
12440   svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
12440   svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
12440   svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
12440   svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
12440   svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
1336    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
1336    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
1336    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
1336    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
1336    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
1336    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
1336    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
1336    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
1336    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
1336    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
1336    msedge.exe      12      SeSystemtimePrivilege           Change the system time
1336    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
1336    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
1336    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
1336    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
1336    msedge.exe      17      SeBackupPrivilege               Backup files and directories
1336    msedge.exe      18      SeRestorePrivilege              Restore files and directories
1336    msedge.exe      19      SeShutdownPrivilege     Present Shut down the system
1336    msedge.exe      20      SeDebugPrivilege                Debug programs
1336    msedge.exe      21      SeAuditPrivilege                Generate security audits
1336    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
1336    msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1336    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1336    msedge.exe      25      SeUndockPrivilege       Present Remove computer from docking station
1336    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
1336    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1336    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
1336    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
1336    msedge.exe      30      SeCreateGlobalPrivilege Default Create global objects
1336    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1336    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1336    msedge.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
1336    msedge.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
1336    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
1336    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
5808    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
5808    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
5808    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
5808    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
5808    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
5808    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
5808    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
5808    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
5808    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
5808    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
5808    msedge.exe      12      SeSystemtimePrivilege           Change the system time
5808    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
5808    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
5808    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
5808    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
5808    msedge.exe      17      SeBackupPrivilege               Backup files and directories
5808    msedge.exe      18      SeRestorePrivilege              Restore files and directories
5808    msedge.exe      19      SeShutdownPrivilege     Present Shut down the system
5808    msedge.exe      20      SeDebugPrivilege                Debug programs
5808    msedge.exe      21      SeAuditPrivilege                Generate security audits
5808    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5808    msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
5808    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5808    msedge.exe      25      SeUndockPrivilege       Present Remove computer from docking station
5808    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
5808    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5808    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
5808    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
5808    msedge.exe      30      SeCreateGlobalPrivilege Default Create global objects
5808    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5808    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5808    msedge.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
5808    msedge.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
5808    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
5808    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
13488   msedge.exe      2       SeCreateTokenPrivilege          Create a token object
13488   msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
13488   msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
13488   msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
13488   msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
13488   msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
13488   msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
13488   msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
13488   msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
13488   msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
13488   msedge.exe      12      SeSystemtimePrivilege           Change the system time
13488   msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
13488   msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
13488   msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
13488   msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
13488   msedge.exe      17      SeBackupPrivilege               Backup files and directories
13488   msedge.exe      18      SeRestorePrivilege              Restore files and directories
13488   msedge.exe      19      SeShutdownPrivilege             Shut down the system
13488   msedge.exe      20      SeDebugPrivilege                Debug programs
13488   msedge.exe      21      SeAuditPrivilege                Generate security audits
13488   msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
13488   msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
13488   msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
13488   msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
13488   msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
13488   msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
13488   msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
13488   msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
13488   msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
13488   msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
13488   msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
13488   msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
13488   msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
13488   msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
13488   msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
12476   msedge.exe      2       SeCreateTokenPrivilege          Create a token object
12476   msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
12476   msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
12476   msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
12476   msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
12476   msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
12476   msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
12476   msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
12476   msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
12476   msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
12476   msedge.exe      12      SeSystemtimePrivilege           Change the system time
12476   msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
12476   msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
12476   msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
12476   msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
12476   msedge.exe      17      SeBackupPrivilege               Backup files and directories
12476   msedge.exe      18      SeRestorePrivilege              Restore files and directories
12476   msedge.exe      19      SeShutdownPrivilege     Present Shut down the system
12476   msedge.exe      20      SeDebugPrivilege                Debug programs
12476   msedge.exe      21      SeAuditPrivilege                Generate security audits
12476   msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
12476   msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
12476   msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
12476   msedge.exe      25      SeUndockPrivilege       Present Remove computer from docking station
12476   msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
12476   msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
12476   msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
12476   msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
12476   msedge.exe      30      SeCreateGlobalPrivilege Default Create global objects
12476   msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
12476   msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
12476   msedge.exe      33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
12476   msedge.exe      34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
12476   msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
12476   msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
6408    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
6408    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
6408    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
6408    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
6408    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
6408    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
6408    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
6408    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
6408    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
6408    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
6408    msedge.exe      12      SeSystemtimePrivilege           Change the system time
6408    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
6408    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
6408    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
6408    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
6408    msedge.exe      17      SeBackupPrivilege               Backup files and directories
6408    msedge.exe      18      SeRestorePrivilege              Restore files and directories
6408    msedge.exe      19      SeShutdownPrivilege             Shut down the system
6408    msedge.exe      20      SeDebugPrivilege                Debug programs
6408    msedge.exe      21      SeAuditPrivilege                Generate security audits
6408    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
6408    msedge.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
6408    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
6408    msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
6408    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
6408    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
6408    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
6408    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
6408    msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
6408    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
6408    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
6408    msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
6408    msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
6408    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
6408    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
13964   msedge.exe      2       SeCreateTokenPrivilege          Create a token object
13964   msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
13964   msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
13964   msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
13964   msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
13964   msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
13964   msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
13964   msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
13964   msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
13964   msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
13964   msedge.exe      12      SeSystemtimePrivilege           Change the system time
13964   msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
13964   msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
13964   msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
13964   msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
13964   msedge.exe      17      SeBackupPrivilege               Backup files and directories
13964   msedge.exe      18      SeRestorePrivilege              Restore files and directories
13964   msedge.exe      19      SeShutdownPrivilege             Shut down the system
13964   msedge.exe      20      SeDebugPrivilege                Debug programs
13964   msedge.exe      21      SeAuditPrivilege                Generate security audits
13964   msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
13964   msedge.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
13964   msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
13964   msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
13964   msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
13964   msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
13964   msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
13964   msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
13964   msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
13964   msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
13964   msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
13964   msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
13964   msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
13964   msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
13964   msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
13620   splunkd.exe     2       SeCreateTokenPrivilege          Create a token object
13620   splunkd.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
13620   splunkd.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
13620   splunkd.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
13620   splunkd.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
13620   splunkd.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
13620   splunkd.exe     8       SeSecurityPrivilege     Present,Enabled Manage auditing and security log
13620   splunkd.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
13620   splunkd.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
13620   splunkd.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
13620   splunkd.exe     12      SeSystemtimePrivilege   Present Change the system time
13620   splunkd.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
13620   splunkd.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
13620   splunkd.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
13620   splunkd.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
13620   splunkd.exe     17      SeBackupPrivilege       Present,Enabled Backup files and directories
13620   splunkd.exe     18      SeRestorePrivilege      Present Restore files and directories
13620   splunkd.exe     19      SeShutdownPrivilege     Present Shut down the system
13620   splunkd.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
13620   splunkd.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
13620   splunkd.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
13620   splunkd.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
13620   splunkd.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
13620   splunkd.exe     25      SeUndockPrivilege       Present Remove computer from docking station
13620   splunkd.exe     26      SeSyncAgentPrivilege            Synch directory service data
13620   splunkd.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
13620   splunkd.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
13620   splunkd.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
13620   splunkd.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
13620   splunkd.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
13620   splunkd.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
13620   splunkd.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
13620   splunkd.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
13620   splunkd.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
13620   splunkd.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
1600    conhost.exe     2       SeCreateTokenPrivilege          Create a token object
1600    conhost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1600    conhost.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
1600    conhost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
1600    conhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
1600    conhost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
1600    conhost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
1600    conhost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
1600    conhost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
1600    conhost.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
1600    conhost.exe     12      SeSystemtimePrivilege   Present Change the system time
1600    conhost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
1600    conhost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
1600    conhost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
1600    conhost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
1600    conhost.exe     17      SeBackupPrivilege       Present Backup files and directories
1600    conhost.exe     18      SeRestorePrivilege      Present Restore files and directories
1600    conhost.exe     19      SeShutdownPrivilege     Present Shut down the system
1600    conhost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1600    conhost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
1600    conhost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
1600    conhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1600    conhost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1600    conhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
1600    conhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
1600    conhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1600    conhost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
1600    conhost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1600    conhost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1600    conhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
1600    conhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1600    conhost.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
1600    conhost.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
1600    conhost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
1600    conhost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
11256   splunk-winevtl  2       SeCreateTokenPrivilege          Create a token object
11256   splunk-winevtl  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
11256   splunk-winevtl  4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
11256   splunk-winevtl  5       SeIncreaseQuotaPrivilege        Present Increase quotas
11256   splunk-winevtl  6       SeMachineAccountPrivilege               Add workstations to the domain
11256   splunk-winevtl  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
11256   splunk-winevtl  8       SeSecurityPrivilege     Present,Enabled Manage auditing and security log
11256   splunk-winevtl  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
11256   splunk-winevtl  10      SeLoadDriverPrivilege   Present Load and unload device drivers
11256   splunk-winevtl  11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
11256   splunk-winevtl  12      SeSystemtimePrivilege   Present Change the system time
11256   splunk-winevtl  13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
11256   splunk-winevtl  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
11256   splunk-winevtl  15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
11256   splunk-winevtl  16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
11256   splunk-winevtl  17      SeBackupPrivilege       Present,Enabled Backup files and directories
11256   splunk-winevtl  18      SeRestorePrivilege      Present Restore files and directories
11256   splunk-winevtl  19      SeShutdownPrivilege     Present Shut down the system
11256   splunk-winevtl  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
11256   splunk-winevtl  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
11256   splunk-winevtl  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
11256   splunk-winevtl  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
11256   splunk-winevtl  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
11256   splunk-winevtl  25      SeUndockPrivilege       Present Remove computer from docking station
11256   splunk-winevtl  26      SeSyncAgentPrivilege            Synch directory service data
11256   splunk-winevtl  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
11256   splunk-winevtl  28      SeManageVolumePrivilege Present Manage the files on a volume
11256   splunk-winevtl  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
11256   splunk-winevtl  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
11256   splunk-winevtl  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
11256   splunk-winevtl  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
11256   splunk-winevtl  33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
11256   splunk-winevtl  34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
11256   splunk-winevtl  35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
11256   splunk-winevtl  36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
10144   MoUsoCoreWorke  2       SeCreateTokenPrivilege          Create a token object
10144   MoUsoCoreWorke  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
10144   MoUsoCoreWorke  4       SeLockMemoryPrivilege   Default Lock pages in memory
10144   MoUsoCoreWorke  5       SeIncreaseQuotaPrivilege        Present Increase quotas
10144   MoUsoCoreWorke  6       SeMachineAccountPrivilege               Add workstations to the domain
10144   MoUsoCoreWorke  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
10144   MoUsoCoreWorke  8       SeSecurityPrivilege     Present Manage auditing and security log
10144   MoUsoCoreWorke  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
10144   MoUsoCoreWorke  10      SeLoadDriverPrivilege   Present Load and unload device drivers
10144   MoUsoCoreWorke  11      SeSystemProfilePrivilege        Default Profile system performance
10144   MoUsoCoreWorke  12      SeSystemtimePrivilege           Change the system time
10144   MoUsoCoreWorke  13      SeProfileSingleProcessPrivilege Default Profile a single process
10144   MoUsoCoreWorke  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
10144   MoUsoCoreWorke  15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
10144   MoUsoCoreWorke  16      SeCreatePermanentPrivilege      Default Create permanent shared objects
10144   MoUsoCoreWorke  17      SeBackupPrivilege       Present Backup files and directories
10144   MoUsoCoreWorke  18      SeRestorePrivilege      Present Restore files and directories
10144   MoUsoCoreWorke  19      SeShutdownPrivilege     Present Shut down the system
10144   MoUsoCoreWorke  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
10144   MoUsoCoreWorke  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
10144   MoUsoCoreWorke  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
10144   MoUsoCoreWorke  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10144   MoUsoCoreWorke  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10144   MoUsoCoreWorke  25      SeUndockPrivilege               Remove computer from docking station
10144   MoUsoCoreWorke  26      SeSyncAgentPrivilege            Synch directory service data
10144   MoUsoCoreWorke  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10144   MoUsoCoreWorke  28      SeManageVolumePrivilege Present Manage the files on a volume
10144   MoUsoCoreWorke  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
10144   MoUsoCoreWorke  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
10144   MoUsoCoreWorke  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10144   MoUsoCoreWorke  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10144   MoUsoCoreWorke  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
10144   MoUsoCoreWorke  34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
10144   MoUsoCoreWorke  35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
10144   MoUsoCoreWorke  36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
11776   svchost.exe     2       SeCreateTokenPrivilege          Create a token object
11776   svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
11776   svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
11776   svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
11776   svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
11776   svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
11776   svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
11776   svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
11776   svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
11776   svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
11776   svchost.exe     12      SeSystemtimePrivilege           Change the system time
11776   svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
11776   svchost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
11776   svchost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
11776   svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
11776   svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
11776   svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
11776   svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
11776   svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
11776   svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
11776   svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
11776   svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
11776   svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
11776   svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
11776   svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
11776   svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
11776   svchost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
11776   svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
11776   svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
11776   svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
11776   svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
11776   svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
11776   svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
11776   svchost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
11776   svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
5148    msedge.exe      2       SeCreateTokenPrivilege          Create a token object
5148    msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
5148    msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
5148    msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
5148    msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
5148    msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
5148    msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
5148    msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
5148    msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
5148    msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
5148    msedge.exe      12      SeSystemtimePrivilege           Change the system time
5148    msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
5148    msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
5148    msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
5148    msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
5148    msedge.exe      17      SeBackupPrivilege               Backup files and directories
5148    msedge.exe      18      SeRestorePrivilege              Restore files and directories
5148    msedge.exe      19      SeShutdownPrivilege             Shut down the system
5148    msedge.exe      20      SeDebugPrivilege                Debug programs
5148    msedge.exe      21      SeAuditPrivilege                Generate security audits
5148    msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5148    msedge.exe      23      SeChangeNotifyPrivilege Default Receive notifications of changes to files or directories
5148    msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5148    msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
5148    msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
5148    msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5148    msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
5148    msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
5148    msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
5148    msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5148    msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5148    msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
5148    msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
5148    msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
5148    msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
9444    svchost.exe     2       SeCreateTokenPrivilege          Create a token object
9444    svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
9444    svchost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
9444    svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
9444    svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
9444    svchost.exe     7       SeTcbPrivilege          Act as part of the operating system
9444    svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
9444    svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
9444    svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
9444    svchost.exe     11      SeSystemProfilePrivilege                Profile system performance
9444    svchost.exe     12      SeSystemtimePrivilege   Present,Enabled Change the system time
9444    svchost.exe     13      SeProfileSingleProcessPrivilege         Profile a single process
9444    svchost.exe     14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
9444    svchost.exe     15      SeCreatePagefilePrivilege               Create a pagefile
9444    svchost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
9444    svchost.exe     17      SeBackupPrivilege               Backup files and directories
9444    svchost.exe     18      SeRestorePrivilege              Restore files and directories
9444    svchost.exe     19      SeShutdownPrivilege             Shut down the system
9444    svchost.exe     20      SeDebugPrivilege                Debug programs
9444    svchost.exe     21      SeAuditPrivilege        Present Generate security audits
9444    svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
9444    svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
9444    svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
9444    svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
9444    svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
9444    svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
9444    svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
9444    svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
9444    svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
9444    svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
9444    svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
9444    svchost.exe     33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
9444    svchost.exe     34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
9444    svchost.exe     35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
9444    svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
5176    Microsoft.Shar  2       SeCreateTokenPrivilege          Create a token object
5176    Microsoft.Shar  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
5176    Microsoft.Shar  4       SeLockMemoryPrivilege           Lock pages in memory
5176    Microsoft.Shar  5       SeIncreaseQuotaPrivilege                Increase quotas
5176    Microsoft.Shar  6       SeMachineAccountPrivilege               Add workstations to the domain
5176    Microsoft.Shar  7       SeTcbPrivilege          Act as part of the operating system
5176    Microsoft.Shar  8       SeSecurityPrivilege             Manage auditing and security log
5176    Microsoft.Shar  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
5176    Microsoft.Shar  10      SeLoadDriverPrivilege           Load and unload device drivers
5176    Microsoft.Shar  11      SeSystemProfilePrivilege                Profile system performance
5176    Microsoft.Shar  12      SeSystemtimePrivilege           Change the system time
5176    Microsoft.Shar  13      SeProfileSingleProcessPrivilege         Profile a single process
5176    Microsoft.Shar  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
5176    Microsoft.Shar  15      SeCreatePagefilePrivilege               Create a pagefile
5176    Microsoft.Shar  16      SeCreatePermanentPrivilege              Create permanent shared objects
5176    Microsoft.Shar  17      SeBackupPrivilege               Backup files and directories
5176    Microsoft.Shar  18      SeRestorePrivilege              Restore files and directories
5176    Microsoft.Shar  19      SeShutdownPrivilege     Present Shut down the system
5176    Microsoft.Shar  20      SeDebugPrivilege                Debug programs
5176    Microsoft.Shar  21      SeAuditPrivilege                Generate security audits
5176    Microsoft.Shar  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
5176    Microsoft.Shar  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
5176    Microsoft.Shar  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
5176    Microsoft.Shar  25      SeUndockPrivilege       Present Remove computer from docking station
5176    Microsoft.Shar  26      SeSyncAgentPrivilege            Synch directory service data
5176    Microsoft.Shar  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
5176    Microsoft.Shar  28      SeManageVolumePrivilege         Manage the files on a volume
5176    Microsoft.Shar  29      SeImpersonatePrivilege          Impersonate a client after authentication
5176    Microsoft.Shar  30      SeCreateGlobalPrivilege Default Create global objects
5176    Microsoft.Shar  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
5176    Microsoft.Shar  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
5176    Microsoft.Shar  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
5176    Microsoft.Shar  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
5176    Microsoft.Shar  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
5176    Microsoft.Shar  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
16344   msedge.exe      2       SeCreateTokenPrivilege          Create a token object
16344   msedge.exe      3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
16344   msedge.exe      4       SeLockMemoryPrivilege           Lock pages in memory
16344   msedge.exe      5       SeIncreaseQuotaPrivilege                Increase quotas
16344   msedge.exe      6       SeMachineAccountPrivilege               Add workstations to the domain
16344   msedge.exe      7       SeTcbPrivilege          Act as part of the operating system
16344   msedge.exe      8       SeSecurityPrivilege             Manage auditing and security log
16344   msedge.exe      9       SeTakeOwnershipPrivilege                Take ownership of files/objects
16344   msedge.exe      10      SeLoadDriverPrivilege           Load and unload device drivers
16344   msedge.exe      11      SeSystemProfilePrivilege                Profile system performance
16344   msedge.exe      12      SeSystemtimePrivilege           Change the system time
16344   msedge.exe      13      SeProfileSingleProcessPrivilege         Profile a single process
16344   msedge.exe      14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
16344   msedge.exe      15      SeCreatePagefilePrivilege               Create a pagefile
16344   msedge.exe      16      SeCreatePermanentPrivilege              Create permanent shared objects
16344   msedge.exe      17      SeBackupPrivilege               Backup files and directories
16344   msedge.exe      18      SeRestorePrivilege              Restore files and directories
16344   msedge.exe      19      SeShutdownPrivilege             Shut down the system
16344   msedge.exe      20      SeDebugPrivilege                Debug programs
16344   msedge.exe      21      SeAuditPrivilege                Generate security audits
16344   msedge.exe      22      SeSystemEnvironmentPrivilege            Edit firmware environment values
16344   msedge.exe      23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
16344   msedge.exe      24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
16344   msedge.exe      25      SeUndockPrivilege               Remove computer from docking station
16344   msedge.exe      26      SeSyncAgentPrivilege            Synch directory service data
16344   msedge.exe      27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
16344   msedge.exe      28      SeManageVolumePrivilege         Manage the files on a volume
16344   msedge.exe      29      SeImpersonatePrivilege          Impersonate a client after authentication
16344   msedge.exe      30      SeCreateGlobalPrivilege         Create global objects
16344   msedge.exe      31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
16344   msedge.exe      32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
16344   msedge.exe      33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
16344   msedge.exe      34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
16344   msedge.exe      35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
16344   msedge.exe      36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
15568   SenseNdr.exe    2       SeCreateTokenPrivilege          Create a token object
15568   SenseNdr.exe    3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
15568   SenseNdr.exe    4       SeLockMemoryPrivilege           Lock pages in memory
15568   SenseNdr.exe    5       SeIncreaseQuotaPrivilege                Increase quotas
15568   SenseNdr.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
15568   SenseNdr.exe    7       SeTcbPrivilege          Act as part of the operating system
15568   SenseNdr.exe    8       SeSecurityPrivilege             Manage auditing and security log
15568   SenseNdr.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
15568   SenseNdr.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
15568   SenseNdr.exe    11      SeSystemProfilePrivilege                Profile system performance
15568   SenseNdr.exe    12      SeSystemtimePrivilege           Change the system time
15568   SenseNdr.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
15568   SenseNdr.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
15568   SenseNdr.exe    15      SeCreatePagefilePrivilege               Create a pagefile
15568   SenseNdr.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
15568   SenseNdr.exe    17      SeBackupPrivilege               Backup files and directories
15568   SenseNdr.exe    18      SeRestorePrivilege              Restore files and directories
15568   SenseNdr.exe    19      SeShutdownPrivilege             Shut down the system
15568   SenseNdr.exe    20      SeDebugPrivilege                Debug programs
15568   SenseNdr.exe    21      SeAuditPrivilege                Generate security audits
15568   SenseNdr.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
15568   SenseNdr.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
15568   SenseNdr.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
15568   SenseNdr.exe    25      SeUndockPrivilege               Remove computer from docking station
15568   SenseNdr.exe    26      SeSyncAgentPrivilege            Synch directory service data
15568   SenseNdr.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
15568   SenseNdr.exe    28      SeManageVolumePrivilege         Manage the files on a volume
15568   SenseNdr.exe    29      SeImpersonatePrivilege          Impersonate a client after authentication
15568   SenseNdr.exe    30      SeCreateGlobalPrivilege         Create global objects
15568   SenseNdr.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
15568   SenseNdr.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
15568   SenseNdr.exe    33      SeIncreaseWorkingSetPrivilege           Allocate more memory for user applications
15568   SenseNdr.exe    34      SeTimeZonePrivilege             Adjust the time zone of the computer's internal clock
15568   SenseNdr.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
15568   SenseNdr.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
15508   svchost.exe     2       SeCreateTokenPrivilege          Create a token object
15508   svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
15508   svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
15508   svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
15508   svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
15508   svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
15508   svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
15508   svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
15508   svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
15508   svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
15508   svchost.exe     12      SeSystemtimePrivilege           Change the system time
15508   svchost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
15508   svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
15508   svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
15508   svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
15508   svchost.exe     17      SeBackupPrivilege               Backup files and directories
15508   svchost.exe     18      SeRestorePrivilege              Restore files and directories
15508   svchost.exe     19      SeShutdownPrivilege             Shut down the system
15508   svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
15508   svchost.exe     21      SeAuditPrivilege        Default Generate security audits
15508   svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
15508   svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
15508   svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
15508   svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
15508   svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
15508   svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
15508   svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
15508   svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
15508   svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
15508   svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
15508   svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
15508   svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
15508   svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
15508   svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
15508   svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
11296   PSEXESVC.exe    2       SeCreateTokenPrivilege          Create a token object
11296   PSEXESVC.exe    3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
11296   PSEXESVC.exe    4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
11296   PSEXESVC.exe    5       SeIncreaseQuotaPrivilege        Present Increase quotas
11296   PSEXESVC.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
11296   PSEXESVC.exe    7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
11296   PSEXESVC.exe    8       SeSecurityPrivilege     Present Manage auditing and security log
11296   PSEXESVC.exe    9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
11296   PSEXESVC.exe    10      SeLoadDriverPrivilege   Present Load and unload device drivers
11296   PSEXESVC.exe    11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
11296   PSEXESVC.exe    12      SeSystemtimePrivilege   Present Change the system time
11296   PSEXESVC.exe    13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
11296   PSEXESVC.exe    14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
11296   PSEXESVC.exe    15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
11296   PSEXESVC.exe    16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
11296   PSEXESVC.exe    17      SeBackupPrivilege       Present Backup files and directories
11296   PSEXESVC.exe    18      SeRestorePrivilege      Present Restore files and directories
11296   PSEXESVC.exe    19      SeShutdownPrivilege     Present Shut down the system
11296   PSEXESVC.exe    20      SeDebugPrivilege        Present,Enabled,Default Debug programs
11296   PSEXESVC.exe    21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
11296   PSEXESVC.exe    22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
11296   PSEXESVC.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
11296   PSEXESVC.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
11296   PSEXESVC.exe    25      SeUndockPrivilege       Present Remove computer from docking station
11296   PSEXESVC.exe    26      SeSyncAgentPrivilege            Synch directory service data
11296   PSEXESVC.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
11296   PSEXESVC.exe    28      SeManageVolumePrivilege Present Manage the files on a volume
11296   PSEXESVC.exe    29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
11296   PSEXESVC.exe    30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
11296   PSEXESVC.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
11296   PSEXESVC.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
11296   PSEXESVC.exe    33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
11296   PSEXESVC.exe    34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
11296   PSEXESVC.exe    35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
11296   PSEXESVC.exe    36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
4672    excel-update.e  2       SeCreateTokenPrivilege          Create a token object
4672    excel-update.e  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
4672    excel-update.e  4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
4672    excel-update.e  5       SeIncreaseQuotaPrivilege        Present Increase quotas
4672    excel-update.e  6       SeMachineAccountPrivilege               Add workstations to the domain
4672    excel-update.e  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
4672    excel-update.e  8       SeSecurityPrivilege     Present Manage auditing and security log
4672    excel-update.e  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
4672    excel-update.e  10      SeLoadDriverPrivilege   Present Load and unload device drivers
4672    excel-update.e  11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
4672    excel-update.e  12      SeSystemtimePrivilege   Present Change the system time
4672    excel-update.e  13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
4672    excel-update.e  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
4672    excel-update.e  15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
4672    excel-update.e  16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
4672    excel-update.e  17      SeBackupPrivilege       Present Backup files and directories
4672    excel-update.e  18      SeRestorePrivilege      Present Restore files and directories
4672    excel-update.e  19      SeShutdownPrivilege     Present Shut down the system
4672    excel-update.e  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
4672    excel-update.e  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
4672    excel-update.e  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
4672    excel-update.e  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
4672    excel-update.e  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
4672    excel-update.e  25      SeUndockPrivilege       Present Remove computer from docking station
4672    excel-update.e  26      SeSyncAgentPrivilege            Synch directory service data
4672    excel-update.e  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
4672    excel-update.e  28      SeManageVolumePrivilege Present Manage the files on a volume
4672    excel-update.e  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
4672    excel-update.e  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
4672    excel-update.e  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
4672    excel-update.e  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
4672    excel-update.e  33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
4672    excel-update.e  34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
4672    excel-update.e  35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
4672    excel-update.e  36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
13752   cmd.exe 2       SeCreateTokenPrivilege          Create a token object
13752   cmd.exe 3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
13752   cmd.exe 4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
13752   cmd.exe 5       SeIncreaseQuotaPrivilege        Present Increase quotas
13752   cmd.exe 6       SeMachineAccountPrivilege               Add workstations to the domain
13752   cmd.exe 7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
13752   cmd.exe 8       SeSecurityPrivilege     Present Manage auditing and security log
13752   cmd.exe 9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
13752   cmd.exe 10      SeLoadDriverPrivilege   Present Load and unload device drivers
13752   cmd.exe 11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
13752   cmd.exe 12      SeSystemtimePrivilege   Present Change the system time
13752   cmd.exe 13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
13752   cmd.exe 14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
13752   cmd.exe 15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
13752   cmd.exe 16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
13752   cmd.exe 17      SeBackupPrivilege       Present Backup files and directories
13752   cmd.exe 18      SeRestorePrivilege      Present Restore files and directories
13752   cmd.exe 19      SeShutdownPrivilege     Present Shut down the system
13752   cmd.exe 20      SeDebugPrivilege        Present,Enabled,Default Debug programs
13752   cmd.exe 21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
13752   cmd.exe 22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
13752   cmd.exe 23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
13752   cmd.exe 24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
13752   cmd.exe 25      SeUndockPrivilege       Present Remove computer from docking station
13752   cmd.exe 26      SeSyncAgentPrivilege            Synch directory service data
13752   cmd.exe 27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
13752   cmd.exe 28      SeManageVolumePrivilege Present Manage the files on a volume
13752   cmd.exe 29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
13752   cmd.exe 30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
13752   cmd.exe 31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
13752   cmd.exe 32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
13752   cmd.exe 33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
13752   cmd.exe 34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
13752   cmd.exe 35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
13752   cmd.exe 36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
12392   conhost.exe     2       SeCreateTokenPrivilege          Create a token object
12392   conhost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
12392   conhost.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
12392   conhost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
12392   conhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
12392   conhost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
12392   conhost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
12392   conhost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
12392   conhost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
12392   conhost.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
12392   conhost.exe     12      SeSystemtimePrivilege   Present Change the system time
12392   conhost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
12392   conhost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
12392   conhost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
12392   conhost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
12392   conhost.exe     17      SeBackupPrivilege       Present Backup files and directories
12392   conhost.exe     18      SeRestorePrivilege      Present Restore files and directories
12392   conhost.exe     19      SeShutdownPrivilege     Present Shut down the system
12392   conhost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
12392   conhost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
12392   conhost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
12392   conhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
12392   conhost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
12392   conhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
12392   conhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
12392   conhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
12392   conhost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
12392   conhost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
12392   conhost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
12392   conhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
12392   conhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
12392   conhost.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
12392   conhost.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
12392   conhost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
12392   conhost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
6988    ScreenConnect.  2       SeCreateTokenPrivilege          Create a token object
6988    ScreenConnect.  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
6988    ScreenConnect.  4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
6988    ScreenConnect.  5       SeIncreaseQuotaPrivilege        Present Increase quotas
6988    ScreenConnect.  6       SeMachineAccountPrivilege               Add workstations to the domain
6988    ScreenConnect.  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
6988    ScreenConnect.  8       SeSecurityPrivilege     Present Manage auditing and security log
6988    ScreenConnect.  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
6988    ScreenConnect.  10      SeLoadDriverPrivilege   Present Load and unload device drivers
6988    ScreenConnect.  11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
6988    ScreenConnect.  12      SeSystemtimePrivilege   Present Change the system time
6988    ScreenConnect.  13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
6988    ScreenConnect.  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
6988    ScreenConnect.  15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
6988    ScreenConnect.  16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
6988    ScreenConnect.  17      SeBackupPrivilege       Present Backup files and directories
6988    ScreenConnect.  18      SeRestorePrivilege      Present Restore files and directories
6988    ScreenConnect.  19      SeShutdownPrivilege     Present Shut down the system
6988    ScreenConnect.  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
6988    ScreenConnect.  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
6988    ScreenConnect.  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
6988    ScreenConnect.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
6988    ScreenConnect.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
6988    ScreenConnect.  25      SeUndockPrivilege       Present Remove computer from docking station
6988    ScreenConnect.  26      SeSyncAgentPrivilege            Synch directory service data
6988    ScreenConnect.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
6988    ScreenConnect.  28      SeManageVolumePrivilege Present Manage the files on a volume
6988    ScreenConnect.  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
6988    ScreenConnect.  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
6988    ScreenConnect.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
6988    ScreenConnect.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
6988    ScreenConnect.  33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
6988    ScreenConnect.  34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
6988    ScreenConnect.  35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
6988    ScreenConnect.  36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
14756   ScreenConnect.  2       SeCreateTokenPrivilege          Create a token object
14756   ScreenConnect.  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
14756   ScreenConnect.  4       SeLockMemoryPrivilege           Lock pages in memory
14756   ScreenConnect.  5       SeIncreaseQuotaPrivilege                Increase quotas
14756   ScreenConnect.  6       SeMachineAccountPrivilege               Add workstations to the domain
14756   ScreenConnect.  7       SeTcbPrivilege          Act as part of the operating system
14756   ScreenConnect.  8       SeSecurityPrivilege             Manage auditing and security log
14756   ScreenConnect.  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
14756   ScreenConnect.  10      SeLoadDriverPrivilege           Load and unload device drivers
14756   ScreenConnect.  11      SeSystemProfilePrivilege                Profile system performance
14756   ScreenConnect.  12      SeSystemtimePrivilege           Change the system time
14756   ScreenConnect.  13      SeProfileSingleProcessPrivilege         Profile a single process
14756   ScreenConnect.  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
14756   ScreenConnect.  15      SeCreatePagefilePrivilege               Create a pagefile
14756   ScreenConnect.  16      SeCreatePermanentPrivilege              Create permanent shared objects
14756   ScreenConnect.  17      SeBackupPrivilege               Backup files and directories
14756   ScreenConnect.  18      SeRestorePrivilege              Restore files and directories
14756   ScreenConnect.  19      SeShutdownPrivilege     Present Shut down the system
14756   ScreenConnect.  20      SeDebugPrivilege                Debug programs
14756   ScreenConnect.  21      SeAuditPrivilege                Generate security audits
14756   ScreenConnect.  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
14756   ScreenConnect.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
14756   ScreenConnect.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
14756   ScreenConnect.  25      SeUndockPrivilege       Present Remove computer from docking station
14756   ScreenConnect.  26      SeSyncAgentPrivilege            Synch directory service data
14756   ScreenConnect.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
14756   ScreenConnect.  28      SeManageVolumePrivilege         Manage the files on a volume
14756   ScreenConnect.  29      SeImpersonatePrivilege          Impersonate a client after authentication
14756   ScreenConnect.  30      SeCreateGlobalPrivilege Default Create global objects
14756   ScreenConnect.  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
14756   ScreenConnect.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
14756   ScreenConnect.  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
14756   ScreenConnect.  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
14756   ScreenConnect.  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
14756   ScreenConnect.  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
1772    ScreenConnect.  2       SeCreateTokenPrivilege          Create a token object
1772    ScreenConnect.  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
1772    ScreenConnect.  4       SeLockMemoryPrivilege   Default Lock pages in memory
1772    ScreenConnect.  5       SeIncreaseQuotaPrivilege        Present Increase quotas
1772    ScreenConnect.  6       SeMachineAccountPrivilege               Add workstations to the domain
1772    ScreenConnect.  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
1772    ScreenConnect.  8       SeSecurityPrivilege     Present Manage auditing and security log
1772    ScreenConnect.  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
1772    ScreenConnect.  10      SeLoadDriverPrivilege   Present Load and unload device drivers
1772    ScreenConnect.  11      SeSystemProfilePrivilege        Default Profile system performance
1772    ScreenConnect.  12      SeSystemtimePrivilege           Change the system time
1772    ScreenConnect.  13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
1772    ScreenConnect.  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
1772    ScreenConnect.  15      SeCreatePagefilePrivilege       Default Create a pagefile
1772    ScreenConnect.  16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
1772    ScreenConnect.  17      SeBackupPrivilege       Present Backup files and directories
1772    ScreenConnect.  18      SeRestorePrivilege      Present Restore files and directories
1772    ScreenConnect.  19      SeShutdownPrivilege     Present Shut down the system
1772    ScreenConnect.  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
1772    ScreenConnect.  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
1772    ScreenConnect.  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
1772    ScreenConnect.  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
1772    ScreenConnect.  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
1772    ScreenConnect.  25      SeUndockPrivilege       Present Remove computer from docking station
1772    ScreenConnect.  26      SeSyncAgentPrivilege            Synch directory service data
1772    ScreenConnect.  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
1772    ScreenConnect.  28      SeManageVolumePrivilege Present Manage the files on a volume
1772    ScreenConnect.  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
1772    ScreenConnect.  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
1772    ScreenConnect.  31      SeTrustedCredManAccessPrivilege Present Access Credential Manager as a trusted caller
1772    ScreenConnect.  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
1772    ScreenConnect.  33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
1772    ScreenConnect.  34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
1772    ScreenConnect.  35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
1772    ScreenConnect.  36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
15172   powershell.exe  2       SeCreateTokenPrivilege          Create a token object
15172   powershell.exe  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
15172   powershell.exe  4       SeLockMemoryPrivilege           Lock pages in memory
15172   powershell.exe  5       SeIncreaseQuotaPrivilege        Present Increase quotas
15172   powershell.exe  6       SeMachineAccountPrivilege               Add workstations to the domain
15172   powershell.exe  7       SeTcbPrivilege          Act as part of the operating system
15172   powershell.exe  8       SeSecurityPrivilege     Present Manage auditing and security log
15172   powershell.exe  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
15172   powershell.exe  10      SeLoadDriverPrivilege   Present Load and unload device drivers
15172   powershell.exe  11      SeSystemProfilePrivilege        Present Profile system performance
15172   powershell.exe  12      SeSystemtimePrivilege   Present Change the system time
15172   powershell.exe  13      SeProfileSingleProcessPrivilege Present Profile a single process
15172   powershell.exe  14      SeIncreaseBasePriorityPrivilege Present Increase scheduling priority
15172   powershell.exe  15      SeCreatePagefilePrivilege       Present Create a pagefile
15172   powershell.exe  16      SeCreatePermanentPrivilege              Create permanent shared objects
15172   powershell.exe  17      SeBackupPrivilege       Present Backup files and directories
15172   powershell.exe  18      SeRestorePrivilege      Present Restore files and directories
15172   powershell.exe  19      SeShutdownPrivilege     Present Shut down the system
15172   powershell.exe  20      SeDebugPrivilege        Present,Enabled Debug programs
15172   powershell.exe  21      SeAuditPrivilege                Generate security audits
15172   powershell.exe  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
15172   powershell.exe  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
15172   powershell.exe  24      SeRemoteShutdownPrivilege       Present Force shutdown from a remote system
15172   powershell.exe  25      SeUndockPrivilege       Present Remove computer from docking station
15172   powershell.exe  26      SeSyncAgentPrivilege            Synch directory service data
15172   powershell.exe  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
15172   powershell.exe  28      SeManageVolumePrivilege Present Manage the files on a volume
15172   powershell.exe  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
15172   powershell.exe  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
15172   powershell.exe  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
15172   powershell.exe  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
15172   powershell.exe  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
15172   powershell.exe  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
15172   powershell.exe  35      SeCreateSymbolicLinkPrivilege   Present Required to create a symbolic link
15172   powershell.exe  36      SeDelegateSessionUserImpersonatePrivilege       Present Obtain an impersonation token for another user in the same session.
13176   conhost.exe     2       SeCreateTokenPrivilege          Create a token object
13176   conhost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
13176   conhost.exe     4       SeLockMemoryPrivilege           Lock pages in memory
13176   conhost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
13176   conhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
13176   conhost.exe     7       SeTcbPrivilege          Act as part of the operating system
13176   conhost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
13176   conhost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
13176   conhost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
13176   conhost.exe     11      SeSystemProfilePrivilege        Present Profile system performance
13176   conhost.exe     12      SeSystemtimePrivilege   Present Change the system time
13176   conhost.exe     13      SeProfileSingleProcessPrivilege Present Profile a single process
13176   conhost.exe     14      SeIncreaseBasePriorityPrivilege Present Increase scheduling priority
13176   conhost.exe     15      SeCreatePagefilePrivilege       Present Create a pagefile
13176   conhost.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
13176   conhost.exe     17      SeBackupPrivilege       Present Backup files and directories
13176   conhost.exe     18      SeRestorePrivilege      Present Restore files and directories
13176   conhost.exe     19      SeShutdownPrivilege     Present Shut down the system
13176   conhost.exe     20      SeDebugPrivilege        Present Debug programs
13176   conhost.exe     21      SeAuditPrivilege                Generate security audits
13176   conhost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
13176   conhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
13176   conhost.exe     24      SeRemoteShutdownPrivilege       Present Force shutdown from a remote system
13176   conhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
13176   conhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
13176   conhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
13176   conhost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
13176   conhost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
13176   conhost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
13176   conhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
13176   conhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
13176   conhost.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
13176   conhost.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
13176   conhost.exe     35      SeCreateSymbolicLinkPrivilege   Present Required to create a symbolic link
13176   conhost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present Obtain an impersonation token for another user in the same session.
14536   spoolsv.exe     2       SeCreateTokenPrivilege          Create a token object
14536   spoolsv.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
14536   spoolsv.exe     4       SeLockMemoryPrivilege           Lock pages in memory
14536   spoolsv.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
14536   spoolsv.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
14536   spoolsv.exe     7       SeTcbPrivilege          Act as part of the operating system
14536   spoolsv.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
14536   spoolsv.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
14536   spoolsv.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
14536   spoolsv.exe     11      SeSystemProfilePrivilege        Present Profile system performance
14536   spoolsv.exe     12      SeSystemtimePrivilege   Present Change the system time
14536   spoolsv.exe     13      SeProfileSingleProcessPrivilege Present Profile a single process
14536   spoolsv.exe     14      SeIncreaseBasePriorityPrivilege Present Increase scheduling priority
14536   spoolsv.exe     15      SeCreatePagefilePrivilege       Present Create a pagefile
14536   spoolsv.exe     16      SeCreatePermanentPrivilege              Create permanent shared objects
14536   spoolsv.exe     17      SeBackupPrivilege       Present Backup files and directories
14536   spoolsv.exe     18      SeRestorePrivilege      Present Restore files and directories
14536   spoolsv.exe     19      SeShutdownPrivilege     Present Shut down the system
14536   spoolsv.exe     20      SeDebugPrivilege        Present,Enabled Debug programs
14536   spoolsv.exe     21      SeAuditPrivilege                Generate security audits
14536   spoolsv.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
14536   spoolsv.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
14536   spoolsv.exe     24      SeRemoteShutdownPrivilege       Present Force shutdown from a remote system
14536   spoolsv.exe     25      SeUndockPrivilege       Present Remove computer from docking station
14536   spoolsv.exe     26      SeSyncAgentPrivilege            Synch directory service data
14536   spoolsv.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
14536   spoolsv.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
14536   spoolsv.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
14536   spoolsv.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
14536   spoolsv.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
14536   spoolsv.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
14536   spoolsv.exe     33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
14536   spoolsv.exe     34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
14536   spoolsv.exe     35      SeCreateSymbolicLinkPrivilege   Present Required to create a symbolic link
14536   spoolsv.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present Obtain an impersonation token for another user in the same session.
6380    taskhostw.exe   2       SeCreateTokenPrivilege          Create a token object
6380    taskhostw.exe   3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
6380    taskhostw.exe   4       SeLockMemoryPrivilege           Lock pages in memory
6380    taskhostw.exe   5       SeIncreaseQuotaPrivilege        Present Increase quotas
6380    taskhostw.exe   6       SeMachineAccountPrivilege               Add workstations to the domain
6380    taskhostw.exe   7       SeTcbPrivilege          Act as part of the operating system
6380    taskhostw.exe   8       SeSecurityPrivilege     Present Manage auditing and security log
6380    taskhostw.exe   9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
6380    taskhostw.exe   10      SeLoadDriverPrivilege   Present Load and unload device drivers
6380    taskhostw.exe   11      SeSystemProfilePrivilege        Present Profile system performance
6380    taskhostw.exe   12      SeSystemtimePrivilege   Present Change the system time
6380    taskhostw.exe   13      SeProfileSingleProcessPrivilege Present Profile a single process
6380    taskhostw.exe   14      SeIncreaseBasePriorityPrivilege Present Increase scheduling priority
6380    taskhostw.exe   15      SeCreatePagefilePrivilege       Present Create a pagefile
6380    taskhostw.exe   16      SeCreatePermanentPrivilege              Create permanent shared objects
6380    taskhostw.exe   17      SeBackupPrivilege       Present Backup files and directories
6380    taskhostw.exe   18      SeRestorePrivilege      Present Restore files and directories
6380    taskhostw.exe   19      SeShutdownPrivilege     Present Shut down the system
6380    taskhostw.exe   20      SeDebugPrivilege        Present Debug programs
6380    taskhostw.exe   21      SeAuditPrivilege                Generate security audits
6380    taskhostw.exe   22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
6380    taskhostw.exe   23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
6380    taskhostw.exe   24      SeRemoteShutdownPrivilege       Present Force shutdown from a remote system
6380    taskhostw.exe   25      SeUndockPrivilege       Present Remove computer from docking station
6380    taskhostw.exe   26      SeSyncAgentPrivilege            Synch directory service data
6380    taskhostw.exe   27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
6380    taskhostw.exe   28      SeManageVolumePrivilege Present Manage the files on a volume
6380    taskhostw.exe   29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
6380    taskhostw.exe   30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
6380    taskhostw.exe   31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
6380    taskhostw.exe   32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
6380    taskhostw.exe   33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
6380    taskhostw.exe   34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
6380    taskhostw.exe   35      SeCreateSymbolicLinkPrivilege   Present Required to create a symbolic link
6380    taskhostw.exe   36      SeDelegateSessionUserImpersonatePrivilege       Present Obtain an impersonation token for another user in the same session.
15784   taskhostw.exe   2       SeCreateTokenPrivilege          Create a token object
15784   taskhostw.exe   3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
15784   taskhostw.exe   4       SeLockMemoryPrivilege           Lock pages in memory
15784   taskhostw.exe   5       SeIncreaseQuotaPrivilege        Present Increase quotas
15784   taskhostw.exe   6       SeMachineAccountPrivilege               Add workstations to the domain
15784   taskhostw.exe   7       SeTcbPrivilege          Act as part of the operating system
15784   taskhostw.exe   8       SeSecurityPrivilege     Present Manage auditing and security log
15784   taskhostw.exe   9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
15784   taskhostw.exe   10      SeLoadDriverPrivilege   Present Load and unload device drivers
15784   taskhostw.exe   11      SeSystemProfilePrivilege        Present Profile system performance
15784   taskhostw.exe   12      SeSystemtimePrivilege   Present Change the system time
15784   taskhostw.exe   13      SeProfileSingleProcessPrivilege Present Profile a single process
15784   taskhostw.exe   14      SeIncreaseBasePriorityPrivilege Present Increase scheduling priority
15784   taskhostw.exe   15      SeCreatePagefilePrivilege       Present Create a pagefile
15784   taskhostw.exe   16      SeCreatePermanentPrivilege              Create permanent shared objects
15784   taskhostw.exe   17      SeBackupPrivilege       Present Backup files and directories
15784   taskhostw.exe   18      SeRestorePrivilege      Present Restore files and directories
15784   taskhostw.exe   19      SeShutdownPrivilege     Present Shut down the system
15784   taskhostw.exe   20      SeDebugPrivilege        Present Debug programs
15784   taskhostw.exe   21      SeAuditPrivilege                Generate security audits
15784   taskhostw.exe   22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
15784   taskhostw.exe   23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
15784   taskhostw.exe   24      SeRemoteShutdownPrivilege       Present Force shutdown from a remote system
15784   taskhostw.exe   25      SeUndockPrivilege       Present Remove computer from docking station
15784   taskhostw.exe   26      SeSyncAgentPrivilege            Synch directory service data
15784   taskhostw.exe   27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
15784   taskhostw.exe   28      SeManageVolumePrivilege Present Manage the files on a volume
15784   taskhostw.exe   29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
15784   taskhostw.exe   30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
15784   taskhostw.exe   31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
15784   taskhostw.exe   32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
15784   taskhostw.exe   33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
15784   taskhostw.exe   34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
15784   taskhostw.exe   35      SeCreateSymbolicLinkPrivilege   Present Required to create a symbolic link
15784   taskhostw.exe   36      SeDelegateSessionUserImpersonatePrivilege       Present Obtain an impersonation token for another user in the same session.
14128   svchost.exe     2       SeCreateTokenPrivilege          Create a token object
14128   svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
14128   svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
14128   svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
14128   svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
14128   svchost.exe     7       SeTcbPrivilege  Default Act as part of the operating system
14128   svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
14128   svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
14128   svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
14128   svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
14128   svchost.exe     12      SeSystemtimePrivilege           Change the system time
14128   svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
14128   svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
14128   svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
14128   svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
14128   svchost.exe     17      SeBackupPrivilege               Backup files and directories
14128   svchost.exe     18      SeRestorePrivilege              Restore files and directories
14128   svchost.exe     19      SeShutdownPrivilege             Shut down the system
14128   svchost.exe     20      SeDebugPrivilege        Default Debug programs
14128   svchost.exe     21      SeAuditPrivilege        Default Generate security audits
14128   svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
14128   svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
14128   svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
14128   svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
14128   svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
14128   svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
14128   svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
14128   svchost.exe     29      SeImpersonatePrivilege  Default Impersonate a client after authentication
14128   svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
14128   svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
14128   svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
14128   svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
14128   svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
14128   svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
14128   svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
14972   CloudNotificat  2       SeCreateTokenPrivilege          Create a token object
14972   CloudNotificat  3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
14972   CloudNotificat  4       SeLockMemoryPrivilege           Lock pages in memory
14972   CloudNotificat  5       SeIncreaseQuotaPrivilege                Increase quotas
14972   CloudNotificat  6       SeMachineAccountPrivilege               Add workstations to the domain
14972   CloudNotificat  7       SeTcbPrivilege          Act as part of the operating system
14972   CloudNotificat  8       SeSecurityPrivilege             Manage auditing and security log
14972   CloudNotificat  9       SeTakeOwnershipPrivilege                Take ownership of files/objects
14972   CloudNotificat  10      SeLoadDriverPrivilege           Load and unload device drivers
14972   CloudNotificat  11      SeSystemProfilePrivilege                Profile system performance
14972   CloudNotificat  12      SeSystemtimePrivilege           Change the system time
14972   CloudNotificat  13      SeProfileSingleProcessPrivilege         Profile a single process
14972   CloudNotificat  14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
14972   CloudNotificat  15      SeCreatePagefilePrivilege               Create a pagefile
14972   CloudNotificat  16      SeCreatePermanentPrivilege              Create permanent shared objects
14972   CloudNotificat  17      SeBackupPrivilege               Backup files and directories
14972   CloudNotificat  18      SeRestorePrivilege              Restore files and directories
14972   CloudNotificat  19      SeShutdownPrivilege     Present Shut down the system
14972   CloudNotificat  20      SeDebugPrivilege                Debug programs
14972   CloudNotificat  21      SeAuditPrivilege                Generate security audits
14972   CloudNotificat  22      SeSystemEnvironmentPrivilege            Edit firmware environment values
14972   CloudNotificat  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
14972   CloudNotificat  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
14972   CloudNotificat  25      SeUndockPrivilege       Present Remove computer from docking station
14972   CloudNotificat  26      SeSyncAgentPrivilege            Synch directory service data
14972   CloudNotificat  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
14972   CloudNotificat  28      SeManageVolumePrivilege         Manage the files on a volume
14972   CloudNotificat  29      SeImpersonatePrivilege          Impersonate a client after authentication
14972   CloudNotificat  30      SeCreateGlobalPrivilege Default Create global objects
14972   CloudNotificat  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
14972   CloudNotificat  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
14972   CloudNotificat  33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
14972   CloudNotificat  34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
14972   CloudNotificat  35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
14972   CloudNotificat  36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
13156   powershell.exe  2       SeCreateTokenPrivilege          Create a token object
13156   powershell.exe  3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
13156   powershell.exe  4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
13156   powershell.exe  5       SeIncreaseQuotaPrivilege        Present Increase quotas
13156   powershell.exe  6       SeMachineAccountPrivilege               Add workstations to the domain
13156   powershell.exe  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
13156   powershell.exe  8       SeSecurityPrivilege     Present Manage auditing and security log
13156   powershell.exe  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
13156   powershell.exe  10      SeLoadDriverPrivilege   Present Load and unload device drivers
13156   powershell.exe  11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
13156   powershell.exe  12      SeSystemtimePrivilege   Present Change the system time
13156   powershell.exe  13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
13156   powershell.exe  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
13156   powershell.exe  15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
13156   powershell.exe  16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
13156   powershell.exe  17      SeBackupPrivilege       Present Backup files and directories
13156   powershell.exe  18      SeRestorePrivilege      Present Restore files and directories
13156   powershell.exe  19      SeShutdownPrivilege     Present Shut down the system
13156   powershell.exe  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
13156   powershell.exe  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
13156   powershell.exe  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
13156   powershell.exe  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
13156   powershell.exe  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
13156   powershell.exe  25      SeUndockPrivilege       Present Remove computer from docking station
13156   powershell.exe  26      SeSyncAgentPrivilege            Synch directory service data
13156   powershell.exe  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
13156   powershell.exe  28      SeManageVolumePrivilege Present Manage the files on a volume
13156   powershell.exe  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
13156   powershell.exe  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
13156   powershell.exe  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
13156   powershell.exe  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
13156   powershell.exe  33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
13156   powershell.exe  34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
13156   powershell.exe  35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
13156   powershell.exe  36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
16024   svchost.exe     2       SeCreateTokenPrivilege          Create a token object
16024   svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
16024   svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
16024   svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
16024   svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
16024   svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
16024   svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
16024   svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
16024   svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
16024   svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
16024   svchost.exe     12      SeSystemtimePrivilege           Change the system time
16024   svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
16024   svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
16024   svchost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
16024   svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
16024   svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
16024   svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
16024   svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
16024   svchost.exe     20      SeDebugPrivilege        Default Debug programs
16024   svchost.exe     21      SeAuditPrivilege        Default Generate security audits
16024   svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
16024   svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
16024   svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
16024   svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
16024   svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
16024   svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
16024   svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
16024   svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
16024   svchost.exe     30      SeCreateGlobalPrivilege Default Create global objects
16024   svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
16024   svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
16024   svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
16024   svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
16024   svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
16024   svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
15932   svchost.exe     2       SeCreateTokenPrivilege          Create a token object
15932   svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
15932   svchost.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
15932   svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
15932   svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
15932   svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
15932   svchost.exe     8       SeSecurityPrivilege     Present Manage auditing and security log
15932   svchost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
15932   svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
15932   svchost.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
15932   svchost.exe     12      SeSystemtimePrivilege   Present Change the system time
15932   svchost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
15932   svchost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
15932   svchost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
15932   svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
15932   svchost.exe     17      SeBackupPrivilege       Present Backup files and directories
15932   svchost.exe     18      SeRestorePrivilege      Present Restore files and directories
15932   svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
15932   svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
15932   svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
15932   svchost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
15932   svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
15932   svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
15932   svchost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
15932   svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
15932   svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
15932   svchost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
15932   svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
15932   svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
15932   svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
15932   svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
15932   svchost.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
15932   svchost.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
15932   svchost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
15932   svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
10392   svchost.exe     2       SeCreateTokenPrivilege          Create a token object
10392   svchost.exe     3       SeAssignPrimaryTokenPrivilege           Replace a process-level token
10392   svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
10392   svchost.exe     5       SeIncreaseQuotaPrivilege                Increase quotas
10392   svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
10392   svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
10392   svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
10392   svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
10392   svchost.exe     10      SeLoadDriverPrivilege   Present Load and unload device drivers
10392   svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
10392   svchost.exe     12      SeSystemtimePrivilege           Change the system time
10392   svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
10392   svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
10392   svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
10392   svchost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
10392   svchost.exe     17      SeBackupPrivilege               Backup files and directories
10392   svchost.exe     18      SeRestorePrivilege              Restore files and directories
10392   svchost.exe     19      SeShutdownPrivilege     Present Shut down the system
10392   svchost.exe     20      SeDebugPrivilege        Default Debug programs
10392   svchost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
10392   svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
10392   svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10392   svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10392   svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
10392   svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
10392   svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10392   svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
10392   svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
10392   svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
10392   svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10392   svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10392   svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
10392   svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
10392   svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
10392   svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
7728    SenseIR.exe     2       SeCreateTokenPrivilege          Create a token object
7728    SenseIR.exe     3       SeAssignPrimaryTokenPrivilege   Present,Enabled Replace a process-level token
7728    SenseIR.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
7728    SenseIR.exe     5       SeIncreaseQuotaPrivilege        Present,Enabled Increase quotas
7728    SenseIR.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
7728    SenseIR.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
7728    SenseIR.exe     8       SeSecurityPrivilege     Present,Enabled Manage auditing and security log
7728    SenseIR.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
7728    SenseIR.exe     10      SeLoadDriverPrivilege   Present,Enabled Load and unload device drivers
7728    SenseIR.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
7728    SenseIR.exe     12      SeSystemtimePrivilege   Present Change the system time
7728    SenseIR.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
7728    SenseIR.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
7728    SenseIR.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
7728    SenseIR.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
7728    SenseIR.exe     17      SeBackupPrivilege       Present,Enabled Backup files and directories
7728    SenseIR.exe     18      SeRestorePrivilege      Present,Enabled Restore files and directories
7728    SenseIR.exe     19      SeShutdownPrivilege     Present Shut down the system
7728    SenseIR.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
7728    SenseIR.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
7728    SenseIR.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
7728    SenseIR.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7728    SenseIR.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7728    SenseIR.exe     25      SeUndockPrivilege       Present Remove computer from docking station
7728    SenseIR.exe     26      SeSyncAgentPrivilege            Synch directory service data
7728    SenseIR.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7728    SenseIR.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
7728    SenseIR.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
7728    SenseIR.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
7728    SenseIR.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7728    SenseIR.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7728    SenseIR.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
7728    SenseIR.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
7728    SenseIR.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
7728    SenseIR.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
10352   powershell.exe  2       SeCreateTokenPrivilege          Create a token object
10352   powershell.exe  3       SeAssignPrimaryTokenPrivilege   Present,Enabled Replace a process-level token
10352   powershell.exe  4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
10352   powershell.exe  5       SeIncreaseQuotaPrivilege        Present,Enabled Increase quotas
10352   powershell.exe  6       SeMachineAccountPrivilege               Add workstations to the domain
10352   powershell.exe  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
10352   powershell.exe  8       SeSecurityPrivilege     Present,Enabled Manage auditing and security log
10352   powershell.exe  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
10352   powershell.exe  10      SeLoadDriverPrivilege   Present,Enabled Load and unload device drivers
10352   powershell.exe  11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
10352   powershell.exe  12      SeSystemtimePrivilege   Present Change the system time
10352   powershell.exe  13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
10352   powershell.exe  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
10352   powershell.exe  15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
10352   powershell.exe  16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
10352   powershell.exe  17      SeBackupPrivilege       Present,Enabled Backup files and directories
10352   powershell.exe  18      SeRestorePrivilege      Present,Enabled Restore files and directories
10352   powershell.exe  19      SeShutdownPrivilege     Present Shut down the system
10352   powershell.exe  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
10352   powershell.exe  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
10352   powershell.exe  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
10352   powershell.exe  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
10352   powershell.exe  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
10352   powershell.exe  25      SeUndockPrivilege       Present Remove computer from docking station
10352   powershell.exe  26      SeSyncAgentPrivilege            Synch directory service data
10352   powershell.exe  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
10352   powershell.exe  28      SeManageVolumePrivilege Present Manage the files on a volume
10352   powershell.exe  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
10352   powershell.exe  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
10352   powershell.exe  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
10352   powershell.exe  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
10352   powershell.exe  33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
10352   powershell.exe  34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
10352   powershell.exe  35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
10352   powershell.exe  36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
7256    conhost.exe     2       SeCreateTokenPrivilege          Create a token object
7256    conhost.exe     3       SeAssignPrimaryTokenPrivilege   Present,Enabled Replace a process-level token
7256    conhost.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
7256    conhost.exe     5       SeIncreaseQuotaPrivilege        Present,Enabled Increase quotas
7256    conhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
7256    conhost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
7256    conhost.exe     8       SeSecurityPrivilege     Present,Enabled Manage auditing and security log
7256    conhost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
7256    conhost.exe     10      SeLoadDriverPrivilege   Present,Enabled Load and unload device drivers
7256    conhost.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
7256    conhost.exe     12      SeSystemtimePrivilege   Present Change the system time
7256    conhost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
7256    conhost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
7256    conhost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
7256    conhost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
7256    conhost.exe     17      SeBackupPrivilege       Present,Enabled Backup files and directories
7256    conhost.exe     18      SeRestorePrivilege      Present,Enabled Restore files and directories
7256    conhost.exe     19      SeShutdownPrivilege     Present Shut down the system
7256    conhost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
7256    conhost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
7256    conhost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
7256    conhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
7256    conhost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
7256    conhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
7256    conhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
7256    conhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
7256    conhost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
7256    conhost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
7256    conhost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
7256    conhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
7256    conhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
7256    conhost.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
7256    conhost.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
7256    conhost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
7256    conhost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
11408   OpenHandleColl  2       SeCreateTokenPrivilege          Create a token object
11408   OpenHandleColl  3       SeAssignPrimaryTokenPrivilege   Present,Enabled Replace a process-level token
11408   OpenHandleColl  4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
11408   OpenHandleColl  5       SeIncreaseQuotaPrivilege        Present,Enabled Increase quotas
11408   OpenHandleColl  6       SeMachineAccountPrivilege               Add workstations to the domain
11408   OpenHandleColl  7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
11408   OpenHandleColl  8       SeSecurityPrivilege     Present,Enabled Manage auditing and security log
11408   OpenHandleColl  9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
11408   OpenHandleColl  10      SeLoadDriverPrivilege   Present,Enabled Load and unload device drivers
11408   OpenHandleColl  11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
11408   OpenHandleColl  12      SeSystemtimePrivilege   Present Change the system time
11408   OpenHandleColl  13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
11408   OpenHandleColl  14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
11408   OpenHandleColl  15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
11408   OpenHandleColl  16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
11408   OpenHandleColl  17      SeBackupPrivilege       Present,Enabled Backup files and directories
11408   OpenHandleColl  18      SeRestorePrivilege      Present,Enabled Restore files and directories
11408   OpenHandleColl  19      SeShutdownPrivilege     Present Shut down the system
11408   OpenHandleColl  20      SeDebugPrivilege        Present,Enabled,Default Debug programs
11408   OpenHandleColl  21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
11408   OpenHandleColl  22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
11408   OpenHandleColl  23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
11408   OpenHandleColl  24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
11408   OpenHandleColl  25      SeUndockPrivilege       Present Remove computer from docking station
11408   OpenHandleColl  26      SeSyncAgentPrivilege            Synch directory service data
11408   OpenHandleColl  27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
11408   OpenHandleColl  28      SeManageVolumePrivilege Present,Enabled Manage the files on a volume
11408   OpenHandleColl  29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
11408   OpenHandleColl  30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
11408   OpenHandleColl  31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
11408   OpenHandleColl  32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
11408   OpenHandleColl  33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
11408   OpenHandleColl  34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
11408   OpenHandleColl  35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
11408   OpenHandleColl  36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
13532   conhost.exe     2       SeCreateTokenPrivilege          Create a token object
13532   conhost.exe     3       SeAssignPrimaryTokenPrivilege   Present,Enabled Replace a process-level token
13532   conhost.exe     4       SeLockMemoryPrivilege   Present,Enabled,Default Lock pages in memory
13532   conhost.exe     5       SeIncreaseQuotaPrivilege        Present,Enabled Increase quotas
13532   conhost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
13532   conhost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
13532   conhost.exe     8       SeSecurityPrivilege     Present,Enabled Manage auditing and security log
13532   conhost.exe     9       SeTakeOwnershipPrivilege        Present Take ownership of files/objects
13532   conhost.exe     10      SeLoadDriverPrivilege   Present,Enabled Load and unload device drivers
13532   conhost.exe     11      SeSystemProfilePrivilege        Present,Enabled,Default Profile system performance
13532   conhost.exe     12      SeSystemtimePrivilege   Present Change the system time
13532   conhost.exe     13      SeProfileSingleProcessPrivilege Present,Enabled,Default Profile a single process
13532   conhost.exe     14      SeIncreaseBasePriorityPrivilege Present,Enabled,Default Increase scheduling priority
13532   conhost.exe     15      SeCreatePagefilePrivilege       Present,Enabled,Default Create a pagefile
13532   conhost.exe     16      SeCreatePermanentPrivilege      Present,Enabled,Default Create permanent shared objects
13532   conhost.exe     17      SeBackupPrivilege       Present,Enabled Backup files and directories
13532   conhost.exe     18      SeRestorePrivilege      Present,Enabled Restore files and directories
13532   conhost.exe     19      SeShutdownPrivilege     Present Shut down the system
13532   conhost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
13532   conhost.exe     21      SeAuditPrivilege        Present,Enabled,Default Generate security audits
13532   conhost.exe     22      SeSystemEnvironmentPrivilege    Present Edit firmware environment values
13532   conhost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
13532   conhost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
13532   conhost.exe     25      SeUndockPrivilege       Present Remove computer from docking station
13532   conhost.exe     26      SeSyncAgentPrivilege            Synch directory service data
13532   conhost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
13532   conhost.exe     28      SeManageVolumePrivilege Present Manage the files on a volume
13532   conhost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
13532   conhost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
13532   conhost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
13532   conhost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
13532   conhost.exe     33      SeIncreaseWorkingSetPrivilege   Present,Enabled,Default Allocate more memory for user applications
13532   conhost.exe     34      SeTimeZonePrivilege     Present,Enabled,Default Adjust the time zone of the computer's internal clock
13532   conhost.exe     35      SeCreateSymbolicLinkPrivilege   Present,Enabled,Default Required to create a symbolic link
13532   conhost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Present,Enabled,Default Obtain an impersonation token for another user in the same session.
12008   svchost.exe     2       SeCreateTokenPrivilege          Create a token object
12008   svchost.exe     3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
12008   svchost.exe     4       SeLockMemoryPrivilege   Default Lock pages in memory
12008   svchost.exe     5       SeIncreaseQuotaPrivilege        Present Increase quotas
12008   svchost.exe     6       SeMachineAccountPrivilege               Add workstations to the domain
12008   svchost.exe     7       SeTcbPrivilege  Present,Enabled,Default Act as part of the operating system
12008   svchost.exe     8       SeSecurityPrivilege             Manage auditing and security log
12008   svchost.exe     9       SeTakeOwnershipPrivilege                Take ownership of files/objects
12008   svchost.exe     10      SeLoadDriverPrivilege           Load and unload device drivers
12008   svchost.exe     11      SeSystemProfilePrivilege        Default Profile system performance
12008   svchost.exe     12      SeSystemtimePrivilege           Change the system time
12008   svchost.exe     13      SeProfileSingleProcessPrivilege Default Profile a single process
12008   svchost.exe     14      SeIncreaseBasePriorityPrivilege Default Increase scheduling priority
12008   svchost.exe     15      SeCreatePagefilePrivilege       Default Create a pagefile
12008   svchost.exe     16      SeCreatePermanentPrivilege      Default Create permanent shared objects
12008   svchost.exe     17      SeBackupPrivilege               Backup files and directories
12008   svchost.exe     18      SeRestorePrivilege              Restore files and directories
12008   svchost.exe     19      SeShutdownPrivilege             Shut down the system
12008   svchost.exe     20      SeDebugPrivilege        Present,Enabled,Default Debug programs
12008   svchost.exe     21      SeAuditPrivilege        Default Generate security audits
12008   svchost.exe     22      SeSystemEnvironmentPrivilege            Edit firmware environment values
12008   svchost.exe     23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
12008   svchost.exe     24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
12008   svchost.exe     25      SeUndockPrivilege               Remove computer from docking station
12008   svchost.exe     26      SeSyncAgentPrivilege            Synch directory service data
12008   svchost.exe     27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
12008   svchost.exe     28      SeManageVolumePrivilege         Manage the files on a volume
12008   svchost.exe     29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
12008   svchost.exe     30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
12008   svchost.exe     31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
12008   svchost.exe     32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
12008   svchost.exe     33      SeIncreaseWorkingSetPrivilege   Default Allocate more memory for user applications
12008   svchost.exe     34      SeTimeZonePrivilege     Default Adjust the time zone of the computer's internal clock
12008   svchost.exe     35      SeCreateSymbolicLinkPrivilege   Default Required to create a symbolic link
12008   svchost.exe     36      SeDelegateSessionUserImpersonatePrivilege       Default Obtain an impersonation token for another user in the same session.
14028   WmiPrvSE.exe    2       SeCreateTokenPrivilege          Create a token object
14028   WmiPrvSE.exe    3       SeAssignPrimaryTokenPrivilege   Present Replace a process-level token
14028   WmiPrvSE.exe    4       SeLockMemoryPrivilege           Lock pages in memory
14028   WmiPrvSE.exe    5       SeIncreaseQuotaPrivilege        Present Increase quotas
14028   WmiPrvSE.exe    6       SeMachineAccountPrivilege               Add workstations to the domain
14028   WmiPrvSE.exe    7       SeTcbPrivilege          Act as part of the operating system
14028   WmiPrvSE.exe    8       SeSecurityPrivilege             Manage auditing and security log
14028   WmiPrvSE.exe    9       SeTakeOwnershipPrivilege                Take ownership of files/objects
14028   WmiPrvSE.exe    10      SeLoadDriverPrivilege           Load and unload device drivers
14028   WmiPrvSE.exe    11      SeSystemProfilePrivilege                Profile system performance
14028   WmiPrvSE.exe    12      SeSystemtimePrivilege           Change the system time
14028   WmiPrvSE.exe    13      SeProfileSingleProcessPrivilege         Profile a single process
14028   WmiPrvSE.exe    14      SeIncreaseBasePriorityPrivilege         Increase scheduling priority
14028   WmiPrvSE.exe    15      SeCreatePagefilePrivilege               Create a pagefile
14028   WmiPrvSE.exe    16      SeCreatePermanentPrivilege              Create permanent shared objects
14028   WmiPrvSE.exe    17      SeBackupPrivilege               Backup files and directories
14028   WmiPrvSE.exe    18      SeRestorePrivilege              Restore files and directories
14028   WmiPrvSE.exe    19      SeShutdownPrivilege     Present Shut down the system
14028   WmiPrvSE.exe    20      SeDebugPrivilege                Debug programs
14028   WmiPrvSE.exe    21      SeAuditPrivilege        Present Generate security audits
14028   WmiPrvSE.exe    22      SeSystemEnvironmentPrivilege            Edit firmware environment values
14028   WmiPrvSE.exe    23      SeChangeNotifyPrivilege Present,Enabled,Default Receive notifications of changes to files or directories
14028   WmiPrvSE.exe    24      SeRemoteShutdownPrivilege               Force shutdown from a remote system
14028   WmiPrvSE.exe    25      SeUndockPrivilege       Present Remove computer from docking station
14028   WmiPrvSE.exe    26      SeSyncAgentPrivilege            Synch directory service data
14028   WmiPrvSE.exe    27      SeEnableDelegationPrivilege             Enable user accounts to be trusted for delegation
14028   WmiPrvSE.exe    28      SeManageVolumePrivilege         Manage the files on a volume
14028   WmiPrvSE.exe    29      SeImpersonatePrivilege  Present,Enabled,Default Impersonate a client after authentication
14028   WmiPrvSE.exe    30      SeCreateGlobalPrivilege Present,Enabled,Default Create global objects
14028   WmiPrvSE.exe    31      SeTrustedCredManAccessPrivilege         Access Credential Manager as a trusted caller
14028   WmiPrvSE.exe    32      SeRelabelPrivilege              Modify the mandatory integrity level of an object
14028   WmiPrvSE.exe    33      SeIncreaseWorkingSetPrivilege   Present Allocate more memory for user applications
14028   WmiPrvSE.exe    34      SeTimeZonePrivilege     Present Adjust the time zone of the computer's internal clock
14028   WmiPrvSE.exe    35      SeCreateSymbolicLinkPrivilege           Required to create a symbolic link
14028   WmiPrvSE.exe    36      SeDelegateSessionUserImpersonatePrivilege               Obtain an impersonation token for another user in the same session.
```



`PSEXESVC.exe` (11296), `excel-update.exe` (4672), and `ScreenConnect` (6988) hold multiple high‑level privileges (notably `SeDebug`, `SeImpersonate` and `SeTcb`), several of which are enabled. This combination grants near‑system capabilities including reading other processes’ memory, impersonating accounts, and modifying system state and therefore these processes should be flagged as high‑priority for containment, binary hashing/signature checks, and a full forensic review.

#### DLLLIST
The `windows.dlllist` command lists modules loaded into each process from the memory image to reveal unusual or injected DLLs and their file paths for further PE/version and IOC analysis. the `--pid=<pid number>` flag helps to filter the output to only the suspiscious processes identified

`vol -f iveep.mem windows.dlllist --pid=6988`

```
Volatility 3 Framework 2.26.2
Progress:  100.00               PDB scanning finished
PID     Process Base    Size    Name    Path    LoadTime        File output

6988    ScreenConnect.  0x7f0000        0x18000 ScreenConnect.ClientService.exe C:\Program Files (x86)\ScreenConnect Client (a17d7bba0dc91adf)\ScreenConnect.ClientService.exe  2025-03-17 17:05:17.000000 UTC  Disabled
6988    ScreenConnect.  0x7ffb50170000  0x1f8000        ntdll.dll       C:\Windows\SYSTEM32\ntdll.dll   2025-03-17 17:05:17.000000 UTC  Disabled
6988    ScreenConnect.  0x7ffb4edc0000  0x59000 wow64.dll       C:\Windows\System32\wow64.dll   2025-03-17 17:05:17.000000 UTC  Disabled
6988    ScreenConnect.  0x7ffb4ed10000  0x83000 wow64win.dll    C:\Windows\System32\wow64win.dll        2025-03-17 17:05:17.000000 UTC  Disabled
6988    ScreenConnect.  0x77260000      0xa000  wow64cpu.dll    C:\Windows\System32\wow64cpu.dll        2025-03-17 17:05:17.000000 UTC  Disabled
6988    ScreenConnect.  0x1010000       0x16000 umppc19205.dll  C:\Windows\System32\umppc19205.dll      2025-03-17 17:05:17.000000 UTC  Disabled
6988    ScreenConnect.  0x7f0000        0x18000 ScreenConnect.ClientService.exe C:\Program Files (x86)\ScreenConnect Client (a17d7bba0dc91adf)\ScreenConnect.ClientService.exe  -       Disabled
6988    ScreenConnect.  0x77270000      0x1a4000        ntdll.dll       C:\Windows\SYSTEM32\ntdll.dll   -       Disabled
6988    ScreenConnect.  0x75c00000      0xf0000 KERNEL32.DLL    C:\Windows\System32\KERNEL32.DLL        -       Disabled
6988    ScreenConnect.  0x756c0000      0x23a000        KERNELBASE.dll  C:\Windows\System32\KERNELBASE.dll      -       Disabled
6988    ScreenConnect.  0x742b0000      0x9f000 apphelp.dll     C:\Windows\SYSTEM32\apphelp.dll N/A     Disabled
6988    ScreenConnect.  0x76260000      0x45000 SHLWAPI.dll     C:\Windows\System32\SHLWAPI.dll -       Disabled
6988    ScreenConnect.  0x734f0000      0x52000 mscoree.dll     C:\Windows\SYSTEM32\mscoree.dll -       Disabled
6988    ScreenConnect.  0x751d0000      0xbf000 msvcrt.dll      C:\Windows\System32\msvcrt.dll  N/A     Disabled
6988    ScreenConnect.  0x762d0000      0x7d000 ADVAPI32.dll    C:\Windows\System32\ADVAPI32.dll        -       Disabled
6988    ScreenConnect.  0x769e0000      0x78000 sechost.dll     C:\Windows\System32\sechost.dll -       Disabled
6988    ScreenConnect.  0x75290000      0xbc000 RPCRT4.dll      C:\Windows\System32\RPCRT4.dll  -       Disabled
6988    ScreenConnect.  0x761a0000      0x19000 bcrypt.dll      C:\Windows\System32\bcrypt.dll  -       Disabled
6988    ScreenConnect.  0x76e70000      0x96000 OLEAUT32.dll    C:\Windows\System32\OLEAUT32.dll        -       Disabled
6988    ScreenConnect.  0x75960000      0x7b000 msvcp_win.dll   C:\Windows\System32\msvcp_win.dll       -       Disabled
6988    ScreenConnect.  0x75460000      0x120000        ucrtbase.dll    C:\Windows\System32\ucrtbase.dll        -       Disabled
6988    ScreenConnect.  0x76bf0000      0x280000        combase.dll     C:\Windows\System32\combase.dll -       Disabled
6988    ScreenConnect.  0x1470000       0x10000 CsXumd32_19205.dll      C:\Windows\System32\CsXumd32_19205.dll  -       Disabled
6988    ScreenConnect.  0x1530000       0x2a000 ScriptControl32_19205.dll       C:\Windows\System32\ScriptControl32_19205.dll   -       Disabled
6988    ScreenConnect.  0x76b00000      0xe3000 ole32.dll       C:\Windows\System32\ole32.dll   -       Disabled
6988    ScreenConnect.  0x76220000      0x23000 GDI32.dll       C:\Windows\System32\GDI32.dll   -       Disabled
6988    ScreenConnect.  0x749e0000      0x8000  VERSION.dll     C:\Windows\SYSTEM32\VERSION.dll -       Disabled
6988    ScreenConnect.  0x76ae0000      0x18000 win32u.dll      C:\Windows\System32\win32u.dll  -       Disabled
6988    ScreenConnect.  0x77140000      0xe6000 gdi32full.dll   C:\Windows\System32\gdi32full.dll       -       Disabled
6988    ScreenConnect.  0x76fa0000      0x19d000        USER32.dll      C:\Windows\System32\USER32.dll  -       Disabled
6988    ScreenConnect.  0x761c0000      0x5f000 bcryptPrimitives.dll    C:\Windows\System32\bcryptPrimitives.dll        -       Disabled
6988    ScreenConnect.  0x73460000      0x8d000 mscoreei.dll    C:\Windows\Microsoft.NET\Framework\v4.0.30319\mscoreei.dll      -       Disabled
6988    ScreenConnect.  0x74450000      0xf000  kernel.appcore.dll      C:\Windows\SYSTEM32\kernel.appcore.dll  -       Disabled
6988    ScreenConnect.  0x72cb0000      0x7b0000        clr.dll C:\Windows\Microsoft.NET\Framework\v4.0.30319\clr.dll   N/A     Disabled
6988    ScreenConnect.  0x72be0000      0x14000 VCRUNTIME140_CLR0400.dll        C:\Windows\SYSTEM32\VCRUNTIME140_CLR0400.dll    -       Disabled
6988    ScreenConnect.  0x72c00000      0xab000 ucrtbase_clr0400.dll    C:\Windows\SYSTEM32\ucrtbase_clr0400.dll        -       Disabled
6988    ScreenConnect.  0x717d0000      0x140e000       mscorlib.ni.dll C:\Windows\assembly\NativeImages_v4.0.30319_32\mscorlib\1c960778124fb2c275142764edfbee19\mscorlib.ni.dll        -       Disabled
6988    ScreenConnect.  0x74d60000      0x13000 CRYPTSP.dll     C:\Windows\SYSTEM32\CRYPTSP.dll -       Disabled
6988    ScreenConnect.  0x74d30000      0x2f000 rsaenh.dll      C:\Windows\system32\rsaenh.dll  -       Disabled
6988    ScreenConnect.  0x74cc0000      0xa000  CRYPTBASE.dll   C:\Windows\SYSTEM32\CRYPTBASE.dll       -       Disabled
6988    ScreenConnect.  0x74800000      0x1a8000        urlmon.dll      C:\Windows\SYSTEM32\urlmon.dll  -       Disabled
6988    ScreenConnect.  0x74500000      0x237000        iertutil.dll    C:\Windows\SYSTEM32\iertutil.dll        -       Disabled
6988    ScreenConnect.  0x76f10000      0x87000 shcore.dll      C:\Windows\System32\shcore.dll  2615-06-09 14:52:36.000000 UTC  Disabled
6988    ScreenConnect.  0x744e0000      0x1d000 srvcli.dll      C:\Windows\SYSTEM32\srvcli.dll  -       Disabled
6988    ScreenConnect.  0x744d0000      0xb000  netutils.dll    C:\Windows\SYSTEM32\netutils.dll        -       Disabled
6988    ScreenConnect.  0x74d00000      0x21000 SspiCli.dll     C:\Windows\SYSTEM32\SspiCli.dll -       Disabled
6988    ScreenConnect.  0x711b0000      0x613000        windows.storage.dll     C:\Windows\SYSTEM32\windows.storage.dll -       Disabled
6988    ScreenConnect.  0x71180000      0x25000 Wldp.dll        C:\Windows\SYSTEM32\Wldp.dll    -       Disabled
6988    ScreenConnect.  0x710b0000      0xc2000 PROPSYS.dll     C:\Windows\SYSTEM32\PROPSYS.dll -       Disabled
6988    ScreenConnect.  0x74350000      0xf000  virtdisk.dll    C:\Windows\SYSTEM32\virtdisk.dll        -       Disabled
6988    ScreenConnect.  0x710a0000      0x8000  FLTLIB.DLL      C:\Windows\SYSTEM32\FLTLIB.DLL  -       Disabled
6988    ScreenConnect.  0x71010000      0x8a000 clrjit.dll      C:\Windows\Microsoft.NET\Framework\v4.0.30319\clrjit.dll        -       Disabled
6988    ScreenConnect.  0x705b0000      0xa55000        System.ni.dll   C:\Windows\assembly\NativeImages_v4.0.30319_32\System\a94f452eecde0f07e988ad14497426a5\System.ni.dll    -       Disabled
6988    ScreenConnect.  0x6fd90000      0x818000        System.Core.ni.dll      C:\Windows\assembly\NativeImages_v4.0.30319_32\System.Core\8c6bf72770b92eaa245248616300f722\System.Core.ni.dll  -       Disabled
6988    ScreenConnect.  0x6fc80000      0x106000        System.Configuration.ni.dll     C:\Windows\assembly\NativeImages_v4.0.30319_32\System.Configuration\ade5c2e4b2537b254ea41b3d743fdef2\System.Configuration.ni.dll    -       Disabled
6988    ScreenConnect.  0x76350000      0x5d7000        shell32.dll     C:\Windows\System32\shell32.dll -       Disabled
6988    ScreenConnect.  0x6f500000      0x774000        System.Xml.ni.dll       C:\Windows\assembly\NativeImages_v4.0.30319_32\System.Xml\48fbd098873eae92cf4bda5b7cf7a6ba\System.Xml.ni.dll    -       Disabled
6988    ScreenConnect.  0x74ce0000      0x1b000 profapi.dll     C:\Windows\SYSTEM32\profapi.dll -       Disabled
6988    ScreenConnect.  0x75360000      0x100000        crypt32.dll     C:\Windows\System32\crypt32.dll -       Disabled
6988    ScreenConnect.  0x74cd0000      0x8000  DPAPI.dll       C:\Windows\SYSTEM32\DPAPI.dll   -       Disabled
6988    ScreenConnect.  0x73a10000      0x18000 amsi.dll        C:\Windows\SYSTEM32\amsi.dll    -       Disabled
6988    ScreenConnect.  0x74dc0000      0x25000 USERENV.dll     C:\Windows\SYSTEM32\USERENV.dll -       Disabled
6988    ScreenConnect.  0x739d0000      0x38000 MpOav.dll       C:\Program Files (x86)\Windows Defender\MpOav.dll       -       Disabled
6988    ScreenConnect.  0x73920000      0xab000 MPCLIENT.DLL    C:\Program Files (x86)\Windows Defender\MPCLIENT.DLL    2616-02-22 07:37:47.000000 UTC  Disabled
6988    ScreenConnect.  0x73900000      0x1e000 gpapi.dll       C:\Windows\SYSTEM32\gpapi.dll   -       Disabled
6988    ScreenConnect.  0x74db0000      0xf000  wtsapi32.DLL    C:\Windows\SYSTEM32\wtsapi32.DLL        2615-06-09 21:33:28.000000 UTC  Disabled
6988    ScreenConnect.  0x738b0000      0x47000 WINSTA.dll      C:\Windows\SYSTEM32\WINSTA.dll  -       Disabled
6988    ScreenConnect.  0x74480000      0x14000 netapi32.DLL    C:\Windows\SYSTEM32\netapi32.DLL        -       Disabled
6988    ScreenConnect.  0x74a10000      0x15000 SAMCLI.DLL      C:\Windows\SYSTEM32\SAMCLI.DLL  -       Disabled
6988    ScreenConnect.  0x73890000      0x1b000 SAMLIB.dll      C:\Windows\SYSTEM32\SAMLIB.dll  2616-02-22 14:18:39.000000 UTC  Disabled
6988    ScreenConnect.  0x76130000      0x63000 ws2_32.dll      C:\Windows\System32\ws2_32.dll  -       Disabled
6988    ScreenConnect.  0x743f0000      0x52000 mswsock.dll     C:\Windows\system32\mswsock.dll -       Disabled
6988    ScreenConnect.  0x73a60000      0x90000 DNSAPI.dll      C:\Windows\SYSTEM32\DNSAPI.dll  2616-02-20 02:10:52.000000 UTC  Disabled
6988    ScreenConnect.  0x743b0000      0x32000 IPHLPAPI.DLL    C:\Windows\SYSTEM32\IPHLPAPI.DLL        -       Disabled
6988    ScreenConnect.  0x75350000      0x7000  NSI.dll C:\Windows\System32\NSI.dll     -       Disabled
6988    ScreenConnect.  0x73880000      0x8000  rasadhlp.dll    C:\Windows\System32\rasadhlp.dll        -       Disabled
6988    ScreenConnect.  0x73820000      0x58000 fwpuclnt.dll    C:\Windows\System32\fwpuclnt.dll        -       Disabled
6988    ScreenConnect.  0x76250000      0x6000  psapi.dll       C:\Windows\System32\psapi.dll   -       Disabled
6988    ScreenConnect.  0x74380000      0x14000 dhcpcsvc6.DLL   C:\Windows\SYSTEM32\dhcpcsvc6.DLL       -       Disabled
6988    ScreenConnect.  0x74360000      0x16000 dhcpcsvc.DLL    C:\Windows\SYSTEM32\dhcpcsvc.DLL        -       Disabled
6988    ScreenConnect.  0x73810000      0x8000  WINNSI.DLL      C:\Windows\SYSTEM32\WINNSI.DLL  -       Disabled

```



`vol -f iveep.mem windows.dlllist --pid=11296`

```
(base) PS C:\Users\princ\Labs\iveep> 
Volatility 3 Framework 2.26.2
Progress:  100.00               PDB scanning finished
PID     Process Base    Size    Name    Path    LoadTime        File output

11296   PSEXESVC.exe    0xa40000        0x51000 PSEXESVC.exe    C:\Windows\PSEXESVC.exe 2025-03-17 17:01:18.000000 UTC  Disabled
11296   PSEXESVC.exe    0x7ffb50170000  0x1f8000        ntdll.dll       C:\Windows\SYSTEM32\ntdll.dll   2025-03-17 17:01:18.000000 UTC  Disabled
11296   PSEXESVC.exe    0x7ffb4edc0000  0x59000 wow64.dll       C:\Windows\System32\wow64.dll   2025-03-17 17:01:18.000000 UTC  Disabled
11296   PSEXESVC.exe    0x7ffb4ed10000  0x83000 wow64win.dll    C:\Windows\System32\wow64win.dll        2025-03-17 17:01:18.000000 UTC  Disabled
11296   PSEXESVC.exe    0x77260000      0xa000  wow64cpu.dll    C:\Windows\System32\wow64cpu.dll        2025-03-17 17:01:18.000000 UTC  Disabled
11296   PSEXESVC.exe    0x1590000       0x16000 umppc19205.dll  C:\Windows\System32\umppc19205.dll      2025-03-17 17:01:18.000000 UTC  Disabled
11296   PSEXESVC.exe    0xa40000        0x51000 PSEXESVC.exe    C:\Windows\PSEXESVC.exe -       Disabled
11296   PSEXESVC.exe    0x77270000      0x1a4000        ntdll.dll       C:\Windows\SYSTEM32\ntdll.dll   -       Disabled
11296   PSEXESVC.exe    0x75c00000      0xf0000 KERNEL32.DLL    C:\Windows\System32\KERNEL32.DLL        -       Disabled
11296   PSEXESVC.exe    0x756c0000      0x23a000        KERNELBASE.dll  C:\Windows\System32\KERNELBASE.dll      -       Disabled
11296   PSEXESVC.exe    0x742b0000      0x9f000 apphelp.dll     C:\Windows\SYSTEM32\apphelp.dll N/A     Disabled
11296   PSEXESVC.exe    0x76fa0000      0x19d000        USER32.dll      C:\Windows\System32\USER32.dll  -       Disabled
11296   PSEXESVC.exe    0x74dc0000      0x25000 USERENV.dll     C:\Windows\SYSTEM32\USERENV.dll -       Disabled
11296   PSEXESVC.exe    0x76ae0000      0x18000 win32u.dll      C:\Windows\System32\win32u.dll  -       Disabled
11296   PSEXESVC.exe    0x76220000      0x23000 GDI32.dll       C:\Windows\System32\GDI32.dll   -       Disabled
11296   PSEXESVC.exe    0x75460000      0x120000        ucrtbase.dll    C:\Windows\System32\ucrtbase.dll        -       Disabled
11296   PSEXESVC.exe    0x77140000      0xe6000 gdi32full.dll   C:\Windows\System32\gdi32full.dll       -       Disabled
11296   PSEXESVC.exe    0x75290000      0xbc000 RPCRT4.dll      C:\Windows\System32\RPCRT4.dll  -       Disabled
11296   PSEXESVC.exe    0x75960000      0x7b000 msvcp_win.dll   C:\Windows\System32\msvcp_win.dll       -       Disabled
11296   PSEXESVC.exe    0x762d0000      0x7d000 ADVAPI32.dll    C:\Windows\System32\ADVAPI32.dll        -       Disabled
11296   PSEXESVC.exe    0x751d0000      0xbf000 msvcrt.dll      C:\Windows\System32\msvcrt.dll  -       Disabled
11296   PSEXESVC.exe    0x769e0000      0x78000 sechost.dll     C:\Windows\System32\sechost.dll -       Disabled
11296   PSEXESVC.exe    0x761a0000      0x19000 bcrypt.dll      C:\Windows\System32\bcrypt.dll  -       Disabled
11296   PSEXESVC.exe    0x76350000      0x5d7000        SHELL32.dll     C:\Windows\System32\SHELL32.dll -       Disabled
11296   PSEXESVC.exe    0x1600000       0x10000 CsXumd32_19205.dll      C:\Windows\System32\CsXumd32_19205.dll  -       Disabled
11296   PSEXESVC.exe    0x761c0000      0x5f000 bcryptPrimitives.dll    C:\Windows\System32\bcryptPrimitives.dll        -       Disabled
11296   PSEXESVC.exe    0x74db0000      0xf000  Wtsapi32.dll    C:\Windows\SYSTEM32\Wtsapi32.dll        -       Disabled
11296   PSEXESVC.exe    0x74450000      0xf000  kernel.appcore.dll      C:\Windows\SYSTEM32\kernel.appcore.dll  -       Disabled
11296   PSEXESVC.exe    0x74d80000      0x29000 ntmarta.dll     C:\Windows\SYSTEM32\ntmarta.dll -       Disabled
11296   PSEXESVC.exe    0x74d60000      0x13000 CRYPTSP.dll     C:\Windows\SYSTEM32\CRYPTSP.dll -       Disabled
11296   PSEXESVC.exe    0x74d30000      0x2f000 rsaenh.dll      C:\Windows\system32\rsaenh.dll  -       Disabled
11296   PSEXESVC.exe    0x74d00000      0x21000 SspiCli.dll     C:\Windows\SYSTEM32\SspiCli.dll -       Disabled
11296   PSEXESVC.exe    0x74ce0000      0x1b000 profapi.dll     C:\Windows\SYSTEM32\profapi.dll -       Disabled
11296   PSEXESVC.exe    0x75360000      0x100000        CRYPT32.dll     C:\Windows\System32\CRYPT32.dll N/A     Disabled
11296   PSEXESVC.exe    0x74cd0000      0x8000  DPAPI.dll       C:\Windows\SYSTEM32\DPAPI.dll   -       Disabled
11296   PSEXESVC.exe    0x74cc0000      0xa000  CRYPTBASE.dll   C:\Windows\SYSTEM32\CRYPTBASE.dll       N/A     Disabled
```

`vol -f iveep.mem windows.dlllist --pid=4672`

```
Volatility 3 Framework 2.26.2
Progress:  100.00               PDB scanning finished
PID     Process Base    Size    Name    Path    LoadTime        File output

4672    excel-update.e  0x400000        0x16000 excel-update.exe        C:\Windows\excel-update.exe     2025-03-17 17:01:18.000000 UTC  Disabled
4672    excel-update.e  0x7ffb50170000  0x1f8000        ntdll.dll       C:\Windows\SYSTEM32\ntdll.dll   2025-03-17 17:01:18.000000 UTC  Disabled
4672    excel-update.e  0x7ffb4edc0000  0x59000 wow64.dll       C:\Windows\System32\wow64.dll   2025-03-17 17:01:18.000000 UTC  Disabled
4672    excel-update.e  0x7ffb4ed10000  0x83000 wow64win.dll    C:\Windows\System32\wow64win.dll        2025-03-17 17:01:18.000000 UTC  Disabled
4672    excel-update.e  0x77260000      0xa000  wow64cpu.dll    C:\Windows\System32\wow64cpu.dll        2025-03-17 17:01:18.000000 UTC  Disabled
4672    excel-update.e  0x640000        0x16000 umppc19205.dll  C:\Windows\System32\umppc19205.dll      2025-03-17 17:01:19.000000 UTC  Disabled
4672    excel-update.e  0x400000        0x16000 excel-update.exe        C:\Windows\excel-update.exe     -       Disabled
4672    excel-update.e  0x77270000      0x1a4000        ntdll.dll       C:\Windows\SYSTEM32\ntdll.dll   -       Disabled
4672    excel-update.e  0x75c00000      0xf0000 KERNEL32.DLL    C:\Windows\System32\KERNEL32.DLL        -       Disabled
4672    excel-update.e  0x756c0000      0x23a000        KERNELBASE.dll  C:\Windows\System32\KERNELBASE.dll      -       Disabled
4672    excel-update.e  0x742b0000      0x9f000 apphelp.dll     C:\Windows\SYSTEM32\apphelp.dll N/A     Disabled
4672    excel-update.e  0x74a60000      0x253000        AcGenral.DLL    C:\Windows\SYSTEM32\AcGenral.DLL        -       Disabled
4672    excel-update.e  0x751d0000      0xbf000 msvcrt.dll      C:\Windows\System32\msvcrt.dll  N/A     Disabled
4672    excel-update.e  0x769e0000      0x78000 sechost.dll     C:\Windows\System32\sechost.dll -       Disabled
4672    excel-update.e  0x75290000      0xbc000 RPCRT4.dll      C:\Windows\System32\RPCRT4.dll  -       Disabled
4672    excel-update.e  0x761a0000      0x19000 bcrypt.dll      C:\Windows\System32\bcrypt.dll  -       Disabled
4672    excel-update.e  0x76260000      0x45000 SHLWAPI.dll     C:\Windows\System32\SHLWAPI.dll -       Disabled
4672    excel-update.e  0x76fa0000      0x19d000        USER32.dll      C:\Windows\System32\USER32.dll  N/A     Disabled
4672    excel-update.e  0x76ae0000      0x18000 win32u.dll      C:\Windows\System32\win32u.dll  N/A     Disabled
4672    excel-update.e  0x76220000      0x23000 GDI32.dll       C:\Windows\System32\GDI32.dll   -       Disabled
4672    excel-update.e  0x77140000      0xe6000 gdi32full.dll   C:\Windows\System32\gdi32full.dll       -       Disabled
4672    excel-update.e  0x75960000      0x7b000 msvcp_win.dll   C:\Windows\System32\msvcp_win.dll       -       Disabled
4672    excel-update.e  0x75460000      0x120000        ucrtbase.dll    C:\Windows\System32\ucrtbase.dll        -       Disabled
4672    excel-update.e  0x76b00000      0xe3000 ole32.dll       C:\Windows\System32\ole32.dll   -       Disabled
4672    excel-update.e  0x76bf0000      0x280000        combase.dll     C:\Windows\System32\combase.dll -       Disabled
4672    excel-update.e  0x76e70000      0x96000 OLEAUT32.dll    C:\Windows\System32\OLEAUT32.dll        -       Disabled
4672    excel-update.e  0x76350000      0x5d7000        SHELL32.dll     C:\Windows\System32\SHELL32.dll -       Disabled
4672    excel-update.e  0x762d0000      0x7d000 ADVAPI32.dll    C:\Windows\System32\ADVAPI32.dll        -       Disabled
4672    excel-update.e  0x74e00000      0x74000 UxTheme.dll     C:\Windows\SYSTEM32\UxTheme.dll -       Disabled
4672    excel-update.e  0x74a30000      0x28000 WINMM.dll       C:\Windows\SYSTEM32\WINMM.dll   -       Disabled
4672    excel-update.e  0x74a10000      0x15000 samcli.dll      C:\Windows\SYSTEM32\samcli.dll  -       Disabled
4672    excel-update.e  0x749f0000      0x19000 MSACM32.dll     C:\Windows\SYSTEM32\MSACM32.dll -       Disabled
4672    excel-update.e  0x749e0000      0x8000  VERSION.dll     C:\Windows\SYSTEM32\VERSION.dll -       Disabled
4672    excel-update.e  0x74dc0000      0x25000 USERENV.dll     C:\Windows\SYSTEM32\USERENV.dll -       Disabled
4672    excel-update.e  0x749b0000      0x26000 dwmapi.dll      C:\Windows\SYSTEM32\dwmapi.dll  -       Disabled
4672    excel-update.e  0x74800000      0x1a8000        urlmon.dll      C:\Windows\SYSTEM32\urlmon.dll  -       Disabled
4672    excel-update.e  0x74780000      0x7d000 WINSPOOL.DRV    C:\Windows\SYSTEM32\WINSPOOL.DRV        -       Disabled
4672    excel-update.e  0x76f10000      0x87000 shcore.dll      C:\Windows\System32\shcore.dll  -       Disabled
4672    excel-update.e  0x74760000      0x19000 MPR.dll C:\Windows\SYSTEM32\MPR.dll     -       Disabled
4672    excel-update.e  0x74d00000      0x21000 SspiCli.dll     C:\Windows\SYSTEM32\SspiCli.dll -       Disabled
4672    excel-update.e  0x74740000      0x1d000 winmmbase.dll   C:\Windows\SYSTEM32\winmmbase.dll       -       Disabled
4672    excel-update.e  0x74500000      0x237000        iertutil.dll    C:\Windows\SYSTEM32\iertutil.dll        -       Disabled
4672    excel-update.e  0x744e0000      0x1d000 srvcli.dll      C:\Windows\SYSTEM32\srvcli.dll  -       Disabled
4672    excel-update.e  0x744d0000      0xb000  netutils.dll    C:\Windows\SYSTEM32\netutils.dll        -       Disabled
4672    excel-update.e  0x74020000      0x285000        AcLayers.dll    C:\Windows\SYSTEM32\AcLayers.dll        -       Disabled
4672    excel-update.e  0x75cf0000      0x438000        SETUPAPI.dll    C:\Windows\System32\SETUPAPI.dll        -       Disabled
4672    excel-update.e  0x759e0000      0x3b000 cfgmgr32.dll    C:\Windows\System32\cfgmgr32.dll        -       Disabled
4672    excel-update.e  0x66680000      0x3000  sfc.dll C:\Windows\SYSTEM32\sfc.dll     N/A     Disabled
4672    excel-update.e  0x744c0000      0x10000 sfc_os.DLL      C:\Windows\SYSTEM32\sfc_os.DLL  -       Disabled
4672    excel-update.e  0x744b0000      0x8000  WSOCK32.dll     C:\Windows\SYSTEM32\WSOCK32.dll N/A     Disabled
4672    excel-update.e  0x76130000      0x63000 WS2_32.dll      C:\Windows\System32\WS2_32.dll  -       Disabled
4672    excel-update.e  0x6b0000        0x10000 CsXumd32_19205.dll      C:\Windows\System32\CsXumd32_19205.dll  -       Disabled
4672    excel-update.e  0x743f0000      0x52000 mswsock.dll     C:\Windows\system32\mswsock.dll N/A     Disabled
4672    excel-update.e  0x75360000      0x100000        CRYPT32.dll     C:\Windows\System32\CRYPT32.dll -       Disabled
4672    excel-update.e  0x73bc0000      0x455000        WININET.dll     C:\Windows\SYSTEM32\WININET.dll N/A     Disabled
4672    excel-update.e  0x73af0000      0xca000 WINHTTP.dll     C:\Windows\SYSTEM32\WINHTTP.dll N/A     Disabled
4672    excel-update.e  0x74d60000      0x13000 CRYPTSP.dll     C:\Windows\SYSTEM32\CRYPTSP.dll -       Disabled
4672    excel-update.e  0x74d30000      0x2f000 rsaenh.dll      C:\Windows\system32\rsaenh.dll  -       Disabled
4672    excel-update.e  0x761c0000      0x5f000 bcryptPrimitives.dll    C:\Windows\System32\bcryptPrimitives.dll        -       Disabled
4672    excel-update.e  0x74ce0000      0x1b000 profapi.dll     C:\Windows\SYSTEM32\profapi.dll -       Disabled
4672    excel-update.e  0x74cc0000      0xa000  CRYPTBASE.dll   C:\Windows\SYSTEM32\CRYPTBASE.dll       -       Disabled
4672    excel-update.e  0x744a0000      0xe000  MSASN1.dll      C:\Windows\SYSTEM32\MSASN1.dll  -       Disabled
4672    excel-update.e  0x74480000      0x14000 NETAPI32.dll    C:\Windows\SYSTEM32\NETAPI32.dll        -       Disabled
4672    excel-update.e  0x76250000      0x6000  PSAPI.DLL       C:\Windows\System32\PSAPI.DLL   -       Disabled
4672    excel-update.e  0x743b0000      0x32000 IPHLPAPI.DLL    C:\Windows\SYSTEM32\IPHLPAPI.DLL        -       Disabled
4672    excel-update.e  0x74460000      0x11000 wkscli.dll      C:\Windows\SYSTEM32\wkscli.dll  N/A     Disabled
4672    excel-update.e  0x743a0000      0xe000  cscapi.dll      C:\Windows\SYSTEM32\cscapi.dll  N/A     Disabled
4672    excel-update.e  0x75350000      0x7000  NSI.dll C:\Windows\System32\NSI.dll     -       Disabled
4672    excel-update.e  0x74380000      0x14000 dhcpcsvc6.DLL   C:\Windows\SYSTEM32\dhcpcsvc6.DLL       -       Disabled
4672    excel-update.e  0x74360000      0x16000 dhcpcsvc.DLL    C:\Windows\SYSTEM32\dhcpcsvc.DLL        N/A     Disabled
4672    excel-update.e  0x73a60000      0x90000 DNSAPI.dll      C:\Windows\SYSTEM32\DNSAPI.dll  N/A     Disabled
```

The presence of modules with unusual names like `umppc19205.dll` and `ScriptControl32_19205.dll` suggests a possible infection or compromise. Such modules might be part of a persistence mechanism or malware.

#### MALFIND
The `windows.malfind` command in Volatility is used to detect potential malicious code injected into processes by scanning memory for suspicious executable regions. It identifies regions of memory marked with executable permissions, commonly associated with malware activity, such as code injection or reflective DLL loading.

`vol -f iveep.mem windows.malfind.Malfind`

```
Volatility 3 Framework 2.26.2
C:\Users\princ\anaconda3\Lib\site-packages\volatility3\framework\deprecation.py:28: FutureWarning: This API (volatility3.plugins.windows.malware.malfind.Malfind.run) will be removed in the first release after 2026-06-07. This plugin has been renamed, please call volatility3.plugins.windows.malware.malfind.Malfind rather than volatility3.plugins.windows.malfind.Malfind.
  warnings.warn(

PID     Process Start VPN       End VPN Tag     Protection      CommitCharge    PrivateMemory   File output     Notes   Hexdump Disasm
C:\Users\princ\anaconda3\Lib\site-packages\volatility3\framework\deprecation.py:105: FutureWarning: This plugin (volatility3.plugins.windows.malfind.Malfind) has been renamed and will be removed in the first release after 2026-06-07. Please ensure all method calls to this plugin are replaced with calls to volatility3.plugins.windows.malware.malfind.Malfind
  warnings.warn(

11296   PSEXESVC.exe    0x1570000       0x1570fff       VadS    PAGE_EXECUTE_READWRITE  1       1       Disabled        N/A
8b ff 55 8b ec e9 db 58 84 73 cc cc cc cc cc cc ..U....X.s......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 ..U.............
00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00 ..........-.....        8b ff 55 8b ec e9 db 58 84 73 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00
11296   PSEXESVC.exe    0x1580000       0x1580fff       VadS    PAGE_EXECUTE_READWRITE  1       1       Disabled        N/A
8b ff 55 8b ec e9 6b de 78 73 cc cc cc cc cc cc ..U...k.xs......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 ..U.............
00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00 ..........-.....        8b ff 55 8b ec e9 6b de 78 73 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00
11296   PSEXESVC.exe    0x1670000       0x1670fff       VadS    PAGE_EXECUTE_READWRITE  1       1       Disabled        N/A
8b ff 55 8b ec e9 fb d5 c5 73 cc cc cc cc cc cc ..U......s......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 ..U.............
00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00 ..........-.....        8b ff 55 8b ec e9 fb d5 c5 73 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00
11296   PSEXESVC.exe    0x1640000       0x1640fff       VadS    PAGE_EXECUTE_READWRITE  1       1       Disabled        N/A
8b ff 55 8b ec e9 6b 2d 5f 74 cc cc cc cc cc cc ..U...k-_t......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 ..U.............
00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00 ..........-.....        8b ff 55 8b ec e9 6b 2d 5f 74 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00
11296   PSEXESVC.exe    0x1630000       0x1630fff       VadS    PAGE_EXECUTE_READWRITE  1       1       Disabled        N/A
8b ff 55 8b ec e9 7b 02 cc 74 cc cc cc cc cc cc ..U...{..t......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 ..U.............
00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00 ..........-.....        8b ff 55 8b ec e9 7b 02 cc 74 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00
11296   PSEXESVC.exe    0x1660000       0x1660fff       VadS    PAGE_EXECUTE_READWRITE  1       1       Disabled        N/A
b8 79 00 00 00 e9 7b 33 c8 75 cc cc cc cc cc cc .y....{3.u......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
b8 79 00 00 00 00 00 00 00 00 00 00 00 00 00 00 .y..............
00 00 00 00 00 00 05 00 2d 00 00 00 00 00 00 00 ........-.......        b8 79 00 00 00 e9 7b 33 c8 75 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 b8 79 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 2d 00 00 00 00 00 00 00
11296   PSEXESVC.exe    0x1650000       0x1650fff       VadS    PAGE_EXECUTE_READWRITE  1       1       Disabled        N/A
8b ff 55 8b ec e9 eb 03 1b 74 cc cc cc cc cc cc ..U......t......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 ..U.............
00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00 ..........-.....        8b ff 55 8b ec e9 eb 03 1b 74 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00
11296   PSEXESVC.exe    0x1680000       0x1680fff       VadS    PAGE_EXECUTE_READWRITE  1       1       Disabled        N/A
8b ff 55 8b ec e9 fb 05 9b 75 cc cc cc cc cc cc ..U......u......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 ..U.............
00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00 ..........-.....        8b ff 55 8b ec e9 fb 05 9b 75 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 8b ff 55 8b ec 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 12 1b 2d 00 00 00 00 00
11296   PSEXESVC.exe    0x1690000       0x1690fff       VadS    PAGE_EXECUTE_READWRITE  1       1       Disabled        N/A
b8 03 10 03 00 e9 2b 10 45 75 cc cc cc cc cc cc ......+.Eu......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
b8 03 10 03 00 00 00 00 00 00 00 00 00 00 00 00 ................
00 00 00 00 00 00 05 00 2d 00 00 00 00 00 00 00 ........-.......        b8 03 10 03 00 e9 2b 10 45 75 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 b8 03 10 03 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 2d 00 00 00 00 00 00 00
11296   PSEXESVC.exe    0x16f0000       0x16f0fff       VadS    PAGE_EXECUTE_READWRITE  1       1       Disabled        N/A
68 98 00 00 00 e9 8b aa cb 73 cc cc cc cc cc cc h........s......
cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 ................
68 98 00 00 00 00 00 00 00 00 00 00 00 00 00 00 h...............
00 00 00 00 00 00 05 00 2d 00 00 00 00 00 00 00 ........-.......        68 98 00 00 00 e9 8b aa cb 73 cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc cc 05 00 68 98 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 05 00 2d 00 00 00 00 00 00 00
```


The malfind results suggest that the process `PSEXESVC.exe` may have injected or loaded malicious code into its own memory space. This could indicate the presence of malware, possibly exploiting code injection techniques for persistence or payload execution. The suspicious memory region and hex dump should be further investigated by performing static analysis or extracting the region for deeper inspection.

#### Summary and IOCs

Based on the investigation conducted, several binary analysis techniques were employed to identify potential Indicators of Compromise (IOCs) within the `iveep memory image`. The following analysis and IOCs are critical for future investigation:

###### Analysis Techniques:

###### Process and Parent-Child Analysis:

`PSLIST`, `PSTREE`, and `PSCAN` were utilized to identify suspicious processes like `PSEXESVC.exe`, `excel-update.exe`, and `ScreenConnect` which were involved in the launching of a command prompt and PowerShell, hinting at potential remote access and persistence mechanisms.

###### Memory Handle Analysis:

The `windows.handles command` revealed that suspicious processes, particularly `PSEXESVC.exe` and `excel-update.exe`, interacted with key system files, registry keys, and other critical components, indicating potential malicious activity.

###### Privilege Escalation:

Using `windows.privs`, it was observed that the suspicious processes had high-level privileges such as `SeDebugPrivilege`, `SeTcbPrivilege`, and `SeImpersonatePrivilege`, enabling them to read memory from other processes, impersonate users, and modify system settings.

###### Network Connection Analysis:

The `windows.netscan` command identified active remote connections from `excel-update.exe` to `35.86.58.69:4443` and `ScreenConnect` to `139.178.70.144:443`, suggesting command-and-control (C2) communication and potential malware exfiltration.

###### Unusual DLL Injections:

The `windows.dlllist` command flagged suspicious DLLs like `umppc19205.dll` and `ScriptControl32_19205.dll`, which could be associated with malware persistence or additional exploitation techniques.

###### Malware Detection:

The `window.malfind` plugin revealed suspicious memory regions in `PSEXESVC.exe`, showing anomalous code segments that could potentially indicate injected or malicious code.

###### Key IOCs:

Suspicious Process Names:

`PSEXESVC.exe`, `excel-update.exe`, and `ScreenConnect`

These processes, especially `excel-update.exe`, executed from unusual directories `(e.g., C:\Windows)` and exhibited malicious behavior `(e.g., invoking cmd.exe and powershell.exe)`.

###### Network Connections:

IP addresses: `35.86.58.69` (C2 communication) and `139.178.70.144` (Remote access).

Ports: `4443` (non-standard HTTPS), `443` (standard HTTPS).

###### Suspicious DLLs:

`umppc19205.dll` and `ScriptControl32_19205.dll`, potentially injected for persistence or exploitation.

###### Memory Regions with Anomalous Code:

Regions with `PAGE_EXECUTE_READWRITE permissions` and abnormal code sequences associated with `PSEXESVC.exe`.

###### Recommendations for Further Investigation:

File System Analysis: Search for `PSEXESVC.exe`, `excel-update.exe`, and any DLLs like `umppc19205.dll` on the disk to confirm if these files exist and if they match the memory image.

Network Traffic Monitoring: Investigate network traffic to and from IP addresses `35.86.58.69` and `139.178.70.144` for signs of ongoing C2 or exfiltration activities.

Binary Analysis: Perform a hash check on suspicious binaries `(PSEXESVC.exe, excel-update.exe, and any DLLs)` and cross-reference them against known threat databases `(e.g., VirusTotal)` to determine if they are known malware.

Endpoint Detection: Utilize endpoint detection tools to check for traces of `ScreenConnect` and related remote access tools.

By correlating these IOCs with further investigation on the disk and network traffic, analysts can confirm the presence of malware and take appropriate containment measures.